In [1]:
import numpy as np
import json
import os
import pandas as pd
import requests
import bs4
from lxml import html
from fake_useragent import UserAgent
import time
import re
from urllib.parse import unquote

In [2]:
def get_yelp_data(path):
    with open(path, 'r') as file:
        raw_data = file.readlines()
    raw_data = map(lambda x: x.rstrip(), raw_data)
    json_data = '[' + ','.join(raw_data) + ']'
    df = pd.read_json(json_data)
    return df

In [3]:
file_type = 'business'
#'checkin' : 'yelp_academic_dataset_checkin.json',
#'review' : 'yelp_academic_dataset_review.json', # the file size is 6.3G 
#'tip' : 'yelp_academic_dataset_tip.json',
#'user' : 'yelp_academic_dataset_user.json' # the file size is 3.3G 

path = os.path.join('..', '..', 'yelp_dataset', 'yelp_academic_dataset_' + file_type + '.json')
df = get_yelp_data(path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


In [5]:
df.head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
5,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."
6,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,"6870 S Rainbow Blvd, Ste 117",Las Vegas,NV,89118,36.063977,-115.241463,3.5,81,1,"{'BusinessParking': '{'garage': False, 'street...","Dry Cleaning & Laundry, Local Services, Laundr...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."
7,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,6910 E Southern Ave,Mesa,AZ,85209,33.393885,-111.682226,5.0,18,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Auto Repair, Oil Change Stations, Automotive, ...","{'Monday': '7:30-17:0', 'Tuesday': '7:30-17:0'..."
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
9,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,700 Kipling Avenue Etobicoke,Toronto,ON,M8Z 5G3,43.624539,-79.529108,3.0,16,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Martial Arts, Gyms, Fitness & Instruction, Act...","{'Monday': '5:30-23:0', 'Tuesday': '5:30-23:0'..."


In [6]:
session = requests.Session()
ua = UserAgent()
#headers = {'User-Agent' : str(ua.chrome), 'Connection': 'Close'}
#headers = {'User-Agent' : str(ua.chrome)}
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
session = requests.Session()

In [21]:
def fetch_url_from_yelp_stop_if_captcha(biz_entry):
    s = (biz_entry['name'] + ' ' + biz_entry['city']).lower().replace('&', 'and').replace('\'', '')
    s = re.sub('[^0-9a-zA-Z]+', ' ', s)
    biz_address_tail = '-'.join(s.split())
    res = session.get('https://www.yelp.ca/biz/' + biz_address_tail, headers=headers, stream=True)
    website = None
    blocked = 0
    #print(res.text)
    idx = res.text.find('/biz_redir')
    if idx > 0:
        while res.text[idx] != '=': idx += 1
        idx2 = idx
        while res.text[idx2] != '&': idx2 += 1
        website = res.text[idx + 1: idx2]
        website = unquote(website)
    
    if website is None:
        if res.text.find('<title>Yelp Captcha</title>') >= 0:
            print(res.text)
            blocked = 1
    return website, blocked

In [8]:
urls = ['http://www.therangeatlakenorman.com/', None, 'http://www.felinus.ca', None, 'https://www.usemyguyservices.com', 'http://oasisautocenter.net', None, 'http://junctiontire.net/tires-auto-repair-mesa-az', 'http://www.theempanadashouse.com', 'http://www.xtremecouture.ca', 'https://landscapingcalgary.org', None, 'http://www.chocolateshoppeicecream.com', 'http://www.convertabath.com/', None, 'http://www.desertstorage.com/henderson-nv-self-storage', 'https://www.manoloblahnik.com/in/boutiques/Wynn%20Las%20Vegas', 'http://www.bloomandblueprint.com', 'http://www.2carguyz.com', 'https://fitbodyaz.clickfunnels.com/sales-page-yelp', None, 'https://www.glendaleaz.com/glitters/', 'http://www.ncbrewsmusic.com/', 'http://www.tipsyryde.com', 'http://www.philthyphillys.com', None, None, 'http://www.hendrickmotorsofcharlotte.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, None, 'https://www.apadana.restaurant', 'https://www.iga.net/fr/recherche_de_magasin/magasin/0293-marche-d-alimentation-beck-inc', None, 'http://www.wetzels.com', 'http://www.beechmontcc.com', 'http://www.gkvapors.com', '1592058805', 'https://www.bannerhealth.com/locations/phoenix/banner-university-medical-center-family-medicine', 'http://www.thegourmetburgerco.com', 'https://www.goodyearautoservice.com/en-US/shop/PA/Pittsburgh/000901034-15206-3149', 'http://stores.disneystore.com/ca/concord/317', None, 'http://meatchix.com', None, None, None, None, 'http://www.basspro.com/', 'https://ccv.church/locations/anthem', 'https://locations.wendys.com/united-states/oh/cleveland/13939-euclid-ave.', None, None, 'http://www.bathandbodyworks.com', 'http://irenestaproom.com', 'http://hingetown.com/', 'http://www.bestitchedneedlepointshop.com', 'https://www.esportafitness.com/Pages/clubhome.aspx?clubid=1', None, 'https://www.smlentertainment.com/calgary-north/', None, None, 'http://fiveguys.com/', 'http://platinumkustomz.com/', None, 'http://locations.tccrocks.com/az/gilbert/2218-e-williams-field-rd.html', 'https://local.fedex.com/nc/charlotte/office-0746/', 'http://www.lacoste.com/us/', 'http://www.helenblakeyflowers.com/', None, 'http://localpetpgh.com', 'http://www.ocotillointernalmedicine.com', 'http://www.mpbattingcages.com', 'https://www.garageexperts.com/location/charlotte', None, 'http://redlinepromotions.com', None, 'http://stqitoronto.com', 'http://saxetheater.com/', 'http://www.sincitypartybuses.com', 'http://www.cjaycomputers.com/', None, 'http://www.visitnc.com/listings/view/52079', None, '1592059164', 'http://www.minthillbarbershop.com/', 'http://actionfurnace.ca', 'http://www.mamanapolipizza.com', 'http://www.monarichiebride.com/', 'http://www.movers4less-il.com/', '1592059211', None, 'http://darrochmotorsports.com', 'http://www.jeffersonchandler.com/?utm_medium=listing&utm_source=yelp', 'http://landing.trugreen.com/yelp?city=Champaign&state=IL&zip=61822&utm_campaign=ORGANIC&marketingpartner=9817', None, 'http://fresh.longos.com', None, None, None, 'http://www.rehabdirectives.com', None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.tavernrestaurantgroup.com', None, None, 'https://www.pizzaville.ca', 'http://www.royalskincare.net', '1592059367', 'http://www.sugarbowlscottsdale.com/', 'http://www.vikingcleanersaz.com', 'http://www.timhortons.com/', None, None, 'http://www.shalecreekgolfclubggp.com/', None, '1592059436', 'http://store-locator.barnesandnoble.com/store/2147', 'https://www.myairtoday.com', None, 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-indian-school?utm_campaign=honorhealth-medical-group-indian-school&utm_medium=local&utm_source=yelp', 'http://www.marcos.com', 'http://www.sleepandwellnessctrs.com/index.php/modules-menu/2012-01-19-13-38-38/henderson', 'http://www.hotelcharlotterestaurant.com/hotel/', 'http://www.handandstone.com', 'https://www.ulta.com/stores/charlotte-nc-590', 'http://www.JesusChurch.life', 'http://www.jjspizzacle.com/', 'http://www.dennys.com', 'https://rinsenride.com', None, 'https://kulinaryaexpress.com', 'http://www.ahwatukeepeds.com/', 'http://www.timhortons.com/ca/en/locations/get-directions.php?id=100934', None, 'http://www.amegashootingsupplies.com/', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, None, 'http://www.freshcoatpainters.com/charlotte', 'http://clk.atdmt.com/HBO/go/165733594/direct/01/', None, 'http://www.hottubliquidators.com', 'https://sharonmtuttle.com/', 'http://www.fleurishflowershop.ca', 'http://www.healthspringlv.net', 'http://www.torontodermatologycentre.com', 'http://chandlersymphony.com', None, None, 'http://www.zavcoffeeshop.com', None, 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=0352&s_cid=lsl0000001', 'http://www.frederickandsons.com', 'http://www.winerack.com', 'http://www.chefalyssaskitchen.com', None, 'https://www.fatmoespizzawings.com', 'http://www.shearsensationsonline.com', 'http://www.movingforwardmadison.com/', None, 'http://www.kegsteakhouse.com/en/locations/ON/toronto/esplanade-keg/', 'https://www.thethingshop.com', None, None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89128/027192/', None, None, 'https://www.target.com/sl/richland-twp-north/2262', 'http://www.fearlessmeat1.com', None, None, 'http://www.qobollabridal.com', None, 'http://www.lcbo.com', 'https://www.dayspascottsdaleaz.com', 'http://leecafesc.com', 'http://www.gofitnessnation.com', None, None, 'https://www.peoriaaz.gov/government/departments/parks-recreation-and-community-facilities/parks-and-trails/pioneer-community-park', 'http://www.nwpoolaz.com', 'http://www.cruffs.com/', 'http://dwellingsinthedesert.com', 'https://agents.allstate.com/oscar-arnold-goodyear-az.html', 'https://www.walgreens.com/locator/walgreens-1725+northport+dr-madison-wi-53704/id=2967', None, 'http://www.purelifeaz.com', None, None, None, 'http://stlouiswings.com', None, 'http://www.massageindavidson.com', 'http://www.aumspatoronto.com', 'http://www.pizzafresh.com', None, 'https://www.raynathans.com', 'http://www.docrequest.com', 'http://www.drsimms.com', None, 'http://www.pitcrewmovin.com', 'http://www.planetfitness.com/gyms/las-vegas-flamingo-nv-611?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'http://www.jealousyonline.com', 'http://www.communalmule.com', None, 'http://www.thebeautyboutiquewhitby.com', 'http://www.torontopolice.on.ca/museum/', 'http://www.goldstandardhomecare.com', None, 'http://www.103yoga.com', 'https://membercuts.business.site', 'https://dutchbros.com/?utm_source=goodyear&utm_medium=yelp&utm_campaign=website_goodyear', 'http://phxstreetfood.org/', None, None, 'http://www.criminalcustomz.com', None, 'http://standandstair.com', None, 'https://www.mazzeolimousine.com', 'http://dentsquad.ca', 'http://www.menchies.com', 'https://www.smalv.com/OurCenters/HealthCareCenters/Siena', 'http://mystonecreekdentist.com', 'http://www.chobuffet.com', 'http://www.pho.com/mesa-az/pho-hien', 'http://www.mrakovic.com', 'http://www.childtime.com/your-local-school/las-vegas-nv-1803/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', None, None, 'https://www.sonoraquest.com/find-a-location/scottsdale-scottsdale-rd-mescal/', None, None, None, None, 'https://www.meineke.com/locations/az/surprise-4338/?utm_source=yelp&utm_medium=xvc', 'https://losolivosristorante.business.site', 'http://sunfoodsupermarket.ca/', 'http://www.2uglassandtint.com', 'http://edwardsbeautycle.com', 'https://www.allmysons.com/cleveland', None, None, 'http://www.thetailwaggersclub.com/', 'https://veggiehouse702.com', '1592060496', 'https://www.mr-bills.com', None, 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://olympiasweettreats.com', 'https://locations.churchs.com/az-phoenix-7235', 'http://MasterMechanicalCleveland.com', 'https://psychbc.com/locations/greater-cleveland/north-olmsted-oh', None, 'http://sevabeauty.com/locations/charlotte_nc', None, 'http://www.uncleotis.com/', None, 'https://www.scottsdaleflighttraining.com', 'http://www.charlotteorthotics.com', None, 'http://www.motherhood.com/shipping.html', '1592060654', 'http://www.shintkd.com', None, 'http://www.thebookshopaz.com/', None, 'http://wandaroche.starkhomes.com', 'http://www.lvchinatownplaza.com/event_index.htm', 'http://www.solissouthline.com', 'http://www.hairrevivalaz.com', 'http://www.pivotimage.com', 'http://kingsfamily.com/?utm_medium=referral&utm_source=yelp', None, None, 'https://www.repairngo.com', 'http://www.fountainhillsrv.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.clubpilates.com/dtphoenix/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'https://wearernd.com', 'http://www.lcbo.com', 'http://www.skinconceptsspa.com', 'https://locations.tropicalsmoothiecafe.com/nv/henderson/10612-south-eastern-avenue', None, 'http://lvmpd.com/', None, 'http://www.pueblomedicalimaging.com', 'http://www.creditunion1.org', None, 'http://www.pjspet.com', 'http://www.menyhartplumbing.com', 'http://www.cuisinemalaya.com', None, 'http://www.truvapenv.com', 'http://www.shopmadmod.com', None, 'http://www.southwestmobility.com/', 'http://www.wendys.com', 'http://www.starbucks.com/store/9298/', '1592061020', 'http://www.liquorstorevegas.com', 'https://www.hottopic.com', 'http://rubberstampevents.com', None, 'http://www.jcsgrillhouse.ca/mississauga/', 'http://www.guyslock.com', 'https://qcontrary.com', None, 'http://www.thecatsmeow.com', 'https://www.thejoint.com/arizona/glendale/arrowhead-ranch-48016', 'http://www.fruitsnroots.com', 'https://www.maids.com/49/?utm_source=Yelp', 'http://www.td.com', 'https://www.usps.com/', None, 'http://www.airrite-supply.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, 'https://www.subarusuperstorewest.com', 'http://allinonerepairs.us/index.php/home/', 'http://www.joesautoaz.com/', 'http://edulisrestaurant.com/', 'http://www.sdil.net', 'http://www.churchs.com', 'http://www.fatbastardburrito.ca', 'http://www.dollarama.com/', 'http://www.blackod.ca', 'http://www.rupertcornelius.net', 'http://www.alliancelv.com/', 'https://harrymarshlaw.com', 'https://nergysolutions.com', 'http://www.elitefloralevents.com', 'http://www.arizonaexecutivellc.com/', None, 'http://www.dunkindonuts.com', None, None, None, 'https://www.azairtime.com/scottsdale', 'https://www.jasonsdeli.com/restaurants/henderson-deli', None, None, None, 'http://www.izumi-sushi.com', 'http://www.logoexecutives.com', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', None, 'http://climbrefuge.com/', None, 'http://asiancafe.weebly.com/', None, 'https://www.brakesplus.com', 'https://www.churrasqueiramartins.com', None, 'http://www.vernspizza.com', 'http://tatmanstowing.com', 'https://www.mbkseniorliving.com/senior-living/az/scottsdale/mcdowell-village/?utm_source=mcdowell_village&utm_campaign=yelp', 'http://www.dhtorontoairport.com/', None, 'http://www.exxon.com', 'https://www.peacockrestaurants.com', 'http://www.thehousecoffee.ca/', None, 'http://www.calgarycurlingclub.com', 'https://clevelandhabitat.org/restore/', None, 'http://www.revelsalonmadison.com', 'http://www.misterchangs.com', 'http://www.smilesforeverpgh.com', 'http://www.charlotterusse.com', 'http://www.bistrogrande.com', None, 'https://ottogelato.ca', None, 'http://twizted-manez-beauty-barber.business.site', None, 'http://uncletetsu-ca.com', 'http://www.bitesandbrewspittsburgh.com/', 'https://www.walmaneye.com', 'http://speparty.com', 'http://www.propertyclaimadjusters.com/', 'http://www.zandrus.com', 'http://www.mercedesbenzofchandler.com', 'http://www.mycasinoquest.com', 'http://concordhairbeautysalon.com', None, None, 'http://ascentdronevideo.com/', None, 'http://www.cineplex.com/theatres/theatredetails/silvercity-crossiron-mills-cinemas-xscape-entertainment-centre.aspx', None, None, 'http://www.northtaphouse.com', 'http://www.mydcdental.com', 'http://www.kadensnails.com', 'http://soulgastrolounge.com/', 'https://greenvalleyranch.sclv.com/Entertainment/Lounges-and-Bars?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://katespade.com', None, None, None, 'https://www.toronto.ca/data/parks/prd/facilities/complex/1748/index.html', 'https://newresultsmedicalweightloss.com/near-you-scottsdale-arizona/', 'http://saigonnailspa.ca', None, 'http://www.bellevuebeer.com/', None, None, 'http://trufusion.com/', 'https://www.heartwoodroastery.com', 'http://www.poshhairandspa.com', 'http://www.yankeecandle.com', '1592062044', 'http://www.footcare4you.com/', '1592062563', 'http://www.livewellpgh.com', '1592062577', 'http://www.bloombc.com/', 'https://all-season-nails-spa.business.site', 'http://acninc.com', None, 'http://www.rexall.ca/', None, None, 'http://www.brookfieldvet.ca', 'http://theflowerroom.com/', 'http://sichuanren.ca', '1592062673', 'http://www.lodidoor.com', 'https://www.zerodegreescompany.com', 'http://thaimassagepro.vegas', 'http://www.pizzanova.com/locations/markham.html', 'http://www.yelp.com/events/chandler-rockin-taco-street-fest-2', None, 'https://www.purehockey.com/store.aspx?utm_content=Pure+Hockey-Monona-WI&loc_id=85', '1592062748', 'http://www.hickoryridgefirearms.com/page8.php', 'http://www.fantasticsams.com', 'http://www.marcs.com/stores/marc-s-medina-grocery-store-44256.aspx', 'http://www.yd.com/torontoyonge-eglinton/default.aspx', 'http://www.31kitchen.com/InfoEng.php', 'http://www.wamplerapartments.com', 'http://www.enterprisecarsales.com/location/53R1/Enterprise_Car_Sales_Charlotte?mcid=iyp:277&CUR=yelp', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PHXWH', 'http://www.starbucks.com', 'http://www.chewchew.net', 'http://jmacnv.com', None, 'http://www.gibs.bar', 'http://www.garciasmexicanrestaurants.net/', 'http://www.bobosbarbeque.com', 'http://www.chandlerinnaz.us', None, 'http://www.crescentoralsurgery.com/', 'http://www.fivestarmovingvegas.com', None, 'http://www.she-lv.com/', '1592062917', 'http://www.platinumpaintandbody.com', 'http://www.phxfoot.com', None, 'http://www.az-insulation.com', None, 'http://www.ilovemilanos.com/', 'http://www.duckislandclub.com/', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', None, None, 'http://skippa.ca', None, '1592063032', None, None, 'http://www.spanishoaksapts.com', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://www.easydrinkeasygo.com', 'http://www.wayneacebusandlimo.com', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/lasbr-renaissance-las-vegas-hotel/', None, None, 'http://karbking.com', None, None, 'http://cooling-az.com', 'https://anniys-massage-spa.business.site', 'http://white-oaks.com/cmswo/', 'http://www.kirstyblattner.com', None, None, None, 'http://www.gap.com?tid=gpme000290', 'http://www.cvrr.com', 'http://www.leelaecospa.com', 'http://www.leakdetectionarizona.com', None, 'http://www.dragonflyhotyoga.com/locations/middleton-west-madison-wi/', 'http://stores.partycity.com/az/goodyear/party-store-pc947.html', 'http://www.1800degrees.ca', 'http://locationsca.michaels.com/qc/saint-laurent/4967/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.chopinrestaurant.com', 'http://www.homedepot.ca', None, None, 'http://www.clemetparks.com/education/facilities/index.asp', 'http://www.bombaybuffet.ca', '1592063291', 'http://www.portagepointe.com', 'http://www.lakeeffectlaundromat.com', None, None, 'https://www.loandepot.com/msander', 'http://xmsengineering.com/', 'http://www.puntogelatotoronto.com', None, 'http://www.rudinospizza.com', 'http://www.mimiscafe.com', 'http://larkandkey.com/', 'https://www.chocolatsfavoris.com', 'http://www.myrotiplace.com', 'http://www.sdmi-lv.com', 'http://www.allcarewellness.com', 'http://www.firstclassvanlines.ca/', 'https://www.honorhealth.com/physicians/peter-ferrara', 'http://www.canyonridgephotography.com', 'http://www.trianglebenefitservices.com', None, None, 'http://www.abrazadental.com?utm_campaign=local&utm_medium=citations&utm_source=yext', 'http://www.lampsplus.com/?sourceid=SMYELPTXLKStore16&cm_mmc=YELP-SM-_-Text-_-Link-_-Store16', 'http://www.wtaetv.com', None, 'http://www.choicechiropractic.net', '1592063514', None, 'http://www.keytohealingmassage.com/', None, None, 'http://www.duquesne.org', 'http://gcglassllc.com', 'https://www.myedgecharlotte.com', 'https://www.pinnaclebankaz.com', 'http://www.johnjayautomotive.com/', 'http://www.barneys.com', 'http://www.torontocoffeeandteahouse.com/', None, None, 'http://timessquareaz.com/', 'http://www.TriPlumb.com', 'http://www.jchobbies.com', 'https://www.adencamera.com', 'http://izakayaju.com', 'http://www.jonesysclassiccleaners.com/', 'http://www.anchorbarcanada.com/square_one/', None, 'http://www.cloudcareclinics.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://bikesauce.org', None, 'http://www.sears.com/stores/arizona/mesa/0001628.html', None, None, None, None, None, 'http://singletrackfactory.com', None, 'http://www.residentialREclosings.com', 'http://www.sharifian.com', None, None, '1592063785', 'http://heritageeyecare.ca/', 'http://chazandodette.com', None, 'http://www.usa-autotransport.com/Phoenix', None, 'http://www.celebritytanning.com', 'http://www.blissdentallv.com', None, None, 'http://reinholdresidential.com/shadysidecommons/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.littlecaesars.com/', None, None, 'http://www.pods.com/phoenix.aspx', 'https://combatzonepblv.com', 'http://www.arizonawalkshop.com/', 'http://drakegeneralstore.ca', None, 'http://www.redhotrobot.net', None, 'http://miragenailsandspa.webs.com', 'https://www.mgmgrand.com/en/nightlife/whiskey-down.html', None, 'http://www.asianbuffet328.com/', 'http://www.nstuffmusic.com', 'http://www.groovyshoes.ca', 'http://www.gotoguyshandyman.com', 'http://www.meltingpot.com/charlotte-lake-norman/welcome', 'http://www.space55.org', None, 'http://fahhairextensions.com', 'http://www.visionsourcewilliamfharveyod.com', None, 'http://www.eggsmart.ca', 'https://www.lvatv.com', None, 'http://www.historybygeorge.com', '1592064139', 'http://korrys.com', 'http://www.cityofhenderson.com/parks/parks/paseo_vista.php', 'http://www.revitaantiagingcenter.com/testimonials', '1592064176', None, 'http://www.santosfairview.com', None, 'http://www.switchinkitchens.com', 'http://www.littlecaesars.com', None, 'http://www.ellekayegroup.com', 'http://7eleven.ca/store-locator/33259', None, None, 'http://www.kitchengourmetaz.com', 'https://www.batteriesplus.com/store-locator/oh/northolmsted/batteries-plus-927?utm_source=yelp&utm_medium=local&storecode=927', 'https://holisticlakewood.com', 'http://www.forestnailsandspa.com', 'http://www.massagehealthcare.com', 'http://tiredoctors.com', 'http://lonespurcafe.com', 'https://www.camdenliving.com/charlotte-nc-apartments/camden-south-end', None, None, '1592064386', 'http://www.UnexpectedVenue.com', None, '1592064415', 'https://holidayinnlongueuil.com/?utm_source=yelp&utm_medium=referral', 'http://azdistilling.com/', 'http://www.bamaspgh.com', 'http://rtcsnv.com', None, 'https://ad.doubleclick.net/ddm/clk/300117429;127049113;q', 'http://www.raphaelsbeautyschool.com', 'http://liznailsrockyriver.com', 'https://www.anytimefitness.com/gyms/866/Oregon-WI-53575/?utm_source=yelp&utm_medium=local', 'http://www.bridgespreschool.com', 'http://www.genoapizza.com', 'http://essostations.com', 'http://www.completeautomotivefairview.com', 'http://www.santanoralsurgery.com', None, 'http://www.thewinkinowl.com', 'http://polishedsalons.com', 'https://magicalstarpreschool.com', 'https://www.cellularsales.com/stores/verizon-4764-s-higley-rd-gilbert-az-85297/', 'http://www.abracleaners.com', 'http://www.liveatthestetson.com', 'https://www.hilton.com/en/hotels/clebhes-embassy-suites-cleveland-beachwood/?SEO_id=YELP-ES-CLEBHES', None, 'http://zombieburlesqueshow.com/', 'https://stores.hallmark.com/nv/henderson/amys-hallmark-shop-henderson-nv-65414.html', None, None, 'http://toronto.shambhala.org/', 'https://onekickgym.com', 'http://www.nctphoenix.com', 'http://www.akirabacktoronto.com', 'https://wolfgangpuck.com/dining/lupo-by-wolfgang-puck-las-vegas/', None, 'http://www.laserenaheights.com', 'http://www.tropicalbreezefrozendrinks.com', 'https://www.choicehotels.com/north-carolina/charlotte/rodeway-inn-hotels/nc168/rates', 'http://www.bramaleatire.com', None, 'https://www.nevadainsuranceenrollment.com', None, 'https://www.goldencoastii.com', 'http://jardin-thai.com/', None, 'https://paulpaddalaw.com', 'http://vegashotwater.com', None, None, None, 'http://www.mooreauto.com', None, None, 'http://MGpatisserie.com', 'https://www.corebellawellness.com', 'http://www.heartcleaneats.com', 'http://www.hhovv.org/', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.BetterThanYiaYias.com', '1592064927', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'http://www.jordanbowden.com?cmpid=D3ZF_BLM_0006', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://stores.jcrew.com/en/j-crew-yorkdale', 'http://www.frostgelato.com', 'http://www.dairykingsoutheuclid.com/', 'https://www.target.com/sl/willoughby/1112', 'https://www.target.com/sl/fitchburg/2106', 'http://www.excalibur.com', 'https://www.zippssportsgrills.com', 'http://theshoebox.com', None, 'http://miracleeyebrows.com/', 'http://www.biteandsip.ca', 'http://www.247acaz.com', 'http://michaelsgenuine.com', None, 'http://www.mtlebanonpubandpizza.com', None, 'http://www.ipowerweb.com', None, 'http://kidzpreschool.com/contact/mountainsedgeandsouthernhighlands', '1592065153', 'http://www.doughboyspizzawaunakee.com/', None, None, 'http://www.evildeadvegas.com', 'https://www.marriott.com/hotel-restaurants/phxdr-jw-marriott-phoenix-desert-ridge-resort-and-spa/stonegrill/5543174/home-page.mi', 'http://streetfoodapp.com/', 'https://atriumhealth.org/medical-services/childrens-services/levine-childrens-hospital', 'http://www.snowpalacecalgary.com', 'http://www.makemydoor.com', None, 'http://www.cjgenterprise.com', 'http://www.rodiworld.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.hottopic.com', 'http://www.gtavapes.com', 'https://www.fortmillderm.com', 'https://www.learningjungle.com/en-US/lone-mountain/', '1592065314', None, 'http://www.arizonafederal.org', None, 'http://www.holeeburger.com/', 'https://www.peterpiperpizza.com/locations/mesa-az/gilbert-baseline', None, 'http://onestopnutrition.com', 'http://www.thebloordalepantry.com/', 'http://www.playitagainsam.com', None, None, None, 'http://www.churchs.com', None, 'http://www.starbucks.com', None, 'http://www.muchoburrito.com', 'https://www.chezmatante.ca/fr', None, 'http://www.thebarndoorstore.com/', 'http://www.carrefourmultisports.com/', 'http://www.loicgourmet.ca', 'http://www.swedishautowerks.com', None, 'http://www.theroomaz.com', None, 'http://sushishop.com', 'http://www.kensingtonriversideinn.com', 'http://www.ghardytours.com', 'http://www.marmiteafricaine.ca/', 'http://www.rcsandbar.com', 'http://www.theaustralianaz.com', 'http://www.pctravelservice.com', '1592065589', 'https://www.global-restaurant.com', 'http://www.tfsgolf.com/', None, '1592065626', 'http://www.dunkindonuts.com', None, 'http://www.locksmithsolutionslv.com', None, 'http://www.victoriaballet.com/', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.dollartree.com/', 'http://www.perfectionistcleaningservices.co', 'http://parkerautotransport.com/', 'http://www.mrfallstree.com', 'http://premiumnailsandspa.com', 'http://www.teamtheraputix.ca/', 'http://www.wholegrainbreadco.com/', 'http://www.sweettoothfestival.com/', 'http://www.hendersonhyundai.com', 'https://www.camelbackcollision.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'http://www.pizzahut.com/', None, 'http://www.xpresslubecharlotte.com', None, None, None, 'http://www.momos.ca', 'http://www.brdiamondsuite.com', None, None, 'https://www.reliablerooteraz.com', 'http://www.pitaland.ca', 'http://www.ActionComicsandSports.com', 'http://www.davisbakery.net', 'http://www.riverstonetaverne.com/', 'http://www.FiveStarPM.com', 'http://www.mrmechanicofaz.com', None, 'http://www.foodlion.com', 'http://www.zombiemfg.com', None, 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://emergencyfamilymovers.com', 'https://useggrestaurant.com/', 'https://www.nationalsports.com/pages/store-pickering-ontario', 'http://airkewld.com', '1592066020', None, None, None, 'http://www.lacasadecristo.com', 'http://cfnsaz.com', None, 'http://macmillanorchards.com/', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTWS', 'http://www.momskoreanfood.com', 'http://www.azpops.com/', 'http://www.mamaspizzamatthews.com', 'https://www.bannerhealth.com/locations/glendale/banner-thunderbird-surgery-center', 'http://www.starbucks.ca', None, None, 'http://www.montroseautooutlet.com', 'http://kidneypuncher.com/', 'http://www.drnoorda.com/', 'https://candrtire.com', None, None, 'http://www.bradfordclinic.com', 'http://www.theepicure.ca', None, None, None, 'http://www.timhortons.com', None, None, 'http://www.abstractahairsalon.com', 'http://hungrybeecatering.com', 'http://livingatlavie.com', 'http://www.pittsburgh-home-inspections.com/', 'https://www.finks.com/birkdale-village', 'http://www.fearsconfections.com', '1592066435', 'http://www.crossfitvenividivici.com', '1592066459', None, 'http://herbologytours.com', 'http://www.lorays.com/home', None, 'http://www.ateraspas.com', None, 'https://www.supercuts.com/locations/nv/henderson/eastern-commons-haircuts-9195.html', 'https://stores.joann.com/oh/hudson/171/', 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=phxch&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', 'https://www.seaboardbrewing.com', None, 'http://www.lavenderoldlace.com', '1592066560', 'https://www.sleepcountry.ca', None, 'http://www.picknsave.com/StoreLocator.aspx', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://newserenityspa.com', 'http://www.pesticon.com', 'http://www.sushieye.com/', None, None, None, None, 'http://www.worldoftile.net/wordpress/locations/scottsdale/', None, None, 'https://fitnesstogether.com/plaza-midwood?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://handy-howies-handyman-services.business.site', 'http://nataliesnailsalonlasvegas.com', 'http://fiveguys.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, None, None, 'http://extremepita.com', None, 'http://www.solaruniverse.com/vegas', 'http://www.securitysupplystore.ca', 'http://jackastors.com/location-map/34/', 'http://www.lakenormancottage.com', 'http://sunsetwater.net', 'http://www.loveyoubackboutique.com', 'http://www.coach.com', 'http://www.scottsdaleoralsurgery.com/', None, 'http://www.davidsonday.org', 'http://www.babypointlounge.com', 'http://www.aerushome.com', 'https://www.dtlvarts.com', '1592066878', None, None, 'https://www.uber.com', None, None, 'https://www.theskincentermd.com/cosmetic-plastic-surgery-shadyside-pittsburgh-pa', 'http://www.PrecisionWindowTint.com', None, 'http://www.juicecore.com/', 'http://locations.einsteinbros.com/az/scottsdale/6501-e-greenway-pkwy-bldg-5.html?utm_source=storelocator', 'http://jewelrycitylasvegas.com', 'http://heartlakeauto.net', 'http://www.mcclurecollision.com/', 'http://canyonmidwifery.com', None, None, 'http://www.superstore.ca', 'http://www.atlascinemas.net', 'http://www.fosterinn.com', 'http://5two3.com', None, 'http://www.girlsgolfofphoenix.org', 'http://www.asu.edu/news/forthemedia/asufacts/campuses.htm', 'http://igniteglassandgifts.com', 'http://www.fixsys.ca', 'http://www.skydivelasvegas.com', None, None, 'http://guildresto.com/', 'http://www.focusbenefits.com', 'http://smokespoutinerie.com', None, 'http://www.jcprinting.net', 'http://www.passautos.net', 'http://www.papagiuseppes.com/', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://www.salonbellarouge.com', 'https://my.clevelandclinic.org', 'http://www.littlecaesars.com/', 'http://www.graciebarra.com/ahwatukee-az', 'http://www.marahselegantbridal.com', None, 'http://albertaballet.com', None, 'http://www.eyebrows4us.com', None, 'https://familydentistryofharrisburg.com', '1592067261', 'http://lenasrotianddoubles.ca', None, None, 'http://www.aquasystems.ca/', '1592067298', 'http://www.limoluxusa.com/', None, 'https://stores.cartier.com/united-states/nv/las-vegas/3131-las-vegas-blvd-south', None, 'https://www.reliablecouriers.com/service-area/arizona/phoenix', 'http://www.theoperahousetoronto.com', 'http://www.pinkberry.com/stores/15324', 'http://facejuvenate.com', None, 'http://www.tooltimehandymanservices.com', 'http://www.concordcustomcleaners.com', None, None, 'http://www.iriefoodjoint.com/', 'http://www.laherradero.com', 'http://www.insuremedave.com?cmpid=ram6_blm_0006', None, 'https://www.caseys.com/general-store/wi-de-forest/602-w-north-st/1936?utm_campaign=website&utm_term=organic&utm_medium=referral&utm_source=yelp&utm_content=1936', None, 'https://www.services.gov.on.ca/services/mapLocations.do?action=m', 'http://www.illinipubcycle.com', 'http://www.automotivespecialtyservices.com', None, 'http://mcmanusandcampbellhair.com', 'http://www.lv-pm.com', 'http://www.brunswickvolkswagen.com', 'http://www.tempe.gov/museum', 'http://www.firstchoicedental.com/locations/east-madison-office', None, 'https://smp-ink.com', 'http://www.lilliasdressmaking.com', 'http://www.firstcareclinic.org', 'http://www.redrockcanyonlv.org/hikes-trails/first-creek-canyon', None, 'http://www.discount-drugmart.com', 'http://rumrunnervegas.com/', 'http://www.starbucks.com/', None, None, 'http://thebco.com', 'http://www.oldworldbrewery.com', None, None, None, 'https://www.mattressfirm.com/stores/az/scottsdale/mattress-stores-scottsdale-az-85260-2922.html', None, 'https://www.walmart.com/store/1380/las-vegas-nm/details', None, None, None, 'http://www.flashtelwireless.com', 'http://www.hornekia.com', None, 'http://www.yogurtology.com', None, 'http://www.iliosnoche.com', '1592067721', None, None, None, 'http://baycathospital.com/', 'http://www.sportinglife.ca/', 'http://www.montebellasalon.com', 'http://www.salvationarmyusa.org', 'http://www.crossfitmecklenburg.com', 'http://Jjsgrubbshacknliquor.com', 'http://10minuteoilchangers.business.site', 'http://www.pizzacafe.ca', 'https://www.theneckandbackclinics.com/locations/southwest-las-vegas', None, None, 'http://nextgenelectrical.com', None, None, '1592067863', None, 'http://www.clarkcountynv.gov/Depts/parks/Pages/charlie-frias-park.aspx', None, None, 'http://www.dunkindonuts.com', 'http://www.shopnofrills.ca', 'http://santemedical.ca/', None, 'http://3rdphazebodyoil.net', 'http://www.mogowestlake.com/', 'https://www.23twelveluxe.com', 'http://painting.vegas', 'https://locations.rallys.com/az/tempe/1410-west-university', 'http://www.swisschalet.com', 'http://www.rfadezbarberspa.com', None, 'http://www.bluedogtraining.com', 'http://www.brueggers.com/', 'https://www.fitnessworks.com/locations/dobson-rd-chandler-blvd', '1592068059', 'http://www.freskamediterraneangrill.com/', 'https://www.choicehotels.com/pennsylvania/gibsonia/quality-inn-hotels/pa122/rates', None, None, None, 'http://www.coyoacangrill.com', 'http://www.dobsonspringsapartments.com', None, 'https://sites.google.com/site/brightcleantollesonlaundromat/', None, 'http://www.ihg.com/candlewood/hotels/us/en/tempe/phxbr/hoteldetail', 'http://www.guerlain.com', None, 'https://www.usps.com/', 'http://www.acelebritylimousine.com', 'http://www.laserspecialists.net', 'http://vegasstronglocksmithlv.com', None, None, 'http://www.centremecanix.com', 'http://www.schmitzdoor.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.fourturnberryplace.com/', None, None, 'http://www.lassendas.com', 'http://www.UnitedBloodServicesAZ.org', 'http://www.wildburger.ca', 'https://www.pizzadivinoaz.com', None, 'http://www.chicagojoesrestaurant.com', 'http://flopatio.com/', 'http://www.undrcard.com', 'https://www.cherrylash.com', 'http://restaurants.quiznos.ca/on/mississauga/CLARKSonCROSSING-L5J2Y4', 'http://www.kitcabinets.com', None, 'http://www.nevadaozone.com/', 'http://www.arizonaturnergroup.com', 'http://samwanna.com/', 'https://www.phxfray.com', 'http://www.azdonutco.com/', 'http://www.mtnroadcycles.com', 'http://www.martinisandmagicshow.com', 'http://www.ghhlasvegas.com', None, 'http://www.tradeinautobrokers.com', 'http://www.enterprisecarsales.com/location/39R2/Enterprise_Car_Sales_Bedford?mcid=iyp:277&CUR=yelp', 'https://local.biglots.com/az/tempe/4604?zcp=pd_local_yelp', None, 'https://tonysgaragelv.com', None, 'http://www.papajohns.com', None, 'http://www.searshomeapplianceshowroom.com/locations/phoenix-az', None, None, None, 'http://www.hivehealinghouse.com', 'http://americansprinklerlandscape.com', 'http://www.ckdesignslv.com', 'http://www.lasvegasoralsurgery.com', 'https://www.signaturestyle.com/locations/on/markham/first-choice-haircutters-51352.html', None, 'http://poopertrooperslv.com', None, 'https://www.charlottebraces.com/davidson-office-location', 'http://www.josephiarussi.com', 'http://www.yankeecandle.com', None, 'https://www.369bugs.com', None, None, None, None, '1592068718', 'http://www.frysfood.com/', None, None, 'https://www.lasvegassedation.com', 'https://vitospizzaglendale.com', 'https://www.amazinglashstudio.com/studios/az/phoenix/phoenix-ahwatukee', 'https://www.azpulm.com', '1592068798', 'http://www.cedarbraemall.ca', None, 'http://megansframing.com/', None, 'http://www.summitapartmenthomes.com', 'http://www.vegasgarageco.com', 'http://www.terroni.com/bar-centrale/', 'https://www.windsorstore.com/blogs/store-locator/nv-henderson-henderson-galleria-at-sunset', None, 'http://www.snaponoptics.com', None, 'http://rockinsul.com', 'http://www.thescottsdaleresort.com/restaurants/palm_court/', 'http://www.salsablancagrill.com', 'http://www.acupuncturevegas.com', 'http://www.blvdfoodcourt.com', 'http://www.drwiner.com/', None, None, 'https://lasvegasbankruptcylawyer.co', 'http://www.celebritytanning.com', '1592069006', 'http://beststripclublasvegas.com', 'http://www.hometastingroom.ca', 'http://www.clubredrocks.com/', 'http://greaterthancoffee.com', 'http://www.caferio.com/', None, 'http://www.MountainsideAir.com', 'http://adrenalinerushslingshot.com', None, 'https://www.take5oilchange.com/locations/oh/southeuclid-140/', 'http://www.pizzahut.com/', 'http://www.micabeauty.com', 'https://www.zeppestavern.com', 'http://www.woodworkerssource.com', None, 'https://www.steammaxx.net', 'http://www.arizonaveterinaryoncology.com', 'http://newellfarmersmarket.com/', 'http://www.theheadquarters.ca', None, 'http://www.sunrisekidsdental.com/', 'http://www.azprecisionpet.com', 'https://store.401games.ca', 'http://www.meineke.com/locations/il', 'http://medpost.com/location/surprise-az', None, None, 'http://www.wickedpixiehair.com', 'http://www.poorlittlerichgirlresale.com', 'http://www.karenwrightins.com?cmpid=G2LN_BLM_0006', 'https://lesliescleaningservice.wordpress.com', 'http://www.sacredseed.com', 'http://fiveguys.com/', 'http://www.raginuptown.com/', 'http://www.meadowspediatrics.org', 'https://www.sprint.com/storefronts/bd/sprint-pittsburgh-pa-15235-fcs-102/?ecid=SEO:Yelp:2020_Q1:LastMile', None, 'https://www.wokwithmingchinese.com', 'http://mdcpgh.com/', None, 'http://www.moochieandco.com', 'http://www.coronalandscape.com', 'http://www.devourphoenix.com', None, 'http://www.vegasvacuumsewing.com', '1592069417', None, 'http://www.rossstores.com', None, 'http://www.abercrombie.com', 'https://www.homedepot.com/l/Tempe-K/AZ/Tempe/85283/489?cm_mmc=seo-yelp&', 'http://www.framezillapgh.com/', 'http://carguyscanada.ca', 'https://www.enterprise.com/en/car-rental/locations/us/al/huntsville-madison-55ha.html?mcid=iyp:8368400', None, None, 'https://www.fyhanimalhospital.org', 'http://burritozone.ca/', 'http://www.malayakitchen.com', None, 'https://www.choicehotels.com/CN478?mc=blypypaf', '1592069575', 'https://clearviewplumbing.ca', 'http://www.azvalleydental.com', 'http://www.hiddencoveapartments.com', 'http://platinumhealth.ca', 'http://www.rachellallen.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, 'https://www.jerseymikes.com/3141', 'http://www.rexall.ca/', None, 'http://www.seah.ca', None, 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', None, 'https://www.kohls.com/stores/oh/stow-331.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', None, None, None, None, 'http://www.starbucks.com/store/9334/', None, 'http://pamperedhair.com', 'http://www.31kitchen.com', 'http://www.acsmartialarts.com', None, 'http://www.innovativeeyecare.net', 'https://theresortatsummerlin.com/dining/casual-dining/market-place-buffet-las-vegas/', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.mzdds.com', 'http://www.jeanniesgrooming.com', 'http://www.mobilemassagebychristina.com', 'http://www.acsm-clinic.org', 'https://www.anytimefitness.com/gyms/4149/Champaign-IL-61820/?utm_source=yelp&utm_medium=local', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.jessicahair.ca', 'http://www.teamafterburn.com', None, '1592070253', None, 'https://www.beemasterlv.com', 'https://sierraspringsdental.com', None, 'https://www.alfaromeousaofhenderson.com/contact.htm', '1592070319', 'http://www.boutiquetalulah.com', None, None, 'http://www.spirithalloween.com', 'http://www.rountreeplantation.com', 'http://www.forever21.com', 'http://serviceexpertscleveland.com/', None, 'http://suncontrollandscapes.com/', 'https://locator.bmo.com/Default.aspx?t=bb&lang=en&sid=8803', 'https://locations.wendys.com/united-states/sc/fort-mill/1800-hwy-160-w.', 'http://www.elmayarestaurante.com', None, None, 'https://www.brightnow.com/dentist-near-me/surprise-dentist/40110', 'http://www.lasvegascc.com', 'http://www.landscapeguy.net', 'https://www.cloverleafsuites.com/charlotte/', 'http://www.ihg.com/holidayinnexpress/hotels/us/en/phoenix/phxun/hoteldetail', 'http://myInsightImaging.com/AZ', None, 'http://www.azfaccs.org', 'http://www.phoenixhairextensions.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://rodneygentry.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006430627392&ds_s_kwgid=58700000443429206&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.onearmor.com', 'http://www.opusglow.com', 'http://www.hurleybrokers.com', 'http://www.Anchoranvilcoffee.com', None, 'http://www.greatclips.com/', 'http://sahniendo.com', '1592070593', 'https://www.chevronwithtechron.com', 'https://www.rayspizzacavecreek.com', None, 'http://www.hotelblackfoot.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://wwww.outlawscavecreek.com', 'http://www.beunailsbeauty.com', 'http://www.maggiesdonuts.com', None, 'https://www.johnfloydpaintingllc.com', None, None, 'http://carstarexpress.ca/locations/toronto/', 'http://www.coyoteplumbingandrooter.com', 'http://rothrockdmd.com/', 'http://www.mcmurraystylingcenter.com', 'http://danmarpetresort.com', 'https://www.MedgausDentalCare.com', 'http://www.starbucks.com', None, 'http://www.vietnampalace.ca', None, None, 'http://advancedanklefoot.com', 'http://www.sscandycigar.com/', 'http://signaturewest.net', None, None, 'http://www.burritoboyz.ca', None, None, 'https://www.moonvalleynurseries.com/services/tree-service-nevada?utm_campaign=yelp&utm_source=yelp-nv&utm_medium=tree-care', 'http://www.majesticeyebrowthreading.com', 'https://www.devitomd.com', None, 'https://www.harveys.ca', None, None, 'http://sofiasbraids.com/', None, 'https://www.aderraapts.com', 'http://goodwillsp.org/shop/retail-stores/second-editions', None, 'https://www.mattressfirm.com/stores/az/phoenix/mattress-stores-phoenix-az-85032-7927.html', 'http://www.azdmvaz.com/', 'http://www.TheMovementDanceExperience.com', 'http://www.hurstbakery.com/', 'http://www.sacktimemattress.com', 'http://www.bedbathandbeyond.com', 'http://www.shell.us', 'http://www.tambellinisbridgeville.com', 'https://electriciansmesaaz.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700007898397760&ds_s_kwgid=58700000649767852&ds_e_adid=%7Bcreative%7D&ds_url_v=2&cmpid=ADV_PG_%5B*KeywordID*%5D', 'http://www.redkingkong.com', 'http://www.bestbuy.com', None, 'http://eatnewchina.com', None, None, None, 'http://www.lvpremierpt.com', 'http://www.osteoklinika.com', 'http://koreangrillhouse.com/yonge-bloor-location/', 'https://www.providencedental.com', 'http://www.cabana-restaurant.com', None, 'http://www.dfwfurniture.com', None, 'http://www.impbaseline.com', 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.rumbi.com', 'https://arbatrestaurant.com', None, None, '1592071168', 'http://www.marshallretailgroup.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.firehousesubs.com/locations/oh/mayfield-heights', None, 'https://www.calldunrite.com', 'http://www.childrenslearningadventure.com/', 'http://www.bankofthewest.com', 'http://www.deweyscoffee.com', 'https://mercado-negro-tapas-restaurant.business.site', 'http://dethaicuisine.com', None, None, '1592071267', 'http://locations.harristeeter.com/nc/charlotte/205/', '1592071283', 'http://www.starbucks.com/store/8167/', 'http://www.amhomeservices.ca/', 'http://www.marbleslab.ca', 'http://castlecoffeelasvegas.myrestaurantpage.website', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://bikesharetoronto.com', 'http://storelocator.pepboys.com/pepboys/stores/chandler/chandler/00735', 'http://Www.makeupbynikkip.com', None, None, 'http://www.sevenhillsdentalcenter.com', 'http://www.stclairtanningspa.com', 'http://www.ontimevegas.com', None, None, 'http://www.lasvegasdiningtours.com/', None, 'http://www.airconditioningarizona.com/', 'http://www.spayaz.com/locations/mesa-az/', 'http://www.crossfitvenividivici.com', None, None, 'http://www.oakasushihouse.com', None, None, 'http://southeastlocksmith.com/', 'http://www.arizonacharliesdecatur.com/Food-Drink/Charlies-Market-Buffet', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://atechautocenter.com/', 'https://ad.doubleclick.net/ddm/clk/434184450;236814625;f', '1592071537', None, 'http://www.bonterrafarmmarket.com', None, 'http://solobace.com/', 'https://www.popeyes.com', 'http://holidayhoundhouse.com', 'http://zebreoptique.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', None, 'https://www.ediblearrangements.ca/stores/250-7th-Ave-SW-Calgary-AB-T2P-0W6', 'http://www.kaypacha.ca', None, 'http://whasn.com', 'http://www.winners.ca', 'http://www.elsantocantina.com/', 'http://www.thehomeshopnv.com/', 'https://stores.maccosmetics.com/us/oh/westlake/262-crocker-park-blvd', 'http://www.valueplace.com', 'https://stores.jcpenney.com/pa/pittsburgh/clothing-stores-pittsburgh-pa-2263.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2263', 'http://blackchickenwinecellar.com/', 'http://www.matthewsastrology.com', 'http://www.corrados.ca', 'http://www.apple.com/fashionshow', None, 'http://www.perfectprincesscut.com/', None, 'http://www.kouski.com', None, '1592071798', 'http://www.patdees.com/', 'https://order.subway.com/Default.aspx', 'http://fiveguys.com/', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTVF', 'http://smartypaws.com', '1592071848', None, 'http://reflexologypedicure.blogspot.com/', 'https://www.lhshealth.com/cedarwood-plaza/', None, 'http://www.scottssharpening.com', 'http://www.starbucks.com/store/11425/', 'http://gratifyinggrains.com', 'http://dreamconnections.com', '1592071927', None, 'http://www.actionazcomputers.com', 'http://www.sushixtra.com', 'http://www.pinevillecomputers.com', 'https://corksncrafts.com', 'http://www.pictureshowent.com/home-mesa/', None, 'https://locations.whataburger.com/az/scottsdale/2955-n-hayden-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', None, 'http://www.simplebistro.com', 'http://www.jimmysbackyardbbq.com', None, None, 'http://dentistrylasvegasnv.com', 'http://fishsandwich.ca/', 'http://yoamotacos.com/woodbridge-canada/', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.valentinos.ca', 'https://computerrepairdoctor.com/locations/cleveland/', None, 'http://www.sugarmarmalade.com/', 'http://www.winners.ca', 'http://www.originalchimneys.com/', 'https://www.hellotech.com/us/nc/mecklenburg/charlotte', None, None, 'http://www.champaigntruetires.com', 'http://www.hokkaidoseafoodsushi.com', None, 'http://www.innerstrengthchiro.com', None, None, None, None, 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2fnv%2flas-vegas%2fmidtown&sisearchengine=1524&siproduct=BEMDT_Bus', '1592072228', 'http://www.lettysonshamrock.com', None, None, 'https://www.wyndhamhotels.com/hotel/29495?tel=18882935571&iata=00093785&cid=fe:bh:20161215:yl:pp:hawus:29495', '1592072279', 'http://charlestonantiquemall.com/', 'http://apple.ca/retail/dix30', 'http://majerles.com/goodyear/', 'http://bebosmacshack.com', None, 'http://pamperohio.com/about.php', 'http://www.baystreetvideo.com/', 'http://www.bistro2182.com', 'http://staples.ca', None, 'http://www.uhhospitals.org/locations/uh-st-john-medical-center/st-john-medical-center-emergency-room', 'https://store.vioc.com/oh/brunswick/3675-center-rd-44212-060023', None, None, 'http://www.unionhillscc.com/', 'http://brighteyespa.ca', 'http://www.cosmictreats.ca', '1592072432', 'http://stardustvapes.org', None, 'http://www.nancyscheese.com', 'http://local.fedex.com/nc/concord/SRWA/', None, 'http://www.ososkatepark.com', 'http://www.alligatorpartyrental.com/', 'https://www.photobooths-charlottenc.com', None, 'http://jovanilinens.com', 'http://www.torontopearson.com/en/parking/#%23', 'http://mrgreek.com', 'http://www.thaihana99.com', 'http://www.FlawlessBeautiAndNailBar.com', None, None, 'http://www.cashamerica.com/SUPERPAWN-stores/AZ/PHOENIX/Pho18', 'http://www.holidayinn.com/hotels/us/en/toronto/yyzct/hoteldetail', None, None, 'http://vibrantcare.com', None, 'http://www.planetfitness.com/gyms/mississauga-dundas-st-on-1431?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'http://www.perfumes4u.com', 'http://www.fitness19.com', '1592072690', 'http://www.brisams.com/', None, 'http://www.dmelv.com', 'http://www.picea997.com', 'http://www.les3maria.com', 'http://www.jazz28.com', 'http://www.petland.ca/store-locator', None, 'http://www.tcccollision.com', 'http://www.advantaclean.com/las-vegas-nv', None, 'http://ccshvac.net', 'https://longevitymedical.com', None, 'https://tottoriallergy.com', 'http://richsautomotive.net', None, None, 'http://tasty-n-healthy.com/', 'http://www.reesors.ca/', 'http://www.tacoshuichoaz.com', 'https://www.azcasona.net', 'http://www.diamondtaxi.ca/', 'http://www.noblewindowcleaning.com', 'https://www.holysheetsusa.com', None, 'https://livepalmvillas.com', 'http://thelodgeatgrandtetonlasvegas2.cafesplanet.com', 'https://www.autumnparkcharlotte.com', 'https://www.hubbardswim.com', 'http://www.poolsnbeyond.com', 'http://www.littlerascalsrescue.com/', 'http://cityfreshpastapgh.com', 'http://www.primeroofingaz.com', 'http://www.lasvegasendodontics.com', None, None, 'http://uspalasvegas.com', 'http://stingraysushi.com/', None, None, None, 'http://www.pelicanssnoballs.com/', 'http://lakewoodmusicacademy.com', 'http://www.snvrha.org/index.htm', 'http://eastvalleysewnvac.com', 'http://www.daysinn.com/hotels/hotel-overview?wid=ps:addppc_yelp&tel=18003291073&reg=appc-_-ylPROPP-_-00405-_-diWES&hotel_id=00405&cid=whg_di_yelp', None, 'http://sushiave.net', None, 'http://www.silvercityrestaurant.ca', 'https://www.dontdrivedirty.com/locationsandpricing/?id=904', 'http://pages.wynnlasvegas.com/showstoppers', 'https://bluelakesassistedliving.com', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'http://www.facetofacefotographie.com', 'http://locations.harristeeter.com/nc/concord/202/', 'http://www.crownbuilders.com', 'http://www.mississaugahonda.com', 'https://www.goodfeet.com/charlotte', 'http://doorclosingsystems.com/', '1592073208', 'http://www.expressautospot.com', 'http://www.chestnutresidence.utoronto.ca/', 'http://www.naturalmeats.org', None, 'https://www.pizzaepazzi.ca', None, None, 'https://saenz-cleaning-llc-house-cleaning-service.business.site', 'https://einsteinplumbing.com/las-vegas-nv', '1592073287', 'http://www.thegreatinflatablerace.com/las-vegas', 'http://phenixsalonsuites.com/locations-detail/?id=94', 'http://www.houndhuddle.com', 'http://www.littlecaesars.com', None, '1592073354', None, 'http://www.stripdistrictmeats.com/', None, None, 'http://www.fearlesskittyrescue.org', None, 'http://www.rollinpizzapies.com', 'http://nsbank.com/', 'http://locator.carolinas.aaa.com/locations/pineville/', None, 'http://www.fortefamilypractice.com', 'http://www.arbys.com/', None, 'http://www.beatitudeschurch.org', None, None, 'https://www.alaskaair.com', 'http://www.discounttire.com/dtcs/makeAppointmentForStore.do?sc=NVL-12&utm_source=Yelp&utm_medium=iyp&utm_campaign=YelpIYPSA', None, 'http://www.parisoptique.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7784&localechange=1', None, 'http://thebreakawaybarandgrill.com/', None, 'https://www.riteaid.com/store-details?storeNumber=11467', None, 'http://storelocator.staples.ca/stores/fr_CA/QC/Saint-Laurent/21', 'https://www.supercuts.com/locations/sc/rock-hill/manchester-village-haircuts-82206.html', 'http://www.nacvegas.com', None, 'http://www.peoriashippingservice.com', 'http://www.bravaboutique.ca', 'http://www.saffronlasvegas.com/', '1592073756', None, None, 'http://gammajeansinhomedogboarding.com', 'http://www.eternitynailsspa.com', 'http://www.foodymart.com/', 'http://www.vogue-salon-spa.com', None, None, 'http://freshco.com/stores/bloor-dundas', 'http://www.tiltedgrill.com', 'http://www.northpointeyestudio.com', 'http://www.rightbuyautosales.com', 'https://www.jiffylube.com/locations/az/tempe/1665', 'https://www.choicehotels.com/wisconsin/madison/comfort-inn-hotels/wi415/rates', 'http://www.theinsidecoup.com', 'http://www.healingroots.center', 'http://www.lululemon.com/toronto/yorkdale?cid=yelp', None, 'http://tiffanynailsandspa.ca', None, 'http://www.ivoryparade.com', 'http://shehadys.com', 'http://www.torontonorthmitsubishi.ca', 'https://www.jiffylube.com/locations/nc/cornelius/3148', 'http://www.bridalboutiqueofaz.com', None, 'http://www.portofsubs.com/', None, 'https://yorkvilles.ca', 'http://www.atmospheresalon.com/', None, 'http://www.concordtavern.com', 'http://tyndallfurniture.com', 'http://www.las-vegas-locksmith.us/', 'http://www.saharamedicalinstitute.com', 'http://3vintagechicks.com/', 'http://www.edandmarks.com', 'http://www.eatoooweebbq.com', None, 'http://www.cbdefa.com', 'http://www.kuhars.com', 'https://www.mattressfirm.com/stores/nc/charlotte/mattress-stores-charlotte-nc-28277-5684.html', None, None, 'http://www.lalalunaspa.com', 'http://www.titansgym.com/index.php?Itemid=47&view=article&id=52&option=com_content', 'http://www.quiznos.com', 'http://www.sunpenang.com', 'http://www.stevenvance.com', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', '1592074181', 'http://makerealestateeasy.com', 'https://koliequestrian.com', 'http://www.goldmedalgym.com/chandler', None, 'http://www.freckaplumbing.com', 'http://www.mamaginaspizzeria.com', 'http://www.hotwire-electric.com', '1592074242', None, None, 'https://www.littledealer.com/pages/location%20phoenix', None, 'http://www.cleanandamazing.com', None, 'http://somagrill.com', 'http://www.xtremeperformance.info/', 'https://www.discounttire.com/store/nv/henderson/s/1300?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://www.gabesstores.com', 'http://5-starair.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://arthurmurray.com/locations/las-vegas-west-studio', None, 'http://www.winebeercigars.com', 'https://www.dairyqueen.com/us-en/locator/Detail/2398', 'http://www.glutenzerobakery.com/', None, 'http://www.cowboyciao.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', 'http://www.rebel-auto.com/', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=443', 'http://salsaritas.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.shoptieoneon.com/about-us.html', 'http://remachines.com', 'http://queencityrides.com', 'http://www.mazajloungecalgary.com', 'http://hahnsurvival.com', 'http://www.charmingcharlie.com/', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'https://www.starbucks.ca', None, '1592074521', '1592074530', 'http://www.scooterinvasion.com', 'http://www.zenewicznotary.com', 'http://www.onyxtheatre.com', 'http://www.mirasnails.com', 'http://www.gostorageone.com/self_storage/Las_Vegas_NV/zip_89130/storageone_self_storage/10494', 'https://www.dahntahndonuts.com', 'http://thirdeyebodyart.com', None, None, 'http://www.cjaimnotary.com', 'https://www.smalv.com/OurCenters/HealthCareCenters/Nellis', 'http://www.sinteriordesign.com', 'https://mstarkeychiro.com', 'http://www.provocateurimages.ca/', 'http://www.newwavemarket.com', 'http://www.dentistrybydesignchampaign.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.scratchkitchenclt.com', None, 'http://www.harlemrestaurant.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://classicchinesecuisine.com', 'http://www.thepreeners.com', None, 'http://www.organicmarketplacenc.com', 'https://www.officedepot.com/storelocator/nv/las-vegas/office-depot-6761/', None, None, 'http://www.surprisespringtraining.com', None, 'http://www.pickeringtoyota.com', None, 'http://www.bonniesprings.com/', 'http://www.divasalonspa.com/', 'http://wi.madison.charmingcharlie.com/fashion_accessories_madison_wi_124.html', None, 'http://roseandcrown.com/', None, 'http://www.buffalochipsaloon.com', 'http://rainbowcinemas.ca', 'http://www.grandchinesecuisine.com', 'http://www.azcommercialproperties.info/', None, 'https://bumpandroll.com', 'http://www.springmeadowliving.com', None, 'http://www.brakeshopaz.com', 'https://www.singyourstyle.com', 'http://lajollaexpress.net', '1592074921', None, 'http://calladams.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=10538&localechange=1', None, 'http://www.bulwarkpestcontrol.com', None, None, None, 'https://yourfeetfixer.com/locations/chandler/', 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', None, None, 'http://www.sofashack.com', 'http://mcneelyfamilydentistry.com', 'http://www.ludexauto.com', None, 'https://www.benjerry.com/gastonia', 'http://www.timhortons.com', 'http://raffin.ca', None, None, 'http://www.freshss.ca', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006435132512&ds_s_kwgid=58700000444952194&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.goodspeedperformance.com', 'http://www.ritzcarlton.com/en/Properties/Toronto/Dining/DEQ_Lounge/Default.htm', 'http://www.thesycamoreatscottsdale.com/', 'http://www.cobsbread.com/local-bakery/kingsland-calgary', None, 'https://fratellosdeliohio.com', 'https://www.wyndhamhotels.com/hotel/11185?tel=18007606483&iata=00093785&cid=fe:tl:20170523:yl:pp:tlus:11185', None, 'http://www.desertsparkle.com', None, 'http://allcityrefrigeration.com/', 'http://www.ipl-laserhairremoval.com', None, 'http://www.monicals.com/locations/monicals-pizza-of-tolono/', '1592075253', 'https://mercedes-benz-thornhill.ca', 'https://www.simonmed.com/locations/all/simonmed_imaging___summerlin', None, 'http://www.watericemesa.com', 'http://www.swisschalet.com', None, None, 'http://acehotel.com/pittsburgh', 'http://www.restauranttranscanada.ca', None, '1592075354', 'https://www.batteriesplus.com/store-locator/az/surprise/batteries-plus-616?utm_source=yelp&utm_medium=local&storecode=616', 'http://www.pioneersand.com', 'http://www.discounttire.com/', 'http://www.jonesoptical.com', 'http://www.illsecondthatonline.com', 'https://dealer.sprint.com/experts-choice', 'http://www.veneciagastonia.com', 'https://www.greatwolf.com/concord/dining-shopping/dining/loose-moose-cottage', 'https://www.scottsdale5thapts.com', 'http://www.shanghaipalacerestaurant.ca/', None, 'http://anytimefitness.com/en-us/clubs/582', None, '1592075489', 'http://lakecityair.com', 'http://www.level20lounge.com', 'http://www.vetteculture.net', 'http://chambersphx.com/', None, None, 'http://www.jimmythegreekgroup.ca', 'http://www.element-dayspa.com/', 'http://www.shopbop.com', '1592075568', 'http://www.the505e.com/', 'http://icyroll20.com', 'http://www.verizonwireless.com/stores/arizona/surprise/surprise-319434/', 'http://www.blizzyogurt.com', None, 'http://www.mygnp.com', '1592075632', 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/etobicoke-(1).aspx', None, '1592075661', 'http://homefabricsonline.com', 'https://kayahempco.com', 'http://www.planetfitness.com/gyms/kannapolis-nc-1212?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', '1592075693', 'http://myarizonasmile.com', 'https://www.navyfederal.org/?cmpid=bn%7Cyelp%7Cevgrn%7Cbrn%7Cnull%7Cnull%7C01/13/2020%7Cml%7Cnull', 'http://www.knitandsip.com', 'http://www.carpetmdonline.com', 'http://www.slcrealty.com', None, 'http://www.gvdentalcenter.com', None, 'https://www.apple.com/ca/retail/yorkdale/', 'http://www.pizzapanorama.ca/index.html', None, None, None, None, 'https://easternarizonaexterminating.com', 'http://envoguestyle.com', None, None, 'http://www.greenvalleypres.com', 'https://www.petsuppliesplus.com/Store/OH/Middleburg-Heights/Middleburg-Heights/28?utm_source=Yelp&utm_medium=Store-Page%C3%8A', '1592075851', 'https://www.hermanheylflorist.com', 'http://arizonanationalinsurance.com', None, 'https://www.sunrisepreschools.com/locations/dysart-thomas/?utm_source=yelp&utm_campaign=online_listing_goodyear_287&utm_medium=referral&utm_term=business_url', 'http://neurocnv.com/', 'http://www.showmethatsmile.com', 'http://www.facesbylorri.com', 'http://www.qinaturalfood.com', 'http://www.longos.com/store/storedetails.aspx?store_no=10', 'https://spbarbershop.com', 'https://www.nofrills.ca/store-locator/Jerry%27s+No+Frill%27s+Mississauga/3101?storeId=3101', None, 'https://www.advantage.com/location/las-vegas-mccarran-intl-airport-las/?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_local', 'https://www.walgreens.com/locator/walgreens-133+w+8th+ave-homestead-pa-15120/id=9674', None, None, '1592076007', None, 'http://www.bikesaviours.org/', 'http://saucelv.com/', None, 'http://www.splendidchinamall.ca', 'http://www.libertytintsolutions.com', None, 'http://scottsdale.hyatt.com', 'https://www.aceofheartscpr.net/locations/ace-of-hearts-cpr-las-vegas', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'http://www.re-newcabinets.com/', 'http://www.lloydsrestaurant.ca', 'http://www.medmassage.com', None, 'http://www.familyfarmcreameries.com', 'http://www.bloomingexpressionsflowers.com/', 'https://www.valero.com', 'http://www.norwinbowl.com', 'http://www.smilestraight.com', 'http://www.sunvalleyscooters.com', 'http://www.paperjoy.com/', None, 'https://www.redclayciderworks.com', 'http://arkvegas.com/restaurants_village-streets.htm', None, None, 'http://www.martelsalon.com/', None, 'http://www.theunicornpub.com', None, 'http://www.superstarmovers.com', 'http://www.tempe.gov/parks/', 'https://shop.wirelesszone.com/ma/hudson/234-washington-street', 'http://www.phoenixcarpetrepair.com', None, None, 'http://www.fernandosmexicanrestaurant.ca', 'http://scaasports.com', 'http://www.deervalleyplumbing.com', 'http://www.nspeds.com', 'http://www.americanapparel.net', 'http://www.sahara.bhhsnv.com', 'http://www.nobutei.com/', 'http://www.marvelmasonry.com', '1592076409', 'http://store.bobbiesflowers.com/?srccode=yelp_track', None, 'http://www.cicis.com/locations/nv-las-vegas-9890-s-maryland-pkwy', 'https://www.dontdrivedirty.com/locationsandpricing/?id=907', None, 'http://www.dilisosmeats.com', None, 'http://thebacklotbar.com', 'https://palmbeachtan.com/locations/AZ/Scottsdale-North-Scottsdale-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://www.procarehospicenv.com', '1592076498', 'https://www.harborinnseafoodcharlotte.com', 'https://www.kohls.com/stores/pa/gibsonia-1271.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.nsbank.com/branch/?branch=durango_and_215_south_branch', None, None, None, None, '1592076576', 'http://www.paradisemedspa.com', None, None, 'http://www.torontolivings.com', 'https://www.usps.com', 'http://www.scottspro.net', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=976', 'https://www.epoxylasvegas.com', 'http://www.pappertdentistry.com/', 'http://www.amjcampbell.com', 'http://www.valleyendodontic.com', 'http://www.encorelasvegas.com/#/entertainment/sinatra_dance_with', 'https://www.target.com/sl/glendale/1806', 'https://www.ulta.com/stores/gilbert-az-354', 'http://habitburger.com', None, 'http://www.greatclips.com/salons/6225', 'http://www.heavenscookie.com/', 'http://www.coolcomfort.net', None, 'https://late-night.business.site', 'https://mydeluxecleaners.com', 'http://www.thaiexpress.ca/', 'http://brotherspizzaandwings.ca', 'http://www.thechefon2wheels.com', 'http://www.reedkrakoff.com', None, None, 'http://www.big5sportinggoods.com', 'http://www.onelovegrapplingarts.com', 'http://www.eatatjoeydeli.com', None, 'http://www.montgomeryross.ca/', 'http://intouchcoachingllc.com', 'http://www.grottoshakersquare.com', 'http://www.ddsdiscounts.com', 'http://martininailbar.com', 'http://www.wedjaz.com', 'http://www.kasecustom.com', 'http://amadioranch.com', None, 'http://www.dworksautotintanddetailing.com', 'http://www.toothhealer.com', 'http://www.chbaz.com', None, 'http://gameworldvegas.com/', 'http://hearingassociateslv.com', 'http://www.kildares.ca', 'http://www.tablewinemadison.com', 'http://oohlalash.ca', 'http://www.ezpawn.com', 'http://www.starbucks.com/store/9298/', 'http://www.gnc.com', 'http://conniesjewelrygalleryandclockshop.com/', None, 'http://www.sushikanda.com/laval.html', None, None, 'http://www.quilles440.com/index_files/Page455.htm', 'http://www.oreillyauto.com', 'http://www.maxandermas.com/locations/333038', 'http://www.abreedapartanimalhospital.com', 'http://www.harbsauto.com', 'http://www.imagepluslasvegas.com', None, None, 'http://www.karachikitchen.com/location/brampton', None, 'http://www.flikandcompany.com', 'http://Skingri-la.com', 'http://www.theperfectshadesalon.com', None, '1592077170', None, None, 'http://www.calgaryautoquest.ca', '1592077203', None, None, None, 'https://www.beautybrands.com/store-locator/ahwatukee.do', 'http://www.rochestersdining.com', None, 'http://www.riverset.com', 'http://www.alohacardiology.com', None, 'https://www.ttc.ca/Subway/Stations/Wellesley/station.jsp', 'http://www.restaurantmissitalia.com/fr', 'http://www.charliesbarkpark.com', 'http://www.heartcenterofnv.com', 'http://www.networkautomotive.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.thaidtogo.com', 'www.edelweissimports.com', 'http://www.mcnallypianomovers.com/', 'http://www.daybreaknewmarket.com', 'http://www.wealthsimple.com', None, 'http://algobertoslasvegas.onlunchroom.space', 'http://phoenixcarnival.com', 'http://cabanatanolympia.business.site', 'http://www.bayrischerhof.biz', 'http://challengeyourgame.com/', None, None, None, None, None, 'http://www.phtowers.com/', 'http://www.raniasgrill.com/', None, 'http://www.dibellas.com/club-bread?source=yelp', '1592078201', 'http://www.burritoboyz.ca', None, 'https://locations.jimmyjohns.com/oh/southeuclid/', 'http://rusanscharlotte.com', 'http://www.brooklinbakery.ca', 'http://dukepubs.ca/westminster', None, 'http://www.regmovies.com/Theatres/Theatre-Folder/Regal-Severance-Town-Center-Stadium-14-822?utm_medium=social+network&utm_source=yelp&utm_campaign=yelp+-+description&utm_term=Regal+Cinemas+Severance+Town+Center+14', 'http://www.longjohnsilvers.com/', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', '1592078324', None, 'https://www.ahn.org/location-detail?name=wexford_Health_%21_Wellness_Pavilion', 'http://www.legendswarehouse.ca', None, 'http://www.gordonsjewelers.com', 'https://www.mukujapaneseramen.ca', 'http://www.kyreneinternalmedicine.com', 'https://www.presleyuptown.com', 'http://www.rapaczdental.com', 'http://www.ethicalaz.com/moquinosbodyandpaint', '1592078404', 'http://www.reesors.ca/', None, 'http://www.absorental.com', 'http://isthmustattoo.com', 'http://azhairdoctor.com', 'http://www.natalyapanchenkostudios.com', 'http://www.thrifttown.com/', 'http://www.lecharmeroses.com', None, 'https://www.northpointecle.com', None, None, 'http://robraymagic.com/', 'http://www.busybuggy.com/', 'https://www.lvhomeinspection.com', 'https://store.vioc.com/nv/las-vegas/9631-trailwood-dr-89134-HC0003', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.arsenal201.com', 'http://www.aplusnanniesinc.com', None, 'http://www.waypointvacation.ca', 'https://www.wastedgrain.com', 'http://www.analogcoffee.ca', 'http://www.chandlerneurologyandsleep.com', 'http://rockinrollssushi.com', None, 'http://www.ymcacharlotte.org', 'http://www.pohlenv.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYZXR', '1592078668', 'http://www.myoworksvegas.com/', 'https://www.cottonlanecitrus.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.villagerapartments.com', None, None, None, 'https://shop.nordstrom.com/c/wedding-suite-locations-beachwood-place?utm_source=yelp&utm_medium=organic&utm_campaign=wedd&utm_content=11227&utm_channel=lcl_brand_p&sp_source=yelp&sp_campaign=wedd', 'http://vegascaralarm.com', 'http://www.lubestop.com/Locations/Solon', 'https://www.fatcatsfun.com/gilbert', 'http://www.pinnaclevistavilla.com/', 'http://hharizona.com', 'http://www.sinatralv.com', 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'http://www.mlsalonspa.com', 'http://www.xplozone.com', '1592078860', None, 'http://bodyworksplus.com', 'https://circlecentreforhealth.com', None, 'http://www.hyatt.com/gallery/torphannon/kosher.html', 'http://www.halalbiryanihouse.com', None, 'http://longoortho.com/', 'http://www.emeraldsuites.com', 'http://www.gophoit.com/index.htm', 'http://www.sherwin-williams.com/store-locator/paint-store/Charlotte/NC/2164', None, 'http://www.attentiontoautodetail.com', 'http://www.club120.ca', 'http://www.onereddoorhudson.com/', 'http://pittsburgh.pirates.mlb.com/pit/ballpark/pncpark-events/venues/brewing-hall/', None, 'http://www.sunburstsmokeshop.com', None, None, 'http://www.DrJoyceChang.com', 'http://www.adogsbestfriendlv.com', None, 'http://www.jumpforfun.com', 'http://www.gonsteadperformance.com/', 'http://www.sunkissedtanninglv.com', 'https://www.purebarre.com/location/brecksville-oh', None, 'https://www.moderntortilla.com', None, 'http://bagelnash.ca', None, 'http://www.elevationweightloss.com', 'http://www.homegoodstore.com', None, None, None, 'http://www.frenchconnection.com', 'https://thirteen55motorsports.com', None, 'http://www.biltmorelifestylesre.com', None, 'https://megafurnitureusa.com', None, 'http://brittneywinklerhair.com', 'https://www.noons.co', 'http://www.eurocarsdetail.com', None, 'http://www.nailbrite.ca', 'http://worldofbeer.com/Locations/Gilbert', 'http://smartbuyavenue.com', None, '1592079287', None, 'http://www.pawsnclawsvet.com', 'https://www.smartstyle.com/en-us/locations/nv/las-vegas/located-inside-walmart-3473-haircuts-3334.html', 'http://www.christownspectrum.com', 'http://www.rebarn.ca', 'http://www.champps.com/', None, 'http://www.thepavilionsattalkingstick.com', 'http://www.merriperry.com', 'http://www.willowsdesign.com', None, None, None, None, None, 'http://www.dollartree.com', 'http://labambaburritos.com/', None, 'https://locations.teavana.com/us/az/scottsdale/7014-east-camelback-rd.html', 'http://whiskyattic.com/', 'http://apexendolv.com', 'https://locations.dennys.ca/AB/CALGARY/248179', 'http://www.alphadrone.com', 'http://sushibarkim.com/index.php', 'https://adamsplacerestaurant.wordpress.com', None, 'http://camelbacktailoringandalterations.com', 'http://www.firedupbbq.com', 'https://calgarycanine.com/auburn-bay/', None, 'http://www.oneshotinstallation.com', 'http://donutconnection.com/', 'http://www.goodfaithcasting.com', 'https://lvccld.org', None, 'https://www.sdi.edu', 'http://www.rslcs.org', 'http://www.coinlesslaundry.com/alma-school.html', None, 'http://www.oggiristorante.net/', 'https://www.ph216.com', 'http://www.shopnsavefood.com', None, 'http://www.manchuwok.com/', 'http://billskupa.com', 'http://www.cellphonerepaircomputerrepairlasvegas.com/', 'https://www.smartstyle.com/en-us/locations/nv/north-las-vegas/located-inside-walmart-2592-haircuts-3346.html', 'http://www.zernichlaw.com', 'http://www.greatdjmusic.com', '1592079719', 'https://www.redshoeslv.com', 'http://www.brighton.com', 'http://www.breadandsalt.ca', 'http://www.verizonwireless.com/stores/ohio/westlake/westlake-202670/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://zeroissuebeer.com', 'https://www.corepoweryoga.com/free-week?utm_source=yelp&utm_medium=cta&utm_campaign=chandler-yelp-free-week', 'http://www.ljkusina.com', 'https://www.kungfutea.com', None, None, 'https://www.discounttire.com/store/az/scottsdale/s/1284?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://desertridgemusicacademy.com', 'https://www.popeyes.com', 'http://www.martislounge.com', None, None, 'http://lylesjewelryandcoin.com', '1592079878', None, 'http://www.luckyhuelv.com', None, 'http://www.colonyroofing.com', None, None, 'http://www.mentorsportsbar.com', 'https://www.randrrefrigeration.com', 'http://kaizenautocare.com/body-shop-phoenix-az/', 'http://www.dsw.com', None, None, 'https://www.pristinemobiledetailaz.com', 'http://frysfood.com', None, 'http://www.hotdeallio.com', 'http://locations.michaels.com/NC/GASTONIA/5006/', 'http://www.enochturnerschoolhouse.ca/', 'http://AutoNationSubaruScottsdale.com', None, None, 'https://www.mezu.ca', None, 'http://racquetclubscottsdaleranch.com', 'https://www.stbank.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://kitcarson.snappages.com/', None, 'http://manchuwok.com', 'http://affinityplumbingaz.com', 'http://www.pizzadepot.ca/locationDetails/21', 'http://www.keratinspecialist.com', None, 'http://www.sincityvapor.com/', 'https://www.thebrick.com/pages/store-mississauga', 'http://www.villasatgreenvalleyapts.com', 'http://sojoramenlasvegas.taverndesigner.site', '1592080235', None, 'http://www.timhortons.com/', 'http://www.ahappydogwash.com', None, 'http://www.lids.com', '1592080274', 'http://www.johnrobertsspa.com', 'https://northscottsdale.tapoutfitness.com', 'https://1transmission.wixsite.com', 'http://www.momdocwfw.com/mercy-gilbert', 'https://my.clevelandclinic.org/locations/directions/225-brunswick-family-health-center', 'http://charmeck.org/city/charlotte/Airport/Parking/Pages/CellPhoneLot.aspx', 'http://idealdentistryaz.com/', 'http://www.DNDC.ca', 'http://www.crescentphx.com/', None, 'http://www.regency.edu/campus/tinley-park-illinois.aspx', None, None, None, 'http://www.coffeetime.com', None, 'http://www.connectionsarizona.com', 'https://mollymaid.com/west-glendale-avondale', 'https://www.baskinrobbins.com', 'http://www.maidbrigade.com/az/phoenix', 'http://azcyclery.com/', 'https://stores.guitarcenter.com/las-vegas-lessons?source=4GOA5AJBB', 'https://rvonthego.com/arizona/paradise-rv-resort/?utm_campaign=Yelp_Directory_Listing&utm_medium=directory&utm_source=Yelp', 'http://www.ranchomanana.com/', 'http://www.stratospherehotel.com', 'http://michaeldaily.com', 'http://www.winnerstraining.com', None, 'http://monkpatisserie.ca', 'http://www.mypopcornheaven.com', 'http://www.mc3salon.com/', 'http://www.andersonentertainment.co', 'http://biggunenterprisesllc.com/', None, 'http://www.glenatmesa.com', 'https://www.hilton.com/en/hotels/msnbjhx-hampton-suites-madison-downtown/?SEO_id=YELP-HP-MSNBJHX', 'http://concordncattorney.net', 'http://www.curetondentistry.com', 'http://eastsidemarios.com', None, None, 'https://www.theflowercart.org', 'http://www.serenevegas.com', 'http://www.outback.com/locations/az/surprise', None, None, 'https://haroldstudio.com', None, 'http://alsshoerepair.com', 'https://www.garciadiamondsusa.com', None, None, 'http://bodyworkscenter.net/', 'http://www.barbershopjack.com', None, None, 'http://www.charlotterestore.org', 'http://www.chopstixhenderson.com', 'http://www.billysroadhouse.com', 'http://www.fieldclub.org/', None, 'http://www.xcitemobilegamingcharlotte.com', 'http://www.dentalhygienegroup.ca', 'https://mbveuro.com', 'http://oaksautotruckservice.com/', 'http://www.securitytitle.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/nj/colonia-2471.html?mcid=iyp:8368400', None, None, 'http://www.thecortinaapartments.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044282967372&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.gnc.com', 'http://www.nexuscarelv.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.agramgardencentre.ca', 'http://www.petvethospitals.ca/morningside-pet-hospital/', 'http://www.restomeridiana.com', None, 'http://www.face2toe.com', None, 'http://teslamotors.com', None, 'http://www.lvsmiles.com', 'http://www.starbucks.com', None, 'http://www.theweavescene.com/', None, 'http://athoriagames.com', 'http://america-sunshine-maintenance.com', 'http://www.arizonatmjpain.com', 'http://www.redrocklasvegas.com/dining/cabo.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=RR&utm_content=RR-PROPERTIES&utm_campaign=RR-YELP', 'http://www.christysnursery.com', None, 'http://www.insuranceofvegas.com/?cmpid=wj16_blm_0006', None, '1592081138', 'http://www.discount-drugmart.com', 'http://www.arrowmedaz.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.Rooneys.Market', 'http://www.vitalant.org', '1592081204', 'http://rsecatering.com/', 'http://rebozos.ca', 'https://www.bbbuggy.com', 'http://www.villageinnrestaurants.com', 'http://www.craigroadcarwash.com', 'http://freshsifoods.com', 'http://www.gogibson.com', 'http://www.fatburger.com/home/', 'http://www.pitapit.com/', 'http://www.aumaubree.com/', 'http://oxygenrestaurant.ca', 'https://www.freshthyme.com/store/oh-westlake/', 'http://www.tinderboxcigars.com', 'http://www.brooksidelawnservice.com', 'https://www.truecocktailsbites.com', None, 'https://www.zaxbys.com/locations/nc/denver/7179-highway-73/', 'http://www.lovethreadingbar.com/', None, 'http://truearizonaliving.com', 'http://pigandduke.com/', 'http://ignite.fortis.edu/?BID=3038&VID=SR_CID-000045038666', 'http://www.moes.com', '1592081418', 'http://www.esourceparts.ca', 'http://www.qualityinn.com/hotel-madison-wisconsin-WI089?source=pmfyelp&pmf=yelp', 'http://www.satisfiedfrog.com', None, 'http://www.fceglobal.com/', 'https://www.lifetimefitness.com/en/programs-and-events/lifestyle/lifespa/arizona/goodyear-palm-valley-lifespa-hair-skin-nail-spa-facilities.html', 'http://phoenixmoversunlimitedinc.com', 'https://curaleaf.com/stores/pl/curaleaf-az-central-85040/', 'http://www.arbys.com/', 'http://www.towcompanymesa.com', 'http://thepurrrfecttouch.com', 'https://dutchbros.com/?utm_source=las-vegas&utm_medium=yelp&utm_campaign=website_las-vegas', 'http://www.linatranstudio.com', None, 'http://www.terryalixsalon.com', 'http://www.cupcrazed.com', 'http://shymcgrath.com', 'http://asiannoodletogo.com', None, '1592084427', None, 'http://www.themandrakeclt.com', None, 'http://www.AppleOne.com', 'http://www.originalhoagieshop.com/', None, 'http://www.modernrevivalevents.com/', None, 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, None, 'http://calibercollision.com/locate-a-caliber-collision-center/las-vegas-downtown/', 'http://barksbubblesgrooming.com/', 'http://www.customalterationsbyanne.com', 'http://www.kingcoleducks.com', 'http://climb-cleveland.com', None, 'http://my.clevelandclinic.org/staff_directory/staff_display.aspx?doctorid=4363', 'https://www.americancampus.com/student-apartments/az/tempe/vista-del-sol', 'https://www.perfectpalmssalon.com', 'http://www.finaticsaquarium.com', 'http://www.jrsweets.ca', '1592084616', 'http://www.acuraeast.com', 'http://www.grillcity.com/nevada/bbq-grill/', None, None, 'http://www.cowlickhairsalon.com', None, None, 'http://www.magnoliaroom.com', 'https://www.wyndhamhotels.com/hotel/52962?tel=18003876085&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52962', 'http://www.pro-edgecutleryaz.com', 'http://www.elitedestinations.net', 'http://www.fourstarsushi.ca', 'http://clickserve.dartsearch.net/link/click?lid=43700006430791267&ds_s_kwgid=58700000444977937&ds_url_v=2', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.attractagency.com', 'http://www.valleyrentabox.com', 'http://Www.garbarinos.com', 'https://www.swoon.cc', None, None, 'https://www.lakenormannailsbar.com', 'http://www.studiofontana.ca', 'http://www.e-zrentacar.com/locations/las-vegas-airport', None, 'https://hpautorepair.com', None, None, 'http://www.pizzahut.com/', 'http://thesman.com/las-vegas-meadows', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://affordableweddingminister.com', 'https://www.drnnadi.com', None, None, 'https://www.autozone.com/locations/pa/wilkinsburg/208-penn-ave.html', None, None, None, None, 'https://stores.dickssportinggoods.com/nv/las-vegas/1266/', None, 'http://www.themadisonapartmentliving.com', None, None, 'http://www.entsoa.com', 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', 'http://www.harborcrestapts.com', None, 'http://www.bottega.ca/index.php', 'https://badaxethrowing.com/axe-throwing-vaughan', 'http://www.rockincleveland.com', 'http://www.enatethiopianrestaurant.com', '1592085010', None, 'http://www.thecamptc.com', 'http://portsalonandspa.com', 'http://www.tso.ca', '1592085055', '1592085064', None, 'http://www.jimboys.com', None, None, 'http://workshopcalgary.com', None, None, None, 'http://charandarestaurante.com/', 'http://www.pattersonroofingco.com', 'http://www.armaantasteofindia.com', None, 'http://www.citywidetowing.com', 'https://www.officedepot.com/storelocator/nv/las-vegas/office-depot-3351/', 'https://www.hilton.com/en/hotels/lasethx-hampton-suites-las-vegas-airport/?SEO_id=YELP-HP-LASETHX', 'http://www.achristmasstoryhouse.com', 'http://www.cannabition.com', None, '1592085206', 'https://www.sleepcountry.ca', 'http://www.tenfeettall.ca/', 'http://myezroofsolution.com', 'http://mellomotive.com', 'http://www.sciullispizza.com', 'https://resurrectioncarpetcleaning.com', 'https://www.dodsonbros.com/location/charlotte-south/', 'http://www.russellchiro.net/', None, 'https://www.goldsgym.com/chandler-az/', 'http://fortyrodroadhouse.com', 'http://www.hendricktoyotaconcord.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, 'https://petmedic.vet/hospitals/providence-pet-hospital/', 'http://www.thewholeplate.ca', 'http://www.richardsautoclinic.com/', None, 'http://www.melonhead.ca', 'http://www.harryrosen.com/store/Vaughan%20Mills', 'http://www.rpspasta.com/', 'http://www.themaidcrusade.com', 'https://www.pintocashforgold.ca', 'http://www.willemandjools.com', 'http://www.abrazohealth.com/facilities/ArizonaHeartHospital/home.aspx', None, 'http://www.surefireinstitute.com', 'https://www.choicehotels.com/pennsylvania/coraopolis/comfort-suites-hotels/pa327/rates', 'http://www.kidszonedurham.ca', None, None, 'http://www.kennywood.com', 'http://www.avriorx.com', 'http://www.marcelwaffles.com', 'http://crumblcookies.com', 'http://www.thaiexpress.ca/', 'http://www.justincrain.com?cmpid=RV5O_BLM_0006', 'http://bestwestern.com', None, 'https://www.peoriapines.com', None, 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.aquarelarestaurant.com/Dominican_Restaurant_Toronto_-_Aquarela/Welcome.html', 'http://www.desertbloomskincare.com', 'https://locations.cibc.com/on/toronto/03902?q=Toronto%2C+ON+M4K+1P7&loc=M4K+1P7%2C+ON', 'http://masterwerks.com/', None, 'http://robertostacoshop.com', None, None, None, 'http://www.weprintvegas.com', 'https://angel-dentalcare.com', 'https://www.urbanoutfitters.com/en-ca/stores/downtown-toronto', 'https://www.repairngo.com', 'http://www.oasisislandvegas.com', 'http://www.airconditioningrepairinphoenix.com/', 'http://www.paloverdedentistry.com', 'https://www.benjerry.com/davidson', 'https://www.anytimefitness.com/gyms/690/Fitchburg-WI-53719/?utm_source=yelp&utm_medium=local', None, None, 'http://southwestcajunfest.com', 'http://locations.scootaround.com/lasvegas', 'https://www.tidedrycleanersaz.com/locations/e-frank-lloyd-wright-blvd', None, 'https://www.fragranceoutlet.com/pages/fragrance-outlet-tanger-outlets-westgate-114', 'https://www.exxon.com/en/gas-station/tempe-az-tholdings2165-321058', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.mygym.com/henderson', 'http://www.southwestbikes.com', 'http://www.thepittstopbbq.com/', 'https://www.missionmontessori.com', 'http://www.chandlerjewishpreschool.com', None, 'http://www.chinaonepgh.com', None, 'http://www.pandaexpress.com/', 'http://www.davisvilleparkvets.com', 'https://lawnlove.com/charlotte-lawn-care', None, 'http://www.stitchit.com', None, 'http://kidscuckoosnest.com', None, '1592085834', None, 'http://www.coolsummercamp.ca', 'http://www.phxskin.com/', 'http://www.magnetitecanada.com', None, None, None, None, 'http://www.diamondbackcleaners.com', None, 'http://asurology.com', 'http://www.maxstudio.com', 'http://thefloatfactor.com', 'http://www.hairdistrict.ca', 'http://www.republicservices.com/', None, 'https://order.iamaflowerchild.com', 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', None, 'https://secondcup.com', None, 'http://www.altour.com', 'https://www.gogeisel.com', 'http://www.sincitytint.com', 'http://www.landeseye.com', 'http://www.superstore.ca', None, 'http://www.academyofnails.net', 'http://www.stevenkretchmer.com/', None, '1592086116', None, 'http://www.cineplex.com/Theatre/cineplex-cinemas-scarborough', 'http://www.crowbarlasvegas.com', 'http://azwindowreplacement.com', 'https://www.jrsmulchsales.com', 'http://www.mywellnessplace.com/', '1592086195', None, 'http://www.summerwindsnursery.com', 'http://palace.awardspace.com/', None, None, 'http://www.danieljsalons.com', 'http://www.southhillsortho.com', None, None, 'http://www.blackcatespressobar.com', None, 'http://jtsconcrete.com', 'http://www.magnolialanelv.com', 'http://www.bergstrommadisonchevrolet.com', 'http://www.thevillagedentalcenter.com', None, 'http://www.walterstation.beer', None, 'https://www.officedepot.com/storelocator/nc/kannapolis/officemax-6480/', 'http://www.aztacoshack.com', None, None, None, 'http://www.tranestudio.com/', 'http://www.gabroy.com', 'http://www.doughnc.com', 'http://www.maxmcqs.com/', 'http://www.popsphotoandtrophy.com', 'http://www.jaysatthelake.com', 'http://summerlinbarbershop.com/summerlinbarbershop/', 'http://www.destihl.com/', None, 'http://www.lasvegasskinspa.com/', 'http://www.northhillstoyota.com/', 'http://www.drlomboydc.com', None, None, None, None, 'https://www.hungryhowies.com/store/hungry-howies-00829?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=00829&utm_content=Website_Link', None, None, None, None, 'https://www.kyrene.org/apr', 'http://www.hoopershealth.com/store-information/store-information-brampton/', 'http://www.cosmoprofbeauty.com', 'https://www.unlvmedicine.org/clinic/internal-medicine-0', 'http://www.hairyt.com', '1592086614', 'https://mettlercenter.com', 'http://stlouiswings.com', 'https://bhmanagement.com/communities/az/phoenix/cielo-apartments-1794/', 'http://woodchesterautoglass.com', 'http://castironwaffles.com', 'http://www.caronpartners.com', 'http://www.debruinchiropractic.com', 'http://proshops.bowlersmart.com/locations/nevada/bowlersmart-henderson-pro-shop-inside-strike-zone-sunset-station', None, 'http://www.windowprollc.com', 'http://www.mildreds.biz/', 'http://theupsstore.ca/45', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Phoenix/AZ/8160', 'http://www.hennaflorist.ca', 'http://murphyslaw.pub', 'http://website-3132959076228282333140-nailsalon.business.site', None, 'http://cksbbqlv.com', 'http://thewhitebrickkitchen.com/', 'http://www.teamonsiteconcrete.com', 'http://www.eighty8family.com', 'http://www.romaninconcrete.com', 'http://www.safelite.com', 'http://madison-west.certapro.com/?source=Yelp', 'http://sassygurl.ca', 'https://www.azspiritualmedium.com', 'http://yourbuybacks.com', None, None, None, None, None, None, None, 'http://www.bhfcfinancial.com', 'http://www.angelblessingslv.com/', 'http://www.stonebowlkorean.com', 'http://www.liveattheurban.com', None, '1592086982', 'http://www.bigotires.com/regions/Nevada', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', None, None, None, 'http://phoenix-9thandbell.titleboxingclub.com/', 'http://www.thevineyard.ca/', 'https://www.anytimefitness.com/gyms/2195/Richmond-Hill-ON-L4E-3L6/?utm_source=yelp&utm_medium=local', 'http://www.mygreek2go.ca', 'http://stephenfolsondds.com', None, None, 'http://keyspleaselocksmith.com', None, 'https://www.woodbridgeinteriors.com', 'http://www.powernailspa.com/', 'https://www.homedepot.com/l/N-Scottsdale/AZ/Scottsdale/85260/472?cm_mmc=seo-yelp&', 'http://www.hair2000.ca', None, None, 'http://www.bestchickenwings.com/', 'http://www.faziopizza.com', 'http://www.sprint.com', None, 'http://shusalon.com/', 'https://www.lawkm.com', 'http://www.thetrteam.com/?agent=mahalalel-leli', None, 'http://5starpdrllc.com', None, None, 'http://www.bennettmedical.com', None, 'http://www.sulwer.com', 'http://www.ryerson.ca', 'https://locations.theupsstore.com/wi/fitchburg/2935-s-fish-hatchery-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.modernsalonandspa.com', 'https://scottsdaledaycare.com', 'http://www.venetian.com/casino/race-and-sports-book.html', 'http://www.waterway.com', 'https://parteeputt.com', 'http://www.vapelandphx.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.preferredpublicrelations.com', 'http://www.flashlightfactory.net', '1592087399', None, None, None, 'http://cloisterhoney.com/', 'https://horizoneye.com', 'https://www.autozone.com/locations/nv/las-vegas/1021-w-owens.html', 'http://www.audiprestigeddo.com', None, 'http://www.hpb.com/041', 'https://stores.joann.com/nv/henderson/2015/', '1592087498', 'http://www.buldak.ca', 'https://www.westwindintegratedhealth.com', 'http://www.rtosullivans.com/', 'http://www.thechefandyou.com', 'https://locations.checkers.com/nv/north-las-vegas/2720-n.-las-vegas-blvd/?utm_source=Yelp', None, 'http://magnoliapetwc.com', None, 'http://subway.com/', 'http://gotrmc.org', 'http://www.quiktrip.com', 'http://capitalacupuncture.janeapp.com', None, 'http://www.firehousegrillingco.com', 'http://www.threesheetsdtlv.com', None, None, 'http://www.cccnevada.com/doctor/josette-e-spotts-md-facs/', 'http://www.greatclips.com/', 'http://www.kokobellibagel.com', 'http://jpsushi.ca', 'http://protechautogroup.com', None, None, None, 'http://www.teresaspizza.com/richfield', None, None, '1592087749', None, 'http://www.donsbees.com', 'http://LugariPet.com', 'http://popeyeschicken.ca', 'https://arizona-firearms.com', 'http://www.paramountbutchershop.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Phoenix-AZ-85027/858046/', 'https://www.hertz.com/rentacar/location/unitedstates/northcarolina/charlotte/CLTT12', 'https://www.greendrop.com/calgary', None, None, None, 'http://www.bluelinegrille.com', 'http://www.shakeys.ca', None, 'https://www.lattaplantation.org', 'https://www.cybermax-pc-phone.repair', 'http://www.latan.com', None, None, 'http://web.archive.org/web/20071110183536/www.city.cleveland.oh.us/around_town/city_highlights/landmarks/freestamp.html', 'https://www.jerseymikes.com/3128', None, 'http://www.thecopperpalmsapartments.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.colorglolasvegas.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.thepotatobarn.com', 'https://northernsmokes.com', 'http://www.pandmpizzainc.com', 'http://www.brakemasters.com', 'http://www.towercitycenter.com/', 'http://www.crazymocha.com', 'http://www.bloordovercourtappliances.com', None, None, 'http://www.carmellaspizzagrill.net', 'http://www.thebluefoodtruck.com', 'http://www.kumon.com/PHOENIX-DESERT-RIDGE', 'http://www.creativeleather.com/', None, 'http://www.lecreuset.com/stores/charlotte-premium-outlets-charlotte-nc', 'http://www.ahwatukeechurch.com', 'https://stores.advanceautoparts.com/pa/carnegie/1011-washington-ave', 'https://charlotte-bjj.com', 'https://www.galleriavillage-apts.com', None, None, 'http://www.theairteamlv.com', 'http://www.northcentralsportsbar.com', None, 'https://epdpestcontrol.com', None, '1592091673', None, 'http://www.innerdivafitness.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.waltsbbq.com', 'http://www.lpoecharlotte.com', None, 'https://maps.rbcroyalbank.com/locator/searchResults.php?t=6702', None, None, None, 'https://www.campbowwow.com/cuyahoga-falls/', None, 'http://www.asba.com', 'http://cccwindowcleaning.com', 'http://metrodiner.com', 'http://www.sunnynutrition.com', None, 'https://www.jamescheese.ca', 'http://mattas.com', None, 'http://www.autoglassnow.com/phoenix', 'http://www.auntieleechampaign.com/', None, 'http://evolvedancecenter.com', 'http://www.luigismadison.com/', None, 'https://www.noodlebarphx.com', 'https://elitefloorandfurniturecleaning.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.yogibar.ca/', 'http://www.centerforbehavioralhealth.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://ineffablecaphe.com', 'http://www.thecitycafeaz.com/', 'http://www.floresfamilydental.com', 'https://www.thejoint.com/north-carolina/mint-hill/mint-hill-12029', 'http://sweetrepublic.com', 'http://www.thecaptainsboil.com?utm_source=Yelp&utm_medium=Click&utm_campaign=ON_5313Yonge&utm_content=mainlink', 'http://www.blueheel.ca/', 'http://sweetiefry.com', 'http://www.aecne.com/', 'http://www.carriepatriciastudios.com', None, 'http://www.platosclosetstrongsville.com', None, None, 'http://thegreatgreekgrill.com', None, None, 'http://www.wilderfood.ca', '1592092073', None, 'http://trunightclublasvegas.com', 'http://www.pouroliveoil.com', None, 'http://www.massagebar.com/pit.htm', 'https://penrose.edu/student-salon-and-spa', 'http://www.foodbasics.ca', 'http://www.t-mobile.com', 'http://www.azmulticare.com', None, 'http://www.yourbrowbar.com', 'https://www.advanceamerica.net', 'http://www.blossomdayspa.com', 'http://www.anticaosteria.ca/', 'http://mtholly.us/park_and_facility.php?3', None, '1592092204', 'http://www.ezpawn.com', None, 'http://wigwamcreekdental.com', None, None, 'http://enchantedcastle.info/', 'http://rosenailbarsummerlin.com', 'http://www.greatlakesbeer.com', 'http://habibizcafe.com', 'http://www.villagediscount.com/', 'http://www.tocayopgh.com', 'http://www.shuku2salon.ca', None, 'https://www.britishairways.com', None, 'http://www.elmolinocafe.com/', 'http://www.vedasfitness.com', 'https://www.infuzelv.com', None, 'http://www.winghouseandsportsbar.com/', 'http://www.eyewearatthehamptons.com', None, 'http://www.funkutzkidshairsalon.com/', '1592092400', 'http://expresssmogandmotorworks.com', None, 'https://www.affordablegaragedoor.biz', 'https://www.nhsteelecreekfamilymedicine.org', 'http://www.dhl.ca', 'http://www.azcrittergetter.com', 'http://www.smokesweinerie.com/', 'http://www.pizzapizza.ca/', 'http://www.southlandvisioncare.com', None, None, None, None, None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/1/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'http://iflyelite.com', 'http://www.crossfits3.com', None, 'https://www.argentaapts.com', 'http://www.guitaror.ca', 'http://hotyogaoncrowfoot.com/', 'http://Localocksmiths.com', 'https://www.tillys.com/store?StoreID=0044', 'http://www.woodbineanimal.ca', 'http://www.shilohcommunity.com', None, 'http://www.cochranvwnorth.com', None, None, 'http://www.fulloutperformance.com', 'http://caveatemptorlv.com/', '1592092620', 'http://www.bigalsbargrill.ca', 'http://www.theglostudio.com/', 'http://www.thematka.com/', 'http://thetkto.com/', 'http://www.pestcontrolguaranteed.com', None, 'http://belmontdrivein.20megsfree.com', 'http://marcellospastagrill.com/', 'http://www.roysrestaurant.com', '1592092690', 'http://aldi.us/index_ENU_HTML.htm', None, 'http://petersenicecream.com', None, None, None, None, 'https://www.hilton.com/en/hotels/phxclhw-homewood-suites-phoenix-chandler-fashion-center/?SEO_id=YELP-HW-PHXCLHW', None, 'http://www.apria.com', 'http://www.bk.com', None, 'https://www.caesars.com/caesars-palace/things-to-do/cleopatras-barge?utm_campaign=Yelp&utm_source=yelp&utm_medium=local&utm_term=CocktailCabaret&utm_content=entertainment', 'http://www.keshaschairlv.com', None, 'http://www.thesweetspotcleveland.com', None, 'http://www.3030southwater.com', 'http://www.presotea.ca', None, 'https://www.biscuitsrestaurants.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sunscreensbymark.com', '1592092906', None, None, 'https://www.ahnenchiropractic.com', None, 'http://www.sincerelyyogurt.com', 'http://vipbilliards.ca', 'http://www.roosterscharlotte.com/', None, 'http://www.autosood.com/fr/', 'https://www.capriottis.com', 'http://www.starbucks.com/store/11135/', None, 'http://www.americanapparel.net', 'http://www.searsucker.com/scottsdale/', 'http://www.gogofresco.org', 'http://www.frankstowngardens.com/?srccode=yelp_track', None, 'https://local.fedex.com/az/scottsdale/office-2256/', 'http://www.sheahomes.com/newhomes/charlotte', 'http://www.integratedmha.com', 'http://arcadia.redemptionaz.com/', None, 'http://www.elevationoncentral.com', 'http://www.gilbertappliancestore.com', 'http://www.firelightbarn.com', None, 'http://www.southcharlotte.youvegotmaids.com', 'http://valleyprocessservice.com', None, None, None, 'http://www.foreverdiamondsjewelers.com', 'http://newmanprop.com', 'http://www.platosclosetlyndhurst.com', 'http://drappliance.business.site', 'http://99only.com/store/Las-Vegas---Beltway/', 'http://www.azbreastfedbabies.com', 'http://www.mobileaxs.ca', 'http://firstpictureframe.ca', None, None, None, 'http://advanceddermatologynevada.com/about-us/jan-fu-md-phd-faad/', None, 'http://www.sccarcenter.com', 'https://www.choicehotels.com/nevada/boulder-city/quality-inn-hotels/nv062/rates', None, 'http://www.animalkennelcare.com', 'http://www.mccandlesscrossing.com/', None, 'http://www.kmart.com/stores/pennsylvania/pittsburgh/3529.html', 'http://www.blomedry.com', 'http://www.steve-n-kids.com', 'http://www.brickandbarleyaz.com', None, 'http://www.residoor.com', None, 'http://www.massageheights.com/matthews-festival', 'https://pranaivtherapy.com', None, 'https://dillonmcdonald.vipmtginc.com/apply/', '1592093393', None, 'http://www.spamadison.com', None, None, 'http://www.drsibel.com', 'http://www.MODERNMILK.com', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', None, 'http://www.tacotimecanada.com/locations/alberta/calgary/deerfoot-meadows', 'http://www.apple.com/retail/northlakemall/', 'http://www.starbucks.com/store/10759/', 'http://www.sharkys.com', 'http://www.riggsroadvets.com/', 'http://www.head2toesmassage.com', None, 'https://www.ikea.com/us/en/stores/las-vegas/', 'http://Geeksquad.com', None, 'http://www.nevadaheadache.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.omwest.com', '1592093574', None, 'https://nolmsted.onyxhealthclub.com/contact/', None, 'http://westgatenailsspaglendale.com', 'http://www.stjosephphoenix.org', 'https://www.dickblick.com/stores/ohio/cleveland-heights/', None, 'https://www.familydentalcharlotte.com', 'http://www.swearlv.org', 'http://www.shawarmabox.ca', 'http://Www.summitpsychologicalservices.org', 'http://www.oldnavy.com/products/peoria-az-store-6364.jsp', 'http://www.myrosatis.com/store-details/Mesa%20Greenfield', 'http://www.mamarosespizza.com', None, None, 'http://www.underwrapslv.com', 'http://www.russellhendrix.com', 'http://www.wisconsinfencing.com', None, None, 'http://www.holatapasbar.com', None, 'http://www.autonationtoyotatempe.com/', 'http://www.shawarmaave.ca', None, 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', None, 'http://www.abcsforchildren.com', 'https://locations.sonicdrivein.com/az/tolleson/9552-west-van-buren-avenue.html', 'http://www.lackscleaning.com', 'https://www.psgdental.com/locations/precision-orthodontics-lyndhurst/', None, None, None, 'https://LimitlessGraphix.com', 'http://kachinapainting.com', None, 'http://www.fuchsiaspa.com', 'https://azfunandsun.com', '1592093870', 'http://www.deserttitle.com', None, 'http://www.trundlemanor.com', 'http://www.wrapsodygifts.com', None, 'http://tsujiri-global.com', 'http://www.sundevilbookstores.com', 'http://www.sophiespizza.ca', None, 'http://greenlifelawncare.net/', None, None, 'http://www.xoxolashes.com', 'https://www.freshcoatpainters.com/glendale-az', 'http://thesmokincue.com', None, 'http://www.centennialparkgolfcentre.com', 'https://hcpnv.com/clinic/la-canada-cardiology/', 'http://www.tanglesmadison.com', None, 'http://www.valleyartcenter.org', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.chatime.com', 'https://www.choicehotels.com/arizona/phoenix/econo-lodge-hotels/az191/rates', None, None, 'http://www.angrygorillamarketing.com', None, None, None, 'http://www.premierdentalal.com', 'https://www.americanfreight.com/?utm_medium=Yelp_Website_URL&utm_source=Yelp', 'http://www.bunkerhillgolf.com/bar_and_grille/', 'http://advancedcleaningscottsdale.com', 'http://www.ckokickboxing.com/las-vegas.html', 'http://www.portcityclub.com/', 'https://locations.davidstea.com/on-markham-167', 'https://www.target.com/sl/streetsboro/2157', None, None, 'http://thebutlerdidit.info', 'http://www.rootcandles.com', 'http://mistertransmissionnewmarket.com', 'https://www.oxifresh.com/locations/az/paradise-valley-carpet-cleaning/?utm_source=yelp&utm_medium=social&promo_code=YP5000', 'http://www.lasvegaspropertyvaluation.com', None, 'http://www.mcknight-kitchen.com', 'http://www.aulddubliner.com/', 'http://www.cabanaondumont.com', 'http://www.lubie.ca/', 'http://www.seventattoostudio.com', None, 'http://www.goldwaterbank.com', 'http://yang39y.wix.com/xiangweixuan', '1592094322', None, '1592094333', 'https://www.johnsonlarsendental.com', 'http://www.mcadenvillemotors.net', 'https://locations.churchs.com/az-scottsdale-685', 'http://mauidogrocks.com', 'http://www.socialeatery.ca', 'http://www.newstylenailshenderson.com', 'http://eosfitness.com/location/centennial?utm_campaign=vegas-centennial&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, 'http://www.nicholaselectric.com', 'http://www.appetitebooks.ca/', 'http://www.futureshop.ca/', 'http://storelocator.pepboys.com/pepboys/stores/glendale/63rd_and_bell/00779', None, 'http://www.phoenix.invisiblefence.com', 'http://www.wilddogdigital.com', 'http://www.verizonwireless.com/stores/nevada/las-vegas/katie-203158/', 'https://www.davescosmicsubs.com', 'http://www.cocoshairstudio.com', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'http://pjobrokerage.com', None, 'http://www.valleywidedoor.com', 'http://www.thaifootmassageaz.com', None, None, 'https://marketbrewingco.com', 'http://groutsmithcharlotte.com', 'http://www.bestfuneralservices.com', None, 'http://www.aactionmovers.com', 'http://www.mcsorleys.ca', None, 'http://www.camelviewpt.com', 'http://vaseklaw.com', None, 'http://www.shayacuisine.com', 'http://www.pizzapizza.ca', 'https://www.moesbarbershopaz.com', 'http://www.regencyheightsapartments.com', 'http://www.asiankitchencleveland.com', 'http://www.nationalspecialtypharmacy.com', 'https://www.spencediamonds.ca/location/vaughan-on/?utm_source=Yelp&utm_medium=Display', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'http://www.valleypaincenters.com', 'http://www.pipercleo.com', 'http://www.ashleysbeautylounge.com', 'http://mobiletecsmallengines.ca', 'http://www.zamrestaurant.com/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', None, 'http://www.partybusesinvegas.com', 'https://www1.shoppersdrugmart.ca', 'https://www.etiquettesystems.com', None, 'http://www.hardrockhotel.com/las-vegas-restaurants/goose_island_pub', 'https://www.absolutedental.com/locations/las-vegas/2301-e-lake-mead/', 'http://uptowncheapskate.com/charlotte', 'http://www.dontortaco.com', 'http://www.spbcnc.org', 'http://www.seeingdoublecle.com', 'http://www.dogpolitan.com', 'http://www.bistrosirwilfrid.ca', 'http://crispycrust.ca', 'http://www.mcgoverns.biz', 'http://www.cadencehealthcentre.com', 'http://www.besolasvegas.com/', 'http://mayaclubaz.com', 'https://www.occphx.net', 'http://www.gamestop.com', 'http://Www.remosbistro.ca', 'https://www.petsmart.com/stores/us/az/phoenix-store0131.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://www.periospecs.com', None, None, 'http://www.azsingingcrepe.com/', 'http://www.andlerdental.com', 'http://bocalocaburritofactory.com', 'http://www.azbestgaragedoors.com', None, 'http://www.lazeezshawarma.com/', None, None, None, 'https://www.rightautoandtruck.com', None, 'http://www.cielostudios.ca', 'https://kirinbuffet.net', 'http://www.beatleshowvegas.com', 'http://www.metroicare.com/', None, 'https://www.bowdog.ca', None, None, 'http://www.greentreeprinting.com', 'http://www.vientianepalacerestaurant.com', 'http://jep-landscaping.com', 'http://www.germanbreadbakerylasvegas.com/', None, None, 'http://www.maxwellspine.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.greatclips.com/', '1592100552', None, None, 'http://www.drtomgreco.com', 'http://www.deerbrookefarm.com', None, None, None, 'https://americanaddictioncenters.org', 'http://www.marriott.com/cmpts', 'http://www.myspace.com/sweetsandbeats', 'https://locations.jackinthebox.com/us/az/scottsdale/20385-n-hayden-rd', 'http://www.budsayathaispalv.com', 'http://www.coderedplumbing.com', 'http://www.abcbartending.com/vegas/index.htm', 'http://www.starbucks.com', 'http://www.silversagebeef.com', None, 'http://lunchboxcafeaz.com', 'http://www.onedaypittsburgh.com', None, 'http://coolblew.com', None, None, 'http://www.oreilyauto.com', 'http://www.lvchamber.com', None, None, 'http://www.owloptometry.com', 'http://www.approvedaz.com', 'http://www.horizonlaservision.com/', None, 'http://zulmamassagetherapy.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.vapeandco.com', None, 'https://goo.gl/2UEpRZ', None, 'http://www.322PizzaBar.com', '1592100862', 'http://www.blackmountainbmx.com', None, 'http://www.dunkindonuts.com', '1592100900', None, 'http://www.crystalrivergems.com', 'http://swimmingpoolslasvegasnv.com', None, 'https://satruck.org', 'http://www.nickscalgary.com', None, None, 'http://www.caldensebakery.ca/', None, 'http://www.azsprinklerpros.com', None, None, None, 'http://www.oakridge-kennels.com', 'http://www.qdoba.com', 'http://www.arizonalodgingexperts.com/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282947950&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.vegasconciergedoctor.com', None, 'http://www.teaspressa.com', 'http://mdrainlaw.com', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.dynastyirondoors.com', 'http://www.pineviewauto.com', 'https://www.extraspace.com/storage/facilities/us/ohio/kent/501645/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.crazymocha.com', 'https://www.royalautosales.ca', 'https://automotivediagnosticspecialties.com', 'http://www.greatescape-to.com', 'http://www.greatclips.com', None, 'https://rigatonys.com', 'http://www.sunrisemontessori.com', None, None, 'http://asavvyskincare.net', 'http://www.gabyetjules.com/', 'http://www.tcvintageaddiction.com', 'http://www.starbucks.com/store/9298/', 'http://www.CompleteLaserAZ.com', 'http://www.expressscreenprinting.com', None, None, 'http://www.tshirthell.com/', None, 'http://www.katkuphotography.com', 'http://www.greatclips.com', 'http://www.nutbrownschristmastrees.com', '1592101294', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3075+West+Liberty+Avenue-Dormont-PA-15216/storeid=4144?cid=ps_ylp_2020', 'http://www.mrshawarmaairdrie.ca/', None, 'http://www.harborddentistry.com', 'http://porterhouseto.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Pittsburgh-PA-15237/809064/', 'http://www.ihg.com/hotelindigo/hotels/us/en/scottsdale/phxin/hoteldetail', 'https://www.homedepot.ca/en/home.html?eid=LS_Google_7061', None, 'http://www.goldes.com', None, 'http://www.splitsville.ca/locations/woodbridge-bowling', '1592101389', None, 'https://www.target.com/sl/chandler-south/2747', 'http://www.worldscoutingmuseum.org', 'http://fuyaosupermarket.ca', None, 'http://www.scottsdaleaz.gov/parks/scottsdalemall', 'http://www.valleyinteriors.com', None, '1592101459', 'https://www.redradishcharlotte.com', None, 'http://www.matsudasushi.com/', None, 'http://www.starbucks.com/store/10759/', None, 'https://www.wellsfargo.com', 'http://www.stowglass.com', 'http://www.ramsey-auto.com/', 'http://mazzeolaw.com', 'https://www.tech1auto.com', '1592101541', 'https://www.suncountryheatingandcooling.com', None, 'http://www.taylorsflooringaz.com', 'http://www.asgnv.net/', 'https://www.autozone.com/locations/nv/las-vegas/1351-n-eastern-ave.html', None, 'http://cpllabs.com/', 'http://www.dandelionretro.com/', 'http://www.freshkabob.com/', None, '1592101619', 'http://www.restorationhardware.com/', 'http://serenityspapeoria.com/', 'http://www.bannerhealth.com/locations/avondale/banner-physical-therapy-mcdowell-107th?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', None, 'http://www.phoenixcountryclub.com', 'http://www.krauterdds.com', None, 'http://sthielpilates.com', 'http://www.advanceddermatologylv.com/', 'http://www.UltraCleanExpress.com', 'http://www.littlecaesars.com', 'http://evansotg.com', 'http://www.pitabasket.com', None, 'https://www.ulta.com/stores/surprise-az-176', None, 'https://www.scaddabush.com/sherway/', 'http://www.carltonsharley.com', None, 'http://shadysidedetail.com/', 'https://www.hendersondentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://vbarbershop.com/locations/gilbert/', 'https://www.lifestorage.com/storage-units/arizona/phoenix/85234/7110-in-gilbert/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-AZ-7110&utm_content=website-link&howheard=IYE', None, None, 'http://karsonskornbread.com', 'http://www.starwoodhotels.com/sheraton/property/dining/index.html?propertyID=205', 'http://www.pi-co.ca', 'http://www.itsbarmagic.com', None, 'http://www.painconsultantsofarizona.com', 'https://www.popeyes.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://harbour60.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.kidsacademyofcopley.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PITMC', None, 'http://www.marriott.com/hotel-restaurants/phxcb-jw-marriott-scottsdale-camelback-inn-resort-and-spa/blt-steak-scottsdale/125727/home-page.mi', 'http://www.mobilicity.ca', None, 'https://www.cricketwireless.com', 'https://www.ycsignsllc.com', 'http://www.fantasticfiredept.com/', 'http://www.busybodiesplayground.com', 'http://exquisitemobiledetailingllc.com', None, 'http://www.imperialflowershop.net/?srccode=yelp_track', 'http://www.yogaplustoronto.com', 'https://www.myessentia.ca/fr/magasins/laval/', 'https://torontovaporizer.ca', 'http://www.GurrBrothers.com', None, 'https://www.cleoms.com/meet-us/meet-the-doctor/dr-urban-picard/', None, 'http://thaipepperlv.com', 'http://www.orthoarizona.org', 'http://www.tirekingdom.com', None, None, 'http://www.barbsbridalboutique.com/', None, 'http://johnandsonsoysterhouse.com', 'http://www.allamericanlandscape.net/', None, 'http://www.kickerspizza.com', 'http://www.jlcolesalon.com/', 'http://obsessionbeautysalon.com/', 'http://www.allstateselfstorage.com/self-storage/bell-85032', 'http://www.risingphoenixllc.com', '1592102184', 'http://www.rasputinvodkabar.com', 'http://www.hkasiancuisine.com/', 'http://prioryfinepastries.com', 'http://alnolashoerepair.com', None, 'http://mvrwholesale.com/', 'http://www.rankinwomenscenter.com', 'http://www.curvyconsignments.com', 'http://www.krantzelectricinc.com/', 'http://www.gilcontool.net', 'http://www.themobilewebapps.com/repairwatchandjewelry', 'http://7eleven.ca/store-locator/33259', 'http://queencitylawn.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=2767&utm_content=bpas', None, None, None, None, 'http://www.hauntedpittsburghtours.com', 'http://www.pebooks.com/lv2/', 'http://www.kavicwinery.net/5701.html', 'http://www.waincapital.com', 'https://www.dentalsummerlin.com', 'http://slslasvegas.com/thebuffet', 'http://swensonelectric.com', None, 'http://www.westernexterminator.com/las-vegas', 'https://www.lebos.com', 'http://www.figarosbarbershop.com/', 'http://www.totallychiinc.com', 'http://www.fiddlersgreenpub.ca/', None, '1592102421', 'http://www.druglessdrs.com', 'http://www.learningrx.com/akron-bath', 'http://projectrefinedlife.com/group-private-yoga/', 'http://www.essenceairllc.com', 'http://www.amazingyoga.net', 'http://www.FatAngelosPizza.com', 'http://blinkeyewear.ca', None, 'http://www.bufords.kitchen/locations/downtown/', 'http://thenailspapgh.com/', 'http://lawanthaikitchen.com', None, 'http://harrisonautos.com', 'http://www.poppydenvegas.com/', 'http://www.reveregolf.com', 'http://clarkcountypoolandlawn.com', 'http://www.carpetsharkslv.com', 'https://www.summerlin.com/downtown-summerlin/overview.html', 'http://velvetlanecakes.com', 'http://www.cameostudio.com', 'http://1therapeuticmassage.wix.com/massage', 'http://www.muchoburrito.com', 'http://athleticmindedtraveler.com/browse-cities/toronto-on/university-of-toronto-athletic-centre', 'http://californiapools.com', None, 'http://www.apricotflowers.com', 'http://www.avon.com', None, 'http://www.MaderaPoint.com', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.mindfulwellnessaz.com', 'http://www.opalskinwax.com', 'http://www.sunkistpoolslv.com', None, 'http://www.greenroadpediatrics.net', 'http://www.room1six.com/', 'http://corktowndental.com', 'https://www.massageaddict.ca/Toronto-TheBeach', 'https://www.changingtimesdiaperco.com', 'http://www.zefiessmoothies.com', 'http://dynoplumbinglv.com', None, 'https://www.permenterchiropractic.com', 'http://scofieldrealty.com', None, None, 'https://stores.hallmark.com/nv/lasvegas/amys-hallmark-shop-lasvegas-nv-63960.html', None, 'http://www.globex2000.ca/fr', None, 'http://www.awrswheelrepair.com/pittsburgh', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Toronto-ON-M9M2R5/848075/', '1592102864', 'http://www.gnc.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.burritoboyz.ca/', 'https://ca.tokyosmoke.com/pages/queen-west', 'http://copperwillowsalon.com', 'http://www.stoughtonbuyandsell.com', 'https://integritydry.com', 'http://zero20kids.com/', None, 'http://walmart.com/', 'http://www.motel6.com', None, 'http://www.caldensebakery.ca', 'http://appliancewarehouse.com/', None, None, 'http://www.cafedejapon.com/', 'http://drshawnae.com', 'http://parrcabinet.com', 'https://www.towndentalgroup.com', 'http://www.theblindmannc.com', 'https://storelocations.ae.com/us/az/phoenix/4550-east-cactus-road.html', None, 'https://www.springspreserve.org/visitor-information/divine-cafe.html', 'http://www.solutionbuild.ca', 'http://www.northwestlimo.com', 'http://fairtradecoffeehouse.com/', 'http://www.starbucks.com/store/10759/', None, 'https://locations.wendys.com/canada/on/ajax/80-bayly-street-west', 'http://www.fredsheatandac.com/', '1592103130', 'http://newyork51pizzeria.com/', 'http://www.starbucks.com/store/14175/', 'http://www.radichisalon.com', 'https://encompasshealth.com/valleyofthesunrehab', 'http://www.fantasyautoinc.com', 'http://www.elitebusinessfilms.com', 'http://shop.pistolwhippedpastry.com', None, 'http://www.evanjstrong.com/', 'http://www.cbsquaredhairstudio.com', None, '1592103225', 'http://www.clevelandprintwear.com', None, 'http://www.jolantainteriors.com', None, None, None, None, 'http://www.pisausa.com', 'http://www.bashas.com/', 'http://www.playitagainsportscfalls.com', 'http://www.rvpaintrepair.com', 'http://www.janetkirkman.com/', 'http://www.ccvglass.com', 'https://www.massagespacharlotte.com', None, 'https://www.ritasfranchises.com/Wexford', 'http://bogartspet.com', 'https://www.jerseymikes.com/9003', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Tempe-AZ-85213/723082/', None, 'http://www.beyondconceptionmidwifery.com', 'https://www.24hourfitness.com/gyms/las-vegas-nv/rainbow-las-vegas-supersport', 'https://craftypots.studio', None, 'https://ad.doubleclick.net/ddm/clk/318369417;146857521;q?https://www.safelite.com/stores/Mesa?utm_source=yelp&utm_medium=list&utm_campaign=Addressblock&utm_content=85210_AZ', 'https://vspa-massagespa.business.site', 'https://winecollective.ca/highfield-store', 'http://www.thetoothwhisperer.com/index.html', 'https://opticas2020.business.site', None, None, 'https://drbeverlysmathisel.wixsite.com/drbeverlysmathisel', None, 'http://www.echosalonphoenix.com/index.html', 'http://www.guyspizzaco.com', None, 'http://www.dphairdesigns.com/', 'http://www.markmitsuphoenix.com', 'http://beegee-kitchen.com/', 'https://www.wm-columbuspark.com', 'http://www.infinitebeautysalon.com', None, 'http://www.sundevilauto.com/locations/az/mesa/east-mesa-power-26', 'http://www.flowersoftheworld.ca', 'http://patonthebackspa.com', 'http://pdrpizza.net', None, 'http://bankerinsulation.com', 'http://www.mainstreetcupcakes.com', None, 'http://www.veinitymd.com', 'http://mcfaddendalehardware.com', 'http://www.massagenirvana.com', 'http://www.timhortons.com', None, None, 'https://www.lush.ca/on/demandware.store/Sites-LushCA-Site/en_CA/Stores-Info?StoreID=129', 'http://www.acrepair4less.com', 'http://locations.jambajuice.com/ll/US/AZ/Scottsdale/15686-N_-Frank', 'http://www.timhortons.com/', None, 'http://www.GreaterPhoenixAreaRealEstate.com', 'http://xtremeairparks.com', 'https://www.steaknshake.com/locations/nv-las-vegas-las-vegas-boulevard-south', 'http://cambiedesign.com', 'https://ca-en.caudalie.com/boutique-caudalie-sherway-gardens-toronto', None, None, None, None, None, 'https://karachi-kabab-wala.business.site', 'http://www.azcellphones.com/', 'http://kjscurrybowl.com', None, 'https://grantmma.ca', 'http://phoenix-phx.worldairportguides.com/terminals.php', 'http://www.oldtimecandy.com', 'http://americanhomewater.com', 'http://www.footsolutions.com/summerlin', 'http://www.brownsshoes.com', None, None, 'http://eyecandispa.com', 'http://www.seanssalon.com', 'https://www.barlouie.com/locations/us/az/tempe/tempe-marketplace?utm_source=other&utm_medium=listings', 'http://www.bikramyoganphx.com/', 'http://www.nationalgolfacademy.ca', None, 'http://www.luxurymotorsports.us/used-luxury-cars-phoenix/', 'http://zcoilcf.zcoil.com/137/', None, 'http://www.fatbobpizza.com', 'http://www.swarovski.com/en-US/?utm_source=YELP&utm_medium=referral_managed&utm_campaign=YELP', 'http://www.west49.com', 'http://geigerfitness.net', 'http://Clearqualityautoglass.com', None, 'http://www.zenshinasianrestaurant.com', 'http://www.veginyyc.com', 'https://www.californiaclosets.com/locations/greater-phoenix/scottsdale-showroom/?CC5=32008&CC6=30593&utm_source=directory&utm_medium=local&utm_content=AZ166-M1-SR2', 'https://lulasewickley.com', 'http://www.fatbastardburrito.ca', 'http://www.wellsfargo.com', 'https://www.gordmans.com', 'http://www.chagrinfalls.net/', None, None, None, 'http://www.coconutclubbargrill.com', 'http://www.imaginedentistry.com', 'http://oldchicago.com/locations/champaign', 'https://www.interdent.com/gentle-dental/locations/az/mesa/1425-s-greenfield-rd-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.haysroofing.com/', 'https://www.integritycontentsservices.com', None, 'http://smilesourceonchagrin.com', 'http://etonhouse.ca', 'https://tsperformance.net', 'http://www.dressbarn.com', 'http://www.buildabear.com?sc_cid=ds:yelp:profile', 'https://locations.theupsstore.com/pa/harrisburg/4075-linglestown-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://greenwoodrentals.ca/', 'https://www.bikesourceonline.com/articles/bikesource-denver-pg502.htm', None, None, None, 'http://www.signaturehealthinc.com', 'https://www.snakesandlattes.com/location/tempe/', None, 'http://yourspacesalons.ca/tonyc', 'http://www.americanairappliance.com', None, None, 'http://www.theacvh.com', None, 'http://www.thisaint.ca', 'http://www.lasvegaswoodflooring.com/', 'https://www.insomniacookies.com/locations/store/1129', None, 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', 'http://shell.us', 'http://www.ghost-armor.com', 'http://www.thegunsmith.com/thegunsmith.html', 'http://treasureisland.com/restaurant/13/the-buffet-at-ti', None, None, 'https://www.rotorooter.com/murrysvillepa/', None, 'http://harrys-market.business.site', None, 'http://shop.hobbylobby.com', 'http://www.noodles.com/locations/south-end', None, 'https://vindalooindiancuisine.business.site', 'https://www.bowlero.com/location/bowlero-avondale?utm_campaign=webclicks&utm_medium=organic&utm_source=yelp', 'http://yardzatmirabelli.com', None, None, 'http://www.chefclaudio.com', None, None, 'http://rockfords.ca/', None, 'http://dannysdeli.net', 'http://www.foodcity.com', 'https://www.bnc.ca/particuliers/cartes-credit-mastercard/salon-aeroport-montreal.html', 'http://www.allforfun.ca', 'http://www.elyspringvalley.com', 'http://www.clubcorp.com/Clubs/Bear-s-Best-Las-Vegas/', 'http://www.dollartree.com/home.jsp', None, 'http://www.jriac.com', 'http://www.starbucks.com', 'http://www.cevw.com', 'http://realsh1ft.com', 'http://www.premiercustombuilders.com', '1592104749', 'http://www.scorelv.com/', 'http://bellaluna.us', 'http://www.jaaz.org', 'https://local.fedex.com/nc/charlotte/office-0746/', None, 'http://www.habibspersiancuisine.com', 'http://www.YouthfulDaySpa.com', None, 'http://morfinserv.com/', 'http://destinychurchlv.com', 'http://www.cabanasislandrestaurant.com', 'https://www.wyndhamhotels.com/hotel/46608?tel=18003370400&iata=00093785&cid=fe:bu:20170523:yl:pp:bayus:46608', None, 'http://downtowndetailslv.com', None, None, None, 'http://sonoranskydental.com', None, 'http://www.northcourtbeverage.net', 'http://www.restaurantdepot.com', 'http://www.hardrockhotel.com/las-vegas-restaurants/johnny-smalls', 'http://www.phoenixbimmer.com', 'https://azdonerightroofing.com', None, 'http://www.atozpediatricdentistry.com', 'https://www.titc.us', 'http://www.eastsidemarios.com/locations/ontario/toronto/scarborough/', 'https://www.assurant.com', 'http://www.westcheyennepharmacy.com', 'http://taylorhouselaw.com', 'http://kingsfamily.com/?utm_source=yelp&utm_medium=referral', None, None, None, None, None, 'http://romanticaboutique.ca', 'http://www.marriott.com/hotels/travel/cltcm-courtyard-charlotte-matthews', 'http://eagleriderhonda.com', None, '1592105051', 'http://www.hungryhollow.ca/', 'http://www.chandlercourtamc.com/', None, None, 'http://www.ultimatesportscards.com', '1592105091', 'https://www.arizonamovingcompanies.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282947950&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', None, 'https://calibercollision.com/locate-a-caliber-collision-center/charlotte-tyvola-rd', 'http://www.totalcigars.com', 'http://www.madisondancelife.com', 'http://www.pghgrille.com', 'http://www.allianceurgentcare.com', 'http://yafokitchen.com', 'http://www.abcautobodylv.com', None, 'http://www.boracaymassagevegas.com/', None, 'http://www.vetcoclinics.com', 'https://saharalasvegas.com/restaurants-bars/beers-and-bets/', 'http://www.robcoelectriclv.com', '1592105228', 'http://www.veneziabakery.com', None, 'http://www.summerspraytan.com', '1592105273', 'http://www.fresh-carpet-care.com/p/693?utm_source=bing&utm_medium=cpc&utm_campaign=EOW%20-%20Exact%20-%20Phoenix&utm_term=carpet%20cleaning', None, 'http://www.gordonwoodoptical.com', None, 'http://www.winghouseandsportsbar.com/', 'http://www.deltaco.com', 'https://www.obonsushi.com/scottsdale-quarter', 'http://www.valleyprotectiveservices.com/phoenix', 'http://www.wildgamegrillaz.com', 'http://thejewelers.com/venetian.html', 'http://www.sushiakaihana.com', 'http://www.wpahumane.com', 'http://menuderiaguanajuato.com', 'http://lambrettapizzeria.ca', 'http://www.clarkcountycourts.us', 'https://i-hvac.net', 'http://www.ichiriki.ca/', None, 'http://www.wordoflifelv.com', 'http://www.pizzaspano.ca', 'http://www.oakley.com/en/store-finder', None, None, None, 'http://overstockvehicles.com', None, 'http://hophausbrewing.com', 'http://www.pmilv.com', 'http://phoenixportablemobilewelding.com', None, None, 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=106&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-106', 'http://www.oldewinecellar.com', None, 'http://www.east4thstreet.com', None, 'http://www.synergyhomecare.com/agencies/az/sun-city/az02/', None, 'http://www.wildpurveyors.com', 'http://www.4monkeybusiness.com', None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', '1592105619', None, 'http://www.starbucks.com/store/14175/', None, 'https://independenceveterinaryclinic.com', None, 'https://www.choicehotels.com/CN281?mc=blypypaf', None, 'https://opasouvlaki.ca', 'https://www.lasvegassedation.com', 'http://www.mondrianscottsdale.com/mondrian_hotel_scottsdale_redbar.asp', 'http://affordablebailbondsphoenix.com/', None, None, 'http://www.todays-market.com', None, 'http://www.jazhairstudios.com/', None, 'http://www.dwellatregency.com', None, 'http://www.skinplicitylaneese.com', None, None, 'http://www.peterpiperpizza.com/', 'http://www.BlueStateElectric.com', 'http://www.thesourcemenswear.com', 'http://pizzaandbeyond.blogspot.com/', 'https://cobblestone.com/?utm_source=yelp%C3%8A', None, 'http://www.alohausedcars.com', 'https://www.carenow.com/locations/las-vegas/eastern-horizon-ridge', None, None, None, None, None, None, None, None, 'https://www.covenantcare.com/stores/silver-hills-health-care-center/', None, 'https://www.bestwestern.com/en_US/book/hotel-rooms.66086.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66086', 'http://avolsbookstore.com', 'http://www.mrshuizenga.com/', 'https://el-mesquite-cocina-mexicana.business.site', 'http://www.arnoldcrescentanimalhospital.com', 'http://www.valorestaurant.com', 'http://www.estheticssalonandspa.com', '1592105962', 'http://www.vape29.com', None, None, 'http://www.akronmassage.co', 'https://www.eddiev.com/', None, 'http://www.lcbo.com', None, None, 'http://www.dynamitecustomsoundsolutions.com', 'http://lacubana.ca', 'http://www.juiceandjava.ca', 'http://liftedtrucks.com', 'http://www.awildhairvegas.com/', 'https://lasvegasliving.com/properties/inspire/', 'https://www.thesteelecreekdentist.com', 'http://www.abbybelladance.com', None, 'http://www.roomstogo.com/stores/pineville/carolina-place-store/1601#utm_source=yelp&utm_medium=localsearch&utm_campaign=yelp', 'http://www.scottsdaleplaza.com/dining/jds-sports-pub-en.html', 'http://www.winners.ca', 'http://www.michaelangelos.ca', 'https://www.kohls.com/stores/pa/pittsburgh-293.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://lifedesignevents.com', 'http://www.rmcf.com', None, 'http://www.nhlakesidefamilyphysicians.org/locations--hours/prosperity-church.aspx', 'http://www.friartuckonline.com', 'http://www.getacoolbox.com', 'https://www.motel6.com/en/motels.az.phoenix.18.html', 'http://www.rockerzandrollerz.com', None, 'http://www.vegasemployment.com', None, None, 'http://kahunaville.com/', None, 'http://www.bathandbodyworks.com', None, 'http://thepinkdoorcarefree.com', 'http://www.scottsdaleaz.gov', 'http://www.midibistro.ca/', None, None, 'http://www.kingdomtitle.net', 'https://www.tdcanadatrust.com/products-services/banking/index-banking.jsp', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.liveatsanmelia.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=organic', None, 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'https://locations.monro.com/oh/bay-village/609-dover-center-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, 'http://www.oakridgepizza.ca', 'http://www.myspace.com/ronnieslocal069', 'http://barbershopstyles.com', None, 'http://www.spotteddogmenu.com/', 'https://www.susanvthomaslaw.com', 'https://www.rittersdinerpittsburgh.com', None, 'http://www.rocklandtrailers.com', 'http://azcustomcoatings.com', 'https://www.bannerhealth.com/locations/peoria/banner-health-clinic-vistancia', 'http://www.bestarizonacars.com', 'http://www.kay.com', 'http://www.digitaljuiceagency.com', 'http://www.bzbarandgrill.com', '1592106459', None, None, 'http://www.aspenplumbingandrooter.com', 'http://www.allcaremedicine.com/', 'http://patriatoronto.com/', 'http://www.awomansplaceobgyn.com', None, None, 'http://www.karlsqualitybakery.com', None, None, None, '1592106550', 'https://www.popeyes.com', None, None, 'http://www.michaelangelosphotography.com', None, 'https://www.wesoothesoles.com', None, 'http://www.thechickery.com/', 'http://www.dunkindonuts.com', 'http://www.treats.com', 'https://myneighborhoodprimarycare.org', 'http://www.scottsdaleaz.gov/trolley/neighborhoodtrolley.asp', None, 'http://www.lemongrassthaicafe.net', 'https://www.bowlero.com/location/bowlero-via-linda?utm_source=yelp&utm_medium=organic&utm_campaign=webclicks', 'http://www.framesunlimited.com/locations/shaker-heights', None, None, None, 'http://www.nicks-automotive.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.goodwill.ab.ca/goodwill-location/varsity/', 'http://www.icaptureevents.com', 'http://www.trubuildersaz.com', 'http://www.formulakartways.com', None, 'http://chandlerdrivingschool.com', 'https://www.jacksonhewitt.com/Tax-Preparation-Services-Locations/NV/North-Las-Vegas/16008/', 'http://www.sushihousebrampton.ca', '1592106768', 'http://www.thehickorytavern.com/home/hickory_tavern-locations.php?l=Charlotte+-+Providence+Rd.', 'http://www.re-wrap.ca', None, 'http://availvapor.com', 'https://threemonksandaduck.ca', 'http://www.hihealth.com', 'http://www.starbucks.ca', 'https://www.keendogtraining.com', '1592106832', None, 'http://www.kangamooplay.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://sunrisemechanical.com/areas-served/peoria-az/', 'http://www.caminoalnorteapts.com/', 'http://www.bidwellhomeimprovement.com', 'http://www.bookbeautician.com', 'http://www.marcellos.ca', None, '1592106911', 'http://www.theroyalambassador.com', 'http://nextlevelcleaners.com/', None, 'http://www.ogdenfuneralhome.com', 'https://www.starbucks.ca', None, 'http://highlifesmokes.net', '1592106971', 'http://www.goodlifefitness.com/locations/ontario/northyork/don-millseglinton?source=yelp-websitelink-northyorkdon-gl', None, None, 'https://www.elementballantyne.com', 'http://tapeface.tv', 'http://www.hussongslasvegas.com', 'http://tidycleaners.ca', 'http://www.hksarizona.com', 'http://www.prettyininkpermanentmakeupaz.com', 'http://www.thefarmersfork.com', 'http://www.ibarbieri.com', 'http://www.barbiesgrill.com/', None, 'http://www.vegasgoldbuyers.com', None, 'http://www.anthemlandscape.com', None, 'http://www.erinssnug.com/', '1592107116', None, 'http://www.gelato64.com/?page_id=409', None, 'http://mesafamilydentistryaz.com/', 'https://locations.dennys.com/OH/ELYRIA/247158', 'https://www.closetworld.com', None, 'http://segwaylasvegas.com', None, None, 'http://getahaircutlv.com', 'http://www.ticozaz.com', 'http://www.brooksdiamonds.net', '1592107221', 'http://www.giadasitaliancucina.com', 'http://stores.barnesandnoble.com/store/2680', 'http://www.bamboobistrolasvegas.com', 'http://cestbonrestaurant.com/', '1592107261', 'http://www.euro-fine.com', None, 'https://www.wrightlawnv.com', 'http://www.booksamillion.com', None, 'http://ballersbistro.com', None, None, None, None, 'http://www.bestwaypoolservice.com', 'http://www.vfwpost7401.com/', 'http://30minutealterations.com/', 'http://www.auroranaturalmedicine.com', 'http://littlecaesars.com/', None, 'http://www.yaymaker.com', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.SecureDataRecovery.com/Charlotte', 'https://smartstopselfstorage.com/storage-locations/usa/nevada/las-vegas/smartstop-self-storage-4851-e-bonanza-rd', 'http://phoenixarc.salvationarmy.org', 'http://www.timbermesaoutdoors.com', '1592107451', 'https://www.chilis.com', 'http://www.phoenixvirtualtour.com', None, 'http://gorgeouslady.ca', 'http://www.tinderboxcigars.com', None, 'https://primpandblow.com/locations/primp-and-blow-chandler/?UTM=yelp', None, None, 'http://www.aladorpetresorts.com', 'http://www.bcbg.com', 'http://feelslikehomefarm.com/index.html', 'https://www.circlek.com/store-locator/us/ruskin/3602-hwy-41-s/2707247', None, None, None, None, None, 'http://www.kazesushi.ca', None, None, 'http://www.fabulousmanicurebar.com', 'http://www.gtatireandauto.com', 'http://www.mannyforinsurance.com/?cmpid=UGOG_BLM_0006', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', 'http://www.torontoairportlimo-taxi.ca', 'http://Www.FlamingoCoolsVegas.com', 'http://www.secondcup.com/location/toronto-355-danforth-avenue', 'http://www.jdbyrider.com/buy-here-pay-here-pittsburg-monroeville-pennsylvania-pa114', 'http://www.djinlasvegas.com', 'http://freshco.com/stores/bloor-dundas', 'http://www.totalwine.com/store-info/arizona-phoenix-desert-ridge/1003', 'http://www.ggfarm.com/welcome-to-grateful-growers/harvest-moon-g', None, 'http://www.apassionforpetssalon.com', 'http://www.trulythaicuisine.com', 'https://www.scottsdalecfdh.com', 'https://www.legendatarrowhead.com', 'http://vehiclesunlimitedinc.com', 'http://vegasairduct.com', None, None, 'http://www.thetcmclinic.com', None, 'http://www.verizonwireless.com/stores/colorado/denver/16th-st-mall-201560/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://www.plazaloftskierland.com/', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'https://www.neohioeyes.com', 'https://www.enterprise.com/en/car-rental/locations/us/az/tempe-holdeman-5015.html?mcid=iyp:8368400', 'http://www.chelseanicole.com', 'http://www.morningsidenissan.com', None, '1592139482', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/yyzsb-delta-hotels-toronto-east/', '1592139508', None, 'https://www.chocolatebywickerhead.com', None, '1592139537', None, 'https://www.kneadtodestress.com', '1592139573', 'http://www.holeeburger.com/', 'http://www.gttdogs.com', None, 'https://www.faded.club', None, 'http://www.autoberry.ca', None, 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=441', 'http://www.pandaexpress.com/', 'http://www.mistyglenkennel.com', 'http://www.krispykremeaz.com', 'http://nsbank.com/', '1592139673', 'http://www.vivefitness.com/', 'http://www.mchair.com', 'http://purefitnessclubs.com/', 'http://www.bstreetrmt.com', 'http://www.rungreatrace.com/', None, 'http://lasvegas.777rentals.com', 'http://www.philipamodei.com/lasvegashairstudio/contact', 'http://www.moongsushi.com', 'http://www.capitalone.ca', '1592139757', 'https://www.discounttire.com/store/az/scottsdale/s/1284?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://www.cmgfi.com/agents/richard-blair', None, '1592139794', 'http://www.lvboneandjoint.com', None, 'http://www.eatgarbanzoaz.com/', None, None, None, 'https://lumenbrite.com/locations/phoenix/', 'http://www.Topspizza.ca', None, None, None, 'http://chefkennysasianvegan.com', None, 'http://www.safestoreaz.com', 'http://www.foothillsrehab.com/', 'http://www.25thstreetautomotive.com', 'https://cobblestone.com/?utm_source=yelp%C3%8A', '1592139942', None, 'https://www.bankofamerica.com', 'https://www.purebarre.com/location/westlake-avon-oh', 'http://www.tigerslairaz.com/', None, 'http://adavenkid.com/', 'http://www.shoregrillandgrotto.ca', None, 'https://www.advancedfootankle.com', 'http://www.insideoutsports.com', 'http://roysliquor.com', 'http://www.bestchoice4insurance.com', 'http://www.redkphoto.com/', 'http://www.firstchoicedental.com/locations/sun-prairie-office', None, 'https://www.target.com/sl/henderson-lake-mead-pkwy/2404', None, None, '1592140058', 'http://substationaz.com', 'http://searchthec4group.com', None, 'https://www.zimbrickaudiofmadison.com', 'http://www.mercuryautospa.com', 'http://mtclaret.org', None, 'https://www.nothingbundtcakes.com/bakery/nv/las-vegas-eastern?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=las-vegas-eastern_2', 'http://bobbyqgrill.com/', 'https://www.spring-green.com/franchise/urbana-area/', 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/etobicoke-(1).aspx', 'https://www.kohls.com/stores/az/scottsdale-620.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.abovegroundartsupplies.com/', None, '1592140190', None, 'http://www.brighterangels.com/phoenix.html', 'http://croywifi.com', 'http://www.jlbridal.com/', 'http://www.thepaperytoronto.com', 'http://brocach.com', 'http://www.balancingyourchemistry.com/', 'http://deeplyrootedhairstudio.com', None, 'http://www.chennaichettinaadpalace.com', None, None, '1592140276', 'http://gohomeplate.com', '1592140287', 'http://www.lvac.com/locations/central', 'http://www.amici-dinos.com', 'http://greenfieldjewelers.com', 'http://oldnavy.gap.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.spoonandfork.ca/spoon-and-fork-eglinton/welcome', None, 'http://fpcmesa.org', 'http://www.goodlifefitness.com', 'http://www.theecigflavourium.com', 'http://www.milagrorestaurant.com', None, 'https://www.thingsremembered.com', None, 'http://www.sambabraziliangrill.com/', None, None, 'http://fusionpowerco.com', 'http://www.michaelkors.com/', 'https://shop.lululemon.com?CID=ps_YELP_website_org', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://agents.farmers.com/az/tempe/greg-krajewski?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'https://www.sephora.com/happening/stores/pittsburgh-shadyside', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://skyvapeaz.com', 'https://pablocanada.com/shop/', 'http://www.nationalcarmart.com/', 'http://home-savings.com', 'http://chopstickexpresstogo.com', 'http://www.pathwayscounselingsvcs.com', 'http://www.petsittingscottsdale.com/', 'http://www.woodbridgevacuum.ca', 'http://www.golfwildhorse.com/', 'http://www.jennysdispensary.com', None, 'http://thericegarden.com/', None, 'https://locations.jackinthebox.com/us/nc/charlotte/7725-pineville-matthews-rd', 'https://performancetherapymassage.net', 'http://www.shinbay.com', 'http://www.solapizza.com', 'http://www.gilbertserenitydental.com/', 'http://jollyrollsicecream.com', 'http://browsbyhilary.com', None, 'http://www.adventdentistry.com/', 'http://www.mastersgalleryltd.com/', 'http://smog-king.com', None, None, None, 'http://www.a-ztennis.com', 'https://www.cellairis.com/gilbert-cell-phone-iphone-repair-yelp', 'https://made-in-argentina-la-esquina-argentina.business.site', None, 'http://www.mustardseedherbary.com', None, 'http://ivorystar.com/shows/friends-the-musical-parody', None, 'http://FoxtailLasVegas.com', 'http://www.starbucks.com/store/1006250/ca/jean-adam-st-denis/75-avenue-de-la-gare-saint-sauveur-qc-j0r-1r6/les', 'https://www.plumbcrazy4plumbing.com', 'https://www.enterprise.com/en/car-rental/locations/us/al/huntsville-madison-55ha.html?mcid=iyp:8368400', 'http://gamblerscafe.com', None, None, 'http://www.themachatgroup.com', 'http://www.mikesbarbershopmentor.com', '1592140868', None, None, 'http://chaseanddestroy.com', None, None, 'http://andersonac.com', None, 'http://www.phoenixazchinesefood.com', None, 'https://www.allnevadainsurance.com', 'https://thepolishedbrow.com', 'http://muramoto.biz', 'https://palmbeachtan.com/locations/OH/Westlake-Detroit-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=CLE', '1592140999', None, 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', None, 'http://www.campusbodysalon.com', 'http://www.lovesick.com/store-details?StoreID=9020', 'https://www.infinityfinancialmortgagecorp.com', 'http://www.atlwingsyourway.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.lvatvtours.com/', None, None, None, '1592141121', 'http://www.psychiatrylasvegas.com', 'http://www.all-starpediatrics.com', None, 'http://roomofonesown.com/', 'http://www.unityrideshop.com', 'http://villemontpizza.com/index.html', 'http://www.2bmod.com', '1592141176', 'http://washworkslaundry.com', 'https://highweedmedshop.com', 'https://mrsub.ca', 'http://gradeaid.ca', None, None, None, None, 'http://www.mrrooter.com/phoenix', None, None, None, 'https://www.sheetz.com', None, 'https://www.wyndhamhotels.com/hotel/36226?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:36226', None, 'https://www.coffeebean.com/store/usa/scottsdale/promenade-at-scottsdale', 'http://www.clearwash.ca', 'http://www.carolinavaluevillage.com', 'http://www.sincitycrossfit.com/', None, 'http://www.epennzoil.com', 'http://www.bojangles.com', 'https://www.americafirst.com/about/branches/branch-details.cfm?siteNo=233', None, 'https://www.sprint.com/storefronts/bd/sprint-south-euclid-oh-44118-fcs-1799/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://townhall.innis.utoronto.ca', 'http://www.azgreencleaning.com', 'http://pa.pittsburgh.charmingcharlie.com/fashion_accessories_pittsburgh_pa_128.html', None, 'http://www.kickaxeguitars.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YULRI', 'http://www.sunlessboutique.com/', 'https://www.dominos.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://redhousepub.co', 'http://zellaspizza.com', 'http://www.palminaleone.com', 'http://comicgirlcoffee.com', 'http://www.andreawhitten.com/', 'https://www.whitakertire.com/location/mode/1/3701-greensprings-ave-west-mifflin-pa-15122.aspx', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.taylorrentalavon.com', 'http://www.starbucks.com', None, None, 'http://www.urbanaparks.org/facilities/16.html', 'http://www.superdry.com/', '1592141552', None, 'http://hazelgeneralstore.com/', 'https://www.supercuts.com/locations/az/glendale/bell-park-shopping-center-haircuts-80215.html', 'http://www.creativetraveler.com', None, None, None, 'http://www.circabella.com', '1592141628', None, 'http://www.usps.com', 'http://www.aaronbrothers.com', 'http://www.roastmadison.com/', 'https://www.americancampus.com/student-apartments/nc/charlotte/university-crossings-charlotte', 'http://calgary.cnty.com/', 'http://locations.michaels.com/AB/CALGARY/3902/', 'http://www.home-computerhelp.com', 'http://www.thebeverleyhotel.ca', None, 'http://sachisalon.com', None, 'http://www.durafastlabel.ca', 'http://www.vegas-salon.com', None, 'http://firstchurch.church', None, 'http://www.sumitsyoga.com', 'http://www.popeyeschicken.ca', None, 'https://kneaders.com', 'https://www.turkishkebabhouse.net', None, 'http://www.subarusaintejulie.com/en/', 'http://www.eyelidsphoenix.com', 'http://www.victoriassecret.com', None, 'http://totalwellnessacupunctureaz.com/', 'http://www.maxandermas.com/locations/333038', None, 'https://www.berkshirecommunities.com/apartments/nc/charlotte/berkshire-dilworth/?utm_source=Yelp&utm_medium=Referral', 'https://www.walmart.com/store/2050/henderson-nv/whats-new', None, 'http://www.discounttirebellevue.com/', 'http://www.pizzanova.com', 'http://www.7-eleven.com/?yelp=29384', None, None, 'http://www.marriott.com/hotels/travel/yyzmk-courtyard-toronto-markham/', 'https://locations.levi.com/nv/lasvegas/denim_las-vegas_nv_188.html', 'https://www.pizzahut.ca/store-details/store-R32814/pizza-hut-Mississauga', 'http://www.onpointhairandnail.com', None, None, 'http://www.climatechnc.com', None, 'http://www.newgalleryofmodernart.com', None, 'http://www.assemblychefshall.com', 'http://www.wisconsincheesemart.com', 'http://www.gamestop.com', None, 'https://flights.hisgo.com/us/', 'http://www.starbucks.com/store/14887/', None, 'http://www.belgo.ca/', None, '1592142134', None, 'https://www.lasvegasbookkeeping.com', 'http://www.dollarama.com/', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', 'https://www.instantplumbing.ca', 'http://www.tintaz.com/', 'http://www.aneyefordetail.com', 'http://vanlanendds.com', 'http://www.loonspub.com/', None, '1592142216', None, '1592142226', None, 'https://www.bigfrog.com/stores/gilbert-az/', 'http://csrcoffee.com', 'http://www.expert-locksmith.ca', 'http://www.manhattanfishgrill.com', 'https://henleysfloor.com', None, 'http://www.skallywagcigars.com', 'http://www.speedandstrengthuniversity.com', 'http://www.lasvegasrestorationllc.com', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, 'http://steelecreekpediatricdentistry.com', 'http://www.mesafarmersmarketandflea.com', None, 'http://theforumatdesertharbor.com', None, 'http://addisonparkapts.com?utm_source=Local%20Listings&utm_medium=referral&utm_campaign=Top%20Line%20Local', 'http://www.greatclips.com', None, 'http://www.citi.com', None, None, None, None, None, 'http://www.7baker.ca', 'https://team-kaizen.com', None, 'https://donroyaldrycleaner.wixsite.com/website', None, 'http://www.goldstartailoring.com', None, 'http://www.planetfitness.com/gyms/charlotte-whitehall-nc-125?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.gerrysmarine.com', None, None, 'https://www.suretempair.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/oh/elyria-3931.html?mcid=iyp:8368400', 'http://www.krispykremeaz.com', 'http://www.breadandcircus.ca', None, 'https://www.millerstavernlv.com', 'http://www.marcs.com/stores/marc-s-lakewood-grocery-store-44107.aspx', 'http://www.ambrosia.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.awardscheesecakes.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNCY', 'http://www.daleskarrparts.com/', 'http://www.bp.com', None, None, 'http://magictailor.ca', 'http://www.distillerypub.com/', 'http://www.deckscapes.com', 'http://www.chromasalonandboutique.com', 'https://www.bfly.ca/experience-bfly/', 'http://www.thrill-bikes.com', '1592142722', None, 'http://www.gotransit.com/publicroot/en/travelling/stations.aspx', 'http://ModManBarber.com', 'https://chocolateshops.sees.com/az/phoenix/chocolate_shops_phoenix_az_102103.html', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://sunboxphotography.biz', None, 'https://locations.monro.com/oh/painesville/1575-mentor-avenue?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', '1592142811', 'http://www.truemedispa.com/', 'http://www.realfruitbubbletea.com', 'https://az.aaa.com/agent-office-locations/az/phoenix/aaa-phoenix-office', None, 'http://jeroldoptical.com/', 'http://www.redrocklasvegas.com/dining/hachi.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=RR&utm_content=RR-PROPERTIES&utm_campaign=RR-YELP', 'http://jenniferwallens.com', None, 'https://clevelandhungarianmuseum.org', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.lakenormanchrysler.com', 'http://latitude360.com/pittsburgh-pa/', 'http://www.brewercommercialservices.com/', 'http://www.winerack.com', None, 'https://www.radisson.com/madison-hotel-wi-53719/madison', 'http://cityofhenderson.com/henderson-happenings/facilities/recreation-centers/whitney-ranch-recreation-center', None, 'http://Terminalbarbershop.com', None, 'http://www.americanapparel.net', None, 'http://www.mariospgh.com/southside/', 'http://www.shamrockroofingservices.com', 'https://www.amctheatres.com/movie-theatres/phoenix/amc-ahwatukee-24?utm_uid=yelp', None, '1592146361', 'https://www.steinmart.com/?utm_source=yelp.com&utm_medium=social&utm_campaign=yelp_site_store97', 'http://meineke.com', 'http://www.partydesignlv.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.margaritavilleglendale.com/', 'https://www.chimneystax.ca', 'http://www.palmettofamilyhearing.com', 'https://airpromaster.com', 'http://www.fresh-burger.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281380115&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://tempecomputerrepairservice.net', None, 'https://www.lvaspensalon.com', 'http://www.pacetax.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/13777+Pearl+Road-Strongsville-OH-44136/storeid=4300?cid=ps_ylp_2020', None, None, 'http://www.pandalawfirm.com', None, None, 'http://www.dominos.com', 'http://www.azuldayspa.org', None, 'http://www.koa.com/campgrounds/madison/', 'https://tinylovesphotography.com', 'https://www.imyourmechaniclv.com', 'http://www.moes.com', 'http://eng.bigbustours.com/lasvegas/home.html', 'http://www.lindleylawoffice.com', 'http://www.actiontours.com', None, 'http://www.thaizone.ca/', 'https://www.islandthymepub.com', None, None, 'https://www.efirstbank.com/customer-service/find-location.htm?loc=BELL', None, 'http://sagoautoglass.com', 'http://www.truevalue.com', 'http://www.clemcopostal.com', 'http://www.barrymorefurniture.com', 'http://www.plazahotelcasino.com/entertainment/superstars_of_magic', 'http://www.peakchiropracticpgh.com/', 'https://locations.sonicdrivein.com/nv/lasvegas/6301-w.-lake-mead-blvd..html', None, None, 'http://www.scooopskidspa.com/', 'http://www.ladybugsandwiches.com', None, 'http://www.separkmansion.com', 'https://www.ironwoodcrc.com/locations/?locations_doc=scottsdale', 'http://www.goldenlobsters.com', 'http://www.FourSonsLandscaping.net', 'https://www.enterprise.ca/en/car-rental/locations/canada/on/north-toronto-eglington-yonge-c245.html?mcid=iyp:8368400', 'https://www.softsurroundings.com/stores/the-district-at-green-valley-ranch/', 'http://wellnesslasvegas.com', 'http://www.southernshows.com/wch', 'http://omfastfoodrestaurant.com/home.html', 'http://fuerstauto.com', 'https://www.papamurphys.com', None, None, 'http://eeleconomicrestaurantequipment.com', 'http://www.haircoloringinphoenixaz.com', None, 'http://www.oakesappliance.com', 'http://whasn.com/providers/julie-goodell', 'http://www.huntersvilleford.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.carminespizzakitchen.com', 'http://www.homedepot.ca', 'https://www.hendersonmentalwellness.com', None, 'http://onetoone.case.edu/', 'http://www.taqueriapastorcito.com', 'https://www.bakerbros.com/visitus', None, 'http://www.fallsheating.com/', None, 'http://www.starbucks.com', None, None, 'http://houlihans.com/', 'http://www.yogurtini.com', 'http://www.donvalleynorthlexus.com', 'https://www.smartstyle.com/en-us/locations/az/phoenix/located-inside-walmart-2113-haircuts-11301.html', None, 'http://www.bk.com', None, 'http://pizzabytheslice.ca/', 'http://www.shapetrain.com', 'http://coronadelsolhighschool.net/', None, None, 'http://www.wyattworks.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1015+East+Ray+Road-Chandler-AZ-85225/storeid=7849?cid=ps_ylp_2020', 'http://massageaddict.ca/oakville', 'http://Inspiradabarberandbeauty.com', 'http://www.nuvohotel.com', 'https://www.hilton.com/en/hotels/lasruru-tru-las-vegas-airport/?SEO_id=YELP-RU-LASRURU', 'http://www.starbucks.com/store/9059/', 'https://azoralsurgery.com', 'http://www.bethanyanimalhospital.com', None, 'http://www.goodeggdining.ca', 'http://www.nevadadivorce.net', None, 'http://www.classykitchenandbath.com', '1592147262', None, 'http://www.service1stenergysolutions.com/', 'http://www.vesperrx.com', '1592147301', 'http://www.chestnuthillsdental.com/locations-providers/pittsburgh-shadyside/', 'http://www.macssushi.com', 'http://www.nanashake.com', 'http://www.eyepriority.com', 'http://www.northcoastbodymod.com', None, '1592147361', 'http://www.lvcc.com', 'http://storelocator.pepboys.com/pepboys/stores/phoenix/cactus/00747', 'http://www.reginospizza.com', 'http://www.travelbughealth.com', None, None, 'http://www.mysisterscloset.com/', 'http://www.carolsconfectionspgh.com', 'http://www.altaderm.com', 'https://www.ymcacalgary.org/program-descriptions/locations/brookfield-residential-ymca-at-seton/', 'http://www.aaaonestopappliance.com', 'https://www.tonysautoshop.com', 'https://911drivingschool.com/ohio/driving-school-rocky-river-ohio/', 'http://www.gaudinporschelv.com', None, 'http://www.tuesdaymorning.com', 'http://www.tlcflooringlv.com', 'http://www.buddystubbshd.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.potterlawaz.com', 'https://www.maaco.com/locations/nc/charlotte-11619/', 'http://www.bartendingschoolc2c.com', 'http://www.millenniumsalonmadison.com', 'http://www.yak.com', 'http://rackempubandbilliards.com', 'https://restoreandrenewmassage.com', 'http://creativeillusionshairstudio.com', 'https://www.TheUltimatePartyStore.net', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.completevisioncare.com/home.html', None, '1592147593', 'https://www.northwest.com', 'http://www.arizonalullabyguild.com', 'http://www.meltwich.ca/richmondstreet/', 'http://www.aroundtheblock.com', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700006435131063&ds_s_kwgid=58700000444951711&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', '1592147657', None, None, 'http://www.littlecaesars.com/', None, 'http://cherryboomboom.com', None, None, 'http://www.azdivalashes.com', None, 'http://www.drteddysim.com/about-dr-teddy-sim/', 'http://access-handyman.webnode.com', None, 'http://www.roomforhealth.com/', 'https://www.autozone.com/locations/nv/las-vegas/1351-n-eastern-ave.html', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', None, 'https://www.salt-tempe.com', 'https://nature.discoveryplace.org', 'http://www.senjivastudio.com', None, None, None, 'https://www.shijosushi.ca', None, None, 'http://www.qinthaicuisine.com', None, 'http://www.invisiblehandyman.com', None, None, 'http://headtripbrewery.com', None, None, None, 'http://www.444park.com', 'http://www.pueblomedicalimaging.com', 'http://www.the-bag-lady.biz', None, None, None, None, 'http://www.bbqisland.com', 'http://www.ohiocitynails.com', 'https://www.realstar.ca/apartments/on/brampton/188-cityview-apartments/index.aspx', None, 'http://walmart-eyedocs-lasvegas.com/Marks-Sunset', 'https://studios.waxingthecity.com/ll/US/PA/Pittsburgh/3605-Butler-Street?utm_source=yelp&utm_medium=local&utm_campaign=localmaps', None, None, 'http://omnidermatology.com', 'http://locations.in-n-out.com/234', 'http://www.eastvalleyfiduciaryservices.com', '1592148033', 'https://azflorist.com', 'https://www.meineke.com/locations/pa/pleasant-hills-2517/?utm_source=yelp&utm_medium=xvc', 'http://www.ezpoolslv.com', None, 'http://alluregardens.com/', 'http://abovethecrustpizza.com', 'http://www.alldogsgotobev.com/', '1592148088', 'http://itdotrepairs.com', None, None, 'http://www.uwhealthkids.org/locations/kidsdetail.jsp?locationId=59', None, 'http://www.visiblevoicebooks.com', 'http://www.juvenaspa.com', 'http://www.aeortho.com', 'http://www.rosestodaylasvegas.com/?srccode=yelp_track', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.charakcenter.org', None, 'http://gear-oh.com', 'http://www.wfamilydental.com', 'http://salonstylush.com/', 'http://www.westaddisonaz.com', '1592148259', 'https://www.barra-sandusky.com', None, 'http://www.denturerepairinglendaleaz.com', None, None, 'http://www.schneidersauto.com', None, 'https://datarecovery.com/about/toronto/', 'https://stores.jcpenney.com/az/phoenix/clothing-stores-phoenix-az-2889.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2889', 'http://www.picknsave.com', None, None, None, 'https://www.eliminexpest.com', 'http://www.starbucks.com/store/9298/', 'http://www.lightrx.com/mesa', 'https://local.biglots.com/pa/monroeville/1960?zcp=pd_local_yelp', None, 'http://rizzosspaghettihouse.com', 'https://www.statefarm.com/agent/US/NV/Henderson/Carl-Endorf-HYSY71YS000?cmpid=ST0N_BLM_0006', '1592148451', 'http://www.lveyedocs.com', 'https://www.usps.com/', 'http://www.fatbastardburrito.ca/locations/burrito-whitby', None, None, 'https://www.scottsdale-duilawyer.com/?utm_source=scottsdale_location&utm_medium=yelp&utm_campaign=website_link_click', 'http://arizonapianomover.com', None, 'http://www.ctrglass.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', None, None, None, 'http://www.rollcare.ca', 'https://locations.panerabread.com/wi/fitchburg/2960-cahill-main.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', None, 'http://www.speedlube.net', 'http://local.skechers.com/ca/glendale/38/', 'http://www.lowes.com', 'https://www.pinnaclenissan.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'https://www.ulta.com/stores/westlake-oh-1423', 'http://www.azphi.com', None, 'http://www.lacasadelahabana.com', None, None, None, 'https://www.ediblearrangements.com/fruit-arrangements?utm_source=Yelp&utm_medium=Listing&utm_campaign=StoreID206', None, '1592148696', None, None, 'http://www.starbucks.com', '1592148737', 'http://www.getenzo.com', 'http://howlerspittsburgh.com', None, None, None, 'http://l.macys.com/fashion-show-in-las-vegas-nv?cm_mmc=Yelp_Stores-_-LasVegas-_-n-_-532&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Las+Vegas_532&m_ac=yelp_stores&m_ag=n', None, None, None, 'https://local.safeway.com/safeway/az/scottsdale/23565-n-scottsdale-rd.html', 'http://local.firestonecompleteautocare.com/arizona/cave-creek/4915-e-carefree-hwy/?lw_cmp=IYP_YPC_MLP_784291', 'http://www.doordoctor.ca', 'https://www.maccosmetics.com', None, 'http://www.popularpizza.ca/locations.phtml', None, None, None, None, 'http://www.craftcellars.ca', 'http://www.valleylights.com', 'http://www.quickspark.com', 'http://www.atlwings.com', None, 'http://www.OnTheSpotLV.com', None, '1592148965', 'http://www.hillsong.com/phoenix', None, None, 'http://www.divinethreading.com', 'http://www.mayfieldohdentist.com/', 'http://www.scottsdaleaz.gov/sports/aquatics/mcdowell-mountain-ranch-pool', None, 'http://www.ryah.ca/', None, 'http://www.bubblesandbowslv.com', 'http://www.houdini.com', None, None, 'http://mmgyn.com', None, 'https://www.openriceclt.com', 'http://www.mccoolspools.com', 'http://musiccentrecanada.com/', None, 'http://www.gioninos.com', 'http://www.ericpalacioslaw.com', None, 'http://www.embassygrand.com', 'http://www.firstprimerealtygroup.propertyware.com', 'http://www.sierraironlv.com', None, None, '1592149192', None, None, 'https://www.bakerbros.com/glendale', 'https://prairieflowersandgifts.com', 'https://www.sunsetgrill.ca', 'https://americanbuilderslv.com', 'https://www.petsfriend.ca', 'https://cobsdoggersandmore.wixsite.com/website', 'http://jacksaloon.com', 'http://www.thesweetpotato.ca', None, None, 'http://www.breatherefuge.com', None, 'http://www.fantasticsams.com', 'http://www.olsonortho.com', 'http://www.bulldogliquidators.com', 'http://www.ladyyorkfoods.com', 'http://storelocator.pepboys.com/pepboys/stores/las_vegas/east_charleston/00685', 'http://www.acaringnanny.com', 'http://www.markhamtheatre.ca', None, None, 'http://www.mermaidsplashnails.com', 'http://www.stephanoslv.com', None, 'https://www.peppywalnutssweetsboutique.com', None, 'http://www.courtesychrysler.com', 'http://creamistryAZ.com', 'https://www.skininjectables.com', 'http://www.lcbo.com', 'https://www.sprint.com/locations/nv/henderson/334-w-lake-mead-pkwy', None, 'http://nourishmassageandskincare.com', 'http://www.timhortons.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044280206066&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://sidelinestreeservice.com', None, 'http://www.evooristorante.com', 'http://www.budacafe.ca', 'http://leesliquorlv.com/locations/sunset-office/', None, 'https://www.jackentertainment.com/thistledown', 'http://www.orangetreepools.com', 'http://www.panorea.ca', 'http://www.cancleanpros.com', 'http://www.phxestatesales.com/', 'http://www.bareoaks.ca', 'http://www.marriott.com/hotels/travel/phxss-springhill-suites-phoenix-north/', 'http://steviespizza.com', 'http://www.7-eleven.com/?yelp=32265', 'http://tricitytowingaz.com/', 'http://www.vanillabeanonline.com', 'http://www.haggen.com', 'https://www.thebrick.com', 'http://www.appleplumbingaz.com/', 'https://www.americaroofingco.com', 'http://www.broadacresm.com', 'http://www.MobileDogWash.dog', None, None, 'http://www.cozymels.com', 'https://cacao70.com/fr/nos-emplacements/atwater', 'http://www.randstadusa.com/', None, 'http://www.soulas.ca', 'http://www.masalahousepgh.com', 'http://www.creativeformalwear.com/', None, 'http://annandalegolfclub.com/', 'http://drmikepace.com', None, None, None, 'http://www.rona.ca/', 'https://www.riosbraziliansteakhouse.com', 'http://johnnyrockets.com', None, None, 'https://www.goldencorral.com/locations/970/golden-corral-william-penn-highway', 'http://www.glendaleaz.com/parksandrecreation/ThunderbirdPaseoPark.cfm', 'https://www.daveandbusters.com/locations/pineville', '1592150792', 'http://whatisbrewing.com', None, None, 'https://www.klinkecleaners.com/locations/dane-county-locations/monroe-street', 'http://www.ymcacharlotte.org', 'http://vegasvisionoptometry.com', 'http://www.photoandgo.com', None, 'https://the-post-sports-bar.business.site', None, None, 'https://www.chevronwithtechron.com', 'https://www.audioexpress.com/locations/15115-n-hayden-road-scottsdale-az-85260/', 'http://www.kidsgardenlearningacademy.com', 'http://www.iamyoga.ca', 'http://vmacauto.com', None, None, None, 'https://www.mansion54.com', 'http://www.gogopokeus.com', 'http://www.allen-equipment.com/', 'http://tandoorigrillcalgary.com', 'http://www.indulgeculinaryacademy.com', 'http://pillartopost.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=13080', None, 'http://siestalandscaping.com/index.html', 'http://storelocations.ae.com/us/pa/pittsburgh/540-south-27th-street.html', 'http://www.rockstartattoolasvegas.com', None, None, 'http://logansroadhouse.com', '1592150921', None, 'http://www.villageatcobblestonecourt.com', 'http://www.burritogordito.ca', 'http://www.tigbbqlv.com', 'http://www.fishfurniture.com', 'http://www.windowandsolarservices.com', 'http://www.pazzopittsburgh.com/', None, 'http://www.richardstreeservicellc.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://sabalimo.com', None, 'http://www.parkway-sc.com', '1592150982', 'https://www.schlotzskys.com/wisconsin/madison/1532', 'https://stores.anntaylor.com/az/scottsdale/7014-east-camelback-road.html', None, None, 'https://www.wingstop.com/location/wingstop-1394-gastonia-nc-28054/menu', None, None, 'http://www.calgarytransit.com', 'http://www.zeebasleep.com/', 'http://camdenlv.com', 'https://chicagoschickenandwafflescle.com', 'http://www.performancemarineaz.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.starbucks.com/store-locator/store/1016078', 'http://www.mullinspub.com/locations/kington-road/hours-location/', 'http://www.christensenoralsurgery.com', 'http://www.med-place.org', 'http://www.goodearthcafes.com', 'http://www.jkhlawoffice.com', None, None, 'https://prolinewindowtint.com', None, None, 'https://dainties-macaron.business.site', 'http://www.abovebeyondcleaner.com', 'https://www.petsmart.com/stores/us/oh/macedonia-store1851.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', None, None, 'http://www.heartcenterofnv.com', 'http://www.zaksbicycleshop.com', '1592151104', '1592151108', 'http://nevadaorthopedic.com/directory/people/show/thomas-vater', 'http://www.pastelesmary.com/', None, 'http://www.sciullos910.com/', 'http://www.simplyflowersnthings.com?srccode=yelp_track', 'https://www.sonoranspine.com', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', None, 'https://www.wellsfargo.com', 'http://www.weymouthfarms.com', 'http://shawarmaxpress.ca', 'http://thebarrelpub.ca', 'http://www.eurekagifts.com', None, 'https://chatters.ca/stores/chatters-square-one/', None, 'http://www.lviglobal.com', 'http://hamptoncrestenc.com', 'https://www.saucepizzaandwine.com/locations/tempe-tempe-union/', None, None, 'http://tandooristyle.ca', 'http://www.associatedretinaconsultants.com', 'http://www.azeuros.com', None, None, None, 'https://www.supercuts.com/locations/az/peoria/north-valley-power-center-haircuts-82958.html', None, None, 'http://www.chassepermanentmakeup.com', 'https://www.expressoil.com/stores/gilbert/az/3502/', None, 'http://www.bestmatt.com/', None, 'http://www.nonnasplace.ca', 'http://www.commonwealthah.com', 'http://www.cigna.com/cmgaz/locations/phoenix-central-multi-specialty-center', 'http://www.safeway.com', 'http://www.roxxsouthend.com', 'http://www.torontohearingservices.com/location-contact/north-york-ontario-ca/', 'http://www.cookiedoughjo.com', 'http://www.nohiddenfeestickets.com', None, 'https://www.simon.com/mall/southpark', 'http://kintarooh.com', None, 'http://nahasushi.com/', None, 'http://www.princessauto.com', 'http://2ndandcharles.com/', 'http://www.NorterraAH.com', 'http://www.torontomenssuits.ca', 'http://www.metalmorphosis.tv', 'https://homewoodfamilyaz.org', None, 'http://www.bestpittsburghhomes.com', 'http://www.awhlawyers.com', None, None, None, None, 'http://www.rightbynaturemarket.com', 'http://www.centrooptical.com/', 'http://www.grodzinskibakery.com/', 'https://www.spoonerpt.com/locations/goodyear/', None, 'http://www.detailingknights.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.secondcup.com', None, 'http://tinyurl.com/hp44240', 'http://tokyogrill.business.site', None, 'http://www.scoopspittsburgh.com/', None, 'http://www.candlelightersnv.org/', 'http://www.grapevinewinebar.com/', 'https://www.torrid.com/store-details?StoreID=5371', 'https://freedsbakery.com/pages/freeds-dessert-shop', '1592151332', None, None, 'http://www.pandysgardencenter.com', 'http://petgrooming-charlotte.com', 'http://southlandlanes.com', None, None, None, None, 'http://www.truenorthbarbers.com', None, None, '1592151370', '1592151372', 'http://www.sunsolarsolutions.com', None, 'http://www.avenuediner.com', 'https://www.hubbardswim.com', 'http://www.canamrealestateservices.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.dreamdealers.com', None, 'http://www.rapidcollision.com', None, 'http://www.homesteadmaid.com/', 'http://www.jackinthebox.com/', 'http://www.doylechiropractic.com', 'https://dutchbros.com/?utm_source=phoenix&utm_medium=yelp&utm_campaign=website_phoenix', 'http://www.spookyfast.com/', 'http://popeyeschicken.ca/', None, 'http://huntfamilychiro.com', 'http://www.stafflink.ca/', '1592151421', None, 'https://dc-glass-glass-repair-service.business.site', 'http://www.ohiocitypizzeria.org', 'http://www.northyorksheridanmall.com', None, 'http://www.concretemadenew.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.samscyclesupply.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://graphicideals.com/', None, 'https://www.excellenceairllc.com', 'http://www.larryjsauto.com', None, 'http://www.gioninos.com', None, 'https://www.pridestaff.com/tempe', 'https://www.cinemark.com/nevada/century-orleans-18-and-xd', None, 'https://thepetclub.net', None, 'http://www.thedartshop.com/', 'https://www.greentreesportsplexpittsburgh.com', 'http://safe.frontpointsecurity.com/yelpsecurity-us', 'http://www.themagicoflearning.com/', 'http://www.westvalleyent.com/', 'http://www.lvccld.org', 'https://pizzeriamoretti.com/moretti-toronto/', 'http://www.crossfitmte.com', 'http://mcmurraych.com', 'http://murderinkproductions.com', 'http://baskonseaboard.com/', 'http://www.gracefulcakecreations.com/', 'http://www.clasensbakery.com/', 'https://jessicanailinc.wixsite.com/feelingnailsandspa', None, 'http://www.mrlube.com/en/mr-lube-near-you/Locate-a-Mr-Lube/richmond-hill-1070-major-mackenzie-drive-east.aspx?storeid=190&postalcode=L4S%201P3', 'https://www.tacobell.com/food?utm_source=yelp&utm_medium=social&utm_campaign=yelp-pilot-test-menu-static-1-CS&utm_content=paid', 'https://www.charlotteautodetailingservices.com', None, 'http://www.helzberg.com', 'http://www.haircutsarefun.com', 'http://lasvegasyardwork.com', 'http://www.innevation.com', 'https://www.cox.com/local/residential/stores/nv/henderson/545-marks-street?sc_id=dp_z_yext_z_store_19Q2?sc_id=dp_z_yelp_z_store_19Q102', 'http://www.nesbitslanes.com', None, 'http://www.eggsmart.ca', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.artistictlc.com', 'http://www.wendys.com', 'https://www.haggardchiro.com/chiropractor-phoenix/', None, 'http://www.starallison.com', 'http://www.pupusaslafamiliar.com', 'http://usonv.com', 'https://www.devadave.com', 'http://www.evdi.com/?location=chandler', 'http://www.raulsoriginal.com/', 'http://www.omasplaceaz.com', 'http://www.legacynails.net', '1592151592', 'http://www.rvvh.ca/', 'https://chargrilldavidson.com', 'http://tijuanatanaz.com/', 'http://scifest.vegas/contact/', 'http://www.davinciflowershop.com', '1592151608', 'http://www.eventsbyfirstrose.com', 'http://outremont.lapiazzetta.ca', 'http://www.eurowise.com', 'http://www.rocrace.com/charlotte', '1592151621', 'https://www.wellsfargo.com', None, None, 'http://www.powerhousegym.com/schedules-i-407.html', 'http://www.dogsbycindy.com', None, 'http://www.kngmg.com', '1592151643', 'http://www.odelaybagel.com/', 'http://www.harbsbarbershop.com', None, 'http://stevedavishtgac.com', 'https://www.lajoliefemme.ca', 'http://www.alphagraphics.com/centers/westlake-ohio-us649/index.html', 'http://www.hellofacepaint.com', None, None, None, 'https://www.southwestmobility.com', None, None, None, 'http://www.lbvgallery.com', None, 'http://www.pageplusaz.com', 'http://gimelconstruction.homestead.com', 'http://www.gelatostar.com', 'https://www.lawncontrolcenter.com', 'http://www.anotherstory.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.rnrcarpetcleaning.com', 'http://www.heistbrewery.com', 'http://www.sharkystaphouse.ca/', 'https://negretelawfirm.com', None, 'http://samsxpresscarwash.com/huntersville', None, 'http://www.righttime-inc.com', None, 'https://locations.monro.com/oh/cuyahoga-falls/1584-state-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'http://www.yclibrary.org/lakewyliebranchdetails.php', 'http://www.publix.com', 'https://locations.monro.com/oh/north-olmsted/4565-great-northern-blvd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, None, None, 'http://tonysicecream.com', 'http://bigapplebagels.com/', 'http://www.clwplumbing.com/', 'http://fcautobody.com', None, 'http://restaurants.basspro.com/FishCompany/Mesa/Default.aspx', None, 'http://www.bk.com', 'http://www.dom.pitt.edu/id/About.html', 'http://vervewellnesscenter.com/', None, 'http://www.oliveoilaz.com', None, 'http://www.nevadadentalgroup.com/', None, 'http://www.castcadearizona.com', 'http://www.JimmyJohns.com', 'http://www.generationsmedcenter.com', None, 'https://www.circlek.com/store-locator/US/phoenix/4135-w-anthem-way/2709192?location=85086', None, 'http://lotusservicesaz.com', 'http://www.mysteryadventureslasvegas.com', 'http://www.swirlyz.ca', None, '1592151813', 'http://www.kfc.ca/store/sunrige-mall-101-2525-36-st-ne-calgary', 'http://www.torontocarwrap.com/', None, 'http://www.countyofdane.com/lwrd/parks/viking.aspx', 'http://www.golfpinehills.net', 'http://www.ddsdiscounts.com', 'http://www.gailambrosius.com/', 'http://wagntails702.wix.com/vegaspetgrooming', 'http://www.rawfitnesspersonaltraining.com', None, None, 'http://www.vedatakeout.com', None, 'http://www.cocinaelvis.com', 'http://www.filosophy.ca', None, 'https://www.interstatebatteries.com/locations/all-battery-center-las-vegas-nv', 'https://www.frysfood.com/storeHours?store=66000064', None, 'http://www.kitchenova.com', 'http://www.cosmiix.com', 'http://www.7-eleven.com/?yelp=29644', 'http://www.jasonsdeli.com/restaurants/az/glendale/glendale-deli', 'http://www.clevelandtallships.com', 'http://www.eathopscotch.com', 'http://www.RentMontereyApartments.com', 'https://stores.hallmark.com/nc/matthews/jansens-hallmark-shop-matthews-nc-23514.html', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.orthocarolina.com/locations/gastonia', 'http://www.phoenixsymphony.org', 'http://balletaz.org', 'http://www.PazanGallery.com', 'http://www.teaspaceus.com', 'https://www.nsm-seating.com/national-seating-mobility-relocates-phoenix-office/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.gotyoucoveredmusic.com', 'http://handelsicecream.com', 'http://www.babessportsbar.com/', 'http://www.thegreenmango.com', 'http://teamzrestaurant.com', 'https://www.chick-fil-a.com/stapleydrive', 'http://car-detailing-cleveland.com', '1592151935', None, 'http://www.a-zequipment.com', 'http://www.adventurecjd.com', 'https://ad.doubleclick.net/ddm/clk/434423236;236814625;d', 'http://www.hotrocksdiner.com/', 'http://www.studiowe.ca', 'http://www.floorcoveringfactoryoutlet.com/', 'http://www.nutrition51.com', 'https://www.hungryhowies.com/store/hungry-howies-01809?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1809&utm_content=Website_Link', 'http://www.zbakeryandcafe.com', None, 'http://www.payless.com', 'http://www.onefinedayphotography.com', 'http://www.sherwin-williams.com/store-locator/paint-store/Phoenix/AZ/8160', 'http://www.acehardware.com/store-details/07473', 'http://www.lacakery.ca/', 'https://agents.farmers.com/nv/las-vegas/jason-suemoto?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://www.scpchiropractic.com', 'http://www.housepaintinginphoenix.com', 'http://www.autoaccessoriespgh.com', None, 'http://www.baskinrobbins.com', '1592151997', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.tcby.com', 'http://www.creativestonecrafters.com', 'http://www.cocosbakery.com/', 'http://www.salonjovon.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.babyramacanada.com', None, 'https://guardlocksmithgaragedoor.com/locksmith-garage-door-repair-scottsdale-az', 'http://www.rei.com', 'https://www.burnsidebarbecue.com', 'http://www.pizzeriaviamercanti.ca/', 'https://www.sprouts.com/stores/details/-/store-details/view/store/19/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.wilsonniblett.com', 'http://www.zoltons.com', None, 'http://www.hirtsflowers.com', None, 'http://www.badboy.ca/en/pg-storelocator-Mississauga', 'http://www.blm.gov/nv/st/en/fo/lvfo/blm_programs/blm_special_areas/red_rock_nca.html', None, 'https://www.pteglv.com/locations/pts-pub/rainbow-cheyenne/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://www.crabcrackernc.com', None, 'http://magiceric.com/', 'http://www.mathnasium.com/scottsdale', None, None, None, 'http://www.textilemuseum.ca/', 'http://www.tpcmadison.com', 'http://www.freshstartaz.com', '1592152094', None, None, 'http://pizza.dominos.ca/Calgary-Alberta-10108', 'https://techfourlife.com', None, 'http://www.ttroadhouse.com', 'https://www.walgreens.com/locator/walgreens-5975+weddington+rd-matthews-nc-28104/id=10255', None, 'https://www.walgreens.com/locator/walgreens-5400+pearl+rd-parma-oh-44129/id=3314', 'http://www.ryanfootandankleclinic.com', 'http://www.hansabrewery.com', 'http://gabbys.ca/locations/gabbys-kingston-road/', 'http://westcitysnackbar.com', 'http://www.litchfield-park.org/index.aspx?NID=288', 'https://www.petsuppliesplus.com/Store/OH/Parma/Parma/120?utm_source=Yelp&utm_medium=Store-Page%C3%8A', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.centralbarkusa.com/madison-southeast', 'http://www.mattress.org', None, 'http://www.hobbylobby.com/StoreDetail/500', None, 'http://www.hairandcosaloninc.com', 'http://www.dennys.ca', 'http://bangzartandhairsalon.com/', None, None, 'http://www.joeslakewoodcomputer.com', None, 'https://milios.com', 'http://www.thescottresort.com/?utm_medium=enhanced-profile&utm_source=yelp', None, 'http://meskeremincharlotte.com/', 'http://sergio-daae.squarespace.com', '1592152184', 'http://www.courtesydental.com', None, 'http://www.soma.com/store/home.jsp?cmp=MOC_yelp', 'http://www.paradisevalleyhomes-forsale.com', 'https://thelasvegaslounge.com', 'http://www.eandgrealestate.com', 'http://www.laundrybasketvillager.com/', 'http://www.marybekitchen.com', 'http://www.gelatospot.com', 'http://professornailsandspa.com', None, 'http://www.buttermilkfinewaffles.com', 'http://www.tubeaudio.com', 'http://jeffreyngmd.com', 'http://www.lenailsonline.com', 'http://www.gnc.com', None, 'http://www.9round.com/fitness/madison-wi', 'http://www.alltattoos.net', None, 'https://www.officedepot.com/storelocator/az/glendale/officemax-6171/', '1592152244', None, 'http://www.downtownphoenixrentals.com', None, 'http://www.strawberryhedgehog.com', None, 'https://azwestpainting.com', 'http://www.abundance-a.com', 'http://silvertoncasino.com/dining', None, None, 'http://www.nellisfamilyhousing.com/', 'http://www.theloftycupcake.com', 'http://impchandler.com', None, 'http://www.lassendas.com/Casual-Family-Dining-Sports-Bar-Happy-Hour-Mesa-The-Grille-Las-Sendas.html', 'http://www.edgewaterbarandgrille.com/', 'https://www.guardianmortgageonline.com/branch-managers/arizona_mortgage_branch_manager_steve-howard/', None, None, 'http://www.marketdistrict.com/Locations/Detail.aspx?LocationID=7', None, None, None, None, None, None, 'http://ingenuitycleveland.com/', 'http://www.lcbo.com', None, 'http://www.starbucks.com', 'http://www.marcellos.ca/', 'http://www.clevelandgaragedoors.net', 'http://www.africanabbqcurriesdrinks.com', 'http://www.pvbarbers.com', 'https://locations.wendys.com/canada/on/mississauga/6966-financial-drive', 'http://www.PawsPetServicesInc.com', None, 'http://svdppitt.org/stores/store-locations-directions/', 'http://www.newliferoof.com', None, None, None, 'https://agents.farmers.com/az/carefree/jason-grifo?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://venturagrill.com/', 'http://www.libertyburgers.ca', 'http://www.mushkaki.com/', None, None, 'http://www.enchantress.com', 'http://www.belfilandscaping.com', '1592152383', None, None, None, None, 'http://www.rubymarie.com', None, 'https://locations.wendys.com/canada/on/mississauga/6966-financial-drive', 'http://www.dmvnv.com/', None, None, None, None, 'http://www.badattitude.com/', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', 'http://www.ogreatcut.com', 'http://www.analoguegallery.com/', 'http://www.arizonadigestivecenter.com', 'http://www.murrysvillespa.com', 'http://www.bobstvaz.com/', None, 'https://www.rcpw.com', 'https://www.arizonatermitespecialists.com', 'http://www.familypetsalon.com', 'https://novasolutions.ca', 'https://www.kwiktrip.com/Locations/Details/673/', 'http://locations.harristeeter.com/nc/charlotte/205/', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_112', 'http://www.greenleafwholesale.com', None, 'https://nestnailwellness.com', 'http://www.lowes.com', 'http://www.soupersalad.com/', 'http://www.mammaspizza.com', 'http://www.sacredrootsaz.com', 'http://bk.com/', 'http://www.eldoradocasino.com', 'https://locations.wendys.com/united-states/wi/sun-prairie/743-w.-main-st.', 'https://salsalocamexicangrill.square.site', 'http://www.tph.ca/print/toronto-yonge-st-eglinton-ave', None, 'http://www.batteriesandgadgets.com/pages/contact_us', '1592152501', 'http://www.westthirddeli.com', 'http://www.charmeck.org/', None, 'https://growgeneration.com/?utm_source=yext', 'https://oldpinevillepremiumpub.com', None, None, None, 'http://www.swspineandsports.com', 'http://www.thepubat333.com', None, 'http://www.quiznos.com', 'http://www.autohausofnorthscottsdale.com/', 'http://www.deathinvenice.ca', 'https://sobamesa.com', 'https://wongs-chinesedining.com', None, 'http://www.megatunes.com', None, 'http://www.phoenixpaver.com', 'https://www.myplacehotels.com/my-place-hotel-north-las-vegas-nv', None, 'http://www.wehnergroup.com', 'http://liveatwynnpalms.com', None, 'http://www.artisanantiquescleveland.com', 'http://chaletvet.com', None, 'http://www.miasalonstudios.com', None, 'http://www.eatchinakitchen.com', 'http://www.williamsstainedglass.com', None, None, 'http://elcompatireshoplv.com', '1592152616', None, None, 'http://www.tbcmovers.com', 'http://www.residencesonfarmer.com/', 'https://jojosjerky.com', 'https://agents.allstate.com/mike-seifer-henderson-nv.html', 'http://www.domyresume.net', 'http://www.hongkongvintage.com/', 'http://www.newtowndds.com', 'http://www.chuparosapainting.com', 'http://ottavio.ca/contact-greenfield-park-rive-sud/', None, None, 'http://www.charmeck.org/', 'http://www.ae.com/web', 'https://www.loandepot.com/branches/Henderson-NV?sc_camp=DE925F8ECA0F4E0D8092C7E108A615A3', 'http://www.judonspainting.com', None, 'http://www.peoplesmortgage.com/loan-officer-team/broker/Tamiarmijo', None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.pinkhousenc.com/', 'http://www.kobootcamp.com', 'http://www.standa.ca', 'https://www.pepsolar.com', 'http://www.onenevada.org', 'http://623pc.com', None, None, 'http://www.expressionswaxspa.com/about-us.html', None, 'http://www.azburgershoppe.com', 'http://italianvillagepizza.com/', None, 'https://www.newportmotorslv.com', None, 'http://mandys.ca', None, 'http://www.joelcohenmd.com', '1592152774', None, 'https://www.walgreens.com/locator/walgreens-220+s+century+blvd-rantoul-il-61866/id=9965', 'http://topnotchnailsspa.com', None, 'http://www.passengerscafe.com/#hello', None, None, None, 'http://enginehouse.com', 'http://www.cartridgeworldphx.com', 'https://www.cellphonerepair.com/mayfield-heights-oh/', 'http://towingscottsdale.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.essentoronto.com/', 'http://avis.com', 'http://www.z-vw.com//?cs:pro=repman&cs:e=yelp', 'http://www.patriciacakes.com', 'http://www.acepoollv.com', None, None, 'http://www.firepatio.com', None, 'http://jerryrhynescollision.com/locations/pineville-location/', None, None, None, 'http://www.michaelkluthe.com', 'https://www.youfit.com/gym/phoenix-32nd-st-7406', 'http://www.woofgangbakery.com', 'http://www.azandersonanimalclinic.com', None, 'http://Phomono.ca', 'http://www.sunscreens-armadillo.com', None, 'http://www.donatos.com', None, 'http://www.noviamia.ca', 'http://www.paseodelpradoapts.com', None, 'http://www.redrockcanyonlv.org', 'http://Www.Tailsandbones.com', 'https://pawnscottsdale.com', None, None, 'https://www.waltsjewelers.com', None, 'http://www.oasishookahlv.com/hookahcatering.html', None, 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.ontrackdj.com', 'http://vintagelucys.com', None, 'http://www.regissalons.com/salondetail/default.asp?salonid=4818', 'http://www.DJEliteAZ.com', 'http://www.stadiumbargrill.com', 'http://kwcafeterias.biz', None, 'http://www.joycitytires.com', '1592152981', 'http://luckithaibistrolasvegas.justcafepages.site', 'http://www.thebakedbear.com', None, 'http://www.finewinereserve.com', 'http://scfapm.net/', 'http://www.shoppersdrugmart.ca', 'http://www.thrivehomeservice.com', 'http://www.atomiccomics.com', None, None, None, 'http://www.regenesisskinstudio.com/', 'https://www.searshomeservices.com/locations/appliance-repair/pa-pittsburgh/1000-robinson-center-dr', None, 'https://stores.jcpenney.com/pa/monroeville/salon-monroeville-pa-0199.html', None, 'http://www.cosmosrecords.ca', 'https://nothingbundtcakes.com/bakery/AZ/glendale?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=glendale_56', '1592153049', 'http://bentoburrito.ca/', 'http://www.bigming.ca', 'http://www.rolenbrothersfence.com', 'https://juliagrambo.com', 'http://www.automaxxon16th.com', 'http://www.flatlinegrinding.com', 'http://www.hopingfurahome.com', None, 'http://www.UrbanPranaYoga.com', 'http://www.aamfg.com', 'http://www.808smokes.com', '1592153093', 'http://restaurantlabelleprovince.com', None, None, None, 'http://www.winners.ca', 'https://www.mamabaers.com', None, None, 'http://www.risquelv.com/', None, None, None, 'https://www.KFYIadvertising.com', 'http://www.christieclinic.com', 'https://showerdoors.binswangerglass.com/nc/charlotte/?utm_source=yelp&utm_medium=social&utm_campaign=showerglass&utm_content=urllink', None, None, 'http://www.thaitouchkitchen.com/', 'http://www.bk.com', 'http://www.spinbikeshop.com', '1592153170', 'https://www.crawfordtransmission.com', 'http://www.simpleshuttle.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.dreamcarrentals.com/', '1592153188', 'http://www.coldstonecreamery.com', 'http://www.tonicpgh.com', 'http://www.hydroshieldphx.com', 'http://www.bk.com', 'https://wellingtonsofcalgary.com', 'https://store.vioc.com/nc/gastonia/2650-e-franklin-blvd-28052-BD0014', 'http://kimaznailspa.wix.com/kim-az-nails-and-spa', 'http://superstore.ca', 'http://www.kilkearysautobody.com', 'https://skinaddictstudio.com', None, None, None, 'http://autopricecanada.com', 'https://westvalleypawn.com', None, '1592153258', None, 'http://www.clevelandymca.org/branches/westpark', 'https://www.azcontainers.com', None, 'http://www.techspector.com', 'http://www.dunkindonuts.com', 'http://www.napoca.ca/', None, 'http://www.brownleejewelers.com', 'http://www.arbys.com', 'http://www.kearnsandco.co', None, None, 'http://www.tommulligan.com', 'http://www.vegaslimocoach.com', 'https://locations.sonicdrivein.com/nv/lasvegas/6301-w.-lake-mead-blvd..html', 'http://www.westhillvet.com/', 'http://www.mark-taylor.com/arizona/laguna/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.fourpeaks.com', None, None, 'https://www.carewithconfidence.com', 'https://elementsmassage.com/tempe', None, 'http://www.millscleaners.com', None, 'http://cluffmechanical.com', 'http://www.buenoburgeraz.com', 'https://www.cinemark.com/northern-ohio/cinemark-cuyahoga-falls-and-xd', None, 'https://www.pittsburghparks.org/schenley-park-cafe-visitor-center', 'http://www.rossocoffeeroasters.com', 'http://www.edgewalkcntower.ca/', 'http://www.dunkindonuts.com', None, '1592153386', 'http://www.shell.us/', '1592153393', 'http://www.tolmachoff-farms.com', 'https://crisp-pizza.com', 'http://www.comebuy2002.com.tw/', None, 'https://www.perfectpalmssalon.com', 'http://www.regionsbistro.com', 'http://advancedtherapeuticsmassage.com', 'http://www.davidofflv.com', 'http://www.kvegas.com', None, 'http://www.farnsworthrealty.com', 'http://blueagavemadison.com', None, None, 'https://howardair.com/showroom/', 'http://www.photographerjun.com', 'https://www.desertdryclean.com', 'http://workoutanytime.com/mint-hill', '1592153469', None, None, 'https://www.supercuts.com/locations/az/scottsdale/scottsdale-pavillions-haircuts-9672.html', 'http://www.merlenormanstudio.com/nv/las-vegas/mn-10293/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.divasalonspa.com/', None, 'http://www.alexandani.com/pittsburgh', 'http://www.antoniospizza.net', '1592153499', None, None, 'http://www.pittsburghsbestremodeling.com', 'https://cvam.com', None, None, None, 'http://www.magomindfulness.org', 'http://www.twolouiesmarket.com/', 'http://www.runningroom.com/hm/', 'http://www.gamestop.com', 'http://www.fabletics.com', 'http://captiverooms.com', None, 'http://ad.doubleclick.net/clk;266326037;92222803;c?http://www.24hourfitness.com/FindClubDetail.do?clubid=205&edit=null&semiPromoCode=null&cm_mmc=Yelp-_-ClubPage-_-BusinessListing-_-Link', None, 'http://www.7-eleven.com/?yelp=19726', '1592153554', None, 'https://www.kalonsandglidewell.com', 'https://roiproperties.com', 'https://www.truenorthmortgage.ca/?utm_source=Yelp&utm_medium=yelp-ads&utm_campaign=yelp-ont', None, 'http://www.scottsdaleautogroup.com', None, None, 'http://www.papamurphys.com', '1592153591', 'http://www.beneficial-insurance.com', None, 'http://katespade.com', 'http://www.purelifehomeopathy.com', 'https://www.nhrandolphobgyn.org/locations--hours/charlotte.aspx', None, None, 'http://www.olgaallenhairsalon.com', 'http://www.cabowabocantina.com/vegas/', 'https://apricotlaneboutique.com/store/champaign/', 'https://locations.noodles.com/wi/fitchburg/2981-triverton-pike-dr--at-mckee-rd.html', 'http://clickserve.dartsearch.net/link/click?lid=43700044282974602&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.tlchudson.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, 'http://www.gmgbarbershop.com', 'https://communitytirepros.com/tires-auto-repair-6-phoenix-az', 'http://www.calgary.ca/CSPS/Recreation/Pages/Athletic-parks/Glenmore-Athletic-Park.aspx', None, 'http://www.2dadsbouncehouses.com', 'http://www.INTLscottsdale.com/', 'http://www.xingfutang.ca', 'http://www.thelashandsugarcompany.com', None, 'https://stores.hallmark.com/nc/pineville/amys-hallmark-shop-pineville-nc-38803.html', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEAA', None, 'https://www.azmonsterair.com', 'http://knockout-fitness.com/locations/gilbert/', None, None, None, None, None, None, '1592153709', None, '1592153714', 'https://vercini.com/stores/vercini-miracle-mile-shops-las-vegas/', 'http://signaturegalleries.com/ACs_Wyland_Signature.html', None, None, None, 'http://marsiglianospizzeria.com', 'http://www.byashleymarie.com', 'http://www.dominos.com', None, None, None, 'https://www.americasbest.com/store-list/pineville-la-71360/', None, None, 'http://www.kbbicycleshop.com', 'http://mastertune.ca', None, None, 'http://www.stevesknows.com', 'http://www.shopplanetblue.com', 'http://www.89002realtor.com', 'http://www.peoplesutility.com', 'http://www.silverstatecu.com', 'http://www.crossfitmagna.com', 'https://www.olivegarden.com/locations/ab/calgary/calgary-northgate-plaza/4345?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:calgary-ab-4345_pl:locurl_rd:1743', 'http://dishosteria.com', 'http://gcbc.gcu.edu', None, 'http://www.bluewaterimaging.ca/', 'https://bikedepot.com/stores/toronto/', 'http://angelosgilbertbarbershop.com', 'http://www.teacoffee.com', 'https://locations.usbank.com/index/arizona/phoenix/west-bell-rd-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.nevadacoinmart.com', 'http://www.cupcakes-canada.ca', 'http://www.mastertoddy.com/', 'http://www.venetian.com/restaurants/italian/tintoretto-restaurant-bakery.html', None, 'http://www.videoproductionscottsdale.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PHXEL', 'http://www.wineandspiritfestival.ca', 'https://www.supercuts.com/locations/ca/oakland/rockridge-district-haircuts-8050.html', 'http://www.cvlt.org', 'http://www.9round.com/fitness/madison-wi', 'http://hobakkoreanbbq.com', 'https://beavertails.com/shops/', 'http://friendsofdormontpool.org/pool.html', 'http://www.applyapprovedauto.com', None, 'http://www.customdentures.us', 'http://www.partysavvy.com', 'http://www.oliverannie.com', 'http://abrazodiabetes.com', 'http://www.i9sports.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.dunkindonuts.com', 'http://www.crossfitcholla.com', 'http://HairMajesty.mysalononline.com', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', 'http://vespucciconsignment.com', 'http://www.dominos.com', 'http://www.hangoverbail.com', 'http://www.harmonybarandgrill.com', 'http://LiveAtArtessa.com', '1592153958', 'http://drkaylaeckart.com', None, 'https://www.va.gov', 'http://www.starbucks.com/store/9334/', 'http://thesweatlab.com', None, None, None, 'http://www.homesense.ca', None, 'http://www.dominos.com', None, 'http://www.zerorezpgh.com', None, None, 'http://whasn.com/locations/whasn-henderson/', '1592154010', 'https://haircutmenraintreeranchchandleraz.com', 'http://themonticello.com', 'http://www.seagullbook.com/', None, 'https://www.publicstorage.com/arizona/self-storage-mesa-az/85210-self-storage/787?pid=wyelp&CID=1245&CHID=AFL', 'http://protecrepair.biz/', 'https://www.vapeloft.com/vaughan-vape-shop', 'http://www.futons-4-less.com/', 'http://www.unitedcabcle.com', 'http://www.pinkladiesmaids.com/', None, None, 'http://www.heroburgers.com', 'http://www.coinlesslaundry.com', 'http://www.mortgageandcreditpro.com', None, None, 'http://www.caesarspalace.com', 'https://www.gohealthuc.com/north-carolina/charlotte/quail-hollow?utm_source=yelp&utm_medium=referral&utm_campaign=yelp%7Clistings%7C2020-nc-null-novant-011520-quail-hollow', None, '1592154093', None, 'http://www.akanails.com', None, 'http://doorfiftyfive.com/', 'http://opasouvlaki.ca?utm_source=yelp&utm_medium=website-click&utm_campaign=enhanced-profiles', 'http://northamericanspine.com/', 'http://www.nutrishopcharlotte.com/', None, 'http://www.plazaappliancemart.com', None, None, 'http://www.chickrotisseriebar.com', None, 'http://www.losolivosrestaurants.com', None, None, None, 'www.rustycage.ca', None, 'http://www.newcityphx.com/', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.pnponecarecleaning.net/', 'http://ronsserviceandtire.com', 'https://simonmed.com', 'http://www.streetsaw.com', '1592154185', '1592154189', 'https://bosshairsalon.com', '1592154198', 'http://www.speedydetail.ca', 'http://www.dentalspalv.com', None, 'http://www.taketimetorelax.com', None, 'http://www.lvcmc.com', 'http://www.fkaiser.com', None, 'http://www.vinorossorestaurant.com', None, 'http://www.dwellstudentmadison505.com', None, 'http://www.DornHardware.com', None, 'https://www.atmosphere.ca/stores/calgary.html', 'http://anvilbaraz.com', 'http://thetreeguylv.com', None, 'http://www.redrockcanyonlv.org', 'http://www.chachis.ca', 'http://www.decadenceculinary.com', None, 'https://www.sierracanyonaz.com', '1592154281', 'https://providenceeye.com', 'https://locations.sonicdrivein.com/il/savoy/101-calvin-street.html', 'https://cosmomusic.ca', '1592154349', 'http://www.banfield.com/veterinarians/az/phoenix/pho?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://www.desertspringsrealty.com', 'https://www.valuecityfurniture.com/store-locator/pa/pittsburgh/451-clairton-blvd', 'https://www.speedy.com/shops/newmarket/en/', 'https://stores.anntaylor.com/factory/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'http://splashwindowcleaning.com', None, 'http://incascuisine.com', 'http://kierlanddentalcenter.com/', 'http://www.cherrystbbq.com', 'http://www.hznvet.com/', 'https://www.enterprise.com/en/car-rental/locations/us/oh/chardon-3928.html?mcid=iyp:8368400', 'http://www.gansercompany.com', 'http://casaritasmexicangrill.com', None, 'https://www.theelementlv.com', 'http://www.olde8automotive.com', 'https://www.241pizza.com', 'http://kindmedsaz.com', None, None, 'http://www.blimpcitybikeandhike.com', None, None, 'http://www.championfenceinc.com', None, 'https://www.anytimefitness.com/gyms/3397/Montreal-QC-H2W-1Y7/?utm_source=yelp&utm_medium=local', '1592154453', 'http://www.enchantedvalleyacres.com/', 'http://furbabiesresort.com', None, 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=586&utm_content=bpas', 'http://www.visualcreations.vegas', 'http://www.phoenixazhomes4u.com/', '1592154476', '1592154480', 'http://www.phoenixmedicalgroup.com', 'http://www.azvpcp.com', 'https://www.petsmart.com/stores/us/nv/las-vegas-store1153.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'https://www.sellitquickhs.com', 'http://a1carpetmasters.com/', None, 'http://www.arcoflifechiro.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', None, 'http://www.sears.com/stores/north-carolina/monroe/0002008.html', 'http://www.EliteBartend.com', 'http://www.adriaticflooring.com', 'http://www.BombHairVegas.com', 'https://www.sonoraquest.com/find-a-location/sun-lakes-alma-school-riggs/', None, 'https://www.bluebreeze216.com', 'http://www.swgeneral.com', 'http://www.wetpetsandfriends.com', 'http://www.dirkcpa.com', 'http://www.alivedanceworks.com', 'http://www.coldstonecreamery.com/stores/20181?latitude=33.6071829&longitude=-111.9403254', 'http://www.silhouettebridallv.com', 'http://www.alibabas.ca/index.php?route=common%2Flocation_details&loc_id=12', 'http://www.jakesbarvegas.com/', None, 'http://www.sav-onplumbing.com?srccode=yelp_track', None, 'http://www.edelmanplumbing.com', 'http://www.starbucks.com/store/11823/', 'http://www.scentair.com', None, 'https://www.ihop.com/en/restaurants-south-las-vegas-nv/7490-s-las-vegas-blvd-3096', None, 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/mississauga-(2).aspx', 'http://www.aspirekidsports.com', 'http://www.bloodbrothersbrewing.com', 'http://www.sherwin-williams.com/store-locator/paint-store/Gilbert/AZ/8286', 'http://www.blackeagletoronto.com/', 'http://www.maderacabinets.com', 'http://www.citycleaner.com', 'http://todaysinteriors.com/', 'http://4seasonslandscapinglv.com', None, None, 'http://ashleynailsspachandler.com', None, 'https://www.walmart.com/store/1646/mesa-az/details', 'http://www.daphnesbakery.com', 'http://www.vynepowdercoating.com/', '1592154609', 'http://www.cityfresh.org', None, None, 'http://www.crystalsatcitycenter.com', None, None, 'http://www.accountingprose.com/accountingprose-arizona/', 'https://order.pizzapatron.com/menu/phoenix-indian-school-road?utm_source=Yelp&utm_medium=Order&utm_campaign=Ordering', None, 'http://www.beachesbrewing.com', 'http://www.robertocarpetclean.com', 'http://skyharbor.com', 'http://www.brodywebdesign.com', None, '1592154648', None, None, 'http://www.puckettsfarm.com', 'http://www.bishopsplum.com', 'http://www.alleen.com', 'https://www.hilton.com/en/hotels/clemehx-hampton-suites-cleveland-mentor/?SEO_id=YELP-HP-CLEMEHX', 'http://www.ariatroofing.com/', 'http://yazlounge.com', 'http://www.safaripetcenter.com', 'http://www.sukhothaibistro.com/', 'http://teavana.com/ca', 'http://www.furniturestopaz.com/', 'http://www.apexoutdoortreeservices.com', 'http://meridianlv.com', None, None, 'http://www.eddys.com', 'http://www.bobscopyshop1.com', '1592154699', 'http://www.kingspointsmallengine.ca', 'http://www.elpolloloco.com/locations/store/5541', 'http://www.skinstudioaz.com/', None, None, 'http://www.liquordepot.ca/?utm_source=Yelp&utm_medium=Online_Listings&utm_content=Location&utm_campaign=LiquorDepot_AB_Q3_July_September_2017', 'http://www.hcdentallv.com', 'http://www.tomduffyinsurance.com', None, 'https://www.chandleraz.gov/explore/chandler-aquatics/centers-and-pools/folley-pool', 'https://jessieleephotos.com', 'http://Www.crawlingsquidtattoo.com', 'http://www.bojangles.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.airportlimocanada.com', '1592154747', 'http://www.wellsfargohistory.com/museums/phoenix', None, 'https://www.xtremevapour.com', '1592154760', 'http://sunvalleynailspa.com/', 'http://www.quantumcollections.com', None, None, 'http://www.grandstayhospitality.com/find-a-hotel/locations/mount-horeb/overview', 'http://www.swingpt.com', 'https://certapro.com/summerlin/?utm_source=YextNetwork&utm_medium=organic&utm_campaign=YextNetwork', None, 'http://biggscamera.com', 'http://www.deekerssidetracks.com/', 'http://www.humblewinebar.com', 'http://www.theofficepub.ca', 'http://hhtattoo.com', 'http://www.blackphoto.com', None, 'http://kakaallyoucaneat.com', 'http://www.restartbar.com', None, None, 'http://kiplinautorepair.com/', 'http://www.thepizzacompany.biz', 'http://www.checkeredmoon.com', '1592154823', 'http://www.michaelpwebbersondds.com', None, 'http://www.potbelly.com/MayfieldHeights', 'https://www.walgreens.com/locator/walgreens-2363+s+lindsay+rd-gilbert-az-85295/id=5215', '1592154837', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, 'http://www.kirchappliance.com/', 'https://www.erinparktoyota.com', 'http://www.parkatdeervalley.com', 'http://www.westgateaz.com', 'http://www.sugo.ca', None, 'http://bellacinos.com', None, 'http://mintstrength.com', 'http://www.napolivince.ca', 'https://parsleymediterranean.com', 'http://sweeteezsweets.com', 'http://www.pspohio.com/', 'http://www.readytuxedo.com', 'http://www.wildchildclothes.com', 'http://www.mexicanbowl.ca', 'https://bostonpizza.com/en/locations/sunridge.html', None, 'https://www1.shoppersdrugmart.ca', 'http://www.baskinrobbins.com', None, 'https://www.long-mcquade.com/location/Ontario/Brampton', '1592154901', 'http://www.uhhospitals.org/westlake/urgent-care', 'http://www.wholehomeinspectionsllc.com', 'http://www.beautif-eyestudios.com', None, 'http://allbritematerials.com', 'http://www.victoriassecret.com', 'http://www.fransrestaurant.com', 'http://www.fandoctor.com', 'https://www.popeyes.com/store-locator/store/restaurant_83325', 'https://www.usstoragecenters.com/self-storage/arizona/glendale/us-storage-centers/126', 'https://stores.nordstromrack.com/us/az/gilbert/2090-e-williams-field-road?utm_source=yelp&utm_medium=organic&utm_campaign=rack&utm_content=154&utm_channel=lcl_acq_p&sp_source=yelp&sp_campaign=rack', 'http://www.trilogylife.com/communities/arizona/vistancia/', 'http://www.carmellaspizza.com/', 'http://www.skysongcenter.com', 'http://www.sessomsandsons.com', 'http://fossilandhaggis.com/', None, None, 'http://casablancanails.com', None, 'http://www.erbertandgerberts.com/locations/detail/sun-prairie', 'https://cafevialinda.com', 'http://www.tesororanchapts.com', 'http://www.icecreampizzaworks.com', 'http://www.pizzanova.com', 'http://www.te-koop.ca', None, 'http://www.cardealsdirect.ca', 'http://getnutslab.com', 'http://www.artinthepark.org/', None, None, None, 'http://www.CampanasPizza.com', None, None, 'http://www.phoenix.gov', 'http://www.pwaesthetics.com', None, 'https://www.encantopoolcare.com', None, None, 'http://greekstop.ca', 'http://www.mortproductions.com', 'http://trendsettereyewear.com', 'http://www.starbucks.com/store/15887/', None, 'https://www.anytimefitness.com/gyms/4028/Mint-Hill-NC-28227/?utm_source=yelp&utm_medium=local', 'https://www.gowireless.com/stores/az/goodyear/goodyear-estrella-marketplace/', 'http://www.kitchenstuffplus.com', None, 'https://www.choicehotels.com/CN283?mc=blypypaf', 'http://phxfloors.com', 'http://www.tenpennyimc.com', 'http://www.winchellspub.com/', None, None, 'http://www.kisellasurgery.com', 'http://www.thegraniteshopofmadison.com', 'http://www.coffeebean.com', 'http://www.completeeyecareaz.com/', 'http://www.palazzo.com/azure', 'http://www.azinspector.com/', 'http://www.dollartree.com/home.jsp', None, 'http://www.cartersgymnastics.org/', None, 'https://www.extraburger.ca', 'http://www.elhefe.com/scottsdale', 'http://www.baschs.com', None, 'http://www.monroevillekia.com', 'http://farmersfurniture.com/', None, None, 'http://www.abestautorepair.com', 'http://www.northlakechiro.com', 'http://drkennedychiro.com/', 'http://www.artgalleryofmississauga.com/index.html', 'http://www.uptowncharlottecleaner.com', 'http://www.buckdeitzphotography.com', 'http://www.manuelsaz.com/', 'http://www.funkymunky.ca', 'http://tandmautomotive-hub.com', 'http://smokespoutinerie.com', 'http://www.galleriashoppingcentre.ca', 'http://www.functionpilates.com', None, '1592155146', 'http://www.allthingseyeslv.com', None, None, None, 'https://www.mgmgrand.com/en/nightlife.html', 'https://arizona.weidner.com/apartments/az/phoenix/arezzo/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'http://www.flawlessfaceandfigure.com/', '1592155164', 'http://www.clerkofcourt.maricopa.gov/', 'http://www.chaska.ca/', 'http://www.highdesertoralsurgery.com', 'http://www.wins.doitbest.com', 'http://www.gtmassages.com', 'https://www.pioneerfamilypools.ca/locations/newmarket/', None, 'https://www.lvspaandnailsvegas.com', 'http://thebikeproject.org', 'http://www.850Degrees.ca', 'https://stores.joann.com/oh/hudson/171/', None, 'http://www.phoenixlockoutsandroadsideassistance.com', 'http://www.bettercontracting.ca', 'https://www.popeyes.com', None, None, 'http://icirides.com/', None, 'http://www.7-eleven.com/?yelp=29384', 'http://www.ritascleaningllc.com', None, '1592155224', 'http://www.rockbottom.com/', 'https://soswaxlv.com', None, 'http://www.pelicanssnoballs.com/lakewylie123.php', 'http://www.shopmetcalfes.com', None, None, 'https://www.themotherhoodco.com', None, 'https://losgirasoles.us', None, 'http://www.bowlofgreensfinesalads.com', None, 'http://planetpetopia.com', 'http://www.platinumentourage.com', 'http://www.drsteed.com', None, 'http://www.chopchopwok.com/', 'http://www.mizumisushi.ca', None, 'http://www.lachotsmiles.com', 'http://www.laketowne.com', 'http://www.carolynes.com', 'http://www.golfsmith.com', 'http://www.themerchant.biz', None, 'https://thepetclub.com', 'https://www.loszarapes.com', 'http://www.dmvnv.com/locat.htm', 'http://www.mrpuffs.com/fr/', None, 'http://distilledbeautybar.com', 'http://www.intothefirepizza.com', 'http://www.womeninneed.net/', 'http://www.rianahairstudio.com', 'http://planetsmoothie.com/', 'http://www.asianhealingartsandacupuncture.com/', 'http://www.sosappliancerepair.com', 'http://www.greaterlasvegasdental.com', None, None, 'https://www.cellphonerepair.com/phoenix-central-az/', 'http://GreenMaid.Info', 'http://www.goldennugget.com/home/entertainment_attractions/defending_the_caveman/', 'http://www.vegasvacuumsewing.com', None, 'http://www.rockwoodnaturalmedicine.com/', 'http://www.club42.com', 'http://newmarket.colmustardspub.com', 'http://www.lazyloafandkettle.com', None, None, 'http://www.thepilatesbodylasvegas.com', None, None, None, 'http://www.laptopxchange.com/phoenix-az-computer-repair/', 'https://www.4pawspetresort.com/?utm_source=yelp&utm_medium=social', 'https://www.cartridgeworld.com/las-vegas-nv/89102-0318/about-us', None, None, 'http://www.economy-glass.com', 'http://www.novaristorante.ca', None, None, 'http://locations.harristeeter.com/nc/davidson/268/', 'https://dentalsolutionslv.com', 'http://www.doggiedoorsontherun.com/', 'http://www.thaicuisineexperts.com', 'http://www.aztechglasstint.com', None, None, 'http://www.designercakeshop.com', 'http://www.myfoodcity.com', 'http://www.kmart.com/stores/michigan/monroe/4098.html', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'https://www.themetatfashioncenter.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.mcqueenplumbing.com/index.html', None, 'http://communityculinary.org/', 'http://www.classroomcentral.org', None, 'http://www.saltydogpetspa.net', 'http://www.thebodyrx.com', None, 'https://www.moonvalleynurseries.com/services/tree-service?utm_campaign=yelp&utm_source=yelp-az&utm_medium=tree-care', 'http://www.wm.com/', '1592155455', 'https://dema.az.gov/army-national-guard/papago-park/arizona-military-museum', 'http://www.glendalesalonaz.com', 'http://www.hcpnv.com/clinic/wynn', 'http://www.noirtraining.com', 'http://www.heritagecrownaz.com', 'http://massageworkscharlotte.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=803&utm_content=bpas', 'http://www.OrganicMassageCleveland.com', 'http://www.BeefOBradys.com', '1592155482', 'http://www.parktowerapts.com', 'http://chartiersanimalhospital.com/', 'http://ordercasadeplata.com', 'https://www.omniwellnesscenter.com', None, None, 'http://www.horsinaround.ca/', 'https://petvalu.com/location/2034', None, None, 'http://www.karlaskarslv.com/', 'http://www.dogsuppliesvegas.com', 'http://www.savon.com', 'http://www.foodbasics.ca', 'http://www.cryscakecreations.com/', 'https://www.autorepairpittsburghpa.com', 'https://vrplayin.ca', None, 'http://www.philsebastian.com', 'http://www.gvdentalcenter.com/', 'http://www.barringtonplaceapts.com/', 'https://locations.theupsstore.com/az/surprise/15508-w-bell-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.hinkleyslighting.com', 'http://laptopexpert.com', 'http://www.bathandbodyworks.com', 'http://www.jimmysnycpizzeria.com', 'https://locations.theupsstore.com/az/phoenix/4802-e-ray-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.rotaryevolution.net/', None, None, 'http://www.baskinrobbins.com', 'http://www.apple.com/fashionshow', 'http://www.gohunanofstow.com', None, 'http://www.piacere.ca', None, 'http://www.azcenturypoolplastering.com', 'http://www.acclivityhealthcare.com', None, None, 'http://www.gallaghersexpertautorepair.com', 'http://www.containerbaryyc.com', None, None, 'https://www.superstitiongrandantiquemall.com', None, None, 'https://beehiveceilings.com', 'http://www.toppackgear.com', None, 'https://www.medexpress.com/location/pa/pleasant-hills/plp/?utm_source=yelp&utm_medium=paidsocial&utm_campaign=Evergreen', None, None, None, 'http://www.phoenixgreyhoundpark.com', None, '1592155627', 'http://www.autobell.com', 'http://slshotels.com/lasvegas/', None, None, 'https://www.hilton.com/en/hotels/pitsmes-embassy-suites-pittsburgh-downtown/?SEO_id=YELP-ES-PITSMES', 'http://bestchinainc.com', None, '1592155648', None, 'http://www.ArrowAH.com', None, 'http://dswcenter.com', None, '1592155660', None, 'https://tonyspokerschool.com', 'http://www.aetna.com', 'http://www.pizzafire.com', 'http://www.cakesonthemove.com', 'http://www.mandeljcc.org/', 'http://www.shakesandfranks.com', 'http://www.erindaleanimalclinic.com/', None, 'http://www.bloorwestvet.com', None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/501/?utm_source=yelp&utm_medium=listing&utm_campaign=website', None, 'http://www.bestmatt.com/', 'https://agents.allstate.com/wes-cain-gilbert-az1.html', None, 'http://www.scoutandcash.com', None, 'https://soccerpostarrowhead.com', 'http://www.saeedsbarandgrill.com', None, None, 'http://www.thesuntanboutique.com', None, None, 'http://www.childrensurgentcare.com', None, 'http://www.nikospizzalasvegas.com', None, None, 'https://enjoi-nails-salon.business.site', None, None, 'http://poolpartsplus.net/', None, None, 'https://locations.davidstea.com/on-toronto-1', 'http://clickserve.dartsearch.net/link/click?lid=43700006435127445&ds_s_kwgid=58700000444950505&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://myamericanfurnituregalleries.com', 'https://www.circlek.com/store-locator/us/peoria/9476-w-olive-ave/2706468', 'http://www.tikiicevegas.com', None, None, '1592155772', 'https://www.challengerschool.com/campus/nevada/las-vegas/silverado', None, 'http://www.fredmeyerjewelers.com', 'https://shadysidehousecleaning.com', None, 'http://serranosaz.com', 'http://www.expressflooring.com', None, 'http://www.clarkcountynv.gov/depts/clerk/services/pages/civilmarriages.aspx', 'http://griegofamilyeyecare.com', None, 'http://www.thunderbirdobgyn.com', 'http://www.bowkerchiropractic.com/', 'https://www.ventospizzapgh.com', 'http://www.minhchauoptical.com', None, 'http://www.michael-a-anderson.com', 'http://www.thewallpapermarketofmesa.com/', 'https://www.lvac.com/locations/henderson', 'http://subway.com/', 'http://www.sarphx.com', 'http://rokvegasnightclub.com', None, 'http://www.dovespa.ca', 'http://www.reprodux.com/', None, 'http://www.birch-dan.ca', 'http://www.nellosauto.com/', 'https://fantasticworldscomics.com', None, 'https://arizonadental.com/contact/phoenix-az-office/', None, 'http://www.strongsvillevacuum.net', 'http://www.ufixitautomotive.com', 'http://www.greenlinehair.com', 'https://www.lifestorage.com/storage-units/arizona/phoenix/85255/787-near-pinnacle-peak/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-AZ-787&utm_content=website-link&howheard=IYE', 'http://www.fremontweddingchapel.com', 'http://eh25.com/tasting-room/', 'http://www.paramountfinefoods.com', None, 'http://www.stevenjosephphotography.com/reviews', 'http://www.trappedcle.com', None, 'http://roxannesonline.com/', None, 'https://www.medusacle.com', None, None, 'http://www.fikesnet.com', 'http://cobraplumbing.net', 'http://www.tranteam.com', 'http://chinaaaadelivery.com', 'http://www.gangnamstreetlv.com', None, None, None, None, None, None, 'http://www.rapidrehablv.com/', '1592155928', 'https://reviveptandwellness.com', 'http://www.bronzebeautyst.com/', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'http://www.dunkindonuts.com', 'http://www.savers.com', 'http://www.kulinars.net', None, None, 'https://www.tesla.com/findus/location/store/calgary', 'http://www.vibebeautylounge.com', None, 'http://www.sunsetstation.com/entertainment', '1592155959', 'https://www.moneytreeinc.com/branch/north-nellis', 'http://refinedlandscaping.com/', 'http://www.enterprise.com', 'http://www.kidsrkidsmatthews.com', None, None, 'http://www.simsroofingandsiding.com', 'http://www.shoplostfound.com/', 'http://www.galaxylg.com', 'https://www.chopsticksngrill.com', None, None, 'http://www.cal-am.com/communities/tropicana-palms', 'http://www.johnsfamilyrestaurant.com', 'https://www.toronto.ca/data/parks/prd/facilities/complex/337/index.html', 'http://www.onehazelton.com/', 'http://www.handinhandpreschool.com/index.html', 'https://diminishedvaluecheck.com', None, 'http://www.BradyLuxury.com', None, 'http://www.bikramyogamesa.com', 'http://www.niceguystechnology.com', '1592156015', None, 'https://www.kohls.com/stores/oh/macedonia-235.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.castlemegastore.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=phoenix_yelp&utm_content=2017', None, 'http://www.kingwongchinese.com', 'http://l.macys.com/scottsdale-az?cm_mmc=Yelp_Stores-_-Scottsdale-_-n-_-450&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Scottsdale_450&m_ac=yelp_stores&m_ag=n', 'http://www.therisingsun.ca/', 'https://www.ahn.org/locations/healthwellness-pavilion/peters-township', None, 'https://www.impactarcherylv.com', None, None, None, 'http://www.greatclips.com/', None, None, 'https://www.rossoutdoors.com', 'http://www.allianceurgentcare.com', None, None, None, 'https://bounceu.com/matthews-nc', 'http://www.thestudiosalon.biz/', 'http://www.jackieo.ca', None, 'http://www.owlremodeling.com', 'http://wiglewhiskey.com', None, None, None, 'https://store.vioc.com/nc/denver/7575-highway-73-28037-GM0004', None, 'http://www.mymindseyerecords.com', None, None, None, None, 'https://www.themovingmasters.com', 'http://www.rodgunandbarbers.com', 'https://locations.davidstea.com/on-toronto-1', 'http://scottsdaleautoglass.com', None, 'http://ad.doubleclick.net/ddm/clk/283420830;110240480;l', None, 'http://prestigebeautysalo.wix.com/prestigesalon', 'https://www.tileshop.com/store-locator/north+charlotte.do', 'http://www.colkerjanitorial.com/', 'http://www.justforpleasurenc.com/just-for-pleasure-2-north-charlotte-nc/', None, 'https://www.accucaredental.com', 'http://www.eltororestaurante.com', 'https://www.dicksspeedotach.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://trunzosauto.com', 'https://www.originaljoes.ca', '1592156149', 'http://RalphieMayVegas.com', None, 'http://www.greengrotto.ca', None, 'http://carolinashomestone.com', 'https://www.montecarloinns.com/markham-suites', 'https://www.wellsfargo.com', 'https://www.marathonpetroleum.com', 'http://www.recordsreduction.com', 'https://www.rmcf.com/AZ/Tempe50423', '1592156173', None, 'http://www.newcenturyhairdesign.com', 'http://www.bloorwestwinery.com', 'http://www.costco.com/warehouse-locations/scottsdale-az-427.html', 'http://thearomatherapygarden.com', 'http://www.aaronlending.com', 'https://local.biglots.com/nv/las-vegas/4500?zcp=pd_local_yelp', '1592156193', None, 'http://nes-electrical-services.business.site', 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2faz%2fphoenix%2fmesa-west&sisearchengine=1524&siproduct=BEMES_Bus', 'http://othermamalv.com', 'http://www.n15salon.com', 'http://www.eastmesapediatrics.com/Home_Page.php', 'http://www.timhortons.com', 'http://www.shoelessjoes.ca', 'http://www.85254.com', 'http://www.funasiancuisine.com', None, 'http://mazzagarden.com/', 'http://www.littlecaesars.com/Locations/tabid/92/location/28079/Default.aspx?id=MTU1MzAwMTc%3d', 'http://www.accesstohealthcare.org', 'https://www.kidsquest.com/location/redrock/', 'http://www.siegelandcompany.com', 'http://www.miraclesofphoenixfitness.com', None, 'https://www.sheetz.com', 'http://www.elysianatstrose.com/', 'http://Www.federalgalley.org/restaurants/Shaka/', 'http://www.bowiechiropractic.com', 'https://www.topnotchwrapslv.com', 'http://simplicitysalonlv.com/', 'http://www.resumesolutions.ca', 'http://www.steamwayservices.com', None, None, 'https://collegehunkshaulingjunk.com?utm_source=yelp&utm_medium=organic', 'https://www.italianrestaurantrivergate.com', 'https://www.booksamillion.com', 'http://www.starskycanada.com/aboutus.php?s=4', 'http://www.platinumwellness.net', None, None, None, 'http://www.sheratoncavaliercalgary.com/calgary-restaurant', 'http://www.massagealacarter.com/', 'http://www.vintagewireandsupply.com', 'http://www.meyersmanagement.com', 'https://www.orthoarizona.org/physicians/Caleb-Behrend', 'https://heartlandfarmsanctuary.org', 'http://benefitcosmetics.com', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://locator.chase.com/ny/glendale/71-04-woodhaven-blvd', 'http://www.thespiritconnection.com', None, 'http://Adventurehelicoptertours.com', None, 'http://www.studiowish.com', 'http://www.luckybrand.com', 'http://www.glamorgananimalclinic.com', 'http://evermorenevermore.com', 'http://www.flooring2day.com', 'http://www.rosedrivingschool.com', None, 'https://www.marquiscompanies.com/facility/marquis-centennial-hills/', 'https://www.queenofheartscasinoparties.com', 'https://store.vioc.com/wi/stoughton/1108-us-51-138-53589-GR0034', 'http://solylunabeautystudio.com', None, 'http://chinatocleveland.com', 'http://www.coffeedaklak.com', 'https://www.ediblearrangements.com/fruit-arrangements?utm_campaign=StoreID561&utm_medium=Listing&utm_source=Yelp', 'http://www.christophier.com', 'http://www.paramountfinefoods.com', None, 'http://www.ballantynevillage.com', 'https://stores.advanceautoparts.com/nc/charlotte/2910-freedom-drive?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', 'https://www.walgreens.com/locator/walgreens-6649+morrison+blvd-charlotte-nc-28211/id=1376', 'http://andysmechanicalser.mechanicnet.com', None, 'http://www.mattressfirm.com/', None, None, 'https://www.sandmanhotels.ca/hotels/calgary-airport/', 'http://www.MontysFurniture.com', 'http://www.urbanamarket.org', 'https://roadside-picnic.business.site', 'https://www.woolfeyecare.net', 'http://www.neatedupey.com', 'http://www.azpluscleaning.com', 'http://www.JimmyJohns.com', None, 'http://www.doubletreeinspectionsofaz.com', None, None, 'http://www.habitburger.com', None, 'http://www.ShaolinArizona.com', 'http://www.providencecourtapts.net', None, 'http://howardstowingpittsburgh.com/', 'http://www.fantasticsams.com/', None, None, 'http://www.contessacatering.com', 'http://projectmanagementacademy.net/las-vegas/pmp-certification.php', 'http://www.scottsdalelibrary.org/', 'http://richtreenaturalmarket.com/', 'http://www.heightslibrary.org/', 'http://www.alibabas.ca/index.php?route=common%2Flocation_details&loc_id=1', 'http://www.lids.com', 'http://www.terracharlotte.com', 'http://l.macys.com/chandler-az?cm_mmc=Yelp_Stores-_-Chandler-_-n-_-453&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Chandler_453&m_ac=yelp_stores&m_ag=n', 'http://www.galleriaatsunset.com', None, 'http://www.primrosegilbertsantan.com?utm_campaign=localmaps&utm_medium=maplistings&utm_source=yelp', 'http://www.glendalebarber.com', 'http://www.costnerlaw.com', 'http://www.permanentbeautyusa.com', 'http://www.scottsdaleaz.gov/Parks/chap.asp', 'http://www.picknsave.com', None, 'http://freshandeasy.com', None, None, 'https://www.teslamotors.com/en_GB/findus/location/store/rosspark', 'http://bmwofchampaign.com', 'https://pvpd.com', 'http://www.poisonivysalon.com.', 'http://www.rmfenceohio.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://motherstamales.com', 'http://www.pamier-kabob.com/', None, None, None, 'http://www.mabelsbakery.ca/', 'https://www.hilton.com/en/hotels/clepwht-home2-suites-cleveland-independence/?SEO_id=YELP-HT-CLEPWHT', 'http://www.costco.com/warehouse-locations/henderson-nv-673.html', 'http://www.smiledesigncenterlv.com', 'http://www.mononachinastar.com/', None, 'https://www.trendssalonllc.com', 'http://www.heysugarstudio.com', 'https://www.choicehotels.com/arizona/scottsdale/rodeway-inn-hotels/az437/rates', 'http://www.thorespressobar.com', None, 'https://izzysbakeryandcafe.com', 'http://tonydblc.webs.com/', None, 'https://www.teamlogicit.com/PhoenixAZ302/', '1592156578', 'http://clickserve.dartsearch.net/link/click?lid=43700006430431858&ds_s_kwgid=58700000444952719&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', '1592156585', None, 'https://www.palmetto-eye.com', 'http://www.solaireonscottsdale.com/', 'http://www.westernreservehospital.org', 'https://www.sleepcountry.ca', 'http://theupsstore.ca/148', 'http://www.vip-pools.com', 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', None, 'http://7eleven.ca/store-locator/33259', 'https://www.popeyescanada.com/maps/toronto-liberty-village.html', 'http://www.theproducebox.com', 'http://www.elitepestcontrolnv.com', 'https://www.worldwidegolfshops.com/locations/van-s-golf-shops-mesa-801-s--power-road.html', None, 'http://www.wearejoviec.com', 'http://www.zajecpainting.com', 'http://www.spicy-garden.ca/', 'http://www.pittsburghoralsurgery.com', 'https://www.alterationsandcreationsaz.com', 'http://www.marriott.com/hotels/travel/yyzfs-fairfield-inn-and-suites-toronto-airport', 'http://www.wetseal.com', None, 'http://www.gioninos.com', None, None, 'http://www.gioiabeautybarspa.com', 'http://homedetectiveohio.com/', 'http://www.4seasonstreesllc.com', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.buylaptops.ca/', 'http://www.pittsburgh.va.gov', 'https://hoame.ca/contact-us/', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', None, 'http://eatzencore.ca', None, None, None, None, None, 'http://www.teamfreemanhomeloans.com', None, '1592156716', None, 'http://www.snapfitness.com/fountainhills', None, 'http://www.scooterscoffee.com', None, 'http://www.szechuanexpress.com/', 'http://baycoins.wordpress.com/', 'http://www.computerlasvegas.com', None, None, 'http://familylawguys.com/contact/scottsdale', 'https://www.sephora.com/happening/stores/phoenix-jcpenney-at-ahwatukee-foothills', '1592156756', None, 'http://www.swanpizza.com', 'http://cartridgeworldphx.com', 'http://www.amerizonaproducts.com', 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', 'http://www.bridlewoodmall.ca/', None, 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=clein&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', None, 'http://www.faithculture.ca', '1592156791', 'http://www.itsugar.com', 'https://local.fedex.com/az/phoenix/office-5048/', '1592156801', 'http://www.monkeybargym.com', None, None, None, None, 'http://www.quiktrip.com/', 'http://www.signaturewellness.org/', 'http://junipergrill.com', 'http://www.michaudscatering.com', 'http://www.bikeandforth.ca', 'https://www.artlewinbespoke.com', None, 'http://azteamania.com', 'http://www.miegakoreanbbqcleveland.com', None, 'http://goldenasiancuisine.com/', 'http://southparkvet.net/', 'http://www.tamcapturesphotography.com', 'http://www.glotique.com/', 'https://www.t-mobile.com/store-locator/az/chandler/chandler-fashion-center?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://asapcity.com', 'https://www.idealimage.com/medspa-locations/nevada/las-vegas-nv-area/lasvegashendersongreenvalley', 'http://Www.ktsfish-chicken.com', 'http://dreamdinners.com/main.php?page=store&id=53', 'http://www.rockstarhotdogs.com/', None, 'http://www.scottsdaleaz.gov/scottsdale-stadium', '1592156886', 'http://www.discountfirearmslv.com', 'http://www.homelife.ca/office_agents/2164', 'http://www.pillarwestgate.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', 'http://www.treasureisland.com/restaurants/little_richies.php', None, 'https://noracleveland.com', 'http://www.cabelas.ca/find-a-store/calgary/14', 'http://www.nevadagastro.com', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'https://www.extraspace.com/storage/facilities/us/arizona/chandler/1000000272/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', '1592156922', 'http://www.big4motors.com', 'http://telemindclinic.com', 'https://www.wildblue.studio', 'http://theistanbulgrille.com', 'http://www.dunkindonuts.com', 'http://www.fluorescentnails.com', None, 'http://www.leveloneaz.com', 'http://www.summitmedicalweightloss.com/', 'http://hopesbridalnc.com', None, None, 'http://www.meninkilts.com/calgary', 'http://www.harkins.com', None, None, None, None, None, None, 'http://www.marriott.com/hotels/travel/cltfm-fairfield-inn-and-suites-charlotte-matthews/', 'http://luvcuts.com', 'http://www.toronto.ca/parks/parks_gardens/allangdns.htm', 'http://www.abrazadental.com?utm_campaign=local&utm_medium=citations&utm_source=yext', None, None, 'https://www.take5oilchange.com/locations/oh/mayfieldheights-130/', '1592157006', 'https://www.ihop.com/en/restaurants-gilbert-az/920-s-gilbert-rd-1529', 'http://www.murillocarpet.com/', 'https://www.levinfurniture.com/furniture-store/pa/pittsburgh/5438-baum-blvd', None, 'http://www.stanfordcapitalloans.com', 'http://www.jpgexhaust.com', None, None, None, 'http://www.cpr4yourcar.com', 'http://lulyssalonllc.weebly.com/', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/328+Cochran+Road-Mount+Lebanon-PA-15228/storeid=5094?cid=ps_ylp_2020', 'https://www.macu.com/branch-locations/az/gilbert/gilbert-branch/0146', None, 'http://www.JimmyJohns.com', 'http://www.cpr-savers-training.com', 'https://www.paviliondentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.trane.com/residential/en/buying-a-trane/find-a-dealer.html/401830', None, None, 'http://www.carvel.com', 'http://baliandbeyond.ca', 'http://www.karaokerentalphoenix.com', 'http://www.eggsmart.ca', None, None, 'http://www.vogava.com', 'http://www.mrcookschinesecuisine.com', None, 'http://streetsofnewyork.com/', None, 'http://www.truphysicaltherapy.com', 'http://savagewalker.com', 'http://www.tntauctionssnelgrove.com/', 'http://goodworksautorepair.com', 'http://www.hikethislasvegas.com/', 'http://www.pacificforesightsecurity.com/free-security-yelp', None, 'http://www.oneluckypup.com', 'http://www.stovalldentistry.com', 'https://www.dominos.com', None, 'http://www.plushsalonaz.com', 'https://www.kaizenprogressivehealth.com', 'http://www.stoppleworthplumbing.com/', 'http://www.customfitdj.com', None, 'http://www.autodealzaz.com', None, 'http://www.smoothieking.com', 'http://floweramaofglendale.com/', 'http://benedictinewomen.org', None, 'http://www.purevanityspa.com', 'http://www.salonentro.com', 'http://www.shell.us/', 'https://canyonstateclassics.com/contact', 'http://Slicersonmain.com', 'http://www.1brotherspizza.com', None, None, 'https://www.aa.com', None, 'http://www.drlfish.com', 'http://dowdentalcare.com', 'http://www.dignitymemorial.com/funeral-homes/mesa-az/melcher-mortuary-mission-chapel/4527', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.rotorooter.com/scottsdaleaz', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', None, None, 'https://kottasushi.ca', 'http://www.medvetforpets.com/location/medvet-akron/', 'http://www.internationalhotel.ca', 'http://www.lonemortgage.com', None, 'https://toothworks.com/toothworks-bay-adelaide-dental/', 'http://www.steves-auto-service.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.valleyhomepros.com', 'http://www.smallbarclt.com', None, 'http://www.toscanacondorentals.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281379878&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'https://www.goldbergcentre.com', 'http://www.roosterskountrykitchen.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89102/838069/', 'https://www.altierus.org/campus/henderson', 'http://www.coach.com', 'https://medmen.com/stores/scottsdale-north-scottsdale-airport', 'https://hihidonuts.business.site', 'http://www.bulwarkpestcontrol.com/', 'http://ranchoflooringvegas.com', '1592157278', 'http://www.JimmyJohns.com', 'https://www.phoenix.gov/publicworks/garbage/disposable', None, 'http://singsing.com', 'https://aestheticrevolutionlv.com', None, 'https://www.pimtmassage.com', 'http://www.cocosbakery.com/', 'https://www.capriottis.com', 'http://acewasherservice.com/', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'https://www.socialsecuritybranch.com/nv/henderson/henderson-social-security-office/10416-s-eastern-avenue/', 'http://www.collisionrepairphoenix.net', None, None, None, None, 'https://www.olivegarden.com/locations/az/scottsdale/scottsdale-e-cactus-rd/1234?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:scottsdale-az-1234_pl:locurl_rd:1175', 'http://www.jimscoins.com', None, 'http://serenitycleaningaz.com', 'http://www.reddoorliving1.com/delta-lakewood/', 'https://rdavis.amerifirstloan.com', None, None, None, None, 'http://www.tbdnursery.com', None, 'http://www.thescottresort.com/?utm_medium=enhanced-profile&utm_source=yelp', 'https://www.hafezpersianrestaurant.com', 'https://www.zaxbys.com/locations/ga/monroe/195-martin-luther-king-jr-blvd/', 'http://www.yetifirearms.com', None, 'http://www.bombaypalacetoronto.com', None, 'http://lebanesebakery.ca/', 'http://www.midabui.com', 'http://www.whoolisbarandgrill.com', 'https://www.yourcupcakequeen.com', 'http://www.banfield.com/veterinarians/pa/monroeville/moe?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'https://www.vannormanlaw.com', 'http://www.eaglehomeinspections.net', 'https://www.anytimefitness.com/gyms/372/Huntersville-NC-28078/?utm_source=yelp&utm_medium=local', 'http://oralsurgeryaz.com', None, 'http://www.cyclevegas.com', 'http://www.masteryoon.net', 'https://www.brighton.com/viewstore/123', None, None, '1592157435', 'https://www.riteaid.com/store-details?storeNumber=11467', 'http://www.soundplumbinglv.com', 'https://www.envision1361.com', 'http://www.gurusweets.ca', 'http://rebathandkitchens.com', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.nikolastreeservice.com', 'https://www.doctorallenwellness.com', None, 'https://www.dominos.com', 'https://www.ncfdentistry.com', 'http://www.vervehairdesign.com', None, '1592157478', 'http://www.lossombreros.com', '1592157484', None, None, 'https://local.fedex.com/az/tempe/MSCA/', None, 'http://www.ilovesushihenderson.com', 'http://www.okraaz.com/', 'http://www.starbucks.com/store/9298/', 'http://mascariauto.com', 'http://www.azdanceproject.com/', 'http://www.thetilestore.ca', 'http://www.germancarscottsdale.com', 'http://fracturedprune.com/location/shops-norterra', 'http://tasty-chinese.com', 'http://www.haydeedocasarmd.com', None, 'http://bubblesandsqueak.com', 'http://www.greatclips.com/salons/6225', 'https://www.unclelouie.com', None, 'http://www.firemeupaz.com', '1592157549', 'https://www.hilton.com/en/hotels/lasdmhw-homewood-suites-las-vegas-city-center/?SEO_id=YELP-HW-LASDMHW', None, None, '1592157562', 'https://yonge-and-eglinton.orangetheoryfitness.com?utm_campaign=yelp&utm_term=072017&utm_medium=print&utm_source=redirect&utm_content=yelp_print', 'http://www.beerexpressisbest.com', None, None, 'http://www.omahasteaks.com/stores', None, 'http://www.commonwealthbar.ca/', 'http://www.silverminesubs.com/', 'http://www.presotea.ca', 'http://www.myharu.ca', 'http://www.piedmontgoldexchange.com', '1592157598', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://crclasvegas.com', 'http://www.wildwingrestaurants.com/franchisees/newmarket-north/', 'http://www.apples4you.com/', None, None, 'http://www.marinersmadison.com/', 'http://www.century21.com', 'https://www.mcdonalds.com/us/en-us/full-menu/123dollarmenu.html?cid=PS:GCM_MOP:NB::Yelp:All', 'https://www.choicehotels.com/CN387?mc=blypypaf', 'http://www.vegasinkandtoner.com/', 'http://www.johnvarvatos.com/storedetails?StoreID=5656', 'http://www.westsidecommunitymarket.org', 'http://richnerair.com', None, 'https://www.kwiktrip.com', '1592157649', 'http://www.starbucks.com', 'http://www.timhortons.com', None, 'https://www.thejoint.com/north-carolina/huntersville/birkdale-12006', 'http://www.lukehousemadison.org', 'http://www.greatclips.com/', 'http://local.firestonecompleteautocare.com/arizona/phoenix/3638-e-thomas-rd/?lw_cmp=IYP_YPC_MLP_13315', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, 'http://www.smokeandjoelounge.com', None, 'http://botecolv.com', 'http://www.keelinglawoffices.com', 'https://puredentalaz.com/home.htm', 'https://www.thewestlake.ca', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.greyhound.com/social?utm_source=Book%20now&utm_medium=Yelp&utm_campaign=permanent%20link%20', 'http://aaronmachineshop.com', 'http://www.vegasprohvac.com', None, None, None, 'http://www.torontoimageworks.com', None, None, 'https://desertrad.com/locations/southwest-wynn-road', None, 'http://rocksolid-surfaces.com', None, None, 'https://www.lappeneyecare.com', 'https://www.bentosushi.com', 'http://www.drrusselo.com', None, 'http://www.dunkindonuts.com', 'http://nouveauxclt.com', None, None, 'http://Pizza-wings.com', 'http://tracysautobody.com', None, None, 'https://www.marquiscompanies.com/facility/marquis-plaza-regency/', 'http://www.nascarspeedpark.com/nc.aspx', None, None, 'https://www.gowireless.com/store/stores/Arizona/Mesa/Mesa---MonteVistaVillageCenter----602609', 'http://www.kwikfill.com', 'http://www.usps.com', 'http://www.dolcegelato.net', 'https://www.hilton.com/en/hotels/pitokhx-hampton-pittsburgh-university-medical-center/?SEO_id=YELP-HP-PITOKHX', 'http://www.exclusivetailoring.com', 'https://www.martinandmacarthur.com/', None, 'http://www.marleypark.com/', 'http://www.corepestsolutions.com', 'https://www.marcustheatres.com/', 'https://www.dippindots.com/loc/ll/US/NC/Charlotte/6801-Northlake-Mall-Dr_', 'http://www.healthydentalalternatives.com', 'http://www.northmountainvisitorcenter.org', '1592157819', None, 'https://www.fjtesq.com/?npcmp=dir:local:4407007:89145', None, '1592157829', 'http://www.mwuclinics.com', 'http://www.windowprollc.com', 'http://www.azairductcleaning.com', None, 'http://www.aamcolasvegas.com', 'http://www.casinonight2you.com', None, None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.dairycream.org/', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_426', 'http://www.ntb.com/store/8522/bedford-oh-44146-2252?utm_source=yelp&utm_medium=maps&utm_campaign=yelp-maps', None, 'http://www.lovebailbonds.vegas', 'http://downtownfood.ca/', '1592157877', '1592157879', None, 'http://www.apollos.com', None, None, 'http://www.dairyqueen.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://bennysva.com', 'http://urbantealoft.com/', None, None, 'http://www.25celciusbeautybar.com', 'http://www.simplicitylaser.com/locations/nevada/henderson/', 'http://www.hopkinsautospa.com', 'http://www.junk-rite.com', 'https://locations.dennys.com/NV/LAS-VEGAS/247828', None, 'http://www.arborealty.com', None, 'http://www.9dragonskungfu.com/', None, 'https://www.storquest.com/self-storage/az/tempe/9052', None, 'http://www.nailswithlovelv.com', 'http://www.daveyplumbingpittsburgh.com', 'https://birdysoulfood.com', 'http://motorsandmoreinc.com', None, 'http://www.steaknshake.com/locations/24097-steak-n-shake-us-hwy-17-s-brunswick', 'http://surprisefamilymedicine.com', None, 'http://www.czcustom.com', 'http://Dogfather.me', 'http://clickserve.dartsearch.net/link/click?lid=43700006430792752&ds_s_kwgid=58700000444978432&ds_url_v=2', 'http://www.kelseys.ca/location_region.php?province=on&s=1&city_drop=North+York', None, 'http://jianphoto.com/', 'http://www.worldofdecorauctions.com', 'http://www.goodwillsp.org/shop/retail-stores/pineville/', None, None, None, 'http://www.childrensorchard.com/stores/las-vegas-nv/', None, 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', 'https://www.pescadoborracho.com', 'http://www.spiceshack.ca', None, None, None, '1592158252', None, None, None, 'http://www.extendasuites.com', None, None, 'http://www.docautorepairs.com', 'https://www.signaturestyle.com/locations/oh/medina/famous-hair-medwick-marketplace-haircuts-67153.html', None, '1592158276', None, 'https://www.luxor.com/en/nightlife.html', '1592158284', 'http://www.southstburger.com/location/35-first-commerce-dr/', None, 'http://shopmelrosevintage.com/', 'http://www.mrsub.ca', 'https://imwireless.net/locations/concord-ma/', 'https://lasvegaspainreliefcenter.com', None, 'https://www.extraspace.com/storage/facilities/us/arizona/mesa/501578/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.fastfreshfoods.ca/', None, 'http://www.zazaespressobar.com', 'http://www.snapfitness.com/gyms/lasvegas-nv-89147/7089', 'http://www.forever21.com', 'http://www.ssmokincafe.com', None, None, None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Carnegie-PA-15106/067741/', 'http://www.saloncafe.net', 'https://www.blumenailsspasolon.com', 'http://www.gehendricks.com/', 'https://www.norfolkhall.com', 'http://missionptaz.com', 'http://www.visithenderson.com/todo/special-events/stpatricksdayparade', 'http://www.skinapeelbeautybar.com', 'http://www.ccdcranch.com', 'http://www.craftdogs.com/', None, 'http://www.tomatoesiipizzeria.com/', 'http://yesidobridals.com/', 'https://www.lumberliquidators.com/ll/stores/az/mesa/1845-south-power-rd?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1388', None, None, 'http://www.walmart.ca', 'http://www.papajohns.com', 'http://www.partycity.com/storedetail.do?id=842', 'http://locations.tccrocks.com/az/scottsdale/8980-e-indian-bend-rd.html', None, None, None, '1592158380', 'http://www.raxlenpharmacy.com', 'http://www.anjapparcanada.ca', None, 'http://www.dennys.com', 'https://www.ispynserve4u.com', 'http://mainstreetautoservices.com', 'http://www.dennyscarwash.com', None, 'http://www.bk.com', 'http://www.eatgenos.com', None, 'https://www.seeeyewear.com/locations/pittsburgh', None, 'https://www.piesbyinge.com', 'https://locations.nekterjuicebar.com/ll/US/AZ/Tempe/2000-E_-Rio-Salado-Pkwy_*-Ste_-1213?utm_source=yelp&utm_medium=link&utm_campaign=Tempe&utm_content=URL', 'http://kohinoortempe.com', 'http://idolizespa.com', 'http://www.naturaltress-salon.com', 'https://www.sunrisepreschools.com/locations/baseline-gilbert/?utm_source=yelp&utm_campaign=online_listing_gilbert_280&utm_medium=referral&utm_term=business_url', None, None, 'http://www.peridotsweets.com/', None, 'http://www.thefitmenswear.com', 'http://thegrillat11union.com', 'http://www.papajohns.com/', None, 'https://www.victoriassecret.com', 'http://www.lostdutchmanroasters.com', 'http://www.petplanethealth.com/stores/village-plaza/', None, None, 'http://www.upinvapor702.com', 'https://www.ronmarhoferchevy.com', None, None, 'http://www.sanealcamera.com', 'https://www.bankofamerica.com', 'http://www.thesinginglady.com', 'http://clickserve.dartsearch.net/link/click?lid=43700006435127049&ds_s_kwgid=58700000444950373&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.arizonaeyecenter.com', None, None, 'http://www.thespicetrader.ca/', None, None, 'http://www.harkinstheatres.com/', 'https://chefmayra.com', None, 'http://www.westmore.ca', None, 'http://www.LaRoccaKitchen.com', 'http://www.craquedecreme.com', 'https://heartandlungnv.com/physicians/profile/Arnold-Chung-MD', None, 'http://www.loveandkissespetsitting.net', None, None, 'http://www.tsgservicesaz.com', 'https://www.staples.com', None, None, None, 'http://carlosthecomputerguy.com/', 'http://info.tradesecrets.ca/locations-reviews/upper-canada-mall/', None, 'https://www.shapirolawaz.com', 'http://www.azpatiocover.us', 'http://bevvyuptown.com', 'http://Www.solasalonstudios.com/salon-professional/liz-mervar1', None, 'http://www.yangschicken.ca', None, 'http://www.dsw.com', None, None, 'http://birdhauscoffee.com', None, 'http://www.jysk.ca', None, 'http://www.puffnstuffstores.com', 'https://www.brakesplus.com', 'https://locations.noodles.com/pa/pittsburgh/476-mcmaster-way.html', 'http://www.michaeltoddsfurniture.com', None, 'http://www.dentistryintoronto.com/', 'https://jmichaelrealestate.com', None, 'https://splittingtimber.com/axe-throwing-phoenix/', 'http://www.crabhouselv.com', None, 'http://wisconsincutlery.com/', None, 'http://www.jerseygiant.ca', 'http://www.mazdaofrichmondhill.com', 'http://www.quiktrip.com', 'http://www.dimsumchineserestaurant.com', 'http://www.marcustheatres.com', None, 'http://kaiburcoffee.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.shishaliciouscafe.ca', None, 'http://charandamexicaneatery.com', 'http://www.rideprlimo.com/scottsdale-limousine/', 'http://www.distillata.com', 'http://www.sobeys.com/en/stores/sobeys-aurora-bayview/', 'http://www.acanac.com', 'https://www.ubreakifix.com/locations/goodyear', 'https://www.robainalaw.com', 'http://www.newdimensionsauto.com/', 'http://www.martinmarineaz.com/', None, None, 'http://www.rexall.ca/', 'http://www.baierlhonda.com', 'http://www.pizzalafelice.com', None, 'https://www.olivegarden.com/locations/wi/madison/madison-east-towne-mall/1300?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:madison-wi-1300_pl:locurl_rd:1227', 'https://www.chilis.com', 'http://www.npfy.org', 'http://www.bedbathandbeyond.com', 'http://www.andersons.pub', 'https://www.lowes.com/store/AZ-Scottsdale/1850', 'http://www.playfactoryparty.com', None, 'http://cityh2o.com', 'http://www.zooecomuseum.ca/', 'http://www.shamsflorist.ca/?srccode=yelp_track', 'https://massagetherapistscottsdale.com', 'http://www.wilshirepines.com', 'http://www.modernfurniturestudio.com', 'http://allontarioapplianceclinic.ca', 'http://www.bridalalterationspittsburgh.com/', 'https://perfumecollection.ca', None, 'https://www.homedepot.com/l/Ray-Road/AZ/Chandler/85226/476?cm_mmc=seo-yelp&', 'http://www.bornandraisedlv.com', None, 'https://www.statefarm.com/agent/us/nv/las-vegas/ladonna-koeller-p89cl1ys000?cmpid=jxep_blm_0006', 'https://www.jewelrymedinaohio.com', 'https://eastendpittsburgh.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'https://www.choicehotels.com/wisconsin/madison/cambria-hotels/wi143/rates', 'http://www.cafe-mirage.com', None, 'http://adagestudios.com', 'https://www.masteroftintllctempe.com', None, 'http://lsmchiro.com/clinics-office-hours/sunprairie.html', 'http://plumbing-repair-now.com', 'http://spookysswirls.com', 'http://www.sunserimove.com', None, 'http://www.nandos.ca/restaurants/bay-street', 'http://www.massageheights.com/south-pointe-plaza', 'http://www.a-zsources.ca', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', 'http://www.smilereef.com', 'http://simplycardio.com', None, 'http://kismetandclover.com/', 'http://www.assouline.com', 'http://www.pkcreek.ca/', 'http://www.internationalgalleriesurbana.com', 'http://www.regularsbar.ca', 'http://www.6DegreesAZ.com', 'http://www.oldnavy.com', None, 'http://www.dmcsystemsllc.com', 'http://www.jumpstartchildcare.com', None, 'http://www.villapizza.com', 'https://www.yourlistingexpert.com', 'http://stores.dsw.com/usa/nv/henderson/dsw-designer-shoe-warehouse-stephanie-street.html', 'https://www.usps.com/', None, 'https://eastvalleyrv.com', None, '1592158816', None, None, None, 'https://www.aircanada.com/ca/en/aco/home/fly/premium-services/maple-leaf-lounges/maple-leaf-lounge-details.html#/!lounge@toronto', 'http://chickswithspiritualgifts.com', None, 'http://www.costco.com/', '1592158831', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.tyddvegas.com', 'http://www.samfawazphotography.com', None, None, 'https://www.schendelpest.com', 'http://www.honeybeeac.com', None, 'https://www.gavishpropertymanagement.com', 'http://www.topnailsaz.com', 'http://www.katesissons.com', 'http://www.JimmyJohns.com', 'https://plexustechnology.com', 'http://www.thedirtydrummer.com', 'http://pac.unlv.edu/index.php', 'http://www.enterprisecarsales.com/location/5482/Enterprise_Car_Sales_Las_Vegas?mcid=iyp:277&CUR=yelp', 'http://www.happitimedaycare.com', None, '1592158875', 'http://www.wyldewoodillinois.com', None, None, 'http://www.tribridpersonaltraining.com', 'http://www.kw.com', '1592158885', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.azheart.com/', None, 'http://www.bellabridesmaids.com/', 'http://www.dollargeneral.com/', None, 'http://www.plazahotelcasino.com/entertainment/the_best_little_wh', 'http://www.yofreshyogurtcafe.com', 'http://mdskinlounge.com', None, '1592158911', 'https://www.itdrsolutions.com', 'https://www.papajohns.com', 'http://hiltongardeninn3.hilton.com/en/hotels/ontario/hilton-garden-inn-toronto-oakville-YTOTVGI/index.html?SEO_id=YELP-GI-YTOTVGI', None, 'https://www.emser.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', None, 'http://www.pitapitusa.com/', None, None, None, 'http://www.tlcdentalcarelv.com', None, 'http://www.chiefsautowash.com', 'https://www.gamesourceinc.net', None, None, None, 'http://www.bigotires.com/Locations/89032', 'http://www.mbenergetics.com', None, '1592158958', '1592158961', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.goldenchinarancho.com', 'http://www.starbucks.com/store/14175/', 'https://www.cellphonerepair.com/monroeville-pa/', 'http://vbarbershop.com/locations/kierland-scottsdale/', 'https://agents.allstate.com/padma-schaudt-las-vegas-nv.html', 'https://www.petsmart.ca/stores/ca/on/oakville-store2002.html?utm_campaign=us-lcl-srv-eg&utm_medium=listing&utm_source=yelp', '1592158979', 'http://www.designedbydyani.com', 'http://spadinaoptometry.ca/', 'https://www.wellschiro.com', 'https://www.justonemoretavern.com', 'https://www.sprouts.com/stores/details/-/store-details/view/store/7/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.framestorepittsburgh.com', None, 'http://totemappliance.com/', 'http://www.silvertoncasino.com', 'http://playnwisconsin.com', 'http://www.tokasticksgolf.com/index.php', 'https://local.biglots.com/nv/las-vegas/4500?zcp=pd_local_yelp', 'https://www.dwellarizona.com', 'http://achermanlaw.com/', 'http://www.odellconstructioninc.com', None, 'http://www.thebownesiangrocer.ca', 'http://outlandcigars.com', 'http://stlouiswings.com', 'http://www.brokeassphone.com/broke-ass-phone-locations/broke-ass-phone-rocky-river-ohio', 'https://www.valleywidecooling.com', None, 'http://www.amrracing.com', 'http://www.cmlibrary.org/locations/branches.asp?id=13', None, 'http://nakedpizza.biz', 'http://www.meltinyourmouthcupcakes.com', 'https://local.fedex.com/az/phoenix/office-5048/', 'https://locations.usbank.com/index/arizona/gilbert/east-baseline-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.dominos.com', 'http://www.treeworkersofphoenix.com', 'http://d-moes.com/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://dansdogsdiner.com', 'http://www.saucepizzaandwine.com/locations/scottsdale-scottsdale-waterfront/', None, 'http://Www.tomjames.com', 'http://www.healinginspirations-wi.com/', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', '1592159067', 'https://stores.advanceautoparts.com/pa/etna/500-butler-street?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'https://local.fedex.com/nc/charlotte/office-0746/', 'https://northvalleysurgerycenter.com', 'http://www.burritozone.ca', 'http://www.enterprise.com/car_rental/deeplinkmap.do?cnty=US&gpbr=53W9&bid=004&arc=YAPL100&cm_mmc=Yelp-_-Local-_-National-_-null', 'http://pddlv.com', 'https://hairbykee.wixsite.com/hair-by-kee', None, None, '1592159093', 'http://www.trianglebarswissvale.com', None, None, None, 'http://freshco.com/stores/bloor-dundas', 'http://www.bojangles.com', 'http://www.mark-taylor.com/arizona/san-sonoma/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.midhafurniture.com', None, None, 'https://www.lashandskinlab.com', None, None, None, None, 'http://www.serviceking.com/locations/service-king-bell-rd?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', None, None, 'http://alatisinspectionservice.com/lp/home-inspection', 'https://www.moes.com/find-a-moes/pennsylvania/bridgeville/938?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', None, 'http://www.wagg-n-walk.com', 'http://www.clevelandcabby.com', 'https://www.chaoshomeloans.com', 'http://www.marondahomes.com', 'http://www.morocochocolat.com/', 'http://anytimefitness.com/North-Royalton', 'http://qualitysystemsac.com', 'https://local.safeway.com/safeway/az/sun-city-west/13503-camino-del-sol.html', 'http://www.buckinghamsports.ca/view/buckinghamsports/the-penalty-box', 'http://geckohomeservices.com', 'http://www.republicwesthome.com', 'http://www.pawlorpets.com/', 'https://www.sevabeauty.com/location/az/avondale-w-rancho-santa-fe-blvd/', 'http://www.tjmaxx.com', None, None, 'http://www.barleyhousecleveland.com/', 'http://www.landmarkcinemas.com/bolton/contact-us', '1592159181', 'http://www.irenevoo.com', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', None, None, None, 'http://www.cumbraes.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, 'http://www.curtisnewsomedds.com', 'http://walmart.com/', 'https://www.scottsdalemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://www.casinomontelago.com', None, None, None, 'http://www.jaguarnorthscottsdale.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Mesa-AZ-85210/723081/', None, 'http://www.cityofmadison.com/parks/parks/park.cfm?id=1368', None, 'https://www.viomedspa.com', 'http://www.paradisetileandnaturalstone.net', None, 'http://www.kinglawofficespc.com', 'http://www.innovateframe.com', None, 'http://www.happyhourarizona.com/champions-sports-saloon', 'http://www.millcrestpark.com', 'http://www.atlanticbay.com', 'http://www.fiddlersdream.org', 'http://www.nationaleventstaffing.com', 'http://holisticfit.life', 'http://www.atwistedtulip.net', 'https://www.carmax.com/stores/7284?adcode=YLP7284&utm_campaign=LG_AFF&utm_source=LG_AFF_YLP&utm_medium=local&utm_content=shop', None, None, None, 'https://stewardship.pro', 'http://www.golfsmith.com', 'http://www.duskscottsdale.com', None, 'http://www.metrosouthbridge.com', 'http://www.pizzahut.com/', 'http://www.80OnGibsonapts.com?campaign_code=yelp_80_On_Gibson_NV_89012', 'http://reliableappliance.ca', 'https://www.legendsofchampaign.com', 'http://www.timhortons.com/ca/en/locations/get-directions.php?id=101357', None, None, 'http://www.spotlesssquadlv.com', None, 'https://asquaredmathtutoring.com', 'http://www.autoxpertslv.net', None, 'http://www.shanklelaw.com', None, 'http://www.cajuncrawfishLV.com', 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=1165', 'https://ohiosbestmassage.com', 'https://handandstone.com/locations/az-glendale', '1592159307', '1592159309', 'http://inesdisanto.com/', 'https://www.mandalaybay.com/en/nightlife/eyecandy-sound-lounge.html', None, 'http://affordablemoverslv.wixsite.com/website', None, 'https://pizzapit.biz', 'http://www.piedmontexpresscaresuttonroad.com/', 'http://www.distinctiveweddingvideos.com', None, 'https://affderm.com/locations/tempe-office/', 'http://vegasproperty.management', 'http://www.gandmstaffingsolutions.org/', 'http://www.bellagio.com/attractions/gallery-of-fine-art.aspx', 'http://www.structube.com', None, 'http://www.autocash.com/auto-cash-in-las-vegas?utm_campaign=places&sf_traffic=paid_listing_yelp&utm_medium=listing&tmx_id=d2c199521a9a9f&utm_source=yelp', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.theavenuebanquethall.com', None, None, None, 'http://trutea.ca', 'http://www.rbomotorsports.com', None, '1592159362', 'http://thebagelshoppe.weebly.com/', 'http://www.freewayinsurance.com', 'http://www.persianbijar.com', 'https://www.phoenixmusiclessons.com', 'http://www.discoverydetectivegroup.com', 'http://www.fortinos.ca/en_CA/storedetail.484.html', 'http://www.motel6.com/reservations/motel_detail.aspx?num=4742', 'http://www.its2di4.com', None, 'http://pawminderspluspetsitters.com', 'http://www.fiatusaofstrongsville.com', 'http://www.skinsopretty.com', 'https://www.ontario.ca/page/serviceontario', 'http://www.northphoenixcampground.com', 'http://www.packbrotherscollisioncenter.com/', 'http://www.gusnewyorkpizza.com/', None, 'http://www.canadianpardons.ca', 'http://www.computerezearizona.com', None, 'http://www.vashticosmetic.com', None, 'http://fitforlifefood.com/', 'https://www.meijer.com/shop/en/store/307/?cmpid=OLA:040319:YelpGrocery', None, 'http://www.suepembertoneducation.com', 'http://www.disktopia.com', 'http://www.wjbonline.com', 'http://www.atriasrestaurantpncpittsburgh.com', 'http://www.discotech.me', 'http://www.myturnlv.com', None, 'http://www.catchacritter.us', None, None, 'http://www.ourfamilyeyecare.com', 'http://www.goodwillaz.org/', 'http://www.BroadstoneParadiseValley.com/?utm_medium=listing&utm_source=yelp', 'http://uptownnailsalon.wix.com/uptown-nail-salon', 'https://www.sticyyc.com/locations-1/', 'http://www.harveyswineburger.com', 'http://esoslifestyle.com', 'http://www.smithenvironmentalsolutions.com', 'https://beyondextensions.mayvenn.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.cardinalewaymazdapeoria.com/?ibp-adgroup=yx_target', None, '1592159462', 'https://local.albertsons.com/az/peoria/8240-w-deer-valley-rd.html', None, 'http://www.plastikwrap.com', 'https://www.petsuppliesplus.com/Store/OH/Lakewood/Lakewood/128', 'http://www.thunderbirddental.com', 'http://poolservlv.com', 'https://www.heinens.com/Heinens-of-Hudson', 'http://www.jackscornertaplkn.com/', None, None, 'https://www.wyndhamhotels.com/hotel/03725?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:03725', 'https://innovativecleaningsvs.com', 'https://www.starbucks.ca/store-locator/store/1017580/bayview-village-2901-bayview-avenue-toronto-on-m-2-k-1-e-9-ca', 'http://www.statusinkllc.com', 'http://bellagiopizza.ca', 'http://www.puptowncharlotte.com', None, 'http://www.starbucks.com/store/10311/', 'http://avis.com', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', 'http://aftbar.com/', 'https://local.skechers.com/az/phoenix/', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.autozone.com/locations/az/peoria/8669-nw-grand-ave.html', 'https://smartmoveinsurance.com', None, None, 'http://www.the-idea-store.org', 'http://www.dunkindonuts.com', 'https://www.i9sports.com/ProgramDirector/309/', 'http://www.rainrestaurant.ca', 'http://www.davidandgoliathtees.com/', 'http://www.starrdickensortho.com/', 'http://mickeles.ca/', None, 'http://www.westsidelaser.ca', 'http://www.lcbo.com/lcbo-ear/jsp/storeinfo.jsp?STORE=629&language=EN', 'https://gilbert.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', None, 'http://www.pitapit.com', 'http://www.cottonwoodstationeatery.com', 'http://superdiscounttrans.com/', '1592159557', 'http://www.bowlingwithleverage.com', 'http://www.saloncentric.com/', 'https://www.carolinashealthcare.org/locations/detail/carolinas-healthcare-urgent-care-union-west', 'http://www.autosonic.ca', None, 'https://www.heirloomtoronto.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66065.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66065', 'http://philipsplastics.com/', 'http://www.bookmans.com', 'http://www.wordofmouthdentistry.ca', 'http://www.starbucks.com', None, 'http://www.yofreshyogurtcafe.com', 'http://www.breadheadslv.com', 'http://www.luxor.com/attractions/attractions_king_tut_museum.aspx', 'http://www.crazymocha.com', 'http://www.vuaurology.com', 'http://www.costco.com/', 'http://www.mydcdental.com', 'http://www.ritzcarlton.com/en/hotels/charlotte/dining/punch-room', 'http://phoenicianestates.com/', 'http://www.bountifulfruitarrangements.com', '1592159602', None, None, None, None, None, 'http://www.azpowerwashpros.com', 'http://www.fishcreekvets.com/', 'https://haircutmensoutheuclidoh.com', 'http://Www.mixnmatchausa.com', None, 'http://consolidatedauto.com', None, 'http://www.cookinsgroup.com', 'https://stores.bestbuy.com/il/champaign/2117-n-prospect-ave-46.html/?loc=ns100&ref=NS', 'http://www.nextcare.com/locations/az/peoria-glendale/?utm_source=Yelp&utm_medium=Local&utm_campaign=peoria_glendale', 'http://darkhorsesportsbar.com', None, None, 'http://www.catseyevintage.ca', 'http://www.searsgaragedoors.com', '1592159643', 'http://www.westcoastkids.ca', 'http://heavenlyhairbodysalon.com', None, None, None, '1592159653', 'http://www.cyclonespitas.com', None, None, None, 'http://www.valledelsol.com', None, 'http://www.villageinn.com/', None, None, None, 'http://www.tascsolutions.org/locations/phoenix-central', 'http://hooverdamstore.com', None, 'http://www.elitepodiatry.net', None, '1592159685', 'http://www.cleverchefscatering.com/', 'http://www.successjustclicks.com', None, 'http://www.theterracegardens.com', 'http://www.nortownfoods.com', '1592159695', None, 'http://www.chinadragonbedford.com', 'http://www.choiceautorepairaz.net', 'http://gpsandtrack.com', 'http://www.pancheros.com/locations/tempe', 'http://www.oldvegastavern.com', 'https://store.vioc.com/wi/madison/3594-e-washington-ave-53704-GR0030', '1592159710', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.youfit.com/gym/scottsdale-scottsdale-rd-7346?utm_source=local&utm_medium=yelp&utm_campaign=club7346&utm_term=website', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.budget.com/', 'https://www.glamparlor.com/?utm_source=Yelp&utm_campaign=Main_Web', None, 'http://www.balanicustom.com/custom-suits-cleveland/', None, 'http://www.cox.com', 'http://www.westendcomics.ca', 'http://www.greenhousemontessorischool.com', 'http://carpetwarehousecleveland.com', None, None, 'http://www.ontrac.com', None, 'http://www.jacobsonbrosdeli.com', 'http://www.altrahomedecor.com', None, None, None, None, 'https://p3hp.org/nevada/p3-medical-group/our-locations/business-park/', 'http://lvmpd.com/', None, None, 'http://www.fastforwardskate.com', 'http://www.midoco.ca', None, None, 'https://www.starbucks.ca', None, None, 'http://www.dunkindonuts.com', 'http://www.findyourcenter.com/', 'http://www.neocoffeebar.com', 'http://www.locksmithbetty.com', None, None, '1592159796', 'https://www.walgreens.com/locator/walgreens-4667+william+penn+hwy-murrysville-pa-15668/id=11768', 'http://www.thespa-sunprairie.com', 'http://www.rushhourlegal.com', 'https://locations.jackinthebox.com/us/az/gilbert/4361-e-baseline-rd', None, 'http://www.celsiussushibar.com', '1592159814', None, None, None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', None, None, 'http://www.SonicAutoHouse.com', None, 'http://www.bayviewmooreauto.com/', 'http://www.jessicabashaw.com', 'http://www.thewoodenvine.com', None, 'https://merkatoethiopianrestaurantlasvegas.com', 'http://www.whitevacuum.com', None, None, None, None, 'http://www.beattheheatpoolservice.com', 'http://www.carmellaspizzagrill.net', 'http://www.tphc.ca', 'http://www.tomscountryplace.com', 'http://www.balaton-restaurant.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.autoexinc.com/', 'https://creativenailsandspascottsdale.com', None, 'http://www.canadiantire.ca/en/store-locator/ab/calgary-south-ab.store.html', 'http://rusticsocial.ca', 'http://wolfwatersports.com', 'https://www.extremearizona.com', 'http://www.theBooKnack.com', None, 'https://elementsmassage.com/fitchburg', None, 'https://www.copart.com/locationDetail/47', 'http://tigotrattoria.ca', None, 'http://www.nevadadesertmermaids.com/', 'http://www.caesarsuniforms.com', 'http://www.wingsntings.ca/', 'http://www.boulderzclimbing.com', 'http://www.SonoranVein.com', 'http://www.goodlifefitness.com', 'http://nievemalandra.com', None, 'http://www.scottsdalespice.org', None, 'http://l.macys.com/madison-wi?cm_mmc=Yelp_Stores-_-Madison-_-n-_-228&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Madison_228&m_ac=yelp_stores&m_ag=n', 'http://www.superiortire.ca/locations/mode/7/208-king-rd-oak-ridges-on-l4e2w1.aspx', 'http://thearizonafishingguides.com/index.php', 'http://www.chapmancollision.com/', 'http://www.quiktrip.com/', None, None, None, 'http://www.littlecaesars.com', 'http://www.onereddoorphotography.com', 'http://www.circuscircus.com/casino/slots_a_fun.aspx', None, 'http://www.milgard.com', 'http://benetflorentine.com', 'http://www.sbistroaz.com', None, 'http://locations.avis.com/us/az/chandler/c5a.html', 'http://www.rockcitydance.com', None, 'http://www.washwerksexpress.com', 'http://AutoFinanceNC.com', 'http://www.mrhandyman.ca/calgary-south', None, 'https://arizona.weidner.com/apartments/az/mesa/pala-mesa-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', None, 'http://www.lesterklebedpm.com', 'http://www.bk.com', 'https://www.bathandbodyworks.com', None, 'http://www.brakemasters.com', 'http://clearfloat.ca', None, 'http://www.watchdogpestcontrol.com', None, 'http://www.budgetbridal.vpweb.com', 'http://advancedheartcare.com/', 'https://moxies.com/?utm_source=yelp&utm_campaign=yelp_enhanced_profile&utm_content=landing-page', 'http://www.ihlcanada.com', 'http://www.parmalaserwash.com', None, 'http://www.anatoliaturkishfood.ca', None, None, 'http://calgarylibrary.ca/locations/shaw', 'https://www.payless.com/stores-location/NV-LAS-VEGAS-5798.html', 'https://peoria.uptownjungle.com', None, 'http://www.rimrepairrx.com', None, 'http://www.apria.com', None, 'http://www.santinocatering.com', 'http://www.fusionsincsalonanddayspa.com', 'http://www.qualityonepool.com', None, 'http://zenarizona.com/', None, 'http://empireplumbingac.com/', 'http://www.contourmedical.com', 'https://website-18496976604484669992-seafoodrestaurant.business.site', 'http://www.swimmingpoolrepairphoenix.com', 'https://locations.visionworks.com/ll/US/OH/Beachwood/26300-Cedar-Rd', 'http://www.barrebodystudio.com', 'http://www.wwsdental.com', 'http://robertostacoshop.com', None, None, 'http://pacificeastcoventry.com/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', 'http://www.labonitadulceria.com', 'http://www.retinalconsultantsaz.com', 'http://www.irideworldwide.com', 'https://titleboxingclub.com', 'http://www.serviceking.com/locations/service-king-north-las-vegas?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'https://www.elegance-living.com/senior-living/nv/las-vegas/wentworth-las-vegas', 'http://www.cafekrishnaaz.com', 'http://mountaincreektrees.com', None, 'http://www.batw.ca/', None, '1592160137', None, 'https://locations.vitaminshoppe.com/nv/lasvegas/vitamins-supplements-las-vegas-nv-394.html', None, None, 'http://www.secondsoleohio.com/#!akron/c1fn6', None, None, None, None, 'http://www.purpleorchidflowers.ca/', None, 'http://www.bluepandalv.com', 'http://calgary.treehouseplay.com/', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'http://www.ncdot.org/dmv/', 'http://www.2meatballz.com/', 'https://www.fit4less.ca/locations/ontario/north-york/409?utm_source=yelp&utm_medium=cta&utm_term=torontolawrence-f4l', 'http://www.mrhero.com/', 'http://www.truevalue.com/', 'http://www.pombalensebakery.com/', None, 'http://www.peoriacpr.com/', 'http://www.terribleherbst.com/', None, 'https://www.smartstyle.com/en-us/locations/on/brampton/located-inside-walmart-3135-haircuts-5184.html', 'http://www.massagebymichael.vegas', 'https://www.myeyedr.com/locations/ballantyne?utm_source=Yelp&utm_medium=Referral&utm_campaign=ballantyne', None, 'http://www.reebokcrossfitramsay.com', 'http://blackowlrealtylv.com', 'http://www.speducci.com/', None, 'https://deadcityvegas.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://bostonpizza.com/centropolis-laval', 'https://satruck.org', 'http://www.richbeckguitars.com/', None, 'http://www.inspiringsmilesdentistry.com', None, 'http://www.morcillapittsburgh.com', 'http://www.kids-supergym.com', None, 'http://www.deserthottubs.com', 'http://www.dimaggiostire.com/', 'https://local.fedex.com/nv/las-vegas/office-1441/', None, None, None, 'http://budstreecare.com', None, 'http://www.saintsandsinnershairdesign.com', 'http://www.woodiesautoservice.com', None, 'http://www.gap.com?tid=gpme000290', None, 'http://rawfitnesslv.com', 'http://www.starbucks.com', 'https://cocosgelatooh.com', 'http://www.sissybeautysalon.com', 'http://m.primarycarewalkinmedicalclinic.com', None, None, 'https://www.sprint.com/storefronts/bd/sprint-matthews-nc-28105-fcs-2516/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.xiangcunfaxian.com/', 'http://Www.LVLegalHelp.com', 'https://scottsdale.andaz.hyatt.com/en/hotel/dining/weft-warp-art-bar-kitchen.html', 'http://www.salzmann.com', None, 'http://glossitdetailstudio.com/', 'http://www.trcaparks.ca', 'http://www.eastofchicago.com', None, 'https://chocolateshops.sees.com/pa/pittsburgh/chocolate_shops_pittsburgh_pa_pa-17.html', 'http://houseoffitness.net/', None, 'http://www.circletree.net', 'https://www.atmosphere.ca/stores/calgary.html', 'http://www.burgers2beer.com/', 'http://www.shipleyortho.com/', None, None, 'http://www.mamathaikitchenaz.com', 'http://www.junctionatantiquity.com', 'http://www.vendettatowing.com', None, None, 'http://www.centennialwellnesscenter.com', 'http://thebiomedcenter.com/', 'http://www.LandisReedHomes.com', 'https://www.thegibsoncharlotte.com', 'http://www.vientianepalacerestaurant.com', 'https://www.allauramedical.com', None, 'http://www.benefitcosmetics.com/?ext_id=yelp_ulta_010713_Yelp', 'http://www.cnroofing.com', None, 'http://www.sabas.com', 'http://www.greatclips.com/', 'http://www.mammothairlv.com', 'http://www.daleshirleys.com', None, 'http://www.baldwinbrothersautomotive.com', 'http://www.gyroworldohio.com/', None, None, None, 'http://www.bistro-jules.com', None, 'http://onsitebuilds.com/', 'http://www.livewellarborsteelecreek.com', 'http://www.augiessportsgrill.com/', None, None, 'https://www.gowireless.com/stores/az/gilbert/gilbert-gilbert-road/', None, None, 'http://www.arrabiatas.com', None, 'http://www.heroburgers.com', 'http://www.amesachiropractor.com', 'https://onginstitute.com', 'https://www.woodlandestatesapartment.com', 'http://www.bamboobakery.com', 'http://www.olsonglassandmirror.com', 'http://www.carolinafoxs.com', 'http://www.kingwongchinese.com', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.theminderbinder.com', 'http://www.wpinsure.com', 'http://www.sagastasalon.com', 'http://velveteenrabbitlv.com', None, None, 'http://pesterminate.ca', 'http://downtown.muramoto.biz', None, 'http://nevadacheapcars.com/', 'https://www.laserquest.com/nv-lasvegas', None, None, None, None, 'http://smogbusters.com', None, 'http://www.theriverrockgrille.com', 'http://www.myresortspa.com', None, 'http://nalysfloralshop.com', 'http://www.plantitsucculent.com', None, 'http://www.greatclips.com/salons/1552', 'https://www.suburbanpropane.com/locations/phoenix-az/', 'http://www.windycityair.net', 'http://www.lavidamassageshadysidepa.com', None, None, '1592160519', 'http://www.upandawaylandscaping.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, None, 'http://www.sincerelyyogurt.com/locations/peters-township', 'http://www.classickids.biz', 'http://mastercrafttires.com/', 'http://glenwoodacresrvpark.com', 'http://forecloseduponpets.org', 'https://www.mediweightloss.com/locations/scottsdale?_ppc=Z8FWKRXKT5', 'http://galaxiedinerwoodbridge.com', 'http://yeoldsoapmill.com/touch', None, None, 'http://www.1866staydry.com', None, 'http://www.sarphx.com', None, None, None, 'https://homesbyclaudiamarion.kw.com', 'http://cloud9windows.com', 'http://www.bevmo.com', None, None, None, 'http://www.tailgatersilprimo.com', 'http://www.rainbow.travel', 'http://www.timhortons.com/', 'http://www.noblebeastpets.com', None, 'http://compadresmexicanrestaurantwi.business.site', '1592160596', '1592160598', 'http://www.sunnymorning.ca', None, 'https://www.westvalleytech.net', 'http://www.lasvegasbraces.com', 'http://www.cicispizza.com', None, None, 'https://stores.dickssportinggoods.com/az/phoenix/365/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281360123&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.gamestop.com', None, 'http://www.apple.com/retail/scottsdalequarter/', '1592160628', 'http://www.mark-taylor.com/arizona/san-marquis/?utm_content=visit-website&utm_medium=listing&utm_source=yelp', '1592160634', 'http://www.quiktrip.com/', 'http://www.yellowcheckercabchampaign.com/', 'https://www.lowes.com/store/PA-Monroeville/1660', 'https://megafurnitureusa.com', 'http://greenbeautyspa.com/', 'https://www.wyndhamhotels.com/hotel/52964?tel=18666154223&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52964', 'http://www.itsnewtomeconsignments.com', 'http://www.valueworld.net/', 'http://www.lunarossaristorante.com/', None, 'https://www.bannerhealth.com/locations/mesa/banner-urgent-care-1', None, 'http://www.planetfitness.com/gyms/willowick-oh-985?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://jakesautorepair.com/', None, None, 'http://www.flippnburgers.ca', 'https://camp.nc', 'http://www.lenotrebistro.com', None, 'http://www.yourfloorz.com', None, '1592160693', 'http://www.paradisejuice.com', None, 'http://www.nailsforyou.ca', 'http://www.vintagebymisty.com', None, 'http://www.chezcora.com', 'http://www.thehealthygreek.ca/', 'http://www.upmc.com/locations/hospitals/western-psychiatric/Pages/default.aspx', None, 'https://www.scottsdalecc.com', 'https://www.anytimefitness.com/gyms/92/Phoenix-AZ-85048/?utm_source=yelp&utm_medium=local', 'http://www.tccarpetcare.com/', 'http://www.phohongnc.com', 'https://us.louisvuitton.com/eng-us/point-of-sale/usa/louis-vuitton-cleveland-saks?utm_source=yelp&utm_medium=organic&utm_campaign=store_listing_storepage', '1592160727', None, 'http://www.nice1nails.com', None, 'http://www.travelsuitesinn.com', None, 'http://priscillaspantrypetsupplies.business.site', None, None, 'http://www.avidalaserandspa.com/', None, None, 'http://www.city-tavern.com', 'http://www.lockquest.com', None, 'https://www.plannedparenthood.org/health-center/arizona/scottsdale/85251/scottsdale-health-center-2809-90030?utm_campaign=scottsdale-health-center&utm_medium=referral&utm_source=yelp.com&utm_content=link', None, None, None, None, None, 'http://tirerepairlasvegas.com', 'http://www.tangdynastyurbana.com', None, None, 'http://www.thegridcl.com', None, 'http://www.janasredroom.com', 'http://atrackout.com', None, None, 'http://www.mancavecigarlounge.com', 'http://quicklaneoflasvegas.com/', 'https://locations.theupsstore.com/nv/north-las-vegas/5575-simmons-st?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://consulmex.sre.gob.mx/lasvegas/', 'http://www.azstoragedoctor.com', 'http://www.greenstreetcommunitiesinc.com', 'http://www.pcpropane.com', None, None, "http://charmeck.org/mecklenburg/county/ParkandRec/Greenways/OpenGreenways/Pages/MallardCreekandClark'sCreek.aspx", 'http://www.paradisevalley.edu/', 'http://www.curiouskidspreschoolaz.com/', 'http://www.gardenialanephotography.com', 'http://www.kandrpainting.com', 'http://bestestiedayspa.com', None, None, None, 'https://locations.panerabread.com/oh/hudson/118-w-streetsboro-street.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', 'https://www.usps.com/', 'http://www.apartmentsnow.online', 'https://www.5280mexicangrub.com', 'http://Destinations.rei.com/Arizona', 'http://www.ferrarimaseratioflasvegas.com', None, 'http://www.justrightlandscape.com', 'https://arizonapain.com/contact-us/chandler/', 'http://www.degnanlawaz.com', '1592160875', None, 'http://azvideomarketing.com', None, 'https://www.marks.com/en/stores/north-york-2625d-weston-road-14-15-crossroads-centre.html', 'http://toterafinefoods.ca', 'http://www.moonhaven.com/', None, 'http://www.torontoguesthouses.com', 'http://massagemaniaglendale.com', None, None, None, None, 'http://www.plannedparenthood.org/health-center/ohio/rocky-river/44116/rocky-river-health-center-2418-91230', None, 'http://www.totaltransit.com', 'https://www.marriott.com/hotels/travel/phxsh-springhill-suites-scottsdale-north/', 'http://www.skatenrd.com', 'https://blueboxmovingcompany.com', 'http://lamsaz.com', 'http://www.designsbylanae.com', None, 'http://arborstownhomes.com', 'http://www.europebound.com/', 'http://www.acehardware.com', 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', 'https://azvent.com/locations/mesa.php', 'http://usacollision.com', None, 'http://www.falloutgames.net', 'http://www.shaanxibiangbiang.com', None, 'http://www.brettbaughman.com', 'http://swgclv.com', 'http://www.kromerradio.com', 'http://www.acmteam.com/acm/outside_home.asp', 'http://www.orvis.com/madison', 'https://www.retreataptsmcalpinecreek.com', None, 'http://www.elvischapel.com', 'http://www.ubreakifix.com/locations/peoria/', None, 'http://www.superchargefoods.com', 'http://www.thesakeexpress.com', None, 'http://www.topnoodle.ca', 'http://www.jazzberryteahouse.com', 'http://www.berkshireplaceapartments.com/', 'http://www.horizonhonorselementary.org', 'https://www.enterprise.com/en/car-rental/locations/us/pa/irwin-4021.html?mcid=iyp:8368400', 'http://www.4colorprint.com', 'https://deserttropicspools.com', 'http://martyscycle.com/', 'https://www.ttc.ca/Subway/Stations/Bay/station.jsp', 'http://www.sunsetgrill.ca', 'http://www.sothlv.com', 'http://www.palms.com/?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.ekgproperties.com', None, None, 'https://www.loandepot.com/branches/las-vegas-nv?sc_camp=DE925F8ECA0F4E0D8092C7E108A615A3', None, 'http://www.newmexicangrill.net', '1592161032', None, None, 'http://www.lenasflowersandgifts.com', 'http://www.cell2fix.com/queen-410-brampton/', None, 'http://www.tonymorenosalon.com', None, None, None, None, 'http://www.callister-law.com', 'http://www.smokeybones.com/', 'http://franklinPhotographyAz.com', '1592161065', 'http://www.thelastwordcharlotte.com/coffee-press', 'http://totalofficenc.com/', None, None, 'https://www.kaltire.com/en/stores/locations/?source=yelp_track&StoreID=864', 'http://www.corinthiapeoples.com', 'http://www.madurbanbees.com/', None, None, '1592161087', 'http://www.petopia.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.BarrowNeuro.org', '1592161101', 'http://www.southpittsburghvet.com', 'http://www.safeguardministorage.biz', 'http://www.lasvegasnvpoolservice.com', None, 'http://www.shopthepig.com', 'https://www.fatburger.com/locations/rancho', 'http://www.preferredparking.com', 'http://charkha.us/', 'http://www.hardboiledinc.com/', 'http://fasttowingaz.com', 'http://www.mephisto.com/us', None, 'https://locator.chase.com/oh/akron/50-s-main-st', 'http://www.lous.ca/', 'http://www.stratospherehotel.com/Food-Drink/Dining/Carvery-108', 'http://onestopnutrition.com', 'https://www.mimiscafe.com/locations/sunset-station/', 'http://www.nice1nails.com', None, '1592161151', None, 'http://diamondcarwash.ca', 'http://delucasplaceinthepark.com', None, 'https://www.torontobuffing.com', 'http://collinscleaners.com', None, 'http://xquiziteaffairz.com', None, 'https://locations.checkers.com/nv/las-vegas/4175-e-desert-inn-rd/?utm_source=Yelp', 'https://www.tedwiens.com', 'http://visionsource-southhills.com', 'http://www.timhortons.com/', 'http://www.arizonaeyedoc.com', None, 'http://www.mrhandyman.com/western-dane-county?sppccampaignid=67352&SPPC=Offline&L=true', 'https://www.whitbylibrary.ca', 'http://www.dangfinerentals.com', None, 'https://pizzariviera.com', 'http://eyetique.com', None, None, 'http://www.haulnassproductions.com/contactus.html', None, 'http://www.kinginklasvegas.com', 'http://www.hartwooddentists.com', 'https://www.ahwfac.com/', 'http://www.southparkoralsurgery.com/', 'http://store-locator.barnesandnoble.com/store/2039', 'http://www.imaginationchildcare.com', 'http://www.palazzo.com/entertainment/shows/baz.html', 'http://www.culturefreshlife.com', None, 'http://lvshotokan.com/', 'http://woolfeye.com', None, 'http://www.cloudadagents.com', 'http://www.viautorepair.info/', None, None, 'http://www.littlecaesars.com', 'http://terros.info', None, 'https://ad.doubleclick.net/ddm/clk/434334113;236814625;y', None, None, 'https://campbellssweets.com/locations/campbells-sweets-factory-ohio-city/', None, 'http://charmeck.org/mecklenburg/county/ParkandRec/Greenways/PlannedGreenways/Pages/McDowellCreekGreenway.aspx', None, None, 'http://www.italianvillagepizza.com/store-locator.html', 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', 'http://www.coldstonecreamery.com', None, 'http://www.smileworkscalgary.com', None, 'http://www.bannerhealth.com/locations/phoenix/banner-physical-therapy-mcdowell?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', None, 'https://rosyeyebrowthreading.business.site', 'http://www.sushiichibanchampaign.com', 'http://www.suenosimports.com', 'https://www.wellsfargo.com', '1592161298', None, 'https://www.davey.com/eastpittsburgh?source=Yelp', 'https://www.aria.com/en/restaurants/aria-patisserie.html', 'http://www.dpdough.com', 'http://www.us.allsaints.com/', 'https://www.payless.com/stores-location/OH-PARMA-6259.html', None, None, None, 'https://www.chick-fil-a.com/scottsdale101', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://thenakedbbq.com', 'http://www.ensurco.ca', 'https://www.mattressfirm.com/stores/az/scottsdale/mattress-stores-scottsdale-az-85266-6664.html', 'http://www.tenthousandvillages.com', 'http://chatime.com', 'http://www.childcareproviderlasvegas.com', 'http://www.meckabc.com', 'http://storelocator.pepboys.com/pepboys/stores/pittsburgh/library_road_service_center/01425', 'http://www.obskincare.com', None, None, None, 'http://www.valleyacademy.com', 'https://www.freshlandflowers.ca/?srccode=yelp_track', 'https://www.saladmeister.com', 'http://yogalovepgh.com', 'http://www.countyofdane.com/lwrd/parks/lake_farm.aspx', 'https://silvertoncasino.com/dine/dining-in-las-vegas-on-a-budget/', 'http://lasilmosoptical.com', 'https://www.solasalonstudios.com/salon-professional/lainie-schmidt2', 'http://winghartburgers.com/', 'http://www.bobzip.com', 'http://absoluteautoservicelv.com', 'http://tacoskissiphoenix.lunchroomconstructor.website', 'http://www.adelaideclub.com', 'http://www.drrudisill.com', None, None, None, 'http://www.maxandermas.com/locations/333038', 'http://mariannasalterations.com', 'http://www.bossplow.com', 'http://www.liquordepot.ca/?utm_source=Yelp&utm_medium=Online_Listings&utm_content=Location&utm_campaign=LiquorDepot_AB_Q3_July_September_2017', 'http://www.assuranceexteriors.com', None, 'http://www.shell.com', None, 'http://www.table51.com', 'http://www.recreation.gov/recAreaDetails.do?contractCode=NRSO&recAreaId=7&contractCode=131', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.wildpcs.com', 'https://ad.doubleclick.net/ddm/clk/301028788;127996861;v#/Live%20Weekend%20Entertainment', None, 'http://www.reebokcrossfitlibertyvillage.com/', 'http://www.pittsburgh-serenity.com', 'http://www.torontopubliclibrary.ca/detail.jsp?Entt=RDMLIB014&R=LIB014', 'http://dogboneranch.com', 'http://www.cleanjuice.com', 'http://dilworthcoffee.com', 'http://www.rompdogs.com', 'https://www.haircp.com', '1592161456', 'https://www.surlatable.com/browse/storeLocator/storeHome.jsp?storeId=140', '1592161460', 'http://filigreebeds.com', 'http://www.colonyamerican.com/?utm_source=Yelp!%20Regional&utm_medium=Text%20Link&utm_campaign=Yelp!%20Charlotte', None, 'http://www.juicebaratlvac.com', 'https://canyoncreekrestaurant.ca/vaughan/', None, None, None, None, None, 'http://www.acehardware.com/store-details/09471', 'http://www.timhortons.com/', 'http://www.barrospizza.com', '1592161494', 'http://smokeworldtempe.com', 'http://www.bestdoggonebookkeeping.com', 'http://www.banfield.com/veterinarians/az/phoenix/pho?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://alphaomegafg.com/', 'https://www.enterprise.ca/en/car-rental/locations/canada/on/north-toronto-eglington-yonge-c245.html?mcid=iyp:8368400', 'https://www.aqua-tots.com/surprise', 'http://www.kortmaninc.com', None, 'http://browsthreadingsalons.com', 'http://www.strideawayequestriancenter.com', 'http://www.mattressfirm.com/', None, None, None, 'http://www.saigonphocuisine.com', 'http://www.salvationoutdoor.com', 'http://us.christianlouboutin.com/us_en/', None, 'http://www.pizzaiolo.ca/?utm_source=Yelp', 'http://edhardyshop.com', 'http://www.parkshairhaven.com', None, 'http://www.hautechixvegas.com', 'http://integrity.helpusell.com', 'https://www.avis.com/en/locations/us/nv/las-vegas/qn4', 'http://www.matabar.ca', 'http://www.vivaldisalonsuites.com', 'http://ritzfurnitureplanet.ca/', 'http://movingteamsix.com', None, None, 'http://www.ecocleaners.ca/', 'https://locations.wendys.com/united-states/oh/parma/6970-ridge-road', 'https://www.discounttire.com/store/az/mesa/s/1294?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://biltmorenailbar.com', '1592161583', None, None, 'http://rumerzpgh.com/', 'http://www.suite114.ca', 'http://www.myspace.com/giant45', None, None, 'https://www.homedepot.com/l/Peoria/AZ/Peoria/85345/453?cm_mmc=seo-yelp&', None, None, '1592161607', None, 'https://www.k9viplv.com', 'https://www.hockeyworld.com/index/page/store_details/store_id/10', 'http://www.peninsulaluggageonline.com', 'https://www.phoeniciangaragedoorandrepair.com', 'http://www.LuminosityWellnessCenter.com', None, 'http://www.easternpodiatry.com', 'http://www.peppersfreshmarket.com', 'http://www.tremontscoops.com', 'https://local.albertsons.com/az/mesa/2727-n-power-rd.html', None, 'http://moviescoop.com', 'http://waikikigoldsilver.com', 'http://applefactory.com', 'http://www.brustersicecream.com', 'http://bigjohnspizza.ca', None, None, '1592161649', None, 'http://www.bigboy.com/', 'https://www.bluemercury.com/?utm_source=2016-Yelp&utm_medium=social&utm_content=49&utm_campaign=general', '1592161660', 'http://www.heritageatdeervalley.com', 'http://www.cellaxys.com', 'https://ireh-salon.business.site', 'http://noblecleaners.vegas', 'http://www.tuesdaymorning.com', 'http://specsonbloor.ca', 'http://www.estheticsbycristina.ca/', 'http://www.silkmanlawfirm.com', 'http://hubcapannielasvegas.com', None, None, 'http://www.eastcalgaryregistry.com', 'http://local.firestonecompleteautocare.com/ohio/akron/1245-firestone-pkwy/?lw_cmp=IYP_YPC_MLP_2046', 'http://www.fallsicecream.com', 'http://www.revitasize.ca/', 'http://www.elitefitnesstrainingandcoaching.com/', None, None, 'http://www.atozmovinganddriving.com', None, None, None, 'http://www.lagravineseappraisals.com', 'http://www.custombuiltpizza.com/', 'http://www.jeanvegetariankitchen.ca', '1592161722', 'http://beautyinvegas.com', 'http://www.countyofdane.com/lwrd/parks/indian_lake.aspx', 'http://www.phillipscliniclv.com', 'https://www.mattressfirm.com/stores/oh/streetsboro/mattress-stores-streetsboro-oh-44241-7782.html', 'http://www.hollywoodtans.com', 'http://budget.com', 'http://www.saltcreekofficefurniture.com', 'http://www.happycampersdental.com', None, None, 'http://www.pizzanova.com/locations/mississauga.html', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, 'http://h1racing.com', 'http://pdqmechanicalllc.com', 'http://www.edenwellnessvibration.com', 'http://yardbusterslandscaping.com', None, None, 'http://www.uwhealth.org/locations/detail.jsp?locationId=254', None, None, 'https://www.backfithealth.com', None, 'http://microsoft.com/en-ca/store/locations/ab/calgary/chinook-centre/store-1053?pointit_retailsustain_sitelinks_Yelp', 'http://ruggedmoosedecor.com', None, 'http://simplekitchento.com', 'https://www.republicservices.com?utm_source=yelp.com&utm_medium=referral&utm_campaign=Yelp_Profile_Click&utm_term=gTepcuWPpm5jqrVjM0gwew', 'https://www.standardplumbing.com/locations/PE', 'https://local.fedex.com/az/phoenix/office-5048/', '1592163820', None, None, 'https://ollyshoes.ca/pages/yonge-street-store', 'http://www.waterfrontoronto.ca/explore_projects2/west_don_lands/don_river_park', 'http://www.biggamblesrummagesale.com', None, None, None, None, 'http://www.arizonabouncearound.com', 'http://lavidamassageballantynenc.com', None, 'http://adultmart.com?utm_source=yelp&utm_medium=social&utm_campaign=yelptest&utm_content=url', 'http://www.firewatercalgary.com/', 'https://www.hertz.com/rentacar/location/unitedstates/arizona/phoenix/PHXC04', 'http://www.sameatshops.ca', 'http://mywheelguylv.com', 'https://locations.theupsstore.com/az/tempe/1753-e-broadway?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.shopsteelcity.com', 'http://www.RiteRug.com', 'http://www.thegolfdome.com', 'http://hianddrycleveland.com', 'https://plan.dignitymemorial.com/yp/plan/?id=542949820&utm_source=yp.com&utm_campaign=ypm&utm_medium=display&utm_term=386&utm_content=mlp', 'http://www.eurogyro.com/', 'http://www.marriott.com/hotels/travel/cltfa-fairfield-inn-and-suites-charlotte-arrowood/', '1592163884', 'http://www.logojoy.com', 'https://locations.theupsstore.com/az/scottsdale/16211-n-scottsdale-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.donttreadonbeans.com', None, None, 'http://www.sleepcountry.ca/', 'http://allcountypet.com', 'http://glenmeadowsauto.mechanicnet.com', 'http://www.parloursalon.com', None, 'http://www.merchantoftennis.com/', 'http://www.neshoffscarpetcleaning.com', 'http://www.fortmcdowelladventures.com', 'http://www.psychicamandamarquez.com', 'http://www.tricountymg.com', 'https://www.alaschools.org/arizona/az-schools/gilbert-k-6', None, 'https://pastureandplenty.com', 'https://palacestation.sclv.com/Gaming/Bingo?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', None, 'http://www.cupidstoybox.com', None, 'http://thesanddollarlv.com', 'http://www.usysnv.net/page/show/110658-bettye-wilson-soccer-complex', 'http://www.themeadowsschool.org', None, 'https://www.esso.ca/en/gas-station/richmondhill-on-beavercreekesso-200302403', 'http://www.studiocsalonsaz.com/studio-c', 'http://www.clorebeauty.com', '1592163956', 'http://klimateking.com/', 'http://www.spinco.ca', 'http://infinitevapor.com', '1592163966', 'http://www.thebrewteabar.com', 'http://www.chopchopchinesekitchen.com', 'http://www.chilitos.ca', 'http://thewienerscirclelv.com/', 'http://www.hob-nobs.com/', 'http://www.legendaryhomeservice.com', 'http://DefiantPestControl.com', 'https://www.napoli.vegas', 'http://www.brunswickkidds.com', 'http://www.ctrca.com/', 'https://www.trekbikes.com/us/en_US/retail/madison_west/', 'http://www.streetsofnewyork.com', 'http://nbdmd.com/', 'http://phoenix.gov', 'http://www.azpodiatrists.com', 'http://www.larryscocktails.com', 'http://www.cookout.com', 'http://spandexcity.com', 'http://www.casalinochiropractic.com', None, 'http://www.samdaebbq.com', 'http://balancingpawsdogtraining.com', None, '1592164037', 'http://www.revivme.com/las-vegas?utm_source=yelp-org&utm_medium=webclicks', '1592164042', 'http://www.beelogicpestcontrol.com', 'http://www.creativestyleslv.com', None, 'http://Www.bellaskinsalon.com', 'https://www.chick-fil-a.com/rossparkmall', '1592164058', None, 'http://www.bluejadechinese.com', 'http://nswhobgyn.com', 'http://brewsters.ca/', '1592164073', 'http://www.luxebrides.vegas', None, None, 'http://paradisebakery.com', None, 'http://actoneyouththeater.org', 'http://www.cityofhenderson.com/henderson-happenings/facilities/overview-of-pools/pool-locations/whitney-ranch-indoor-pool', 'http://www.7-eleven.com/?yelp=29384', 'http://www.johnniewalkerrv.com/index.php', 'http://www.ColleenSellsAZ.com', None, None, 'https://www.haircutmengibsoniapa.com', 'https://www.payless.com/stores-location/IL-CHAMPAIGN-5067.html', '1592164114', 'https://www.cafetempeaz.com', None, None, 'http://www.bartongrouphomesearch.com', 'http://foleysac.com', 'http://www.foodandwineexpo.ca/sitepages/', None, None, '1592164141', None, 'http://www.jaginchystuff.com', 'http://www.tremontpointe.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975319&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.qashtonbeauty.com', 'http://www.phoenixairport.hilton.com', 'http://www.sweetaffairsbakeshop.com', 'https://www.verizon.com/home/storelocator/pa/pittsburgh-15206/', '1592164167', None, 'http://local.firestonecompleteautocare.com/arizona/peoria/10661-n-84th-ave/?lw_cmp=IYP_YPC_MLP_11932', None, 'http://www.azcoolmotors.com', 'http://www.shoproka.com/web/', 'http://www.gostorageone.com/self_storage/North_Las_Vegas/zip_89084/storageone_self_storage/10487', 'https://www.anytimefitness.com/gyms/1820/Markham-ON-L6B-0P2/?utm_source=yelp&utm_medium=local', 'http://www.pinkpolishnailspa.com', None, None, 'http://www.boyacklawgroup.com', 'http://dekra.us/en/nevada/las-vegas-3209-n-rainbow-blvd/', 'http://www.goplaycanada.com', 'http://www.crispfoods.com', 'http://www.restechservices.net', 'http://www.polepositionraceway.com', '1592164211', None, 'http://geraldinetoronto.com/', 'http://www.bakerperformingarts.com', 'https://dutchbros.com/?utm_source=las-vegas&utm_medium=yelp&utm_campaign=website_las-vegas', None, None, None, 'https://tortillafeliz.com', 'http://www.charlotterecyclery.org', 'http://www.maxmoving.ca', None, 'http://www.daggerdiesel.ca', 'https://my.clevelandclinic.org/locations_directions/Regional-Locations/beachwood-fhsc', None, 'http://www.cannonpharmacies.com', 'http://www.bettysonking.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.waffluv.com/find-us', 'http://www.dunkindonuts.com', 'http://www.ytr.com', 'http://www.scoopnswirl.com', 'http://www.piedmont-eyecare.com', 'http://www.thefoodconnectionlv.com', None, 'http://www.starbucks.com/', 'http://www.copperrestaurant.com', None, 'http://www.peachykleenlaundry.com', 'https://wildfire.sclv.com/Wildfires/Wildfire-Rancho?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://extremepita.com', None, 'http://www.axelmortgage.com', 'http://www.hobbylobby.com/StoreLocator/Search?Latitude=33.2774819&Longitude=-111.73165570000003&SearchOn=85297&SearchDistance=50', None, None, None, 'http://www.myerstar.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', '1592164296', 'http://www.sportclips.com/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975058&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, '1592164307', 'http://www.busterstransmission.com', 'http://www.the-alley.ca', 'http://www.mifamiliamexicanfood.com', 'http://Www.thebubblegumgallery.vpweb.com', None, None, 'http://www.lincolnlandexpress.com/index.php', 'http://www.kaiulanidetailing.com', 'http://restaurants.quiznos.com/oh/lorain/amherst-44053', 'http://www.angelscremationandburial.com/', None, None, None, 'http://www.luxepethotels.com', None, 'http://www.officedepot.com/storelocator/az/chandler/officemax-6515/?cm_mmc=GMB-_-6515-_-Website-_-NS', 'https://www.hughesfloorcovering.com/yelp-offer', 'http://www.pinesatcarolinaplace.com', None, None, 'http://www.charlotteobserver.com', None, 'http://www.tmdish.com', 'http://www.insideoutpatio.ca', 'http://www.firecreekcoffee.com', None, None, 'http://www.QuickKeyLocksmithLasVegas.com', 'http://www.loftea.ca', 'http://sandersonplumbinglv.com', None, 'http://www.sopra.ca', 'http://mylifelinemd.com/', 'http://EUsweets.com', 'https://vinoandcanvas.com', 'http://www.desertsunpeds.com', 'http://www.allstardiscountmuffler.com', '1592164392', None, None, 'http://www.pokerpalace.net/', 'http://www.pizzapros.ca', 'https://www.thebso.com/locations/on/toronto/beauty-supply-outlet-haircuts-11813.html', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_112', 'http://www.salstuscangrill.com/', 'http://www.arizonahikebikeboard.com', 'http://www.providentlawyers.com', None, 'http://www.unicarcollision.com', 'http://kickstandcharlotte.com/', 'http://www.american-self-storage.com/avondale', None, 'http://www.TopScottsdaleHomes.com', 'http://www.ssrealtylv.com', 'http://www.thelimited.com', None, 'http://www.shawarma-palace.com', 'http://www.picturepeople.com/default.aspx?PK=e3adeef4-54b4-43c6-990a-619531e6637b', 'http://www.linsseafoodbuffet.com', None, 'http://www.whichwich.com', 'http://redlinediagnosticsllcauto.com', 'https://www.walmart.com/store/5085/belmont-nc/whats-new', None, 'https://stores.whitehouseblackmarket.com/boutique/white-house-black-market-kierland-commons', 'http://www.theconcordcove.com/', None, 'http://thenailchamber.com', '1592164461', None, 'https://www.paradisegrapevine.com', 'http://www.chrispuhlmanflowersandgifts.com/?srccode=yelp_track', 'http://phbalancedpool.com', 'http://www.timhortons.com', 'http://petemac.com/gilbert/', 'https://millersalehouse.com/locations/henderson/?utm_source=Yelp&utm_campaign=Yelp', 'http://nailing.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://vehbrothers.com', None, None, None, 'http://www.noizeboyz.com', 'http://www.originalchimneys.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.blackhawkcc.com', 'http://www.dollargeneral.com', 'http://kingdom-restaurant.business.site', 'https://www.choicehotels.com/illinois/urbana/comfort-suites-hotels/il366/rates', None, 'http://www.sunburstmusic.com', 'http://torontoauto.autotrader.ca/contact-form.htm', 'http://www.thecleverkoi.com/', 'http://www.madcitypetclinic.com', 'http://www.tandori.ca', 'http://www.waitressworldllc.com', 'http://homeinsulationaz.com', 'http://www.woodworkersemporium.com', None, 'https://www.elnuevotaquito.com', None, None, 'http://luxenailsspaphoenix.com', 'http://www.masakosushibar.ca', 'http://www.pandawokscarborough.com', None, 'http://www.tacobell.com', None, 'https://www.sunbritedental.com', 'http://atticsalt.co', None, None, 'http://www.drfineberg.com', 'http://www.uscryotherapy.com/location/scottsdale-az/', 'http://www.sureshotbilliardsaz.com', 'http://uspizzasurpriseaz.com', 'https://www.statefarm.com/agent/US/NC/Charlotte/Jack-English-YKJTS2RZ000?cmpid=M8V7_BLM_0006', None, 'http://www.bedbathandbeyond.com', 'http://www.arizonaministers.com', 'http://www.usairways.com?c=maplisting_Yelp_21041', 'http://www.arbys.com', 'http://www.kuohua.com', 'https://brighthomeenergy.com', None, 'https://www.meineke.com/locations/pa/monroeville-512/?utm_source=yelp&utm_medium=xvc', None, '1592164595', None, '1592164600', 'http://www.shoelessjoes.ca', 'http://azsportsmedicine.com', 'http://www.decadentdesserts.ca', '1592164609', 'http://www.whynotpizzaclt.com', None, 'http://www.diventures.com', 'http://www.ncix.com/', '1592164621', None, None, None, None, 'http://locations.michaels.com/oh/avon/1532/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.allamericanmodernsportsgrill.com', 'https://www.windsorstore.com/blogs/store-locator/az-scottsdale-scottsdale-fashion-square', 'http://www.trumphotels.com/las-vegas', 'http://www.grldcheese.com', 'http://www.wowwash.com', 'http://www.zoyogurt.com/arcadia/', 'https://hatfieldmedicalgroup.com/red-mountain-primary-care-arizona/', None, 'http://www.lafitness.com/Pages/clubhome.aspx?clubid=669', None, 'http://shop.tuesdaymorning.com/', 'https://www.pizzaville.ca/stores/view/4335-bloor-street-w', 'https://www.meijer.com/shop/en/store/146/?cmpid=OLA:040319:YelpGrocery', None, 'http://local.skechers.com/AZ/GILBERT/237/', None, 'http://www.arbys.com', None, None, 'http://www.gratefulgardens.net', 'http://www.wynnlasvegas.com/Shows', 'http://www.recess.net/', 'https://www.aaautocarelv.com/locations/decatur-blvd/', 'http://www.remnanthealth.com', 'http://computerdocnc.com', None, None, 'http://www.papajohns.com', 'https://bananarepublic.gap.com', 'https://www.boehmerheating.com', 'http://fancynailsnorthlasvegas.com', 'https://www.vibenailbarlv.com', None, 'http://www.littlemountaincc.com', None, 'http://azglaucomaspecialists.com/our-locations/phoenix/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://icedoutbarbershop.com', 'http://www.moxies.ca', 'http://www.villamadina.com', 'https://www.davidyurman.com/stores/s/david-yurman-the-forum-shops-at-caesars?ecid=listing%20marketing%20store%20LasVegas%20all%20082216%20all', 'http://www.coconutsaz.com', 'http://www.superstarcarwashaz.com', None, 'http://naildistribution.com/', 'http://www.bk.com', 'http://www.screenlounge.ca/', 'http://beachpizzavegas.com/', 'http://www.anthropologie.com', 'http://www.comicheaven.com', 'http://www.quiktrip.com', 'http://richardsandoval.com/commissary', 'http://www.coldstonecreamery.com/stores/20030', None, 'http://lvbraces.com', 'http://www.silverspoonphoenix.com', 'http://www.ashleyfreehan.com', 'https://radiantdivine.com', 'http://www.dahmensathawks.com', 'http://www.kidsu.ca/', 'https://culinaryhc.com', None, None, 'http://www.luckybreakaz.com', 'http://www.3dudesquilting.com', 'http://www.theprincessmargaret.ca/en/pages/default.aspx', None, None, 'http://www.estrellamountain.edu/', 'http://www.greendrycleaning.ca', 'http://www.westshorehome.com', 'http://www.greatclips.com', None, 'http://laurel-md.yellowusa.com/', 'http://www.honestjohnspgh.com', None, 'http://www.tandoorichaska.com', 'https://www.tempemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://myporcellis.com', None, 'http://www.JimmyJohns.com', None, 'https://www.godsgardentreasures.com', None, None, 'http://www.burns-scalo.com', 'http://www.mrsteamcarwash.ca', None, 'http://www.nouvellemariaspa.com/', 'http://rinckerlaw.com', 'http://www.drchemdry.com', 'http://nationsfreshfoods.ca', 'https://www.kohls.com/stores/nv/lasvegas-1372.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.fourpeaksrentals.com', 'http://www.canuckamusements.com', 'https://greenvalleyranch.sclv.com/Dining/Hanks?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYZGS', 'https://stores.bestbuy.com/nv/las-vegas/6950-arroyo-crossing-pkwy-1044.html/?loc=ns100&ref=NS', '1592164907', 'http://www.roccuzzorealty.com', 'http://www.surprisechildcare.com', 'http://www.shuttlersuniform.com', None, 'http://www.ecochoicewindows.ca', 'http://www.sushiandrolls.ca', None, 'http://www.backofhouse.ca/', 'http://www.handmunlimited.com', None, None, 'http://www.asimpleaffairlv.com', 'http://www.kegsteakhouse.com/en/locations/ON/toronto/mansion-keg/', 'http://www.spinellikia.com', 'http://www.saborcharlotte.com', 'http://www.kctowinginc.net/', 'https://www.crudowine.com', 'https://www1.shoppersdrugmart.ca', 'http://www.jaipurgrill.com', None, 'http://madisoneyecare.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.blackbearcharlotte.com/', 'http://mycupoftea.ca', '1592164974', None, 'http://olympiclimoservice.ca', 'http://www.beddingtons.com', 'http://villagecp.com', None, None, 'http://www.aonefingerprinting.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66054.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66054', 'http://elementsdrycleaner.com/', 'http://www.hungfooktong.ca', 'http://www.ayscater.com', '1592165005', None, 'http://www.mwuclinics.com', None, None, 'http://amelia-c.com/', 'https://www.hilton.com/en/hotels/pitmcht-home2-suites-pittsburgh-mccandless-pa/?SEO_id=YELP-HT-PITMCHT', 'http://www.dunkindonuts.com', 'http://local.buckle.com/PA/PITTSBURGH/415/', 'http://www.pleasantlakeapartments.com', 'https://www.supercuts.com/locations/az/phoenix/village-square-haircuts-9668.html', '1592165031', 'http://www.ateamair.net', 'http://GoldenRuleFB.com', 'https://locations.jackinthebox.com/us/nv/boulder-city/1101-nevada-hwy', None, 'http://www.greengardenflowers.ca/', None, 'http://local.skechers.com/AZ/GILBERT/237/', None, None, 'http://www.bestbuy.com', 'http://www.drshalupal.com/about.html', 'http://www.downwarddoggrooming.ca', None, 'http://www.exxonmobilstations.com/home.php?c=20140424134755&lang=en-us&storenum=757cfd0e77ee7a4ea34c6a6a556f7058', 'https://www.staples.com', 'http://www.federicosmexicanfood.com/', 'http://www.rainbowchiropracticcenter.com', 'http://www.janefendelman.com', None, 'https://glendaleparts.com', None, '1592165090', 'http://www.hairhappiness.com/ca/index.php', None, 'https://www.walmart.com/store/2112/scottsdale-az/details', 'https://www.druryhotels.com/locations/pittsburgh-pa/drury-plaza-hotel-pittsburgh-downtown', 'https://sincityent.com', 'http://www.bondplace.ca', 'https://allyouneedsmokeshop.com', 'http://www.suntechnc.com', 'http://www.mariepain.com', 'http://lazarbakery.ca', 'http://www.dollargeneral.com', 'http://www.platinumblondestudios.com', None, 'http://Chandlersmilesbrighter.com', None, 'http://www.meritagehomes.com/', None, '1592165136', 'http://fiveguys.com', None, None, None, 'http://www.barbarycoastcasino.com', 'http://abcsanitary.com', 'http://meatmarketvintage.com/', None, 'http://coconutcleaningco.com', None, 'http://bostonmarket.com', 'http://drlaffa.com', 'http://www.davescosmicsubs.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=723&utm_content=bpas', 'http://www.lrlv.com', None, None, 'http://www.bluediamondairductcleaning.com', 'http://www.castelloristorante.com', 'http://bit.ly/103bJzP', 'http://www.usps.com', 'http://aromawcs.ca', 'http://lifeguardsolutionsllc.com', None, 'http://www.chandlermall.com', None, 'http://www.aceretailer.com/aceonwilly/', 'https://reliancehomecomfort.com/toronto/', None, 'http://pizza.dominos.ca/Mississauga-Ontario-10348', None, 'https://www.clearchoice.com/locations/dental-implants-phoenix/?trans_id=webdisyelp&utm_campaign=Yelp&utm_source=YelpListing&utm_medium=Listing', 'http://www.4BCafe.com', 'http://www.onabun.ca', 'https://roxybar.ca', 'http://watchoutaz.com', '1592165236', 'http://www.drglick.com', 'http://licecrew.com', 'http://www.cafecentralclt.com', 'https://www.thebrick.com/pages/store-calgary-sunridge', '1592165249', None, 'http://wolfgangpuck.com/restaurants/casual-dining/wolfgang-puck-express/7730', '1592165258', 'https://www.dsw.ca/en/ca/', 'http://www.studio223az.com', 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.andoverwoodsapts.com/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.7-eleven.com/?yelp=29384', 'http://www.evelinecharles.com/', 'http://www.bffgyms.com', 'http://bbdrybar.com/', 'https://stores.advanceautoparts.com/nc/charlotte/2910-freedom-drive?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'http://drakids.com', 'http://Www.lasvegaspsychic.com', 'https://www.vegasdeserttours.com', 'https://www.lasikvisioninstitute.com/lasik-eye-surgery/tempe-surgeon-center/?subid=TMP&venid=YLP31', 'http://www.7-eleven.com/?yelp=29384', 'http://www.la-mestiza.com/', None, None, 'http://check-please-cafe.business.site', 'http://www.sweetitis.net', 'http://www.papyrusonline.com/', 'http://cutenailscharlotte.com', 'http://www.drumsnflats.com', 'https://www.uhaaz.com', 'http://www.butlerplaza.net', 'http://www.filibertos.com', None, None, 'http://nynbeauty.com/', 'http://www.tastebudsyogurt.com', '1592165348', 'http://pellathehandyman.webs.com/', 'http://www.azmediquip.com', 'http://www.efilenevada.com/', 'https://himali-kitchen.business.site', None, 'https://gynecarelv.com', None, 'http://www.annschophouse.com', 'http://www.fitnessinmotionlv.com', 'http://wwwtowingwithtlc.wixsite.com/website', 'https://repestservices.com', 'https://www.ulta.com/stores/charlotte-nc-590', None, 'https://local.safeway.com/safeway/az/chandler/4970-s-alma-school-rd.html', 'http://www.dogsbollockspub.com', 'http://www.freshii.com', 'http://www.havencraft.org', 'http://www.ampm.com', 'http://www.rockyrivervineyards.com', 'http://www.gleemhomecleaning.com', 'http://www.peets.com/?cm_mmc=yelp*peets*8_28_12_homepage*na', None, 'http://www.aubreyshouseofroots.com', 'https://steakfrites.ca/fr/restaurants/detail/saint-sauveur/', 'http://www.theartshoppe.com', None, 'https://www.twiceasnicefood.com', 'http://motorlessmotionbicycles.com', 'http://www.bathandbodyworks.com', 'http://www.tayloredphotomemories.com', 'https://www.moes.com/find-a-moes/ohio/mentor/6467?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', 'http://bk.com/', 'http://www.marriott.com/hotels/travel/cltsd-springhill-suites-charlotte-uptown/', 'http://www.a1carlinemuffler.ca', None, 'http://www.sharpmyknife.com', None, 'https://www.arizonapartybike.com/scottsdale/', 'http://bradyhendersoninspections.com/', 'https://www.jamba.com/nc/charlotte/charlotte-douglas-int-l-airport-613', None, None, 'https://www.take5oilchange.com/locations/oh/bedfordheights-121/', 'http://www.ullaeyewear.com', None, 'http://www.lunaacupunctureaz.com', 'http://dignityvethospital.com', 'http://bodywaxinglasvegas.com', None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpwestorganic', 'http://www.legermainhotels.com/en/torontomls/', 'http://www.realcoolhvac.com/', 'http://www.meckafitness.com', 'http://vitalitybowls.com/locations/las-vegas/', 'http://www.southtrailhyundai.com', None, 'http://www.midas.com/pittsburgh/store.aspx?shopnum=6365&dmanum=688', 'http://www.ttc.ca', 'http://grainger.com/', 'http://www.thelakeshorespa.com', None, None, 'http://www.laurentianlanes.com', 'http://www.Park2300.com', 'https://classicwaterheaters.com', None, None, 'http://carpetcleanertoronto.ca', 'http://www.charlesalexanderdistribution.com', 'https://www.planetfitness.com/gyms/monroe-nc?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.thnclinic.com', 'http://www.rosenails.ca', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'http://www.jetluxuryresorts.com', 'http://www.arwright.ca', 'http://www.tugawaytowing.com/home.html', 'https://www.ttc.ca/Subway/Stations/Ossington/station.jsp#Schedule_', 'https://tempestoysterbar.com', None, 'http://www.locksmithsmadison.com', 'http://spicensabzi.us', 'http://www.moviesoncentral.com', 'http://www.jamisymonssalon.com/', None, 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.saloncercone.com', None, 'http://eggsquis.com/fr/restaurants.php?restaurant=lasalle', 'https://www.montecarlo.com/en/restaurants/brand-steakhouse.html', None, None, None, 'http://www.carwashcleveland.com', 'http://sophiainsapphire.com', 'http://russospizza.net/', 'http://www.secondcup.com', 'http://edeninlove.com', 'http://www.rogersrepairservice.com/', 'http://doubletopping.com', 'http://www.charlotteautogroup.com', None, 'http://www.rebelbook.com', 'https://www.anylabtestnow.com/franchises/huntersville-28078', 'http://www.eatnpark.com', 'http://www.doubledoorinn.com', 'http://robertostacoshop.com', 'http://www.spirithalloween.com', 'http://www.marriott.com/hotels/hotel-rooms/yyzmd-courtyard-toronto-mississauga-west/', 'http://www.dunkindonuts.com', 'http://clevelandsteamercleaner.com', None, 'http://oaklandhallinn.com/', 'http://www.starbucks.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.startrekexp.com', None, 'http://us60selfstorage.com', None, None, 'http://www.jensontotalservices.com', 'https://www.olivegarden.com/locations/nv/las-vegas/las-vegas-cheyenne-ave/1363?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:las-vegas-nv-1363_pl:locurl_rd:1274', '1592165677', 'http://305kustomzdesigns.com', 'http://www.arthurmurrayscottsdale.com/', '1592165689', None, 'http://southwestsmilesdental.com/', 'http://www.prestigemusicacademy.com', None, None, None, None, 'https://www.wontonking.com', None, 'http://www.dunkindonuts.com', None, None, 'http://www.notecollection.com', None, None, 'http://rajafoods.ca', 'http://www.phoenixalterations.com', None, 'http://www.insurewithmelissa.com?cmpid=d3u7_blm_0006', 'http://www.cafe-mirage.com', 'http://livingyogacenter.net/', 'http://deserttintlasvegas.com', 'http://www.restoreandreplenish.com', 'http://www.palaciosupholstery.com', 'http://elisabat.com/', 'http://www.yogurttimedt.com', 'http://cityproperty.com', 'http://www.Ricosacai.com', 'http://www.alternativepaths.org', 'http://deerfootmeadowsoptometry.com/seton-optometry-location-and-hours/', None, 'https://thefaceshop.ca/en/pacific-mall-center-32-32-32-32/', 'http://www.silverminesubs.com/', 'http://seasaltbar.com', None, None, 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', None, 'http://www.menchies.com/frozen-yogurt-shops/frozen-yogurt--northland-plaza-ab', 'http://www.fosdalbakery.com', 'http://www.BombHairVegas.com', '1592165799', None, '1592165804', None, 'http://www.justroughinit.com', None, 'https://www.hazukiphotography.com', None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://style-encorenorthhills.com', 'http://matchacafemaikoclt.com', 'http://Hydra-tattoo.com', 'http://www.sunautoservice.com/auto-repair/nv/las-vegas/las-vegas-tropicana-52', None, 'http://www.chura-hair.com', 'http://www.spoiledchics.com', 'http://www.docwalters.com', 'http://www.sushionbloor.com/', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.litehouse.com', '1592165859', 'http://www.specializedflooringaz.com/', None, None, None, 'http://www.crywolfclothing.com', 'http://www.disantoimmigrationlaw.com', None, 'http://www.rossstores.com', 'http://www.popuppizzalv.com', 'http://www.thomaseyecare.org', None, 'https://budgetblinds.com/DowntownPittsburgh/?utm_source=yelp&utm_medium=naf-listing&utm_campaign=NAF-LOCAL-YELP', None, 'https://www.hankinsplasticsurgery.com', None, None, None, 'http://www.spirithalloween.com', 'http://www.heersmanagement.com/properties/property/desert-point-apartments', 'https://www.hilton.com/en/hotels/cltephf-hilton-charlotte-executive-park/?SEO_id=YELP-HI-CLTEPHF', 'https://www.foodandthingsliquoranddeli.com', 'https://www.hairclub.com/locations/on/toronto/hair-loss-clinic-242/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_242', 'http://tubbystubs.com', None, None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.dipsidoo.com', 'https://www.annaksalon.com', 'https://www.republicservices.com?utm_source=yelp.com&utm_medium=referral&utm_campaign=Yelp_Profile_Click&utm_term=a7viRDrdChmaTJUqBxg4EQ', 'http://www.urbanairgoodyear.com', 'http://ajsalterations.com', 'http://www.nevadacleanup.com', 'http://qdoba.com', 'http://www.purplepenguinsnow.com/', 'https://www.vietbeautysupply.com', 'http://www.nashvillenorth.ca/', None, None, None, 'http://www.lauerrealtygroup.com', 'http://tpmlasvegas.com', 'http://www.clerebar.com', None, None, None, 'https://www.elizabethtatelaw.com', 'http://www.elegancehaircare.com', None, None, 'http://yourspacesalons.ca/tonyc', 'http://www.greatclips.com/salons/3065', None, 'http://www.cherokeehillsgolf.com/restaurant', 'http://www.housetohomeinspection.com/', 'http://prairiefirechampaign.com/', None, None, 'http://cervantesrestaurant.com/', 'https://www.doanrestoration.com', 'http://www.tjbakerconst.com', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.lasvegasartificialgrass.com/', None, None, '1592166030', None, None, 'http://www.sonobello.com/locations/cleveland/?utm_source=yelp&utm_medium=site&utm_campaign=yelp_loc_page_url', None, None, None, None, None, 'https://www.kitchenstuffplus.com/storelocator/#kspST020', 'http://punjabibynature.ca/home.html', 'https://www.charlottefoot.com', '1592166055', 'http://www.kentlanes.com', 'http://www.pghtacotruck.com', 'http://www.tiltonlmt.wixsite.com/mysite', 'http://www.ghadirmeatmarket.com', 'http://www.starbucks.com/store/15887/', 'http://www.pizzanova.com', None, 'http://actionjacksfurniture.com/', 'http://taylorandcolt.com/commerce-court-toronto/', None, 'http://www.blindstogo.com/en/stores/east-mississauga?ua=L4X+1M1&rds=25', '1592166085', 'http://www.menchies.ca/frozen-yogurt-shops/frozen-yogurt-creekside-crossing-ab', 'https://www.searshomeservices.com/locations/appliance-repair/nv-las-vegas/4000-meadows-lane', 'http://www.sundeviltrophy.com/', None, '1592166100', 'http://www.lakeshorepediatric.com', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.popularlube.ca', 'https://www.southhillspetresort.net', None, 'http://www.plumbperformingartscenter.com/', 'https://www.vaqueroscarneasada.com', 'http://www.giantlasvegas.com', None, 'http://www.richsrestaurant.com/', 'https://riosecco.com', None, None, 'https://www.ritasfranchises.com/Brodheadsville', 'http://www.drivingtoindependence.com/', 'http://www.piazzamanna.com', 'http://www.lcbo.com', 'http://www.wxdx.com/main.html', 'http://thebelsize.pub', 'http://www.bannerhealth.com/locations/gilbert/banner-physical-therapy-gateway?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', 'http://www.caromonthealth.org/Locations/Hospitals/CaroMont-Regional-Medical-Center.aspx', 'http://tigerlily.com', 'https://www.mielesitalian.com', None, 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', None, 'http://www.peaceofmindandbodymassage.com', 'https://www.eatnpark.com', 'http://scarletdoorcafe.com', 'https://stores.maccosmetics.com/us/nv/las-vegas/3200-s-las-vegas-blvd', 'http://www.dominos.com', 'http://www.vinylwraptoronto.com', 'http://www.brillorestoration.com', None, 'http://www.dunkindonuts.com', None, '1592166203', 'https://www.ediblearrangements.com', 'http://www.ecosmartpros.com/', 'http://www.accurateautoglass.net', 'http://jennysalterations.wordpress.com/', 'http://www.whiskersandpawsvet.com', 'http://happyfeetnailsspa.com', 'http://www.stonefamilyservices.com', None, 'http://www.animalhospitalatgrayhawk.com', 'http://www.restaurantsawadee.com/', 'http://toronto.thepint.ca', None, 'http://cutie-giftshop.business.site', 'http://signaturebarbershop.net', 'http://cuyahogalibrary.org/', 'https://www.wellsfargo.com', None, None, 'http://mrniceguygames.wpengine.com/contact-us/', 'http://www.dunkindonuts.com', 'http://fiveguys.com/', 'http://www.papajohns.com/', None, 'http://www.sonicdrivein.com/', 'http://602autosports.com/', 'http://www.salonescapeaz.com', 'http://www.rocklobster.com.php53-12.ord1-1.websitetestlink.com/ossington/', 'https://www.4wheelparts.com/stores/north-carolina/4wp-charlotte-nc-54', 'https://www.petsmart.com/stores/us/az/phoenix-store0131.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://www.daango.com', 'http://restaurantspago.com', None, None, 'https://www.amr.net/home/lasvegas', 'http://www.hennashoppe.com', None, 'http://www.lickshomeburgers.com', 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'https://www.pizza73.com', 'http://www.cigna.com/cmgaz/locations/stapley-multi-specialty-center', 'http://tempeimportsaz.com', 'http://www.restaurant-divino.com/accueil', None, 'http://www.big5sportinggoods.com', None, 'http://www.idoupdolasvegas.com', 'http://www.westlibertyanimalhospital.com', None, None, 'http://www.anteloelectric.com', 'http://www.happyhomeservicesaz.com', 'https://www.upmc.com/locations/community/south-side', None, None, 'http://www.newhighend.com', None, 'http://www.Visionworks.com', 'http://patio-pleasures.com/', None, 'https://www.unlv.edu/donnabeamgallery', 'http://www.virtualworldmadison.com', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', None, None, 'http://desertwestobgyn.com', 'https://www.twistedmindssmokeshop.com', 'http://www.PalmLane.ca', None, 'https://www.sportchek.ca', None, 'http://www.mezzaninehair.com', 'https://www.fatfreddyscatering.com', 'http://www.asepestandweedsupplies.com', 'https://www.ihg.com/holidayinnexpress/hotels/us/en/rantoul/cmirt/hoteldetail', None, None, 'http://www.babassupperclub.com', 'http://www.champagnestrands.com', 'http://www.boulevardvegas.com', '1592166417', 'http://bloomcigar.com', 'http://extensionroom.com', 'https://www.fascinations.net', 'http://www.theskullzsalon.com', None, 'http://www.indianpunjabibazaar.ca', 'https://www.bobevans.com/our-restaurants/locations/22?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', None, 'http://bestasianmassagelv.com', 'https://locations.davidstea.com/ab-calgary-111', 'http://www.papajohns.com', None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEWI', 'http://www.vivintsmarthomesecuritycalgary.com', None, 'https://www.terravita.com/neighborhood', None, '1592166467', None, 'http://www.kelseys.ca/', 'http://ago.ca/ago-bistro', '1592166475', None, 'http://www.cbcautism.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'https://www.circlek.com/store-locator/us/tolleson/11450-w-southern/2701130', None, 'http://www.simpleaz.com', None, '1592166498', 'https://scottsdaleearnoseandthroat.com/about-us/meet-our-team/cheryl-a-mackechnie-md', '1592166505', 'http://www.lvcabs.com', None, 'https://hpautorepair.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FPHXPC', 'https://www.purebarre.com/location/san-tan-village-az', 'http://www.canadapost.ca/cpotools/apps/fpo/personal/findPostOfficeDetailPrint?outletId=0000104239', 'http://www.onlinetickets.com/', 'http://www.cwpress.com/', 'http://thesaucebse.com', 'http://www.caferio.com/', None, None, 'http://www.heldtlumber.com', '1592166545', 'https://www.afw.com/stores/az/glendale', 'http://www.tenniswerx.com', 'https://www.publicstorage.com/north-carolina/self-storage-waxhaw-nc/28173-self-storage/2073?pid=wyelp&CID=1245&CHID=AFL', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', 'http://www.purcelltire.com/retail/retail-locations.aspx?store=85', None, None, 'http://www.envynailspaclt.com', None, 'http://www.organicgarage.com', 'http://abbottcourtesyplumbing.com', None, 'http://www.americanrealtylv.com', 'http://www.angelosohio.com/', 'http://www.sipcoffeeandbeer.com', None, 'http://www.osteriausa.com', None, 'http://www.homegoods.com', 'http://www.vegas-asian-massage.com/', 'http://www.cristinahindslaw.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.shangrilabanquet.com', 'http://japanesespecialties.blog.fc2.com/', 'http://www.gordonbiersch.com', 'http://www.awildhairaz.com', 'http://www.switchbladesalon.com', 'https://www.dairyqueen.com/us-en/locator/Detail/?store-id=5175&localechange=1', 'http://www.theloreleiboutique.com', 'http://www.chandleraz.gov/tumbleweed', 'http://www.pizzahut.com/', None, 'http://www.harrisonlawaz.com', 'https://www.menswearhouse.com/store-locator/2513?utm_source=yelp&utm_medium=listing&utm_content=2513&utm_campaign=AZ', 'http://palacenaillounge.net', None, 'http://www.falconlandscapes.com', 'http://www.trueharmonywellness.com', 'http://lodgecharlotte.com', None, 'http://www.moonflightcoffee.com', 'http://subamanautocare.com', 'http://www.theotheroomlv.com', None, None, 'https://tannertileandstone.com', 'http://rebathandkitchens.com', 'https://www.target.com/sl/mccandless/1218', None, 'https://jamboree-bagel.business.site', None, None, 'https://www.arrowheadccaz.com', None, 'http://floydsbarbershop.net/', 'http://www.livemagnoliaterrace.com', None, 'http://www.oasisspasalon.com', 'https://www.homedepot.com/l/Bethel-Park/PA/Bethel-Park/15102/4176?cm_mmc=seo-yelp&', 'http://www.platosclosetcuyahogafalls.com', 'http://www.quesada.ca', 'http://www.sdsdental.com', 'http://www.ipdiet.com', 'http://www.NEwomensclinic.com', None, None, None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700044280216620&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.customcarconcepts.net', None, 'http://www.fabricboutique.com/', 'http://stores.save-a-lot.com/crafton-pa', None, 'http://www.acu-painclinic.com/index.html', 'https://mollymaid.com/southwestern-charlotte-fort-mill-rock-hill', None, 'http://99only.com/store/Mesa---Main/', 'http://www.regaleaz.com/', 'http://www.centennialcollege.ca', 'http://www.carinobistro.ca', 'https://www.ridgeatrobinson.com', 'https://www.integramedical.com', 'https://local.fedex.com/oh/fairlawn/office-1229/', 'http://www.timhortons.com/', 'http://www.graftmadison.com/', 'http://www.arcadiafamilyclinic.com', 'http://www.bayequityhomeloans.com/gilbert', None, None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=12197&localechange=1', None, 'http://www.islandpethospital.com', None, None, 'http://www.marshallscanada.ca', 'http://www.tzatziki.ca', None, 'https://www.thejoint.com/arizona/mesa/red-mountain-48026', 'http://www.callcoolaid.com/', None, 'http://www.hoerner.ca', 'http://www.greatlakesoutdoorsupply.com', 'http://www.thefatbelgian.com/', 'http://thailandspamassage.com', None, 'http://www.surgicalweightcontrolcenter.com', None, 'http://www.sonoranorthotrauma.com', None, None, 'http://www.vcahospitals.com/elliot-park/?utm_source=yelp.com&utm_medium=referral&utm_term=VCA_Elliot_Park_Animal_Hospital&utm_campaign=main_url', 'http://www.eyebrowsrus.com', 'http://www.herriotts.com', 'http://www.longsontherapy.com', None, None, 'http://www.ardene.com', None, None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.dominicspizzamedina.com', 'http://atrautoglass.com', None, 'http://livingwellhomes.com/apartments/details/escondido_apartments', 'http://www.swisschalet.com', 'http://justincookhomes.com', 'http://octanecf.com/', 'http://www.theladysilvia.com/', 'http://www.whitfillnursery.com/', None, None, 'http://theodorewilliamslaw.com/', 'https://upscalemixologist.com', None, 'http://fancynailsaz.com', 'http://www.vivalasguestlist.com', 'http://www.dwr.com/category/find+a+studio/scottsdale.do?mobile=off', None, 'http://gilbert.donutworryaz.com', 'http://greenleafeinncharlotte.com', 'http://www.ahsappliance.com/', None, 'https://www.villageeastcleaners.com', None, None, 'http://info.lululemon.com/stores/us/las-vegas/fashionshowmall', 'http://www.involtini.ca/', 'http://www.blackforestbooksandtoys.com/', 'http://drjimshipley.com', None, 'http://www.arizonaorganic.com/', 'http://www.jasontheplumberllc.com', None, 'http://www.benfranklinplumbingcharlotte.com', 'http://www.aznypdpizza.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://Geostacos.com', 'https://www.thingsremembered.com', 'http://bikepirates.com/', 'http://manhattanpizza4lv.com', 'http://www.greatclips.com/salons/9261', None, 'http://www.divinedoggies.ca', 'http://www.desertdove.com', 'http://www.LocalMotionPGH.com', 'http://www.shopchristophertodd.com', 'http://www.umamipoke.com', '1592167022', 'http://www.marathon.com', 'http://www.timhortons.com/', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.premiercoachworks.com', None, 'http://swigs.ca', 'https://globalgifts.ca', None, 'http://www.sfxphx.org', None, 'https://www.napaonline.com', None, '1592167056', 'http://directpcc.com/', 'https://www.cricketwireless.com', 'http://www.gotaticket.net', None, None, None, 'http://www.rentalstempe.com', None, 'http://www.costco.ca/warehouse-locations/vaughan-woodbridge-on-547.html', 'http://www.momosushilasvegas.com', 'http://threeamigoscharlotte.com', 'http://www.sincitycomputerservices.com', 'https://www.petahead.ca', None, 'http://www.acropoliscg.com', 'https://www.interdent.com/gentle-dental/locations/az/phoenix/702-e-bell-rd-dentist-office/', 'http://www.foundery.is/', 'http://waxhawdentalcenter.com', 'http://www.trainwithgabe.com', 'http://laplanchagrills.com', None, 'http://eastwestblinds.com/?utm_source=Yelp&utm_medium=Yelp%20CPC&utm_campaign=Yelp%20East%20West%20Blinds', 'http://bespokeinn.com/', 'http://www.dunkindonuts.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.craigrd.com/', None, 'http://www.cash4chaos.com', None, None, 'https://www.autozone.com/locations/az/scottsdale/7401-e-mcdowell-rd.html', 'http://www.acebartending.com', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'http://newbeginningscounselingcenters.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://stockmanos.com', None, None, 'http://www.blingtour.com', 'http://charlotteswim.com/', 'http://www.chiro-phoenix.com', None, 'http://alterios.webs.com/', 'http://azteccaraudio.com', 'http://www.arizonaveterinaryoncology.com', 'http://www.robertcoomergroup.com', None, 'http://www.portofsubs.com', 'http://gastondentalassociates.com/', 'http://www.aapglv.com', 'http://www.practiceofpositivity.com', 'https://hirehometeam.com', 'https://www.kohls.com/stores/nv/henderson-667.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.hillsideappliancerepair.com', 'http://www.lorain.lib.oh.us', 'http://www.dunkindonuts.com', None, 'http://www.Konbeasushi.com', 'https://www.walmart.com/store/1598/phoenix-az/whats-new', 'http://www.agaveyaguacate.blogspot.com/', 'http://www.periodentalimplant.com', 'http://www.nandos.ca/restaurants/calgary-17th-coming-soon', 'https://www.signaturestyle.com/locations/on/richmond-hill/first-choice-haircutters-51417.html', 'http://aproposcreations.com/', 'http://www.theshopprinting.com', 'http://fritesstreet.com/', 'http://www.cyanimalclinic.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', None, 'http://local.buckle.com/OH/WESTLAKE/419/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', None, 'http://www.starbucks.com/store/9334/', None, 'http://rebeltoronto.com', 'http://mikasgreek.com', None, 'http://www.premiumgranite.com', '1592167276', 'http://pouloslawfirm.com', None, None, None, 'https://andersondentalaz.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.devicepitstop.com', 'http://www.keungsrestaurant.com', None, 'https://www.chickasawdemolition.com', None, 'http://washmeohio.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://Lasajspa.com', 'http://transnationalmoving.com', None, 'http://www.pinkberry.com', 'http://www.paradiseflowermarket.net', None, None, 'http://spamontst-hilaire.com', 'http://www.aamco.com/centerinfo.asp?centerNum=4128569500', None, 'http://www.banfield.com/veterinarians/nc/cornelius/coe?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', None, None, 'http://www.wbt.com', 'http://tanakasmartialartslasvegas.com', None, 'http://www.arrowprofessionals.com', 'http://www.princetonclubxpress.net/', 'http://www.parkersdowntown.com', '1592167562', None, None, None, None, 'http://www.hibachigrillphoenix.com', None, 'https://www.kaltire.com/en/stores/locations/?StoreID=694&source=yelp_track', '1592167582', 'http://www.theartshoppe.com', 'http://www.stjohnsmusic.com/locations/calgary', None, None, None, 'http://www.spanishtrailcc.com', None, 'https://www.kruyerdental.com', None, 'http://www.fellinislv.com', '1592167611', 'http://sincitybrewtours.com', None, '1592167616', None, 'http://www.caseysdrafthouse.com', None, 'http://www.tradesecrets.ca/', None, 'http://otrocafe.com', 'http://www.lvnailspanv.com', 'http://www.letseat.at/paparazzipizzeria#home', None, None, 'http://lapaloma.ca/about-us/', '1592167648', 'http://www.healthyplanetcanada.com', None, None, None, 'https://www.aplaceformom.com/assisted-living/arizona/phoenix', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', '1592167664', 'http://www.Kaizensushi.ca', None, 'http://helpinghandhandymanservice.com/handyman-services/', None, 'http://www.furniturebank.org', 'http://www.verizonwireless.com/', 'http://www.buddhabodyyoga.ca', None, None, None, 'https://www.rodiziogrill.com/henderson-nv/', 'http://www.sunridgemazda.com', None, 'http://www.breastforbaby.com', 'http://www.bowlingdynamics.com/', 'https://www.americanrealtyacademy.com', 'https://www.ashleycourt-apartments.com', None, 'http://www.lasvegashairandmakeup.com', None, None, 'http://www.campbowwow.com/summerlin', 'https://centennialhillsrealty.managebuilding.com/Resident/PublicPages/Home.aspx', 'https://www.anytimefitness.com/gyms/2194/Cottage-Grove-OR-97424/?utm_source=yelp&utm_medium=local', None, 'http://www.glickdental.com/', 'http://Lawdawgtrailtours.com', 'http://www.citymarketdeli.com', None, 'http://www.regalnails.com', 'http://www.tipitytoenails.com', 'https://www.foodforthought.ca/contact-us/markham', 'http://www.salonlola.net', None, None, 'https://www.publicstorage.com/arizona/self-storage-phoenix-az/85027-self-storage/1113?pid=wyelp&CID=1245&CHID=AFL', None, None, None, 'https://bridalxoxo.com', 'http://www.statewidelightinglasvegas.com', 'https://www.enterprise.com/en/car-rental/locations/us/oh/mentor-3921.html?mcid=iyp:8368400', 'http://www.cozinedental.com', 'http://www.lessermeansmore.com', 'http://www.drinkbambu.com', None, 'http://www.villaserenahenderson.com', None, '1592167790', 'http://www.sevencalgary.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.hilton.com/en/hotels/cltsohw-homewood-suites-charlotte-southpark/?SEO_id=YELP-HW-CLTSOHW', None, 'https://www.menswearhouse.com/store-locator/2503?utm_source=yelp&utm_medium=listing&utm_content=2503&utm_campaign=AZ', 'http://www.eastwestbank.com', 'https://stores.godiva.com/us/nv/las-vegas/3200-las-vegas-blvd-s.html', None, 'http://www.fracturedprune.com/location_charlotte.php', 'http://www.arizonapoolpros.com', 'http://www.kleinhenzjewelers.com', None, 'http://www.eastvalleypetdoors.com', 'http://www.chatpattachowk.ca', None, 'https://www.bostondentalgroup.com/locations/happy-dental-west-valley/', 'http://www.dasonii.com', 'http://gourmetschnitzelhouse.com/', 'https://www.kohls.com/stores/pa/monroeville-289.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.parasitetesting.com', 'http://www.picnicfood.ca', 'http://counseling.healingcouples.org', 'http://www.ssrtowing.com/', None, '1592167861', 'http://www.Vegasjj.com', 'http://www.jerianchiropractic.com', None, 'https://www.sprint.com/storefronts/bd/sprint-chandler-az-85226-fcs-1172/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.starbucks.com', 'http://fitnesstogether.com/waxhaw?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://www.carefreecoverings.com/home', None, 'https://www.sunsetgrill.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://arizonareptilecenter.co/', 'http://www.tastidlite.com/', None, None, 'http://www.victoriassecret.com', 'http://www.arrowoodmed.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.indiascafeandkitchen.com', None, 'http://www.dollartree.com', 'http://www.soshandyman.ca', 'http://www.greencleanphoenix.com/', None, None, None, 'http://paninisgrill.com/locations/euclid', None, 'http://www.cellxperts.ca', 'http://www.mamarobertos.com', 'http://www.subaruofmaple.com/', '1592167950', 'http://www.aaronvargaphotography.com', 'http://modernvascular.com/modern-vascular-of-glendale/', None, '1592167966', 'http://www.yourserenity.ca', 'http://fiveguys.ca', 'http://www.eddiebauer.com', 'http://cosmonailsmesa.com', 'http://www.christieclinic.com', None, 'http://vacationownershipconsultants.com', 'http://www.mcdonalds.com', 'http://www.wardenandsons.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.affirmhartman.com', 'http://rdnplumbing.net', 'http://www.allegromusicaz.com', 'http://www.bevsbeadz.com', 'http://www.phatpackshow.com/', None, 'http://www.bdcdental.ca', 'http://www.sweetnote.ca', 'https://4tenrealestate.com', 'https://www.painteddesertgc.com', 'http://www.marinoswickliffe.com/', 'http://www.delimarkcafes.com/477mtpleasant.html', None, 'https://www.r2smile.com', 'http://www.yelp.com/events/charlotte-hops-and-shops-2', '1592168035', 'http://www.rentcoldair.com/', None, 'http://7keystohealthinc.com/contact-us/', None, 'http://www.massageaddict.ca/mississaugameadowvale', 'http://www.stripper101.com/', None, None, 'http://www.quiktrip.com', 'http://pageperpage.com', 'http://www.stephencooley.com', '1592168063', 'http://www.torontopubliclibrary.ca/detail.jsp?R=LIB060&Entt=RDMLIB060', 'http://tradition.market', 'http://www.thepeacepipe.net', 'http://www.lasvegassuites.com/', None, None, None, '1592168082', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.hoopershealth.com', 'https://arizona.weidner.com/apartments/az/phoenix/roosevelt-row/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'http://www.centuryelectricllc.com', None, 'http://www.vieweyecare.com/', None, None, None, 'http://www.carusogourmetpizza.com/', 'https://locations.wendys.com/united-states/az/goodyear/1178-n-litchfield-rd', None, None, 'http://www.massimodutti.com/ca', 'http://www.restaurantrouge.com/', 'http://www.thesportspub.com/', 'http://www.ektaxandaccounting.com', 'https://locations.vitaminshoppe.com/az/scottsdale/vitamins-supplements-scottsdale-az-571.html', 'http://www.AgaveTreeAndLandscape.com', 'https://www.marks.com/en/stores/rexdale-etobicoke-55-woodbine-downs-blvd.html', None, 'https://www.thearrogantbutcher.com', 'http://www.adenearthworks.com', None, None, None, None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', 'http://www.roserunkennel.com', 'https://www.thebso.com/locations/on/aurora/beauty-supply-outlet-haircuts-11847.html', 'http://www.allfireduplv.com', 'http://www.visitingangels.com/AZ', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3075+West+Liberty+Avenue-Dormont-PA-15216/storeid=4144?cid=ps_ylp_2020', 'http://www.conns.com/store-locator/north-carolina/charlotte-28273', None, None, None, None, 'http://www.superstarcarwashaz.com/w_services2.php', 'http://www.goldstockjewelers.com', None, 'https://www.uhaul.com/Locations/Self-Storage-near-Litchfield-Park-AZ-85340/721037/', 'https://locations.theupsstore.com/nv/henderson/631-n-stephanie-st?utm_source=Yext&utm_medium=organic&utm_campaign=Listings', None, 'http://eastvalleypulmonary.com', 'https://www.gucci.com/us/en/store/7014-east-camelback-road?utm_source=yelp_us&utm_medium=storemarketing&utm_campaign=retail_storepages_marcomm&utm_content=23016_bookapp', None, 'https://www.ritasice.com', None, 'http://www.joinrmxprofessionals.com', None, None, 'http://bartlegibson.com', 'http://www.progressivehome.com', 'http://www.elpolloloco.com/locations/store/3855', 'http://www.problindrepair.com', 'http://stacyneal.wvmbmesa.com', 'http://www.hyattcarcompany.com/', None, 'http://www.thebeehivegifts.com/', None, None, 'http://luckyboyburgershopphoenix.bestcafes.online', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Chandler-AZ-85226/922030/', '1592168214', 'http://jwmechanical.com', 'http://www.dogdaystempe.com', None, 'http://www.venetian.com', 'http://villageclubs.com', None, None, 'https://www.titlemax.com/store/phoenix-e-warner-az-41-title-loans/?utm_source=yelp&utm_medium=listing&utm_campaign=places&tmx_id=771c0159ac754cf&sf_traffic=paid_listing_yelp', 'https://risasdental.com/find-a-location/tempe/?utm_campaign=LocationPage&utm_medium=TempeYelpWebsite&utm_source=Yelp', 'https://www.robinsplumbing.com', None, 'http://www.kierlandcigars.doodlekit.com/', 'http://www.rtthomegroup.com', None, 'http://www.samifruits.com', None, 'http://www.zeppes.com', 'http://www.southshoresah.com', '1592168256', 'http://www.meetfreshcanada.com', 'http://www.chandlerfamilycare.com', 'http://www.dadanddaughterphotography.com', 'https://www.themattressmania.com', None, 'http://uberadventures.net', 'http://megatirecalgary.com', None, 'https://stayatthei.com', None, 'http://www.harborfreight.com', '1592168284', None, 'https://runpigrun.ca', None, 'http://pranamanana.com', 'http://www.isscogenerator.com/main/', None, 'http://www.runplusfun.com', '1592168301', 'http://www.bowlology.com', 'https://www.ulta.com/stores/henderson-nv-1191', 'http://chase.com', '1592168311', 'http://www.floorsforless.com', None, 'http://www.3030dundaswest.com', 'http://www.krankensigns.com/', 'http://www.laurellake.com', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', 'http://www.graceaz.com', 'https://www.roostersmgc.com/locations/nc/huntersville/torrence-village-haircuts-13194.html', 'http://www.greatclips.com/', 'http://omegamassage.ca/', None, 'http://www.wiler.org', None, 'http://azrattlers.com/', 'http://hawksgym.com', None, None, None, '1592168350', 'http://xaviermortimer.com', 'http://www.teejaysbrampton.com', 'http://leesliquorlv.com/locations/the-crane-bar', 'http://portofinocrs.com', 'http://www.mississauga.ca/', 'https://www.foresthillortho.com', None, 'http://my.clevelandclinic.org/departments/eye', '1592168367', 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', None, 'http://www.bk.com', 'http://www.urbanhomeandgarden.com', 'http://N8tiveArts.com', 'https://www.publicstorage.com/south-carolina/self-storage-lake-wylie-sc/29710-self-storage/2566?pid=wyelp&CID=1245&CHID=AFL', 'http://www.valleyairworks.com', None, 'https://www.vagaro.com/nailsbyyulie', 'http://www.schaffers.com/', None, 'https://www.eaglerider.com/phoenix', None, 'https://www.SearsHomeServices.com/locations/appliance-repair/nv-henderson/1245-w-warm-springs-rd', 'https://www.wellsfargo.com', 'http://www.makeupbykyra.com', 'http://encorecatering.com/', 'http://www.cbdemporiumaz.com', '1592168410', 'http://pawntnt.com', 'http://baywoodclinic.com', None, None, 'http://www.corvobartoronto.com', 'http://www.spicersprinklerworks.com/', None, 'http://www.expertnailandspa.com', 'http://cleaningservicecharlotte.net', None, 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', None, None, None, 'https://www.pods.com/locations/united-states/nevada/las-vegas', 'http://www.blackmountainortho.com', 'https://advancedvisioninstitute.net', 'http://christophersfinedrycleaning.ca', None, 'http://www.canvasgallery.ca', 'http://browart23rosspark.com', None, 'https://local.safeway.com/safeway/az/phoenix/810-e-glendale-ave.html', 'http://www.epiclandscapelv.com', 'http://local.buckle.com/nv/las-vegas/457/', 'http://italianoasispizzeria.com', 'http://www.lmca.org/about/lmcafe.cfm', None, 'http://www.flutedmushroom.com', None, 'http://cleaningwonders.com', 'https://www.msisurfaces.com/phoenix-countertop-showroom-distribution/?utm_source=yelp&utm_medium=cpc&utm_campaign=yelp_monthly', 'http://www.memphisstationbar.com/', 'http://www.berdenas.com', 'http://www.tacobell.com', None, None, 'http://www.jacobsonbrosdeli.com/', 'http://www.dornellas.com/', None, 'http://www.bikebike.ca', None, 'http://www.beautybrowsalon.com', 'http://www.theupsstore.ca/60', 'http://www.azmistsystems.com', 'http://www.ajsspamillennium.com/', None, 'http://www.dsw.com', 'http://www.elitefamilydental.com', None, 'http://solita.ca', None, 'http://www.hugoboss.com', 'https://horsebacktrailriding.com', None, 'http://www.cpk.com/location/details/Chandler', None, 'http://www.salonlevel6.com', 'http://www.twainandswensonstorage.com', None, None, None, None, 'http://www.fanniemay.com/', 'http://tavernedelaferme.ca', None, None, 'http://www.essentialskinaz.com', 'http://www.newcreationssalon.com', 'https://www.nevadalegalforms.com', 'http://www.masterhardwarepa.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.theinnersleeve.com', '1592168571', 'http://www.serviceking.com/locations/service-king-falconis?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'https://cinemark.com/nevada/century-16-santa-fe-station-and-xd', 'http://www.metrogreenscape.com', 'http://mkautomotive.com', 'http://www.teuschertoronto.com', None, None, None, None, 'http://www.dininginaz.com', 'http://sc-clinic.com/clinic/spadina-and-bloor/', None, None, 'http://www.maxumconstruction.net', 'https://www.walmart.com/store/6586/concord-nc/whats-new', 'http://www.smartpointresearch.com', 'https://www.cfshops.com/toronto-eaton-centre.html', 'http://www.elmnt.ca', 'http://dubaijewellers.ca', 'http://www.pizzabreak.ca/', 'http://labahiaasadosymariscoslasvegas.cafesonly.online', None, None, None, None, '1592168629', 'http://www.precisiontune.com/ConcordNC', 'http://beyerapplianceservice.com', 'http://www.completemarine.com', 'https://www.food4less.com/', None, None, None, 'http://sundevilpool.com', '1592168649', 'https://www.handandstoneballantyne.com', 'https://blackbeardiner.com/location/glendale/', 'http://fastlanepizza.com', 'http://www.arizonasrealty.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, None, None, None, 'http://k9-games.com', None, 'http://www.carolinashealthcare.org/charlotte-medical-clinic', 'http://www.gmurrayphoto.com', 'http://stcharlesonline.org', 'http://www.classiccakesaz.com', 'http://www.tiptopautocollision.com', None, 'http://www.fresh-burger.com/', 'http://www.lakesidesportschiro.com', 'http://flobarlounge.com', None, None, 'http://www.fat-tuesday.com', 'http://www.stowintegrityauto.com', 'http://experiencetheblackout.com/', 'http://www.charlotteappliancerepairmen.com', None, '1592168709', 'http://www.redtailgolfclub.com', 'https://www.balancinghormonesnaturally.com', 'http://www.emb-plus.com', 'http://famouslaffa.com/', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://cigarsdaily.com', 'http://www.centralphoenixanimalhospital.com', 'https://lvfamilyvision.com', None, 'http://www.cuttingedgecalgary.com', 'http://www.rentaffordable.com', 'http://www.sunnydayslearningcenter.com', 'http://www.oasisspringsbuilders.com', 'http://www.surf-ski.com/', 'http://www.kwakballet.com', 'http://www.setmefree.ca', 'http://cashcanada.com/pawn-shop-location/cash-canada-forest-lawn', 'http://universalenroll.dhs.gov', 'http://www.kfc.com', None, 'http://www.haircutie.com', 'http://www.mcdonalds.com', 'http://frugalmuse.com', None, 'https://www.snaderlawgroup.com', 'http://www.acehardware.com/store-details/15917', None, 'http://www.stlouiswings.com', 'https://www.abuelos.com/restaurants/chandler/', None, 'http://www.shenanigansbarandgrille.com', None, 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', None, 'https://www.anytimefitness.com/gyms/3912/Litchfield-Park-AZ-85340/?utm_source=yelp&utm_medium=local', None, 'http://wynnfitness.com/location/mississauga', None, 'http://newvisionpropertymanagement.com/', 'http://www.islanddentalcenter.com', 'http://www.hootersarizona.com', 'https://www.circlek.com/store-locator/us/scottsdale/20721-n-scottsdale-rd/2705915', None, 'https://lernerandrowe.com/?utm_source=Yelp', None, 'http://www.murrayhillfoods.com', '1592168804', None, 'https://locations.panerabread.com/il/aurora/1304-fox-valley-center.html', 'http://Therapeuticretreat.Com/', 'http://www.instantbeautyspa.com', None, 'http://www.nest.to/', 'http://alexblackgolf.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.verizonwireless.com/stores/ohio/streetsboro/streetsboro-357953/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.dollarama.com', 'http://www.gustafsonlawoffice.com', 'https://www.core704.com', 'https://dealer.sprint.com/experts-choice', None, 'http://www.dragonscandy.com', None, 'http://www.suncitysummerlinflorist.com/?srccode=yelp_track', 'http://www.smoknpig.com/', 'http://www.rahrawrahpetfood.com', None, None, 'https://arizona.weidner.com/apartments/az/phoenix/trailside-at-hermosa-pointe-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'https://dentistryofthecarolinas.com/locations/monroe/', None, 'http://www.timothys.ca', 'https://www.nofrills.ca/store-locator/?utm_campaign=Loblaws&storeId=1384&utm_medium=LPM&utm_source=G', None, 'http://www.lu-az.com', 'https://www.circlek.com/store-locator/us/glendale/5848-w-olive-ave/2706657', 'https://tahdesign.com', None, None, 'https://www.alphagraphics.com/centers/charlotte-north-carolina-us287.html', None, None, None, 'http://www.sabas.com', None, None, 'http://www.picknsave.com', 'http://locations.harristeeter.com/nc/harrisburg/92/', '1592168889', 'http://all-tranz.com', None, 'http://www1.shoppersdrugmart.ca/en/store-locator/store/359', 'http://thefrogandcrown.com', 'http://popeyeschicken.ca/', 'http://www.muchoburrito.com', 'http://www.starbucks.com/store/17296/', 'http://www.stratospherehotel.com/american_superstars.html', 'http://www.foodlion.com', None, 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', 'http://www.hayashihibachi.com', 'http://www.aboutfaceaz.com', None, 'http://www.pizzanova.com', 'http://www.westowntire.com', '1592168925', 'https://www.potterybarn.com', 'http://www.burritoboyz.ca', 'http://www.oldnavy.com', 'https://firstcu.net', None, None, '1592168939', 'http://www.dontortaco.com', None, 'http://astoriacafemarket.com', 'http://downtownbrewfestival.com', None, 'http://www.papajohns.com', 'https://el-pollito-charro.business.site', None, 'http://www.asiaaz.com', 'https://www.currys.com', 'http://www.discount-drugmart.com', None, None, 'https://www.alamo.com/', 'https://www.coffeeatrio.com', 'https://www.studioemphotography.com/photographer-fairmont-scottsdale-princess', None, None, 'http://www.louiesnoodle.com', None, None, 'http://desertcovenursingcenter.com', 'http://parkviewmanor.ca', None, 'https://www.lumberliquidators.com/ll/stores/az/phoenix/2120-s-7th-street?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1019', None, None, 'http://www.chinapearl1.com/home', 'http://amoving.company', None, 'http://www.macgyverautobody.ca', None, 'https://local.fedex.com/oh/north-olmsted/office-1234/', None, 'http://www.poochesnpalsinc.com', 'http://www.hmsmokeshop.com', 'http://www.bankofamerica.com', 'http://www.a2bmovers.com', 'https://www.casareynapgh.com', None, 'http://www.smilecreationsaz.com', 'https://curaleaf.com/stores/nj', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.rounduplakecampground.com/', 'https://www.sunlifemaids.com', 'https://www.chevronwithtechron.com', 'http://sugarshacksweets.net/', None, 'http://www.stuffingtonbear.com/', None, 'http://www.myflightsurgeon.com', None, 'http://www.stavrossportsbar.com', 'http://www.teffngreens.ca', 'http://www.modern.yoga', 'http://www.fillsgas.com', '1592169057', 'https://local.fedex.com/pa/pittsburgh/office-2064/', 'http://www.scoopnyc.com', 'http://ourjourneytraining.com', None, 'https://www.camelbackautoglass.com', 'http://www.tempespeedandperformance.com', None, 'http://www.spartantrainingchandler.com/', 'http://www.tbirdmed.com', None, None, 'https://www.petsuppliesplus.com/Store/OH/Elyria/Elyria/123?utm_source=Yelp&utm_medium=Store-Page%C3%8A', 'http://www.enzositalianmarket.com', 'http://www.drlonlawrenz.com', 'https://www.anylabtestnow.com/franchises/las-vegas-89146/', 'https://stores.jcrew.com/en/south-hills-village', 'http://coachzinger.com', 'http://www.footsensation.ca/', 'http://www.assistbooks.com', None, 'http://www.goodlifefitness.com/locations/ontario/toronto/kingliberty?source=yelp-websitelink-torontoking-gl', 'http://www.mcdonalds.com', 'http://www.adore2.com', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://transcanadacarrental.com', 'http://www.berryvinegifts.com', 'http://www.thewellnesscenter.org', 'http://www.lux-spa.com', 'http://www.shopkawaii.com/', '1592169123', 'http://www.playitagainsportsmadison.com', None, 'http://www.audichandler.com/index.htm', None, 'https://www.peets.com', None, None, 'http://www.hollandresidential.com/talise', None, 'http://www.insurrectionaleworks.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.domesticblissdesign.com', 'http://taniahairbraiding.com/', None, 'http://www.graceinlasvegas.com', 'http://www.damcomputermedic.com', 'http://lokacupunctureclinic.com', None, 'http://www.artistsplay.com/hub-cafe', None, None, 'http://www.thrifty.com/?campid=DS:YL_THR_20150109_HP', None, 'https://www.wingstop.com/location/wingstop-421-phoenix-az-85015/menu', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006430792986&ds_s_kwgid=58700000444978510&ds_url_v=2', 'http://cogburnsbar.com', None, 'http://www.kidscoop.org/', 'http://sangrock.com', 'http://www.expertconcretecoatingsinc.com', None, 'http://www.braizenfoodtruck.com/wp1/', 'http://www.drraykim.com/', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.cjsitalianice.com', None, 'http://www.awinadvancedautobody.ca', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Calgary-AB-T2H1L6/840051/', 'http://www.doddslaw.com', 'http://www.stacyspitstopbbq.com', 'http://www.hbotofaz.org', 'http://www.flowershoppetempe.com', 'http://www.moderationsuit.com', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.montzauto.com/', None, '1592169237', 'http://www.foodkingmenu.com', None, 'https://cortland.com/apartments/cortland-whitehall/', None, 'http://tiffanynailshairscottsdale.com/', 'http://www.chocolatsfavoris.com', 'http://www.yogadigs.com/', None, 'http://www.estheticaglenmore.com/index.php', 'http://www.richmondhilltoyota.com/', None, 'http://www.advant-edgecurbing.com', 'http://www.juicestandard.com', None, 'http://nlasvegasvets.com', 'https://footsolutions.com/peoria/', None, 'http://www.themanorspa.ca/', 'http://www.intownsuites.com/locations/charlotte-northeast-chn/', None, None, None, '1592169289', 'http://www.allearzsavannahs.com/', 'http://www.CoronadoHeights.com', 'http://www.aimeecaballero.com', 'http://www.pinklotusmenu.com', 'http://www.myscrubsandmore.com', 'http://www.parmatireservice.net', None, 'http://myvillaroma.com/', 'https://boulderoaksca.com', 'http://www.livingstoninnmadison.com', '1592169315', 'http://www.duffsfamouswings.ca', 'https://www.sourceforsports.ca/en-CA/Stores/3-Adrenalin-Calgary-Source-For-Sports.aspx', None, 'https://budgetbicyclectr.com/used-bicycle-showroom-museum', 'http://arizonagoldendoodles.com', 'http://www.einsteinbros.com', None, 'http://chickenfootestatesales.com', 'http://www.joyli.net', 'http://stuartdayguitars.com', 'http://www.foodlion.com', 'http://www.mellowbentertainment.com', 'http://avenueb-pgh.com', 'http://www.superelectric.tv/pinball', 'http://www.backonthebar.com', 'http://www.dickssportinggoods.com/', None, None, None, 'https://www.6ixcycle.com/contact-us/', '1592169360', 'http://azcountertop.com', 'http://www.turnberrytowers.com', 'http://www.liftedtrucks.com', 'http://elwoodmotorsportsandcollision.com', 'https://www.wellsfargo.com', 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=1243', None, 'http://www.tntmobiledetail.com', 'http://mariscoensenada.com', 'http://www.rachelsyoungatart.com', None, 'http://www.outback.com/locations/az/gilbert', None, 'http://redmodernfurniture.com', 'http://www.savorcafecharlotte.com', 'https://www.finessecateringandevents.com', 'https://www.petpeoplestores.com/rocky-river-store.html', 'https://www.gregbryandds.com', 'https://www.searsclean.com/pittsburgh-pa', None, 'http://www.cobsbread.com/home/', 'http://www.mcnallyauto.com', None, '1592169413', 'https://palmbeachtan.com/locations/NC/Indian-Trail-Old-Monroe-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=AUG', 'http://www.monroevilleconventioncenter.com', 'http://www.moes.com', None, 'https://www.umcsn.com', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_399', 'http://www.markhambowl.com', 'http://www.maidpro.com/henderson-nv', 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=119&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-119', 'http://www.hairrehabaz.com', 'https://abraauto.com/abra-auto-location/1051-charlotte-nc-albemarle-road', None, 'http://www.gingerflowerstudio.com/', 'http://domenicspub.com/', 'http://www.angelpark.com/-ninos-kitchen', 'http://www.redonionlounge.com', None, 'https://introboutique.com', 'https://www.faema.ca', 'http://johnnyvegasgrill.com/', 'http://www.bakingatitsbest.com', None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Ocotillo.aspx', None, 'http://www.thedistrictatscottsdale.com', 'https://www.marriott.com/spas/lasjw-jw-marriott-las-vegas-resort-and-spa/spa-aquae/5013971/home-page.mi', None, None, 'http://www.chopstickfoods.com', 'http://www.denimanddiamondsmesaaz.com', 'http://www.azenergy.com/', None, 'http://motorlessmotionbicycles.com', None, 'http://Centurionmgmt.com', 'http://www.crossfitwestvalley.com', '1592169495', 'http://www.oldnavy.com', 'http://www.palaroos.com', '1592169503', 'https://www.libertytax.com/income-tax-preparation-locations/19248.html', 'http://www.timhortons.com', 'http://www.gusbrown.com', None, '1592169513', 'http://www.citypharmacylv.com', 'http://lm2investments.com', 'http://www.silverstatesoccerleague.org', None, None, 'http://www.xlntshutters.com', None, None, 'http://www.tomaydo.com/', None, None, 'https://www.gallusdetox.com', 'http://www.laveenalterations.com', None, 'https://www.redrobin.com', 'http://elcortezhotelcasino.com/', 'http://www.fasttracksolar.com/', 'http://www.maricopaac.com', 'http://www.pagesbooks.ca', 'http://www.shaneonealphotography.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://sites.google.com/publikfire.com/publikfire', None, 'http://www.patatemallette.com/fr', 'http://palmvalleyrehabilitation.com/?utm_source=yelp&utm_medium=organic', 'http://criticalaz.com', 'http://sweetfrogyogurt.com/locations/store/tarentum-pa', 'http://www.kmart.com/stores/nevada/las-vegas/3680.html', 'http://www.snugasabug.com/store', 'http://www.luplv.com', 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-bethany-home?utm_source=yelp&utm_medium=local&utm_campaign=Honorhealth_Medical_Group_Bethany_Home', 'http://www.greatclips.com', 'http://www.toysrus.com/family/index.jsp?camp=MISC:yelp:store:page:tru:111910&categoryId=2255956', '1592169583', None, 'http://www.cherrybombcoffee.ca', None, None, 'http://www.courtneyvista.com', 'http://lisasean.com', 'https://www3.hilton.com/en/hotels/nevada/elara-by-hilton-grand-vacations-center-strip-LASCSGV/index.html?SEO_id=YELP-GV-LASCSGV', 'https://shopused.com', 'http://swingshiftauto.com', 'http://nail-salons.dapeem.com/', None, None, 'http://www.lcbo.com', 'https://www.chick-fil-a.com/lee', 'http://www.starbucks.com/store/9298/', 'http://www.sidestogobbq.com/bbq-location/cleveland-hopkins-international-airport', 'http://www.itsjustlunch.com/', 'http://onestopnutrition.com', None, None, None, 'http://www.jamminjerkvegas.com', 'https://www.lifestorage.com/storage-units/north-carolina/charlotte/28270/109-near-matthews/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NC-109&utm_content=website-link&howheard=IYE', 'https://www.marriott.com/hotels/hotel-information/restaurant/yyzda-delta-hotels-toronto-airport-and-conference-centre', None, 'http://www.dragonpearl.ca', None, 'https://www.gianteagle.com/Stores/Store/?storeId=8%2F', None, None, None, None, 'http://www.adampinksalon.com', 'https://essencevegas.com/locations/las-vegas-strip/', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.blushlasvegas.com', 'http://www.autoglassfactoryaz.com', 'http://www.womensinternational.com', 'https://www.advancedmotionchiropractic.com', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.blastedbarley.com', 'http://www.cvcclub.com', 'http://www.falafelcafecleveland.com', 'http://www.munails.com', 'https://www.gotenda.com', None, 'http://www.coffeerushcafe.com', 'http://www.azdeq.gov/EmissionsTest', 'http://www.RedesignUpholstery.com', None, 'http://www.olbrichbiergarten.com', 'http://doubletree3.hilton.com/en/hotels/ontario/doubletree-by-hilton-hotel-toronto-downtown-YTOCSDT/index.html?SEO_id=YELP-DT-YTOCSDT', 'http://www.northshorecommercialdoor.com', 'http://www.thewhiningpig.com', None, 'http://www.eastmeatswesthalal.com/', None, 'https://www.speedy.com/shops/toronto-danforth/en/', 'http://sandiegonailsphoenix.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, 'http://www.ingleside.com', 'https://soslawnmaintenance.com', None, None, 'http://www.deltamechanical.com', 'https://www.wyndhamhotels.com/hotel/02493?tel=18002521709&iata=00093785&cid=fe:ra:20161215:yl:pp:ramus:02493', '1592169733', 'https://www.speedway.com', 'http://www.gianteagle.com', 'http://www.theredzonemadison.com/', 'https://azcdps.com/locations/paradise-valley/', 'http://www.kitchenland-lv.com', 'http://www.squirrelhilloptometrist.com', 'https://stepuptowellnessnow.com', 'http://www.WestonTeamMortgageLoans.com', 'http://www.dreamlanes.com', '1592169756', None, 'http://www.eatfromatruck.com', None, 'http://www.coffeebytes.net', 'http://tahoepower.net/', None, 'http://www.cafedepariscanada.com', 'http://www.mecklenburgbridal.com', None, 'https://www.lasvegashomes.vegas', 'http://www.DrThompsonAZ.com', None, 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.ups.com/ca/en/help-center/contact.page', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.capstonerealtypros.com', 'https://ad.doubleclick.net/ddm/clk/459148250;262878939;b', None, None, None, 'https://www.clubpilates.com/tempe/', 'http://www.starbucks.com/', None, 'http://www.7-eleven.com/?yelp=29384', 'http://www.tribeamericaleathers.com', 'http://www.bravobridals.com', 'http://www.ipolished.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.vegasonerealty.com', 'https://procellmotorsports.com', None, 'http://www.subzimandi.ca', None, None, None, 'http://www.starbucks.com', 'http://theflatsatgranitereef.com', 'https://www.budsglassjoint.com', 'http://www.city-tavern.com', None, 'http://www.onceuponachildstow.com/', 'http://www.sonoranlifestyle.com/', 'http://auraspacalgary.com/', 'http://tfwestrella.com', 'http://www.lasvegaspaininstitutes.com', 'https://locations.monro.com/oh/fairlawn/111-ghent-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, 'http://www.doctorsofhair.com/', None, '1592169853', 'https://www.purebarre.com/location/central-phoenix-az', 'http://goodnbr.com', 'http://www.warholdmechanicals.com/', None, 'http://gmhairdesigncolorspecialist.com', 'http://www.michaeljohnsoncpa.com', 'http://robertsidell.com', 'http://www.lakeeriecrushers.com', None, 'https://www.wellsfargo.com', None, 'https://local.fedex.com/nc/charlotte/office-0746/', 'http://www.mrsub.ca', '1592169882', None, None, 'http://www.tempetrophy.com', 'http://www.pvlaserlipo.com/', 'https://www.quicknclean.net', 'https://www.signaturestyle.com/locations/on/toronto/first-choice-haircutters-5677.html', 'http://www.proficientclean.com', 'http://www.einsteinbros.com', 'https://www.kurevapes.com', None, 'https://unlvmedicine.org/family-medicine', 'http://www.7-eleven.com/?yelp=32265', None, 'https://theeggworks.com', 'http://www.aamcophoenixcentral.com', 'http://www.somacafe.com', 'http://www.ronherman.com', 'http://www.culligantotalwater.com', 'http://www.theweaveshop.com/north-carolina-charlotte/', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', 'http://www.starbucks.ca', 'https://www.bluevaportech.com/', 'http://www.missbrow.com/', 'http://gqcutslasvegas.com', 'http://lasvegas.carenow.com/location/ann-simmons', 'http://allure4me.com', None, None, None, 'http://snvscooters.com', 'http://www.bownessauto.com', None, 'http://cowbridgefurniture.com', 'https://www.staples.com', 'http://www.youngbeautifulsalonspa.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://grilladestorino.com/fr/grillades-torino-place-vertu/', 'https://candy-cravers-coffee.business.site', 'https://www.xpo.com', 'https://www.supercuts.com/locations/oh/lakewood/lakewood-city-center-haircuts-83162.html', 'http://www.carters.com', 'https://arrowheadcoin.com', None, 'http://bytecle.com', 'http://www.bombayschutney.com', 'http://www.bookparkngo.com/index.php?main_page=charlotte', None, 'http://www.familydollar.com', 'http://thedetroiterlakewood.com', 'http://www.crownandanchorlv.com/', 'http://www.papajohns.com/', 'http://www.keffervw.com', 'https://www.choicehotels.com/arizona/scottsdale/comfort-suites-hotels/az816/rates', 'http://www.nardisfinedining.com', 'http://balilaffa.com', 'http://www.cabarruseye.com', 'https://stores.advanceautoparts.com/az/scottsdale/15475-northsight-blvd?utm_content=listing-2017-12-21&utm_medium=local&utm_source=yext&utm_campaign=aap', 'http://www.studiojewelerswi.com', 'http://www.mootowncreamery.com', None, 'https://locations.vitaminshoppe.com/oh/strongsville/vitamins-supplements-strongsville-oh-854.html', 'https://6ixtips.ca', 'http://www.pickeringhonda.com/', 'http://www.ericwelchpainting.com', 'http://www.harborfreight.com', None, 'http://www.childsplayaz.org', 'https://www.beallsplumbing.com', 'http://www.tatesac.com', None, None, 'http://www.roiimprovementsllc.com/', None, None, 'https://nevadaoralandfacialsurgery.com', 'https://www.barrhouseonmain.com', 'http://www.bankandbaronpub.com/', '1592170054', 'http://bluediamondlv.com', None, 'http://www.jfdayspa.com', 'http://www.pizzanova.com', 'https://www.thegalleryapthomes.com', 'http://www.elyriafence.com', 'http://www.bakerykaffeehaus.com', 'http://www.drteddysim.com/about-dr-teddy-sim/', 'http://www.gopropoolservice.com/', 'http://www.redressnyc.com/', 'http://charmeck.org/mecklenburg/county/ParkandRec/Parks/ParksByRegion/NorthRegion/Pages/Jetton.aspx', 'http://www.kbhome.com/new-homes-phoenix', 'http://www.freshbonsainails.ca', None, '1592170086', 'https://www.ediblearrangements.com/stores/2875-W-Ray-Rd-Chandler-AZ-85224', None, None, None, None, 'https://www.prosourcewholesale.com/showrooms/oh-prosource-of-cleveland', 'http://www.merrymaids.com', 'https://www.purchasegreen.com/locations/henderson-nv/', None, 'http://the-toy-box.com', None, None, None, None, 'http://www.buckle.com/', 'http://www.nostalgic-furniture.com', '1592170122', None, 'http://www.maximes.ca', 'http://www.allanimalsrescue.com', 'http://www.streetsvilleflorist.com/?srccode=yelp_track', 'http://www.roadrunnerexpresscarwash.com', 'http://www.cafemia.com', None, None, None, None, 'http://Tshsalon.simdif.com', None, None, 'http://www.rossstores.com', None, 'http://www.dirtyfoodto.com', None, None, 'https://www.bellissimafashions.com', None, 'http://www.dollarama.com/', None, None, 'http://www.azdentalheights.com', 'https://www.discounttire.com/store/az/mesa/s/1294?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://clickserve.dartsearch.net/link/click?lid=43700006430627338&ds_s_kwgid=58700000443429188&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.urbancrawfishstation.com', 'http://www.foothillsrehab.com/', 'http://www.holidayinn.com/hotels/us/en/brampton/ybmca/hoteldetail', 'http://www.eatkickshaw.com/index.php', 'http://www.hannahscaramelapples.com', 'http://center-for-growth.com', 'http://www.lux.salon', None, 'http://robertostacoshop.com', 'http://wagnwash.com/locations/phoenix-az/', 'http://www.thepoolmagician.com', 'http://pierrefonds.carstar.ca', None, 'http://www.riosantafeapts.com/?campaign_code=yelp_Rio_Santa_Fe_AZ_85323', 'http://www.arrowheadmartialarts.com', 'https://www.houseofhandsome.com', None, 'http://kristapsons.com', 'http://www.oldcreditbrewing.com', 'http://www.applehillanimalhospital.com/', 'http://www.ortconline.com', 'http://www.dasgasthaus.ca/', 'http://www.campthunderbird.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.34182.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:34182', None, 'http://www.starbucks.com/store/9298/', None, None, None, 'http://www.cronometro.com', None, 'http://www.salonmichaels.com', 'http://www.burlingtoncoatfactory.com/store-details-Store-213--5959-West-Sahara-Avenue--Las-Vegas--NV--89146-3126.aspx?id=2d282a9a-6b10-4437-bb42-7194af655b83&from=89146', 'http://www.fracosandfroccs.com/', 'http://luchitashudson.com', None, None, 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', '1592170265', 'http://www.hm.com', None, 'http://cranberry.sagahibachi.com', 'http://www.tortoisesupply.com', 'http://www.dominos.com', 'http://www.anchorsuprescuegroup.org', 'http://www.lovemycolon.com', None, 'https://grayhawkgolf.com/dining/the-morning-joint', 'http://etherealnails.com', '1592170292', 'https://www.niudanoodles.com', 'http://www.southwestrainbow.com', None, 'http://ad.doubleclick.net/ddm/clk/283420830;110240480;l', 'http://www.foodtruckcaravan.com', 'http://www.portraitinnovations.com/location/143/OH/Mayfield%20Heights/44124/Mayfield-Town-Center', None, 'http://www.hudsonoverheaddoors.com', 'http://www.meltingpot.com/greenvalley', None, 'http://aguamentiaz.com', None, None, 'http://www.rue21.com', '1592170325', 'http://www.valleyvistapoolcare.com', 'http://www.spirale.net/', 'http://fuegobistro.com/fuegotaco/', 'http://www.liasian.com', 'http://DeconResponseteam.com', None, 'http://www.gurtnerdental.com', 'http://www.timhortons.com/', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', 'https://www.oakandfort.ca/', 'http://jcls.jazzercise.com/facility/fountain-hills-jazzercise-center-14600', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/cltad-ac-hotel-charlotte-city-center/', 'http://www.dvmcvet.vetsuite.com', None, 'http://plumbingsource.net', 'http://www.middleburgheights.com/?q=rentals_community_center', 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', None, 'http://www.sparkletime.net', None, 'http://www.playitagainsports.com', 'http://www.hrcleveland.com/tparrot.php', 'http://www.desertstorage.com/las-vegas-nv-boat-rv-storage', '1592170380', 'http://www.nakedcitylv.com', None, 'http://www.bathandbodyworks.com', None, 'http://www.hoipolloicafe.com', None, 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'http://www.plazahotelcasino.com', None, None, 'http://www.thespiritbox.com', 'https://www.patagoniaelements.ca/store/content/10/Calgary/', '1592170409', None, None, 'http://www.countrytireautomotive.ca', 'https://www.royaldistributing.com', 'https://www.presidentialpools.com/gilbert-swimming-pool-builder', 'http://www.gingersdesserts.com', None, None, None, 'http://www.vegasairductcleaning.com', None, 'http://www.betterlifemobility.com', 'http://www.azoral.com/', 'https://www.bellacinosofstow.com', '1592170442', 'https://totalimageoakville.biz', 'http://www.dayonepizzadelivery.com/', None, None, 'http://desotocentralmarket.com', None, 'http://www.eyedoctorchandleraz.com', None, 'http://eosfitness.com/location/pecos?utm_campaign=vegas-pecos&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, '1592170464', None, None, 'https://stores.jcpenney.com/pa/pittsburgh/clothing-stores-pittsburgh-pa-2263.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2263', None, None, 'http://www.thepamperedpinkie.com/', '1592170483', None, None, 'http://www.flightcentre.ca/', 'http://www.marshallsonline.com', '1592170491', 'https://www.gilbertaz.gov/departments/parks-and-recreation/parks-facility-rentals/parks-info-ramada-rentals/parks/crossroads-park', 'http://www.pair.com', '1592170499', 'https://organicnaillounge.business.site', None, None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.mathnasium.com/northwestlasvegas', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.eddietaxplus.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=3317&localechange=1', 'http://www.heroburgers.com', 'https://www1.shoppersdrugmart.ca', 'http://www.sackstierney.com', 'http://www.beastmodeonline.com', 'https://www.waxmans-carpet.com', 'http://myazinjury.com', 'http://www.mrappliance.com/east-valley', None, 'https://www.aritzia.com/en/store?StoreID=aritzia-bloor-flagship-toronto', 'http://www.northcalgarypediatricdentistry.com', 'https://skylineliquorstores.com', None, None, 'https://womensfitnessclubs.com/ajax/?utm_source=yelp&utm_medium=search&utm_campaign=yelpcampaign', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.siennalv.com', 'http://www.tomohibachiandsushi.com/', 'http://www.marriott.com/phxrn', None, None, 'http://www.avis.ca/', None, 'http://www.barriquesmarket.com', 'http://www.spirithalloween.com/', None, 'http://www.bigbenbarber.com', 'http://www.andoaz.com', 'http://www.klassykidsacademylv.com/', 'http://www.freddysusa.com', 'http://repweddings.com', None, None, 'https://www.voguerecoverycenter.com', '1592170593', None, None, 'https://www.planetfitness.com/gyms/streetsboro-oh?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=13080', None, 'https://hawkesclinic.com', None, 'https://www.johnnyscleveland.com', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Sun%20Prairie/WI/3289', 'http://libertyshawarma.com', None, 'http://www.altatempe.com', None, 'https://angel-dentalcare.com', 'http://www.pvcglendale.com', 'http://www.letspartyconsultants.com', 'https://www.wyndhamhotels.com/hotel/53380?tel=18003876085&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:53380', None, 'http://www.v-dubfurniture.com', 'http://www.sovpreschool.org', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.physio-logic.com', None, 'http://www.buca.ca/king.html', 'https://www.100thbgrestaurant.com', 'http://www.arizonaeyes.net', '1592170657', None, 'http://www.dollarama.com/', None, 'http://www.foxspizzamtwashington.com', None, 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', None, None, None, 'http://www.GoodByeDebt.com', 'http://www.starbucks.ca', 'http://www.icecastlearena.com', 'http://www.bestmatt.com/', None, 'http://www.fix8fitnesscafe.com', None, 'http://proem.org', None, None, None, 'http://www.sojoblau.com/', None, 'http://www.canyonstateedu.com', 'http://www.weavebaratl.com', 'http://www.assemblychefshall.com', None, 'https://dealer.sprint.com/preferred-wireless/find-a-store/67331-bridgeville-pa-15017', None, 'https://www.hydroplumbinginc.com', 'http://www.johnwallickjewelers.com', 'http://www.roastednutfactory.ca', 'http://www.hottopic.com/', 'http://www.azulorganicskincare.com', None, None, 'http://www.joyestes.net', 'http://www.jambajuice.com', None, 'http://www.traditionalelectric.ca', 'http://www.hatchyoga.com', None, 'http://www.harbsauto.com', 'http://www.TheKyotoBowl.com', 'http://www.sunnyviewdental.ca/georgetown', 'http://www.inkbombaz.com', 'http://locations.traderjoes.com/oh/woodmere/676/', 'https://www.starbucks.ca/store-locator/store/1017580/bayview-village-2901-bayview-avenue-toronto-on-m-2-k-1-e-9-ca', None, None, 'http://www.bulkbarn.ca/', None, None, 'https://www.supercuts.com/locations/az/surprise/albertsons-shopping-center-haircuts-82334.html', None, 'http://www.applewoodstorage.com', None, '1592170796', None, 'https://theprimerestaurantphx.wordpress.com', None, 'http://fitlifefusion.com', 'http://pazzoscatering.com', 'http://kaffirlimeindonesiancuisine.blogspot.ca', 'http://www.comicoasis.com', None, 'http://www.bigbztexasbbq.com', 'http://www.mrservice.ca', 'http://perennial-yoga.com', 'http://www.lovenspoonfuls.net', 'https://www.davidwmartinlaw.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.kireitoronto.com/', 'https://www.directcarpetone.com', 'http://www.bestratereferrals.com', None, '1592170850', 'http://www.dimpflmeierbakery.com', None, 'http://fabulashclt.com/', 'https://www.georgesflowers.net/?srccode=yelp_track', None, 'https://mintdentallasvegas.com', 'http://www.summerlink.com/wp-content/uploads/2010/03/2010ParkMap', None, 'http://www.southpittsburghvet.com', None, 'http://www.hpsc.ca', 'http://atlanticdentalphoenix.com/', 'http://www.middletonsports.com', 'http://www.californiathai.ca', 'http://www.albertaparks.ca/fish-creek.aspx', None, 'http://summitrestorationlv.com', 'http://serviceprowashandlube.com', 'http://bobcoffeebar.com', 'https://www.lhmfordmesa.com', 'http://www.dailyfreshgrillandcafe.com', 'http://www.houseofblues.com/lasvegas/crossroads', 'http://www.owensautorepair.com', None, None, None, 'http://www.halisangels.com', '1592170925', 'http://www.elixirspa.ca', 'http://www.phillyssportsgrill.com/', 'http://www.charlottecomedytheater.com/', 'http://www.dutildenim.com', 'http://www.treasureisland.com/restaurants/khotan.aspx', 'http://painstopclinics.com/pain-clinic-uptown/', None, 'https://www.modlash.net', None, None, 'http://www.cledogwalk.com', 'http://drivetime.com', '1592170963', 'http://www.bankofamerica.com', None, 'http://www.skyloft.com', 'http://nodayoga.com/', '1592170978', 'http://www.boosterjuice.com', 'http://www.letusrollyouone.com', None, 'https://www.anytimefitness.com/gyms/1069/Pineville-LA-71360/', 'https://iflycu.com', 'http://www.ysl.com/gb', 'http://www.southsidemotorspgh.com', 'https://www.ribeirojiujitsu.com/las-vegas-nevada.html', 'https://www.wellsfargo.com', None, 'https://medmen.com/stores/tempe-airport-phoenix-sky-harbor', None, None, 'http://www.allisonlovesfinejewelry.com', None, 'https://www.uhaul.com/Locations/Truck-rentals-near-Champaign-IL-61820/760043/', 'http://www.ojoslocos.com', 'http://beitshofarot.com', 'http://www.key.com', 'http://www.rainbowchiropracticcenter.com', 'http://www.rivenailandskincarespa.com', 'https://www.pancheros.com/locations/fitchburg', 'https://www.smilesquadlv.com', 'http://www.southfacese.com', 'https://www.bankofamerica.com', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', None, '1592171054', None, 'http://Rentmyrims.com', '1592172433', 'http://www.nevadaspca.org', None, 'http://extremecleantoday.ca/', None, 'http://www.runningroom.com', 'http://www.primarypetcare.com', 'http://www.extremeautocustoms.com', 'http://www.drivingforce.ca', 'http://www.clarkbrands.com', 'http://www.ilovemichaels.com/', 'https://hansenfamilyplumbing.com', 'https://www.ductz.com/north-phoenix-deer-valley/', 'http://www.klosekniturbana.com/', 'http://purebarre.com', None, 'http://renewedbeautyspecialists.com/', 'https://elevationchurch.org/locations/blakeney/', 'http://www.goodlifefitness.com/', 'https://www.selfstoragesolutions.com/self-storage/az/gilbert/85297/421?utm_source=yelp&utm_medium=cpc&utm_campaign=Gilbert', None, None, None, None, 'http://www.drainproplumbing.com', 'http://www.Charleys.com', 'http://www.mtsystems.ca/', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.olivegarden.com/locations/az/scottsdale/scottsdale-e-cactus-rd/1234?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:scottsdale-az-1234_pl:locurl_rd:1175', None, 'http://www.woofswigglesnwags.com', None, None, None, 'http://www.lasvegasnannies.com', 'http://www.sixthcitycycles.com', 'https://fancypetsaz.com', 'http://haymakersportsgrill.com', 'http://Elegantentrysarizona.com', 'http://www.avalonmeatcandy.com', None, 'http://www.theargusrealestatecompany.com', '1592172552', 'http://www.pizzasola.com/eastside.html', 'http://www.arizonabartenders.com', None, None, None, 'http://www.morningstarstorage.com/location/308/sc/fort-mill/carowinds/?ref=yelp', 'http://www.dynamitemontessori.com/', None, 'http://aandbroofing.net', 'https://www.kenganleynissan.com', 'http://www.janthonysalonspa.com/', 'http://rawlicious.ca/canada/locations/markham', '1592172590', 'http://www.illinistor-it.com', None, 'http://www.truevalue.com/mcfarlandtruevalue', None, None, None, None, 'https://www.brakesplus.com', 'https://www.scottsdaleaz.gov/parks/scottsdale-civic-center', 'http://www.athenaswindows.com', 'http://fishkinggrill.com', 'https://locations.davidstea.com/ab-calgary-111', 'http://www.salaradesserts.com', None, None, 'http://www.belle-image.ca/', None, None, 'http://bootlegsmokehouse.com', 'https://www.jasonsdeli.com/restaurants/maryland-pkwy-deli', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', '1592172650', 'http://gotorickys.com', 'http://www.morrowparkcityapts.com', 'http://epicentrenc.com', 'http://www.marketonmacleod.com', '1592172664', None, 'http://subsecreto.com', 'http://www.Barking-Dogs.com', 'http://www.punjabitadka.ca/', None, None, 'http://southeastcrabfeast.com/', 'https://www.tanrepublic.com/locations/nevada/las-vegas/maryland-parkway', 'http://www.mattressfirm.com/', 'http://www.greathousesportsgrill.com', 'http://www.thaihouse.us/', 'https://www.aircanada.com/us/en/aco/home.html', 'http://www.substationii.com', 'http://www.greatwallchinese-lasvegas.com', 'https://www.childhelp.org/advocacy-centers/childrens-center-of-arizona/', None, 'http://www.therefinerycc.com', 'http://onsitetinter.com', 'http://harvestcompassioncenter.org', None, None, None, 'http://www.ohstudiolv.com', 'https://sgplaw.com', 'http://www.failteirishpub.ca/', None, 'https://stores.loft.com/outlet/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'http://www.paradisebakery.com/', 'http://www.osigym.com', None, None, 'https://www.uniquenailbargateway101.com', 'https://www.quarryparkchiropractic.com', 'http://www.easternfootspa.ca', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://paintingwithatwist.com/avon/', 'http://www.urepmaids.com', 'http://www.entebella.com', None, None, 'https://www.springboardrecovery.com', None, 'http://www.pekingexpress.ca', None, None, 'http://www.red9ine.com', 'http://swift-moving.com', 'http://www.frontgate.com/Frontgate/US/CustomerService/Scottsdale/landing-path', 'http://www.greatexpressions.com/dental-offices/ohio-offices/severance', None, None, 'https://www.ulta.com/stores/macedonia-oh-1004', None, None, 'https://www.sitesmartmarketing.com', '1592172812', 'https://www.hilton.com/en/hotels/phxsrhx-hampton-suites-phoenix-surprise/?SEO_id=YELP-HP-PHXSRHX', 'http://patriciangrill.com/', 'http://www.1800waterdamage.com/phoenix-az', None, None, 'http://www.theelranchomotel.com', 'http://falbobrospizza.com', None, 'http://www.noahsnaturalfoods.ca', 'http://vitalshipmed.com', 'https://www.walgreens.com/locator/walgreens-4900+library+rd-bethel+park-pa-15102/id=11296', None, 'http://www.aboutadog.ca/', None, 'http://www.palacestation.com/entertainment/jacks/?utm_term=PS&utm_content=PS-PROPERTIES&utm_medium=SOCIAL&utm_source=YELP&utm_campaign=PS-YELP', None, 'http://www.telaverdeapts.net', 'http://www.skinpossible.ca', None, 'https://www.fordonfifth.com', None, None, 'http://www.gamestop.com', None, 'http://www.rentawrecktoronto.com', 'http://www.fitworks.com/rockyriver', 'http://www.calderawealth.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://extendedstayhotels.com', 'http://jc101services.com', 'http://www.palladinoscleaning.com/', 'https://www.haydenparkapartmenthomes.com', 'http://www.cyclepathnorthyork.com/', 'http://www.starbucks.com/store/69863/', None, None, 'https://www.visioncenterslv.com', None, 'http://makeupmagic.net', None, 'http://therapeuticmethod.net', '1592172922', 'http://www.tlc-airconditioning.com', None, 'https://urbanlivestockvet.com', None, 'https://www.nvisioncenters.com/locations/las-vegas/nevada-eye-care-east/?utm_source=directory&utm_medium=yelp', 'https://www.spectrumdermatology.com', 'https://www.intownsuites.com/extended-stay-locations/arizona/phoenix/west-2/', 'https://beautyenhancementsbyd.com', 'http://www.veneziabakery.ca', None, None, 'http://www.dentscharlotte.com', None, 'http://www.pethavenaz.com', None, 'http://exclusivelytankless.com', None, None, 'http://insurance-agency.amfam.com/AZ/tryna-hoerger-owens/default.aspx?sourceid=agp0001015578', 'http://laterna.ca/', None, 'http://affordableframingaz.com', None, 'http://blackcablimo.wix.com/blackcab', '1592172980', 'http://www.newfoodorder.com', 'http://lolasgelato.ca', 'http://www.tonibulloni.com/', None, None, None, 'http://www.rebath.com/location/lasvegas', 'http://www.acuhealings.com/', 'http://viapanini.ca', None, 'https://www.wyndhamhotels.com/hotel/14408?tel=18008899735&iata=00065402&cid=fe:hj:20170523:yl:pp:hjus:14408', None, None, 'http://ccsunscreens.com/', 'http://tazikiscafe.com/location/charlotte', '1592173013', 'http://www.effortlessmobileauto.com/servicesPhoenix/servicesPhoenix.html', 'http://fukuburger.squarespace.com', None, 'http://www.empowerpt.com', None, 'http://khalsamontessori.org/', 'http://www.mayfieldnissan.com/', 'http://quintealias.com', 'http://www.starbucks.com/store/9229/', None, 'http://www.fatburger.com/home/', 'http://cvdentalcare.com/contact-us-citrus-valley/', 'http://www.bonniesyarncrafts.com', 'http://www.nedrebos.com', 'http://www.ThePMFirm.com', None, 'https://www.xmsweetconnections.com', None, '1592173062', None, 'http://www.ivandianaphotography.com', 'http://www.dunkindonuts.com', '1592173071', 'http://www.cbac.com/sun-prairie?utm_source=Social&utm_medium=Yelp', 'http://www.threebrothersbarandgrill.com', None, 'http://www.cabinforlife.ca', None, 'https://www.azsolarconcepts.com', 'http://mwsvegas.com', 'http://scottsdalecosmeticdr.com', 'http://www.southsidejewelers.com', None, 'https://www.drtroell.com', 'https://www.coolrunningsautomotiverepair.com', 'http://www.stop-n-go.com', 'https://www.zimbrickchevrolet.com', 'http://www.preciouspets702.net', 'http://www.structube.com', 'http://www.middleroadbarbershop.com', 'https://karenshallmarkshops.com', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.springdayspa.com', None, 'http://ridgeparkurgentcare.com', 'https://www.spectrumeyecarenc.com', 'http://vinoandpanino.com', 'http://orangeleafyogurt.com', 'http://weddingsbybonnie.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.richardmilleusa.com', 'http://www.chaneyscollision.com', 'http://www.townshoes.com/', 'http://www.indulgeyourselfsalon.com/', None, 'https://www.luxurypad.vegas', 'http://wanleeloy.com/', 'http://mindbrainlv.com', 'https://www.eatnpark.com', 'http://lerock403.com/', 'http://www.quiznos.com', 'http://www.crossfitculmination.com', None, None, 'http://www.shadowridgeapartmenthomes.com', None, 'http://le-cue.com/', 'http://www.carolinashealthcare.org', 'http://americanbrosplumbingco.com', 'http://www.thepalmsapts.com', None, 'http://www.bk.com', 'http://www.daracap.com', 'http://www.selectphysicaltherapy.com', 'https://www.premierworkspaces.com/nevada/henderson-augusta-park?utm_campaign=Yelp&utm_source=Yelp%20Organic', 'http://www.aulaz.com', 'https://locations.whataburger.com/az/goodyear/14251-west-indian-school-bypass.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', 'http://bullandfinchbars.com', 'http://www.harkinstheatres.com/theatreDetails.aspx?theatreId=4527', None, 'http://www.stfrancisdesaleslv.org', 'http://www.SallyBeauty.com', None, None, 'http://www.ljsilvers.com', None, '1592173210', 'www.brokenplate.ca', None, 'https://www.taylorstmgmt.com', None, 'http://www.swrandalltoys.com', 'http://www.factorydirect.ca', 'https://champsfamilyautomotive.com', 'http://www.orbitzcleaning.com', None, 'https://www.choicehotels.com/pennsylvania/pittsburgh/sleep-inn-hotels/pa744/rates', 'http://www.krispykreme.com', None, 'http://www.jacksonpt.com', None, None, None, None, 'http://www.6az.com/', 'http://www.ynhcclinic.com/', 'http://www.altafillmore.com', None, 'http://www.untitledandco.com', 'http://www.thesodashop.co', 'https://www.themarketphx.com', 'http://www.dqlunchforless.com', 'http://www.yourmovingsolutions.com/?utm_source=Yelp', 'http://miketheplumbercharlotte.com', 'http://www.lasvegasjeepandoffroad.com', 'http://www.oldnavy.com/products/las-vegas-nv-store-5811.jsp', 'http://www.starliquor.ca/', 'http://www.thefrenchbee.com', 'http://www.jeffsnotary.com/', None, None, 'http://pettis-pizza.com', None, 'http://www.rajdhanisweets.ca', 'http://www.simsgm.com/', None, 'https://joefilter.com', 'https://www.maaco.com/locations/az/glendale-11376/', 'http://www.pandaexpress.com/', 'http://www.meineke.com/', 'http://www.dunkindonuts.com', 'http://www.urbanhakka.ca', None, 'http://www.sweetrubyjane.com', None, None, None, 'http://www.legacybungalowsapts.com', 'http://www.getbooked.com', 'https://www.moes.com/find-a-moes/pennsylvania/murrysville/3428?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', 'http://www.mrpuffs.com/', 'http://www.dibeads.com', 'https://www.carstar.ca/en/locations/ab/calgary/carstar-calgary-sunridge-16118/', None, 'http://www.colormebella.com', 'https://www.wilhelmautomotivelitchfield.com', 'http://www.valleverdecarwash.com', None, 'http://www.lapetite.com/your-local-school/las-vegas-nv-7300/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', 'https://local.fedex.com/nc/gastonia/MRNA/?cmp=LOC-1001545-3-2-971-1110000-US-US-EN-YEXTLISTFEDEXSC', None, 'http://www.escapeloungelasvegas.com', 'http://azlifts4less.com', 'https://scottsdaleair.com', None, None, 'http://www.studiofivehair.com', None, 'http://www.insidethebungalow.com', 'http://www.health4lifenv.com', 'http://www.llbcpa.com/#dxEvq', 'http://www.fantasticsams.com', 'http://www.greatclips.com/', None, 'http://www.carolinafireplacerepair.com', None, 'http://www.dickies.com', None, 'http://www.cavabienspa.com/', None, None, None, 'http://beijingchinesecuisine.com', 'https://www.shoeshow.com', 'http://www.orangeleafyogurt.com', 'http://www.starbucks.com/store/8054/', None, None, None, 'http://www.sunshinemoon.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.stevesultimateair.com', 'https://www.wingstop.com/location/wingstop-1117-cleveland-oh-44128/menu', None, 'http://www.inspectslasvegas.com', None, None, None, 'http://www.barefoot-acupuncture.com/', 'http://www.kxyorkville.com/', 'http://flamebroilerusa.com', None, None, None, 'http://www.dragongemchinese.com/', 'https://www.collegecutiescleaningcars.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.fieldservicesaudio.com', 'http://www.lasvegastransportation.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.redrockcanyonlv.org', 'http://www.yongesheppardcentre.com', None, None, 'http://locations.jjill.com/az/scottsdale/65', 'http://www.savourfinefoods.com', 'http://www.twosphotobooth.com', 'https://ccv.church/locations/midtown-phoenix', '1592173504', None, None, None, 'http://www.pmcpropertygroup.com/', 'http://www.citywidemovers.ca/', 'http://www.apnursery.com', 'https://www.extraspace.com/storage/facilities/us/california/glendale/299955/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, 'http://www.sndk9.com/', 'https://local.pharmacy.safeway.com/az/scottsdale/7920-e-chaparral-rd.html', 'https://zoeskitchen.com/locations/store/az/mcdowell', '1592173529', None, 'http://kickinkitchenfoodtruck.com', None, 'http://www.lxvestudios.com', 'http://www.themugshottavern.com/', 'http://www.kravelasvegas.com/', None, 'http://www.noodlewok.com/', 'http://www.gigiscupcakesusa.com/ahwatukee-arizona', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.classictattoowilloughbyoh.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.davidduncanhouse.com', 'http://www.Totallymadeover.com', None, 'http://www.cccakesbybrandi.com', 'http://www.thankqrentals.net', '1592173562', None, None, None, None, 'http://www.grcogrill.com', 'http://www.cantinalaredo.com/location/scottsdale-arizona/', 'http://www.japaneats.ca', None, 'https://www.esso.ca/en/gas-station/vaughan-on-majormacvelloreesso-200302606', 'http://www.jordansrefurbishing.com', 'http://www.gyrostown.com', 'http://www.eddiebauer.com', None, 'http://www.serenityspalounge.com', 'http://www.timhortons.com', 'http://www.iphotoshop.com/', 'http://h5helicopters.com', 'https://www.fahadrestaurant.com', 'http://www.relocationservicescanada.ca', 'http://redcanyondentistry.com', 'http://www.quaker.org/pghpamm/', 'http://www.loveyourposture.com', 'http://www.manilabbq.com', 'http://www.kelseys.ca', None, 'http://www.baskinrobbins.com', 'https://www.boutiquewebsites.ca/website-designs.html', None, 'http://www.decatarina.com', 'http://www.felixswimschools.com/locations-schedules/woodbridge', 'http://www.magictouchmobiledetail.com', 'http://www.shenandoahmill.com', 'http://www.lovinghut.ca', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FLASHH', 'http://www.molonlabe.com', None, None, 'http://www.bundlebuilders.net', 'http://www.thebrewteabar.com', '1592173643', 'http://thecellarsalon.com', None, 'http://carambamex.com', 'http://villagepubcasino.com', None, 'http://www.cainpark.com', 'http://www.fairwaysapartmenthomes.com', 'https://www.athome.com/store-detail/?StoreID=North%20Carolina-Charlotte&utm_source=yelp&utm_medium=paidprofile', 'http://www.midas.com/toronto/store.aspx?shopnum=9827&dmanum=857001', None, 'https://www.bannerhealth.com/locations/phoenix/surgicenter', 'http://www.bestdealsauto.ca', 'http://www.dunkindonuts.com', None, 'http://dekra.us/en/nevada/henderson-26-w-horizon-ridge-pky/', 'http://www.pizzapizza.ca/', 'http://www.urbanfarm.org', 'http://fairgreenlawncare.com', 'http://www.flamingicecube.com', 'http://tnailslasvegas.com/', 'http://countrybarpeoria.com/', 'http://www.magicofjasonbird.com', 'http://kidsinclearningcenters.com/chandler-watermark/', 'http://www.jimenezarmsinc.com', 'https://www.squawpeakhilton.com/dining?_ga=2.50950569.1853949617.1562623293-1431667612.1561583712', 'http://www.madisontranquilitymassage.com', None, 'http://spgilbert.com', 'http://www.siamthaipgh.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.doraziolawgroup.com', 'http://www.highlandinnlasvegas.com', 'https://www.uhaul.com/Locations/Trailer-Hitches-near-Charlotte-NC-28217/780053/', 'https://www.walgreens.com/locator/walgreens-6906+university+blvd-moon+township-pa-15108/id=10917', 'http://www.rinxtoronto.com', 'http://kingskitchen.org/', 'http://www.wholehealth4life.org', None, 'http://www.velocityautobody.com', 'http://www.hotpinkteahouse.com', None, 'http://www.showtimetourslv.com/', 'https://www.dominos.com', None, '1592173746', 'https://us.petvalu.com/location/6058', 'http://illinois.edu', None, 'http://www.arizonaeliteproperties.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', '1592173758', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.missspice.ca', 'http://www.24-7electricalservices.com/', None, 'http://www.familydollar.com', 'http://bchcares.org', 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', None, 'http://www.unforgettablecoatings.com/', 'https://www.purebarre.com/location/fort-mill-sc', 'http://www.starbucks.com/store/9289/', '1592173785', 'http://someburros.com/', 'https://chatters.ca', 'http://salsaritas.com', 'http://www.barbiesgrill.com', None, 'http://www.cleyards.com', 'http://www.onenevada.org', 'http://www.spectrac.com', 'http://www.hortfrontiers.com', None, 'http://pursuitcf.com', None, None, 'http://www.qcreflex.com/', 'http://www.azwhiteningspa.com/index.html', 'http://www.houseofathens.com', 'http://www.firstclassprinting.ca/', None, 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://crissangel.com', 'http://TignorGroup.com', 'http://www.davismagic.com/', 'https://www.kellerhearing.com', 'http://www.cannabisclinics.ca/medical-marijuana-canada-clinic/#1440516223690-62dfe946-9e59', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=976', 'http://www.oasisbeautyspa.ca', 'http://www.sushiplus.ca/', '1592173848', '1592173850', 'http://www.searshometownstores.com/home/az/fountain-hills/1841', '1592173854', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', None, None, None, 'http://www.furnitureworld1.com', None, None, 'http://www.bellagioapts.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', None, None, None, 'https://cell-computers-and-games.business.site', '1592173881', 'https://mamajuliannes.com', 'https://backyourhero.com', 'http://www.theplacerestaurant.ca', 'http://www.urbanbulkandrefill.com', 'https://www.lumberliquidators.com/ll/stores/nc/matthews/11101-east-independence-blvd?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1264', None, 'http://www.marengospa.com', 'http://dialabottle.ca', None, 'https://ad.doubleclick.net/ddm/clk/434342189;236814625;k', None, None, 'http://riotanlasvegas.com/wordpress1/', None, 'http://www.alyciamoore.com', 'http://abdappraise.com', 'http://www.fatbastardburrito.ca', 'http://abakeshop.com', 'http://www.mesabilliards.com', None, 'http://www.marbleslab.ca', 'https://townandcountrybarbershop.com', None, 'http://www.covenantday.org', 'http://www.lakenormanpaintandbody.com', None, None, 'http://www.elitemassagecalgary.ca', 'http://www.cccnevada.com/doctor/g-h-kashef-md/', None, 'https://www.huntington.com/Community/branch-info?locationId=23476', None, 'http://www.bivfaz.com', 'http://www.metro.ca', 'http://www.vegashot.com', 'http://www.crossroadsvet-painesville.com', 'http://www.baskinrobbins.com', None, 'http://www.XtremeExtensions.com', 'http://www.arbys.com', 'http://www.scottsdaleaz.gov/preserve.asp', None, '1592173969', None, 'http://vivaviolins.com/', 'http://www.fionnmaccools.com/locations/brampton-ray-lawson', 'https://www.wingstop.com/location/wingstop-421-phoenix-az-85015/menu', '1592173979', 'https://www.sprouts.com/stores/details/-/store-details/view/store/2/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'http://www.maidpro.com/phoenix-north', 'http://www.kingputtlv.com', 'http://mysuboxonedoctor.com', 'http://www.ivylasersalon.com', 'http://www.kofeinstitute.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.laredosrestaurante.com', 'http://www.wizardofsuds.net', 'http://www.ontarioparks.com/park/voyageur', None, 'https://excelprimarycare.com', None, None, 'https://www.malakjewelers.com', 'http://www.clevelandishome.com', 'http://www.pattysheart.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.marrakechrestaurant.ca', 'http://the-yellow-penthouse.business.site', None, None, None, 'http://www.jjbrewco.com', 'http://www.piccoloantiquemall.com', 'http://www.divinethreading.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.allthingstea.ca', 'http://www-rivera-custom-painting.com/PROFILE', 'http://www.moxieinkdesigns.com', 'http://www.royaljade.ca', 'https://honorhealth.com/locations/primary-care/honorhealth-medical-group-east-tempe', 'https://www.napaonline.com', None, None, 'http://www.abpreschool.com/', 'http://www.ryanssalons.com', 'http://www.scottsdaleaz.gov/parks/cactuspool.asp', 'https://carolinacleanerscharlotte.com', 'http://www.goldsgymcalgary.ca', None, 'https://diceandbeansboardgamecafe.com', None, None, 'https://www.desertfinancial.com/Home/BranchDetails?branchId=1', 'http://www.trendautomotive.com', None, None, 'http://www.isaacjewelers.com', None, '1592174106', 'http://www.ashfordplaceapartments.com', 'https://store.vioc.com/nv/las-vegas/9631-trailwood-dr-89134-HC0003', '1592174114', '1592174118', None, None, None, 'http://pitapitusa.com', None, 'http://meridianlv.com/', None, 'https://www.casinoarizona.com/dining/eagles-buffet/', None, None, None, 'http://www.haggen.com', 'http://www.brickpointingpittsburgh.com', None, None, None, 'http://pennhills.org', 'http://rrrc.run', 'http://www.sunterramarket.com', 'http://www.pastorsautoservice.com', 'http://lvhottubs.com', 'http://l.macys.com/university-heights-oh?cm_mmc=Yelp_Stores-_-UniversityHeights-_-n-_-578&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=University+Heights_578&m_ac=yelp_stores&m_ag=n', None, 'http://www.mychickenshack.com', 'http://angelschristianacademy.com/landing.php', None, 'http://www.housing.wisc.edu/universityapartments/eheights', 'http://www.evaszabo.com', 'http://www.dentalemergencyservices.ca', None, None, None, '1592174202', 'https://container36.wixsite.com/wtrn', 'http://www.artisanhairstudio.net', 'http://www.desertheatwindowwash.com', 'http://www.thedoctorfence.com', None, None, '1592174224', None, 'https://www.choicehotels.com/north-carolina/charlotte/suburban-hotels/nc533/rates', 'http://www.goodmanslandscape.com/', None, None, None, '1592174241', 'http://www.getthelook.com/salondetail/default.asp?salonid=61146', 'http://www.adamsroadsidetowing.com/', None, None, None, 'http://www.aalltechmobile.com', 'http://www.hoopdome.com', None, 'http://www.nrsinjurylaw.com', None, 'http://www.coldwaterroofing.com', 'http://www.MassagesByChelsa.squarespace.com', 'https://www.wallacebethelparkflower.com', None, 'https://www.richerlifecounseling.com', 'https://someburros.com', None, 'http://marcelos-az-barbershop.business.site', None, None, 'http://cowtownbeefshack.ca', 'http://cooperscarpetandtilecleaningaz.com', 'http://alexbeauregard.com', None, None, '1592174306', None, None, None, 'http://www.wrapcitydesigns.com', None, 'http://www.southshoresah.com', 'http://www.balzacs.com/locations/distillery-district/', '1592174326', 'https://npidb.org/organizations/physician_assistants/family_363lf0000x/1386110443.aspx', None, None, None, None, None, None, 'https://www.americandiscountfoodsaz.com', 'http://www.JenniferKostuchowski.MyHomeHQ.biz', 'http://www.sweetdaddycupcakes.com/locations.html', None, None, 'http://www.drbarin.com', 'https://nuancelasvegas.com', '1592174369', None, 'https://www.petsmart.ca/stores/ca/on/scarborough-store2009.html?utm_campaign=us-lcl-srv-eg&utm_medium=listing&utm_source=yelp', 'https://goppa.org', None, None, 'https://www.perplexitygames.com', 'http://www.printystamp.com', 'https://alfredopatisserie.com', 'http://ipacupuncture.com/', None, 'https://hirosushi.to', 'http://www.verizonwireless.com/', 'https://www.peopleready.com/locations/las-vegas-nv-1144', 'http://hancockfabrics.com/', None, 'http://www.welltracnv.com', 'http://www.greenstarautocenter.com', 'http://www.pmkbar.com', None, None, None, None, 'http://www.bhhsnv.com', 'http://www.dhslv.com', None, 'http://www.airportparkingreservations.com/lot_ez_airport_parking_yyz', None, 'http://www.andysfishandchips.com', None, None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1212+South+Greenfield+Road-Mesa-AZ-85204/storeid=5026?cid=ps_ylp_2020', None, None, 'http://www.alwaysjumping.com', 'https://www.mimiscafe.com/locations/glendale/', 'https://www.streetsideclassics.com', 'https://healinghandshannah.com', 'http://www.wendys.com', 'https://nationwidevision.com/locations', 'http://www.canadiantire.ca/en/store-details/on/toronto-main-danforth-on.store.html', None, 'https://globalautoscarrentals.com', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'http://www.gnc.com/', 'https://www.residentculturebrewing.com', 'http://www.ucalgaryrecreation.ca', 'http://www.sheaparc.com', None, 'http://www.windowuniversecleveland.com', 'http://www.marvelousmousetravels.com', None, '1592174529', 'http://www.eraagelessfuture.com', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', None, 'http://www.gap.com', '1592174544', 'http://www.floweramacleveland.com', 'http://www.mycdi.com/arizona/insight_imaging_-_arrowhead/', None, None, 'http://www.perfectblend.ca', 'https://locations.theupsstore.com/pa/moon-township/5990-university-blvd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://www.laseraway.com/locations/nv/las-vegas/?utm_source=yelp+organic&utm_medium=website+link', 'http://www.fabfred.com/locations/trailwood-drive/', None, 'http://www.srsengineshop.com', 'http://studiooftens.com', '1592174581', None, 'http://www.pcfarm.ca', None, 'https://www.cartelcoffeelab.com/pages/5th-avenue', 'http://www.disantopizza.com', None, 'http://www.heritagefishnchips.com', 'http://www.needle-pin.com', 'https://speeders.ca/calgary/', 'http://www.countryhillsmassage.com', 'http://www.fedmc.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360246&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://tempeautoshop.com', None, 'http://www.silverstatecu.com', 'https://www.monicals.com/locations/monicals-pizza-of-st-joseph/', 'http://www.steveedaniellehairandmakeup.com/', 'http://www.myhomegroup.com', 'http://all4you.cleaning', None, 'http://www.drkennedychiro.com', 'https://www.petsmart.com/stores/us/az/surprise-store1040.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', '1592174651', 'http://www.moestire.com', 'http://theartpartystudio.com/', 'https://www.handandstone.ca/locations/richmond-hill', 'http://www.smallcakesphx.com', 'http://www.ratatouillerestaurante.com', 'https://rosyeyebrows.com', 'http://www.phoenixcylinderhead.com', 'http://www.gobencars.com/', 'http://www.bellamiahairstudio.com/', 'https://www.hotelartskensington.com', 'http://www.qimontessori.com', 'http://www.AZbodySugaring.com', 'https://www.ceenta.com', 'https://www.firehousesubs.com/locations/nc/indian-trail', None, 'http://startersaz.com', None, None, 'http://pdubsbbqlasvegas.mypub.website', 'http://porterchiropracticaz.net', 'http://www.alexanderslv.com/', '1592174720', 'http://www.milestonesrestaurants.com/', 'http://www.eddiesalehouse.com/', None, 'http://www.xclusivesparlor.com/', 'http://www.christianjamesart.com/#!/page/183244/home', 'https://www.wyndhamhotels.com/hotel/49487?tel=18002521709&iata=00093785&cid=fe:ra:20170406:yl:pp:ramus:49487', 'http://www.torontomarriagecounselling.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.asianimages.biz', 'http://www.sallybeauty.com', 'https://www.jeansflowers.org/?srccode=yelp_track', 'http://www.lassendasspineinstitute.com', None, 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', None, 'http://www.caplans.ca/', None, None, 'https://www.ewingirrigation.com', 'http://www.six02.com/scottsdale', None, None, None, 'https://internetmarketingteam.com', 'http://lucerestaurant.net', 'http://www.penn-station.com/', '1592174806', 'http://loandaodentistry.com', '1592174812', None, None, 'https://bmcmattress.com?utm_campaign=PhoenixJanMar2018&utm_medium=organic&utm_source=Yelp&utm_content=website-organic', 'http://www.homesurfacesaz.com/', 'http://thelivingroomhairstudio.com', 'http://www.lecreuset.com/stores/tanger-outlets-phoenix-glendale-az', 'http://clinicalsolutionsskincare.com', 'http://www.danier.com', 'http://www.hollisfamdental.com', 'https://drivingmba.com/?utm_source=Yelp&utm_medium=CPC&utm_campaign=Yelp-Ads', 'https://genkijapanese.info', 'http://www.contactlensking.com/', 'http://www.head2toessalon.com', None, None, None, 'http://americarestaurant.ca/', None, 'http://www.amresupply.com/locations/calgary', None, None, 'https://dekalash.com/find-a-studio/pennsylvania/wexford/', None, 'https://www.mjchristensen.com', 'http://www.pusateris.com', 'http://ProCS.us', 'http://swandetailing.com', 'https://ironglovesboxing.com/', 'http://www.bellunionautoservice.com', 'http://www.shoepalace.com?utm_source=yelp&utm_medium=yelp149', 'https://axethrowingpgh.com/booksouthhills/', 'http://www.tuttifruttidejeuners.com/en', 'https://www.courtesyjeepsuperstitionsprings.com', None, 'http://www.londoncleaners.com', None, 'http://www.helvighealth.com/', 'http://www.bebe.com', '1592174941', 'http://www.verizonwireless.com/stores/arizona/scottsdale/promenade-330170/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.lilliputhats.com/', 'https://www.number7honda.com', None, None, 'http://www.wineamplified.com/', 'http://www.glennstoolservice.com', 'http://www.thevictorianmonkey.com', None, 'https://fntarizona.com/scottsdale-kierland', None, 'http://www.anthemchildcare.com', 'http://www.nicols.ca/', 'http://blueskyoils.com', None, 'http://faceofanangel.net', 'http://www.intersport.ca', 'http://shessheik.com', None, None, None, None, 'http://www.adventcoworking.com', 'https://www.walmart.com/store/2512/phoenix-az/details', 'http://www.universalmotorcarslv.com', 'http://northpawdogs.com', '1592175027', 'http://www.pawsinthebath.com', 'http://www.auntieannes.com', 'https://www.walmart.com/store/2256/henderson-nc/whats-new', None, None, None, None, 'http://www.scavolinitoronto.com', 'http://holidaypetfestival.com', 'http://www.precisealterations.com', None, '1592175060', '1592175063', None, None, None, 'http://cateringcompanieslasvegas.com', 'https://www.wyndhamhotels.com/hotel/09499?tel=18003291073&iata=00065402&cid=fe:di:20161220:yl:pp:dius:09499', None, 'http://www.samosagrill.com/', 'http://www.lvcrystalnailspa.com', 'http://www.jamesgallery.net', None, None, 'http://www.thekentstage.com', None, 'http://noirtacgroup.com', 'https://www.goodyearmoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://yuriyscustomjewelry.com', None, 'http://www.vdevmaison.com', 'http://www.newprairieconstruction.com', 'http://www.parkroadhealingarts.com/', 'https://www.ecolinewindows.ca/location/calgary/', None, 'https://www.target.com/sl/mesa-west/1429', None, None, None, None, None, 'http://www.peterparvez.ca', None, 'http://www.teavana.com', 'http://www.lifecaremedicalrx.com', 'http://www.saladessensations.com', 'https://www.jerseymikes.com/3020', 'https://www.uneedmartialarts.com', '1592175182', 'http://aldi.us/index_ENU_HTML.htm', 'http://lincolndoor.com', 'http://bk.com/', None, None, 'http://designerartdirect.com', 'http://www.eljesseautorepair.com/', '1592175206', 'http://www.marriott.com/hotels/travel/yyzbp-fairfield-inn-and-suites-toronto-brampton', 'https://bolmergc.com', 'http://www.snapfitness.com', 'http://www.arrowheadlakesdentistry.com', 'http://www.simonesmtlebanon.com', 'http://www.jewelrysalonlv.com', 'http://www.whitehousechicken.com', 'http://www.carlitosmokes.com', None, None, 'http://www.riderscycleboard.com', None, None, 'http://www.conns.com/store-locator/nevada/las-vegas-89121', 'http://www.glowbeautyloungellc.com', None, 'http://www.sugarmarmalade.com/', None, None, None, None, 'http://scottsdalelavishnails.com/', 'http://www.bambooandspine.com', 'http://www.eyeglass-store.com', 'http://www.pioneercardiovascular.com', 'http://www.davisvillemedicalclinic.com', None, 'http://www.robbellplumbingservices.com', None, None, 'https://oralsurgerylv.com', 'http://www.ritzcarlton.com/en/Properties/Phoenix/Dining/Bistro24/Default.htm', None, None, 'http://envynailsspa.ca', 'https://happytreeslandscaping.com', None, None, None, 'http://kimmiesrestaurant.com', 'http://www.7thdegreeclothing.com', None, None, 'http://www.168sushibuffet.com/mississauga', 'http://www.chipofftheblocklv.com', None, 'http://www.ezpawn.com', 'http://www.applebees.com', 'http://www.portofsubs.com', 'http://www.thecharlotteforum.com', 'https://lastresrosasllc.yahoosites.com', 'http://www.amayaexpress.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Phoenix-AZ-85023/858021/', None, '1592175367', 'https://agents.farmers.com/nv/las-vegas/yebel-escalona?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://buddysautorepair.com', 'http://www.mactechcomputer.com', 'http://www.azmountainviewmassage.com', 'http://vandykeaesthetics.com', None, 'http://www.faithfulwordbaptist.org', None, 'http://www.calgarylabservices.com/', 'http://onestoprealtylv.com', 'http://farmhousefoods.com/', None, None, 'http://koreangrillhouse.com/yonge-bloor-location/', 'http://rollinrustgarage.com', None, 'http://vintageconservatory.com', 'http://phatbuddhatattoos.com', 'https://www.241pizza.com', 'http://www.DentistryoftheCarolinas.com', None, 'http://www.rockstaraccountantsllc.com', 'http://www.chandleraz.gov/default.aspx?pageid=625', None, 'http://www.onemanandahammer.com', None, 'http://www.sushinotori.com', '1592175450', 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.scriptpharmacy.ca/', 'http://www.pancheros.com/locations/urbana', 'http://www.cultivategoods.com', 'http://www.starbucks.com', 'http://www.reynoldsplumbingllc.com/', 'http://www.clevelandcupcakecompany.com/', 'http://www.esmil.com', 'https://www.destinationhotels.com/scottsdale-resort/dining/bar-six40', 'https://www.wwwilliams.com/location/phoenix-az/', None, 'http://www.quiktrip.com', '1592175489', 'http://paradisegreens.com', 'http://www.aerobatics.com', None, None, None, 'http://bellybombz.com/lasvegas', None, 'http://www.madtownrentals.com/', 'http://www.eyedistrictbyjo.com', 'https://www.bathandbodyworks.com/store-locator', None, 'https://www.zamzamhalalmeat.com', 'http://www.walmart.ca/en/walmart-meadowvale-supercentre/3054', None, None, 'http://www.perfectcredit60days.com', 'http://todaysauto.ca', None, None, None, 'http://stores.tuesdaymorning.com/nc/matthews/home-decor-matthews-nc-char0146.html', None, None, 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', None, 'http://www.tntwindowtint.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, 'http://galaxyktv88.com', 'http://www.hollandmarshsoupfest.ca/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.cellularsourcestore.com', 'http://aldi.us/index_ENU_HTML.htm', 'http://www.openstudioartcafe.com', None, None, '1592175611', 'http://www.haifengchinese.com', 'http://www.centennialpediatricslv.com', None, None, 'http://www.mississauga.ca/portal/discover/lakeviewgolf', 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'http://www.abceurodeli.com/', None, '1592175639', None, 'http://www.malkinlegal.com', 'http://www.nextlevelpaintersaz.com', None, None, None, '1592175656', 'https://www.centurylink.com', None, 'https://www.mytightface.com', None, 'http://www.suncoastaz.com/', 'https://www.indochino.com/showroom/charlotte', None, 'http://www.indulgeandco.com', None, 'http://tradeandlore.com', None, 'http://heartlandgsbeautybar.simplybook.me/sheduler/manage', 'https://www.emser.com', 'http://www.quiktrip.com/', None, 'https://www.smithsfoodanddrug.com/storeHours?store=70600376', None, 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', '1592175713', 'http://www.The-Dawg.com', 'http://www.southtempeapartments.com', 'http://shop.hobbylobby.com', 'http://www.thefitalterations.com', 'http://www.vpicasso.com', None, 'http://www.healingheartspeds.com/', 'http://sportsperformanceandspine.com', 'http://www.racewaychrysler.com', 'http://www.fendi.com/ii/store-locator', 'http://clickserve.dartsearch.net/link/click?lid=43700044280205718&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', '1592175749', 'http://www.barbqueboss.com', 'http://adjaafricanbraiding.com', '1592175758', 'http://beyondexpectationdental.com', None, 'http://www.petvethospitals.ca', 'http://www.azenaphoto.com', 'http://www.axisgear.ca', 'https://www.trebilcockcoffee.ca', 'https://www.thetouchofitaly.com', 'http://Rickslawnandlandscaping-lv.com', '1592175785', 'http://www.onceuponachildmacleod.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.espressoroyale.com/', 'http://dtrschooloftrucking.com', 'http://newoutlookopticalnv.com', None, 'http://sushi930.com', 'https://www.mediweightloss.com/locations/ballantyne?_ppc=Z8FWKRXKT5', None, 'http://www.richardsgardencenter.com', None, 'https://locations.levi.com/nc/charlotte/denim_charlotte_nc_258.html', 'http://mojoscoffeeandmore.com', 'http://www.barefootpoolman.com', 'http://www.lushexpress.com', None, 'http://www.stclair.org', None, '1592175848', 'http://www.palmettopethospital.com', 'http://www.mountainsidechiropractic.com', 'http://www.exclusiveballoons.com', 'https://diamondsstrip.ca', 'http://valleywidehouses.com', None, 'http://www.dayspring-umc.org', None, 'https://www.northtownvethospital.com', None, 'http://www.candy-addict.com', 'http://www.harthouse.ca', 'http://www.elitephoenixseo.com/', None, 'http://shea-blvd-chiropractor.com/', 'http://www.alluniformwear.com', None, 'http://www.draskovich.com', None, 'https://lvcollectables.com', 'http://jonesmoverz.com', None, 'http://www.gobeautybar.com', None, 'http://calgary.regency.hyatt.com/en/hotel/home.html?src=corp_yelp_businesslisting_calrc_en_homepage_desktop&code=YB', None, None, 'http://www.thesundanceclinic.com/', 'http://www.sunvalleyfamilyoptometry.com', 'http://www.vipbeautysalon.org', 'http://www.murdockmeals.com', None, 'http://www.pghfilmmakers.org', 'http://pferguson.pillartopost.com', None, 'http://www.dunkindonuts.com', None, None, None, None, 'http://www.masterpieceescapes.com', 'https://nothingbundtcakes.com/bakery/AZ/goodyear?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=goodyear_138', 'http://www.bobbiepinbeauty.com', 'http://www.firkinpubs.com', 'http://murphyusa.com', 'http://Swanpizza.com', 'http://www.advancedpreventivemedicine.com/', 'http://www.starbucks.com/store/6597/', 'http://www.infinityhd.tv', 'http://www.starbucks.ca', 'http://www.lasvegashomewatchers.com', 'http://torontopawn.ca', '1592179247', 'http://www.vegasdiscountnutrition.net', 'http://www.bulkbarn.ca/en-ca/index.html', 'http://adventuredogpark.com', 'http://www.timhortons.com/ca/en', 'http://www.apple.com/retail/southpark', 'http://www.iltartufo.ca', 'http://www.hydrotherapiesplus.com/', 'http://www.desertspringschiro.com', 'https://www.pizzahut.ca/store-details/store-R42101/pizza-hut-Etobicoke', 'http://www.thetirexpress.com', 'http://www.tuckedawaycraftkitchen.com', 'http://www.renewedlifechiropractic.com', 'http://www.phxplastering.com', None, None, 'http://www.muchoburrito.com', 'http://www.theriverwoodconservancy.org/', None, None, None, 'http://blackforgecoffee.com', 'https://www.slatescottsdaleapartments.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://locations.dennys.ca/AB/CALGARY/248179', None, 'http://www.babysafehomes.com/united-states-baby-proofing-locations/child-proofing-phoenix-arizona', 'http://www.clevelandmoto.com', 'http://www.applerepaircanada.ca', 'https://www.druryhotels.com', 'http://www.sabaiaz.com', None, 'http://www.sunsationalswimschool.com/', 'http://www.lapetgrooming.com', 'http://www.gamestop.com/store/us/az/scottsdale/2760/sonoran-village-gamestop', None, 'http://www.ezyes-takeout.com', None, 'http://www.trendvape.com', 'https://glassnickelpizza.com/location/sun-prairie/', 'http://www.arizonarvresorts.com/Pioneer', None, '1592179997', 'http://www.myhealingplaceaz.com', None, 'http://www.restaurantconfluence.com', None, 'http://www.bioprollc.com/', 'http://www.valleyofthesunpharmacy.com', 'http://www.arizonasandwichco.com', None, 'http://www.frogbox.com/toronto', 'https://sushisanrestaurant.business.site', 'http://www.jwoodflooring.com', 'https://www.lefrancisstudios.com', 'http://www.aboutudental.com', 'http://www.kokomogrills.com', 'http://www.ivanhairsalon.ca', '1592180030', None, 'http://www.granadalakestempe.com', 'http://www.capellaeyecare.com', None, None, 'http://www.onthelistdenver.com', None, 'https://www.lvscc.com/south-rancho', 'http://www.lotsmartsolarscreens.com', 'http://www.ontariotrails.on.ca/trails/view/cheltenham-hills', 'http://georgesbrasserie.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://freshandeasy.com/', None, None, 'http://www.northwoodslanes.com', 'http://cleanmycarpetlv.com', 'http://www.nonnispizzamenu.com', None, 'http://globalbikes.info/', 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', '1592180077', None, '1592180081', 'http://www.hotdogonastick.com', None, 'http://www.luchitas.net/elyria.php', None, None, None, 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, None, 'https://www.fitnessworks.com/locations/dobson-rd-chandler-blvd', None, None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.buckysportabletoilets.com', None, None, 'https://www.mirage.com/en/restaurants/osteria-costa.html', 'https://www.cheyennedentalgroup.com/?utm_source=yelp&utm_medium=link&utm_campaign=yelp', 'http://www.atobmove.com', 'https://locations.waxcenter.com/nc/charlotte/charlotte-arboretum-0169.html', None, None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Yucca.aspx', None, None, 'http://www.qsdiner.com', 'http://www.pb.com', 'http://www.amenzone.com', 'http://www.cremacoffee.ca', 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpsouthwestorganic', 'http://www.blueberryhillrestaurants.com/', 'http://www.wishacake.ca', 'http://www.littlecrittersvet.com', 'http://www.c3lasvegas.com', '1592180154', None, 'https://www.thebrick.com/pages/store-north-york', 'http://www.rightc3.com', 'http://imboardgames.com', 'http://www.bwilliamsdds.com', 'http://www.stephita.com/', 'http://www.underdogpetrescue.org', 'http://www.canadianamphitheatre.net/', 'http://www.anthropologie.com', 'http://www.noblecare.us/', None, None, 'http://titanhomeimprovements.com', 'http://nvjawdoc.com/', 'https://www.redlobster.ca/locations/list/on/brampton/368-queen-street-e', '1592180189', None, 'http://www.scottsdale-sc.com', 'http://www.zappos.com', 'http://www.naturalgourmetfoods.com/', 'http://www.walgreens.com/locator/Walgreens-10503-W-Thunderbird%2C-Suite-101B-Sun-City-AZ-85351/id=10018', None, '1592180205', 'http://www.pittsburghcitywide.com', None, 'http://www.inspectioneeringpgh.com/', 'http://www.arizonacharliesboulder.com/Food-Drink/Sourdough-Cafe', 'http://www.regency.edu/campus/madison/?guest', None, 'https://www.leprovencalbistro.com', '1592180219', 'http://www.fallasstores.com', None, 'http://sabiskitchen.com/', 'https://www.nofrills.ca/store-locator/Enzo%27s+nofrills+%28Yonge+St.%29/0770?storeId=0770', 'http://www.sassiskin.com', '1592180232', None, 'http://www.kyrene.org', 'https://www.aqualineplumberselectriciansacrepairscottsdaleaz.com', None, '1592180244', None, None, 'https://bananarepublic.gap.com', 'http://www.artisticintegrity.ca', 'http://www.plumberphoenixboydco.com', '1592180257', 'http://www.goldengeckocoffee.com', None, 'https://www.kfc.ca/store/3199-blvd-taschereau-greenfiled-pk', 'http://www.carolynannryan.com', 'http://www.hyperproductionsdjs.com', 'http://www.menchies.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.coldstonecreamery.com/stores/20030', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.globalpetfoods.com', 'https://www.nbc.ca', None, 'https://www.maids.com/106/?utm_source=Yelp', 'http://www.preferredhomecare.com/', 'http://www.lighthouseNMT.com', None, 'http://www.coiffeurssalon.com', None, 'http://www.ganleywestsidehyundai.com', 'http://www.marcsimonlaw.com', None, 'https://www.lamaithaicuisineaz.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=1943&localechange=1', None, 'http://www.currentdentallasvegas.com', 'http://www.coupebizzarre.com', None, 'http://hallfamilyfarm.com/', 'http://www.ohiyo68.com', 'http://www.knuckleheadstobacco.com', 'https://www.livingspaces.com/stores/phoenix', None, 'http://aldi.us/index_ENU_HTML.htm', 'https://www.prestigecleaners.com', '1592180331', 'http://www.allwashedwindowcleaning.com', 'http://www.chiropractichealthmatthews.com/', 'https://www.phxrisingfc.com/stadium-information', 'https://www.ritasice.com', 'http://logansroadhouse.com', 'http://bluetruckwindowcleaning.com', 'https://restaurants.applebees.com/az/tempe-88083.html?utm_campaign=Local&utm_medium=Display&utm_source=Yelp', 'http://moonlitetavern.com', 'http://www.fiveguys.com', None, 'http://www.HealthyOutlookFamilyMedicine.com', 'http://www.fresh52.com', 'http://www.meckneurology.com', 'https://www.erieinsurance.com/about-us/locations/charlotte', 'https://www.rosewoodcandlebar.com', 'http://www.lvzendog.com', None, 'http://www.centralazsupply.com', 'http://www.costco.com/warehouse-locations/henderson-nv-673.html', 'http://www.coldstonecreamery.com/stores/20030', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, None, 'https://www.revampextensions.com', 'https://breezemassage.wixsite.com/85301', 'http://www.costcutters.com', None, 'http://www.rodefshalom.org', 'http://www.haychiro.com', None, 'http://www.nailsupspahuntersville.com', 'http://fountainhillsbowling.com', '1592180403', 'http://azmrfixit.com/', 'https://probateattorneyvegas.com', 'http://www.siegelsuites.com/rent-apartment-las-vegas/paradise-las-vegas-apartments.php', None, None, 'http://www.shopthemensroom.com', None, None, '1592180420', 'http://www.lilbaci.com', 'http://www.toronto.ca/parks/prd/facilities/complex/1099/index.htm', None, None, 'https://ezautosaleslv.com', 'http://www.dmscshea.com', 'https://haircutmenbillgreenpittsburghpa.com', 'http://www.dentalcaretoronto.com', None, None, '1592180441', 'http://www.mirrorbombstudio.com', 'http://www.2020visionphotos.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YULMV', None, 'https://www.tgifridays.com/locations/1110/fridays-robinson-pennsylvania', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYZTS', 'http://www.annalakshmi.ca', 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://lvcraftshows.com', None, 'http://ktran1401.wix.com/pamperednailsalon', None, None, 'http://www.poolman.com', 'https://www.travelex.com/stores/phoenix-sky-harbour-international-airport-terminal-4-checkpoint-b-phoenix-az/7283', None, 'http://oldnavy.gap.com/customerService/store.do?storeid=6528', None, None, 'https://www.cantina15eleven.com', None, None, 'http://locations.truereligion.com/az/scottsdale/51049/', 'http://www.nolldentistry.com', 'http://www.annexales.com', 'http://dekra.us/en/nevada/las-vegas-6020-n-decatur-blvd/', 'http://www.swanksalon.info', 'https://visionsource-loepervisionteam.com', 'http://hoopssportsbar.com', 'http://www.ArgonPest.com', None, None, None, 'http://www.arizonainterlocks.com/tempe', 'http://www.MellonsPub.com', 'http://arichtreatments.com', 'http://www.deserttailsanimalclinic.com', 'http://www.billsbarandburger.com/venues/pittsburgh/', '1592180530', 'http://www.hcxcharlotte.com/carmel-commons-hair-salon/', 'https://bignons.com/s-tryon-location', '1592180537', 'https://nothingbundtcakes.com/bakery/NC/charlotte?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=charlotte_145', None, 'https://stjudesranch.org/', 'http://www.graciebarra.com/phoenix-az', 'https://www.outback.com/locations/oh/cuyahoga-falls', 'http://www.pizzapizza.ca', 'http://www.eatatfellinis.com/', 'http://www.santurbanospa.com', 'http://www.theartofshaving.com', None, None, 'http://www.sugarpeony.com', 'https://www.lifecelebrationpetcrematory.com', 'http://www.hassanphoto.com', 'https://www.littlejohnslawns.com/', 'http://www.tiptoptrough.com', None, None, None, 'https://www.sephora.com/happening/stores/pittsburgh-shadyside', 'http://clevelandcinemas.com/location/6347/Chagrin-Cinemas-Showtimes', 'http://www.lifetimeeyecarecenter.net', None, '1592180585', 'http://www.wetseal.com', 'http://www4.ncsu.edu/~jtdemars/theshakeshack.html/', 'http://www.usfries.com/', 'https://www.carx.com/madison/5633-odana-rd/', 'http://www.ihg.com/staybridge/hotels/us/en/madison/msnhc/hoteldetail', 'https://siennajewelsalon.com', 'https://andersonsdelivery.com', 'http://www.azwood.com/', None, '1592180608', 'https://www.sunbaraz.com', 'http://www.splishsplashpools.com', 'http://www.swisschalet.com', 'http://www.hutchsbicyclegarage.com', None, 'https://www.levinfurniture.com/furniture-store/oh/solon/6130-kruse-drive', 'http://waternicetempe.com', 'http://lafontanagelati.com/', None, 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-jomax?utm_source=yelp&utm_medium=local&utm_campaign=honorhealth-medical-group-jomax', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', '1592180634', 'http://www.mezes.com', 'https://www.crystalglass.ca/location/calgary-south', None, None, 'http://bikebarnaz.com/', None, None, 'http://www.tashabradyphotography.com', None, 'http://www.sunrisecarpetupholsterycleaning.com/', '1592180654', 'http://www.geniecarwash.com', 'http://www.phoenixlocksmithoncall.com', 'http://www.bronzfx.com', None, 'http://www.urbancurls416.com', 'http://www.einsteinbros.com/', 'https://www.chick-fil-a.com/surprisemarketplace', None, 'http://www.andrewblairs.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.caferio.com/', 'http://spencerstv.com', 'http://www.cuttlefishscottsdale.com/', 'https://www.terragreenhouses.com/terra-milton-garden-centre', 'http://www.performanceext.com', None, None, 'http://charlottenc.gov/cats/rail/lynx-blue-line/Pages/parkwood-station.aspx', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.cinemark.com/nevada/century-16-south-point-and-xd', 'http://www.bgchenderson.org', 'http://popeyeschicken.ca', None, 'http://www.shanghaiwokbyby.com', 'https://www.phoenixmechanicshop.com', 'http://www.mysichuanpalace.com', None, 'http://www.lowes.com', 'http://www.extremepestcontrolLasVegas.com', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', None, 'http://www.sidernobakeryllc.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'https://www.wolbertautobody.com', 'http://www.mscarolsgymkids.com', None, 'https://www.redlobster.com/locations/list/nc/concord/8012-concord-mills-blvd', 'http://www.crazydaisieslv.com', 'http://abbeyroseflorist.com', 'https://positive-movement-logistics.business.site', 'http://www.delaneyauction.com', None, 'http://www.freakstattooemporiumnv.com', 'https://www.oxifresh.com/locations/nc/charlotte-carpet-cleaning/?utm_source=yelp&utm_medium=social&promo_code=YP5000', None, None, None, 'http://www.alnoorkebab.com', None, 'http://www.avalondentistry.ca/', 'http://jwoodsonderm.com', None, None, 'http://www.palmcleaningsystems.com/wp/', None, None, None, 'http://www.innovativevapors.com', 'http://www.naturalbeautyhairboutique.com', 'http://www.bethelbarbershop.com', 'http://www.livescanlasvegas.com', None, 'https://hotelxtoronto.com/en/falcon-skybar.html', 'http://www.danielsdiscount.com', 'http://www.thesmilecreators.com', None, None, None, None, 'http://www.koolthairestaurant.com', None, None, 'http://www.cowboycooling.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', '1592180808', 'http://www.jimpeplinski.ca', 'https://www.avis.ca/car-rental/location/CAU/CA/PQ/Dollard-des-Ormeaux/PCQ', 'http://www.vasilis-yyc.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.avecpanacherestaurant.com', 'http://www.shell.com', 'https://www.creativekidslearningcenter.com/schools/las-vegas-nv-6705/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhancedprofile', None, None, '1592180834', 'http://www.eatrastapasta.ca', 'http://www.chmuseums.org', 'http://www.settlersridgepa.com/', 'http://www.rightgear.net', 'http://www.bodyrevolution.net', 'https://istanbulsofra.com', 'http://www.3riversartsfest.org/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/32331+North+Scottsdale+Road-Scottsdale-AZ-85262/storeid=7839?cid=ps_ylp_2020', None, None, 'http://www.nvcoin.com', 'http://specialtyrvaz.com', 'http://www.azultimatechiropractic.com', 'http://www.moldanddefectlawyer.com', None, 'http://www.ultrasmoothskin.com/', 'http://www.tbmelectriclasvegas.com/', None, None, None, 'http://www.imperialpalace.com/casinos/imperial-palace/restaurants-dining/ginseng-3-detail.html', 'http://www.ventureoutrvresort.com', None, None, 'http://smittysresidentialpainting.com', 'http://royalthaihakka.com', 'http://www.instantac.com', None, 'http://www.rawfitnesslv.com/summerlin.html', 'https://calgarymovers.net', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', None, None, 'http://www.cobrinhabjjlasvegas.com', 'http://www.rooterranger.com', 'https://www.ladym.com', 'http://www.doubledouble.ca', 'http://www.orleanscasino.com/entertain/bourbon-street-cabaret?disid=wmm_yelp_to_corporate_bourbonstreetcabaret_organiclink', None, 'http://pawncorp.com', '1592180934', 'http://www.visionexpress.ca', 'http://www.mashawiaz.com/', 'https://www.pearsoneyecare.com/location/superstition-springs-mall-mesa/', 'https://trivenisupermarket.com', 'https://leader-barber-style-shop.business.site', 'http://www.unclesmoke.com', None, 'https://www.lasvegasbouquet.com', None, 'http://sachisalon.com', 'http://www.bixelsstudios.com', 'http://www.hairsalonlulu.com', None, 'http://www.thriftypropane.com/', 'http://www.balloonatix.com', None, None, 'http://www.starbucks.com/store/9298/', 'http://mississaugaemission.ca', 'http://www.dirtyat1230.com', None, 'http://www.11BravoMobileAuto.com', 'http://www.lagoeastbank.com', None, 'http://www.vitabellasalonandspa.com/', None, 'http://www.climatechangeyyc.com', 'http://fingerprintingink.com', 'http://www.oudejansinteriors.com', None, 'http://www.japco.ca', None, '1592181015', 'http://www.explace.on.ca', 'http://www.kensautorepair-hub.com', 'https://shop.nordstrom.com/store-details/nordstrom-scottsdale-fashion-square?utm_source=yelp&utm_medium=organic&utm_campaign=fls&utm_content=380&utm_channel=lcl_brand_p&sp_source=yelp&sp_campaign=fls', 'http://www.sfagentonline.com?cmpid=FVUJ_BLM_0006', None, 'https://www.aqua-tots.com/ahwatukee/', 'https://myhoneybakedstore.com/charlotte/114', None, 'http://phoenix.gov/parks', None, None, None, 'http://www.dryaged.ca', None, 'http://www.cpk.com/Location/Details/Ross-Park', None, 'http://www.chavezboxinggym.com', 'http://www.curvyconsignments.com', None, None, 'http://www.swlegalgrp.com/', None, None, 'http://www.superstore.ca', None, 'http://www.luxor.com/amenities/amenities_the_spa.aspx', 'http://chucksroadhouse.com', None, None, 'https://www.restaurant-mesa.com', 'http://www.gap.com/products/westlake-oh-store-7181.jsp', 'http://www.gotitcoverednv.com', None, 'http://www.ihg.com/holidayinnexpress/hotels/us/en/mentor/mntoh/hoteldetail', None, None, None, None, 'http://nanadeesdiner.com', 'http://shopping.schubachstore.com/', None, None, 'https://aarons-ark-mobile-veterinary-services.business.site', None, 'http://www.eldoradocasino.com/dining.cfm', 'http://www.commutehome.com/', None, 'http://www.badgergymnastics.com', 'http://www.hightvet.com', None, 'https://www.wedgewoodweddings.com/venues/arizona/palm-valley?utm_source=yelp&utm_medium=social&utm_campaign=palm-valley-yelp', 'http://Inglewoodpizza.ca', 'http://www.gianteagle.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.boulangerielefournil.com/', 'https://www.walmart.com/store/4148/charlotte-nc/details', 'http://www.byrdspub.com', None, 'http://www.headwaystudio.com', 'http://www.bronzecafe.com', None, 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'https://www.choicehotels.com/arizona/phoenix/sleep-inn-hotels/az140/rates', 'http://www.alcornhair.com', 'http://www.topshotlasvegas.com', None, 'http://www.bigmouthluckdumplinghouse.com/', 'http://www.matsudasushi.com/', None, '1592181177', 'http://nile-seafood-market-restaurant.business.site', 'http://versatileautoappearance.com', 'http://www.pizzahut.com/', 'http://www.walmart.ca/en/richmond-hill-supercentre/3195', None, 'https://attorneysforfreedom.com', 'https://havensmedispa.com', 'http://www.bannerhealth.com/locations/glendale/banner-physical-therapy-eugie?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', 'http://www.allproawninglv.com', None, 'http://www.woodbinetoyota.ca', None, 'http://www.ihg.com/holidayinnexpress/hotels/us/en/calgary/yyccg/hoteldetail', 'https://sukiesbridal.com', None, 'http://green-pepper-ot-xanh-asian-cuisine.business.site', 'http://www.sodadispenserdepot.com', 'http://www.7-eleven.com/?yelp=36121', 'http://www.medusaslv.com', 'http://www.cozzeria-pizzeria.com', 'https://www.alanhuirealty.com', 'http://www.clevelanderbar.com/', 'http://www.streamersicecream.com/', 'http://www.fraziersbarbershop.com', None, 'http://www.mufflermasters.com', None, 'https://functionbar.ca', 'http://worldsalon.ca', 'http://www.PetLossAtHome.com', 'http://www.summerlindance.com/', 'http://voxcurvy.com', 'http://www.paintingbyhector.com', None, None, 'https://marsautoservice.net', 'https://www.wyndhamhotels.com/hotel/08797?tel=18007606483&iata=00093785&cid=fe:tl:20170523:yl:pp:tlus:08797', None, 'https://locations.paychex.com/us/pennsylvania/pittsburgh', 'http://www.ymcacharlotte.org', 'https://www.chandlerair.com', 'http://www.poshmommy.ca', '1592181276', None, 'http://www.beancurd.biz', 'http://hummusfactorylv.com', 'http://gilberttowncommons.com/?utm_campaign=YelpWebsite&utm_medium=YelpBusinessProfile&utm_source=Yelp', 'http://www.starloancenters.com', 'http://www.pnc.com', 'http://skincarecleveland.com/index.html', None, 'https://www.kalologie-lasvegas.com', None, None, 'http://www.westlibertycycles.com', 'http://www.marriott.com/hotels/travel/phxcs-springhill-suites-phoenix-chandler-fashion-center/', 'http://Chagrin-Falls.certapro.com/?source=Yelp', 'http://www.keepitcut.com', 'https://www.lugayfamilydental.com', 'http://k-momo.com', 'http://www.executivelounges.com', 'http://www.trulyengravables.com', 'http://www.ecodryrestoration.com', 'https://www.target.com/sl/arcadia-crossing/950', 'http://www.hobbylobby.com', 'http://www.hubnest.com/', 'http://fandombar.com', None, None, None, 'http://www.gianteagle.com', 'https://www.lasvegas-propertymanagement.com', 'http://www.boxheartgallery.com', 'http://corneaaz.com', 'http://www.aveda.com', 'http://www.dralbertli.com', 'http://www.thegatewaybarandgrill.com/', 'http://www.meckabc.com', None, 'http://sugarshackrecords.net', 'http://www.starbucks.com/store/89998/ca/vmr-lucerne/2362-chemin-lucerne-ville-mont-royal-qc-h3r-2k5', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', 'http://www.bennysbar.net', 'http://www.ringaudio.com', None, 'http://dlautomotive.com', 'http://401dixieinfiniti.ca/', 'http://www.operationhalloween.com', 'http://www.fetchpetcare.com', 'http://bluebirdcarpet.com', 'http://auclv.vegas', 'http://www.coachcanada.com/coachcanada/ss.casino.asp', 'http://www.novellenail.com', None, 'https://ad.doubleclick.net/ddm/clk/414776926;215768644;y', 'https://www.apexskin.com/locations/solon-oh-dermatology-office/', 'https://vyiannibas.wixsite.com/juliana', '1592181407', None, 'https://ca.louisvuitton.com/fra-ca/magasin/canada/louis-vuitton-montreal-ogilvy?campaign=b_yelp/storepage', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700007898397760&ds_s_kwgid=58700000649767852&ds_e_adid=%7Bcreative%7D&ds_url_v=2&cmpid=ADV_PG_%5B*KeywordID*%5D', 'https://Scottsdalequarter.com', 'http://www.healthpalace.ca', 'http://www.aamcogilbertchandler.com', None, 'https://www.hungryhowies.com/store/hungry-howies-01823?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1823&utm_content=Website_Link', None, 'http://www.generate.to', None, 'http://campagnolotoronto.com/', '1592181442', 'https://www.24hourfitness.com/gyms/las-vegas-nv/silverado-las-vegas-sport', 'http://www.captainscovenc.net/', 'http://sanctuaryongreen.com/', 'http://www.nulawn.net', 'https://www.simonmed.com/locations/all/simonmed_imaging___greenfield', None, None, None, 'http://www.papajohns.com', None, 'http://restaurantgalleria.ca/fr/', None, 'http://www.sbcfoods.com/', 'https://freshfixofheights.com', 'http://www.myautoamerica.com', None, 'http://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEEM', 'http://www.fonda-las-cazuelas.com', 'http://zaxe.ca', 'http://www.jmphoenixcriminalattorney.com/', None, 'http://www.aztrailstravel.com', 'http://www.allstartaxi.ca', 'http://www.autobell.com/ab_locations/333-n-new-hope-rd/', 'https://stopandshop.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.bergamotmassage.com', 'http://rossispizzamonona.com/', 'http://www.lakeeriemonsters.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89128/027192/', 'http://go.georgestreetphoto.com/l/9752/2014-06-04/m1c3t', None, 'http://www.danspoolcare.com', 'http://www.earlycollegenv.com', 'https://www.oliversplumbing.com', 'https://www.lowes.com/store/AZ-Surprise/1607?cm_mmc=YEXT-_-SiteLink-_-1607-_-Lowes', 'https://stores.guitarcenter.com/avondale?source=4GOA5AJBB', 'https://www.perfectteeth.com/office/az/surprise/85379/waddell/', 'http://www.conciergemakeup.com', None, 'http://soilextractors.com', 'http://greengarageaz.com', '1592181555', None, 'https://www.apparelprousa.com', None, 'http://www.adamahmadison.com', None, None, 'http://www.dieselworldwide.ca', 'http://heritagetattoolv.com', None, 'http://www.phoeniximprovfestival.com', None, 'https://www.cosmeticlasercenters.com', None, 'http://www.zapitwash.com', 'http://therocksportsbarandgrille.com', 'https://www.sprint.com/storefronts/bd/sprint-phoenix-az-85037-d-4610/', None, 'http://www.taverninthehills.com', 'https://dream-cut-barber-and-stylist.business.site', None, None, 'http://www.solsticesunglasses.com/', None, None, 'http://www.patisseriefleur.com', '1592181622', 'http://locations.michaels.com/AB/CALGARY/3902/', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreID=62', 'http://www.kristenmarieweddings.com', 'https://www.eatnpark.com', 'http://www.middletonpower.com/', 'http://www.lakeviewautopartsinc.com', 'http://www.avedamontreal-lifestyle.com', 'https://www.laplazataqueria.com', 'http://discountfloorsaz.com', 'https://www.bannerhealth.com/locations/mesa/banner-urgent-care-1', 'https://www.shopfuego.com', None, 'https://www.labl.ca', None, None, 'http://www.cortezindustriesllc.com', None, 'http://www.skilletsforbreakfast.com/', None, None, 'http://lvfurnituredirect.com', 'https://www.petalsofloveflorist.com', 'https://www.aairofcharlotte.com', 'http://launchhouse.com/', None, None, 'http://www.friendlys.com/', None, 'http://american1stheating.com', 'http://chase.com', '1592181697', 'http://www.bcbg.com', 'http://www.signaturerentalsbyowner.com', 'http://www.traditionalstitches.com/', 'https://www.supercuts.com/locations/az/peoria/north-valley-power-center-haircuts-82958.html', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.mixxiemixologist.com', 'http://carolinawinesandspirits.com', None, 'http://www.altf.com', 'http://www.classiccarspa.com', None, 'https://palmbeachtan.com/locations/AZ/Tempe-East-Broadway-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://www.trustedepa.com', None, '1592181735', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', '1592181744', 'https://www.activedecorating.com', 'http://www.TheWomansCenter.com', None, 'http://moderneraweddings.com', 'http://www.mrdetailohio.com', 'http://winginout.com', 'http://www.bigotires.com', 'https://superiorplay.com/dealerlocator/north-carolina/charlotte', 'http://www.bk.com', 'http://www.lasvegasvasectomy.com', 'http://www.wildbunchdesertguides.com', '1592181773', None, None, 'http://avonurgentcare.net/', None, None, 'http://www.mimiscafe.com', 'http://tenrenstea.com/start/', None, 'https://santafestation.sclv.com/gaming/sports-book?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.look-stylesociety.com', 'http://www.callsp.com', 'http://www.datadoctors.com/locations/101', 'https://cbdemporium.com', 'http://www.ledolci.com/', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.marriott.com/hotels/hotel-information/fitness-spa-services/details/phxcb-jw-marriott-scottsdale-camelback-inn-resort-and-spa/5014642/', 'http://www.cocokala-lasvegas.com', None, None, None, '1592181829', None, 'http://www.twintranslations.com', 'http://www.target.ca', None, None, 'http://www.proximityatpapago.com/index.aspx', 'https://www.massageaddict.ca/calgary-sagehillcrossing/', 'http://www.donaldjpliner.com', None, 'http://bombatacos.com/fairlawn-oh', 'https://www.vitbeo.com', 'http://lanierparking.com/', 'http://SellWithOne.com', 'http://www.phoenixtitleloan.com/', 'https://www.eyesonhayden.com', 'http://CATCh.theater', 'http://www.goldenharboronline.com', 'https://www.kohls.com/stores/az/gilbert-621.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.nuturfofarizona.com', 'http://www.topgearcarwash.net', 'http://www.sansotei.com', None, 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', None, None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.bestbuyflooringcenter.com', 'http://www.flowerchildvintage.com/', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', None, 'http://www.theglassguru.com/scottsdale/', 'http://www.clevelandrockgym.com', 'https://jessebrowns.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.colossalcupcakes.com', 'http://www.phoenixpediatrics.com', 'http://www.klearlykleaned.com', 'http://811kitchenbarlounge.com', 'http://www.blacklanterntattoo.com', None, 'http://www.dedicatedaudio.com', 'http://www.flipsideburger.com', None, 'http://www.teamorthodontics.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.appleone.com/?yelp_track', 'http://www.confectionarycupboard.com/', 'http://www.neworientalrecipes.com', None, None, None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', None, 'http://www.beckhamspub.com', 'http://www.7-eleven.com/?yelp=29644', 'http://www.goldentrianglebike.com', None, 'http://www.wildsagert8.com/', None, 'https://www.grifolsplasma.com/en/-/mesa-az', 'http://www.handsoncharlotte.org', 'https://blackbeardiner.com/location/las-vegas-tropicana/', 'http://www.resumerick.com', 'https://locations.davidstea.com/on-thornhill-17', 'http://www.trinitybellwoods.ca/', 'http://www.threes-company.com', None, 'http://www.braidyjewelers.com', 'https://acrirlty.com', None, 'http://popeyeschicken.ca', 'http://www.aactionmovers.com', 'http://www.lovhairbymarianne.com', 'http://www.pinklime.com', 'http://www.hanshiktaco.com/', 'http://www.littlecaesars.com/', 'https://local.firestonecompleteautocare.com/nevada/las-vegas/3770-s-maryland-parkway/', None, None, 'http://www.precisionguitaraz.com', 'http://www.elitehomeinspections.us', None, 'http://www.vantageloftslv.com/#Home', None, 'https://makeupbyalmah.business.site', 'http://www.salutelounge.ca', 'https://arizonaurologyspecialists.com/contact-scottsdale-thompson-peak', 'http://www.starbucks.com/store/11823/', 'http://www.azmobilereflash.com', 'http://www.baskinrobbins.com', None, 'http://www.fye.com', 'http://www.ampm.com', None, None, 'http://www.nygh.on.ca', 'https://sheridannurseries.com/store-locations/toronto-garden-centre/', None, 'http://www.weathervaneplayhouse.com', None, 'http://www.littleotterswim.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.ebtfloral.com/', 'https://bostonpizza.com/en/locations/sunridge.html', None, 'http://www.sayteaonbloor.com', 'http://www.anewtable.ca', 'http://www.bathurstbowlerama.com/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/109+Dabney+Drive-Henderson-NC-27536/storeid=7055?cid=ps_ylp_2020', 'http://www.lesleyscreativecakes.com', None, None, 'http://www.gandermountain.com', 'https://housedoctoraz.com', 'http://www.greatclips.com/', None, None, 'https://locations.cariboucoffee.com/az/phoenix/2080-w.-northern-ave.html', None, 'http://oriondetailing.co', 'https://business.google.com/website_shared/launch_bw.html?f=vegas-noodle-house.business.site', None, 'http://www.southernpawsanimalhospital.com', 'http://www.pnpkaz.com', 'http://www.eventprostrategies.com', 'http://www.penn-station.com/westvirginia/', '1592182138', None, 'http://www.bohme.com', 'http://www.fatzombietattoo.com', 'http://www.globalmarketretail.com', None, None, 'https://www.wyndhamhotels.com/hotel/28120?tel=18885951878&iata=00093785&cid=fe:mt:20180223:yl:pp:MTUS:28120', '1592182158', 'https://www.amctheatres.com/movie-theatres/phoenix/amc-westgate-20?utm_uid=yelp', None, 'http://Www.piedmontvets.com', 'http://www.chachago.ca', 'http://rebelluxeaz.com', 'https://www.kingdomplumbing.com', None, 'http://www.sincitypc.com', 'http://www.weprintwhatyouwant.com', None, 'https://ballantynecleaners.com/drop-store', None, None, 'http://www.fastfreshfoods.ca/', 'https://comfortcare.ca', 'http://www.calgary.ca/CSPS/Parks/Pages/Locations/SW-parks/North-Glenmore-Park.aspx', None, 'http://www.dontbebroke.com/?utm_source=Yelp&utm_medium=Location%2BPage&utm_campaign=Yelp', 'http://bobochinabuffet.com/', '1592182216', None, 'http://www.quiznos.com', 'http://www.bolocakery.com', 'http://benvenutos.com/locations/middleton', 'http://yourvalleyproperty.com', None, None, None, 'http://www.mehowey.com', 'https://www.alfredoshairsalon.com', None, 'http://www.flooptical.com/index.html', 'http://www.fineskincare.ca', 'http://www.iautocars.com', None, None, None, 'http://www.bk.com', '1592182267', 'https://www.walgreens.com/locator/walgreens-1801+philo+rd-urbana-il-61802/id=11368', None, 'http://www.closetworld.com', 'http://www.chicsincity.com', 'http://www.terribleherbst.com/', 'http://www.altardstate.com/', None, None, 'http://www.madisonpubliclibrary.org/monroe-street', None, 'http://www.corekitchenaz.com', None, 'https://stores.jcpenney.com/nv/henderson/clothing-stores-henderson-nv-2626.html', 'https://gymvmt.com', 'http://www.nynyhotelcasino.com/entertainment/bars/bar-at-times-square.aspx', 'http://www.symdonchevy.com/?cs:pro=repman&cs:e=yelp', 'http://www.dgapplianceandair.com', None, None, None, None, None, None, 'http://imageeyecarenv.com', 'http://www.mondaynightauction.com/', None, None, 'http://www.pool-spaoutlet.com', 'https://ameritekusa.net', 'http://www.nepps.net/contact-us', None, 'http://www.diamondcharlotte.com/', 'http://www.caferumba.net/', 'http://www.libertystationtavern.com', 'https://surrealnightlife.com/las-vegas/', None, None, 'http://7eleven.ca/store-locator/33259', 'http://www.thestretchdoc.com', None, 'http://www.drumandmonkey.ca/', None, 'http://www.thelivermorepgh.com', None, 'http://www.marriott.com/hotels/travel/pitel-springhill-suites-pittsburgh-bakery-square/', 'https://gelsbystormi.com', 'http://www.starbucks.com/store/9298/', 'http://www.magicwindow.ca', 'https://www.wellsfargo.com', None, 'https://www.marcosorder.com/?store=3582', None, 'http://yorktaps.ca', 'https://www.lasikmd.com/fr?utm_source=yelp&utm_medium=cpc&utm_campaign=calgary&utm_content=general-link&p_utc=YELP_paid', 'https://www.extraspace.com/storage/facilities/us/nevada/north_las_vegas/300030/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, None, 'http://www.fleetfeetcleveland.com', 'https://www.walgreens.com/locator/walgreens-8816+e+pinnacle+peak+rd-scottsdale-az-85255/id=3602', None, None, 'https://local.lenscrafters.ca/on/thornhill/one-promenade-circle.html', None, "https://www.brueggers.com/?utm_source=yelp&utm_medium=homepage&utm_campaign=mayfield_heights&utm_term=Bruegger's%20Bagels%20Home%20Page", 'https://locations.wendys.com/united-states/al/madison/8814-hwy-20-w', None, 'https://www.bathdepot.ca/bath-depot-vaughan', 'http://www.lupitafernandezins.com?cmpid=d92d_blm_0006', 'http://www.aaatowing.ca', None, 'http://www.gilbertdentalcenter.com', 'http://fix8fitnesscafe.com/', 'http://www.jaroblue.com', 'http://fairwayindependentmc.com/', 'http://www.pittsburghwinery.com', 'https://www.wow1day.com/locations/house-painting-york?utm_campaign=yelp&utm_source=york&utm_medium=ubl', None, 'https://5starautocol.com', 'http://www.metrolinatire.com', None, None, 'http://palmerslawncare.net', None, None, None, 'http://www.tiffanymaidservice.com', 'http://www.reallysmooth.com', 'http://studio-kim.business.site', None, None, 'http://www.phohoa.com/', None, 'http://softmechanic.com', 'http://mastairaz.com', None, 'http://robertostacoshop.com', 'https://www.sonoraquest.com', 'http://www.dollar.com/Locations/gen.aspx?locationId=YTOC18', 'http://www.cpllabs.com/locations/patient-service-centers/cheyenne-psc.aspx', 'http://www.nguyenhuong.ca/menus.htm', 'http://www.fashionq1982.com', None, 'http://www.maddenbros.com', '1592182541', None, 'http://TresaApts.com', 'http://www.sanriva.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281379494&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, 'http://www.chevron.com', 'http://www.ruelo.com', 'http://burgerking.ca', None, None, 'https://www.uptowncleveland.com', None, 'http://www.dollartree.com/home.jsp', None, 'http://aerosportsparks.ca/brampton/', 'http://hummingbirddental.ca/', 'http://www.pghtours.com', '1592182590', 'http://www.amigotireslv.com', 'http://www.majesticmemorieslv.com/', 'http://www.eliteappliance.ca/', 'http://www.timhortons.com/', 'http://www.absolute-offroad.com', 'http://locations.harristeeter.com/nc/charlotte/205/', 'http://www.deanshousestrongsville.com', 'http://www.sushiakaihana.com', None, None, 'http://www.sunocoinc.com', None, 'http://lvmpd.com/', 'http://lvdancesport.com', 'http://blasioleseatery.com', 'http://qualitycarstereoaz.com', 'http://www.sambucagrill.com', None, 'http://manhattandelimentor.com/', None, 'http://www.rockler.com/retail/stores/pa/pittsburgh-store', 'http://www.taxcomexicangrill.com', 'http://www.mobilespaelite.com/', 'https://www.walgreens.com/locator/walgreens-8500+w+cheyenne+ave-las+vegas-nv-89129/id=4197', 'http://www.bkreflex.com', 'http://Www.haringtonsplumbing.com', 'http://rjcrosby.com', None, 'http://www.coremotus.com/book-now', None, 'http://www.mathnasium.com/ocotillo', 'https://www.cleaneatz.com', None, 'http://www.gunsmithlasvegas.com', '1592182676', '1592182679', 'http://www.wachovia.com', 'http://www.secondcup.com', 'http://www.thelezzet.com', None, 'http://www.polishfestivalaz.org/', 'http://www.cmkpool.com', 'http://www.anniesgourmetitalian.info', None, 'http://www.cannabispatientcenter.com/', 'http://www.traderjoes.com', 'http://www.abcvalet.com', 'http://www.popshoppinautorepair.com', 'http://www.gnc.com', 'http://www.boldbeautyco.com', 'https://stores.thenorthface.com/WI/MADISON/USA24/', 'https://www.bannerhealth.com/locations/phoenix/banner-university-medical-center-internal-medicine', None, 'http://kimpovietnamese.com', 'http://familysbarbershop.com', 'http://www.olio.ca', None, 'http://www.dominos.com', 'http://onestopnutrition.com/', 'http://www.platinumcars.ca', 'http://www.fantasticfleamarket.com/', 'http://www.sapporopgh.com/', 'http://www.uniquedentalaz.com', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', None, None, 'http://www.theyogurtbaraz.com', 'http://www.papajohns.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/7499+West+Bethany+Home+Road+Corner+Of+75th-Glendale-AZ-85303/storeid=7872?cid=ps_ylp_2020', 'https://www.alohacleanersaz.com', 'https://www.mingsbubbletea.com', 'http://www.gokartslasvegas.com', 'https://www.brakemasters.com/location/5180-w-peoria-glendale-az/', '1592182778', None, 'http://www.cocochocolates.ca', 'http://www.showcaselandcare.com', None, 'http://www.bluewaveautospa.com', 'http://starfishandcoffeeshop.com', 'http://cohoots.com', None, 'http://shop.trisports.com/tempe', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975370&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.nhmintviewobgyn.org/', None, 'http://www.stewartdetailing.com', None, 'http://bsalon11.com', 'http://www.northcoastlaser.com/', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7781&localechange=1', None, 'http://www.starbucks.com/store/9334/', 'http://kasianchiro.com', None, None, None, 'http://rtcsnv.com', None, 'http://monkeysushi.ca', '1592182939', 'https://www.bellagio.com/en/restaurants/sadelles.html', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'http://www.siegelsuites.com', 'http://www.bankofamerica.com', 'http://www.sloppykisseslv.com/', None, None, 'http://anthemhillsmusic.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.eliterepeatconsign.com/', 'https://www.zoup.com', 'http://thegymsportsbars.com', 'https://blackbeardiner.com/location/las-vegas-tropicana/', None, None, None, None, None, 'http://www.pachamamadc.com', 'http://www.econrads.com', '1592182988', None, 'https://www.jerseymikes.com/3128', 'http://www.indacutbarbershop.com/', 'http://www.pdubsbbq.com', 'http://www.bonaterrapgh.com', 'http://www.cessiespizza.com/', 'http://www.mark-taylor.com/arizona/ravenwood-heights/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.greatermatthewshabitat.org', None, 'http://www.modernchairrental.com', 'http://www.rhbarkleyhvac.com', None, None, 'http://www.riolasvegas.com/casinos/rio/restaurants-dining/hamadas-asiana-detail.html', 'http://accentricsalons.com', None, None, 'https://www.foodsocietyx.com', 'http://www.kay.com', 'https://www.eglintonsquare.ca', 'http://www.nicholascoffee.net', 'http://pinevilleplace.com', 'http://www.furry-pals.com', None, 'http://www.fantasticfades.ca', 'http://stratospherehotel.com/Food-Drink/Roxys-diner', 'http://www.villapizza.com', 'http://phoenixshoulderandknee.com/', '1592183053', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.sjnc.org/', '1592183060', None, 'http://fixautousa.com/las-vegas-north', None, 'http://www.lasvegas-cars.com', 'http://www.brimhallwellnesscenter.com', 'http://www.english.fgs2.ca/', None, None, 'https://locations.churchs.com/nc-charlotte-598', 'http://www.europeanautosalon.com', 'https://www.interdent.com/gentle-dental/locations/az/peoria/14155-n-83rd-ave-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.secondheartyoga.com', None, 'https://www.infinitiofbrampton.com', None, 'https://www.jlrichardscatering.com', 'https://madison.madhouseexperience.com', 'http://www.tempeuniversityinn.com', None, 'http://www.andersonaudiology.com', 'http://www.fionnmaccools.com', None, 'http://www.gelogisticsolutions.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.hairclub.com/locations/on/mississauga/hair-loss-clinic-228/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_228', 'http://www.hotyogatnt.com', 'https://locations.theupsstore.com/pa/mcmurray/4017-washington-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.cinemark.com/theatre-detail.aspx?node_id=97350', 'http://www.cuyahogalibrary.org/Branches/South-Euclid-Lyndhurst.aspx', 'http://www.wepoundemfishing.com', 'http://www.serviceking.com/locations/service-king-puritas?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'http://www.alsautorepair.mechanicnet.com', 'http://novabellasalon.mysalononline.com', 'http://www.cleatsinc.com/', 'https://phoenix.legacytraditional.org', 'https://beautyboxbyella.business.site', 'http://costcutters.com/salondetail/default.asp?salonid=62180', 'https://www.lafolieresto.ca', None, 'http://www.costco.com/WarehouseLocatorDetailsView?warehouseNumber=1121&storeId=10301&catalogId=10701', 'http://lakesidegrillestrella.com', 'http://www.barrecleveland.com/', 'http://www.cafemoka.us', 'http://www.midas.com/lasvegas/store.aspx?shopnum=1192&dmanum=480', 'http://www.verizonwireless.com/stores/arizona/tempe/tempe-marketplace-1088402/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://junskoreanlv.com/', 'http://www.smoothieking.com', 'http://restorecoldpressed.com', None, 'https://www.choicehotels.com/CN280?mc=blypypaf', 'http://www.earlyeducationpros.org', 'https://www.milansalonaz.com', 'http://www.tappsbar.com/', None, 'http://www.makisushi.ca', 'http://vibrantcare.com', None, 'http://www.marshallscanada.ca/en/directions.aspx?sid=741', 'http://www.thornhillskinclinic.com', 'http://www.union.lib.nc.us', '1592183182', 'http://buysellrent702.com', None, 'http://www.sugarshackaz.com', 'http://www.pizzahotwings.com', 'https://www.pizza73.com', 'http://allpages.com/', 'http://hairtiquedesignersinc.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.hunkmansion.com', 'http://www.nellos-pizza.com/', None, None, 'http://www.andrewleohairsalon.com', None, 'http://www.fullhousedesserts.ca', 'https://www.symphonywed.com', 'http://www.homesteadkennels.com', None, None, 'https://www.lasvegaseliteoffroad.com', None, None, None, 'http://www.dogworksmpg.com', 'http://www.rosarium.ca', 'http://www.piecesunimagined.com/', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', None, 'http://www.groundsguys.com/madison', 'http://www.lovetowax.com', 'https://nailuxx-nail-salon.business.site', 'http://www.mooreinjurylaw.com', 'http://myfriendsrestaurant.com/site/', None, 'https://www.tillys.com/store?StoreID=0045', 'https://www.mosquito-authority.com/locations/scottsdale-az/', 'http://www.thesteammasters.com', 'https://www.thetreesurgeonpa.com', 'http://www.albertsgifts.com', None, None, 'https://www.usps.com/', 'http://www.kingorthodontics.com', None, None, 'http://tailsandscales.ca/', '1592183275', None, 'http://www.papajohns.com/', 'http://www.buonafurniture.ca', 'https://local.biglots.com/ky/henderson/190?zcp=pd_local_yelp', 'http://www.freshlysqueezed.ca', None, 'https://stores.nordstromrack.com/us/oh/westlake/30121-detroit-rd?utm_source=yelp&utm_medium=organic&utm_campaign=rack&utm_content=277&utm_channel=lcl_acq_p&sp_source=yelp&sp_campaign=rack', None, 'https://celebritytheatre.com', None, 'http://www.ryanhomes.com', 'http://www.bwmpetresort.com', 'http://www.diblasisbakery.com', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281380850&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.goldenspoon.com/', 'http://www.starrtreatments.com/', 'https://greatchoicechiro.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://cardenaslawgrouplv.com', 'http://www.coldstonecreamery.com/stores/20008', None, 'http://www.circlek.com/', 'http://www.vegasdiscountnutrition.net', 'https://islandpacificmarket.com/stores/listing/tropicana', 'http://www.madisonwindowtint.com', 'http://www.brueggers.com', None, 'http://giu.ca', None, None, 'http://www.paintingcontractorsohio.com', None, 'http://www.hmsmokeshop.com', 'http://www.sunsetboatrv.com', 'http://www.welderup.com', None, 'http://www.revolution-games.com', 'http://www.rocksidefamilydentalcare.com', 'http://www.accphx.com', 'https://locations.monro.com/pa/sewickley/209-ohio-river-blvd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'http://www.lumosarts.com', None, 'http://www.saswax.com', 'http://www.crutcherauto.net', 'http://www.puttersvegas.com', 'https://palmbeachtan.com/locations/AZ/Peoria-West-Thunderbird-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://clevelandhenna.com', None, None, 'http://www.sageandsandgrill.com', 'http://www.fillsgas.com', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', None, None, 'http://www.tonycordellphotography.com', '1592183388', None, 'http://www.cornerbarrestudio.com', None, 'http://www.bk.com', 'http://www.balletbarreworks.com', 'https://www.ghcscw.com/clinic/Hatchery%20Hill%20Clinic', 'https://www.la-z-boy.com', 'http://www.ecig-emporium.com', 'http://abcpoolservice.net', 'https://etobrestaurant.business.site', 'http://www.otherwideshoes.com', 'https://catchersfish.com', 'http://www.tigerkimstkd.com', 'https://www.take5oilchange.com/locations/oh/euclid-125/', None, 'http://www.esspa.net', 'http://www.allsewingmachinerepairs.com', 'https://www.sunrisepreschools.com/locations/broadway-mcclintock/?utm_source=yelp&utm_campaign=online_listing_tempe_274&utm_medium=referral&utm_term=business_url', None, 'http://planetzong.com/', 'https://www.choicehotels.com/pennsylvania/pittsburgh/comfort-inn-hotels/pa354/rates', 'http://www.starbucks.com/store/8054/', 'http://www.pillarcrowncourt.com', None, 'http://www.greenvalleyranchresort.com/dining/sushi-sake.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://CallAdvancedPro.com', 'https://www.bagelnash.ca', 'https://www.homedepot.com/l/Arrowhead-Ranch/AZ/Glendale/85308/475?cm_mmc=seo-yelp&', None, 'http://www.mattressfirm.com/', None, None, 'http://www.thepowerplant.org', '1592183453', 'http://www.donsbusbox.com', 'http://cheshirecataz.com/', 'http://www.dentistinstow.com', 'http://www.antiquecenterofstrabane.com', 'http://www.pointmanagementgroupllc.com', 'http://brightonevents.ca', 'http://www.autorepairaz.com', 'http://www.timelesshealthclinic.com', None, 'https://www.flyswoop.com', 'http://southpointcasino.com/dining/michaels-gourmet-room?cid=ypmic', None, None, None, None, 'http://www.bedfordnissan.com/', '1592183486', None, 'http://www.planetbouncepgh.com', None, None, None, 'http://www.jrlighting.com', 'http://monsoongroup.ca', '1592183503', 'http://www.familydollar.com', 'http://www.sachisalon.com', None, 'https://www.stancatos.com', 'http://www.substationii.com', None, 'http://www.ccdgp.com/', 'http://uhaul.com', 'http://wienerschnitzel.com/', 'https://www.wherefamiliesgrow.com', 'http://www.corksplayhousesquare.com', 'http://www.tempedogs.com', None, 'http://www.blowit.ca', None, 'http://www.thewokker.com', None, 'http://www.gianteagle.com', 'http://www.calgarybikram.com', 'http://www.fortresscreditpro.com/', 'http://www.honeymee.com', 'http://schnitzel.ca/', 'http://www.freshens.com', None, None, 'http://www.squashblossomaz.com', 'http://www.saharacenterlv.com/', None, 'http://www.unitedmayflower.com?Cpgn=Yelp', None, None, None, 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', 'http://ahwatukeehoa.com', None, 'https://www.scenariopgh.com', 'https://www.carowinds.com/things-to-do/dining', None, None, 'https://www.petsmart.com/stores/us/az/glendale-store0157.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://fastfix.com', 'http://www.starbucks.com/store/11823/', 'http://www.shapirocho.com', 'https://www.hungryhowies.com/store/hungry-howies-01824?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1824&utm_content=Website_Link', None, 'http://www.metromoversgtanorth.ca', None, 'https://westernhonda.com', None, 'http://www.quiktrip.com', 'http://www.jeffspeakmanlv.com', 'http://www.fmentertainment.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=891&utm_content=bpas', 'https://www.gggflorist.net', None, None, None, 'http://www.birtsbistro.com', 'https://www.thepercantileandcreamery.com', 'http://www.geniusnetworkevents.com', 'https://www.esportafitness.com/Pages/clubhome.aspx?clubid=531', 'https://www.pinnaclemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.glazecraze.ca', None, 'http://adfarm.mediaplex.com/ad/ck/24347-208272-18214-3', 'http://charlotteplumbing.com', 'https://stores.loft.com/outlet/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'https://www.wellsfargo.com', None, 'http://www.guess.com', '1592183651', 'http://www.therosehairstudio.com', 'http://www.cookmortgagegroup.com', None, 'http://www.LasVegasSegway.com', 'http://bennettautocare.mynapatools.com', None, None, None, 'http://www.kfc.ca/store/27-ruth-ave-brampton', None, 'http://www.lcbo.com', 'http://88noodlepapalasvegas.cafescentral.xyz', 'http://www.endurancerehab.com', 'http://www.rennsportire.ca', 'http://ladydianascakes.com/', 'https://locations.wendys.com/united-states/oh/cleveland/13939-euclid-ave.', None, 'http://www.MDA-Dentist.com/', 'https://ad.doubleclick.net/ddm/clk/300113982;127047679;f', None, 'http://www.toucheh.com', 'http://www.deathsdoorspirits.com', 'https://www.appliancepartscompany.com', 'http://www.toronto.ca/parks/island/', 'https://www.cinemark.com/arizona/cinemark-mesa-16', 'http://www.madisoninternationalspeedway.com', 'http://www.koolkatpgh.com', 'http://www.appliancesrepaircalgary.ca', None, None, None, 'http://www.reno4men.com/', None, 'http://scottsdale.house.hyatt.com/en/hotel/home.html?src=corp_yelp_businesslisting_phxxs_en_homepage_desktop&code=YB', 'http://www.servicecanada.gc.ca/tbsc-fsco/sc-dsp.jsp?lang=eng&rc=4802', 'http://ginassalonandwigs.com', None, 'http://phxnightclub.com/', '1592183735', None, 'http://www.heersmanagement.com/properties/property/del-mar-terrace-apartments', 'https://www.minutesuites.com', 'http://www.urban-baker.ca/', 'http://www.cactuslv.com', 'http://www.kandagreenhouse.com', 'http://englishhedgerow.com', 'http://www.universalcupcoffee.com', 'https://www.carmax.com/stores/7197?adcode=YLP7197&utm_campaign=LG_AFF&utm_source=LG_AFF_YLP&utm_medium=local&utm_content=shop', None, 'https://locations.dignityhealth.org/st-joseph-s-westgate-medical-center-glendale-az', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1015+East+Ray+Road-Chandler-AZ-85225/storeid=7849?cid=ps_ylp_2020', None, None, 'http://yotapros.com/', 'https://www.giantmanhattanpizzaphoenix.com', None, None, 'http://www.masalakitchen.ca', 'http://www.zachmonroytraining.com/', None, 'http://www.livinggrace.net', 'http://homewoodsuites3.hilton.com/en/hotels/ontario/homewood-suites-by-hilton-toronto-markham-YYZHWHW/index.html?SEO_id=YELP-HW-YYZHWHW', 'https://stores.columbia.com/chandler-factory-store', 'http://www.wegotgamez1.ca', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, None, '1592183793', 'http://www.murphyappliance.ca', 'http://www.silkroadsushiandsteakhouse.com', 'http://www.mycdi.com/arizona/locations__services/', None, 'http://www.mypeacepreschool.com', 'https://www.inschoolwear.com', 'http://www.silverminesubs.com/', 'http://www.Arizonarvservice.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.saharavegas.com/food/HouseofLords/', 'http://www.applianceservicecentre.ca', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.lesolson.com', 'http://www.phoenixpublicmarket.com/', None, None, None, None, 'http://www.theprophysiques.com/#home-top', 'http://www.eta-az.com', 'https://www.pizza73.com', None, None, 'http://radiocitypizza.com', 'http://www.yaharabay.com', None, None, 'http://www.ontariocourts.on.ca/scj/en', None, 'http://www.primroseestates.com', None, 'http://www.zonacaffe.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.jcsgrillhouse.ca/markham/', 'http://www.womensspecialtycare.com', None, 'http://www.ipetgrooming.com', None, 'http://www.perkykettle.com', None, 'http://www.tazzyandboo.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://alorestaurant.com/', None, 'http://Vegasonlinerealty.com', None, 'http://www.cynthiaschinese.com', 'https://locations.traderjoes.com/az/gilbert/258/', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.bestfloorcoatingsaz.com', 'http://www.scottsdalememorycare.com', 'http://www.evans-properties.com', 'http://www.lidlounge.com', 'https://www.supercuts.com/locations/az/surprise/albertsons-shopping-center-haircuts-82334.html', 'http://www.apnihavelly.com/', 'http://www.swisschalet.com', None, None, 'http://www.yangteashop.com', '1592183914', None, 'https://www.originaljoes.ca', None, None, 'http://www.BASISgoodyear.org', 'https://www.goodyear.com/en-US/tire-shop/Phoenix+AZ+85016-6808+US/000121764', 'https://www.pizzahut.ca/store-details/store-R42601/pizza-hut-Stouffville', None, 'http://www.foodlion.com', 'http://www.kegsteakhouse.com/en/locations/ON/toronto/esplanade-keg/', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', 'http://www.patelbros.com', None, 'http://blomedry.com/locations/blo-strongsville-ohio/', 'http://stylehousebeautybar.com', 'http://calgarysbestpubs.com/joyce/', None, 'http://www.artbuscoffee.com', None, 'http://www.galleriasm.com/branch/store/th', 'http://www.morcandirect.com', None, 'http://daluhairdressing.com', None, 'http://www.theencorequartet.com', 'http://www.unclebenspawnshop.com', 'https://www.olivegarden.com/locations/az/phoenix/metro-center/4410?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:phoenix-az-4410_pl:locurl_rd:1754', 'https://www.daveandbusters.com/locations/canada-oakville', 'https://www.westerndental.com/en-us/find-a-location/arizona/phoenix/13651-n-35th-ave', None, 'http://cargilleyecare.com', 'https://www.charmecochinchine.com', 'http://locations.michaels.com/nc/charlotte/5032/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://womensbookstore.com/', None, None, 'http://www.stellamiawestlake.com', 'http://cahairsalons.com', None, 'http://www.svahayogacenter.com', None, 'https://www.choicehotels.com/north-carolina/pineville/quality-inn-hotels/nc384/rates', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.theartofvapes.com', 'https://www.jetspizza.com/stores/illinois/champaign/512-s-neil-st?utm_source=Yelp-Ads&utm_medium=Yelp-Ads-SEO&utm_campaign=IL-008&utm_content=Website', 'http://www.cactuscrossfit.com', 'http://www.safeway.ca', 'http://justdogsgourmet.com', None, 'http://www.studio480thesalon.com/', None, 'https://southcharlotteservices.org', None, 'http://btnfurniturerepair.com/', 'http://www.bojangles.com', 'http://www.mississauga.ca/portal/discover/braebengolfcourse', 'http://exclusivecustomslasvegas.com', 'http://www.somersfurniture.com', 'https://no900.com', None, None, None, None, None, 'https://www.stateandmain.ca', 'https://www.ttc.ca/Subway/Stations/Pape/station.jsp', 'http://www.washwag.com', 'http://www.ritasfranchises.com/ArcadiaTowneCenter', 'http://www.loova.com', 'http://www.xpandrealty.com', '1592184055', 'http://www.petagreemobilespa.com/', 'https://www.simplyravishingsalon.com', 'http://www.dreamtherapies.com/', None, 'http://www.perrinofurniture.com', 'https://www.guaranteedrate.com/loan-expert/joecordone', 'http://www.strudelteig.com', '1592184073', 'http://www.bremec.com/BremecOnTheHeights.htm', 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.arizonaroofingcompany.com/', None, 'http://www.anthropologie.com/', '1592184083', 'http://www.firstcu.coop', 'http://www.gristle.catering', 'http://www.hartswastedspace.com', 'https://www.chilis.com', 'http://www.elementalplumbing.com', 'http://www.northmountaingrille.com', None, 'http://www.meridiancu.ca', 'http://www.elitemoversofcleveland.com', 'http://www.jackalopejacks.com/', None, 'http://www.avenueliving.ca', None, 'https://www.hudsonsrestaurant.com', 'http://www.yojuicylife.com', 'http://bloorwestcosmetics.ca/', None, 'https://locations.nekterjuicebar.com/ll/US/AZ/Phoenix/4340-Indian-School-Rd_*-Ste_-19?utm_source=yelp&utm_medium=link&utm_campaign=Indian-School&utm_content=URL', 'http://www.vintagethai.ca', None, '1592184122', 'http://www.artistsuncorkd.com', None, 'http://www.tiedemannglobe.com', 'http://saveamuttkennel.info', None, 'http://www.bedsidemanor.com/', 'http://www.silversevenscasino.com/', None, 'http://www.lobbyrestaurant.com/', None, 'http://lindeyslakehouse.com', None, 'https://www.ubreakifix.com/ca/locations/thornhill', 'http://kwikkleaning.webs.com/', 'http://www.gemoftheearth.com', 'http://www.charlottelaserdentist.com', None, None, None, 'http://www.exquisitenailsandspa.ca/', None, 'http://www.jfcsaz.org/site/c.ajIQK6NMLfJ0E/b.6211177/k.2AE8/Contact_Us.htm', 'https://tabletennisandmore.com/phx/tabletennisandmore/', None, 'http://www.7-eleven.com/?yelp=22886', None, None, 'http://www.bodytherapyaz.com', None, 'http://www.westvalleypetsitting.com', None, 'http://www.capitano.ca', 'http://www.renusskincare.com', 'http://scorpionridge.com', 'http://www.cfcgiftcard.com', 'http://www.sandovalkarate.net', 'https://abrazomedicalgroup.com/amg-parkway/robert-l-bierman-md/', 'http://www.lifeisbeautifulsalon.com', 'http://www.upnsmokepitbbq.com', 'http://www.kensingtonmakeup.com', 'http://www.saryocafe.com', None, 'https://www.interdent.com/gentle-dental/locations/az/phoenix/4910-e-chandler-blvd-dentist-office?utm_source=organic&utm_medium=yext', None, 'http://www.reposesalonandspa.net', None, None, 'http://www.xingfutang.ca', None, None, 'http://www.challengerbasic.com', 'http://www.farrarjewelers.com/', 'http://www.oakandvine.ca', None, None, 'http://northernhillsdental.com', 'http://www.madvapesindianland.com', 'http://4sonsstores.com/29', None, 'http://www.cashamerica.com/SUPERPAWN-stores/NV/LASVEGAS/LV02', 'https://www.ubreakifix.com/ca/locations/avenueroad', None, 'https://www.hamiddonair.com', 'https://only-inspections.com', 'http://www.restauranttrentina.com/', None, 'https://www.petsuppliesplus.com/Store/SC/Fort-Mill/Fort-Mill/4189', None, 'http://www.skinnywimpmoving.com', 'http://www.auntieannes.com', None, 'http://www.toshoknifearts.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216608&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://wisconsinrepair.com', None, 'http://www.valleysolaraz.com', 'https://www.walgreens.com/locator/walgreens-1159+e+southern+ave-mesa-az-85204/id=3670', 'https://www.cineplex.com/Theatre/cineplex-cinemas-winston-churchill', 'http://www.peaksathleticclub.com', 'https://www.servproscottsdale.com', 'https://www.ventitobakery.com', 'http://www.northscottsdalehyundai.com/', None, 'http://www.metrofireaz.com', None, 'http://www.wingstop.com/', 'http://www.sallybeauty.com', None, None, 'http://hrhc.lv/PinkTaco', 'http://www.addisononerinmills.com/', None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'http://www.heartpathacupuncturemadison.com/', '1592184312', 'http://www.carolinaexpressclinic.com/', 'http://www.insuranceinwhiteoak.com?cmpid=ts6n_blm_0006', 'http://www.apachewellscountryclub.com', 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/mississauga-2191-dundas-street-e.aspx', None, 'http://www.bathandbodyworks.com', None, 'http://www.bestbuy.com', 'https://iisnv.com', 'https://www.topmasterlocksmith.com', 'http://www.lowes.ca', 'http://www.primrosehuntersville.com?utm_source=yelp&utm_medium=maplistings&utm_campaign=localmaps', 'https://stores.jcpenney.com/wi/madison/clothing-stores-madison-wi-0891.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-0891', 'http://www.mcdonalds.com', 'https://1stpetvet.com/locations/north-valley-az/', None, 'http://www.rxboilerroom.com', None, 'http://www.brakemasters.com', 'http://www.familydollar.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://liasgardenbanquethall.com', 'https://www.menswearhouse.com/store-locator/2505?utm_source=yelp&utm_medium=listing&utm_content=2505&utm_campaign=AZ', 'http://freshmexican.wix.com/hermanos', 'http://www.miramaestudio.com/#!services/c166e', None, None, 'http://www.harrisgrill.com', 'http://www.tiffany.ca/', 'http://www.saloonofmtlebanon.com', 'http://priorityonefit.com', None, 'http://www.miseryinktattoobodypiercing.com/', 'http://hongto.weebly.com/', 'https://solsouthwestkitchen.com/#locations', 'http://nexusrehabspine.com/phone/index.html', 'http://www.vivaladivalv.com', 'http://www.eastcoastweapons.com/', 'http://www.arizonaglasspros.com', None, 'https://www.fishertools.com', None, 'http://panzapiena.ca', None, 'http://www.sunocoinc.com', 'http://redwagondiner.com/', None, 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', '1592184420', None, 'http://local.firestonecompleteautocare.com/arizona/peoria/10661-n-84th-ave/?lw_cmp=IYP_YPC_MLP_11932', 'http://www.starbucks.com', 'http://www.thenailroomlv.com', 'http://www.nunziospizzeria.com', None, 'https://shopmarketandco.com/merchant/seed-to-sausage/', 'http://www.clarkcountynv.gov/Depts/parks/Services/pages/picnic-area-reservations.aspx', 'http://www.azstudentmovers.com', 'https://www.itcu.org/about/locations/green-valley', 'http://www.runningroom.com/hm/inside.php?id=2332', None, 'http://www.pareagyros.com', 'http://paladinautomotivelasvegas.com', 'http://automaticgaragedoorcompany.com', 'http://www.azcentral.com', 'http://www.gotourscanada.com', 'http://commercialcenterdistrict.com', 'https://thesaguaro.com/scottsdale/#hotel-spa', 'http://www.einsteinbros.com', 'https://www.desertstateinspections.com', 'http://local.firestonecompleteautocare.com/north-carolina/charlotte/5340-e-independence-blvd/?lw_cmp=IYP_YPC_MLP_10790', 'http://www.clarkcountynv.gov/Depts/parks/Pages/clark-county-museum.aspx', 'http://www.goodlifefitness.com', 'http://www.eSMIL.com', None, 'https://moxies.com/restaurant-locations/ontario/richmond-hill-beaver-creek', 'https://twomaidsgilbert.com', None, 'https://my.clevelandclinic.org/locations_directions/regional-locations/avon-lake-fhc', None, None, 'https://www.redlobster.ca/locations/list/on/toronto/20-dundas-street-west', 'http://www.dvfmaz.net', None, 'http://thevaporloft.com/', 'http://www.dineoncampus.com/UNCCharlotte/', 'http://www.celebrationsbarandgrill.com', 'http://heyrestaurant.tumblr.com/', 'http://www.dunkindonuts.com', None, None, None, None, 'http://www.clarkcountynv.gov/depts/parks/Pages/parkslocator.aspx', 'http://www.dillonsontopofcentral.com/', 'http://www.omegahomeimprovements.com', 'http://talleyproperties.com', None, 'http://www.chandelier-hair.com', 'http://www.serenityflorals.com', 'http://hbsalonnc.com', 'https://www.crawfordsautoservice.com', 'http://www.newsons.ca', 'https://www.tempevistabyavanti.com', None, 'http://www.leons.ca/', 'https://www.giuseppeson28th.com', 'https://diamondlaw.ca/toronto', None, None, 'https://www.greulichs.com', None, None, 'http://www.scottsdaleskininstitute.com/', 'https://germancarspecialist.com', 'https://locations.wendys.com/united-states/pa/swissvale/1703-s.-braddock-ave', 'http://www.greekvillagegrille.com/', 'http://www.neptunehairstyling.com', 'http://www.dollartree.com', None, 'http://www.trucksonlysales.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Calgary-AB-T2Z4M8/058072/', 'http://www.karenshairdesigns.com/', 'http://mothernaturesdiapers.com/', 'http://www.arizdental.com', 'http://www.johnnysonfulton.com', 'https://agents.allstate.com/adam-brown-las-vegas-nv-14515251.html', 'http://www.lasvegasdiscountbeds.com', 'http://www.cactuspools.com/', 'http://www.skytecsecurity.com/', 'http://paintingcontractorglendale.com', 'http://www.carters.com/home?id=carters&cm_mmc=Retail-_-Carters-_-Yelp-_-April', None, 'http://azempiredoors.com', 'https://www.millers.com', None, 'https://thegoatbarandgrill.com', None, 'https://davids-diner.business.site', 'http://www.clevelandart.org', None, 'http://theclevelandarcade.com/', 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', 'http://www.gap.com', 'http://www.janetscustomsewing.com', '1592184613', 'http://xtremecleanmobile.com', 'http://sfsourdougheatery.com/', 'http://www.randitas.com', 'http://steliotesdentalspa.com/', None, 'http://www.sushisushi.ca/', None, 'http://www.duzz.ca', None, None, 'https://arizona.weidner.com/apartments/az/phoenix/skyline-lofts-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'https://dealer.sprint.com/experts-choice', 'http://www.aztecatowingarizona.com', 'http://mannysdiscount.com', None, None, 'http://www.sterlingsteelecreek.com', 'https://www.nofrills.ca/store-locator/Dennis%27+nofrills+%28Erin+Mills+Parkway%29/0700?storeId=0700', 'http://www.libertyfordbrunswick.com/', 'http://pro-techtransmission.com', 'http://www.cozumelrestaurante.com', 'https://www.choicehotels.com/pennsylvania/pittsburgh/quality-inn-hotels/pa476?source=pmfyelp&pmf=yelp', 'http://www.scottmillerdds.com/', 'https://www.escapecampervans.com/locations/las-vegas-nevada/', 'http://automotiverepairphoenix.com', None, 'http://www.bathandbodyworks.com', None, None, None, 'http://www.a1qualitycare.com/', 'http://madisongaragedoorservices.com', 'http://local.firestonecompleteautocare.com/arizona/mesa/1553-s-crismon-rd/?lw_cmp=IYP_YPC_MLP_784290', 'http://petstellthetruth.com/', 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'https://www.dentistryatgreenway.com', 'http://duckyschampaign.com', 'http://www.paddyspourhouse.com', None, 'http://www.saffrontakeout.ca', 'http://www.overflowcafe.com', 'https://www.bankofamerica.com', 'http://msquarecoffeeco.com/', 'http://Clearqualityautoglass.com', 'http://www.nicolebodyworks.com', 'https://www.homedepot.com/l/Avon-Vail/CO/Avon/81620/1525?cm_mmc=seo-yelp&', '1592184707', None, None, 'http://99only.com/store/Phoenix---12th-St./', 'http://www.oasispediatrics.com', 'http://www.krispykreme.com', 'http://www.calgaryscrapautoremoval.com', None, None, None, 'https://jamesgoodnow.com', 'http://www.nps.gov/lake/index.htm', 'http://www.aiellospizzeriallc.com', None, 'https://local.fedex.com/az/glendale/office-1471/', 'http://pecoslawgroup.com', 'https://www.handlebarj.com', 'http://www.courtoneaz.com/', 'http://www.desertrad.com', 'http://trubliss.com', 'http://hotduckaz.com', 'http://www.starbucks.com/store/14887/', 'https://glassdoctor.com/henderson', 'http://www.momdocwfw.com/scottsdale', 'https://www.calforex.com/en/locations/foreign-exchange-vaughan-on-vaughan-mills/', 'http://www.truwest.org', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.designateddriversinc.com', None, 'https://f45training.com/southwestlasvegas/home', None, None, 'http://bellavitaitaliangrille.com/', None, 'http://www.mark-taylor.com/arizona/san-lagos/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.azhomedefense.com', '1592184786', None, 'http://www.hooters.com', 'http://www.atlccleaningcompanyllc.com', 'http://christinanailboutique.com', None, None, 'http://tamidairiki.md.com/', 'http://www.azallergy.com', 'http://www.DymeBoxing.com', None, 'http://www.toplimolasvegas.com', 'http://livingwellhomes.com/apartments/details/brookfield_homes', 'http://www.dermsolutionsaz.com/', 'https://wickenburg-alternative-medicine.business.site', 'http://www.lapetite.com/your-local-school/north-las-vegas-nv-7306', 'http://lasvegasstripweddings.com/', None, None, 'http://www.newamericanagent.com/steveschauer', 'http://www.ocean-prime.com/locations-menus/phoenix', None, 'http://tajindiancuisineaz.com', 'http://www.adidas.com/us/storefinder', None, '1592184835', 'http://ranchosecurity.com/', None, 'http://www.foodlion.com', 'http://www.thebeadery.ca', 'http://www.othermothersaz.com', None, 'http://www.esmokercanada.com', None, 'http://www.carmodysgrille.com', None, 'http://littlemacau.com/', 'http://AnimalCareClinicInc.com', None, None, None, 'https://www.hilton.com/en/hotels/phxruru-tru-gilbert/?SEO_id=YELP-RU-PHXRURU', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.tocasierraspa.com', 'http://www.buffalowildwings.com/?utm_source=yelp&utm_medium=yelp_profile&utm_campaign=yelp_locations', None, 'http://www.fortmcdowellcasino.com/home.php', 'http://www.pridebeautystudio.com', 'http://www.restaurantrouby.ca', 'http://sawasdeethaitogokitchen.com', 'https://mountainsidefitness.com/locations/carefree/', 'http://www.copps.com/', 'http://locations.saksoff5th.com/en/phoenix', 'http://www.canadiantire.ca/en/store-locator/ab/calgary-south-ab.store.html', 'http://www.hansonwater.com', 'http://www.gussdiner.com/', None, 'http://ladoniacafe.com/', '1592184905', None, None, 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/toronto-center-(1).aspx', 'https://www.floristinrockyriveroh.com/?srccode=yelp_track', 'http://www.menasgrill.com', 'https://gohoneyhut.com', 'https://footsolutions.com/phoenix/', None, None, None, 'http://www.minervaindiancuisine.com/', 'http://savoypgh.com/', None, 'http://www.peoples-gas.com', 'http://www.shopjustice.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'http://www.bollywood-bites.com', None, 'http://myazlawyers.com/', 'http://heartschoices.com/calgary-farmers-market/', None, 'http://www.dicey.biz', 'http://crazyburrito.ca', 'http://www.Montblanc.com', '1592184965', 'http://www.beadworldinc.com', 'http://www.lvcharleston.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, None, None, None, None, 'http://www.muchoburrito.com', 'http://www.pitaland.ca', None, None, None, 'http://www.rideabike.com', 'http://www.shicklaw.com', 'http://www.revivestainlesssteel.com', 'http://www.thebespokeman.com', 'http://www.cavecreeklock.com', 'http://www.chestermerelanding.com', 'https://www.canyonstateauto.com', 'http://6ixbar.com', 'https://www.regmovies.com/theaters/ua-showcase-theatre-8/C00542780062', None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7784&localechange=1', 'https://www.aldoshoes.com/us/en_US/store-locator/store/2103', 'http://www.kingsrestaurant.net', None, None, None, 'http://www.sarahhorne.org', None, 'https://www.rentvegas.org', 'https://cocktailantabli.com', 'http://dosgringosaz.com', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', None, None, None, 'https://www.fsequence.photography', 'http://www.petgroominglv.com', None, 'http://www.mississauga.ca/portal/residents/mississaugavalleylibrary', None, 'https://oakarizona.com', 'https://www.nightowlcontractors.com', None, 'http://robertostacoshop.com', 'http://gravady.com', 'http://www.humblepieusa.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=14057&localechange=1', None, 'http://www.farwellpools.com', 'http://www.bin110.com', 'http://www.crocs.com', 'http://tricitycardiology.com', 'http://www.larkspurcatclinic.com/', 'https://www.sunrisechildrenshospital.com', None, 'http://www.theroofdoctorlv.com', None, 'http://www.altacamelback.com', 'https://locations.timhortons.com/ca/on/oakville/494-dundas-st-w.html', 'https://www.currys.com', 'http://perfecthomefurniture.com', None, 'https://www.abraauto.com/abra-auto-location/1058-concord-nc', 'http://www.patricksimonedds.com', 'http://www.lvendocrinology.com', 'http://www.chandlerapthomes.com', 'http://www.desirableskinaz.com', 'http://www.flooringsurfacesinc.com', 'http://www.travelgolfcenter.com', None, 'https://www.jetblue.com', 'https://www.bahamabreeze.com/locations/nv/las-vegas/las-vegas/3020?cmpid=br:bb_ag:ie_ch:dry_ca:BBQ121LOCURL_dt:20200605_sn:yelp_gt:las-vegas-nv-3020_pl:locurl_rd:1013', 'http://www.weknowurban.com/Phoenix/', 'http://www.thorncliffebowlerama.com/', 'http://www.queenspectacle.com/', 'http://www.linenchest.com', 'https://www.redrobin.com', None, None, '1592185140', 'http://lemontblancbanquets.ca', 'http://nolimitska.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.highlandfarms.ca', 'http://www.firkinpubs.com/thefrogandfirkin/', None, 'http://www.papamurphys.com/', None, None, 'http://www.dacordplumbing.com', 'https://www.championwindow.com/Cleveland/?lpc=UU-OD-Cleveland-Yelp&source=ol-yel01&utm_source=yelp&utm_medium=local&utm_campaign=Cleveland', 'http://bundtz.com', 'https://caringhandsvetlv.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://clearchoicetechnical.com/service-locations/las-vegas', 'http://www.devinetouchmassage.com', 'http://www.nirvanahome.ca/', 'http://www.robertgodridgephotography.com/', 'https://www.maidsinblue.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.enblumcatering.com', 'https://atriumhealth.org/locations/detail/carolinas-medical-center-mercy', 'http://www.gilbertazpediatricdentist.com/', 'http://www.screwielewies.com/', '1592185190', 'http://www.sapporolv.com', 'http://www.galaxyhobby.ca/', 'https://azpyramidservices.binbooker.com', None, None, None, 'https://www.modernround.com/locations-peoria.html', 'http://livingwelltakeout.net/', 'http://www.balletossington.com/', None, '1592185212', 'https://agents.farmers.com/az/chandler/joe-baselice?SourceID=AMPALFYEAG&utm_medium=Local&utm_source=Yelp', None, 'https://www.get-go.com', 'https://www.ulta.com/stores/glendale-wi-646', 'https://www.cox.com/local/residential/stores/az/gilbert/2150-east-williams-field-rd?sc_id=dp_z_yext_z_store_19Q2?sc_id=dp_z_yelp_z_store_19Q36', 'https://www.olivegarden.com/locations/az/phoenix/metro-center/4410?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:phoenix-az-4410_pl:locurl_rd:1754', None, 'http://www.passionfordesserts.ca', 'https://www.aqua-tots.com/chandler/', 'https://www.handy.com/yelp50cc?utm_source=yelp&utm_medium=CPC&utm_campaign=TOR-Handy-Yelp', 'http://www.xcash.ca', None, 'http://www.mysweetbasil.com', 'https://www.sassoon-academy.com/en/academy/north-america/toronto', None, None, 'http://www.toytownplaycenter.com', 'https://my.clevelandclinic.org/locations_directions/Regional-Locations/twinsburg-fhsc', None, 'http://www.tabbyandjacks.com', None, None, 'http://enhancedcare.ca', None, 'https://www.gianteagle.com/Stores/Store/?storeId=224', None, 'http://www.kay.com', None, 'http://www.jimshorkeyford.com/?ibp-adgroup=YX_Target', None, None, None, 'https://www.granitetransformations.com/location/granite-transformations-of-northeast-ohio/?utm_source=Yelp&utm_campaign=2018', 'http://www.greentreefootandankle.com', 'https://medmen.com/stores/las-vegas-the-strip-paradise-and-harmon/', None, None, None, None, None, 'http://www.paradisehalls.com', 'http://www.jamrockcafe.com', None, 'http://www.ocadu.ca/', 'http://www.bathandbodyworks.com', 'http://www.hollywoodtans.com', 'http://gehomeservicesllc.com', 'http://eosfitness.com/location/peoria?utm_campaign=peoria&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', 'https://www.firehousesubs.ca', 'https://nationwidevision.com/locations', 'http://www.doubledspizzas.com', 'http://www.waxandlash.ca', 'https://www.tidesonthomas.com', 'http://l.macys.com/superstition-springs-center-in-mesa-az?cm_mmc=Yelp_Stores-_-Mesa-_-n-_-459&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Mesa_459&m_ac=yelp_stores&m_ag=n', 'http://fiveguys.com/', None, 'http://www.fastlanecoatings.com', None, None, None, None, 'http://peignedancestudios.com', None, 'http://www.scottsdalesilveradogolfclub.com/', None, 'http://www.fithcc.com', 'https://tattoocorner.business.site', None, 'http://www.fransrestaurant.com', 'http://www.thelittleclinic.com', None, 'http://www.copperstatehandymanservices.com', None, 'http://www.soniasbridalstudio.com', 'http://sollidcabinetry.com', 'http://www.dukephototempeaz.com', None, 'http://www.city.cleveland.oh.us/CityofCleveland/Home/Government/', '1592185371', 'http://www.saukpointvet.com', 'https://www.duckdonuts.com/location/orange-village-oh-2/', 'https://www.starlitebbq.com', None, None, 'http://www.tartucollege.ca/', 'http://sanctuary-salon-spa.com', None, 'http://www.swignsweets.com', 'http://englishbayfish.ca', 'http://www.riderta.com/facilities/airport', None, 'http://www.arbys.com/', '1592185397', 'http://northmesaauto.com', None, 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', 'https://www.phillipslaw.com/phoenix-arizona-personal-injury-lawyers', 'http://www.momdocwfw.com/estrella', 'https://thelearningexperience.com/preschoolsuccess?utm_source=Yelp&utm_medium=Referral&utm_campaign=Sponsored+Business+Listings', 'http://www.harveyspaint.com', 'https://www.screensunlimited.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.bombayexpresstoronto.com/', None, 'http://www.dominos.com', 'http://www.beautyavenue.us', 'http://www.thethirdestimate.com/', None, None, 'http://1stchoicemechanicalaz.com', 'https://redrock.sclv.com/Dining/Grand-Cafe?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.glockmeister.com', 'http://greenjadechampaign.com', None, '1592185448', None, 'https://escapetheroom.com/phoenix/', None, 'http://www.atthebarns.org/', 'http://www.thebagelcafelv.com/', None, 'http://www.spicypickle.com/restaurants/arizona.html', None, None, 'http://www.dollarama.com', 'http://www.aaaactiondoors.com', None, None, None, 'http://www.lobtoronto.com', 'http://www.happyspizza.com', 'https://locations.comerica.com/location/75th-thunderbird', None, 'http://servitaxlv.com', 'http://www.azdot.gov/mvd', 'http://www.perinatal.com', 'https://www.imagedentallasvegas.com/?utm_source=yelp&utm_medium=link&utm_campaign=yelp', 'http://www.pghoptical.com', None, None, 'https://www.mckenziesmidtown.com', 'http://www.redsearestaurant.net', 'http://www.mallorcarestaurant.com', 'http://broadviewbeauty.ca', 'https://www.homecitybnb.com', 'https://local.fedex.com/oh/bedford-heights/LNNA/', 'http://www.guschaletrestaurant.com', None, 'http://www.tinyboxvapeshop.com', 'http://www.barrettfinancial.com', '1592185518', None, 'http://www.delanolasvegas.com/en/nightlife/skyfall-lounge.html', None, '1592185526', None, None, 'http://www.openarmspreschool.ca', 'https://www.jerseymikes.com/9004', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', 'http://www.phoenixfamilymedical.com/', 'http://www.capitaldrip.com', None, None, 'http://www.handelsicecream.com', None, 'http://www.kaleidoscope-living.com', '1592185552', 'http://www.head2toechiro.com', 'http://www.eastvalleyfloors.com', 'http://www.scrapbookbarngilbert.com', None, 'http://www.nextcare.com/locations/az/peoria-glendale/?utm_source=Yelp&utm_medium=Local&utm_campaign=peoria_glendale', 'http://www.relaxologywellness.com', 'https://theoaksevents.com', 'http://www.imsiamlasvegas.com', None, 'https://www.restaurantohsushi.com', 'http://www.troplv.com/meetings-and-events/havana-room', None, 'http://www.lvms.com/bullring', 'http://www.antoniospizza.net', 'http://arbortrimming.com', None, 'https://www.venetian.com/towers/the-palazzo.html?utm_source=yelp&utm_medium=partner&utm_campaign=businesslisting', None, None, 'http://www.rajdhanisweets.ca', 'http://www.cashmanphoto.com', None, None, 'https://agent.amfam.com/nv/north-las-vegas/colleen-drinnon/', 'http://www.grandtouringautos.com/', 'https://locations.wendys.com/united-states/az/laveen/3520-w-baseline-road', 'http://www.fuzesalon.ca', 'http://gerchickrealestate.com/', 'https://greenvalleyranch.sclv.com/?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://hamptoninn3.hilton.com/en/hotels/ontario/hampton-inn-by-hilton-toronto-airport-corporate-centre-YTOSOHX/index.html?SEO_id=YELP-HP-YTOSOHX', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.everythinginsight.net/our-practice.html', None, None, None, None, None, 'http://www.petco.com', 'http://www.willietys.com/', 'http://www.sidebarphoenix.com', None, None, 'http://www.pandahobby.store', 'https://dealer.sprint.com/experts-choice', 'http://illinishuttle.com', None, 'http://www.instylenail.ca/', 'http://www.mcdonalds.com', 'http://www.clhairartstudio.com', None, None, None, 'http://www.garciaswm.com/', None, 'https://locations.einsteinbros.com/us/az/surprise/13746-w-bell-rd', 'http://www.michaelsbridalgalleria.com', 'http://www.penn-station.com', 'http://www.prmvineyards.com', 'http://Mindbodysoles.net', 'http://EvansLasVegas.com', None, None, None, 'http://www.carolinavaluevillage.com', 'https://locations.whataburger.com/az/scottsdale/2955-n-hayden-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', 'http://www.coolclubs.com', 'http://Azkickbacktours.com', 'https://www.phoenix.gov/parks/sports/complexes/desert-west', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.trilliumweightloss.com', 'http://www.runningwiththebullsusa.com', 'http://saporebyzafferano.ca/', None, None, 'http://www.thehkan.com', 'http://www.hayleylynnedoeshair.com', None, 'http://www.cotswoldfarmersmarket.com', 'http://www.kaleidoscopejuice.com/', 'http://www.harrysgrilleandtavern.com', None, 'http://local.biglots.com/az/gilbert/1913?zcp=pd_local_yelp', 'http://www.scottsdaleobgyns.com/russellBartelsMD.html', 'http://www.yourcarinsiders.com', 'https://www.supercuts.com/locations/pa/pittsburgh/shadyside-plaza-s-highland-haircuts-8953.html', '1592185734', None, None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://www.yknotparty.com', None, None, 'http://local.firestonecompleteautocare.com/ohio/north-olmsted/4580-great-northern-blvd/?lw_cmp=IYP_YPC_MLP_14796', 'https://www.saksfifthavenue.com/locations/s/sherwaygardens', None, None, 'http://www.vintagehomecharlotte.com', 'http://Www.aglowbeautybarlv.com', 'http://www.carolinaherrera.com', None, 'http://www.activegreenross.com', None, None, 'https://locations.whataburger.com/az/mesa/1947-w-broadway-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', None, None, 'http://locopatron.com/', 'http://livewellmassage.vpweb.com', None, 'http://www.mammaspizza.com', 'http://www.victoiredayspa.ca', 'http://www.outback.com/locations/nv/las-vegas-craig', 'http://bandnautosalesllc.weebly.com', 'http://us.shop.ecco.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://firenzastone.com/', None, None, 'http://auxiliarycrossfit.com', 'http://www.capitolpetroleumllc.com/site/', 'http://www.pogllc.com', None, None, 'http://www.legendscafelv.com', 'http://www.sweetlifegelateria.com', 'http://www.clutchlandscaping.com', 'https://www.lesinsulaires.ca', 'https://www.circlek.com/store-locator/us/scottsdale/20721-n-scottsdale-rd/2705915', 'http://mine-sushi.com/', '1592185823', 'http://www.tsingtaomesaaz.com', 'http://www.Friendly-Wireless.com', 'http://www.bmvbooks.com/', 'http://www.smoochiepoochiedoggrooming.com', 'http://www.LaurenParis.com', 'https://www.wyndhamgardencalgaryairport.ca', None, 'https://www.calgarycoop.com/stores/shawnessy/', None, None, 'http://bosadonutsaz.com', 'https://www.ganleylincolnohio.com', 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/brampton-7688-hurontario-street.aspx?postalcode=L7A+3R9&storeid=66', None, 'http://www.thestratford.org/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', None, 'http://www.eddiebauer.com', 'http://www.tempritelv.com', None, 'https://www.saksfifthavenue.com/locations/s/phoenix', 'http://www.carolinavet.com/locations/matthews-nc/', 'http://aestheticbodycontouring.net', None, 'http://www.ganleysubaru.com', 'https://vbarbershop.com/location/chandler-fulton-ranch/', 'http://www.bellanailshop.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://hashtaggallery.com', 'https://locations.theupsstore.com/nc/matthews/2217-matthews-township-pkwy?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Henderson-NV-89011/807058/', 'http://www.heislersmarket.com', '1592185901', None, 'https://www.esso.ca', 'http://www.hogwildpitbbq.com', None, None, '1592185915', 'https://www.kohls.com/stores/az/glendale-652.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.indiasflame.com', 'http://www.brightoncollisioncenter.com/', None, 'http://workoutquest.com', 'http://www.eagerbeaverplumbing.com', 'http://allens-pool-catering.business.site', '1592185934', 'http://www.ostlerfamilydental.com', 'https://chutneyb.com', 'http://www.ae.com', None, 'http://vspconsignment.com', 'https://www.zipscarwash.com/location/rock-hill-sc/', 'http://www.teresaspizza.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.morphebrushes.com', None, 'http://schwans.com', 'http://lovekimnailsspa.wix.com/home', None, 'http://www.earthangelsholistic.com', None, None, 'http://8thwonderstudio.com', 'http://www.bowtiepizza.com', None, 'http://www.eggsmart.ca', None, None, 'http://www.chapters.indigo.ca/home/storelocator/storeDetails/929/', 'http://www.donvalleyvw.net', 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'http://www.aemediaproduction.com', 'http://www.wingmachine.com', 'http://www.lvmis.com', None, 'http://www.vegaspubcrawler.com', None, 'http://www.jeromesalon.com', 'http://www.transformerstation.org/', 'https://lasvegas.adamevestores.com', 'http://www.dvtowing.com', None, None, 'http://Longevityinfitnessexcellence.com', 'http://www.smilebeautifuldentistry.com', 'http://www.marriott.com/hotels/travel/yyzap-courtyard-toronto-airport', 'http://www.urbanbloomcurls.com', 'https://www.willowpark.net', None, 'http://local.biglots.com/az/phoenix/4085', None, None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://rogersandhollands.com/east-towne-mall', 'https://starliterestaurant.ca', 'https://texasstation.sclv.com/Dining/Slices-Pizzeria', 'http://www.oldmillgm.ca', None, 'http://www.maac.com/alabama/huntsville/colonial-grand-at-madison?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://www.drbarbaranewman.com', 'http://www.clecabinets.com', None, 'http://www.mcmagain.com/home.html', 'https://locations.theupsstore.com/nc/charlotte/2820-selwyn-ave?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.bloorfitness.com/', 'http://www.kouskousexpress.com', None, 'http://www.sweetalamode.ca', None, None, None, None, 'http://www.globalpestlasvegas.com', None, 'http://www.foliclesalonandspa.com/', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://www.givemiacookie.com', 'http://www.chatime.com', 'http://www.therobotoproject.com', 'https://www.bushmillsirishpub.com', 'http://ladlows.com', 'https://sfaservice.com', None, None, 'https://luxxestudio412.glossgenius.com', 'http://www.alankells.com', 'http://www.gianteagle.com/store-locator', 'http://www.marytripi.com', 'http://rajplasticsurgery.com/', None, 'http://www.deltaco.com', 'http://www.steelcitysteak.com', 'http://nickelsdeli.com/saint-laurent/', None, 'http://momentumsalon.com', 'https://octapharmaplasma.com/donor/center/18568', 'http://www.mastercleaners.us', 'http://www.justfresh.com', None, 'https://www.gowireless.com/stores/az/phoenix/phoenix-ahwatukee-foothills/', 'http://shop.helixtelecom.com/', 'http://www.CerberusDesigns.ca', 'http://www.betsysicecream.com', 'https://www.newyorknewyork.com/en/nightlife/pour-24.html', 'http://pinogelato.com', None, 'http://www.vergerlacroix.ca/', 'http://www.starbucks.com/store/113209/', 'http://www.alexreidosleitoes.com', 'http://www.habitatlivingsound.com', 'http://www.attackpestcontrollv.com', 'https://www.kindercare.com/our-centers/mesa/az/000632?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=000632', 'http://www.pawsplaygrounds.com', None, 'http://www.zagg.com', 'https://www.donmillersubaruwest.com', None, 'http://www.ArcadiaCoveApartments.com', None, 'http://www.bodyheattanning.com', 'http://www.litespressobar.com', 'http://www.barbiespetsitting.com', 'http://www.hungryhut247.com', 'http://www.restaurantmochica.com/en/toronto', '1592186182', None, 'http://www.evoautoaz.com', '1592186188', 'http://phochay88.com', None, 'http://www.rockyscustomdetailing.com/', 'http://www.azeyes.com', 'https://www.therackandcue.com', 'http://www.backdoorlv.com', None, 'https://www.capriottis.com', None, 'https://lemonsharkpoke.com/locations/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://www.usps.com', 'https://www.labcorp.com', None, None, 'http://www.omzhair.com', 'https://www.marcs.com/stores/marcs-middleburg-hts-southland-grocery-store-4413', None, 'http://woodsrestaurant.ca/', None, None, None, 'http://fourpalmsautosales.com', 'http://gesu.com/', None, 'https://www.purdys.com/shops/details/79', 'http://www.scilv.com', 'http://tjsbutcherblock.com', 'http://aquatherapyclinics.com/', 'https://locations.costavida.com/costa-vida-a2660367671e', 'http://ragingspoon.ca/', 'http://www.queenfishandchips.com/', 'http://www.brownleejewelers.com', '1592186258', 'http://www.gypsybarphoenix.com', 'http://www.tbsstrong.com', 'http://www.titanrelocationservices.com', 'http://www.majormilliken.com/', 'http://www.bestautobuy.com/', 'http://Www.GotGoddessHair.com', 'http://www.cleatswings.com', 'http://outpostcoffee.com', None, None, 'http://www.ampm.com', 'http://www.photoantiquities.org', 'http://www.atmosphere.ca/', 'http://affordablelakenormanshuttle.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'https://stores.mooresclothing.com/ab/calgary/415---3545-32nd-ave--ne.html?utm_source=yelp&utm_medium=listing&utm_content=41&utm_campaign=AB', 'http://www.disciacca.com', None, None, None, '1592186300', 'https://sparklenailsbar.com', 'http://southpointcasino.com/dining/michaels-gourmet-room?cid=ypmic', 'http://www.fiveyearsyoungerfacial.com', 'http://www.thecrepecafe.com', 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.JohnsonEstateLaw.com', 'http://walmart.com/', None, None, None, 'https://www.pizza73.com', 'https://www.batteriesplus.com/store-locator/az/tempe/batteries-plus-333?utm_source=yelp&utm_medium=local&storecode=333', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', None, '1592186333', 'https://dwss.nv.gov', None, None, 'http://noftz.com', 'https://santanvillage.benefitbrowbars.com', '1592186344', 'http://www.cardiacsolutions.net', 'http://natureworksbest.com/', 'http://www.mdclv.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, 'http://www.petsmart.com', None, None, 'http://www.sawhilbanquet.com', 'https://lernerandrowe.com/?utm_source=Yelp', 'http://www.maccosmetics.com', None, 'http://www.midas.com/calgary/store.aspx?shopnum=9304&dmanum=151', 'http://www.timhortons.com', None, 'https://www.rolex.com/rolex-dealers/dealer-locator/retailers-details/rolexboutiquelvluxury.rswi_17986.lasvegas.unitedstates.rolexdealers.com', 'http://www.vosgeschocolate.com', 'http://www.a2zrm.com/', None, None, 'http://www.pittsburghprestigephotobooth.com', 'http://www.carmeldayspa.com', 'http://reddiservices.com/phoenix', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.abcpediatricshenderson.com/', 'http://www.wolsteincenter.com', 'https://stores.chicos.com/s/the-summit-at-scottsdale', 'http://www.monsterpgh.com', 'http://www.soccerzonevegas.com', 'http://www.northhillpizza.com', 'http://www.oxigenspa.com', None, 'https://eyetique.com/squirrel-hill', None, 'http://www.azresearchcenter.com/', '1592186416', 'https://www.cielohendersonapts.com', 'http://www.baileyorthoaz.com/', 'http://www.hobbypeople.net', 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=clesv&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', 'https://mandarinrestaurant.com/mississauga/', 'http://www.switchconsignment.com', 'http://www.porschenorthscottsdale.com', 'http://www.carolinapacknc.com', None, None, 'http://www.guyandgalsalongilbert.com', '1592186443', 'https://www.wilhelmautomotive.com/locations/tatum-ranch/', None, None, 'https://twomenandatruck.com/movers/il/champaign', 'https://castlerents.com', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', 'http://www.dukescycle.ca', 'http://greatwhaletattoo.com', '1592186463', None, 'http://www.trilliummoving.com', None, 'https://www.dominos.com', 'http://local.buckle.com/AZ/TEMPE/385/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', 'http://myorangeleaf.com/frozen-yogurt-store/AZ/chandler/2875-west-ray-rd%2c-suite-1?id=8', 'http://www.17thavechiropractor.com', 'http://www.abiaz.com', 'http://www.thevenetian.com/RIVA.aspx', '1592186483', None, None, 'http://vwnorthscottsdale.com', '1592186491', '1592186493', 'http://www.gordsappliance.com', 'http://www.robinsongarden.com', 'http://www.cityandeast.com', 'http://ccvet.com', 'http://www.treatmentsalon.com', 'http://www.cottonon.com', 'http://fixauto.com/scarborough-town-centre', None, None, 'http://www.whitetoronto.com/', 'http://www.ercfixtv.com', 'http://www.jimmyscoffee.ca', 'http://www.ngvlanes.com', 'https://www.anytimefitness.com/gyms/288/Las-Vegas-NV-89139/?utm_source=yelp&utm_medium=local', 'http://inandoutmoving.ca/', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.madisonavecakes.com', 'http://www.manestudio.ca', 'http://blackcatpgh.com', None, 'http://www.smileshopdental.com', 'https://www1.shoppersdrugmart.ca', 'http://www.bullfrogspas.com', 'http://www.childrenslearningadventure.com/', None, '1592186547', 'http://www.desimoneglobalmarketing.com', '1592186550', 'http://www.timetoronto.com', 'http://restaurant.chineseonlineorder.com/goldengatehenderson', 'http://restaurantletandoor.ca', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', 'https://www.snapdragon-salon.com', 'http://www.thetamalestore.com', 'http://www.peterpiperpizza.com/', None, 'https://mononabarbershop.com', 'http://www.carolinasdish.com/', 'http://www.sparkleanlaundryphoenix.com/', 'http://www.lagunaspanv.com/', None, 'http://www.foxfamilydental.net', None, '1592186583', 'http://www.usps.com', 'http://www.sprint.com', None, 'http://islandheatdinnershow.com', 'http://flashforwardsites.com', 'http://www.bishoplawoffice.com', 'http://www.grandavenueproduce.com', '1592186601', 'http://www.balconycleaning.net', 'https://www.mapleteahouse.com', None, '1592186607', 'http://www.thechesterfieldshop.com', None, 'https://cftn.com', 'https://stoughtontireandautorepairinc.com', 'http://www.jiffylube.com', 'http://www.starbucks.com/store/9298/', 'http://www.thaidelightaz.com', 'https://www.redmountainweightloss.com/scottsdale/', 'https://www.libertysquare-apartments.com', '1592186628', 'http://guireitoronto.com', '1592186632', None, 'http://k1speed.com/phoenix-location.html?utm_source=Yelp', 'http://www.jasesouder.com', 'http://www.thecalgarygroomroom.com', None, 'http://www.littleones.ca', 'http://www.leagueofvapes.ca', 'http://www.scottsleedmd.com', None, 'http://mex-city.com/', None, 'http://www.azrapidmaintenance.com/index.html', 'http://www.coveredwagonaz.com/', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', None, 'https://www.thepaincenter.com/?utm_source=yelp&utm_medium=referral&utm_content=arizona-gilbert', 'http://ritzasiaaz.com/location.php/', 'http://www.meineke.com', 'http://www.soraonrose.com', None, 'http://www.ritzcarlton.com/en/hotels/canada/toronto', 'http://paradisevalleyseptic.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281357318&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://torrancepharmacy.com', None, 'http://www.littlecaesars.com', 'http://www.hummusbowls.com', '1592186689', 'http://www.audioworkx.com/', 'http://restaurantvago.com', None, 'https://www.batteriesplus.com/store-locator/nc/gastonia/batteries-plus-423?utm_source=yelp&utm_medium=local&storecode=423', 'https://beezybooth.com', 'https://www.montecarlo.com/en/nightlife/brand-lounge.html', None, 'http://www.sovarizona.com', 'https://www.orangetheory.com/en-us/locations/arizona/phoenix/2501-west-happy-valley-road-suite-32-1020/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.stylevsalon.com', None, 'https://www.ahn.org/location-detail?name=AGH_Suburban', 'http://shopurbo.com', 'https://www.josbank.com/store-locator/mt-lebanon-pa-356?utm_source=yelp&utm_medium=listing&utm_content=356&utm_campaign=PA', 'http://www.campers-village.com', None, 'http://www.bouldercreekgc.com', None, None, None, 'http://www.bigalscanada.com/stores/locations/NorthYork/northyork.html', '1592186732', None, 'http://www.oakdalepodiatrist.com', 'https://www.penrosecharlotte.com', None, 'http://www.wildwingrestaurants.com/franchisees/brampton-trinity-commons/', 'http://www.scottsdalehillsah.com', 'https://www.sundancecinemas.com/', '1592186748', None, None, None, 'http://www.opticgallerysummerlin.com', None, 'http://www.cottonlanervresort.com', '1592186762', 'http://franklygoodcoffee.com', '1592186767', 'http://www.thebeerstore.ca/locations/sheppardwillowdale', 'https://www.moxieservices.com/locations/phoenix/?utm_source=yelp&utm_medium=cpc&utm_campaign=2019_cpc', 'http://www.valleymedped.com', None, '1592186777', 'http://www.camelotinnelyria.us', 'http://www.kevinpowellmotorsportscharlotte.com', None, 'http://www.spiceasianbiz.com', 'http://www.santabarbaracatering.com', 'https://www.gamestop.com/store/us/wi/madison/740/prairie-towne-center-gamestop', 'http://www.lasvegasspinesurgeon.com', 'http://songbirdartistry.com', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'https://budgetblinds.com/Goodyear/?utm_source=yelp&utm_medium=naf-listing&utm_campaign=NAF-LOCAL-YELP', None, None, 'https://www.amazinglashstudio.com/studios/nc/charlotte/park-towne-village', None, 'http://www.therollhouse.com/wickliffe', 'http://www.dunkindonuts.com', 'https://www.purebarre.com/location/chandler-az', None, 'http://www.eliteacademyofgymnastics.com', 'http://advancedmetal.com', 'http://www.pitaland.ca', None, 'http://www.evex.ca', None, None, 'https://www.anytimefitness.com/gyms/92/Phoenix-AZ-85048/?utm_source=yelp&utm_medium=local', None, 'https://aamcobayarea.com/locations/concord', 'http://www.starbucks.com', 'http://www.spectrumbrands.com/', None, 'http://www.bellybumpsandbabies.com/', None, 'http://www.theknittingroom.ca', 'http://www.simplesimonpies.com', 'http://www.ohiojunkforce.com', None, 'http://www.publix.com', '1592186858', None, 'http://www.furniturestorelasvegas4u.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://tumbleweedhotel-hub.com/', None, None, 'http://www.polishdnailbar.com', 'http://uppercrustwinebar.com', 'https://www.kfc.ca//store/1595-cote-vertu-st-laurent', None, None, None, 'http://www.25liberty.ca', 'http://www.yehyogourt.com/', 'http://www.lasvegasfloral.com/?srccode=EF_YELP', 'http://www.wigsrustoronto.com', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'https://ramclean.com', None, None, 'http://mintlocker.com', None, None, 'http://www.greenfieldgroomingsalon.com', None, '1592186914', 'http://www.tankstap.com/', None, 'http://www.ouestantoniospizza.com', 'http://www.badwaterbrewing.com', 'https://local.albertsons.com/az/chandler/3145-s-alma-school-rd.html', None, None, 'https://quantumcleaning282.wixsite.com/website', 'https://locator.chase.com/ny/glendale/71-04-woodhaven-blvd', None, 'http://www.milestonesrestaurants.com/', None, None, None, 'http://www.wmdouglas.com/charlotte', 'http://www.nvscc.com', None, None, 'http://www.planetdog.ca', 'https://www.arizonarestorativepsychiatry.com', None, None, 'http://www.mevamekitchenexpress.com/', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', None, 'http://nevadaeyephysicians.com', 'http://www.ranchorelaxotogo.com', 'https://hakkasan.com/las-vegas/', 'http://www.dysartdental.com', 'http://stacyannsscissorhands.com', 'https://www.supercuts.com/locations/az/scottsdale/scottsdale-pavillions-haircuts-9672.html', 'https://locations.churchs.com/nv-las-vegas-8616', 'http://www.landmarkautoanddiesel.com', 'http://www.familyvideo.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://stores.dickssportinggoods.com/az/scottsdale/1194/', 'http://www.drallenhuioptometry.com/', 'http://www.fabfred.com/locations/ft-apache/', 'http://www.theteakeeper.com/', 'https://agents.allstate.com/juan-deglane-glendale-az.html', 'http://www.ctctree.com', 'http://www.saharaair.com', None, None, None, 'http://www.holtrenfrew.com/store/holt/stores/hr2/VaughanMills', None, 'http://www.bennett2rentit.com', 'https://locations.theupsstore.com/az/scottsdale/16211-n-scottsdale-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=11088&localechange=1', '1592187020', 'http://www.auburnbayvet.com', 'https://www.swiftaviation.com', 'http://www.sundevilauto.com/locations/az/scottsdale/north-scottsdale-25', None, None, 'http://www.pinotspalette.com/southend', 'http://www.pinkynails.ca', 'http://smartrepairsaz.com', 'http://www.lizyhoeffer.com', 'https://www.spiritsandtales.com', '1592187046', 'http://www.greatclips.com/salons/4614', 'http://www.dunkindonuts.com', None, None, 'http://walkinwalkin.com', None, '1592187060', None, None, None, 'http://www.seespotclean.com', 'http://gogreenac.com/', 'https://iworkoncars.com', 'https://silvernuggetlv.com', None, 'http://www.bk.com', 'http://www.mgmgrand.com/restaurants/crush.aspx', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNMW', 'http://www.AbsolutePetCareAZ.com', 'http://www.scrubtastic.com', 'http://broadbentdentistry.com/', 'https://www.bestbuy.com/site/electronics/geek-squad/pcmcat138100050018.c?id=pcmcat138100050018', 'http://www.honmachirestaurant.com', 'http://www.bluearrowrecords.com/', 'http://www.phoenixrackandaxle.com', None, 'http://www.parkcentraldeli.com', None, 'https://www.mygabes.com', 'http://thriftdlux.com/', None, 'http://www.clinicsalveo.ca', 'http://www.pandora.net/en-us/stores/united-states/85254/pandora-scottsdale-quarter', None, 'https://www.sprint.com/storefronts/bd/sprint-north-las-vegas-nv-89032-fcs-4123/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.housemaster.com', None, None, 'http://nodabarkandboard.com/home.html', 'https://www.destinymartialartsacademy.com', 'http://emakravmaga.ca', 'http://www.stateoftheartdesigners.net', 'https://www.chevronwithtechron.com', 'http://www.bigjimsrestaurant.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=626&utm_content=bpas', None, None, 'https://www.lasikvisioninstitute.com/lasik-eye-surgery/cleveland-surgeon-center/?venid=YLP31&subid=CLE', None, '1592187147', None, 'http://www.verizonwireless.com/stores/arizona/mesa/val-vista-204086/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://www.midwoodsmokeshack.com', 'http://www.chapters.indigo.ca', 'https://www.vistasureno.com', 'http://www.therumormillmarket.com', 'http://www.drywallperfection.com', '1592187163', 'http://artscapeyoungplace.ca', 'https://www.supercuts.com/locations/oh/rocky-river/rocky-river-haircuts-81630.html', 'http://www.katieskorner.com', 'http://www.camh.net', 'http://www.southwestfunctionalmedicine.com', None, None, 'http://www.lionsmanegrooming.com', 'http://www.amazinginc.net', 'https://haircutmenfairlawnoh.com', None, 'http://www.cherryberrieslasvegas.com', 'http://www.shopwoc.com', 'https://www.amcofdv.com', 'http://www.starbucks.com/store/10759/', 'http://usmaniagrill.ca', 'http://www.igoturback.com?cmpid=GSDG_BLM_0006', 'http://www.cal-am.com/communities/chaparral-village', 'http://azmajorsoccerleague.com', None, 'http://www.valleygoldmine.com', 'http://www.saludjuicery.com/aspinwall/', 'http://www.undefeated.com', None, 'http://www.whatalesya.com', 'https://www.burialplanning.com/cemeteries/resthaven-memory-gardens/', 'http://collegestreetanimalhospital.com', 'http://pueblosereno.com/', 'http://www.autonationbuickgmchenderson.com', None, 'http://fortmill.burnbootcamp.com/', 'http://www.orangedumpster.com/', 'http://portofsubs.com', 'https://www.newdermamedlaserclinic.com', None, None, 'http://www.svmmjcc.com', 'https://stores.bareminerals.com/wi/madison/us-2313/', None, 'http://www.rdquake.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sears.com', None, 'https://www.storage-solutions.org/storage/arizona/storage-units-peoria/8266-W-Lake-Pleasant-Parkway-115', 'http://www.darinaelectrolysis.ca', 'https://www.allamericanpubclt.com', 'http://www.sasshoescharlotte.com', 'http://taylormadecustompools.com', 'http://www.kitchen73.ca/laval', 'http://colorworkslandscape.net', 'http://www.bodyrestorationmassage.com', 'http://fujiyamacalgary.com', 'http://www.castleapartments.net', 'https://www.myeyedr.com/locations/independence-boulevard?utm_source=Yelp&utm_medium=Referral&utm_campaign=independence-boulevard', 'http://carlretterdebtattorney.com', None, 'https://zelaya-jr-painting.com', 'http://www.swarovski.com/Web_CA/en/index', 'http://www.trapezeu.com', None, 'http://www.marshallsonline.com', 'http://kokofitclub.com', 'https://www.thirdfederal.com?=ad.doubleclick.net/ddm/clk/458420725;262568795;w', 'http://www.tropicalsmoothie.com', 'http://www.jonesysclassiccleaners.com/', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', None, 'http://tradelandsaz.com', None, 'http://www.sweetfixbakery.com', None, 'https://www.cellphonerepair.com/charlotte-university-city-nc/', 'http://www.happyvalleyplumbingllc.com', 'http://www.oneofakindpets.com/', 'http://jacksonswash.com', None, None, 'http://www.talkingstickresort.com/spa.aspx', 'http://nathansfamous.com', 'http://fiveguys.com/', None, None, None, None, 'http://christensenconstruction.net', '1592187329', 'http://www.realliving.com/sj-fowler-real-estate', 'http://www.gnargnartours.com/', None, 'http://www.apachestone.com', 'http://www.doddsdoors.com', 'http://www.subway.com', 'http://www.canyon-pt.com/location/surprise-office/', 'http://trustarts.culturaldistrict.org/production/37630', None, 'https://stores.savers.com/nv/lasvegas/savers-thrift-store-1047.html', 'https://milk-n-more.business.site', None, 'http://www.sfldentistry.com', 'https://extremewheels.com', None, 'http://www.JimmyJohns.com', None, None, None, 'https://www.thehouseofused.com', 'http://www.nhhuntersvillepedsandim.org', 'http://www.origintmassage.com/', None, 'http://www.hskaraoke.com', 'http://mesa.floorstogo.com', 'http://www.nveyesurgery.com', 'http://www.stoutflooring.com', None, None, 'https://square.site/book/0GPTRC4PSJ0WB/sunny-girl-spray-tan-phoenix-az', None, 'http://www.leanonmeals.ca', 'http://www.colossusgreektaverna.com/locations/colossuspc/contact/', 'http://sonoranserenityspa.com/', '1592187400', 'http://www.blm.gov', 'https://thrifty.com/loc/ll/CA/OT/TORONTO-~-FRONT-ST-(OT)/330-FRONT-STREET-WEST', 'http://www.benchworksteering.com/', 'http://visit.brownmackie.edu/SERCH/dynamic/bmc-3-step.aspx?RIFID=3322&source=YELP&sourceCode=YELP&cid=SERCH_BMWA_096_SRCH_002&SearchEngine=Yelp&utm_source=Yelp&utm_medium=cpc&utm_campaign=Yelp', None, 'http://www.suzannewigginton.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.carepatrol.com/', '1592187418', None, None, 'http://artgeckoshop.com', None, None, None, 'http://www.loftsonthomas.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/pa/turtle-creek-4072.html?mcid=iyp:8368400', None, 'http://www.thebierhausbistro.com/', 'http://www.wheatbyjay.ca', None, None, None, '1592187446', '1592187449', None, 'http://brathausaz.com/brat-haus-events/', None, 'http://www.stardustbuilding.org', None, 'http://www.blackcanyonconferencecenter.com', 'http://www.pittsburghyellowcab.com', None, 'http://www.casbahrestaurant.ca', 'http://www.solosmallmotors.ca', 'http://www.empirepizzabar.com', 'http://www.jacksonswash.com/location-84th-avenue', 'http://www.dakinemobiledetailing.com', 'http://www.adrenalinetoronto.com', 'http://www.drlockney.com', 'https://locations.churchs.com/az-phoenix-681', 'http://venues.calgarystampede.com/venues/bmo-centre/', 'https://bikanersweetsandrestaurant.ca', '1592187485', None, 'http://www.prayosha.ca/', None, 'http://blackcanarycoffee.com', 'http://www.massagewellvegas.com', 'http://www.brookdaledentalcare.com', 'http://www.mcdonalds.com', None, 'http://www.customcaninecompany.com', 'http://www.dcslv.org', None, 'http://www.terribleherbst.com/', None, None, None, None, 'http://gocheckers.com/team/game-guide', 'http://paradisemelts.com', 'http://www.tribebarrestudio.com', 'http://www.fuguasian.com/menu.aspx', None, None, 'http://www.billygrahamlibrary.org', 'http://www.purolator.com/purolator/ship-track/find-location.page?location=ridRC0715', None, 'http://www.fedex.com', None, 'https://www.snwa.com', 'http://www.freshandeasy.com', None, 'http://www.orthopaedicspecialists.org', None, 'https://www.kwiktrip.com/Locations/Details/650', 'http://www.phoenixbedbugexpert.com', None, 'http://www.videoconversionexperts.com/', 'https://www.cash1loans.com/locations/arizona/phoenix-thomas.aspx', 'http://local.firestonecompleteautocare.com/ohio/cleveland/6874-pearl-rd/?lw_cmp=IYP_YPC_MLP_1694', 'http://www.canadiantire.ca/en/store-details/on/warden-eglinton-on.store.html', '1592187562', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://supremetouch15.wixsite.com/supreme-touch/contact', 'http://www.perryellis.com?utm_source=Yelp&utm_medium=Social&utm_campaign=PE_Yelp_BioCTA', 'http://www.mrtire.com', None, 'http://www.sunbirdgolf.com', None, 'http://www.mocca.ca', 'https://www.pizzaville.ca/stores/hours/2889-derry-road', None, 'http://www.maranellocafe.ca', 'http://www.torontosteamnclean.ca', 'http://www.jogoyaseafoodbuffet.com/', None, 'http://www.joefresh.com', 'http://ladolcepesca.com', 'https://www.theshadestore.com/showrooms/nevada/las-vegas?utm_source=yelp&utm_medium=cpc', 'http://www.tristatetowingandrecovery.com', None, '1592187599', 'http://www.royalvinewinebar.com/', None, 'http://SecondsCity.com', 'http://www.ljsilvers.com', None, 'http://scacciaunion.ca', 'http://truhitfitness.com/north-phoenix/', 'https://orlandoautobody.com', 'http://www.coyoteoaties.com', 'https://www.jiffylube.com/locations/az/phoenix/2098', 'http://www.bigmanspizza.ca', 'https://www.dogtrainer-charlotte.com', None, None, 'http://www.goodwillaz.org/location/goodwill-mckellips-mesa/', 'http://www.azspinalcare.com/', 'http://www.ranee.ca', 'http://www.thecornflower.com', 'http://www.springspreserve.org', None, 'http://www.dakotamoversaz.com', None, '1592187645', 'http://www.bownesshealthfood.ca', 'https://rapidresponders.com', None, 'https://amazingleakdetection.com', None, 'http://www.midas.com/cleveland/store.aspx?shopnum=6355&dmanum=212', 'https://www.fountainpalmsapts.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=organic', 'http://www.bojangles.com', 'http://www.themagicshoplv.com', None, 'http://www.sewmuchfun4you.com/', None, 'http://www.hickorynutgolfcourse.com', 'http://www.mrsbridalboutique.com/', 'https://www.harveys.ca', 'http://www.AZLawnGuys.com', '1592187676', 'https://www.spencediamonds.ca/location/calgary-ab/?utm_source=Yelp&utm_medium=Display', 'http://www.LasVegas1RealEstate.com', None, 'http://www.samosafactory.com', 'http://www.thebrubakerspub.com', None, 'https://www.homestreet.com', 'http://www.nikosgardening.ca/', None, 'https://www.victra.com/?utm_content=Yelp&utm_medium=URL&utm_source=Yelp', 'https://www.mindbodhiyoga.com', 'http://www.virginamerica.com/', None, None, 'http://www.health-a-licious.com', 'http://lacampagnaonline.com', 'http://www.canampm.com/sun-valley-ranch', 'http://www.ranefamilyinsurance.com', None, 'https://stores.nyxcosmetics.com/directory/nv/las-vegas/3663-south-las-vegas-blvd.html', 'http://trilogycle.com', None, 'http://www.bloomersto.com', 'https://www.walmart.com/store/5334/aurora-co/details', 'http://www.fithcc.com', None, 'http://99only.com/store/Las-Vegas---Beltway/', '1592187730', None, 'http://www.vbinspect.com', 'http://www.makinolasvegas.com', None, 'http://samspizza.co', 'http://www.swimmerinsurance.com', 'http://www.lipsboutiquebyjka.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.themarvelexperience.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.bannerhealth.com/locations/banner-urgent-care-6', '1592187760', 'http://www.taitjointreplacement.com', None, 'https://www.weecarelv.com', None, 'http://www.pizzanova.com', None, None, 'http://www.entebellamedical.com', 'http://www.deansgreenhouse.com', 'http://www.jazz.fm', 'https://locations.churchs.com/az-phoenix-7235', 'http://www.hashhouseagogo.com/', 'https://www.wyndhamhotels.com/hotel/15081?tel=18008899735&iata=00093785&cid=fe:hj:20170523:yl:pp:hjus:15081', 'http://www.hawaiianbarbecue.com', None, 'http://www.pizzahut.ca/', None, None, 'http://www.hawaiipoke.ca', None, 'http://www.bogdananghelmd.com', 'http://www.thegreenhousetavern.com', 'http://www.farmgirlsfood.com', 'http://www.museums.nevadaculture.org/nsmlv-home', 'http://www.graysbooks.com', None, 'https://www.clubpilates.com/fortmill', 'http://www.swc', 'http://www.4feldco.com/locations/madison/', 'https://www.simon.com/mall/arizona-mills/stores/disney-store-outlet-the', None, 'http://www.raymillphotography.com', 'http://www.thomasyu.ca', None, 'http://www.animalinnlv.com', None, None, 'http://www.ajlawplc.com', 'http://www.metrobark.com', 'https://www.get-go.com/', None, 'http://www.clevelandairport.com', None, 'http://jbamotors.com', '1592187838', 'http://www.wagonthedanforth.com/', None, 'http://www.toryburch.com', 'https://samurairamen-sushi.com', None, 'http://pizza.dominos.ca/Mississauga-Ontario-10348', 'http://www.universalscreenarts.com', None, None, None, 'http://www.keybank.com', None, 'https://cannacabana.com', '1592187864', 'http://www.sweetpeas.ca', 'http://cornercafewi.com/', 'http://www.restaurantbellissimo.com/fr', 'http://pigtailsandcrewcuts.com/charlotte-cotswold', None, 'http://www.calldanielhull.com?cmpid=F27M_BLM_0006', 'http://www.sidestreetpubandgrill.com/', None, 'http://www.DesertFitnessAZ.com', 'http://www.beardedchefaz.com', 'http://www.WickedBakery.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.gumbyspizzamadison.com/', None, None, 'http://www.makeafoodiecall.com', 'http://www.dunkindonuts.com', None, 'https://www.shadowmountaindentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.biennails.com', 'https://www.americasbest.com/store-list/avon-oh-44011/', 'http://www.wynnlasvegas.com/#Shopping/blacksatin', None, None, 'http://farmersagent.com/gelfond', None, 'http://cherryblossompools.com', 'http://www.eatnpark.com', None, 'http://www.activekidspediatrics.com', 'https://daisybluesummit.business.site', 'http://www.cedarcenterhardware.com/', 'http://www.hectorsmovingservices.com', 'http://locations.maurices.com/nv/lasvegas/clothing-stores-lasvegas-nv-1751.html', None, None, 'http://www.sonoramesquitegrill.com', 'http://www.airbornetrampoline.ca/locations/woodbridge/', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'https://arizonaorganix.org', 'http://www.coinlesslaundry.com/23rd.html', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://littlerascalslearningcenter.com', 'http://www.barreljunction.com/location', None, None, '1592187957', None, None, 'http://www.kensingtonmakeupacademy.com', 'https://www.greulichs.com', None, 'http://www.eastwinddentallv.com', None, 'http://www.metroparktune.com', 'https://locations.theupsstore.com/az/gilbert/70-s-val-vista-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.trinitycathedral.com', 'http://kokosmiles.com', None, 'http://www.millerswaterdamageandrestoration.com/henderson-nv.html', None, 'http://ltgrill.com/', 'https://eastvalley.bintheredumpthatusa.com', 'http://www.swgas.com', 'http://www.FletcherHeightsAH.com', None, 'http://www.mooredalesailing.com', 'http://www.disgraceland.ca/', 'http://www.michaelkors.com/stores/united-states/nv/las-vegas/michael-kors-palazzo', None, 'https://www.kurevapes.com', 'http://www.sossystems.com', None, None, 'https://stores.petco.com/nc/gastonia/pet-supplies-gastonia-nc-1586.html', None, 'http://www.Azbiltmorebarbershop.com', 'https://www.zaxbys.com/locations/nc/concord/7781-gateway-ln-nw/', '1592228118', 'http://eyelovebeautybar.com/', 'http://www.justrolldup.com', 'http://www.7-eleven.com/?yelp=22886', None, 'https://www.simonmed.com/locations/all/simonmed_imaging___estrella', None, 'https://www.gostorageone.com/self-storage/nv/las-vegas/maryland-pkwy/', 'http://www.cineplex.com/Theatre/cineplex-odeon-ajax', None, None, None, 'https://www.zimbrickhonda.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', 'http://www.kgautoworkspgh.com', 'https://www.americafirst.com/about/branches/branch-details.cfm?siteNo=104', 'http://www.goldenwestmanagement.com/', None, None, 'http://www.adega9.com', 'http://www.freshandwild.ca', 'http://www.scottsdalerestaurants.co', None, 'http://headtotoeinc.com', 'https://theeggworks.com', 'http://www.aqualunch.ca', 'http://www.steakonastone.us', 'http://www.thesavannah.com', None, None, 'http://www.ktnv.com/', None, 'https://honeycombsaloncollective.com', None, 'http://sunwest-az.com', 'http://www.rustikrotisserie.com', 'http://www.eliesfamilyrestaurant.com', 'https://www.lennar.com/New-Homes/Nevada/Las-Vegas/Las-Vegas/Mountains-Edge/Monterey-Ranch-The-Gardens/INHC', 'https://playgroundsdartzone.com', 'https://stores.peoplesjewellers.com/on/markham/5000-highway.html?cid=YEXT', 'http://www.tuttisantibynina.com', 'http://www.redsealnotary.com', 'http://www.cheekychicbb.com/', None, None, None, 'http://www.garciabailbonds.company', 'http://www.el-chameleon.com', 'http://www.miamistarglo.com', 'http://www.felinefabulous.com/', 'http://firebyjames.com', 'http://www.lumiermedicalspa.com', 'https://www.thaithaispaandmassage.com', 'http://www.lisaeleyphotography.com', 'https://corebodypilatesaz.com/locations/north-gilbert-pilates-studio', 'https://stores.thenorthface.com/NV/LAS-VEGAS/USA314/', 'http://local.firestonecompleteautocare.com/ohio/rocky-river/21210-center-ridge-rd/?lw_cmp=IYP_YPC_MLP_2089', 'https://leancanteengcu.wixsite.com/healthy', 'http://www.hargta.com', 'http://www.vanish-ink.com', 'http://www.elsombrerovegas.com/', 'http://www.SkatelandMesa.Com', 'http://www.aiellospizza.com', 'https://www.tristastevensonstylist.com', 'http://www.marisolcu.org/', 'http://www.theforumbarpar.com', 'http://www.usps.com/', None, 'https://www.carcolv.com', 'http://www.blacks.ca', 'http://barredefinitions.com', None, 'https://nailsatthelakes.business.site/?m=true', None, 'http://www.menschssecurityfilms.com', 'https://hovdeproperties.com', 'http://www.carolinasmedicalcenter.org/body.cfm?id=14', None, 'http://www.dentproaz.com', 'http://www.littlebangkokthairestaurant.com', 'http://thegallerysouthend.com/', 'http://www.desertsandsrvresort.com', None, None, 'http://www.townofyork.com', None, None, 'http://www.onedaytours.com', None, 'http://www.abercrombie.com', None, None, 'http://www.elpolloloco.com/locations/store/3859', 'https://dripedgeroofing.com', '1592228355', '1592228358', 'http://www.sleepnumber.com/?utm_source=Yelp&utm_medium=CPC&utm_term=CTA&utm_campaign=9f4Lf6iyxkIXfQw-hozi2w&cm_mmc=Yelp-_-Yelp-_-WebsiteLink-_-9f4Lf6iyxkIXfQw-hozi2w-904825&tag=904825', None, 'http://www.fairviewmall.ca/EN/Directory/Stores/Pages/NormaReed_1084A.aspx', None, None, None, '1592228370', 'http://www.qualityhomeinspectionspittsburgh.com/', 'https://mattressbyappointment.com/locations/nv/lasvegas/', 'http://retinanevada.com/', '1592228381', 'https://drcooperortho.com', None, 'http://www.brookcreekapartments.com', None, 'https://www.chevronwithtechron.com', 'http://www.citi.com', '1592228400', None, 'http://www.hobobillstrainhousellc.com', 'http://www.maidstr.com', 'http://www.teavana.com', 'http://www.webuycars.com/location', 'http://drphonerx.com', 'http://www.jaquelineandrea.com', 'http://sc-clinic.com/clinic/bay_and_college/', 'http://www.quailhollowresort.com/page/dining', None, None, 'http://www.chtools.ca', 'http://www.silvercollectionatthepark.com/?utm_knock=y', None, 'http://www.nvcoin.com', None, 'http://www.petco.com', None, 'https://www.giggles.ca/fr', 'https://www.associatedbank.com/location/WI/madison/2001-londonderry-dr', None, 'http://www.cascadeautogroup.com', 'https://eatcopperbranch.com', 'http://www.ZahlerProperties.com', 'http://www.somethinscooking.com', 'https://www.levinfurniture.com/furniture-store/oh/mayfield-hts/6051-mayfield-rd', '1592228466', None, 'http://www.bvinsurance.ca', None, 'http://www.csn.edu/library', 'http://www.waxon.ca', 'https://www.davidsbridal.com/?utm_source=Yelp&utm_medium=OrganicSearch&utm_term=84&utm_content=DisplayURL&utm_campaign=WebsiteListing', 'http://affyspremiumgrillscarborough.com', 'http://www.asapcity.com', '1592228491', 'http://www.libertymutual.com/insurance-office-madison-wi', 'https://www.ihop.com/en/restaurants-south-las-vegas-nv/7490-s-las-vegas-blvd-3096', None, None, 'www.ripspub.com', 'https://melodys-chair.business.site/#gallery', 'https://www.upack.com', '1592228513', 'http://www.customcoaches.ca', 'http://www.midabui.com', 'http://www.bostonsgourmet.com/', 'https://www.arrowheaddentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.saberhealth.com', None, 'http://www.trainersclubaz.com', 'http://acdentalvegas.com', 'http://www.crateandbarrel.ca/Stores/Store.aspx?storeid=201', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'https://impactptaz.com/scottsdale/', 'http://choicehotels.com/Suburban', None, '1592228544', 'http://www.plasticmaninc.com', 'http://www.mnmshop.com', None, 'http://www.desertstorage.com/scottsdale-az-storage-units', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', None, 'http://www.clubmonaco.ca', 'https://parkwoodslaundry.com', None, None, '1592228568', None, 'http://venushomesaz.com', None, None, '1592228579', 'http://fabulousfloorscleveland.com', 'http://www.clorebeauty.com/', 'http://bluediamondbestbitedental.com', 'http://www.wildeeasttownehonda.com', None, None, 'http://www.bootmaster.com', 'http://www.freevegasclubpasses.com/yelp', 'http://www.planetfitness.com/gyms/glendale-downtown-az-1110?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://booyah-inc.com', 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', '1592228604', 'https://www.autonationbuickgmcwestsahara.com', None, 'http://www.highlandparkclubapartments.com', 'http://www.russellaquatics.com', 'http://www.eatatbobsdiner.com', 'http://www.huffsautomotive.com', '1592228619', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', None, 'http://www.freds-flowers.com', 'http://www.handandstone.ca/locations/jarvis-adelaide', 'https://www.azmobiletanning.com', None, 'https://www.audinorthlake.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, 'http://www.bunkhousekids.com/', 'http://www.flooringamericaofchandler.com/', 'http://bestbridalandtuxedo.com/', 'http://mcghies.com', None, 'http://www.lhsstudio.com', 'http://summerlinsmiles.com', 'http://alfornopizza.com/', 'http://www.thebatteryguy.com', 'http://www.piola.it', 'http://www.thaihousecalgary.com', None, 'https://agents.allstate.com/rosalie-gonzalez-las-vegas-nv1.html', 'https://www.champssports.com', 'http://www.greenlifecafemadison.com', None, None, None, 'http://www.napolipizzasqhill.com/', 'http://www.amptover.com', 'http://www.bronzecafe.com', 'http://www.envisionnv.com', None, 'http://www.conspirephoenix.com', 'http://findlayhonda.com', 'http://www.LendingPathway.com', None, None, 'https://www.walmart.com/store/2768/mesa-az/details', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://islingtongolfclub.com', None, 'http://HealthySkinLasVegas.com', 'http://cherryst.ca/', 'http://www.popsexotics.com', 'http://StrawberryPropertyManagement.com', 'https://www.extraspace.com/storage/facilities/us/pennsylvania/west_mifflin/1000000568/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, 'http://floresdecoloresart.com', 'http://www.chicos.com/store/home.jsp?cmp=MOC_yelp', None, None, 'http://www.cbdamericanshamanlv.com', None, 'http://www.sheacoco.ca', 'http://dollarcanada.ca', 'http://www.sapphirerealtyaz.com', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://greenwithenvyaz.com', None, None, 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'https://www.redlobster.ca/locations/list/on/toronto/20-dundas-street-west', 'http://lemodernjapanese.com', 'http://www.edwardsmovinglabor.com', 'http://www.deforestfamilyrestaurant.com/', 'http://www.stusmotorcycletowing.com', 'http://www.foxpeterson.com', 'http://luckyspizza67.com', 'http://www.catboardingcharlotte.com/', None, 'https://familybraces.ca', 'http://www.letsdodinneronline.com', None, None, 'http://allstarhairnailslasvegas.com', 'http://www.coldstonecreamery.com/stores/22323', None, 'http://www.straightedgewallpapering.ca/default.htm', 'http://www.tsalonmesa.com', 'https://spiceclub.foodticket.ca/foodticket/cgi/bestel.cgi', None, 'http://mardaloopwellness.com', 'http://www.vegasCapital.com', 'http://7eleven.ca/store-locator/33259', None, 'http://shaggychic.com', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', 'http://www.abbott-trophies.com/', 'http://www.aspca.org', 'http://archibaldmicrobrasserie.ca/index.php/fr/restaurant-blainville.html', 'http://cpm-apts.com', None, None, 'http://poshnailscharlotte.com', 'http://www.sallybeauty.com/', 'https://www.4wheelparts.com/stores/ohio/4wp-cleveland-oh-51', 'http://www.davesducts.com', None, None, 'http://www.trianglerealty.us', 'http://www.findlaychevy.com/', 'http://mountainpie.com', 'http://www.Districtlending.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.redmangousa.com', 'http://wtfexp.com', 'http://www.mi5tentertainment.com', None, 'https://www.bmoharris.com', 'http://www.ymcacharlotte.org', None, 'http://sphinx-cafe.com/', '1592228860', 'http://subway.com/', None, 'https://stoneandvine.com', 'http://www.papajohns.com', None, 'http://www.deltaco.com', 'http://illinishop.com', 'http://www.canadiantire.ca/en/store-details/on/warden-eglinton-on.store.html', 'http://medifastarizona.com', 'http://www.amorino.com', 'http://pdmtailor.com', '1592228887', 'https://www.casamiascottsdale.com', None, 'http://www.paragonphysicaltherapy.com', 'http://www.newviewrealtygroup.com', None, 'http://charlotteorthodontists.com/', 'http://www.clarkcountycourts.us/', 'http://www.redspaceevents.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.steaknshake.com/locations/23413-steak-n-shake-kenyon-road-urbana', 'https://www.cresthavengolf.ca', '1592228921', 'https://locations.wendys.com/united-states/ga/brunswick/164-altama-connector', 'https://www.palladiumeventcenter.com', None, 'http://www.royalganesha.com', 'http://mlk-smog-check.business.site', 'http://www.wemovewi.com', None, None, 'http://www.stevensplumbingac.com', 'http://akronfitnessandwellness.com/', None, 'https://www.wellsfargo.com', 'http://www.portofcleveland.com/environment-infrastructure/cleveland-lakefront-nature-preserve/', 'https://ad.doubleclick.net/ddm/clk/434046915;236895156;q', None, 'http://tiltedkilt.com/richmond-hill/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.investarrealestate.com/agent?xid=040101-01&name=Jacqueline-Kenoun', 'http://www.mirage.com/casino/race-and-sports-book.aspx', None, 'http://www.lilacbakeryaz.com', 'http://www.vegasvalleyphotography.com', 'http://desertmesapainting.com', 'https://www.chick-fil-a.com/arboretum', None, 'http://www.usmenuguide.com/honmachi.htm', 'http://www.cox.com', 'http://www.sgi-usa-lasvegas.org/', 'http://www.pancho-and-leftys.com', 'http://www.shell.com/', 'http://www.lofthairlounge.ca', 'https://www.metro.ca/trouver-une-epicerie/44', None, None, 'http://www.vfwpost3513.com', None, 'https://lasvegasurology.com', 'http://www.animalhousevetqueencreek.com', 'http://www.CLTvictor.com', None, None, 'https://www.shearexcellence.com/dilworth/', None, 'https://www.choicehotels.com/CN803?mc=blypypaf', 'https://thejoyfullkitchen.com', 'http://www.dbksalon.com', 'http://qdoba.com', None, 'http://www.francescasfaceliftoflasvegas.com', None, None, 'https://local.safeway.com/safeway/az/gilbert/90-s-val-vista-dr.html', None, None, 'http://massagemaniaglendale.com', 'http://www.chivino.com', None, 'http://www.integritytransportationpgh.com', None, 'http://www.dynastybuffetnc.com', 'http://www.dvf.com/', 'http://www.casitassanmarcos.com', 'https://sincitymoto.com', 'http://www.bodyandsoultherapy.ca', None, 'http://www.grapevinecabinets.com', 'http://www.arizonadanceartistry.com', 'https://www.simonmed.com/locations/all/simonmed_imaging___metro_ii', 'http://www.nps.gov/jaga/', None, 'http://www.foothillsautoglassphoenix.com', 'http://www.iloveichibanaz.com', None, None, None, 'http://lahainagrillLV.com', 'http://www.adcopaint.com', None, 'http://everydaycafepgh.com', '1592229087', 'https://desertcastlepc.com', 'http://www.stonecreekroofingaz.com', None, None, 'http://www.bfshomewarranty.com', 'http://thomashinds.ca', 'http://www.dermatiquemd.com', 'http://www.bathemp.ca', 'https://nice-garden-landscape.business.site', 'http://www.browart23.com/location.html', None, 'http://www.millerframe.com', 'http://www.JimmyJohns.com', 'http://www.fabriziovegas.com', 'http://chensnoodlehouse.business.site', None, None, 'http://www.ridersinn.com', 'http://www.radjewellery.com', 'https://www.statefarm.com/agent/us/az/phoenix/eric-jackson-x4xfz21d000?cmpid=r6sy_blm_0006', 'http://yelp.com/elite', 'http://www.itsabaotime.ca', 'http://www.azchoicechiro.com', 'http://www.apple.com', '1592229145', 'http://www.superstarcarwashaz.com', 'http://futuredirectionsspa.com', 'http://www.heatherkphotography.net', 'http://www.cvcheart.com/scottsdale-princess-office/', 'http://jfkfinancial.com', 'http://www.canadiantire.ca', None, None, None, '1592229167', 'http://www.samuraicomics.com', None, 'https://nationwidevision.com/location/las-vegas-vision-center-silverado-ranch', '1592229176', None, 'https://ad.doubleclick.net/ddm/clk/435408715;237471434;h?https://www.hrblock.com/block-advisors/local-tax-offices/arizona/gilbert/67-s-higley-rd/2404?otppartnerid=9308&campaignid=pw_mcm_9308_9763', 'http://www.menchies.com', 'https://www.getpreloved.com', 'http://www.thainaka.ca/', 'http://retail.monroehardware.com/', 'http://cualv.com', 'http://www.starbucks.com', 'https://www.pteglv.com/locations/sean-patricks/deer-springs-5th-st/', 'http://www.mooseandbeargrill.com', 'http://assortedtablewine.com', '1592229204', 'https://www.haircutmenblakeneycharlottenc.com', 'https://www.basculebrewpub.com', 'https://www.aramarkuniform.com/our-locations/aramark-uniform-services-phoenix2', None, 'http://myloannailslasvegas.com', 'https://www.walmart.com/store/1093/rantoul-il/whats-new', None, None, None, None, 'http://www.mrsub.ca', 'https://www.metropcs.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.francisandsonscarwash.com/category/7889/francis-and-sons-car-wash-of-peoria-az.htm', 'http://www.swisschalet.com', None, 'http://www.nail-pretty.com', 'http://www.southernhighlandscyclery.com', None, 'http://www.wpxi.com', 'http://docromeopark.tk', 'http://guitartreemusic.com', None, 'http://www.dennysallnighter.com/', 'http://www.queencityclosets.com/', 'http://onestopnutrition.com', 'https://www.flavours-ajax.com', None, None, 'https://heightspestcontrol.com', 'http://vsnails.com', 'http://www.desertflowerseniorliving.com', 'http://www.reclaimedart.org', 'http://www.drdonnawoo.com/', 'https://www.sunsetgrill.ca', 'http://cuttingedgebarbershoplv.com', 'http://www.lllreptile.com/', 'http://www.cencispizza.com/wexford', '1592229284', 'https://www.gowireless.com/stores/az/scottsdale/scottsdale-scottsdale-fiesta/', 'http://haneysfamilyjewelers.jewelershowcase.com', 'http://www.evpools.com', None, None, 'http://www.activatechiropracticclinic.com', 'http://glendale.mensvitalitycenter.com/', 'https://www.potterybarn.ca', 'http://www.learningexpress.com/store/robinson/', 'http://www.letzrolloffroad.com', 'http://quickresponseplumbing.ca', 'http://hamptoninn3.hilton.com/en/hotels/arizona/hampton-inn-and-suites-phoenix-goodyear-PHXGYHX/index.html?SEO_id=YELP-HP-PHXGYHX', None, None, 'http://www.districtflats.com', None, 'http://www.urbanashop.com', None, 'http://www.goodwill.org', 'http://lvfurnituredirect.com', 'http://locations.maggianos.com/us/ohio/beachwood/beachwood', 'http://www.mobil1mentor.com', 'https://cocofreshtea.ca', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Gilbert-AZ-85234/922024/', '1592229337', None, None, 'http://bk.com/', '1592229346', 'http://hkrealtygroup.com', 'https://www.sparkletts.com', None, 'http://isabelcleaning.blogspot.com/', None, None, 'https://www.jackentertainment.com/cleveland/', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://www.90degreesapartments.com', None, 'http://www.fourlakesfamilydental.com', None, 'https://www.choicehotels.com/north-carolina/matthews/sleep-inn-hotels/nc032/rates', None, 'https://valleywiderestoration.com', None, 'http://fitnesstogether.com/matthews?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://www.albertsons.com/home.html', None, 'http://www.bk.com', 'https://www.videogameswap.ca', 'http://www.summitdentallv.com', 'http://pekingman.ca', 'http://ismiledentists.com', 'http://handymanphoenix.com', None, 'http://www.vocellipizza.com/pleasanthills_pa', 'https://stores.sportsmans.com/sportsmans-warehouse/us/az/avondale/10145-w-mcdowell-road', 'http://bennybbq.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://thepetclub.com', 'http://www.zuppasdeli.com/', 'http://bacusbroshotdogs.com', 'http://www.toysrus.com/family/index.jsp?categoryId=2255956&camp=MISC:yelp:store:page:tru:111910', 'http://www.cityofmadison.com/parks/parks/park.cfm?id=1207', None, 'http://www.dominos.com/', 'http://www.mindfulmotionphysicaltherapy.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.centralonbroadway.com/', 'http://www.stashcharlotte.com', 'https://locations.sonicdrivein.com/sc/tegacay/3039-highway-160.html', 'http://www.jtblandscapers.com/', 'http://www.exxonmobilstations.com/1721617-tesoro-62540-las-vegas#_62540-mobil-las_vegas', 'http://www.newimagephotography.com', '1592229448', 'http://sweetvirginiawax.com/', None, 'http://www.perfectflooringpgh.com', 'http://www.macmediainc.com', 'https://alohanailsblakeney.com', 'http://www.rinaldisdeli.com', 'http://www.wizardworld.com/cleveland.html', 'http://www.restaurantcrescendo.com/', None, 'https://www.performancemuffler.net', None, 'http://royal10cigar.com/', 'http://ashavecomics.com', 'http://www.brdiamondsuite.com', 'http://www.laurasecord.ca', '1592229483', 'https://5thavedeliandgrill.com', '1592229487', 'https://www.simon.com/mall/south-hills-village', 'http://www.positivechanges.com', 'http://www.aeacarizona.com', 'http://ajiichibansushi.com/', 'https://www.clearchoice.com/locations/dental-implants-las-vegas/?trans_id=webdisyelp&utm_campaign=Yelp&utm_source=YelpListing&utm_medium=Listing', None, 'http://www.dunkindonuts.com', None, 'http://www.superpumper.com', '1592229508', 'https://www.stance.com/retail/stance-scottsdale', 'https://www.tombstonetactical.com', 'http://www.lebouquetfloristandgifts.com', 'http://www.azmagician.com', 'http://www.whitehouseblackmarket.com/store/home.jsp?cmp=MOC_yelp', 'http://www.monroevillechryslerjeep.com', 'http://www.familylifecenters.org', 'http://locations.harristeeter.com/nc/charlotte/205/', 'http://365gymllc.com', 'https://milios.com', None, 'http://www.padellaonavenue.com', None, 'http://crurestaurant.ca', 'http://www.arizonapaintingcompany.com/', None, 'http://www.equityapartments.com/Arizona/Phoenix-apartments/Scottsdale/San-Marcos-apartments.aspx', '1592229547', 'http://www.calakmulaz.com', 'http://www.pamperedpoocheslasvegas.com', None, 'http://torontovintage.ca', 'http://www.preservationcleaning.ca', 'https://www.homedepot.com/l/West-Aurora/IL/Aurora/60506/1957?cm_mmc=seo-yelp&', None, None, 'http://www.glowbeautycharlotte.com', 'http://www.attorneyramsey.com', None, None, None, 'http://www.kjsdental.net', '1592229581', 'http://www.buchanans.ca', None, 'http://www.khaanakhazanaaz.com/', None, 'https://www.torrid.com/store-details?StoreID=3512', 'http://www.croatiarestaurant.ca/', 'http://kblossomshop.com', 'http://www.phoenixlosburritos.com', 'http://drorona.com/', None, None, 'https://www.signaturestyle.com/locations/wi/madison/cost-cutters-maple-grove-shopping-center-haircuts-16214.html', None, None, 'http://www.mountainpark.org', 'http://www.alamo.com/', 'http://sweettomatoes.com', 'https://nixrockstargarage.com', 'http://glownailbarhenderson.com', 'http://www.heartbreakersalon.ca', 'http://lahmajounsevan.com/', 'http://www.tapino.com', 'http://www.hotdogonastick.com', None, 'http://www.ScruffyToFluffyLV.com', 'http://www.sagamorehillsanimalhospital.com', '1592229642', None, 'https://www.wellsfargo.com', 'http://www.balzermotorworks.com', None, 'http://www.pdqstores.com/gmaps/Wisconsin/MadisonArea/131.html', 'http://www.meadows-chiro.com', 'http://www.levinfurniture.com/furniture-store/pa/pittsburgh/5846-forbes-ave', None, 'http://www.futurekiddie.com', None, None, None, None, 'http://www.starbucks.com', None, '1592229673', 'http://southpointcasino.com/landing?cid=ypsp', None, 'http://www.shigatsusushi.com', 'http://lasvegasballroomdancing.com', None, None, 'http://www.brakemasters.com', '1592229689', 'http://www.homehardware.ca/en/dealer-microsites/1582-7/contact-information.htm', None, None, None, 'http://www.ascentcle.church', 'https://www.5spice.ca', '1592229702', 'http://www.hairbymegananton.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.charlottepediatricclinic.org', 'http://mastcoffee.ca', 'http://strutbridalsalon.com/', 'https://doctors.bannerhealth.com/provider/Daniel+Arrhenius/776226', 'http://www.bk.com', None, 'http://www.ultralighting.ca', None, 'http://ak-chinpavilion.com', 'http://www.zara.com', 'http://amyhogan.ca', 'http://www.chilis.com/', 'http://www.rainstormcarwash.com', 'http://Moreheadtavern.com', 'http://www.camelbackwomenshealth.com', 'https://az.aaa.com/agent-office-locations/nv/las-vegas/aaa-west-lake-mead-branch', 'https://www.luvasub.com', 'http://www.citysmokeclt.com', None, 'http://www.nailparade.com', 'http://stores.sallybeauty.com/az/mesa/beauty-supply-mesa-az-3292.html', None, 'http://www.Jerksogood.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.accuratecargo.com', 'https://losprimosmeats.com', 'http://www.grottolounge.com', 'https://www.plazaexecutive.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=KL%20execsuite', None, 'http://matthewscarwash.com', 'http://www.bbqislands.net', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://zscreamandbean.com', 'http://www.waxmuseum.net/', 'https://www.dominos.com', None, 'http://www.mccleanerslasvegas.com', 'http://jcls.jazzercise.com/facility/carnegie-jazzercise-center', 'http://duclostoys.com', 'http://www.therightguyshvac.com', 'http://www.thegourmetcave.com', 'http://tremontfarmersmarket.com/', 'http://www.nemdelight.com', None, 'http://www.mohaveeyecenter.com', 'http://www.yumeisushi.ca', None, 'https://www.early-bird.ca', None, '1592229815', 'http://www.pbjpittsburgh.com', 'https://www.starbucks.ca', 'http://clickserve.dartsearch.net/link/click?lid=43700006435129272&ds_s_kwgid=58700000444951114&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.conditandassoc.com', None, 'http://www.immortalsinconline.com', 'http://www.butterchickenroti.com', 'http://www.kirklands.com', 'http://www.eatgarbanzoaz.com/', 'http://thevertigogames.com', 'http://www.stadtcafetoronto.com', 'https://locations.waxcenter.com/nc/charlotte/charlotte-arboretum-0169.html', None, 'http://www.privyskincare.com', 'https://www.koekkoekfood.com', 'http://www.burgundysrestaurant.com/', None, 'http://www.usagymaz.com', 'http://www.bodhitreeacupuncture.com', 'http://www.tendertouchmoving.com', 'https://thaiexpress.ca', 'http://www.tenniscabana.com', '1592229868', 'http://www.yogurt-land.com', None, 'http://drivenfix.com', 'http://www.rocknes.com/index.htm', None, 'http://www.econolodge.com/hotel-rock_hill-south_carolina-SC139?source=pmfyelp&pmf=yelp', 'http://www.petco.com', 'http://www.countryinns.com/scottsdaleaz', 'http://www.bigsurffun.com', 'https://studiocpb.mysalononline.com', None, 'http://www.acoresrotisseriechicken.com/', None, 'http://www.oglolas.com', 'https://www.idealimage.com/medspa-locations/pennsylvania/pittsburgh-area/pittsburghwexford', None, 'http://www.bojangles.com', 'http://www.5cinco5.com', 'https://www.indiapalace.ca', 'https://crownpublichouse.com', None, None, 'https://twomaidswexford.com', None, 'http://www.luxdesign.ca', 'https://www.arizonaanimalwellnesscenter.com', 'http://www.clarkcountynv.gov/Depts/parks/Pages/winchester-cultural-center.aspx', 'http://www.renewalbyandersen.com/window-company/720-phoenix-az.aspx/?utm_source=Yelp&utm_medium=localization&mm_campaign=B5863EAA37E0931E94FEB25E4DEEDB54&mm_replace=true&engName=social_Yelp', 'http://www.shopjustice.com', None, None, 'http://www.perfecttimingentertainment.com', 'http://www.bsyhairstudio.com', None, None, 'http://www.welcomepharmacy.ca', 'http://www.friendshipsuites.com/', None, 'https://locator.chase.com/az/cave-creek/28238-n-tatum-blvd', None, '1592229955', 'https://www.panerabread.com/en-us/home.html', '1592229959', 'http://www.crossfitfury.com', 'https://order.qdoba.com', 'http://infinitydentallv.com/', 'http://www.maintenance-pro.com', None, 'https://www.brennansmarket.com', 'http://jetspizza.com', 'http://www.onthegrill.us', 'http://www.myrivertowne.com', 'http://www.plapreschool.com', 'http://www.dpauljewelry.com', None, None, None, 'http://www.forgoodnesscakesaz.com', '1592229997', None, None, 'https://www.eyeglassworld.com/store-list/las-vegas-(s)-nv-89121/', None, 'http://www.swansofinglewood.ca', 'https://www.aqualineplumberselectriciansacrepairphoenixaz.com', 'https://www1.shoppersdrugmart.ca', 'http://www.elirelocation.com', None, None, 'http://www.premiumsweets.ca', 'https://earls.ca/locations/calgary-tin-palace', 'http://www.beadworldinc.com', None, 'http://www.eastsidecannery.com', None, None, 'http://www.pooranivilaas.ca', 'https://www.navyfederal.org/branches-atms/locations/usa/az/phoenix/cmb/', 'http://www.landainails.com', 'https://www.accountingprincipals.com/locations/mesa-az-accounting-staffing-agencies', 'http://tendershawarma.ca', 'http://www.cherryaz.com', 'http://www.theindiegathering.com', None, 'http://www.prontomexicanfood.com', 'http://www.greatclips.com/', 'http://lasvegasrealestatetoday.com', 'http://www.terribleherbst.com/', 'http://freshdelicious.net/', 'http://www.selfconnection.ca', 'https://dealer.sprint.com/experts-choice', 'http://greatlakesbakingcompany.org', None, 'http://www.threedogbakerylkn.com', 'http://www.kiedrowskibakery.com/', None, 'http://www.boernercompany.com', 'https://www.ihop.com/en/restaurants-streetsboro-oh/9700-state-route-14-3606', '1592230084', 'http://pavelsjewelry.com/', 'http://www.suspectvideo.ca', 'http://www.tomasos.com', 'http://www.orenisbetter.com', 'http://az-ent.com', None, None, None, 'http://www.thespottedcouch.com', None, 'https://giorgio.ca/fr/restaurants/succursale/laval/', None, 'http://chiropractorpeoriaaz.com', None, 'http://www.gandgmarine.com', 'http://www.azdot.gov/mvd/', 'http://reeis.com', None, 'http://www.cuffsclothing.com', 'http://www.candomoving.com', 'http://www.cappysonwalnut.com', None, None, 'http://baraongles.ca', 'http://www.ellenmerkinod.com/', 'http://www.lynnbuss.com?cmpid=lvji_blm_0006', 'https://www.unitypoint.org/madison/home-health.aspx', None, None, 'https://www.ditkasrestaurants.com/Locations/Wexford', 'http://embassysuites3.hilton.com/en/hotels/pennsylvania/embassy-suites-by-hilton-pittsburgh-international-airport-PITATES/index.html', '1592230150', 'http://greenvalley.schoolofrock.com/?labelSource=Yelp&utm_campaign=yelp&utm_source=yelp&utm_medium=yelp&utm_content=yelp', None, 'http://www.lamexicana.ca', 'http://www.mymexquite.com', None, None, 'http://www.bayti.ca', None, 'http://www.rhythmcatsshow.com', None, 'http://bcpoolsupply.com', 'http://www.besolasvegas.com/', 'http://www.allclimatenc.com', 'https://www.wellsfargo.com/locator/bank/32__CATAWBA__ST_BELMONT_NC_28012/', 'http://www.innzonerainbow.com', 'http://www.airparkbike.com/', 'http://www.tolmtaz.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3825+Monticello+Boulevard-Cleveland+Heights-OH-44121/storeid=3337?cid=ps_ylp_2020', 'http://www.breathinghands.com', 'http://www.tropicalsmoothiecafe.com', 'http://phoenixnap.com/', 'http://www.canadel.ca', None, 'http://www.rogerkdowney.com/?cmpid=rkr3_blm_0006', 'http://www.caldensebakery.ca/', None, 'http://www.cegdealers.com', 'http://www.toysrus.ca', None, None, None, None, None, 'http://www.lvprestigecleaning.com', None, 'http://www.chtoyota.com', 'https://www.petersendean.com/?utm_source=yelp&utm_medium=referral', 'https://locations.waxcenter.com/pa/mccandless/mccandless-0287.html', 'http://www.turcopersian.com', 'http://www.haji-baba-tempe.com/Haji-Baba.com', None, None, 'http://www.jojosmenu.com', 'http://www.proficientclean.com', None, 'https://www.popeyes.com/store-locator/store/restaurant_82776', 'https://www.mattressfirm.com', 'http://www.snohc.com', 'http://www.napaautopro.com/auto-repair/alberta/calgary-renfrew-auto-service/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://www.ezpawn.com/locations/nv/henderson/10075-eastern', None, 'http://www.ultimatecrc.com', 'http://www.camelbackcab.com/', 'https://www.duluthtrading.com/locations/?city=Belleville&StoreID=004&street-address=1107RiverSt.', 'http://www.chandlersmiles.com', '1592230275', 'https://www.forever21.com', None, None, None, 'http://www.nextcare.com/locations/az/98th/?utm_sourceYelp&utm_mediumLocal&utm_campaign98th', None, None, 'https://www.mattressfirm.com', 'http://www.gianteagle.com', None, 'http://www.boltonpizzapanini.ca', 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2foh%2fakron%2fcopley-east&sisearchengine=1524&siproduct=2557_Bus', None, 'http://www.thanglong.ca', 'http://azfamilydentistry.com', 'http://www.deltaco.com', 'http://www.awakennutrition.com', 'http://www.alphasdiscoveryclub.com/indoorplayground', '1592230318', 'http://www.flourandbarley.com', None, 'http://www.nevadacareerinstitute.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/az/fountain-hills-5029.html?mcid=iyp:8368400', 'http://sonedental.com', 'http://www.get-go.com/storelocator/storedetails.aspx?storeid=294', None, 'https://www.goodyearautoservice.com/en-US/shop/AZ/Goodyear/000908232-85338-1430', None, 'http://www.atlasespressobar.com', 'http://www.foothillsrehab.com/', None, 'http://www.aliveandfreehypnosis.com', None, 'http://www.pizzaiolo.ca/?utm_source=Yelp', None, 'http://floorshop.ca', None, 'http://www.southcharlotteanimalhospital.com', None, 'http://www.morfurniture.com', 'http://www.royalpgh.com', 'http://navarautoandtire.com', 'https://www.jerseymikes.com/8009', '1592230379', 'https://www.hilton.com/en/hotels/clehtht-home2-suites-cleveland-beachwood/?SEO_id=YELP-HT-CLEHTHT', 'http://www.kay.com', '1592230386', 'https://www.davidcreechmd.com', 'http://www.rabbatphoto.com', 'http://eddysautoservice.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYZOT', 'http://www.estrestaurant.ca', 'https://www.180automotive.com', 'http://www.proaquatic.ca/', 'http://www.fastbiryani.com', None, 'http://asairoofing.com', 'http://www.lightrx.com/arrowhead', None, 'http://demetres.com', 'http://www.cre8iveevents.com/', 'https://www.4patientcare.ws/wsv3pro/web/webschedulerv3.aspx?SessionID=2050212', 'https://www.bcautospecialists.com/index.php', 'http://www.JimmyJohns.com', 'http://www.stonetheatres.com/#/sun-valley-14/4579599036', 'https://canyonstateac.com', 'http://www.inexcreativefloors.com', 'http://www.hotelvalleyho.com/vh-spa', None, 'http://www.rhodesranchgolf.com', 'https://www.isluxury.com', 'http://www.yogastyles.com', 'https://pho-oc-vietnamese-restaurant.business.site', None, None, 'http://www.arrowheadoralsurgery.com', '1592230459', 'http://www.fnfcarpetcleaning.com', None, None, 'http://www.slashsalon.com/', 'https://www.mgmresorts.com/en/hotels/united-states/mgm-northfield-park.html', 'http://musa-restaurant.business.site', '1592230477', 'http://www.wellsfargo.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://gourmetgringos.com', 'http://www.enlivant.com/communities/arizona/kingswood-place-surprise/?utm_campaign=local&utm_medium=Y&utm_source=listings', 'http://www.tiltedkilt.com/', 'http://www.azskindoc.com', 'https://www.tranquilwatersmedspa.com', 'http://www.KulturaToronto.com', None, 'http://www.kristinachartier.com', '1592230508', 'https://www.theprosperitygoddess.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FCLTCG', 'http://www.walmart.ca/en/walmart-meadowvale-supercentre/3054', 'http://www.harlemglobetrotters.com/', 'http://www.affordableappliancerepair.net', 'http://phoenixchildrens.org?utm_medium=local-listing&utm_source=chatmeter', None, None, 'http://www.avenuetaphouse.com', 'http://www.torrid.com', 'https://whiteoakanimalsafehaven.com', None, 'http://www.campusdish.com/en-US/CSW/UNLV', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.boomerangpilates.ca', 'http://www.lvsubzero.net', 'http://genghisgrill.com', 'http://www.pinkhouseboutique.com', 'http://www.sdmattress.com', 'http://www.prairiewindsvet.ca', None, None, None, None, 'https://www.pteglv.com/locations/pts-gold/st-rose-eastern/', None, 'http://theloftmedspa.healthcare', None, 'http://gottabegarys.com', None, 'http://www.thebiddlelawfirm.com', None, '1592230586', None, 'http://www.cookiesbygeorge.com', None, 'http://aquinolawgroup.com', 'http://skinnywimpmoving.com', 'http://locations.whichwich.com/AZ/Tempe/Tempe-Gateway', 'http://clevelandfurniturebank.org', 'http://thedizzy.ca/', 'http://www.libertyclinic.com', 'http://www.windowtreatmentslasvegasnv.com', None, 'http://www.portofsubs.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.familydollar.com', 'https://visionsource-studio-2020.com', 'https://locations.traderjoes.com/az/tempe/93/', None, 'https://www.galleryhiart.com', 'http://dyslexiacenter.com', 'http://www.latinmasstimes.com/Nevada/446', None, 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=891&utm_content=bpas', 'http://pipscafe.com', None, None, 'http://www.royaloakdentalcalgary.com', 'https://www.choicehotels.com/arizona/tempe/comfort-inn-hotels/az498/rates', None, None, None, None, 'http://www.aquasealwaterproofing.com', 'http://www.lovinghut.ca', 'http://www.rhinoexterior.com', 'http://store.concept1.ca/store/', 'http://havenhg.com', 'http://WorldsTallestChiropractor.com/', 'http://www.leahs.ca', 'http://thevaporsocial.com/', 'http://www.drpalluck.com', 'http://www.sunsetstation.com/dining/hooters/?utm_source=YELP&utm_medium=SOCIAL&utm_term=SS&utm_content=SS-PROPERTIES&utm_campaign=SS-YELP', 'http://www.newsonganthem.org', 'http://www.rosynails.ca', 'http://www.touchofthai.info', '1592230681', 'http://www.statebicycle.com', 'https://www.bodynbrain.com/scottsdale', 'http://greatscience.com/explore/exhibits/william-g-mather-steamship', 'http://www.arizonacharliesdecatur.com/', 'http://www.trainerjo.ca', 'http://www.myhenrybrown.com/?cs:pro=repman&cs:e=yelp', None, None, 'http://www.g1tour.com', None, 'https://urgentvet.com', 'https://www.mgmgrand.com/en/entertainment/grand-garden-arena.html', 'http://www.blimegreen.com', None, 'http://www.murgatroid.to', 'https://www.segwayofscottsdale.com', 'http://www.iandjfountainrestaurantaz.com', '1592230719', None, None, 'http://www.renegadecoffeeaz.com', None, 'http://www.earnhardtcjd.com', None, 'https://www.target.com/sl/north-olmsted/2016', 'http://schnitzelhub.com/', None, 'http://terramassagepgh.com', 'http://madjackspub.com/', None, None, '1592230745', 'https://www.mountainshadows.com', 'http://www.dukesupperdeck.com', None, 'http://www.qdoba.com/', None, 'https://nevadaeyephysicians.com/Henderson-Office', 'https://www.lebos.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://www.jetblue.com', 'http://doughboyspizzaandwings.com/', 'http://www.gordon-law-offices.com', 'http://www.callahans-flooring.com', 'https://madison.floorcoveringsinternational.com', None, None, 'http://www.festivalofcolorsusa.com/festival-of-colors/festival-colors-las-vegas-nv/', 'http://www.gigglingtomatoes.com', 'https://www.dominos.com', 'https://jose-cansecos-showtime-car-wash.business.site', 'http://lvpaiutesmokeshop.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNME', 'https://wholehealthphoenix.com', 'http://www.salondecheveux.com', 'https://www.sportchek.ca/stores/hillcrest-mall.html', 'http://www.thebrooklineplumber.com', 'http://www.arcadiafarmscafe.com', None, None, 'http://lasvegashears.com/audiologist-las-vegas/', 'http://carlitosmokes.com', None, None, 'http://www.mypittsburghlocksmith.com/', 'http://goodwillsp.org/shop/retail-stores/weddington-road', None, 'http://www.magiclaserandaesthetics.com/', None, 'https://www.nextdoorwinebar.ca', 'http://store-locator.barnesandnoble.com/store/2212', None, 'http://www.fallasstores.com', None, None, 'http://www.coopercoons.com', 'http://www.aatexlawn.com', 'https://zacharylawgroup.com', 'https://local.executivecoin.com', 'http://www.azrealsolutions.com', 'http://www.thunderbirdim.com', 'https://www.insynco.com', 'http://www.drstevensabatino.com', None, 'http://www.lasvegasdogtrainers.com/', 'http://progressivedentalandortho.com/', 'https://www.chapmanhyundaiphoenix.com', None, None, 'http://www.a-natural-u.com', 'https://www.redstarvapor.com/stores/vape-shop-gilbert/', 'http://www.hcpnv.com', 'http://www.twtdbooks.com', 'https://www.stylistics.net', None, 'http://www.ashandkris.com', None, 'http://Www.SerenityRichmond.com', None, None, 'http://www.patrickssigns.com', 'http://www.havanacafe-az.com/', 'http://www.mogheltandoori.com', 'http://alpremium.ca/', 'http://www.wincofoods.com', None, None, None, None, None, 'http://www.mygreentreeapts.com', 'http://www.bearcreekaz.com', None, 'https://fixautousa.com/las-vegas-craig', 'http://www.inmotionstores.com', 'http://www.cadencecoffee.com', None, None, 'https://randrpaintingaz.com', 'http://www.labcorp.com', 'http://www.sunautoservice.com/auto-repair/nv/henderson', None, 'http://www.dubinwellness.com', None, 'https://www.alamo.com/en_US/car-rental/locations/location-details/US/PITT71-pittsburgh-international-airport.html?iataNumber=AL022237&affiliateRes=Y&cm_mmc=Yelp-_-IYP-_-National-_-null', 'http://www.pitaland.ca', 'http://www.tuttigelati.com/', 'http://www.burroburrito.com', None, 'http://terravitavet.com/', None, 'https://www.wingilariver.com/index.php/vq-vee-quiva-hotel-casino/dining/fatburger', 'http://imperialexcavatingaz.com', None, 'http://www.bartholomewphotography.com', 'http://lucillemadison.com', None, None, None, 'http://www.starbucks.com/store/9334/', 'http://www.decoraport.ca/francais/', 'http://www.johnnyrockets.com', 'http://boucherville.delfriscos.ca', 'http://beerbatter.ca', 'http://www.quiznos.ca', None, None, None, 'http://withoutwalls.tv/', 'http://Www.kristinlloy.com', 'http://www.letswrap.biz', 'http://www.truevalue.com/hometruevalue', 'http://www.allmetalsrecyclingllc.com', None, 'http://www.jumprestaurant.com/', None, None, 'http://www.chandlercustomcycles.com', None, 'http://www.bwh-services.com', None, 'https://locations.dignityhealth.org/oasis-hospital-phoenix-az', None, 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'https://locations.panerabread.com/oh/bainbridge-township/8480-east-washington-street.html', None, 'http://www.fred-vincentvw.com/', 'http://www.audreyjanellebeauty.com', None, 'http://www.lauraleesalon.com', 'https://www.bartlett.com/locations/Phoenix-AZ.cfm/?yelp_ads_lop%C3%8A', 'http://appliance-repair-pittsburgh.pa-biz.com', None, 'http://www.starbucks.ca', 'https://sandees-water-n-ice.business.site', None, 'http://www.orvis.com/cleveland', 'http://www.acehardware.com/store-details/06234', 'http://www.fantasticsams.com', None, 'https://www.availvapor.com/north-olmsted', 'http://www.georgiossalonspa.com/', 'http://www.sexyplusclothing.com', '1592231074', None, 'http://eatmegourmetcatering.com', 'http://www.onceuponachildstow.com/', None, None, 'http://www.pianotuneraz.com', 'http://www.framersworkshop.net', None, 'http://www.sunglasshut.com/us/?cvosrc=display.yelp.LasVegas', None, '1592231097', None, None, 'https://www.mandalaybay.com/en/nightlife/rhythm-riffs.html', 'https://www.walmart.com/store/4356/las-vegas-nv/details', 'http://www.bosmeadery.com/', 'https://www.steaknshake.com', None, 'http://www.dolcelucano.com/', None, 'http://storelocator.pepboys.com/pepboys/stores/phoenix/cactus/00747', 'http://www.jumparoundpartyjumpers.com', 'http://www.NextLevelTreeLLC.com', None, 'http://www.jimmyscoffee.ca', 'http://www.bigotires.com/regions/Nevada', None, 'http://www.onegooddoctor.com', 'https://www.aqualityelectric.com', 'http://www.actionappliancepitt.com', 'http://www.fetishfalls.com', None, 'https://www.camdenliving.com/scottsdale-az-apartments/camden-foothills?cm_mmc=Yelp', None, 'http://www.themilkshakefactory.com', 'http://www.opalasvegas.com/', None, 'http://relaxologywellness.com', 'http://www.greatclips.com/', 'http://l.macys.com/biltmore-fashion-park-in-phoenix-az?cm_mmc=Yelp_Stores-_-Phoenix-_-n-_-451&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Phoenix_451&m_ac=yelp_stores&m_ag=n', None, None, 'http://hmshost.com', 'http://www.pagoda.com', None, 'http://www.2fixmyair.com', 'http://lazioitalianbakery.com/', 'https://www.flyfrontier.com', 'http://warraichmeats.com/', 'http://bhelwalaindianfood.com/', 'https://www.enterprise.com/en/car-rental/locations/us/nv/north-las-vegas-craig-rd-5404.html?mcid=iyp:8368400', None, None, None, 'https://azvent.com/locations/estrella.php', None, None, 'http://www.ahealershand.com/', None, 'http://www.thedelevagroup.com', '1592231200', None, None, 'http://www.libertywildlife.org/', None, None, 'https://www.emilianos.net', None, 'http://www.klassynailscalgary.com', 'https://stores.cortfurnitureoutlet.com/arizona/chandler/150-south-kyrene-suite-1/?utm_source=yelp&utm_medium=organic&utm_campaign=localmaps', 'http://www.pintoranch.com', None, 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'http://www.institutefordriverssafety.org/', None, 'http://allautobattery2007.com', 'http://www.paulstailors.com/', None, None, 'http://www.purplesagesalon.com', 'http://www.rockwoodnaturalmedicine.com/', None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'https://www.westelm.com', 'http://robertostacoshop.com', None, 'http://www.banfield.com/veterinarians/nc/charlotte/bky?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89015/601-in-henderson/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-601&utm_content=website-link&howheard=IYE', 'http://banditolv.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.bigotires.com/regions/Nevada', 'https://www.appletreedentalforkids.com', 'http://www.lizzynco.com', 'http://www.skylarparkeraz.com', None, 'http://www.btvape.com', 'http://www.tenrestaurantaz.com', 'http://www.zoestrunk.com', None, 'https://www.popalock.com', 'http://www.zaborfh.com', 'https://www.sunamg.com', None, 'https://www.awomanschoiceinc.com', 'https://www.walmart.com/store/2720/madison-ms/whats-new', 'http://www.cannonschophouse.com', 'http://www.school-for-massage.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216494&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, 'http://www.aquariuslandscapinglasvegas.net', None, 'http://www.snowdentistry.com', None, 'http://www.westhillsnissan.com', 'http://www.marcos.com/', 'http://bermanchiro.net', None, 'http://www.lcbo.com/lcbo-ear/jsp/storeinfo.jsp?STORE=629&language=EN', 'http://www.myazsmile.com', 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89148/591-near-spring-valley/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-591&utm_content=website-link&howheard=IYE', None, '1592231336', None, None, 'http://agentpestcontrol.com', 'http://www.gilleyslasvegas.com', 'http://www.drsmarkantone.com', 'http://www.dollarama.com', 'http://www.vbsconstruction.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.rockruninn.com', 'http://www.jamburritos.com', 'http://www.icenowaz.com', 'http://www.thebarrelinn.com', 'http://www.pierdeorleans.com', None, 'http://www.transitionsquad.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.hennashoppe.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'http://www.buffalophoto.net', 'http://www.skyzone.com/ca/mississauga', 'https://www.destinationhotels.com/tempe-mission-palms/dining/tempe-restaurants', 'http://www.summitfamilydentist.com', 'http://www.notoriousautodesigns.com', 'https://www.wellsfargo.com', 'http://www.walmart.com/', '1592231397', 'https://centralusa.salvationarmy.org/danecounty', 'http://www.thetheodore.beer', 'http://www.michaelruffattodmd.com', 'http://www.quiktrip.com', 'http://www.metropolis-living.com/', 'http://www.buildabear.com?sc_cid=ds:yelp:profile', None, '1592231415', 'http://lagourmandinebakery.com', 'https://www.artclassesforkids.com', None, 'https://www.coulternissan.com', 'http://www.harborfreight.com', 'http://www.loveandcaresitters.com/', 'http://scottsdaleprinting.org', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.wallacefloral.com', 'http://kellyclark.com', 'http://www.papa-luigi.com', None, 'https://www.wellsfargo.com', 'https://nativegrillandwings.com/location/pebble-creek/?utm_source=Yelp&utm_medium=Local&utm_campaign=Pebble_Creek', 'http://www.skinnydippingpools.com', 'http://www.kfc.com', '1592231454', 'https://eyetique.com/squirrel-hill', None, None, 'http://www.bargainballoons.ca', 'http://www.sonicdrivein.com/', 'https://www.erbertandgerberts.com/locations/?store_address=53703', 'http://www.papagnos.com', None, 'http://www.johnnymcguires.com/vegas/', 'http://www.beaumontalignment.com', None, 'https://www.dapperandstout.com', 'http://www.uhhospitals.org/north-olmsted/urgent-care', 'https://www.barnettfalls.com', None, 'http://www.greatdanepub.com/', 'http://www.skodagordon.com', 'http://www.raffmobilemechanic.com', '1592231495', None, 'http://www.maac.com/south-carolina/columbia/the-fairways?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'https://www.walgreens.com/locator/walgreens-8816+e+pinnacle+peak+rd-scottsdale-az-85255/id=3602', None, 'http://www.dannburg.com', None, None, None, 'http://routes.ca', 'https://locations.tiaa.org/nc/charlotte/8500-andrew-carnegie-blvd?tc_mcid=bn_yelplocal_0618', 'http://sweetpeacafe.moonfruit.com/', 'http://www.EveryKidsDentist.com/?sc_cid=Yelp_Listing', 'http://www.healthyplanetcanada.com/stores/index/view/store/13', None, 'https://squarefeat.com', 'http://www.advancedtherapeutics.ca', 'http://www.eighteeneight.com/beachwood-oh', '1592231531', 'https://www.moes.com/find-a-moes/pennsylvania/wexford/100189?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', None, 'http://www.arizonabestmaids.com', None, 'http://www.amournailsandspa.com', None, None, 'https://www.hilton.com/en/hotels/phxgthx-hampton-suites-phoenix-gilbert/?SEO_id=YELP-HP-PHXGTHX', 'https://www.cooperfamilydentistryaz.com', None, 'http://www.hendersonchevrolet.com', 'http://www.dentalassistingaz.com', 'http://www.senecacollege.ca', None, 'http://philthyphillys.com', 'https://www.kaltire.com/en/stores/locations/?StoreID=694&source=yelp_track', 'https://www.restoration1ofgreaterlasvegas.com', None, 'http://www.arizonalasik.com/', 'http://www.downtownhomeinn.com', 'https://stores.petco.com/pa/pittsburgh/pet-supplies-pittsburgh-pa-1812.html', 'https://www.saigeboutiquesalon.com', 'https://barbaraannsalterations.com', 'http://www.vivitaspa.com', None, 'http://footreflexologychandlergilbertaz.com/', 'http://www.rycoplumbing.com', 'https://www.petpeoplestores.com/strongsville-store.html', 'http://www.paulsacehardware.com', None, '1592231602', 'http://www.thegrubclub.com/a/daddydukes', 'http://mesaunderground.com/', 'https://refinedlightingaz.com', 'http://www.thechildbirtheducator.com', None, None, None, None, 'http://www.desertstorage.com/scottsdale-az-storage-units', None, 'http://www.petiterestaurant.ca', None, None, '1592231633', 'http://www.falcononeclt.com', 'http://www.fracascleveland.com/', 'https://stores.arhaus.com/oh/akron/2246-west-market-street.html', 'https://www.publicstorage.com/north-carolina/self-storage-charlotte-nc/28210-self-storage/1196?pid=wyelp&CID=1245&CHID=AFL', 'https://www.lolplay.ca', 'https://www.desertfinancial.com/Home/BranchDetails?branchId=101', 'http://www.somethingnewmedia.com', 'http://www.mughaircuts.com/', 'http://www.pcmloan.com', None, None, 'http://sanfrancescos.com', 'https://www.thephoenician.com/phoenician-tavern/', 'http://www.aeopowersports.com', 'https://www.redlobster.com/locations/list/az/peoria/7921-west-bell-rd', 'http://locations.arbys.com/us/nv/north-las-vegas/1465-w-craig-rd.html', None, 'http://www.usps.com/', 'https://az.aaa.com/agent-office-locations/az/phoenix/aaa-paradise-valley-auto-repair', None, 'http://www.ellamentssalon.com', 'https://shuyi.ca', 'http://www.macxn.com/stores/madison-wisconsin', 'http://pedispanailbar.com', 'http://www.vaudreuilvolkswagen.com', 'https://www.anytimefitness.com/gyms/2160/Scottsdale-AZ-85266/?utm_source=yelp&utm_medium=local', 'http://www.mcdonalds.com', 'http://www.patriotmovers.us', 'http://www.furnitureappliancemart.com/current/Stores/Madison_West_Wisconsin_Furniture_Store.aspx', '1592231701', 'http://www.yorkvilleespressobar.com', None, 'http://www.italianvillagepizza.com/', None, None, None, 'http://www.rossstores.com', None, 'http://www.honeycombedhair.com', None, None, 'http://frostyvalleygolf.com/', 'http://www.builderzwarehouse.com', 'https://www.callawayirrigation.com', 'http://www.Boothbuilt.com', 'https://www.lalasercenter.com/centennial/?utm_source=yelp&utm_medium=centennial&utm_campaign=centennial%20dermatology%20and%20laser%20cancer', None, None, 'https://www.idealimage.com/medspa-locations/nevada/las-vegas-nv-area/lasvegassummerlin', 'http://www.epoxyfloorsphx.com', None, 'http://www.bmdprinting.com', 'http://www.warfieldhairbar.com', 'http://www.jandsautoservice.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'http://Ljhaircuts.com', 'https://www.caesars.com/flamingo-las-vegas?utm_campaign=Yelp&utm_source=yelp&utm_medium=local&utm_term=FlamingoLasVegas&utm_content=hotel', 'http://www.bitsandbytesllc.com', 'http://www.timhortons.com/', 'https://stores.dickssportinggoods.com/nv/las-vegas/1266/', 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', 'https://www.zerorezmadison.com', 'http://www.sheppardvillagedental.com', None, 'http://www.avenue.com', '1592231786', None, '1592231790', 'https://www.camdenliving.com/tempe-az-apartments/camden-tempe', None, 'http://www.qdoba.com/', None, None, 'http://www.pizzahut.ca/', 'http://www.cafelabella.com/', None, 'http://www.agassiprep.net', 'http://www.scarlettplus.com', 'https://gilberteast.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.planetfitness.com/gyms/madison-west-broadway-wi-797?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://lisnoodles.com', 'http://idealcustomflooring.net', '1592231823', 'http://www.picanhalv.com', None, None, 'http://elpalenqueeastlake.com', None, None, 'http://universitycoinandjewelry.com', 'http://www.rajskitchennc.com', 'http://www.strongsvilleheating.com', None, 'http://www.thebutcherspantry.com', 'https://locations.dignityhealth.org/emergency-room-at-arizona-general-hospital-glendale', 'http://www.artisticalloys.com/', 'http://www.silkmedspa.ca/', 'http://www.arizonalodgingexperts.com/owner-support/scottsdale-az-property-management/', 'http://allntattoo.com', 'http://www.belleetboeuf.com', 'http://www.spice9cuisine.com', 'http://lensfactory.ca', 'http://www.mark-taylor.com/arizona/stone-oaks/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.aldoshoes.com/ca/en', 'http://www.shopartmart.com', 'http://www.womeninneed.net/', 'https://www.atozdiscountdepot.com', 'http://lifetimegaragedoorsaz.com', 'https://www.alphaandomegadogtraining.com', 'http://ajustbrewcoffee.org', 'https://www.brooklynbedding.com/showrooms/arizona/phoenix-buckeye/?utm_source=yelp.com&utm_medium=referral&utm_campaign=yelppage', 'http://www.yellowjersey.org/', None, None, 'http://www.cafelouise.ca', None, None, 'http://www.izzazu.com', 'http://www.posterjack.ca', 'https://local.firestonecompleteautocare.com/nevada/las-vegas/3770-s-maryland-parkway/', 'http://orangeleafyogurt.com/frozen-yogurt-store/OH/gates-mills-/6679-wilson-mills-rd?id=384', 'http://www.zaidmoving.com', 'http://www.exoticnailstouch.com/', 'http://www.cartelbrewery.com', 'http://www.greasemonkeyhuntersville958.com', 'http://dresslersrestaurant.com', 'http://superdupermovers.ca', 'http://aquasportscuba.com', 'http://mortlandpt.com/', 'http://www.firstimpressionnails.ca', 'http://www.coffeepubs.ca', 'http://www.empirebagels.com/', None, None, 'http://www.mcramyun.com', 'http://www.ledesmafootandankle.com', 'https://www.bobevans.com/our-restaurants/locations/146?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'https://www.papillon.com/las-vegas-tours/?utm_source=yelp&utm_medium=cpc&utm_campaign=papillon-las-vegas', 'http://www.bytesbackdatarecovery.com', 'https://www.macu.com/Branch-Locations/AZ/Mesa/Mesa-Branch/0033', None, None, None, None, 'https://www.regmovies.com/theatres/regal-manor-twin/1841?utm_medium=social+network&utm_source=yelp+listing&utm_campaign=Regal+Manor+Twin', None, 'http://www.barneys.com/location#!/stores/14', 'http://www.getenergysmarter.org', 'http://hooterscasinohotel.com', None, None, None, None, 'http://www.dianasseafood.com', 'http://www.theloadeddog.ca/', 'https://www.restaurantszibo.com/les-restos/boisbriand', 'https://www.prolinkprotection.com', None, 'http://www.hairhappiness.com', None, 'https://www.playfalcongolf.com', 'http://www.cpracademylv.com', 'http://www.quiktrip.com', 'http://www.kennelcarevet.com', None, 'http://www.athletico.com/locations/urbana/', 'https://www.sgtclean.com', 'http://brucehalllaw.com', 'http://www.westernskiesgolf.com/', 'http://homewoodsuites3.hilton.com/en/hotels/ontario/homewood-suites-by-hilton-toronto-mississauga-YTOHWHW/index.html?SEO_id=YELP-HW-YTOHWHW', None, 'https://www.medicentres.com/chiropractic/', None, '1592232037', 'http://home.vivoli.ca/', 'http://www.ncdentist.com', None, 'http://www.poetryjazzcafe.com/', 'https://locations.tropicalsmoothiecafe.com/nv/henderson/10612-south-eastern-avenue', 'http://www.malleys.com', None, 'http://www.junk-works.ca/locations/toronto-north', None, None, 'https://www.frontieraptmovers.com', 'https://www.hattendo.ca', None, 'http://www.housecallnotary.ca', 'http://bunsofsteelbootcamp.com/', '1592232072', None, 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=118&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-118', 'http://thekeyrealtyteamaz.com', 'http://www.arcadiachiro.com', None, 'https://www.180smoke.ca/vape-shop-etobicoke-islington-electronic-cigarettes', None, 'http://www.soulpepper.ca/', None, 'http://www.autotruckandfleetservices.com', 'http://hiltongardeninn3.hilton.com/en/hotels/north-carolina/hilton-garden-inn-charlotte-concord-CLTCOGI/index.html?SEO_id=YELP-GI-CLTCOGI', 'http://igniteperformance.net', 'http://www.smartbookingtravel.com/', None, 'http://www.sashandbustle.com/', 'http://www.bestratemtg.com', 'http://hibachisan.com/', None, None, 'http://www.acneadvisors.com', None, None, 'https://www.classictan.biz', 'http://www.cityofhenderson.com/henderson-happenings/parks-and-trails/locations-and-features/reunion-trails-park-and-amargosa-trailhead', 'http://juicdlife.com', '1592232129', 'http://www.tutortime.com/your-local-school/phoenix-az-6086/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', 'http://www.letstalktherapy.com', 'http://www.downtownerlv.com', 'http://www.melrosekitchenaz.com', 'http://www.somethinsweet.ca', None, None, 'http://www.climateprollc.com/', 'http://Www.artistichandsmassage.com', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', 'http://www.scottsdaleeyesurgerycenter.com', 'https://www.brewerairporttoyota.com', 'http://springfieldgolfresort.com/', None, 'http://www.bodyworkscentral.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.elpasobarbeque.com/', None, 'http://www.mitchiesmunchies.com', 'http://www.jardinalex.com/?srccode=yelp_track', 'http://www.homesmartinternational.com', 'https://mkbjj.com', 'http://www.outside-inspirations.com', 'http://www.mayereyecarelv.com', 'https://kimmynailsandspa.weebly.com', 'http://www.tree-removal-phoenix.com', None, None, 'http://www.e-mg.com', 'http://www.amayaexpress.com/', None, 'http://www.winners.ca', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FLASGR', '1592232211', 'https://www.loblaws.ca/store-locator/Glen+Erin+Drive%2C+Mississauga/1011?storeId=1011', 'http://www.tamalerialaurita.com', None, '1592232222', None, 'http://www.CharlotteHealthCenter.com', 'http://www.ihg.com/intercontinental/hotels/gb/en/cleveland/clehb/hoteldetail', 'http://www.rockymountaintails.ca', None, 'http://www.meineke.com', 'http://www.semprepizzapasta.com', 'http://symposiumcafe.com/woodbridgerestaurants.html', None, 'https://elementsmassage.com/Phoenix-Arcadia', 'https://graylinelasvegas.com', '1592232247', None, None, 'http://iwearthebest.com', 'http://www.repeteslv.com', 'http://www.managestaff.com', None, 'http://www.urbanpalm.com', 'http://www.livelifeblended.com', 'http://www.empire.edu/cosmetology-schools/north-carolina/charlotte-pineville', None, 'https://rightspacestorage.com/self-storage-locations/nevada/las-vegas-nv-storage-units/3043-N-Pecos-Rd/', None, 'http://www.phxortho.com', None, 'http://www.petvalu.com', 'http://hummusfactorylv.com', None, None, 'http://www.pelicanssnoballs.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4755+West+Ann+Road-North+Las+Vegas-NV-89031/storeid=2989?cid=ps_ylp_2020', 'http://nvpicapart.com', 'http://www.mrosmow.com/', 'http://www.lofgreens.com/', None, 'https://rtcatcafe.org', 'https://www.surpriserental.com', 'http://tjmaxx.tjx.com/store/stores/storeLocator.jsp?_requestid=62549', 'http://www.brooksbrothers.com/?CMP=SMC_Yelp_6006', 'http://www.btbouncies.com', 'http://www.tinnenfamilychiropractic.com', 'http://www.blushlane.com', 'http://archwaychandler.greatheartsacademies.org', None, None, 'http://www.grillies.ca', None, 'http://www.backcarechiropractic.com', 'https://www.mypetmarket.com', 'http://www.visionworks.com', None, None, 'https://www.anytimefitness.com/gyms/288/Las-Vegas-NV-89139/?utm_source=yelp&utm_medium=local', 'http://www.happyhauscleaning.com', None, 'http://www.steaknshake.com/locations/23949-steak-n-shake-cherry-road-rock-hill', 'http://www.irepex.com', 'http://glutenfreegoat.com', 'http://www.rossstores.com', 'http://www.dogdayslasvegas.com', None, 'https://www.allvalleycarcare.com', 'http://www.thepigeonguy.com', None, 'http://www.etchsalon.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.sugarfactory.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281357789&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', None, 'http://quicksmiles.com', 'http://www.southbrookpumpkins.com', 'http://www.starbucks.com/store/9298/', None, 'http://www.fender.com', 'http://www.arizonaveterinaryoncology.com', None, 'http://chollafamilydentistry.com', 'http://www.aliveprintshop.com', 'http://www.skipro.com', 'https://www.cyclegear.com', 'http://sunriseautosaleslv.com', '1592232418', None, 'http://www.warmspringssurgicalcenter.com', None, 'http://www.safewaytire.net/locations/mode/22/4623-superior-avenue-cleveland-oh-44103.aspx', 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=8248', None, 'http://radioshack.com', None, None, None, 'https://ad.doubleclick.net/ddm/clk/300150600;127077036;j', None, 'http://www.kinospainting.com/', 'http://www.themercurylounge.com', 'http://www.westcanadian.com', None, '1592232457', None, 'http://www.tequilaranchcleveland.com', 'https://www.peaceloveandlittledonuts.com/locations/hudson-oh', 'http://www.azsportscenter.com', 'http://www.UnitedBeautySupply.com', None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Cholla.aspx', None, 'http://www.chilinoodlecorner.com/', None, 'https://3scocoffeebar.com', 'http://leafandbloom.com', 'https://www.fastmed.com/urgent-care-centers/phoenix-az-walk-in-clinic-maryvale-parkway/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', 'https://www.homedepot.com/l/Cave-Creek/AZ/Phoenix/85022/480?cm_mmc=seo-yelp&', 'https://www.wyndhamhotels.com/hotel/30821?tel=18882935571&iata=00093785&cid=fe:bh:20161215:yl:pp:hawus:30821', 'http://mcithedoctorsoffice.ca/patients/37-clinic-Mt.-Pleasant', None, None, 'http://www.sabatinospizza.com', '1592232509', 'https://chatters.ca', None, None, 'http://www.greenvalleyrange.com', 'http://www.gangnamasianbbqdining.com', 'http://www.gemini-arcade-palace.com/', 'https://www.mcalistersdeli.com/locations/nc/cornelius/19930-west-catawba-avenue-suite-150-1067', 'http://www.ProOutdoorServices.com', 'http://www.winners.ca', 'http://la-dulce-espera.business.site', 'https://ad.doubleclick.net/ddm/clk/434331165;236814625;c', 'https://cortatefurniture.com', 'http://a519chocolate.com', 'http://gregstruckandautorepairil.com', None, 'http://www.visionsourcewestsahara.com', None, None, 'http://mobileglowaz.com/', 'http://donutparlor.com', 'http://www.greenvalleyobgyn-nv.com', 'http://www.awhitepondparadise.com', 'https://www.circlek.com/store-locator/us/glendale/5848-w-olive-ave/2706657', None, 'http://latinoservices.net', 'https://www.xtendbarre.com/studio/charlotte', 'http://sharpedgebeer.com/index.php/locations/brasserie', 'http://www.cavopgh.com/', 'http://www.ymcaofpittsburgh.org/sampson/', 'http://www.planetfitness.com/gyms/las-vegas-flamingo-nv-611?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.lovesac.com/', 'https://www.samsclub.com/sams/pagedetails/content.jsp?pageName=aboutSams&couponId=YLPADS&PID=YLPADS', None, 'http://thewingfactory.ca', 'https://lmp360media.com', 'http://locations.in-n-out.com/234', 'http://www.Learn2PlayNow.com/', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.JimmyJohns.com', 'http://www.lakeforestcc.org', 'http://www.brittwiththebrush.com', 'http://www.gentlemenscutlakewylie.com/', 'http://www.premierfoliage.com', 'http://www.playacabana.ca/', None, None, 'http://areyoukenilworthy.com/', 'https://www.interstatebatteries.com/locations/all-battery-center-peoria-il', 'https://www.homedepot.com/l/Arrowhead-Ranch/AZ/Glendale/85308/475?cm_mmc=seo-yelp&', None, 'http://www.scottclarknissan.com/?utm_source=yelp&utm_medium=business_url&utm_campaign=local-citations', None, 'http://www.goodlifefitness.com/locations/ontario/northyork/don-millseglinton?source=yelp-websitelink-northyorkdon-gl', 'http://fightertownpaintballpark.com/', '1592232641', 'http://www.barbecuedoctor.com', 'http://www.zinburgeraz.com/locations/gilbert', 'http://www.ifrrealestate.com', 'http://www.botanyflowers.ca', 'http://www.homedepot.com/l/Curity/ON/Toronto/M4B-3L8/7012', 'http://dimurasmoving.com', 'http://www.academyofballroomdance.com', 'https://www.redroof.com/extendedstay/hometownestudios/property/AZ/Phoenix/HTS1059', 'https://mhausnailbar.com', 'http://www.cataldi.ca', 'http://www.thenorvalpancakefactory.com', 'http://www.greenmontvet.com', 'http://realalbertamoving.com', 'https://www.hopewayfoundation.org', None, 'http://www.midas.com/phoenix/store.aspx?shopnum=1241&dmanum=684001', 'http://www.oomomostore.com', 'http://yuphas.com', 'http://lekensington.ca', 'http://us.christianlouboutin.com/ca_en/', 'https://www.redlobster.com/locations/list/il/champaign/1901-n-prospect-ave', None, None, 'http://www.calgaryexpo.com', 'http://www.cheesecakeetc.biz', None, 'http://www.cocktailemporium.com', 'http://west50.ca/', 'http://www.donvalleyendoscopy.com', None, 'https://www.interdent.com/gentle-dental/locations/az/peoria/14155-n-83rd-ave-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.chillinaz.com', 'https://www.oberweis.com/ice-cream-and-dairy-stores/Champaign', 'http://www.orthodoc.aaos.org', None, 'http://www.pregocalgary.com', 'https://www.canadiantire.ca/en/store-details/on/markham-on0.store.html', 'http://www.robstreeremoval.com', 'http://www.sodexousa.com/', 'http://www.tempestlukeshospital.com/', 'https://ikonick.com', '1592232742', 'http://www.yourvacationhome.ca', None, 'http://crockerparkwinefestival.com', 'http://svpsports.ca/stores/steeles', 'http://www.calgarycoop.com/stores/brentwood', 'http://www.papaceo.ca', 'http://www.bulkbarn.ca/en-ca/index.html', 'http://www.alphagraphics.com/centers/tempe-arizona-us004/index.html', 'http://www.floppypawspetsupplies.com', 'http://www.phxsecrets.com', 'http://www.apria.com', 'http://www.onceuponachildscottsdale.com/', 'https://www.belfor.com/en/us/belfor-usa-offices/nevada/las-vegas', None, 'https://www.riteaid.com/store-details?storeNumber=11467', 'http://www.adept.fit', 'http://cibocalgary.com', None, 'http://www.thoseguysmotorcycle.com/', 'https://chadloveservices.com', 'http://phoenixtenniscenter.org', 'http://lagerheads.us', 'http://pizza22ribs.com', 'http://www.funnymagicforkids.com', 'http://barksidebistro.com', '1592232800', 'https://summithomesnv.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.frock.ca', 'http://www.myspace.com/rockyseastside', 'http://www.smgvegas.com', 'http://www.toronto.ca/parks/prd/facilities/complex/467/index.htm', 'http://www.monro.com', 'http://www.wildwingrestaurants.com/franchisees/angus-glen/', 'https://www.goodeyes.com/locations/phoenix/?utm_source=yelp&utm_medium=local&utm_campaign=clinic-phoenix&utm_content=website', 'http://www.einsteinbros.com/', 'http://www.jiffylube.com', 'https://www.moonvalleynurseries.com/locations/northwest-las-vegas?utm_campaign=yelp&utm_source=yelp-nv&utm_medium=rancho', 'https://la-canasta.com', None, 'https://locations.panerabread.com/oh/sheffield-village/5382-abbe-rd.html', '1592232836', 'http://www.frizzhairexp.com', None, None, None, 'https://vespersalon.com', 'http://www.michaelhill.ca/on/demandware.store/Sites-MichaelHillCA-Site/en_CA/Stores-Details?StoreID=148', None, 'http://rudyspub.org', 'http://www.lavishmotoring.com', 'http://www.warehousegroup.ca/el-furniture-warehouse-queen-st', 'http://www.naillacquersalon.com/', 'https://www.benjaminfranklinplumbing.com/las-vegas/', 'https://stores.pepboys.com/nv/las-vegas/506-s-decatur-blvd.html', '1592232869', 'https://www.metropolish17.com', 'http://www.streetsofnewyork.com', None, 'http://www.thepostalroute.com', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://theritzoflasvegas.com/', 'http://www.greentreehealthandmassage.com', 'http://www.dmvnv.com/', None, None, None, 'http://www.hotelarts.ca', 'http://www.runnymedewalkinmedical.com/', None, None, None, 'http://www.vitalitytap.com', 'http://www.bk.com', None, 'http://sekaisushi.net', 'https://www.frysfood.com/storeHours?store=66000623', None, 'http://ptownbar.com/', 'http://www.limozarizona.com/', 'http://www.lakeshorewellness.ca', 'http://en.hestiatea.com', 'http://www.snapfitness.com/torontohighpark', None, 'http://www.yogapulsestudio.com', 'http://www.veraiconsalonspa.com', '1592232943', None, 'http://www.jaybuickgmc.com', None, 'https://www.menzieschryslerdealer.com', 'http://www.dickblick.com/stores/pennsylvania/pittsburgh/', 'http://www.handandstone.ca/locations/aurora', None, None, 'http://BluBurger.com', 'http://www.nurturedmama.com', 'http://www.cwtow.com', None, 'http://www.fitnesscafemenu.com/', None, 'http://www.vegasmartialarts.com', None, 'http://drtangeyecare.com', 'http://b2burgersbrews.com', 'http://www.copyrite.ca', None, None, 'http://wingardsalon.com/', 'https://hilalgrill.com', None, 'http://www.extremeenergypros.com', 'http://www.ciaobabycatering.com', 'https://baycity-movers.com', None, 'https://www.bannerhealth.com/locations/peoria/banner-urgent-care', None, 'https://www.associated-dental.com/practices/associated-dental-care-chandler', 'https://www.starbucks.ca/store-locator/store/1020633/177-holland-street-east-177-holland-street-east-bradford-on-l-3-z-1-x-8-ca', None, 'http://www.spencersonline.com', None, None, 'http://w.subway.com', 'http://myomedicalmd.com', None, 'http://www.karma.salon', None, None, 'http://powerautobrokers.com/', None, None, 'https://www.ihg.com/holidayinnexpress/hotels/us/en/las-vegas/lasqr/hoteldetail', 'http://robertostacoshop.com', None, 'http://www.allbrightsmile.com', None, 'http://www.publicenemytattoo.net', 'http://www.lucysseacove.com', None, None, 'https://www.nofrills.ca/store-locator/Mario+%26+Selina%27s+nofrills+%28Royal+York+Rd.%29/1371?storeId=1371', None, 'http://beersoftheburgh.com', None, 'http://dnister.ca', 'http://www.wilsonparkerhomes.com', 'http://www.priyashairsalon.com', 'http://www.patoisjamaicanrestaurant.com', 'http://www.thebeautyspotstore.com', 'https://opdaz.com', 'http://www.bearmountainsports.com', 'http://www.foresightchiropractic.com/', 'http://www.palominolv.com/', 'http://www.ralphlauren.com', 'https://wildfire.sclv.com/Barleys?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.lawrysonline.com/lawrys-primerib/las-vegas', None, None, '1592233109', 'http://www.chnv.org/OurCare/HealthyLivingInstitute.aspx', 'http://www.dunkindonuts.com', 'http://602massage.com', 'http://www.zaffron.ca/', 'http://www.bodyvision.com/arrowhead-studio/', None, 'http://www.hhgregg.com', None, 'https://cityofspice.com', None, 'https://varmintgone.com/locations/varmint-gone-matthews-north-carolina/', 'http://eastlakecanoe.com/', 'http://www.auroraarizona.com', 'http://www.papamurphys.com/', 'http://www.marcstreeservice.com', 'http://www.majesticpalmsapartments.com/', 'http://www.peoplesjewellers.com', None, 'http://www.tempe.gov/city-hall/community-services/community-recreation-centers/diablo-stadium', 'http://stonecreekfurniture.com', 'http://www.centralavenuedental.com', 'https://www.planetfitness.com/gyms/kent-oh?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', '1592233167', 'https://www.atozlandscapinglv.com', None, 'http://www.sundevilauto.com/locations/az/chandler/chandler-22', 'http://www.wickeremporium.ca/', None, 'https://paulmitchell.edu/pittsburgh', 'https://www.chapters.indigo.ca/en-ca/home/storelocator/storeDetails/939', None, '1592233187', 'http://www.rainbowfeatherco.com', None, 'https://www.coca-colacoliseum.com', 'http://www.masterworksautomotive.com/', 'https://www.gianteagle.com/Stores/Store/?storeId=5831', 'https://www.goldfishswimschool.com/wexford/', 'http://www.elcampesinospgh.com', 'http://www.noforksgivenclt.com', None, 'http://www.gokartslasvegas.com', None, None, 'http://www.jacksonlimousines.com', 'http://www.thelashlounge.com/salons/waverly/', None, 'http://www.ACabLV.com', 'https://www.jumagallery.com', 'http://www.suncityawning.com', 'http://www.carrvalleycheese.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.ashleyfurniturehomestore.com/furniture-stores/united-states/nv/henderson/henderson?cmpid=2016YELP', None, 'http://ratewisemortgage.com', None, None, 'http://www.krispykreme.com', 'https://www.oyorooms.com/76426?utm_campaign=US_PHX004&utm_medium=listing&utm_source=yelp', 'http://www.comda.com/en_ca/', 'https://www.veinscarolina.com', 'http://wedosystems.com', 'https://stores.bestbuy.com/az/avondale/10035-w-mcdowell-rd-1109.html/?loc=ns100&ref=NS', '1592233263', 'http://safransmarket.com', 'http://www.kitchenbathroomremodeling.net', 'http://www.atengarofalodental.com', 'http://diamondnailsofmahomet.com', 'https://www.orangetheory.com/en-us/locations/arizona/surprise/16846-w-bell-rd-suite-106/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.pizzuvio.com', 'http://www.ayrsleynailbar.com', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', None, 'http://www.sushimaster.ca', 'http://silverstatelocksmith.com', 'https://svdpmadison.org/shop/verona/', None, 'http://www.totalsportsmedicine.com', 'http://www.brazascharlotte.com', 'http://www.nutrition-rush.com/stores.html', 'http://brazilianblowoutsbykatie.com', 'http://www.stammhouserestaurant.com/', 'http://www.eyetecheyes.com', None, 'http://www.girly-girlz.com', None, 'http://www.gatesmillsplace.com', 'https://www.flawlessbyhanna.com', None, 'http://www.sdpetresort.com', 'https://locations.togos.com/ll/US/AZ/Scottsdale/7369-N_-Via-Paseo-Del-Sur*-Ste_-104', 'https://www.aireserv.com/red-mountain', 'http://www.ampm.com', 'http://www.custommedicalmassage.com/pre-and-perinatal.html', 'http://www.nanakcarwash.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://masteldrycleaning.com/?utm_medium=CTA&utm_source=VYRLYelp', 'http://www.verizonwireless.com/stores/arizona/mesa/val-vista-204086/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.gotransit.com', 'http://www.geckoshadesolutions.com', 'https://www.blissnailspasc.com', 'https://locations.timhortons.com/ca/on/oakville/494-dundas-st-w.html', None, 'https://locations.massageenvy.com/pennsylvania/bethel-park/5223-library-road.html', 'http://blueiguanapools.com', 'http://www.heirloomsalon.net', None, 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'http://www.kingswayvanlines.com', 'http://www.todaysmenu.ca', 'http://lasvegasnora.com/home.html', None, 'http://www.maac.com/florida/orlando/retreat-at-lake-nona?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://www.7-eleven.com/?yelp=19726', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89015/601-in-henderson/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-601&utm_content=website-link&howheard=IYE', None, 'https://www.calgarycoop.com/stores/shawnessy/', 'http://www.lawroofing.net', 'http://www.palaisroyale.ca/', 'http://www.blackpeppergrill.com', '1592233399', 'http://www.copps.com', 'http://www.greeningenuity.ca', None, 'https://modellacewigsandhair.com', 'http://www.lebowitzdental.com/', 'http://JamesonAndStaggs.com', 'http://www.atlas-autobodyinc.com/', 'http://www.eyebrowsnatural.com/', 'https://www.miragegm.com', 'https://www.hilton.com/en/hotels/clttohx-hampton-suites-charlotte-ballantyne/?SEO_id=YELP-HP-CLTTOHX', None, 'http://www.designingwomanohio.com', None, 'http://www.southwestjudo.com', 'http://www.healingedgeaz.com', None, None, 'http://stores.fossil.com/az/scottsdale/scottsdale-fashion-square-8304', 'http://www.mrfallstree.com', None, None, None, None, None, None, 'https://www.hilton.com/en/hotels/pitsrhx-hampton-suites-pittsburgh-airport-south-settlers-ridge/?SEO_id=YELP-HP-PITSRHX', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.sweetlanasbridal.com', 'http://www.nakedpizza.biz', None, None, 'http://www.generaltsoskitchen.com/', 'http://www.alteredimage.net', 'http://www.azclar.com', None, 'http://www.songlowboutique.com', '1592233480', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.excellenceindentistry.com', 'http://www.roofitright.com', 'http://www.3riverslocksmith.com/', 'https://locations.dollartree.com/il/champaign/546/', None, None, None, 'http://www.belllexusnorthscottsdale.com', 'http://www.osbornplace.com', None, 'http://tavernamericana.com/', 'http://www.paybackpest.com', 'http://www.donscigars.com/', 'http://parkinn.com/hotel-torontoca', 'https://www.carolinaasthma.com/location/eastover/', None, 'http://www.lickshomeburgers.com/', None, 'http://www.atsol.com', None, None, None, 'http://oxurban.com', 'http://salonliftohio.com/', 'https://www.cvs.com/minuteclinic/clinic-locator/nc/belmont/2233.html', None, 'http://www.dairyqueen.com', '1592233542', 'http://www.thecarolinadental.com', 'http://www.dentistryofarizona.com', 'http://goodearthcoffeehouse.com', 'http://www.newmarketinn.com/', 'http://www.gilbertperfectskin.com', 'http://www.dermimatch.com', 'https://joeyrestaurants.com/location/joey-crowfoot', 'http://www.health.gov.on.ca/en/public/programs/telehealth/', 'http://www.proarmament.com/index.html', '1592233565', 'http://www.bodricksbbq.com', None, 'http://www.yalebrothers.com', 'http://www.supremeshutters.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://excel-pro-services.com', 'http://www.minpermanentmakeup.com', 'http://www.brainandspineaz.com', None, 'http://www.harmonyvet.com/', 'https://m.costco.com/warehouse-locations/summerlin-las-vegas-nv-685.html', None, 'http://SolisWaverly.com', None, 'http://www.ziggystavernpgh.com', 'http://mustangairmechanical.com', 'https://zestyclean.com', 'https://mvpmodernbarbers.com/bridgeland/', 'http://www.arizonahomeworks.com', None, 'http://www.perfectionistdetailing.com', None, 'http://www.mkt.com/desertrootskitchen', None, 'http://www.rotorooter.com', None, None, None, 'http://www.nafloans.com/', None, 'http://www.ztrip.com', None, 'http://www.umc.org', '1592233641', 'https://www.getgocafe.com/stores/pa/pittsburgh/baum-blvd-getgo/3064', '1592233647', None, '1592233652', 'http://onlyfaces.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=14314', 'http://www.erx247.com', 'http://www.marcos.com/', 'http://www.azextensionsbyjess.com', None, None, 'https://locations.dennys.com/OH/PARMA/247530', None, 'http://www.thaionyonge.com', None, 'http://www.pawstwoclaws.com', '1592233683', 'http://www.reliable.vegas', 'http://www.a1thevacuumshop.com/', 'http://www.zumiez.com', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.francisandsonscarwash.com', 'http://arizonapinatas.com', None, 'http://saachispa.com', None, 'http://www.papaprimoz.com', 'http://www.thevoodoolounge.ca/', None, None, 'http://www.themesshall.ca', 'http://goldstarsvc.com', None, 'http://www.neatfreakz@vpweb.com', 'http://www.cityofmadison.com/parks/find-a-park/park.cfm?id=1139', 'http://www.eggspectation.com', 'http://www.yamasushihouse.com', 'https://mylocalmcds.com/willow-pass-concord', 'http://www.tlcpedsaz.com/', None, 'http://www.lvthaimassage.com', 'http://www.frenchconnection.com', 'https://www.giascanvas.com', 'https://affordablebathroomsofaz.com', 'https://www.regmovies.com/theatres/regal-franklin-square/1845?utm_medium=social+network&utm_source=yelp+listing&utm_campaign=Regal+Franklin+Square', None, 'https://www.optumcare.com/state/specialty-care/specialist-locations/rose-parkway-oncology.html', 'http://levetto.com/', None, 'http://www.richlane.ca', 'http://www.pureandsimple.ca', 'http://www.moneyinminutesnv.com', 'http://www.MyMightyMoversAz.com', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://nathansfamous.com', 'http://www.wvimaging.com/', 'http://www.motimahal.ca', 'http://4lifenaturalfoods.com', 'http://www.uptowncarservice.com', 'http://www.classicmobileairbrushtanning.com', None, 'https://www.walmart.com/store/1464/charlotte-nc/whats-new', 'http://www.medinaoh.org/Parks/Reagan_Park', 'http://westendfood.coop/farmers-market', 'http://www.larichs-pizza.com', None, 'http://aaapws.com', 'http://www.3Brewers.ca', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', None, 'http://blushcleveland.com', None, '1592233815', None, None, None, 'http://northcarolinafertility.com', 'http://www.lasvegascustomloans.com', 'http://www.tacomamaonline.com', None, 'http://www.jacksonswash.com', 'http://perfectpastryonline.com', 'http://www.clevelandexpress.net/index.html', 'http://www.azturfmasters.com', 'http://sidewalkcafeeuclid.com', 'http://mpmvegas.com', 'http://www.streetsofnewyork.com', 'https://www.thejoint.com/arizona/surprise/surprise-48024', 'http://www.playderecord.com', 'http://www.tranquil-bodyworks.massagetherapy.com', 'http://elsombrero.ca/', None, None, 'http://vipbestlasvegaselectrician.com', None, 'http://www.creativedds.com', None, 'http://www.apexendo.com', 'http://senortequilacantinagrill.com/', 'http://abloomsalon.com', 'http://marketcitycaffelasvegas.mypub.website', 'http://www.starbucks.com', 'http://www.quiznos.com/', 'http://www.mesaaz.gov/parksrec/Parks/MountainView.aspx', 'http://www.victoriasfamily.com', None, None, 'https://northtryonlaundromat.com', None, 'http://www.theastrologystore.com/', None, 'http://www.elomsbraiding.com', 'http://www.downtowntorontochiropractor.com', None, None, 'http://www.matthewsfarmersmarket.com/Index.html', 'https://www.wyndhamhotels.com/hotel/52963?tel=18666154223&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52963', 'http://www.secondlookbooks.org', 'http://www.firstqualityroof.com/', None, 'http://www.visionquestbooks.com', None, None, None, None, 'http://www.sallybeauty.com', 'https://www.bowlero.com/location/bowlero-christown?utm_source=yelp&utm_medium=organic&utm_campaign=webclicks', 'http://www.babysbadassburgers.com', 'http://www.phoenixelectricianstoday.com', 'http://www.saulterstreetbrewery.com', None, None, None, 'http://www.labcorp.com', 'https://www.giovannisonline.com', None, None, 'http://www.marbleslab.ca', 'https://www.lush.ca/on/demandware.store/Sites-LushCA-Site/en_CA/Stores-Info?StoreID=129', 'http://www.eatatovereasy.com', 'http://chirotrendz.com/', 'http://www.greenvalleyranchresort.com/dining/china-spice.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://www.directv.com', 'https://www.gymboreeclasses.com/en/locations/NC/Charlotte/?utm_campaign=Charlotte&utm_medium=website&utm_source=Yelp&utm_content=yelp+page', 'http://brosandbettys.com/', None, 'http://www.lemonfreshcleaning.com', 'http://jetaction.com/', None, None, 'http://www.getitfixed.me', 'http://primozzpizzeria.com', 'http://www.cascadenc.com', 'https://www.fantasticsams.com/about/regions/phoenix-az/fantastic-sams-mesa-az-4', 'http://www.isaute.ca', 'http://www.piesquared.com', 'http://www.thewhitenailsspa.com', 'http://www.bestmassagecenter.com', None, '1592234018', None, 'https://www.brakemasters.com/new-site/stores/1901-n-scottsdale-rd-scottsdale-az/', 'http://happyjoes.com', None, 'http://www.ArrowheadScoopers.com', 'http://www.neatmethod.com', None, 'https://www.usps.com/', 'http://hancockfabrics.com/', 'http://www.albertayogacollege.com', 'http://www.sterling2cleaners.com', 'http://www.royalyorkmeatmarket.com', None, None, None, 'http://signaturenorthwest.com/agent/andrew-lunsford/', 'http://www.madcatpets.com', 'http://www.chasin-tails.ca', 'http://www.roxannesdriedflowers.com', 'https://www.amco-autoglass.com', None, 'http://handymanserviceslasvegasnevada.com', 'http://www.arrowheadnails.com', 'http://www.mcsorleysrestolounge.com/', 'http://www.cityofhenderson.com/parks/parks/sonata_park.php', 'http://www.heartlandpethospital.com', 'http://www.cruisin7th.com', '1592234082', 'http://www.abbotthearingaidcenter.com', 'http://www.townetavernrestaurants.com', None, 'http://www.subvetclinic.net', 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', 'http://wendyandersonlaw.com', 'http://www.globalpetfoods.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://nailskathylinhdesign.com/', None, 'http://www.meadowslv.com', 'http://www.ccrfurniturecleaning.com', None, 'https://www.bostonmarket.com/location/university-heights-oh-483/', 'http://www.senecacollege.ca', 'https://www.walmart.ca/en/stores-near-me/markville-mall-supercentre-3053', '1592234121', 'https://www.munichmotorco.com/service-your-car', None, 'http://www.guardianautoglassphoenix.com', 'http://www.thetacobros.com', 'https://www.popeyes.com', 'http://www.impactps.com/', 'http://www.phoenixmentalist.com', 'https://cagiautoglass.com', None, None, 'https://periodontics-dentalimplants.com', 'http://www.nowandthenretro.com', None, None, None, '1592234160', 'http://www.lesportsac.com', 'https://lvsmilecenter.com', None, 'http://www.mysewingroom.ca', 'https://ad.doubleclick.net/ddm/clk/458685338;262921492;w', None, None, 'http://www.shawnessysouthvet.com/', 'http://www.jandbappliance.com', None, None, 'http://www.piscessushi.com', None, 'http://www.techniquerestaurant.com/', 'http://www.scubamagic.biz', None, None, None, None, None, 'https://www.blackmetalfirearms.com', 'https://www.walgreens.com/locator/walgreens-4965+w+bell+rd-glendale-az-85308/id=3049', 'https://locator.chase.com/nv/henderson/637-n-stephanie-st?jp_cmp=rb/LocalListings/aff/branch/na', 'http://www.laalv.com', None, 'http://www.lasvegasswimacademy.com', 'https://www.target.com/sl/waterfront/1253', 'https://www.cloudcarpetonelasvegas.com/about-us/las-vegas', 'http://phoenixcenterforthearts.org/', 'http://www.phoenixknifehouse.com', None, 'http://www.calranch.com', 'http://honeydont.work', None, 'http://www.orobrewing.com', None, 'http://seefuhair.com', 'https://www.petsmart.com/stores/us/az/scottsdale-store0130.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.youngshawaiianstylekitchen.com', 'http://www.brickyardgrounds.ca/', 'http://www.donrandleslawoffice.com/', 'http://johnnyrockets.com', 'https://www.planchatoronto.ca', 'http://www.pawgo.co', None, 'https://www.hertz.com/rentacar/location/unitedstates/northcarolina/charlotte/CLTT12', 'http://www.mtnparkchiropractic.com/', 'http://aquatechplumbingaz.com', None, None, 'http://www.larrygjones.com', 'http://www.myguardianselfstorage.com', '1592234284', None, 'http://www.orleanscasino.com/entertain/bowling?disid=wmm_yelp_to_corporate_bowlingcenter_organiclink', 'http://www.starbucks.com/store/15154/', 'http://www.thefrenchbee.com', None, 'https://finercustomjewelry.com', 'http://colournailssalon.com/', 'http://yummymarket.com/', None, None, 'http://robertostacoshop.com', None, 'http://azpermanentcosmetics.com/', 'http://www.midas.com/toronto/store.aspx?shopnum=9259&dmanum=857001', 'http://www.ocotilloeyecare.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.amerisent.com', 'http://backwards.club', 'https://antikka.ca', 'http://www.turnstyleconsign.com/ahwatukee-phoenix.php', None, None, 'http://www.pjsdayspa.com', '1592234337', 'http://thebrowbomb.com', 'https://www.dunnedwards.com/directory/store-list/summerlin-078', 'http://drscottminer.com', 'https://www.peoriaford.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', None, 'https://www.crashofrhinospainting.com', 'https://www.kohls.com/stores/nv/lasvegas-668.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.thebakeryphx.com/', 'https://www.usps.com/', None, None, 'http://www.burlingtoncoatfactory.com', 'http://dalesbodyworksllc.com/', 'http://www.petworksco.com', None, 'http://yumyumsofcle.com', 'http://www.thedogdasher.com', 'http://www.hartshornstudios.com/', 'http://europeanpaverssouthwest.com/', None, 'http://www.maleesonmain.com/', 'http://www.equityresidential.com/', 'https://www.publicstorage.com/arizona/self-storage-gilbert-az/85233-self-storage/850?pid=wyelp&CID=1245&CHID=AFL', 'https://www.flavorsoflouisianacajun.com', 'http://www.madbatterbakers.com/', None, 'http://tempefarmersmarket.com/', 'http://www.8bitentertainment.com', 'http://machinodonuts.com', '1592234413', None, 'http://www.westmarine.com', 'https://www.nasm.org', None, None, None, 'https://locations.jackinthebox.com/us/nc/charlotte/7725-pineville-matthews-rd', None, 'https://locations.dennys.com/PA/NEW-KENSINGTON/249565', None, None, None, 'https://www.serviceheroesaz.com', 'http://zabblv.com', None, '1592234448', 'http://www.quiznos.com/', 'http://www.workhorsemarine.com', None, 'http://www.readytorolltire.com', 'https://www.beefjerkyoutlet.com', '1592234462', None, None, '1592234467', None, 'http://christophersfinedrycleaning.ca', None, None, 'http://www.donatos.com', 'http://menu6.org', 'https://www.charlottemeetings.com/charlotte-convention-center', 'http://www.lubestop.com/Locations/Wadsworth', 'http://www.papamurphys.com/', 'http://www.adidas.ca', 'http://www.arizonastoragecenters.com', 'http://www.excelauto.biz', 'http://www.edingersurgicaloptions.com', 'https://www.949atthedistrict.com', None, 'http://www.lvtattoo.com', 'http://www.hobbylobby.com/StoreLocator/Search?Latitude=43.1438143&Longitude=-89.35114950000002&SearchOn=+53704&SearchDistance=25', 'http://www.tacorrito.ca/', 'http://www.pintville.com', '1592234514', None, 'http://www.functionalaestheticdentistry.com', 'http://www.bsjplumbing.com', 'http://www.unitedmessengers.com', 'http://www.kelilabee.com', 'http://www.lovecloudvegas.com', 'http://www.vaco.com', 'http://www.aptfind.com', 'http://www.piesunlimited.com', None, 'http://www.vitalant.org', 'https://locations.usbank.com/index/arizona/tempe/east-elliot-rd-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', '1592234547', None, 'https://agents.allstate.com/the-bell-agency-henderson-nv.html', None, 'http://www.heroburgers.com', 'http://www.kidsrkidslasvegas.com', 'http://artistichair.studio', None, None, 'https://www.qualtire.com', None, None, '1592234573', 'http://www.animalsupplyhouserockhill.com/', 'http://www.muchoburrito.com', None, 'http://suitenectarsalons.com', 'https://www.filibertos.com', None, 'http://www.truwest.org', 'https://www.dipnailboutiqueocotillo.com', 'http://www.paradisedentalgroup.com', None, 'http://dreamydrawdental.com', 'http://www.mcmurray-manalodental.com', 'http://www.thompsonsautorepair.com', '1592234607', None, None, 'http://www.homewatchcaregivers.com/phoenix', 'http://www.tailgateclothing.com', 'http://www.buybuybaby.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.uwhealth.org/locations/detail.jsp?locationid=124', None, 'https://davidguyettphotography.smugmug.com', 'http://www.midge.ca', None, 'http://mkhouseconsulting.com', 'http://www.torontosculpturegarden.com', None, 'http://EventRents.net', None, 'http://usedcars12.com', 'http://www.azartrestoration.com', 'https://www.circlek.com/store-locator/us/peoria/9476-w-olive-ave/2706468', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://waxingaroundtown.com', 'http://www.xtremehomeservicesaz.com', 'http://lasmejorespupusas.weebly.com/', 'http://www.redendos.com', 'http://www.CrowdControlLV.com', 'https://www.kaltire.com/en/stores/locations/?source=yelp_track&StoreID=867', 'http://rickjohn.com', 'http://www.pplusic.com', 'http://www.jabzboxing.com/glendale', None, 'http://www.madisonautodetail.com', 'http://www.fourquet-fourchette.com', 'http://www.dazzlingrestaurant.com', 'http://montereyvetclinic.ca', 'http://www.mrbsfamoussale.com', 'http://www.carserviceoflasvegas.com', 'http://www.suntreehealingarts.com', None, 'http://cirqnightclub.com', 'http://www.ampm.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.zabas.com/', 'http://summerwindsnursery.com/az/home?utm_source=yelp&utm_medium=text_link&utm_campaign=website', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYCWA', 'http://www.santanfamilymedicine.com', 'https://www.circlek.com/store-locator/us/gilbert/888-s-higley-rd/2703373', None, 'https://canyoncreekrestaurant.ca/square-one/', 'http://www.artificialgrassinstallers.com', 'http://www.customcyclerescue.com', 'https://www.usstoragecenters.com/self-storage/nevada/las-vegas/us-storage-centers/135', 'http://www.inland-group.com/location/phoenix/', 'http://www.phxprep.com', 'http://www.DLCphoenix.com', 'http://www.babiesrus.com', 'http://www.ashtangayogalv.com', 'http://upmccancercenters.com/network/upmchillman.cfm', 'https://www.swimco.com/?utm_campaign=Market&utm_source=YELP', None, 'http://www.the-alley.ca', 'https://stores.advanceautoparts.com/pa/murrysville/4010-william-penn-highway?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'http://www.donalfonsotoronto.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', 'https://bikeworldlv.com', None, 'http://www.mathnasium.com/litchfieldpark', 'https://haircutmenbocaparklasvegasnv.com', 'http://www.partycity.com', 'http://www.donuttyme.com', '1592234787', '1592234790', None, None, None, '1592234800', None, 'http://www.scottsdaleaz.gov/parks/mountain-view-park', None, 'https://www.gowireless.com/stores/az/fountain-hills/fountain-hills-fountain-hills-plaza/', 'https://www.acesdental.com/areas-served/dentist-north-las-vegas-nv/?utm_campaign=yelpnlasvegas&utm_medium=organic&utm_source=local', 'http://www.rivierahotel.com/las-vegas-entertainment/crazy-girls.aspx', None, 'https://clevelandhomes.com', 'http://www.clipncleandoggrooming.org', None, 'http://www.centurylink.com', 'https://www.chick-fil-a.com/indianland', 'http://www.stabley.net', 'http://www.ledolci.com/', 'http://www.kemptonpt.com', 'http://www.bk.com', 'http://www.schneidersdairy.com', 'http://desertfoothillsgardens.com', 'https://aguayobunch.wixsite.com/a1electric', 'https://www.gianteagle.com/Stores/Store/?storeId=3043', 'http://www.chandlerpediatricdental.com', 'http://www.thereikigal.com', None, 'http://www.slepakinsurance.com?cmpid=RZYD_BLM_0006', None, None, 'http://www.colussy.com/?cs:pro=repman&cs:e=yelp', '1592234868', None, 'http://www.racersedgeperformance.com', 'http://www.waterconnectionaz.com', 'http://www.goindustrial.ca', 'http://www.bohofurnituregallery.com', 'http://www.dannysfamily.com/', 'http://www.sammysstudentexchange.com', 'http://bellagio.com/nightlife-diversions/hyde.aspx', 'http://thetoytank.com', 'http://www.ironcityarmory.com', 'http://journeyyogawellness.com', 'http://www.gatewayanimalohio.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=jan16', 'http://www.rocksolidconcrete.com', None, 'http://www.ajimenu.ca', 'http://www.precisionpoolcarellc.com', None, '1592234912', 'https://app.visionnv.com', 'https://www.calgarycoop.com/stores/shawnessy/', None, 'http://www.amcofh.org', 'http://www.ironoakshoa.com', 'http://www.z-teca.com', 'http://kendalaquatics.com', 'http://www.hunnerspizza.com', 'http://www.thedistrictatgvr.com/', 'http://www.apple.com/retail/glendale/', 'http://www.marquisdiagnosticimaging.com', None, 'http://www.thewirelesscenter.com', 'https://www.extraspace.com/storage/facilities/us/arizona/phoenix/501788/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'https://athleta.gap.com/stores/wi/madison/athleta-7080.html', 'https://www.usps.com/', 'http://katespade.com', None, 'http://www.nicksauto-tech.aaro.ca', 'http://www.hairmonet.com', 'http://www.greenwaymoving.ca', None, None, 'http://www.winemandental.com', None, 'http://www.mfmassage.com', None, 'https://bluepearlvet.com/hospital/avondale-az/', 'http://www.antheaspa.com', 'http://www.thechateauaz.com', 'http://www.coronetpizzeria.com', None, 'http://www.sabuddyrestaurant.com', 'http://www.avis.com', 'http://www.hendrickmotorsports.com', None, 'http://www.legacyjewelersofpittsburgh.com', 'https://www.dabuzispizza.com', None, 'http://alysskincare.com', 'http://vikingfishandchips.net', 'https://www.applebees.com/en/restaurants-champaign-il/2121-n-prospect-93091', 'http://www.thenailtique.com', 'http://www.shopsys.ca/', '1592235014', None, 'https://www.extraholidays.com/las-vegas-nevada/worldmark-las-vegas-boulevard?lid=yelp&utm_medium=referral-affiliate&utm_source=yelp.com&Identifier=yelp&hotelid=77219&utm_campaign=localpage-worldmarklasvegasboulevard&promoid=localpage-worldmarklasvegasboulevard', None, None, 'http://www.stmlv.org', None, 'http://www.purrclothing.ca', 'http://call4handymanlv.com', 'http://305kustomzdesigns.com', None, 'http://www.theoriginaldicksspeedotach.com/', 'http://visitpittsburgh.com', 'https://batlgrounds.com/axe-throwing-yorkdale', 'http://www.MyLasVegasChiropractor.com', 'http://www.planetdailies.com', 'http://liquorlibrarylv.com', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, None, 'http://tofoodfest.com/', 'http://snipssalonclt.com/', '1592235069', 'https://www.spencersonline.com/store/Ross-Park/32682040.uts', 'http://thestudioacademyofbeauty.com/locations/chandler-campus/', 'http://www.tinytots.ca', 'http://www.supremecollisioncentre.ca', 'http://www.mobilia.ca', 'http://www.westernexterminator.com/chandler', 'http://www.yorkmillspa.com', 'http://www.stampsautoqueencreekqueencreekaz.com', None, 'http://www.vitalitymwi.com', None, 'https://www.thespacelv.com', 'http://www.rosenbaumrealtygroup.com/', 'http://coolpools.services', None, 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', 'http://longwongsofnorthphx.com', 'http://www.winedowntc.com', 'http://www.mitcoremodeling.com', None, 'http://www.holdenarb.org/home/', 'http://kmedspa.com', None, 'http://www.quickerrooterplumbing.ca/', 'http://pittsburghrugcompany.com', 'https://suncountry.com', 'http://www.fujimiddleton.com', 'http://www.thebeergrowler.net/charlotte', None, 'https://sandpipers.ca', None, 'http://www.starbucks.com', 'http://www.picazzos.com', 'https://www.easystreetgalleria.com', None, None, '1592235155', 'http://www.goodlifefitness.com/locations/ontario/mississauga/toronto-pearson?source=yelp-websitelink-torontopearson-gl', 'https://www.pteglv.com/locations/pts-gold/st-rose-eastern/', None, 'http://www.weldrickanimalhospital.com/', 'https://www.cosmopolitanlasvegas.com/resort/spa?utm_source=yelp&utm_medium=organic&utm_campaign=yelp', 'http://www.idotactical.com', 'http://www.progressive.com/locations/mayfield-heights-campus3/', 'https://elitevacuumsplus.com', 'http://www.vha.ca/', 'http://mwtravelvegas.com/', 'http://www.ezpawn.com/locations/nv/las-vegas/6032-cheyenne', 'https://www.eggsupgrill.com/locations/charlotte-nc/', 'http://www.popculture-comics.com', None, 'http://www.wimpysdiner.ca', 'http://www.shearshades.com', 'https://www.oroexpresschandler.com', 'http://www.forcesofnature.ca/', 'http://tommysdrivein.com', 'http://www.saintbernadette.com', None, 'http://steelcreationsfab.com', 'http://www.VerveStudio.net', 'http://www.performancemufflershop.com', 'https://po-ke.ca', None, 'https://www.usps.com/', 'http://grandcourtmesa.com', 'https://azfoodcrafters.com', 'http://www.ohiocitycycles.org', 'https://www.biltmorecounseling.com', 'http://www.budgetmoversaz.com', 'http://bluewavepoolsaz.com', 'https://www.haggar.com/stores/las-vegas-nv/?utm_campaign=websitelink&utm_medium=Referral&utm_source=yelp', None, 'https://www.coultertempe.com', 'http://www.swimzone.info', 'http://walmart-eyedocs-lasvegas.com/Eastern-Russell', None, 'https://www.calgarypromovers.com', 'http://www.gyrotime.com', 'http://www.dinaalonzibridal.com', 'http://www.massagelifewellness.com', None, 'http://www.shellhospitality.com', None, 'http://www.lakemetroparks.com/parks-trails/farmpark', 'http://www.charlottegeneraldentist.com/', 'http://tomyumthaiaz.com', '1592235283', '1592235286', 'https://www.30sixnoda.com', None, 'https://www.laserquest.com/on-whitby', 'https://yogatothepeople.com/tempe-az/', None, 'http://hairbygk.com/', 'http://www.crookedcue.ca/', None, 'http://www.mission4th.ca', 'http://www.lisatrainer.net', 'http://www.greektownrestaurant.com', None, 'http://arizonasports.com', 'http://www.itmp.com', None, 'http://www.scarboroughnissan.com', None, 'https://www.walgreens.com/locator/walgreens-401+w+main+st-waunakee-wi-53597/id=5669', 'http://www.andrewsco.com', 'https://www.sheridancollege.ca', 'http://www.bloombrossupply.com/', 'https://www.esso.ca/en/gas-station/richmondhill-on-beavercreekesso-200302403', 'http://www.eatatgreenmix.com', 'http://www.seasonsboulangerie.com', 'https://tanwithkare.com', '1592235353', None, 'http://bestnails.webs.com/', 'http://Brilliantnailsspa.com', None, 'http://www.lowes.com', '1592235367', 'http://www.lasvegascaralarms.com', 'http://scottsdaleshadowsarizona.com', None, 'http://www.jjcontractingnv.com', 'http://www.dualimagesmiles.com', 'https://agents.allstate.com/j-c-alvarado-charlotte-nc1.html', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://mnandipies.com', None, 'http://www.ArnoldSmithLaw.com', 'http://www.customcobblersofmadison.com', 'https://stores.anntaylor.com/nv/las-vegas/1013-south-rampart-blvd.html', 'http://www.suncityupholstery.com/', None, 'http://www.dropshade.com', 'http://popeyes.com/', 'http://gyro-spot.com', 'http://www.aircanada.ca', 'http://3apools.com/', 'http://www.associatedasset.com', '1592235418', 'http://www.terribleherbst.com/', 'http://www.studio262salon.com', None, 'https://ad.doubleclick.net/ddm/clk/434191077;236814625;i', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'http://www.mattressfirm.com', 'http://www.monkeypantsbar.com/', '1592235439', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://westvalleydentalaz.com/', 'https://abraauto.com/abra-auto-location/1051-charlotte-nc-albemarle-road', 'http://www.mrantenna.com', 'http://www.hlwp.co', None, 'http://www.sweetlifebakeshoppe.com/', 'http://www.brueggers.com/', 'http://footsolutions.com/strongsville', None, 'https://locations.dennys.ca/AB/CALGARY/248179', 'http://www.whitneymurphyfuneralhome.com', 'https://essencevegas.com/locations/henderson/', None, 'https://accutempaz.com', 'http://www.chefbar.ca/', 'http://www.mtlebanon.org/index.asp?NID=25', 'http://www.faregamecc.com', 'https://www.umcsn.com/Quick-Cares/Quick-Care-Locations-Summerlin.aspx?intMenuID=112&intPageID=197', 'http://www.safeworksillinois.com', 'https://www.hendrickhonda.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', 'http://www.access1alarm.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://vegasnailspa.com', 'http://sarahdiaz.com', None, 'http://www.theblackbullpub.ca', 'http://www.upmc.com/locations/community/family-health-centers/Pages/lawrenceville.aspx', None, None, None, None, None, None, None, 'http://www.vaughnautoglass.com', 'http://www.experttechauto.com', 'http://am.pizza', 'http://dennys.com', 'https://www.smithsfoodanddrug.com/storeHours?store=70600371', 'http://www.themodclub.com/', None, 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', 'http://www.valleyfirefightermovers.com', 'https://www.choicehotels.com/north-carolina/charlotte/sleep-inn-hotels/nc005/rates', None, None, 'http://vcacanada.com/royaloak', 'http://dbarkadz-filipino-cuisine.business.site', 'https://www.chilis.com', None, 'http://nakedbeautybar.com/', None, 'http://www.lakesidespa.ca', 'http://www.trinitybellwoodsflea.com/', 'http://www.drcarolford.com', '1592235578', 'http://sinfulskinaz.com/', 'http://www.elxrjuicelab.com', 'http://www.laide.ca', 'https://totalstoragesolutions.com/storage-locations/arizona/peoria-storage-units/7590-w-olive-ave', None, None, 'http://www.ebgames.com', 'http://www.atomicdogboutique.com', None, 'http://www.championappliancecare.com', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://www.grafgrowers.com', 'http://www.gameworks.com/', 'http://www.septic-systems-calgary.ca/', 'https://www.americanfreight.com/?utm_medium=Yelp_Website_URL&utm_source=Yelp', None, 'http://hmartca.com/hmart/?page_id=17', 'https://ad.doubleclick.net/ddm/clk/458685305;262921492;q', 'http://pastorcameron.com', 'http://www.leslievilledentistry.com/', None, 'http://joemommasgrille.com', 'https://www.cathaybank.com', 'https://www.verizon.com/home/storelocator/pa/pittsburgh-15206/', 'http://www.cartridgeworld.com', 'http://www.electricdaves.com', 'http://www.mistwoodapartments.com', 'http://daveandbusters.com/?utm_source=DB&utm_medium=Text&utm_term=DB&utm_content=WebsiteLink&utm_campaign=DB_DB_Text_WebsiteLink', 'http://www.callitspring.com', 'http://wreckamendedautobody.com', 'http://www.thisiscleveland.com', 'http://www.chevron.com', 'http://www.marijuanadeliveryline.com/shop', 'https://www.plantarestaurants.com/location/planta-queen/', 'http://www.kumon.com/SUMMERLIN', None, 'https://locations.wendys.com/canada/qc/lasalle/8699-boul.-newman', 'http://www.kabobnmore.com', None, None, 'http://www.kfc.ca/store/sunrige-mall-101-2525-36-st-ne-calgary', 'http://perpetualfitnesscle.com', 'http://sfdslv.org', None, 'https://whitesintegrityauto.com', None, 'http://www.tommiegirlphotography.com', 'https://www.sprint.com/storefronts/bd/sprint-phoenix-az-85022-fcs-1908/?ecid=SEO:Yelp:2020_Q1:LastMile', 'https://www.ceenta.com/locations/huntersville', 'http://flyingmonkeytruck.com', None, 'http://www.allysphotography.ca', 'http://www.nailganic.me', '1592235698', 'https://bailbondsusa.com', None, 'https://fullypromoted.com/locations/scottsdale-az/', '1592235706', None, 'https://www.localfirstaz.com/events/fall-festival', None, None, None, 'https://sugardaddyscafe.com', 'http://vitalitymds.com', 'http://www.muramoto.biz', 'http://www.7forallmankind.com', 'http://www.susysoup.com', 'http://treesbymikearizona.com', '1592235733', 'http://www.pilatesofcharlottenc.com', '1592235738', 'https://www.hungryhowies.com/store/hungry-howies-00876?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=00876&utm_content=Website_Link', 'http://www.riddleroom.ca/', 'http://www.torchedgoodness.com/', 'http://www.precisionspecialtypharmacy.com', 'https://www.lhmnissanmesa.com', 'http://bit.ly/RealtyExecsTempeTower', 'https://smallpeoplepreschool.com', None, 'https://www.bobevans.com/our-restaurants/locations/571?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.wetzels.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=8658&localechange=1', None, 'http://Bundtsnbutter.com', 'http://www.arizonajewelryappraisals.com', 'http://paisleymonkey.com', 'https://www.euroworks.ca', 'http://www.maltzmuseum.org/', 'http://www.akronphotographer.com', None, 'http://www.firegrill.com/', 'http://www.hairoriginalsalonstudio.com', 'http://azdollhouse.com/', 'http://www.gracelutheranhenderson.org', 'http://thepopplacestore.com', None, 'http://www.widmaierchiropractic.com', None, None, 'http://www.toronto.ca/parks/prd/facilities/complex/287/index.htm', 'http://tokyodiscountvegas.com', None, None, None, None, 'https://www.redrobin.com', 'http://www.pranjbeauty.com', None, None, 'https://www.enterprise.com/en/car-rental/locations/us/nc/charlotte-south-woodlawn-53c4.html?mcid=iyp:8368400', 'http://www.structube.com', 'http://popeyes.com', 'http://in.puma.com', 'https://www.discounttire.com/store/az/gilbert/s/1418?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', '1592235832', 'http://totum.ca', None, None, 'http://www.bestofthebestdjs.com', None, 'http://www.gungarage.com', 'http://www.sunsetvillaaz.com', 'http://cornerstonegiftsandpo.com', None, None, None, 'http://calibercollision.com/locate-a-caliber-collision-center/surprise', 'http://www.desertstorage.com/scottsdale-az-storage-units', 'http://www.scottsdalecc.edu', 'http://www.cupkateslv.com', 'http://www.caesarspalace.com/restaurants/empress-court.html#.Ugfc5su9KSM', 'http://www.brossardbagel.com', 'http://www.mrchickencle.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://locations.avis.com/us/az/chandler/ca3.html', 'http://www.modoyogalasvegas.com', '1592235882', '1592235884', 'https://www.valleyhealthphysicians.com/find-a-doctor/hazem-afifi', 'http://Surprisecomics.com', 'http://www.smileycrane.com', None, 'http://allstateappliancelv.com/', 'http://johnnyrockets.com/', None, 'https://www.mesaparks.com/parks-facilities/parks/countryside-park', 'http://www.monarkhome.com/showrooms/las-vegas', 'http://www.steinmart.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/109+Dabney+Drive-Henderson-NC-27536/storeid=7055?cid=ps_ylp_2020', 'http://www.dillonazrealty.com', 'http://www.arizonaplumbingservices.com', '1592235919', 'http://www.lasvegaspilatesyoga.com', 'http://www.deeppilates.com', 'http://www.irepairicustomize.com', None, 'https://www.hilton.com/en/hotels/clttght-home2-suites-charlotte-i-77-south-nc/?SEO_id=YELP-HT-CLTTGHT', 'http://www.capitalacctpc.com/', None, 'http://www.jamminjumperoos.com/', '1592235936', 'http://www.frozen75.com', 'http://Tamerabbit.com', 'https://bounceu.com/gilbert-az', 'http://www.bullseyesvc.com', None, 'http://popeyes.com', 'http://www.fastfreshfoods.ca/', None, 'http://www.palmsouthwest.com', 'http://www.jeguitars.com', 'http://www.carrefourrichelieu.com/fr', None, 'http://Www.2womenandasolution.com', 'https://www.simonmed.com/locations/all/simonmed_imaging___maryland', 'http://www.RobinsonAutoDetails.com', 'https://stores.fossil.com/az/gilbert/santan-village-8252/', None, '1592235970', 'http://www.cleveland.hfitcenter.com/', 'http://www.bk.com', 'http://phoenixcoffee.com', None, None, None, 'https://www.lovencarevet.com', 'http://www.saintjohncathedral.com', 'https://www.ryokujudo.com', None, 'http://www.tacotimecanada.com/locations/ontario/ajax/ajax', '1592235997', None, 'http://qdoba.com', None, 'http://roomsonline.ca', 'http://www.prada.com/', 'http://www.cigna.com/cmgaz/locations/san-tan-medical-office', 'http://www.innergroovebrewing.com', 'http://www.drtorgerson.com', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://freshandclear.com', 'http://www.starbucks.com/store/9298/', None, 'http://www.kingbalancing.com', None, None, 'http://www.atlwings.com', 'http://www.cuttyfresh.com', 'https://cleancutlandscaping.us', 'http://www.luxuryhomesofhenderson.com', None, 'http://www.sagebrushranch.com/', 'http://snapfitness.com/waxhawnc', 'http://www.bergo.ca', 'http://www.smilesavvydentistry.com/', 'https://www.purebarre.com/location/dana-park-az', 'https://www.hooters.to/toronto-airport', 'http://www.zenq.ca', 'http://www.drshaunachristensen.com', None, None, 'http://www.kingflyspirits.com', 'http://www.theradiodoctor.com', 'https://www.tirediscountdepotinc.com/cleveland', 'https://www.bobevans.com/our-restaurants/locations/357?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.strattonrestoration.com', 'http://www.Liorthebaker.com', 'http://rmtmassagetherapytoronto.com', None, 'http://fiscalfitnessphx.com/', 'http://banderarestaurants.com/locations/scottsdale/', 'https://www.state48brewery.com', None, '1592236095', 'http://www.quiktrip.com', 'http://www.almikestavern.com', 'http://www.earnhardttoyota.com', 'http://www.clairhursteyecare.com', 'http://mybodyworkz.com', 'http://www.envoquemd.com/', 'http://rvcomplete.com', None, 'https://www.steinmart.com/?utm_source=yelp.com&utm_medium=social&utm_campaign=yelp_site_store48', 'http://www.familyfocusedeyecare.com', 'https://www.obonsushi.com/scottsdale-quarter', None, None, 'https://www.choicehotels.com/arizona/youngtown/quality-inn-hotels/az453/rates', 'http://pizzeriaenzo.com', 'http://www.fitzgeraldsbnb.com', 'http://www.steeltownpaintball.com', '1592236142', 'http://www.timelesstattoolv.com', 'https://www.starbucks.ca', None, 'http://www.landmarkworldwide.com/customerservice?utm_source=yelp_sea&utm_campaign=support&utm_medium=website&utm_content=https', 'http://www.southtowndentalcare.com/', 'https://shinynailsonhappyvalleyglendale.com', 'http://www.foothillsrehab.com/', 'http://www.kay.com', None, 'http://www.azcustomhats.com', None, 'http://www.gokartsatpolsonpier.com', 'https://www.jamesacarytreeservice.com', 'https://artscommons.ca/rentals/venues/max-bell-theatre/', 'http://www.medoneaz.com/', 'http://www.destinocalgary.com/', None, 'http://www.futureshop.ca/', 'http://www.thingsremembered.com', 'http://www.restoamir.com/', 'http://www.kay.com', 'http://www.sunguntanning.com', None, 'https://www.scottsdalemusicaltheater.com', 'http://www.terrysauto.extremewrench.com', None, 'http://www.tupelohoneyteas.com', 'http://www.cliftonmartini.com/', 'http://www.oddballpets.com', None, 'http://www.lcbo.com', 'http://www.azrollerderby.com', 'https://www.chilis.com', 'http://www.brooksbrothers.com/?CMP=SMC_Yelp_6038', None, 'http://thecornerspot.ca', 'https://www.wyndhamhotels.com/hotel/03733?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:03733', 'https://stores.bareminerals.com/az/chandler/us-3/', 'http://deluxenailsbar.com', 'http://www.watsonsglen.com', None, 'http://www.2andhighbar.com', 'http://www.alpineroofing.ca', 'http://www.klingshirnwine.com', 'https://www.theranchestates.com', None, None, None, 'http://www.crowntownathletics.com', None, None, None, 'https://www.westgateresorts.com/hotels/nevada/las-vegas/westgate-las-vegas-resort-casino/', 'http://www.marshallsonline.com/store-finder/204', 'https://www.bellamirageapts.com', 'http://www.dvpeds.com', None, 'http://sololiquor.com', None, 'http://www.shampoochgrooming.ca', 'https://keepitcut.com', None, 'http://www.benjerry.com/', None, 'http://www.weissmeats.com', 'http://www.thegreenhouseflorist.com', None, 'http://www.halecoffee.com', 'http://www.mycompletepoolcare.net', 'http://www.ranchodetiarosa.com/rancho-chandler.asp?expanddiv=all', 'http://www.mokshayogaaurora.com', 'http://mrlube.com', None, 'http://www.textbookpainting.com', 'http://www.hansenortho.com', 'https://www.highlandparknorthlake.com', 'https://www.ulta.com/stores/madison-ms-477', 'http://www.saltrivereyecare.com', None, 'http://www.knuckledownsaloon.com/', None, 'http://www.westsidemarket.org/vendor.aspx?id=69', None, None, 'http://deseretindustries.org', 'http://www.marcs.com/stores/marcs-north-ridgeville-grocery-store-44039.aspx', None, None, 'http://www.phoenix.edu', None, None, 'https://www.marcos.com/locations/nc/gastonia/w-garrison-blvd', 'http://www.steelcityflooring.com/', 'https://vrjunkies.com/vr-junkies-az/', 'http://sears.com/stores/massachusetts/cambridge/0001343.html', 'http://www.legionbrewing.com', None, None, 'http://www.dukesbread.com', None, 'http://www.jellybeenz.com', 'http://www.urbanbean.ca', None, 'http://www.labellavitacleveland.com', 'http://www.sash.ca', 'https://www.choicehotels.com/CN659?mc=blypypaf', 'http://www.aceparking.com/downtown-phoenix-parking.com', 'http://www.careytechsolutions.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, 'http://armorpestdefense.com', 'http://www.heidisbrooklyndeli.com', 'http://ilanmassagestudio.com', 'http://bdcustomtattoos.com', 'http://www.ntb.com/store/6234/mayfield-heights-oh-44124-2905?utm_source=yelp&utm_medium=maps&utm_campaign=yelp-maps', None, None, '1592236410', None, '1592236415', None, 'http://www.hightails.ca', 'https://www.agelesskarate.com', 'http://www.mpautobodyrepair.ca', 'http://www.carlstreeservice.net', 'http://www.tailoredeventslv.com', None, '1592236432', 'http://www.solairespa.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.exosathome.com', 'http://dentalbobw.wix.com/denturerepairs', 'https://www.sleepez.com/?utm_campaign=Yelp&utm_source=yelp&utm_medium=referral', 'http://hamiltonortho.com', 'http://www.rbinc-sports.com', 'https://www.pikebeverage.com', 'http://ccandcompanysalonspa.com/', None, 'https://www.cleanjuice.com/locations/uptown/', 'http://srwc.unlv.edu/', 'http://www.theronceyhotel.com/home.html', None, None, 'http://www.702-pools.com', None, 'http://www.belmontapts.com', 'https://www.thehealingquest.com', 'http://robertostacoshop.com', 'http://www.charlottegolf.com', 'https://www.haircutmencorneliusnc.com', None, 'http://www.tymaz.com/', None, None, None, 'http://www.coquettepatisserie.com', None, None, 'http://www.alisfashion.com', None, 'http://www.yogurtys.com', 'https://torontoanimaleyeclinic.com', None, 'http://www.teamrossman.com', 'http://www.firehousemovingco.com', None, 'http://constanthomecomfort.com', None, 'http://www.wcequestrian.com', 'http://warehousewars.net', None, None, 'http://www.baselineflowers.com/', 'http://luccaclothing.com/', 'http://www.ilovekickboxing.com/kickboxing-studio/index.php?id=374?utm_source=review&utm_medium=yelp&utm_campaign=referral%C3%8A', 'http://www.azdmovers.com', None, None, 'http://www.harveycatshotel.com/', 'http://www.bk.com', 'https://www.allinclusiveconcretesystems.com', 'http://www.raintreerestaurantchagrinfalls.com/', 'http://nisstech.com', None, 'http://www.ciaociaopizza.com/', 'https://www.bestcharlottemassage.com', 'http://www.LisaYoungDDS.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=1962&localechange=1', 'https://fullpoolservices.com', 'http://www.theparlorlv.com', None, 'http://tonysautoworks.net', None, 'http://nolinesbydesign.com', 'http://local.buckle.com/AZ/GLENDALE/331/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', None, '1592236581', None, None, 'http://wyattorthodontics.com', 'http://www.staxburgerbistro.com', None, 'http://www.progressivepavers.com', 'https://www.storagewest.com/facilities/baseline/?utm_source=YelpCTA&utm_medium=referral&utm_campaign=baseline-business', 'http://www.crowdpleezersbarbershop.blogspot.in/', 'http://www.AltierDental.com', 'https://www.victoriassecret.com/store-locator#/store/442', 'http://www.wickedclub.com', None, '1592236611', 'http://www.containerstore.com/locations/showStores.htm?zipCode=28211&searchArea=100', None, None, 'https://lljcustoms.com', None, 'http://www.kohakitchen.com', 'https://locations.wendys.com/united-states/pa/bridgeville/1125-washington-pike', 'http://toba.ca/', None, None, None, 'http://www.renue.ca', 'http://www.mobilehomedepot.net', 'https://www.rubytuesday.com/', 'http://www.odiagasautosales.com', None, None, 'http://www.rivieraopticare.com/', 'http://www.showcasemotorspa.com/', None, 'http://www.azsleepyteepee.com', 'http://giantjones.com', None, 'http://www.marcos.com/locations/oh/berea/w-bagley-rd?s=true', 'http://www.pimadental.com', None, None, None, 'https://www.centerforsightlv.com', None, None, 'http://stores.bestbuy.com/122/', None, 'http://www.averyonthegreen.com', 'http://www.krispykrunchy.com/locations/wy-market', None, 'http://www.thecamptc.com/location-gilbert', 'http://www.saigonpearl.ca', 'http://www.posaresalons.com', 'http://www.sanrio.com', 'http://Islandstylepools.net', 'https://turningpointbeauty.com', None, 'http://www.reneesmassotherapy.com', 'http://johnnyrockets.com', None, 'http://funkytoque.com/wellington-accueil/', 'https://www.smithsfoodanddrug.com/storeHours?store=70600354', None, 'http://therabbitspot.com/the-rabbit-hole/', 'http://www.psdm-madison.com', 'http://www.tizzerts.com/', 'http://flahertytree.com', 'http://www.saddlebroncgrill.com', 'http://www.dwmeats.com/', 'http://www.malleys.com', 'https://www.sprint.com/storefronts/bd/sprint-goodyear-az-85338-d-8157/', 'http://expertcarcareandtransmissions.com', 'http://www.themakeden.com/', None, 'http://www.arbys.com', None, None, None, 'http://www.cupcakejunkiecafe.ca/', None, 'http://www.carolinascornerstone.org', None, 'http://joinswift.com/landing-pages/DR/student/yp/careerhorizon/lp', 'http://monogramcoffee.com', 'http://www.littlecaesars.com', None, 'https://www.fastmed.com/urgent-care-centers/scottsdale-az-walk-in-clinic-mcdowell-road/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', 'http://www.stagsfamilychiropractic.com', None, 'http://www.luxhairsalon.com', 'http://www.fiveguys.com', 'http://www.smittys.ca', 'https://locations.dennys.com/MO/INDEPENDENCE/248531', '1592236785', 'http://www.wellnessmedspacosmetics.com/', 'http://sungshouse.com', None, 'http://www.gaatithaikitchen.com', 'http://www.starbucks.com/', 'http://www.augustoalgusto.com/fr/', None, 'http://www.thegroceryohc.com', 'https://www.mcalistersdeli.com/locations/il/peoria/4810-north-university-street-1289?utm_campaign=yelp&utm_medium=referral&utm_source=yelp&utm_content=1289', 'http://www.kangsbba.com', 'https://www.4patientcare.ws/wsv3pro/web/webschedulerv3.aspx?SessionID=2050207', 'http://noahsarkpediatrics.com/', 'http://eosfitness.com/location/north-scottsdale?utm_campaign=north-scottsdale&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', 'https://www.jandbhauling.com', 'http://goodboydogs.ca', None, 'https://www.tajo-one.com', 'https://www.barkbusters.com/dog-training-charlotte-south', 'http://studiobell.ca', None, 'https://stores.jcrew.com/en/crocker-park?srccode=X%7CPMG%7CNaturalSearch%7CLocal%7CYelp%7C677', None, 'http://www.allseasonsmaintenanceandlandscaping.net/', 'http://www.reborncabinets.com', 'http://mikadosushirobata.dining.ca', 'http://www.orchidspa.net/', None, 'https://www.rolandswigsalonmatthews.com', 'http://www.vegashottubs.com', 'http://hammertoronto.com', None, 'http://www.badassjacks.ca', 'https://www.hilton.com/en/hotels/phxmdgi-hilton-garden-inn-phoenix-midtown/?SEO_id=YELP-GI-PHXMDGI', 'https://www.180smoke.ca/vape-shop-etobicoke-islington-electronic-cigarettes', None, 'http://www.burtonstudio.vegas', 'http://mississaugabarbershop.com', None, None, 'http://www.terribleherbst.com/', None, 'http://www.alovingleash.com/', None, 'http://emeraldcomputers.com/', None, 'http://www.cookout.com', 'http://www.acupuncturecenteraz.com/', 'http://www.TheKyotoBowl.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216347&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://skintologyrx.com', 'http://leafandplate.tumblr.com', 'http://www.ballyslasvegas.com/restaurants/sea-the-thai-experience.html#.UkB2BsZnGoM', 'http://www.beinlovedesigns.com', 'https://www.athome.com/store-detail/?StoreID=Pennsylvania-Pittsburgh&utm_source=yelp&utm_medium=paidprofile', 'https://locations.theupsstore.com/pa/white-oak/1985-lincoln-way?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://eclipseautobody.com/', None, None, 'http://www.saltchaletarizona.com', '1592236923', 'http://www.rlemb.com', None, 'http://earlymercy.com', 'http://www.lpmproduction.com', 'http://www.nobullburgers.ca', None, None, 'http://www.wilsonstaekwondo.com', 'https://www.autozone.com/locations/az/avondale/10815-w-indian-school-rd.html', 'http://www.chilis.ca', 'http://eighteeneight.com/north-scottsdale-az/?utm_source=yelp&utm_medium=organic&utm_content=weburl', None, 'http://www.zenlia.com', None, None, 'http://www.bowwowdoggiedaycare.com', 'http://www.teakwoodstavern.com/', 'https://rolliciouscreamery.com/madison-1', 'http://www.goldcoastcasino.com', 'http://www.beinsoccer.com', None, None, 'http://www.vegasthedj.com', 'https://local.fedex.com/oh/mayfield-heights/office-1225/', 'https://stores.golfgalaxy.com/nc/charlotte/3039/', 'https://www.positivelyfrosted.com', None, 'http://www.azactorsacademy.com', None, None, 'http://www.rockymountainsoap.com/', 'https://pracharlotte.com/location/charlotte-cotswold-eastover/', 'https://www.taravietnamesegrill.com', None, 'http://www.ubreakifix.com/locations/pittsburgh', None, 'http://www.splaw.net/', 'https://www.olivegarden.com/locations/az/mesa/southern-fiesta-mall-west-mesa-alma-school-rd/1661?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:mesa-az-1661_pl:locurl_rd:1517', 'http://www.portofsubs.com/', None, None, 'http://www.concentra.com', 'https://www.deshonpullenlaw.com', 'http://www.mcfarlandlibrary.org/', 'https://www.wellsfargo.com', 'http://www.LegalRide.com', '1592237028', 'https://www.trainerpaul.com', None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', '1592237043', 'http://www.cefiore.com', 'http://www.roncesvallesanimalhospital.com/', 'http://jokercafe.info/', 'http://www.traderjoes.com', 'http://www.alohayogurt.com', 'http://www.outdoorlivinglv.com', 'http://www.litespressobar.com', None, 'http://www.mikesgranite.com', None, None, 'https://www.spectrumoralandfacialsurgery.com', None, 'http://www.mpi-services.com/', None, 'http://www.senseandcolor.com/weddings', 'http://gexwomenscare.com/', 'http://www.alleghenyvet.com', 'http://www.reclaimedsecrets.com', 'http://www.desertskyfamilydental.com', 'http://www.greenleafarrowheadranch.com', 'http://www.discount-drugmart.com', 'http://www.trueshinerimrepair.com', 'http://www.brars.ca', 'http://www.eagleindustriescorp.com', 'http://www.discountcab.com/', 'http://muddypawslv.com', 'http://cherryontop.net/', None, 'https://www.zoup.com', 'http://www.beckseuropean.com', 'http://www.ejsushi.com', 'https://locations.monro.com/pa/crafton/2175-noblestown-rd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'https://egl-junk-removal-lasvegas.business.site', 'http://pandora.net', None, None, 'http://www.drpease.com', 'http://www.zimbrickbuickeast.com', 'https://www.gessweinsclocks.com', 'https://www.huntington.com/Community/branch-info?locationId=625', 'http://www.tremontmusichall.com/', None, 'https://killumpestcontrol.com', None, 'http://fitnessconnection.com/clubs/southpark/', None, 'http://smiledocsaz.com', 'http://www.pinkislaserspa.com', 'http://www.empowerpt.com', '1592237155', None, 'http://www.artofshoerepair.com', 'http://www.starbucks.com/store/1013599/', 'https://www.londondrugs.com/london-drugs-store-030-calgary/heritage-plaza-030.html', 'http://www.libertymutual.com/brian-weaver', 'http://www.beansproutscafe.com', 'http://www.capitalone.com', 'http://www.kidzrockprep.org', 'http://www.lvsurgical.com', 'http://goharvestmarket.com', '1592237180', 'http://www.scottsdaleautosalon.com/home', 'http://www.studioadt.com/', 'http://www.bamdogwash.ca', 'http://www.staytruebodyart.com', 'http://www.liveatthepavilions.com', 'http://www.restaurantkarma.com', None, 'http://ifadevegas.com', 'http://www.loirerestaurant.ca/', 'http://www.flrpropertymgmtphoenix.com/', 'http://www.mrchickencle.com', None, 'http://cibronebakery.com/index.html', 'https://locations.timhortons.com/ca/on/richmond-hill/1070-major-mackenzie-dr.html', None, None, 'http://www.newlookhairnailandspa.com', 'https://www.elitemusic.ca', None, None, 'http://pageskensington.com/', 'https://www.bobevans.com/our-restaurants/locations/266?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.deseretfamilymedicine.com', 'http://algobertoslasvegas.onlunchroom.space', None, 'http://www.prettyyongething.com', 'http://www.solarpenny.com', 'http://www.southernnevadalocksmith.com/', 'http://www.wongsphoenix.com', 'http://www.dsw.com', 'https://palmbeachtan.com/locations/AZ/Peoria-West-Thunderbird-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'https://mountainsidefitness.com/locations/gilbert/', None, 'https://ad.doubleclick.net/ddm/clk/434177199;236814625;r', 'http://stlouiswings.com', None, 'http://www.posaresalons.com/', 'http://actionappliances.net', 'http://www.riteawayservices.co', None, None, None, 'http://www.buymaytag.ca', None, 'http://www.zibabeauty.com/', 'http://www.eternalhealthandwellness.ca', 'http://hoangthynailsspa.com/', 'http://www.kimmyztatumpoint.com', 'http://www.fisherkingwinery.com', 'https://lasvegas.colormemine.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.surlatable.com/browse/storeCalendar.jsp?storeId=137', None, 'http://www.spalv.com', 'http://www.fhtaz.org', 'http://www.shell.com', 'http://hgsystemintegrators.com', 'http://littlawfirm.com/', None, 'https://duckdonuts.com/locations/huntersville-nc-rosedale-shopping-center/', None, 'http://pittsburghpetgrooming.com', 'http://www.drkintonis.com', 'http://www.daniadown.com', None, 'http://www.azrealtycare.com/', 'http://www.basspro.com/webapp/wcs/stores/servlet/CFPageC?storeID=55&storeId=10151&langId=-1&catalogId=10051&tab=3&appID=94', 'http://www.omgbakedgoodness.com', None, None, None, 'https://www.usps.com/', 'http://drupatis.com', '1592237356', 'http://www.launicahuntersville.com/', 'http://www.ardene.com', 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=187&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-187', 'http://yourazbraces.com', 'http://www.parkwaychiropractic.net', 'https://cutrateglass.com', '1592237372', 'https://www.kohls.com/stores/nv/lasvegas-669.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.centropolis.ca', None, 'http://www.mvnailspa.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.wildwestmercantile.com/cgi-bin/Category.cgi?category=home&type=store', 'http://www.rosetuxedoaz.com', 'http://www.starbucks.com/store/9681/', 'https://www.therange702.com', 'http://www.mccalls.ca', 'http://www.chefkebab.ca', 'http://drjsoffice.com', '1592237404', 'http://www.waterjack.com', 'http://gonyfl.com', 'https://www.walgreens.com/locator/walgreens-275+davison+dr-sun+prairie-wi-53590/id=3291', 'http://www.kwalitysweets.com', 'http://velocitycx.com/', 'http://www.carolinashealthcare.org/cabarrus-pediatrics-map-and-directions', 'http://www.sheawomenscare.com', 'https://www.anytimefitness.com/gyms/2744/University-Heights-OH-44118/?utm_source=yelp&utm_medium=local', None, 'https://signup.myiclubonline.com/iclub/newAgreement/0389/selectPlan.htm', 'http://www.yogurtvi.com', 'http://www.venetian.com/nightlife/bars-and-lounges/the-bourbon-room.html', 'http://tropicaldelightscharlotte.com/', 'http://www.countryinns.com/champaignil', '1592237438', None, 'http://www.thecandybar.ca', None, 'http://www.pdqstores.com/gmaps/Wisconsin/MadisonArea/135.html', None, 'https://pho-fordays.com', None, 'https://www.sonoraquest.com/find-a-location/glendale-59th-ave-eugie/', 'http://www.queencitytours.com', None, None, 'http://www.atriasunlake.com', 'http://www.duetcatering.com', 'https://www.target.com/sl/charlotte-midtown/2244', 'http://www.starbucks.com/store/9298/', 'https://www.chick-fil-a.com/arboretum', 'http://marusakebar.com/', None, 'http://www.salazarusedcars.com', 'http://www.arizonadeck.com', None, 'http://www.jerusalemgrill.com', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.drtarapeyman.com', '1592237494', None, 'http://www.redcareers.com/kwrpp/', 'http://www.spiedini.com/', 'http://www.marvelousepoxyflooring.com', 'https://order.littlecaesars.com/en-us/stores/15150003/10720-e-southern-ave/mesa/az/menu', None, 'https://bhmanagement.com/communities/the-venue-on-camelback/', 'http://www.greencardadvice.com', 'http://erabyciara.com/', 'http://www.zocalobistro.com', 'http://tyrell710.wix.com/fingertips2thesoul', 'http://www.marshallsonline.com/', 'http://650cafe.com/', 'http://www.astaraskincare.com', 'http://www.arrowheaddermatology.com', '1592237535', 'http://www.serviceking.com/locations/service-king-penn-hills?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'http://locations.michaels.com/az/mesa/2778/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.centurylink.com', 'https://locations.jackinthebox.com/us/ca/concord/1710-monument-blvd', None, 'http://www.dads-smokeshop.com', 'http://www.friendlyneighborhoodhelp.com', 'https://www.olivegarden.com/locations/oh/parma/parma-town-mall/1128?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:parma-oh-1128_pl:locurl_rd:1083', 'http://www.beautifulskinbylucy.com', 'http://www.ritasfloraldesigns.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.arcelectriccompany.com', 'http://www.7-eleven.com/?yelp=29384', 'https://locations.massageenvy.com/arizona/scottsdale/8752-east-shea-blvd-ste--c-3b.html', None, None, 'https://redrock.sclv.com/Gaming/Race-and-Sports-Book?utm_source=Yelp&utm_medium=organic&utm_campaign=fb&utm_content=listinglink', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'https://www.arashijapansteakhouse.com', 'http://www.mgpaz.com', 'http://www.silversnail.com', None, None, 'http://www.ajsfinefoods.com', 'http://www.lasvegasguitarteacher.com', 'http://www.trilogypoolsaz.com/', 'http://poshpooches.ca', 'http://www.aleromoving.com/', None, 'http://www.litchfieldauto.net', 'https://www.crowncustomtattoo.com', 'http://www.illinismilesdental.com', None, 'http://www.udallattorneys.com', 'http://www.rvmastertechs.com', None, 'http://www.buffaloscafe.com', 'http://www.bengioielli.com', None, 'http://www.moulinbouge.com', 'http://birkdalevillage.net/portfolio/birkdale-village-cleaners', 'http://Munhallvet.com', 'http://www.hillhurstmortgage.com', 'https://www.zaxbys.com/locations/sc/fort-mill/3401-hwy-21/', None, 'https://www.bestwestern.com/en_US/book/hotel-rooms.03157.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:03157', 'http://ilfornello.com/richmond-hill/', None, None, None, '1592237668', 'http://www.bigotires.com/regions/minneapolis-metro', None, None, None, 'http://Www.electrikneedletattoo.com', 'http://www.apachetaxi.com', 'http://www.ibdaz.com', 'http://www.gfsbjj.com/', None, 'http://www.matthewsalehouse.com', 'http://www.antoniospizza.net', None, 'http://eastvalleyurban.com/phoenix/stella', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://DDLDental.com', 'http://familydentalstudio.net', None, None, 'http://www.friendsofthewild.org', 'http://www.libertymutual.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.einsteinbros.com/', 'http://www.paradisebaypetresort.com/', 'http://www.everlastingchanges.com', 'http://www.quiktrip.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://thebeatbooth.com', 'http://babycakeswilloughby.vpweb.com', 'http://www.arizonastone.com', 'http://www.newdirectionfertility.com', 'https://www.vegaslashlounge.com', '1592237755', '1592237758', 'http://www.thecurzon.ca', None, 'http://www.samsbarbershops.com', 'http://www.imeldasmith.com', 'http://tinkers-handyman-service.business.site', 'http://www.starbucks.com', None, None, 'http://www.shaggyshaggyshake.com', None, None, None, None, None, 'https://www.supercuts.com/locations/nv/henderson/eastern-commons-haircuts-9195.html', None, None, None, 'http://pho87.net', None, 'http://poolpartsplus.net/', 'https://www.dominionselfstorage.net/self-storage/az/queen-creek/dominion-self-storge-queen-creek/unit-sizes-prices', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Las%20Vegas/NV/8114', 'https://www.greatvaluestorage.com/self-storage/las-vegas-nv-92971', 'https://konjikiramen.com', 'http://yackodentistry.com', 'https://www.podiatryinc.com', 'http://www.freewayinsurance.com', 'https://chocolateshops.sees.com/az/phoenix/chocolate_shops_phoenix_az_102103.html', 'https://acesandarrows.com', 'http://www.noisette.ca/index.html', 'https://locations.cinnaholic.com/ll/CA/ON/Scarborough/6015-Steeles-Ave-E', 'https://www.sunnysidehonda.com', None, None, None, 'http://www.bwsweetsbakery.com', '1592237855', None, 'http://www.charlotte.studio-cellar.com', 'http://www.xooro.com', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'https://www.cellphonerepair.com/charlotte-uptown-nc/', 'https://ad.doubleclick.net/ddm/clk/458685299;262921492;c', 'http://www.somunsuperstar.com', '1592237871', 'http://www.archeryhq.com', 'https://www.agoracleveland.com', 'http://victoryenergywellness.com/', 'http://www.geeksnthings.com', 'https://nodabrewing.com', None, 'http://phoenix.sentrymgt.com', 'http://www.caringheartsanimalclinic.com', 'http://www.stellavapors.com', None, 'http://www.prohibitionclt.com/', 'http://scenicbrewingcompany.com', None, None, None, 'http://justpuckit.com', 'https://www.avioflower.com', 'http://www.bbhairringtons.com', 'https://www.olivegarden.com/locations/oh/north-olmsted/north-olmsted-great-northern-mall/1133?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:north-olmsted-oh-1133_pl:locurl_rd:1087', 'http://www.chousky.com', 'https://speedstarautotransport.com', 'http://www.kolornkiln.com', 'https://www.b1locksmith.com/chandler-locksmith', 'http://www.gamestop.com', 'http://www.gamestop.com', 'http://www.littlecaesars.com', 'http://www.fetaolivesgrill.com', 'http://pittsburgh.vettechinstitute.edu', 'http://luna-street-food.business.site', 'http://centurymill.com', 'http://www.BeestungLingerie.com', 'http://www.rapunzelhair.ca', 'http://www.reliableautolv.com', 'http://extremevaluestore.com/', None, 'https://www.happyaxe.com', 'https://kipc.vegas', None, None, None, None, 'http://www.coastappliances.com', 'http://www.phoenix.egoscue.com', 'http://www.bartonsbakeshop.com', 'http://www.banfield.com/veterinarians/wi/madison/emn?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://www.homehardware.ca/en/dealer-microsites/1274-8/contact-information.htm', 'http://www.europeanmotorstudio.com', 'https://wildfire.sclv.com/The-Greens', 'http://www.boosterjuice.com/', '1592237980', None, 'http://uniquepatiocreations.com', 'http://www.wexlerstavern.com/', 'http://theplacearizona.com', None, 'http://www.mtlebanonfloral.net/?srccode=yelp_track', '1592237997', 'https://thepetclub.com', 'http://www.encompasshealth.com', 'http://www.dimanddensum.com/', 'http://www.wingatehotels.com/hotels/hotel-overview?wid=ps%3Aaddppc_yelp&tel=18885953511&reg=appc-_-YLPROPP-_-30442-_-winwes&hotel_id=30442&cid=whg_winwes_yelp', 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', 'http://lasvegassolarinstallation.com', 'http://www.hairpieceslv.com', 'http://www.thecharlestondtlv.com', '1592238018', 'https://www.caferio.com/locations/eastern', 'http://www.skysalon.com', None, 'http://www.tourneau.com/custserv/locate_store.cmd?storeNumber=33', 'https://www.bootbarn.com', '1592238033', 'http://www.modelcitizentoronto.com/', 'http://www.epiphanychandler.org', 'http://www.trueblueautomotive.com/', 'https://www.anatomica.ca', 'http://churchschicken.ca', 'http://facialsbysofia.com', None, 'http://www.davidesbakeryandcafe.com/', 'http://www.calgarygold.ca', 'http://www.laroccaspizzeria.com', 'http://www.rowhousecinema.com', 'http://www.buddhapie.ca', '1592238062', '1592238065', 'http://www.sunshinegrandcanyon.com', 'http://www.handysacehardware.com', 'http://www.honeybutterbakery.com/', None, 'http://carrepairvegas.com', 'http://www.tortoiseandharesports.com', None, None, 'http://www.buyerschoicelv.com/', None, None, 'http://www.erieislandcoffee.com', 'http://www.baytowne.net', 'https://www.sephora.com/happening/stores/gilbert-san-tan-village', None, '1592238099', 'http://www.glendaleca.gov/', 'http://www.youreliteplumber.com/', 'http://bestimpressionscaterers.com/main.php', None, None, None, None, None, 'http://www.newvisionaz.org', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', None, None, 'http://larmco-oh.com', 'http://www.maisonettedistillery.ca', None, 'http://www.colonandrectalclinic.com', 'http://www.sirpizzafranklinpark.com', 'https://www.t-mobile.com/store-locator/nv/henderson/galleria-at-sunset-mall?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://www.dressbarn.com', 'http://www.choicehotels.com/en/choice-privileges', None, 'http://lrcorralescatering.weebly.com/', 'https://www.floxondanforth.com', None, 'https://dresspeopleltd.com/', None, 'http://Leonscaribbean.com', None, 'https://www.zippssportsgrills.com', 'http://www.clancyspubscottsdale.com/', 'http://www.alimentarimadison.com', None, None, 'http://www.atlashomeinspections.com', None, 'http://zinicolaitalian.com', 'http://www.sunsetclubcharlotte.com/', 'http://www.airspecialiste.com/', 'http://www.deltaco.com', 'http://www.wincofoods.com/', 'http://www.DFgym.com', 'https://www.smilesbysmith.com', 'http://www.drbahmanguyuron.com', 'http://salonstudiowest.com', None, '1592238192', None, 'http://royaldutchgrooming.net', 'http://www.dentistsonbloor.ca/', 'http://www.presidentspizza.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.03133.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:03133', 'http://www.scottsdaledentalcare.net', None, 'http://www.nailsforyou.ca', 'http://www.zumiez.com', None, 'http://www.expressgaragedoorservice.com', 'http://acebroslv.com', 'http://www.tacarbon.com', 'https://www.onemainfinancial.com/branches/PA/Pittsburgh/15234/4721', 'http://www.starkmans.com', 'https://preciousmetalsnv.com', 'https://www.extraholidays.com/las-vegas-nevada/worldmark-las-vegas-spencer-street?lid=yelp&utm_medium=referral-affiliate&utm_source=yelp.com&Identifier=yelp&hotelid=103712&utm_campaign=localpage-worldmarklasvegas-spencer&promoid=localpage-worldmarklasvegas-spencer', None, 'https://www.lovestbrides.com', 'http://www.sakunthaionline.com/', 'http://www.gocomtrans.com', 'http://www.sanremobakery.com/', 'https://piedmontmaids.com/local-house-cleaning/nc/monroe', 'http://www.fluidmotionpgh.com', 'http://www.blisscupcakes.com/', None, 'http://www.tanprousa.com', None, 'http://www.rebekhasushi.com', 'http://www.crispfoods.com', 'http://www.usps.com', 'http://www.lvmpd.com', None, None, None, 'http://www.park2go.ca', 'http://www.horstmancomputers.com', None, 'https://www.walmart.ca/en/whitby-supercentre/3113', None, 'http://colosseumtraining.com', 'https://www.kindercare.com/our-centers/phoenix/az/000477?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=000477', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216206&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.anytimedent.com', None, 'https://www.westelm.com', 'http://www.grayhawkbeerco.com', 'http://www.nextcare.com/locations/az/phoenix-thomas/?utm_source=Yelp&utm_medium=Local&utm_campaign=phoenix_thomas', 'http://goclarendon.com/restaurant', None, None, None, 'http://www.quiktrip.com', 'http://charliepalmersteak.com/locations/las-vegas', 'https://www.sprouts.com/store/az/laveen/laveen//?utm_campaign=website&utm_medium=listing&utm_source=yelp', 'http://www.thomasharman.com?cmpid=czhr_blm_0006', None, 'https://www.autoactionaz.com/dealership-locator/locations-1909-west-van-buren-street-phoenix-az/', 'http://www.ridgecleaners.com', 'http://www.plantworks.com', '1592238330', 'http://www.powerhouseplatinum.com/', 'https://thefortunatefox.com', 'http://www.ghazale.ca/', 'http://www.outback.com/locations/nv/las-vegas-craig', None, 'http://www.boom973.com/index.aspx', 'http://www.maac.com/texas/dallas/colonial-grand-at-hebron?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://ArizonaSailingAdventures.com', None, 'https://myhearingcenters.com/locations/nevada/summerlin', None, None, 'http://www.wellerracing.com/', 'http://www.timhortons.com', 'https://local.fedex.com/nc/pineville/PNVA/?cmp=LOC-1001545-3-2-971-1110000-US-US-EN-YEXTLISTFEDEXSC', 'http://primmed.org', None, 'https://www.cleanjuice.com/menu/', 'http://www.innovativehclinic.com', 'http://www.quickchangeoil.com/', 'http://www.azmortgagebrothers.com', 'https://valleywisehealth.org/locations/community-health-center-el-mirage/', 'https://www.menswearhouse.com/store-locator/2504?utm_source=yelp&utm_medium=listing&utm_content=2504&utm_campaign=AZ', 'http://www.liveatavanteapts.com', 'https://www.staples.com', 'http://www.northamerican.com/?refcode=844-434-1236&utm_source=yelp&utm_campaign=desktop', None, 'http://99only.com/store/Las-Vegas---Beltway/', 'https://theresortatsummerlin.com/dining/casual-dining/market-place-buffet-las-vegas/', 'https://www.jph4ever.com', None, 'http://www.imsaz.com', 'http://www.afabaz.com', None, 'http://www.perfectlookhairbraiding.com', 'http://www.regissalons.com/salondetail/default.asp?salonid=448', 'http://armor-plumbing-boiler.com/index.php', 'http://www.greatwraps.com', 'http://www.steepery.com/', 'http://nailsqueentoronto.ca', None, 'http://www.rodneysbybay.com', None, None, 'http://www.fitnesspropelled.com/', 'http://www.hawksbar.com', None, None, 'http://www.shakeitupbartending.com', 'http://www.salts-of-the-earth.com', None, None, 'http://store-locator.barnesandnoble.com/store/2039', 'http://www.vbflv.org', 'http://www.outlettags.com', 'http://www.702wingspot.com/', 'http://www.kitchenconnectionllc.com', 'http://www.lvparking.com', 'http://www.radair.com/', None, None, None, 'http://magroclothing.com', 'http://www.furnituretoronto.com', None, None, 'http://www.k2elec.com/', 'http://www.delanolasvegas.com/pool/delano-beach-club.aspx', 'http://woksupreme.com/', 'http://www.markandterri.com', 'http://www.goodlifefitness.com/locations/ontario/vaughan/milani-blvd?source=yelp-websitelink-vaughanmilani-gl', 'http://www.thegreatsteak.com/stores/7525', 'http://www.newdragonexpress.com/', 'http://www.daalroti.ca/', None, None, 'http://www.yourstarpupil.com', 'https://www.mccarran.com/Fly/WhichTerminal.aspx', 'http://www.lasvegascounselingcenter.com', None, None, 'http://www.glastek.com', '1592238519', None, None, 'http://www.burlingtoncoatfactory.com', '1592238526', 'https://www.galleriamc.com', 'https://www.tcclinic.com', 'https://www.midas.com/phoenix/store.aspx?shopnum=1109&dmanum=684001', 'http://az.scottsdale.charmingcharlie.com/fashion_accessories_scottsdale_az_45.html', 'http://www.shell.com', 'http://www.lapetite.com/your-local-school/monona-wi-7148/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', None, 'https://www.jabzboxing.com/locations/32ndandshea/', None, 'http://www.tipsetreats.com', None, 'http://www.timhortons.com', 'http://www.amazingphotocamera.com/', None, 'http://www.waddellslonghorncorral.com/', 'http://www.desertsplashadventures.com', 'http://suburban-sanitation.com', None, None, None, 'http://www.thegilcreaseorchard.org', None, None, 'http://www.yasisplace.ca', None, 'http://philhousegrill.com', '1592238583', 'http://www.opticcreations.com/', 'http://www.trainingrevelations.com', None, 'http://www.statclinix.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, '1592238598', 'http://locations.daltile.com/az/tempe/118', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, '1592238610', 'http://www.josephsfoodandcakes.com', None, 'http://www.rowdyhair.com', 'http://www.starbucks.com/', 'http://jackastors.com/location-map/18/', None, 'https://www.choicehotels.com/ohio/wickliffe/econo-lodge-hotels/oh335/rates', None, 'http://www.blondesalon.ca', None, 'https://www.papajohns.com/international', 'http://luv2play.com/surprise/', 'http://independenceplumbing.net', 'https://pawn1st.net/our-locations/pawn1st-surprise-arizona/', None, 'http://www.lasvegasprostho.com', 'http://www.isshoniramen.com', None, 'http://www.eatatnakama.com', 'http://www.macklems.com/', '1592238663', 'http://www.towerofjewels.com/', 'https://stores.dickssportinggoods.com/oh/aurora/121/', 'http://www.lomography.ca', 'https://bostonpizza.com/fr/locations/st-bruno.html', None, 'https://www.kindercare.com/our-centers/goodyear/az/301663?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=301663', None, 'http://www.tinytooth.com', 'https://custommufflersaz.com', None, 'http://www.hairsmith.co', 'http://www.medicineshoppe.com/pharmacy-locations/pennsylvania/mt-washington/home', 'http://www.virgoesthetics.com', 'http://fawnboutique.ca/', 'http://www.btrustsupermarket.com/', None, None, 'http://www.allhoursplumbinginc.com', 'http://www.pridemgmt.com', 'https://thrifty.com/loc/ll/CA/OT/TORONTO-~-DANFORTH-(OT)/1245-DANFORTH-AVE', 'https://www.wisconsinrvworld.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sfnevada.com?cmpid=UN7I_BLM_0006', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.shibuyala.com', 'http://www.bell.ca/internet', '1592238730', None, None, 'http://www.deltaco.com', 'http://www.pizzapizza.com', 'https://habanerofreshmex.com', 'http://www.golfanything.com', None, 'https://la-coppa.com', 'https://www.montanas.ca', '1592238753', 'https://www.rallyhouse.com/rally-house-parma', 'http://balloonscharlotte.com', 'http://tton10.wix.com/pizanocustomautobody', 'http://www.secondcup.com/location/toronto-355-danforth-avenue', 'http://www.mmpguns.com', None, 'http://www.hair-dosalon.com', 'http://www.oxfordcleanersaz.com', None, 'http://hookahlounge.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7781&localechange=1', 'http://www.silverstatevapor.com', None, 'http://www.sessionstattoo.com', '1592238788', None, None, None, 'http://wokintercuisine.com', 'http://sushi-ten.com/', None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=2179&localechange=1', 'http://madisoncircusspace.com', None, None, 'https://myhoneybakedstore.com/charlotte/114', 'https://wexford.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', None, None, None, None, 'http://www.marshallsonline.com', 'http://www.tiashortys.com', 'http://www.mariscosacapulco.org/Mariscos_Acapulco/Inicio.html', 'https://cmwireless.com', None, None, None, None, 'http://thelocalbarberandshop.com', None, 'http://www.mcgrathcpa.net', 'http://www.northranchvet.com/', 'http://www.unosalonlv.com', 'http://michelsbakerycafe.com/?q=node/1/en/', 'https://www.ebgames.ca/', 'http://www.drivesafeaz.net', 'http://www.journeys.com', 'https://www.engagestudio.com', None, 'http://www.azcostumeshop.com/', 'http://www.painsetsaveurs.com/', 'http://www.extraordinarytoys.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=12605&localechange=1', None, 'http://www.fabricland.ca', None, 'http://www.quiktrip.com/', 'http://www.ahwatukeehoa.com', None, 'http://chase.com', None, 'http://Butlerfamilydentalaz.com', None, None, 'http://www.haydenelectricinc.com', 'http://www.parmatireservice.net', 'http://www.marcos.com', None, 'https://www.eliterooterphoenix.com', None, None, 'http://www.ilovepokebar.com', 'http://sushi-mori.ca', 'https://www.propertymanagementscottsdaleaz.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.jnphoenixfamilylawyer.com', 'https://www.vans.com', 'http://www.rowefarms.ca', 'https://www.sephora.com/happening/stores/hillcrest', None, None, 'https://www.cubesmart.com/nevada-self-storage/las-vegas-self-storage/90.html?utm_source=yelp&utm_medium=seo&utm_campaign=yelp-test-bp', 'https://shakeshakeseafood.com/contact_monroe', 'http://www.montessorivisions.com', None, None, None, 'http://www.MarshProperties.com', 'http://www.setispolishboys.biz', 'http://www.ctuckerandassociates.com', 'https://www.everbrookacademy.com/our-schools/charlotte-nc-4608/?utm_campaign=enhanced-listing&utm_medium=paid-directory&utm_source=yelp', None, 'http://www.costco.ca/', 'http://www.azdermacare.com/', 'http://www.yumbudz.ca', 'https://theeggworks.com', 'http://www.forgetaboutitsupperclub.ca', 'http://www.lbcexpress.com', None, 'https://www.reflectionzdetail.com', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://www.lebelandcrowe.com', 'http://www.greencleanerslv.com', 'https://drgeoffreysherivf.com', 'http://www.beadworksjo.com', 'http://www.craigscustomupholstery.com', 'https://www.walmart.com/store/5066/avon-oh/details', None, None, 'http://www.philotavern.com/', 'http://www.abercrombiekids.ca/', None, 'http://www.jessieraesbbq.com', 'http://www.vetemergency.ca', 'http://www.carolinadoughnuts.com', '1592239007', 'http://www.starbucks.com', 'http://www.thespainthehollow.com/', '1592239015', 'http://www.bestclean-llc.com/', 'http://galenaeuropeanspa.com', 'https://www.kidsquest.com/location/boulder/', 'http://www.vsschnitzelhouse.com', None, None, 'http://www.fluidconcepts.net', 'https://www.myfitfoods.com/', 'http://kathysalterations.biz', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/16545+East+Palisades+Boulevard-Fountain+Hills-AZ-85268/storeid=9279?cid=ps_ylp_2020', 'http://reflectionsautodetailing.me', None, 'http://www.thecannerygrill.com', None, None, None, 'http://www.paintboy.com/', '1592239059', None, None, 'http://arizonawindowwashers.com', 'http://www.courtyardatencanto.com/', 'https://titanicaz.com', None, 'http://labayoucasino.tumblr.com/', None, 'http://www.sapporoscottsdale.com', None, None, 'http://kappatoys.com', 'http://www.shoelabworld.com', 'http://www.goodwillaz.org', 'http://bk.com/', 'http://www.soldentalaz.com', 'http://www.lasvegaskidsrkids.com', 'http://www.caninecountry.ca/', 'https://www.halibuthouse.ca/locations/halibuthouse560mulockdrive/', 'http://www.pinoyscoffeeandboba.com', 'http://cp.cuyahogacounty.us/internet/index.aspx', None, 'http://www.airdrieautotrends.com/', 'http://www.leonesflorist.com', 'http://www.wavescoffee.com', 'http://viennamarketbroadway.com', None, 'http://www.desertskysurfaces.com/', None, 'http://www.chinachilirestaurant.com', 'http://www.bethanychristianschool.org', 'http://www.mazonac.com', 'http://stores.dickssportinggoods.com/az/mesa/725/', 'http://charlottefiberglass.com', None, 'http://www.chinapalacesewickleypa.com/', 'http://www.aceaxethrowing.com', 'http://www.exlacademy.com', 'http://www.deltastore.com', 'http://www.desertsunpt.com', 'https://bloomingnails-nailsalon.business.site', 'http://www.caremd.com/', 'http://giovanniandcompany.com/', None, None, 'http://www.razzles.biz/', None, None, None, 'http://www.dollartreecanada.com', 'http://www.socialsolutionsaz.com', 'http://www.springdayspa.com', None, 'http://www.bbkingclubs.com/', 'http://haciendasolanoresort.com', 'http://www.xperiencevapes.com', 'http://www.leakpoint.com', 'https://www.abccollisionrepair.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'https://stores.loft.com/us/pa/homestead/126-west-bridge-street.html', None, None, 'http://eosfitness.com/location/tempe?utm_campaign=tempe&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, None, '1592239206', None, 'https://dealer.sprint.com/experts-choice', 'http://local.biglots.com/oh/stow/453', 'http://www.yayaspizza.com/', 'http://www.portioncontrolnv.com', None, '1592239219', 'http://www.usmgcorp.com', None, None, 'https://www.boydautobody.com/locations/calgary-1st-st-ab', 'http://www.tranquil-lily.com', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'https://www.aireserv.com/north-valley', None, 'http://www.bluerosepottery.com', None, 'https://locations.panerabread.com/oh/highland-heights/6345-wilson-mills-road.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', 'http://www.retroburger.ca/', 'http://crunchbistro.com', 'http://www.valuevillage.com', 'http://www.bleedingrosetattoo.com', None, None, None, 'https://www.goodyearautoservice.com/en-US/shop/AZ/Glendale/000908234-85308-2429', 'http://www.lisboabakeryanddeli.com', None, None, 'http://www.swingpgh.com/', 'http://www.daydreamsdropin.com', 'http://valleypoolcleaning.com/', None, None, 'http://www.vaultnc.com/', 'http://www.tonicsalonnv.com/', '1592239283', 'https://airpromaster.com', 'http://www.queencityautosalesnc.com', 'http://pageperpage.com', None, 'https://www.firehousesubs.com/locations/oh/willoughby-commons', '1592239298', None, 'http://laptoprepairs.com', None, None, 'http://www.lashliqueur.com', 'http://www.airportgatecafe.com/', 'http://www.naturallyyours.ca', 'http://www.shopkkbloom.com', 'http://www.honestyelectricllc.com', None, 'http://www.husd.org', 'http://www.ucalgaryrecreation.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.exacttemphvac.com', None, None, 'http://www.madisonorientalwok.com', 'http://www.crossfitarchangel.com', None, 'http://www.jolieannephotography.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://www.rockabettyssalon.com/', 'http://c2cnd.org', 'http://www.fantasticsams.com/', None, 'http://www.gamestop.com', 'https://www.safelite.com', 'http://cleanenvision.com', 'http://www.ahugheshairdesign.com', None, 'http://www.filibertos.com', 'http://www.kmjwebdesign.com', None, 'http://www.dillysdeli.com/', None, 'http://www.kimchikoreahouse.ca/', None, None, 'http://www.mississauga.ca/portal/discover/parksandgardens?paf_gear_id=9700018&itemId=300019', 'https://www.publicstorage.com/arizona/self-storage-mesa-az/85210-self-storage/787?pid=wyelp&CID=1245&CHID=AFL', '1592239382', 'http://www.teavasee.com/', 'http://www.popularpizza.ca/', None, 'http://www.steffiblackcoaching.com', 'http://www.virginiajohnson.com', 'http://www.kindercollege.ca', None, 'http://www.toysrus.com/family/index.jsp?categoryId=2255956&camp=MISC:yelp:store:page:tru:111910', 'http://www.gdpeds.com/', 'http://www.coldwaterspringsapts.com/', 'http://www.eyecarecenter.com/optometrist-locations/matthews-mint-hill-rd/?address=28105', 'http://www.minus5experience.com/', None, None, 'http://premiercleaners.com', None, 'http://www.theherbcard.com', None, None, 'http://www.maac.com/north-carolina/charlotte/post-ballantyne?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', None, None, 'https://www.unlv.edu/dental', 'https://www.thestonecollection.com/phoenix', 'http://www.sweetpetes.com', None, 'https://www.aaautocarelv.com/locations/decatur-blvd/', None, 'http://www.allnationsdriving.ca', 'http://icehousebrooklyn.com/', 'http://www.carusogourmetpizza.com/', None, None, 'http://www.sparrshspa.com', None, 'https://www.mezzo1.com/?utm_source=yelp', 'http://www.jimtuckerroofing.com', 'https://www.redmountainweightloss.com/ahwatukee/', None, 'http://www.jackinthebox.com/', None, 'http://www.thebeautyloftaz.com', 'http://www.papadelspizza.com/', 'https://www.maricopa.edu', None, None, 'https://www.lasereyecenteraz.com/eye-doctors/kianoush-kian-m-d/', 'http://www.Altus-HPO.com', 'http://yourspacesalons.ca/natural-solutions', None, None, None, 'http://www.buckle.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'https://www.glassdoctor.com/phoenix', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=1131', 'http://goodfoodmadison.com', None, None, 'http://www.suicidejims.com', None, None, 'http://www.thelaughshopcalgary.com', None, 'http://www.skyzone.com/phoenix', None, None, 'http://www.animalcareclinicinc.com/henderson-animal-hospital', None, 'http://www.center-towing.com', 'http://www.regency.edu/campus/north-olmsted-ohio.aspx', 'https://www.purdys.com/shops/details/79', 'http://www.hottopic.com', 'http://www.carolinatotalcare.com', None, 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://archstonecare.com/', 'http://www.kibosushi.com', 'http://rulysauto.com', None, 'https://www.earthfare.com/stores/', 'http://www.allapplianceneo.com', 'http://denver.unique-accounting.com', None, 'https://www.wyndhamhotels.com/hotel/33877?tel=18003291073&iata=00093785&cid=fe:di:20161220:yl:pp:dius:33877', None, 'https://vansoncuisine.ca', 'http://www.donsappliances.com', 'http://alphillipslv.com/', '1592239567', 'http://www.larrystow.com', 'https://www.marleneharriscol.com', 'http://www.consonantskincare.com', None, 'https://www.tgifridays.com/locations/1742/fridays-charlotteuniversity-north-carolina', 'https://locations.eatandys.com/ll/US/NC/Concord/8835-Christenbury-Pkwy', 'http://jmacvideo.com', 'http://www.aosmlv.com', 'http://enticare.com', 'http://www.marriott.com/hotels/travel/pitmv-springhill-suites-pittsburgh-monroeville/?scid=bb1a189a-fec3-4d19-a255-54ba596febe2', 'http://www.theciderfarm.com', 'https://www.presslyanimalhospital.net', None, 'http://www.remax.com/realestateagentoffice/lasvegas-nv-89128-sandrathomas-id11154472.html', 'http://www.meadowlarkestables.com/', 'http://www.salon-atmosphere.com/', 'http://chicagobrewinglv.com', None, None, None, 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/mississauga-2191-dundas-street-e.aspx', 'http://www.doitbest.com', 'http://sterlingssteakhouse.ca', 'http://www.saloncollage.com', 'http://www.eatatbobsdiner.com/', 'http://www.kamasutrarestaurant.ca', 'http://www.swisschalet.com', 'http://www.dunkindonuts.com', None, 'http://locations.atipt.com/grayhawk-scottsdale-az', 'http://www.magicvaporllc.com', 'http://www.starbucks.com', 'http://www.AllBreedMadison.com', 'http://www.montanameatco.com', 'http://bk.com/', 'http://www.starbucks.ca', None, 'http://www.autobell.com/ab_locations/750-n-church-st/', None, None, 'http://www.marriott.com/hotels/travel/cleta-towneplace-suites-cleveland-airport', None, None, None, 'http://workoutanytime.com/charlottewest', 'http://carolinasmiledesignsc.com', 'https://depotbarbershop.business.site/?m=true', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', '1592239672', 'http://www.libertyiron.com', 'http://aaffordablebabysitting.com', 'http://www.quesada.ca', 'http://bk.com/', 'https://locations.theupsstore.com/wi/sun-prairie/1223-w-main-st?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.kay.com', 'https://www.ugg.com/stores/s/fashion-show-mall?latitude=36.127172&exclude=18231&page=1&longitude=-115.171661&storesPerPage=3', 'http://www.creolecajunbistro.com', None, None, None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpsoutheastorganic', 'http://www.wolfcamera.com', 'http://www.corygoldbergmd.com/', 'http://www.harborfreight.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.paksweets.com', 'http://www.airpads.com', None, 'http://www.norterrafamilymedicine.com', None, 'http://www.kensingtonbrewingcompany.com', 'http://25chiropractic.com/', '1592239728', 'http://coffeedaklak.com', None, None, 'http://freddyrguez.googlepages.com/', 'https://www.crossborderpickups.ca', 'http://www.dilworthbilliards.com/', None, 'http://www.ahghomewarranty.com', None, None, '1592239754', 'http://www.guggenheimhermlasvegas.org', 'http://www.pearl-factory.com', None, 'https://local.biglots.com/pa/pittsburgh/1478?zcp=pd_local_yelp', 'https://www.navyfederal.org/?cmpid=bn%7Cyelp%7Cevgrn%7Cbrn%7Cnull%7Cnull%7C01/13/2020%7Cml%7Cnull', 'http://www.sallybeauty.com', 'http://www.salonfair.com', 'https://sunvek.com', 'http://www.midas.com/calgary/store.aspx?shopnum=9304&dmanum=151', 'http://www.bakersbookkeeping.com', None, 'https://phoenix.swimspas.us', None, 'http://torontopremiumoutlets.ca/outlet/toronto', 'http://www.midwifeskaleidoscope.org', 'http://www.redrockfencingcenter.com', 'http://www.azairworks.com/', 'http://auraapts.com', None, 'http://www.discountroofingnv.com', 'http://www.drvinyl.com', 'http://www.newskyrestaurant.ca', None, 'http://www.portauthority.org', 'http://www.downtownbrewfestival.com', None, 'http://www.globallylocal.ca', 'http://www.kakabeautyspa.ca', None, 'http://www.noelinwheeler.tattoo', 'http://concourseroffler.blogspot.com', 'http://www.rexall.ca/', 'http://www.trailsidetreasures.com', 'http://www.nevadatirecity.com', 'http://www.cornerstonewomenscare.com', 'http://www.easyhome.us', 'http://www.mondashaircare.com', 'https://www.desertfinancial.com/Home/BranchDetails?branchId=1', 'http://newks.com', 'http://www.dcspoolbarriers.com', 'http://www.drrochlin.com', 'https://www.fit4less.ca/locations/alberta/calgary/408?utm_source=yelp&utm_medium=cta&utm_term=calgarymacleod-f4l', 'http://www.julietlefleur.com', 'http://www.arizonahairco.com/', 'http://www.dunkindonuts.com', 'http://montana-hookah-longue-and-smokeshop.business.site', 'http://miniso.ca', 'http://www.westairaviation.com', 'https://www.sunrisepreschools.com/locations/baseline-gilbert/?utm_source=yelp&utm_campaign=online_listing_gilbert_280&utm_medium=referral&utm_term=business_url', 'http://www.mackdentistry.com/', 'http://thayersselectmeats.com', 'http://undergrounddiesels.com', '1592239872', 'http://www.samuraicomics.com', '1592239878', 'http://www.michaelkors.com/stores/united-states/nv/las-vegas/michael-kors-palazzo', 'http://willytechshop.com', 'http://www.vernspizza.com', 'http://www.clearviewinstitute.com/', 'http://www.berkeleyplace-apts.com', '1592239893', None, None, 'https://venecia-beauty.business.site', 'http://www.haircutmenriverbendcalgaryab.ca', 'https://www.hungryhowies.com/store/hungry-howies-01805?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=01805&utm_content=Website_Link', 'https://sophiaslounge.com', 'http://www.taxicabchandler.com', 'http://www.bestchickenwings.com/', 'http://www.athletico.com/locations/tempe/?utm_campaign=yelp-listing&utm_medium=web&utm_source=yelp-tempe', 'http://www.solsticerestaurant.ca/', 'http://www.azclubluxx.com', 'http://www.sethkoreytennis.com', None, None, 'http://www.flairtrade.ca', None, None, None, 'http://www.vegasmartialarts.com', 'http://riobel.ca', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.hairea51.net', 'http://www.dmvdrop.com', 'http://carolklemp.wix.com/aefitness', 'http://www.3Brewers.ca', 'http://www.lisaturley.com', 'http://oldtownweddingcenter.com/', None, None, None, None, None, 'https://zoomrentals.com/toronto-airport-car-rentals', 'http://www.rehabplusscottsdale.com', 'http://azdirectmobility.com', 'http://www.torpedocomics.com', None, 'https://www.aladdinswindowtinting.com', None, None, 'https://www.ewingirrigation.com', None, None, None, '1592239999', 'http://www.dapaulsalon.com', None, 'http://productnightclub.com', 'http://desertmesapool.com', None, 'http://www.cltchristmasvillage.com', None, '1592240015', 'http://www.animalhospitalhighpark.com/', 'https://bhrcenter.com/location/boca-park-nv/', 'http://www.conns.com/store-locator/arizona/scottsdale-85250', None, None, None, 'http://www.bestfootspascottsdale.com/', 'http://elitepetcare.com/', 'http://www.reservesquareapts.com', 'http://www.livelovelashe.ca', 'http://www.ibrandyourbusiness.com', 'http://wingsonbroadway.com/', 'https://mugshotzofficial.wixsite.com/mugshotz', 'https://www.luckysmarket.com', 'http://linkfinancialadvisory.com', 'https://www.gocleancredit.com', 'http://www.rothmanawningco.com/', 'http://www.fredastaire.com/scottsdale', 'http://www.sweettomatoes.com', None, None, 'http://www.bellairconditioningaz.com', None, 'http://www.arbys.com', None, 'http://tempeautorepairshop.com', None, 'http://martinjlevinedds.com/', '1592240069', 'http://www.carolinaurology.com', 'http://www.whiteningxpressions.com', 'http://www.chevron.com/', None, 'http://www.jambajuice.com', 'https://www.honorhealth.com/locations/hospitals/scottsdale-thompson-peak-medical-center?utm_source=yelp&utm_medium=local&utm_campaign=Scottsdale_Thompson_Peak_Medical_Center', '1592240085', 'https://locations.wafflehouse.com/nc-pineville-1157', None, 'http://www.lasvegasapartments-r-us.com/Mirabella-Apartments-for-Rent-in-Las-Vegas.html', '1592240094', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', 'https://www.waterstreetpizzeria.com', 'https://familymoversexpress.com', None, None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.asphaltpavingranaldi.com', 'https://locations.jackinthebox.com/us/az/peoria/8236-w-deer-valley-rd', 'http://www.tjmaxx.com', None, None, 'http://www.swarovski.com/en-US/?utm_source=YELP&utm_medium=referral_managed&utm_campaign=YELP', 'http://www.chiefdodge.com/', 'https://www.titlenine.com/store-locator/title+nine+-+madison.do', 'http://www.tiremanusa.com/locations.aspx', None, 'http://www.wolfsonmedicalcenter.com', 'http://greenwayfamilyautoandtowing.com', 'https://www.orangetheory.com/en-us/locations/arizona/peoria/25314-n-lake-pleasant-pkwy-suite-1260/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.woodysdogbath.com', 'http://www.mayfieldbrainard.com', None, None, 'http://www.scrubsandbeyond.com', 'http://fitnessfactorycharlotte.com/', 'http://azsportsmedicine.com', 'http://www.idealservices.com', None, None, None, 'https://www.brasstapbeerbar.com/Charlotte', 'http://www.seaminglystraight.com', None, 'http://www.whitfillnursery.com', None, 'http://www.chip-n-daleslandscaping.com', 'http://www.illinitechcenter.com', None, None, 'http://www.villageicecream.com', '1592240192', None, 'http://www.carpet-cleaning-pittsburgh.com', 'http://www.thelowcarbgrocery.com', 'http://lezah.ca', 'https://www.brakesplus.com', 'http://paulmitchell.edu/phoenix', None, None, None, None, 'http://www.alohabbqgrill.com', 'https://www.esso.ca/en/gas-station/etobicoke-on-geesso-200302279', 'http://www.edencafelounge.ca', 'http://www.signsbytomorrow.com/mesa/', 'http://www.borders.com', None, None, 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://peoriafootandanklespecialists.com', None, 'http://tshirtdinervegas.com', None, None, '1592240240', 'http://lebistrodubeaujolais.com/', 'http://farmershouse.ca', 'http://www.sarkujapan.com/', None, None, 'http://www.charlotterusse.com', None, 'http://kpiteophotography.com/', 'http://www.aceshighpartyrental.com/', 'http://www.harveys.ca', 'https://www.walgreens.com/locator/walgreens-11001+s+eastern+ave-henderson-nv-89052/id=6545', 'https://www.altimadental.com/locations/first-canadian-place', 'http://www.rockcitytaphouse.com', 'http://www.thelucky20cafe.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYCSC', None, None, 'https://www.yeahyeahspizza.com/toronto', None, 'https://locations.panerabread.com/oh/parma/8109-w-ridgewood-dr.html', 'http://www.clearsound.co', 'http://www.sunrisecoffeelv.com', 'http://www.herbergertheater.org', None, None, 'https://pittsburghmoving.net', 'http://www.phoenixcakecompany.com', 'http://www.timothys.ca', 'http://www.olivetree-apartments.com', None, None, 'http://www.ltayh.com', 'https://www.walgreens.com/locator/walgreens-1745+e+southern+ave-tempe-az-85282/id=1197', 'http://dobsonranchdentalcare.com', None, 'http://www.elitedancestudiolv.com', 'http://www.boosterjuice.com', 'http://www.meckabc.com', 'https://www.realestatelawfirmphoenix.com', '1592240325', 'http://www.goodyearsfamilymedicine.com', 'http://www.drdanstrinkoski.com', 'https://www.petpeoplestores.com/avon-store.html', 'http://www.bedfordautocare.com', 'http://www.shininghopefarms.org/', None, 'http://www.freshandeasy.com/', 'http://www.csuohio.edu/artgallery', 'http://www.desertpalettepainting.com/', 'http://www.premierairsolutions.com', '1592240349', 'http://www.desertroselandscaping.com', '1592240354', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://www.dmconcrete.biz', 'http://bluestardiner.ca/', None, 'https://sweetiepiescatering.com', 'https://www.azbouncepro.com', 'http://www.ritters.com', None, 'http://Arizonasoldsisters.com', 'http://www.restaurantdepot.com', 'http://pizzafire.com', 'https://www.ikea.com/us/en/stores/tempe/', None, 'http://www.regaloaksdental.com/', 'https://www.signaturestyle.com/locations/pa/penn-hills/borics-hair-care-penn-hills-shopping-center-haircuts-75141.html', 'http://99only.com/store/Las-Vegas---Beltway/', None, 'https://www.jovannalv.com', 'http://www.coppercanyondentalaz.com/', '1592240397', 'http://journeyeducation.org', None, 'http://www.tandyleatherfactory.com', None, None, None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'http://www.madameboeuf.com', 'https://www.fitzroyrentals.com', 'http://www.resumetarget.ca/location/ontario-on/toronto/', '1592240420', 'http://callnoworriesjanitorial.com', 'https://www.autozone.com/locations/in/avon/7474-e-us-36.html', None, 'https://www.zoup.com', 'http://www.thedistrictatgvr.com/alsGarage.php', 'http://www.kixs.ca', 'http://www.immigroup.com', 'http://www.burlingtoncoatfactory.com', None, None, None, 'http://www.swensensaz.com', None, 'http://www.arrowheadautocare.com', None, 'http://www.lucasautomotive.ca/', 'http://lululemon.com/toronto/queenstreet?cid=yelp', None, 'https://www.nothingbundtcakes.com/bakery/az/surprise?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=surprise_300', None, 'http://www.rnbflooring.com', 'http://www.fortpittblockhouse.com/', 'https://www.hilton.com/en/hotels/cltbahw-homewood-suites-charlotte-ballantyne-nc/?SEO_id=YELP-HW-CLTBAHW', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.cortebellagolfclub.com', 'http://www.popsbarber.com', None, '1592240482', None, 'http://www.greenbeauty.ca', 'http://24hrlockoutsmadison.com/', 'https://kevindeon777.wixsite.com/remodel', 'https://www.needforsweetslv.com', 'http://suzieqluxe.com', None, 'http://www.torontoislandsup.com', None, 'http://www.azhumane.org', 'http://www.providenceprepschool.com/', '1592240510', 'http://kierlandresort.com/fine-dining-at-scottsdale-restaurants/scotch-library/', 'http://www.vocellipizza.com/mccandless_pa', 'http://shawarmaroyale.com', 'http://bullseyervrepair.com', None, None, 'https://www.westelm.com/stores/ca/on/toronto-shops-at-king-liberty/', 'http://www.sterlescountryhouse.com', 'http://www.dawgdayzcare.com/', 'http://www.superiortire.ca/locations/mode/4/3-laidlaw-blvd-markham-on-l3p1w5.aspx', 'https://experigreen.com/service-area/charlotte-nc/', 'http://www.michaelscateringevents.com', 'http://www.footdocvegas.com', 'http://www.InfinityAcuAZ.com', 'http://www.eggsmart.ca', 'http://www.cakestoyou.net', None, None, None, 'http://www.thecorneralley.com', 'http://www.pinkpetalsflorist.com', 'http://www.thecroftdowntown.com/', 'http://www.dbacks.com/', 'https://www.smilefitnessdentalcenter.com', 'https://stores.savers.com/nv/lasvegas/savers-thrift-store-1047.html', 'https://www.appliancepartscompany.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTWS', None, 'http://www.MortgageAssociatesOntario.com', None, 'https://locations.rallys.com/oh/parma/6201-brookpark-road/?utm_source=Yelp', None, 'https://www.bestbuysoccer.com', 'http://www.aeropostale.com', 'http://www.alternativehealingconnection.com/', None, 'http://www.mastercleaningaz.com', 'http://www.thepoddrop.com', 'http://www.dozoaz.com/', 'http://luckysloungelv.com', 'http://www3.hilton.com/en/hotels/north-carolina/hilton-charlotte-center-city-CLTHHHF/index.html', 'http://bestofsevenbarbers.com/', 'http://www.lorispetagree.com', 'https://deltaco.com', 'http://firstwindfitness.com', 'https://lakehealth.org/Locations/TriPoint-Medical-Center', None, 'http://www.swisschalet.com', None, None, None, None, '1592240633', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.pricklypeachwaxbar.com', None, 'http://asappoollv.com', 'http://www.snipits.com/locations/avon/', 'http://www.timhortons.com/', 'http://www.43north.biz', 'http://www.mixxmethod.com', 'http://www.rizegaragedoor.com', None, 'http://www.papagostables.com', None, None, None, 'http://theswedishconnection.net', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66083.html?iata=00171890&ssob=BLBWI0003G&cid=BLBWI0003G:yext:website:66083', 'http://federicosmexicanfood.com', None, None, 'http://www.calvarychristiandaycare.com/', 'http://www.phxdj.com', None, 'https://www.ezwindowsolutions.com/cleveland/', 'http://www.mrappliance.com/pittsburgh', None, 'http://www.opsiseyecare.com', 'http://www.avanteoptics.com', None, 'https://www.chronicinktattoo.com/locations/tattoo-shop/chronic-ink-midtown-toronto-yonge-eglinton/', None, 'http://www.adaptiveforce.com', None, 'http://www.thecatnfiddlepub.com', 'http://www.valleyymca.org', 'http://www.ladybugpestcontrol.com/', None, None, '1592240712', None, 'http://www.greenfieldcitrus.com', 'http://www.prada.com', None, None, None, 'http://www.towncentredentalclinic.ca', 'http://www.hobbyhobby.com', None, 'https://southpointcasino.com/entertainment/grandview-lounge/?cid=ypgl', 'http://crystalclearreflectionsaz.com', 'http://danielacaltarumd.com/', None, 'http://www.cleantheworld.org/', 'https://www.famousfootwear.com/?partnerid=YELP', 'http://www.newhorizonlittlelearners.com', 'https://www.hairclub.com/locations/nv/las-vegas/hair-loss-clinic-292/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_292', 'http://galatimarketfresh.com', '1592240751', None, 'http://www.toysrus.ca', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/9302+Olde+Eight+Road-Northfield+Center-OH-44067/storeid=4301?cid=ps_ylp_2020', None, 'http://www.salonritualsaveda.com', 'http://www.jetfuelcoffee.com', None, 'http://www.mrhero.com', 'http://www.lululemon.com/thornhill/promenade?cid=yelp', 'http://www.destinationweddingphotographycalgary.com', 'http://www.golfthebridges.com/barandgrill/', '1592240777', 'http://funtropolis.ca', 'http://www.bilmarcompanies.com', None, 'https://www.aznaturalselections.com', 'http://locksmith4Uvegas.com', 'http://scotiaplaza.kidsandcompany.com', None, '1592240796', 'http://www.greatclips.com', 'http://gremassagetherapy.com', 'http://www.fortunenails.ca', 'https://locations.massageenvy.com/arizona/peoria/10006-w-happy-valley-parkway.html', 'https://www.extraspace.com/storage/facilities/us/north_carolina/charlotte/1000000404/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.kodisnaturalpetfoods.com', None, 'https://www.papamurphys.com/store-finder/position?utm_source=yelp&utm_medium=2018ads&utm_campaign=SC011', 'http://www.arizonahairco.com', None, None, 'http://www.desi.bar', None, 'http://mkt.com/mobile-bubbles', 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', None, 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', 'http://www.waborasushi.com/', 'http://salon7000.com/', None, None, 'http://www.whitecastle.com', None, None, None, '1592240855', None, None, '1592240862', None, None, 'http://www.averyonthegreen.com', None, 'https://www.southviewdentallv.com/index.php', None, None, 'http://www.firstcu.net', 'http://www.anytimegaragedoor.com/', 'https://sardellaspizza.com', None, 'http://www.panevinorestaurant.net/about.html', 'http://www.newhealthdentalaz.com/', None, 'http://www.hatleystore.com', 'http://xtremeairaz.com', 'https://www.phoenixconventioncenter.com/orpheum-theatre', None, 'http://www.veronadentalcare.com', 'http://www.canyonridge.org', 'http://mint-landscape-creations-llc.business.site', None, None, 'http://www.academyofsound.org', None, 'http://lowell.wisc.edu', None, None, 'http://www.chandlervisioncenter.com', None, 'http://www.sugarmarmalade.com', 'http://shoparooni.com', 'http://www.dieselbarbershop.com', 'http://www.allcitytransportation.com', None, 'http://premierdental702.com', 'http://www.dominos.com', 'http://www.lazylizardaz.com/', 'http://www.qqteaplus.com', 'http://snailmotorsports.com/', 'https://www.autozone.com/locations/az/phoenix/8325-w-camelback-rd.html', 'http://www.lacasalatina.net', 'http://www.itzzapizza.com', 'https://www.aritzia.com/en/store-locator?q=M9C+1B8&view=results', 'https://www.plannedparenthood.org/health-center/wisconsin/madison/53713/madison-south-health-center-3937-91860?utm_campaign=madison-south-health-center&utm_medium=referral&utm_source=yelp.com&utm_content=link', None, None, 'http://www.azkarateacademy.com', 'http://olivebranch412.com', None, None, 'http://susiesestateaz.com/estate_sales', None, None, 'http://www.thaiexpress.ca', 'https://www.advantagestorage.net/self-storage/az/phoenix/advantage-storage-moon-valley/', 'https://florist.1800flowers.com/us-nv/1-800-flowers-shoppe-las-vegas', 'http://www.weavebaratl.com', 'http://www.migrationlaw.com', 'http://www.iceburghs.com', 'http://www.drgreencerts.com', 'http://modernnailsandlash.com', 'https://recreationalartstudio.com', None, 'http://design-ramon.com', 'http://www.sallybeauty.com', 'http://www.just4funsportingclub.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=1378&utm_content=bpas', 'http://www.lush.ca', None, 'http://www.madsens.ca/', 'http://www.sportracks.com', None, 'https://www.fatburger.com/locations/rancho', 'http://www.azsuperiorhomeinspections.com', 'http://www.earnhardtcadillac.com/', None, '1592241033', 'http://barneyslounge.ga', 'http://brianwilliamsdds.com/', 'http://www.simplyflowerstoronto.com', None, 'http://www.arizonaautowashscottsdale.com', 'http://www.dunkindonuts.com', 'http://www.PattersonHomesAZ.com', None, 'http://www.outback.com/locations/oh/independence', 'https://pablocanada.com', None, 'http://www.mychiropractoraz.com', 'http://www.hollospapercraft.net/', 'https://www.searsauto.com/stores/6601?psid=campaigncode-6601', 'http://www.sayyestoherdress.com', None, 'http://www.eyestudio.ca', 'http://www.highlandfarms.ca', 'https://tendollarshirtstore.com', 'http://www.cineplex.com/Theatre/cineplex-odeon-eglinton-town-centre-cinemas', 'http://www.myhusky.ca/station_locator.html?provinceID=AB', 'http://peaksandrafters.com', 'http://www.junkmastersinc.com', None, 'https://www.filerchiropracticpittsburgh.com', 'http://www.westsidemarket.org/vendor.aspx?id=77', 'http://www.ttc.ca/Subway/Stations/St_Patrick/station.jsp', 'http://www.gzcomics.com/locale.html', 'http://www.deltaco.com', 'https://jade-chinese-kitchen.business.site', None, 'http://www.bojangles.com', 'http://Clocktowerbar.ca', 'http://valleyislepumping.com/', 'http://www.quiznos.ca', None, None, None, 'http://www.bigotires.com', None, 'http://stpaulpgh.org', 'http://coopersbbqco.com/', 'http://www.ecokingaz.com', None, None, 'http://www.powervac.ca/power-vac-toronto-gta', None, 'http://www.myphoenixair.com', 'https://www.gianteagle.com/Stores/Store/?storeId=6299', 'http://www.Team1stRealEstate.com', 'http://www.fickleonline.com', 'https://vcahospitals.com/bakerstown?utm_source=yelp.com&utm_medium=referral&utm_term=VCA_Bakerstown_Animal_Hospital=main_url', 'http://www.jglennhome.com', '1592241164', '1592241167', 'https://www.doggiedistrictaz.com?utm_source=yelp&utm_medium=social', None, None, 'http://changjiangbestmadison.com', 'https://www.theacteam.com', None, 'http://www.eatoooweebbq.com', 'http://fourcornersbakery.com', None, 'http://www.paintstoretoronto.com', 'https://stores.toysrus.com/az/goodyear/baby-store-goodyear-az-b5641.html', 'https://www.fiix.io', 'http://www.visualharmonyphotography.com/', 'https://landmarkautoanddiesel.com', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=830', 'https://finedetails.ca', 'http://www.tbdc.com', 'https://www.liveedgelust.com', 'https://www.kinjosushiandgrill.com', 'http://GrizzlyAC.com', 'http://www.opticausa.com', None, 'https://www.simonmed.com/locations/all/simonmed_imaging___sun_city_peoria', 'http://www.advancedhearinggroup.com', 'https://sacredpi.com', None, 'https://www.transformyourself.ca', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282967270&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'https://arirangkoreanbbq.business.site', 'http://www.marqueetheatreaz.com', None, 'http://peoriaartisanbrewing.com/tap-room/', 'http://www.holeechow.com', 'https://www.abcgaragedoor.biz', 'http://webersvintageicecream.com', None, 'http://www.leewiyoga.com', None, 'http://www.lucapizza.ca/', None, 'https://www.wangskitchenpa.com', 'http://northofbombay.com', 'http://www.eandrsweetery.com', 'http://indodesignerrugs.com', 'http://www.landmarkhomeserv.com', None, None, 'http://www.saharaautolv.com', 'http://www.drlizcruz.com', 'https://www.wingstop.com/location/wingstop-230-glendale-az-85306/menu', 'http://www.northcoastjawcenter.com', None, '1592241292', None, 'https://www.loblaws.ca/store-locator/Loblaws+-+St.+Clair+Ave+West/1212?storeId=1212', None, 'http://glassking.com', None, 'http://www.glamoursalonandspa.ca', None, 'http://www.CapstoneBrokerage.com', 'http://www.myrivertowne.com', 'http://www.hotrodpiercingcompany.com/index.html', 'http://www.tuckyshotdogs.com', None, 'http://www.getdynamicdetailing.com', None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/20/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://local.tiresplus.com/store/Madison-East?lw_cmp=YLP_TP_53714&utm_source=YLP&utm_medium=TP_53714', None, 'http://www1.shoppersdrugmart.ca/en/store-locator/store/359', None, 'http://www.theyardpgh.com/market-square', None, None, None, 'http://www.nissanairport.com', 'http://pilateselementlv.com', 'http://bodega13.com/', 'http://freedomclothingcollective.com', 'http://trackofficefurniture.com/', None, 'https://locations.visionworks.com/ll/US/PA/Pittsburgh/120-Fifth-Ave-Pl', 'http://www.healthree.ca', None, None, '1592241373', 'http://www.glamourco.com', 'http://www.goodlifefitness.com/locations/ontario/toronto/kingliberty?source=yelp-websitelink-torontoking-gl', 'http://www.rei.com/stores/paradise-valley.html', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.cornerstone-familypractice.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, None, 'http://www.fitkitchen.ca', 'https://www.mastercuts.com/locations/nv/henderson/mastercuts-galleria-at-sunset-haircuts-3331.html', 'https://locations.wendys.com/united-states/pa/mcmurray/3500-washington-rd', 'http://www.mccormickandschmicks.com/Locations/pittsburgh-pennsylvania/pittsburgh-pennsylvania/FifthAve.aspx', 'https://www.strongsville.org/departments/parks-recreation', None, 'http://www.ppcp.org', None, '1592241411', None, 'http://www.allcomfortservices.com/', 'http://www.eliteautoaz.com', 'http://www.stacysplacescottsdale.com', 'http://ductmanofnevada.com', 'http://www.compolastastypizza.com', None, 'http://www.brighton.com', 'http://www.maclinford.com', None, 'http://www.regalcarpetcleaners.com', None, 'https://mrjims.pizza/149.cfm', 'http://www.amberoaksrestaurant.com/', None, None, 'http://www.birkdaledental.com', 'https://www.fastmed.com/urgent-care-centers/chandler-az-walk-in-clinic-west-ray-road/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', None, 'http://www.levelsfitness.net', 'http://www.borealis-salon.com', 'https://www.nickandmikebarandgrill.com', 'http://uniongrilltap.com', 'http://lululemon.com', None, 'http://www.pacificfitnesslv.com/', 'http://hairbyhall.schedulista.com', 'http://www.kierlandresort.com/dining/nellie-cashmans/', 'http://denierlaundromat.findalaundry.org', None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpnortheastorganic', 'http://www.myvalleyagent.com', 'http://www.littleotterswim.com', None, None, 'http://www.peoriakayakrental.com', 'https://www.enterprise.com/en/car-rental/locations/us/nc/charlotte-south-woodlawn-53c4.html?mcid=iyp:8368400', 'http://www.kingwestvets.com', None, 'http://www.pointerscrapbooking.com/', 'http://www.cocktailemporium.com', None, 'http://www.nyahsmobilegrooming.com', None, 'http://www.gaetanosmayfield.com/', 'http://www.pool-mart.ca', None, 'http://tullys2.com/', 'https://www.signatureplasticsurgery.net', 'http://www.spinink.ca', 'http://www.wildhorsepassresort.com/kosin', 'http://sol-nails-spa.business.site', 'https://pandalibre.com', None, 'http://www.awaxmanlaw.ca/', None, 'http://www.nellisasc.com', 'https://shop.nordstrom.com/st/last-chance-clearance-store?cm_mmc=yelp-_-ycm-_-rack-_-780', None, None, 'https://www.canadianair.info', 'http://toothfairychildrensdental.com', None, None, None, 'http://www.handandstone.ca/locations/bloor-west', None, None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://www.europeanlv.com', 'http://www.priorityhealthgroup.vegas', 'http://www.freewayinsurance.com', 'https://locations.wendys.com/united-states/oh/stow/1051-w.-graham-street', 'http://www.foodlion.com', 'http://www.joedufek.com?cmpid=CKMF_BLM_0006', 'https://www.orleanscasino.com/dine/casual-dining/medley-buffet?disid=wmm_yelp_to_corporate_medley_organiclink', 'https://phoenixtrimatree.com', 'http://www.greenmangopest.com', 'https://locations.sonicdrivein.com/az/glendale/13751-west-glendale-avenue.html', 'https://www.kindercare.com/our-centers/mesa/az/301365?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=301365', None, 'http://www.sweetmangoohio.com', 'http://www.kfc.com', None, 'http://www.altfitpilates.com', 'http://www.nakedmagicians.com', None, 'http://prestocalzonebakery.com', 'http://www.starnail.com/', '1592241614', 'http://serenityhospiceaz.com/home/1792243', 'http://www.vans.com', 'http://familysqueezedlemonade.com/', 'http://codyscavecreek.com/', None, 'http://www.dwcpinc.com', 'http://www.glendaleazsda.org/', 'http://www.timhortons.com', 'http://www.bankofamerica.com', None, None, 'http://spa4pawsaz.com', 'http://www.turnhousegrille.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=2486&localechange=1', 'http://encompasshealth.com/desertcanyonrehab', 'http://www.doc4women.com', 'https://www.bloomingdaysfloral.com', 'http://www.maddissouthernbistro.com/', None, 'http://www.cabcpr.com', 'http://www.upcinematics.com', 'http://drjazzdetailing.com', 'http://www.thedogstop.net/', 'http://www.azbone.com', 'http://www.cleatswings.com', 'http://loparos.localmobil.com/', 'http://www.scottsdale-dentistry.com/index.php', 'http://treasureisland.com/restaurant/17/the-coffee-shop', None, 'http://www.adelaidehallto.com', None, 'http://www.vantagehotels.com/hotel-details.cfm?idp=30026', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, None, 'http://www.einsteinbros.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.fineartspermanentcosmetics.com', None, 'http://www.zyng.com', 'https://www.downeastbasics.com', None, '1592241718', 'http://www.enlightanspraytan.com', None, None, None, 'http://paparocks.com/', 'https://www.wyndhamhotels.com/hotel/13899?tel=18002521709&iata=00093785&cid=fe:ra:20161215:yl:pp:ramus:13899', 'http://qnailscottsdale.com/', 'http://www.economytowinglv.com', None, None, None, 'http://www.centrestauto.ca', None, 'https://www.NEXAMortgage.com', 'http://www.presstalent.com', 'http://www.k9couture.ca', None, 'https://www.sleepcountry.ca', None, 'http://joynailsspa.business.site', None, 'http://www.mipueblo.us', '1592241770', 'http://www.cafe-mirage.com', 'http://www.sammyspizza.com/', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://undergroundbutcher.com', 'https://stores.chicos.com/s/chico-s-the-district-at-green-valley-ranch', None, 'http://www.diamondloungeaz.com/', 'http://www.marcs.com/stores/marc-s-willowick-grocery-store-44095.aspx', 'http://www.boosterjuice.com/', 'http://www.smoothieking.com', 'http://www.mensclothingaz.com', None, 'http://thebodyshopgilbert.com', 'http://www.mark-taylor.com/arizona/san-hacienda/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.amfamilyspine.com', None, 'http://www.clarkcountynv.gov/vote', None, 'https://www.gianteagle.com/Stores/Store/?storeId=440', 'http://www.ocarizona.com', 'http://www.citynationalarenavegas.com', 'http://www.accesshealthdental.com', None, None, None, 'http://www.amscharters.org/peoria', 'http://www.oldchicago.com/mesa-superstition-springs', None, None, 'https://thrifty.com/loc/ll/US/AZ/PHOENIX-AP/1805-E-SKY-HARBOR-CIRCLE-SOUTH', 'http://www.celebritypatios.com/', 'https://www.walmart.com/store/1464/charlotte-nc/whats-new', 'http://www.arizonafireplaces.com', None, 'http://www.calvarycenter.com', 'http://www.bigabaldis.com/', 'http://www.bigplusfashions.com/', 'http://www.shampoochdogspaw.com', None, 'http://lowestpricedepot.com', None, None, 'https://www.shortrunprintingltd.com/Quote/index.php', 'http://www.palazzo.com/', 'http://superiorplumber.ca', 'http://www.chambersobgynlv.com', 'http://stanleysteemer.com', 'https://www.laytondentistrync.com', 'https://www.sonoraquest.com/find-a-location/scottsdale-scottsdale-rd-mescal/', 'https://buffetmandarin.weebly.com', 'http://www.americandrain.com', 'http://www.realfruitbubbletea.com', 'https://www.PUREfectReflection.com', None, 'https://www.telus.com', 'http://millermassage.com', 'http://www.weststar.org/pages/about/b_sunset.html', None, 'http://shanghai1938.biz', 'http://www.qualitycleanohio.com', 'http://www.silverstatedental.com', '1592241931', 'https://www.eddiev.com/locations/nc/charlotte/charlotte/8521?cmpid=br:ev_ag:ie_ch:dry_ca:EVQ420LOCURL_dt:20200511_sn:yelp_gt:charlotte-nc-8521_pl:locurl_rd:1019', None, '1592241939', None, 'http://desertwindawards.com', None, None, 'http://mintlounge.ca', 'http://dsw.townshoes.ca', None, None, None, 'http://www.peterpiperpizza.com/', None, None, 'http://shawbuttebarbershop.com/', 'http://www.christopherspub.net', 'http://locations.arbys.com/us/wi/sun-prairie/829-bunny-trail.html', 'http://www.mysweetblessingsbakery.com', 'http://kaizokupokebowl.com', None, None, 'http://www.coveralllasvegas.com', 'https://mikeshaulingservices.com', None, 'http://www.5rchathai.com', 'http://www.pittsburghplayhouse.com', None, 'http://stapleygaragedoor.com', 'http://www.mbco.ca/', 'http://www.union.wisc.edu/', 'http://www.icedenscottsdale.com', 'http://www.sunchain.com', None, 'http://www.laylaspalace.com', '1592242020', None, 'http://www.ackermanhomeinspections.com/', 'http://www.mysisterscloset.com/', None, 'http://www.big5sportinggoods.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.indacutbarbershop.com/', None, 'http://www.monkeesofcharlotte.com', 'http://www.financeofamerica.com/jrencher', 'http://www.littlecaesars.com/Locations/tabid/92/directions/85240/Default.aspx?id=MTUxNTAwMDQ%3d', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'https://www.reginellapizzeria.com', 'http://www.eastsidemarios.com', 'http://www.pittsburghcyclecenter.com', None, None, 'http://www.fleetfeetcleveland.com', None, 'https://www.brightnow.com/dentist-near-me/chandler-dentist/40050?utm_campaign=local_profiles&utm_medium=profiles&utm_source=yelp', 'http://www.securitydatadestruction.com', 'http://www.innovativehealinglv.com', 'http://www.urbandog.ca', 'https://woodiesautoservice.com', 'https://www.scene75.com/cleveland/eat-drink/food-truck-alley/', 'http://massagemaniaglendale.com', 'http://www.bottleshop48.com', 'http://www.chrislopez.net?cmpid=H3LE_BLM_0006', 'http://www.dunkindonuts.com', 'http://a-zhandymanservicesllc.com', 'https://www.faucetdoctorplumbingaz.com', 'http://signaturenevada.com', None, 'http://www.msfstrong.com/', 'http://www.parkbluesky.com', 'http://www.TheSchielkeGroup.com', None, 'http://www.motel6.com/reservations/motel_detail.aspx?num=2012', 'https://www.simondentalcare.com', 'http://sandblastinglasvegas.com', 'http://www.thegreatsteak.com', 'http://www.linsasianfusion.com', 'http://www.rubinsfurniture.com', None, 'http://www.hairstory.ca', None, 'http://www.abcfest.com', 'http://www.mybenzparts.com/', None, 'http://www.MariachiColonial.com', None, 'http://www.marbleslab.ca', 'http://www.lushfloral.com', 'http://www.llcrv.com/', 'https://madisonsbarandgrill.netwaiter.com/monroeville', 'http://www.ogradysoncollege.com', '1592242202', 'http://www.4naturesfinest.com', 'http://marcellopointeclaire.com', None, 'http://www.barargon.com', None, None, 'https://bodybeautifullasermedi-spa.com/locations-near-me/laser-hair-removal-bethel-park/', 'http://www.footlocker.com', 'http://www.daverubinsonpools.com', 'https://www.decathlon.ca/fr/c/135-decathlon-brossard', 'http://spicnspanspotlesscleaning.webs.com', None, 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.rowefarms.ca', 'http://www.spectaclelovesyou.com', 'http://www.boasteak.com/balboa', 'http://hawgndawgexpress.com', None, None, 'http://sfsourdougheatery.com/', 'http://foosing.weebly.com', None, 'http://www.encorecleveland.com/', 'https://www.joysquad.com/locations/las-vegas-photo-booth-rental', None, None, 'http://chase.com', 'http://www.ginzacuisine.ca', '1592242278', None, 'http://www.peaksathleticclub.com', 'https://www.hilton.com/en/hotels/phxsaht-home2-suites-phoenix-airport-south/?SEO_id=YELP-HT-PHXSAHT', 'http://southpointcasino.com/dining/coronado-cafe?cid=ypcor', 'https://www.cash1loans.com/locations/arizona/phoenix-thomas.aspx', 'http://www.co-operateon.com', None, '1592242301', None, None, 'http://www.mossgardenhome.com', 'http://www.amazingyoga.net/', 'https://br.coffee']


In [9]:
saved = urls
print(saved)

['http://www.therangeatlakenorman.com/', None, 'http://www.felinus.ca', None, 'https://www.usemyguyservices.com', 'http://oasisautocenter.net', None, 'http://junctiontire.net/tires-auto-repair-mesa-az', 'http://www.theempanadashouse.com', 'http://www.xtremecouture.ca', 'https://landscapingcalgary.org', None, 'http://www.chocolateshoppeicecream.com', 'http://www.convertabath.com/', None, 'http://www.desertstorage.com/henderson-nv-self-storage', 'https://www.manoloblahnik.com/in/boutiques/Wynn%20Las%20Vegas', 'http://www.bloomandblueprint.com', 'http://www.2carguyz.com', 'https://fitbodyaz.clickfunnels.com/sales-page-yelp', None, 'https://www.glendaleaz.com/glitters/', 'http://www.ncbrewsmusic.com/', 'http://www.tipsyryde.com', 'http://www.philthyphillys.com', None, None, 'http://www.hendrickmotorsofcharlotte.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, None, 'https://www.apadana.restaurant', 'https://www.iga.net/fr/recherche_de_magasin/magasin

In [10]:
urls[-1], len(urls)

('https://br.coffee', 23556)

In [22]:
delays = [.01,.02,.03,.04,.05,.06,.07]
for i, entry in df.iterrows():
    if i < len(urls): continue
    #time.sleep(np.random.choice(delays))
    #website, yelp_url, blocked = fetch_url_from_yelp_stop_if_captcha(entry)
    website, blocked = fetch_url_from_yelp_stop_if_captcha(entry)
    if blocked == 1:
        print('We are blocked at {}'.format(i))
        break
    #print(i, 'Website not found! Check out: ' + yelp_url if website is None else 'Found: ' + website)
    urls.append(website)

TooManyRedirects: Exceeded 30 redirects.

In [123]:
print(urls)

['http://www.therangeatlakenorman.com/', None, 'http://www.felinus.ca', None, 'https://www.usemyguyservices.com', 'http://oasisautocenter.net', None, 'http://junctiontire.net/tires-auto-repair-mesa-az', 'http://www.theempanadashouse.com', 'http://www.xtremecouture.ca', 'https://landscapingcalgary.org', None, 'http://www.chocolateshoppeicecream.com', 'http://www.convertabath.com/', None, 'http://www.desertstorage.com/henderson-nv-self-storage', 'https://www.manoloblahnik.com/in/boutiques/Wynn%20Las%20Vegas', 'http://www.bloomandblueprint.com', 'http://www.2carguyz.com', 'https://fitbodyaz.clickfunnels.com/sales-page-yelp', None, 'https://www.glendaleaz.com/glitters/', 'http://www.ncbrewsmusic.com/', 'http://www.tipsyryde.com', 'http://www.philthyphillys.com', None, None, 'http://www.hendrickmotorsofcharlotte.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, None, 'https://www.apadana.restaurant', 'https://www.iga.net/fr/recherche_de_magasin/magasin

In [124]:
urls = ['http://www.therangeatlakenorman.com/', None, 'http://www.felinus.ca', None, 'https://www.usemyguyservices.com', 'http://oasisautocenter.net', None, 'http://junctiontire.net/tires-auto-repair-mesa-az', 'http://www.theempanadashouse.com', 'http://www.xtremecouture.ca', 'https://landscapingcalgary.org', None, 'http://www.chocolateshoppeicecream.com', 'http://www.convertabath.com/', None, 'http://www.desertstorage.com/henderson-nv-self-storage', 'https://www.manoloblahnik.com/in/boutiques/Wynn%20Las%20Vegas', 'http://www.bloomandblueprint.com', 'http://www.2carguyz.com', 'https://fitbodyaz.clickfunnels.com/sales-page-yelp', None, 'https://www.glendaleaz.com/glitters/', 'http://www.ncbrewsmusic.com/', 'http://www.tipsyryde.com', 'http://www.philthyphillys.com', None, None, 'http://www.hendrickmotorsofcharlotte.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, None, 'https://www.apadana.restaurant', 'https://www.iga.net/fr/recherche_de_magasin/magasin/0293-marche-d-alimentation-beck-inc', None, 'http://www.wetzels.com', 'http://www.beechmontcc.com', 'http://www.gkvapors.com', '1592058805', 'https://www.bannerhealth.com/locations/phoenix/banner-university-medical-center-family-medicine', 'http://www.thegourmetburgerco.com', 'https://www.goodyearautoservice.com/en-US/shop/PA/Pittsburgh/000901034-15206-3149', 'http://stores.disneystore.com/ca/concord/317', None, 'http://meatchix.com', None, None, None, None, 'http://www.basspro.com/', 'https://ccv.church/locations/anthem', 'https://locations.wendys.com/united-states/oh/cleveland/13939-euclid-ave.', None, None, 'http://www.bathandbodyworks.com', 'http://irenestaproom.com', 'http://hingetown.com/', 'http://www.bestitchedneedlepointshop.com', 'https://www.esportafitness.com/Pages/clubhome.aspx?clubid=1', None, 'https://www.smlentertainment.com/calgary-north/', None, None, 'http://fiveguys.com/', 'http://platinumkustomz.com/', None, 'http://locations.tccrocks.com/az/gilbert/2218-e-williams-field-rd.html', 'https://local.fedex.com/nc/charlotte/office-0746/', 'http://www.lacoste.com/us/', 'http://www.helenblakeyflowers.com/', None, 'http://localpetpgh.com', 'http://www.ocotillointernalmedicine.com', 'http://www.mpbattingcages.com', 'https://www.garageexperts.com/location/charlotte', None, 'http://redlinepromotions.com', None, 'http://stqitoronto.com', 'http://saxetheater.com/', 'http://www.sincitypartybuses.com', 'http://www.cjaycomputers.com/', None, 'http://www.visitnc.com/listings/view/52079', None, '1592059164', 'http://www.minthillbarbershop.com/', 'http://actionfurnace.ca', 'http://www.mamanapolipizza.com', 'http://www.monarichiebride.com/', 'http://www.movers4less-il.com/', '1592059211', None, 'http://darrochmotorsports.com', 'http://www.jeffersonchandler.com/?utm_medium=listing&utm_source=yelp', 'http://landing.trugreen.com/yelp?city=Champaign&state=IL&zip=61822&utm_campaign=ORGANIC&marketingpartner=9817', None, 'http://fresh.longos.com', None, None, None, 'http://www.rehabdirectives.com', None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.tavernrestaurantgroup.com', None, None, 'https://www.pizzaville.ca', 'http://www.royalskincare.net', '1592059367', 'http://www.sugarbowlscottsdale.com/', 'http://www.vikingcleanersaz.com', 'http://www.timhortons.com/', None, None, 'http://www.shalecreekgolfclubggp.com/', None, '1592059436', 'http://store-locator.barnesandnoble.com/store/2147', 'https://www.myairtoday.com', None, 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-indian-school?utm_campaign=honorhealth-medical-group-indian-school&utm_medium=local&utm_source=yelp', 'http://www.marcos.com', 'http://www.sleepandwellnessctrs.com/index.php/modules-menu/2012-01-19-13-38-38/henderson', 'http://www.hotelcharlotterestaurant.com/hotel/', 'http://www.handandstone.com', 'https://www.ulta.com/stores/charlotte-nc-590', 'http://www.JesusChurch.life', 'http://www.jjspizzacle.com/', 'http://www.dennys.com', 'https://rinsenride.com', None, 'https://kulinaryaexpress.com', 'http://www.ahwatukeepeds.com/', 'http://www.timhortons.com/ca/en/locations/get-directions.php?id=100934', None, 'http://www.amegashootingsupplies.com/', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, None, 'http://www.freshcoatpainters.com/charlotte', 'http://clk.atdmt.com/HBO/go/165733594/direct/01/', None, 'http://www.hottubliquidators.com', 'https://sharonmtuttle.com/', 'http://www.fleurishflowershop.ca', 'http://www.healthspringlv.net', 'http://www.torontodermatologycentre.com', 'http://chandlersymphony.com', None, None, 'http://www.zavcoffeeshop.com', None, 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=0352&s_cid=lsl0000001', 'http://www.frederickandsons.com', 'http://www.winerack.com', 'http://www.chefalyssaskitchen.com', None, 'https://www.fatmoespizzawings.com', 'http://www.shearsensationsonline.com', 'http://www.movingforwardmadison.com/', None, 'http://www.kegsteakhouse.com/en/locations/ON/toronto/esplanade-keg/', 'https://www.thethingshop.com', None, None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89128/027192/', None, None, 'https://www.target.com/sl/richland-twp-north/2262', 'http://www.fearlessmeat1.com', None, None, 'http://www.qobollabridal.com', None, 'http://www.lcbo.com', 'https://www.dayspascottsdaleaz.com', 'http://leecafesc.com', 'http://www.gofitnessnation.com', None, None, 'https://www.peoriaaz.gov/government/departments/parks-recreation-and-community-facilities/parks-and-trails/pioneer-community-park', 'http://www.nwpoolaz.com', 'http://www.cruffs.com/', 'http://dwellingsinthedesert.com', 'https://agents.allstate.com/oscar-arnold-goodyear-az.html', 'https://www.walgreens.com/locator/walgreens-1725+northport+dr-madison-wi-53704/id=2967', None, 'http://www.purelifeaz.com', None, None, None, 'http://stlouiswings.com', None, 'http://www.massageindavidson.com', 'http://www.aumspatoronto.com', 'http://www.pizzafresh.com', None, 'https://www.raynathans.com', 'http://www.docrequest.com', 'http://www.drsimms.com', None, 'http://www.pitcrewmovin.com', 'http://www.planetfitness.com/gyms/las-vegas-flamingo-nv-611?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'http://www.jealousyonline.com', 'http://www.communalmule.com', None, 'http://www.thebeautyboutiquewhitby.com', 'http://www.torontopolice.on.ca/museum/', 'http://www.goldstandardhomecare.com', None, 'http://www.103yoga.com', 'https://membercuts.business.site', 'https://dutchbros.com/?utm_source=goodyear&utm_medium=yelp&utm_campaign=website_goodyear', 'http://phxstreetfood.org/', None, None, 'http://www.criminalcustomz.com', None, 'http://standandstair.com', None, 'https://www.mazzeolimousine.com', 'http://dentsquad.ca', 'http://www.menchies.com', 'https://www.smalv.com/OurCenters/HealthCareCenters/Siena', 'http://mystonecreekdentist.com', 'http://www.chobuffet.com', 'http://www.pho.com/mesa-az/pho-hien', 'http://www.mrakovic.com', 'http://www.childtime.com/your-local-school/las-vegas-nv-1803/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', None, None, 'https://www.sonoraquest.com/find-a-location/scottsdale-scottsdale-rd-mescal/', None, None, None, None, 'https://www.meineke.com/locations/az/surprise-4338/?utm_source=yelp&utm_medium=xvc', 'https://losolivosristorante.business.site', 'http://sunfoodsupermarket.ca/', 'http://www.2uglassandtint.com', 'http://edwardsbeautycle.com', 'https://www.allmysons.com/cleveland', None, None, 'http://www.thetailwaggersclub.com/', 'https://veggiehouse702.com', '1592060496', 'https://www.mr-bills.com', None, 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://olympiasweettreats.com', 'https://locations.churchs.com/az-phoenix-7235', 'http://MasterMechanicalCleveland.com', 'https://psychbc.com/locations/greater-cleveland/north-olmsted-oh', None, 'http://sevabeauty.com/locations/charlotte_nc', None, 'http://www.uncleotis.com/', None, 'https://www.scottsdaleflighttraining.com', 'http://www.charlotteorthotics.com', None, 'http://www.motherhood.com/shipping.html', '1592060654', 'http://www.shintkd.com', None, 'http://www.thebookshopaz.com/', None, 'http://wandaroche.starkhomes.com', 'http://www.lvchinatownplaza.com/event_index.htm', 'http://www.solissouthline.com', 'http://www.hairrevivalaz.com', 'http://www.pivotimage.com', 'http://kingsfamily.com/?utm_medium=referral&utm_source=yelp', None, None, 'https://www.repairngo.com', 'http://www.fountainhillsrv.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.clubpilates.com/dtphoenix/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'https://wearernd.com', 'http://www.lcbo.com', 'http://www.skinconceptsspa.com', 'https://locations.tropicalsmoothiecafe.com/nv/henderson/10612-south-eastern-avenue', None, 'http://lvmpd.com/', None, 'http://www.pueblomedicalimaging.com', 'http://www.creditunion1.org', None, 'http://www.pjspet.com', 'http://www.menyhartplumbing.com', 'http://www.cuisinemalaya.com', None, 'http://www.truvapenv.com', 'http://www.shopmadmod.com', None, 'http://www.southwestmobility.com/', 'http://www.wendys.com', 'http://www.starbucks.com/store/9298/', '1592061020', 'http://www.liquorstorevegas.com', 'https://www.hottopic.com', 'http://rubberstampevents.com', None, 'http://www.jcsgrillhouse.ca/mississauga/', 'http://www.guyslock.com', 'https://qcontrary.com', None, 'http://www.thecatsmeow.com', 'https://www.thejoint.com/arizona/glendale/arrowhead-ranch-48016', 'http://www.fruitsnroots.com', 'https://www.maids.com/49/?utm_source=Yelp', 'http://www.td.com', 'https://www.usps.com/', None, 'http://www.airrite-supply.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, 'https://www.subarusuperstorewest.com', 'http://allinonerepairs.us/index.php/home/', 'http://www.joesautoaz.com/', 'http://edulisrestaurant.com/', 'http://www.sdil.net', 'http://www.churchs.com', 'http://www.fatbastardburrito.ca', 'http://www.dollarama.com/', 'http://www.blackod.ca', 'http://www.rupertcornelius.net', 'http://www.alliancelv.com/', 'https://harrymarshlaw.com', 'https://nergysolutions.com', 'http://www.elitefloralevents.com', 'http://www.arizonaexecutivellc.com/', None, 'http://www.dunkindonuts.com', None, None, None, 'https://www.azairtime.com/scottsdale', 'https://www.jasonsdeli.com/restaurants/henderson-deli', None, None, None, 'http://www.izumi-sushi.com', 'http://www.logoexecutives.com', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', None, 'http://climbrefuge.com/', None, 'http://asiancafe.weebly.com/', None, 'https://www.brakesplus.com', 'https://www.churrasqueiramartins.com', None, 'http://www.vernspizza.com', 'http://tatmanstowing.com', 'https://www.mbkseniorliving.com/senior-living/az/scottsdale/mcdowell-village/?utm_source=mcdowell_village&utm_campaign=yelp', 'http://www.dhtorontoairport.com/', None, 'http://www.exxon.com', 'https://www.peacockrestaurants.com', 'http://www.thehousecoffee.ca/', None, 'http://www.calgarycurlingclub.com', 'https://clevelandhabitat.org/restore/', None, 'http://www.revelsalonmadison.com', 'http://www.misterchangs.com', 'http://www.smilesforeverpgh.com', 'http://www.charlotterusse.com', 'http://www.bistrogrande.com', None, 'https://ottogelato.ca', None, 'http://twizted-manez-beauty-barber.business.site', None, 'http://uncletetsu-ca.com', 'http://www.bitesandbrewspittsburgh.com/', 'https://www.walmaneye.com', 'http://speparty.com', 'http://www.propertyclaimadjusters.com/', 'http://www.zandrus.com', 'http://www.mercedesbenzofchandler.com', 'http://www.mycasinoquest.com', 'http://concordhairbeautysalon.com', None, None, 'http://ascentdronevideo.com/', None, 'http://www.cineplex.com/theatres/theatredetails/silvercity-crossiron-mills-cinemas-xscape-entertainment-centre.aspx', None, None, 'http://www.northtaphouse.com', 'http://www.mydcdental.com', 'http://www.kadensnails.com', 'http://soulgastrolounge.com/', 'https://greenvalleyranch.sclv.com/Entertainment/Lounges-and-Bars?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://katespade.com', None, None, None, 'https://www.toronto.ca/data/parks/prd/facilities/complex/1748/index.html', 'https://newresultsmedicalweightloss.com/near-you-scottsdale-arizona/', 'http://saigonnailspa.ca', None, 'http://www.bellevuebeer.com/', None, None, 'http://trufusion.com/', 'https://www.heartwoodroastery.com', 'http://www.poshhairandspa.com', 'http://www.yankeecandle.com', '1592062044', 'http://www.footcare4you.com/', '1592062563', 'http://www.livewellpgh.com', '1592062577', 'http://www.bloombc.com/', 'https://all-season-nails-spa.business.site', 'http://acninc.com', None, 'http://www.rexall.ca/', None, None, 'http://www.brookfieldvet.ca', 'http://theflowerroom.com/', 'http://sichuanren.ca', '1592062673', 'http://www.lodidoor.com', 'https://www.zerodegreescompany.com', 'http://thaimassagepro.vegas', 'http://www.pizzanova.com/locations/markham.html', 'http://www.yelp.com/events/chandler-rockin-taco-street-fest-2', None, 'https://www.purehockey.com/store.aspx?utm_content=Pure+Hockey-Monona-WI&loc_id=85', '1592062748', 'http://www.hickoryridgefirearms.com/page8.php', 'http://www.fantasticsams.com', 'http://www.marcs.com/stores/marc-s-medina-grocery-store-44256.aspx', 'http://www.yd.com/torontoyonge-eglinton/default.aspx', 'http://www.31kitchen.com/InfoEng.php', 'http://www.wamplerapartments.com', 'http://www.enterprisecarsales.com/location/53R1/Enterprise_Car_Sales_Charlotte?mcid=iyp:277&CUR=yelp', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PHXWH', 'http://www.starbucks.com', 'http://www.chewchew.net', 'http://jmacnv.com', None, 'http://www.gibs.bar', 'http://www.garciasmexicanrestaurants.net/', 'http://www.bobosbarbeque.com', 'http://www.chandlerinnaz.us', None, 'http://www.crescentoralsurgery.com/', 'http://www.fivestarmovingvegas.com', None, 'http://www.she-lv.com/', '1592062917', 'http://www.platinumpaintandbody.com', 'http://www.phxfoot.com', None, 'http://www.az-insulation.com', None, 'http://www.ilovemilanos.com/', 'http://www.duckislandclub.com/', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', None, None, 'http://skippa.ca', None, '1592063032', None, None, 'http://www.spanishoaksapts.com', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://www.easydrinkeasygo.com', 'http://www.wayneacebusandlimo.com', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/lasbr-renaissance-las-vegas-hotel/', None, None, 'http://karbking.com', None, None, 'http://cooling-az.com', 'https://anniys-massage-spa.business.site', 'http://white-oaks.com/cmswo/', 'http://www.kirstyblattner.com', None, None, None, 'http://www.gap.com?tid=gpme000290', 'http://www.cvrr.com', 'http://www.leelaecospa.com', 'http://www.leakdetectionarizona.com', None, 'http://www.dragonflyhotyoga.com/locations/middleton-west-madison-wi/', 'http://stores.partycity.com/az/goodyear/party-store-pc947.html', 'http://www.1800degrees.ca', 'http://locationsca.michaels.com/qc/saint-laurent/4967/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.chopinrestaurant.com', 'http://www.homedepot.ca', None, None, 'http://www.clemetparks.com/education/facilities/index.asp', 'http://www.bombaybuffet.ca', '1592063291', 'http://www.portagepointe.com', 'http://www.lakeeffectlaundromat.com', None, None, 'https://www.loandepot.com/msander', 'http://xmsengineering.com/', 'http://www.puntogelatotoronto.com', None, 'http://www.rudinospizza.com', 'http://www.mimiscafe.com', 'http://larkandkey.com/', 'https://www.chocolatsfavoris.com', 'http://www.myrotiplace.com', 'http://www.sdmi-lv.com', 'http://www.allcarewellness.com', 'http://www.firstclassvanlines.ca/', 'https://www.honorhealth.com/physicians/peter-ferrara', 'http://www.canyonridgephotography.com', 'http://www.trianglebenefitservices.com', None, None, 'http://www.abrazadental.com?utm_campaign=local&utm_medium=citations&utm_source=yext', 'http://www.lampsplus.com/?sourceid=SMYELPTXLKStore16&cm_mmc=YELP-SM-_-Text-_-Link-_-Store16', 'http://www.wtaetv.com', None, 'http://www.choicechiropractic.net', '1592063514', None, 'http://www.keytohealingmassage.com/', None, None, 'http://www.duquesne.org', 'http://gcglassllc.com', 'https://www.myedgecharlotte.com', 'https://www.pinnaclebankaz.com', 'http://www.johnjayautomotive.com/', 'http://www.barneys.com', 'http://www.torontocoffeeandteahouse.com/', None, None, 'http://timessquareaz.com/', 'http://www.TriPlumb.com', 'http://www.jchobbies.com', 'https://www.adencamera.com', 'http://izakayaju.com', 'http://www.jonesysclassiccleaners.com/', 'http://www.anchorbarcanada.com/square_one/', None, 'http://www.cloudcareclinics.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://bikesauce.org', None, 'http://www.sears.com/stores/arizona/mesa/0001628.html', None, None, None, None, None, 'http://singletrackfactory.com', None, 'http://www.residentialREclosings.com', 'http://www.sharifian.com', None, None, '1592063785', 'http://heritageeyecare.ca/', 'http://chazandodette.com', None, 'http://www.usa-autotransport.com/Phoenix', None, 'http://www.celebritytanning.com', 'http://www.blissdentallv.com', None, None, 'http://reinholdresidential.com/shadysidecommons/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.littlecaesars.com/', None, None, 'http://www.pods.com/phoenix.aspx', 'https://combatzonepblv.com', 'http://www.arizonawalkshop.com/', 'http://drakegeneralstore.ca', None, 'http://www.redhotrobot.net', None, 'http://miragenailsandspa.webs.com', 'https://www.mgmgrand.com/en/nightlife/whiskey-down.html', None, 'http://www.asianbuffet328.com/', 'http://www.nstuffmusic.com', 'http://www.groovyshoes.ca', 'http://www.gotoguyshandyman.com', 'http://www.meltingpot.com/charlotte-lake-norman/welcome', 'http://www.space55.org', None, 'http://fahhairextensions.com', 'http://www.visionsourcewilliamfharveyod.com', None, 'http://www.eggsmart.ca', 'https://www.lvatv.com', None, 'http://www.historybygeorge.com', '1592064139', 'http://korrys.com', 'http://www.cityofhenderson.com/parks/parks/paseo_vista.php', 'http://www.revitaantiagingcenter.com/testimonials', '1592064176', None, 'http://www.santosfairview.com', None, 'http://www.switchinkitchens.com', 'http://www.littlecaesars.com', None, 'http://www.ellekayegroup.com', 'http://7eleven.ca/store-locator/33259', None, None, 'http://www.kitchengourmetaz.com', 'https://www.batteriesplus.com/store-locator/oh/northolmsted/batteries-plus-927?utm_source=yelp&utm_medium=local&storecode=927', 'https://holisticlakewood.com', 'http://www.forestnailsandspa.com', 'http://www.massagehealthcare.com', 'http://tiredoctors.com', 'http://lonespurcafe.com', 'https://www.camdenliving.com/charlotte-nc-apartments/camden-south-end', None, None, '1592064386', 'http://www.UnexpectedVenue.com', None, '1592064415', 'https://holidayinnlongueuil.com/?utm_source=yelp&utm_medium=referral', 'http://azdistilling.com/', 'http://www.bamaspgh.com', 'http://rtcsnv.com', None, 'https://ad.doubleclick.net/ddm/clk/300117429;127049113;q', 'http://www.raphaelsbeautyschool.com', 'http://liznailsrockyriver.com', 'https://www.anytimefitness.com/gyms/866/Oregon-WI-53575/?utm_source=yelp&utm_medium=local', 'http://www.bridgespreschool.com', 'http://www.genoapizza.com', 'http://essostations.com', 'http://www.completeautomotivefairview.com', 'http://www.santanoralsurgery.com', None, 'http://www.thewinkinowl.com', 'http://polishedsalons.com', 'https://magicalstarpreschool.com', 'https://www.cellularsales.com/stores/verizon-4764-s-higley-rd-gilbert-az-85297/', 'http://www.abracleaners.com', 'http://www.liveatthestetson.com', 'https://www.hilton.com/en/hotels/clebhes-embassy-suites-cleveland-beachwood/?SEO_id=YELP-ES-CLEBHES', None, 'http://zombieburlesqueshow.com/', 'https://stores.hallmark.com/nv/henderson/amys-hallmark-shop-henderson-nv-65414.html', None, None, 'http://toronto.shambhala.org/', 'https://onekickgym.com', 'http://www.nctphoenix.com', 'http://www.akirabacktoronto.com', 'https://wolfgangpuck.com/dining/lupo-by-wolfgang-puck-las-vegas/', None, 'http://www.laserenaheights.com', 'http://www.tropicalbreezefrozendrinks.com', 'https://www.choicehotels.com/north-carolina/charlotte/rodeway-inn-hotels/nc168/rates', 'http://www.bramaleatire.com', None, 'https://www.nevadainsuranceenrollment.com', None, 'https://www.goldencoastii.com', 'http://jardin-thai.com/', None, 'https://paulpaddalaw.com', 'http://vegashotwater.com', None, None, None, 'http://www.mooreauto.com', None, None, 'http://MGpatisserie.com', 'https://www.corebellawellness.com', 'http://www.heartcleaneats.com', 'http://www.hhovv.org/', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.BetterThanYiaYias.com', '1592064927', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'http://www.jordanbowden.com?cmpid=D3ZF_BLM_0006', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://stores.jcrew.com/en/j-crew-yorkdale', 'http://www.frostgelato.com', 'http://www.dairykingsoutheuclid.com/', 'https://www.target.com/sl/willoughby/1112', 'https://www.target.com/sl/fitchburg/2106', 'http://www.excalibur.com', 'https://www.zippssportsgrills.com', 'http://theshoebox.com', None, 'http://miracleeyebrows.com/', 'http://www.biteandsip.ca', 'http://www.247acaz.com', 'http://michaelsgenuine.com', None, 'http://www.mtlebanonpubandpizza.com', None, 'http://www.ipowerweb.com', None, 'http://kidzpreschool.com/contact/mountainsedgeandsouthernhighlands', '1592065153', 'http://www.doughboyspizzawaunakee.com/', None, None, 'http://www.evildeadvegas.com', 'https://www.marriott.com/hotel-restaurants/phxdr-jw-marriott-phoenix-desert-ridge-resort-and-spa/stonegrill/5543174/home-page.mi', 'http://streetfoodapp.com/', 'https://atriumhealth.org/medical-services/childrens-services/levine-childrens-hospital', 'http://www.snowpalacecalgary.com', 'http://www.makemydoor.com', None, 'http://www.cjgenterprise.com', 'http://www.rodiworld.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.hottopic.com', 'http://www.gtavapes.com', 'https://www.fortmillderm.com', 'https://www.learningjungle.com/en-US/lone-mountain/', '1592065314', None, 'http://www.arizonafederal.org', None, 'http://www.holeeburger.com/', 'https://www.peterpiperpizza.com/locations/mesa-az/gilbert-baseline', None, 'http://onestopnutrition.com', 'http://www.thebloordalepantry.com/', 'http://www.playitagainsam.com', None, None, None, 'http://www.churchs.com', None, 'http://www.starbucks.com', None, 'http://www.muchoburrito.com', 'https://www.chezmatante.ca/fr', None, 'http://www.thebarndoorstore.com/', 'http://www.carrefourmultisports.com/', 'http://www.loicgourmet.ca', 'http://www.swedishautowerks.com', None, 'http://www.theroomaz.com', None, 'http://sushishop.com', 'http://www.kensingtonriversideinn.com', 'http://www.ghardytours.com', 'http://www.marmiteafricaine.ca/', 'http://www.rcsandbar.com', 'http://www.theaustralianaz.com', 'http://www.pctravelservice.com', '1592065589', 'https://www.global-restaurant.com', 'http://www.tfsgolf.com/', None, '1592065626', 'http://www.dunkindonuts.com', None, 'http://www.locksmithsolutionslv.com', None, 'http://www.victoriaballet.com/', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.dollartree.com/', 'http://www.perfectionistcleaningservices.co', 'http://parkerautotransport.com/', 'http://www.mrfallstree.com', 'http://premiumnailsandspa.com', 'http://www.teamtheraputix.ca/', 'http://www.wholegrainbreadco.com/', 'http://www.sweettoothfestival.com/', 'http://www.hendersonhyundai.com', 'https://www.camelbackcollision.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'http://www.pizzahut.com/', None, 'http://www.xpresslubecharlotte.com', None, None, None, 'http://www.momos.ca', 'http://www.brdiamondsuite.com', None, None, 'https://www.reliablerooteraz.com', 'http://www.pitaland.ca', 'http://www.ActionComicsandSports.com', 'http://www.davisbakery.net', 'http://www.riverstonetaverne.com/', 'http://www.FiveStarPM.com', 'http://www.mrmechanicofaz.com', None, 'http://www.foodlion.com', 'http://www.zombiemfg.com', None, 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://emergencyfamilymovers.com', 'https://useggrestaurant.com/', 'https://www.nationalsports.com/pages/store-pickering-ontario', 'http://airkewld.com', '1592066020', None, None, None, 'http://www.lacasadecristo.com', 'http://cfnsaz.com', None, 'http://macmillanorchards.com/', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTWS', 'http://www.momskoreanfood.com', 'http://www.azpops.com/', 'http://www.mamaspizzamatthews.com', 'https://www.bannerhealth.com/locations/glendale/banner-thunderbird-surgery-center', 'http://www.starbucks.ca', None, None, 'http://www.montroseautooutlet.com', 'http://kidneypuncher.com/', 'http://www.drnoorda.com/', 'https://candrtire.com', None, None, 'http://www.bradfordclinic.com', 'http://www.theepicure.ca', None, None, None, 'http://www.timhortons.com', None, None, 'http://www.abstractahairsalon.com', 'http://hungrybeecatering.com', 'http://livingatlavie.com', 'http://www.pittsburgh-home-inspections.com/', 'https://www.finks.com/birkdale-village', 'http://www.fearsconfections.com', '1592066435', 'http://www.crossfitvenividivici.com', '1592066459', None, 'http://herbologytours.com', 'http://www.lorays.com/home', None, 'http://www.ateraspas.com', None, 'https://www.supercuts.com/locations/nv/henderson/eastern-commons-haircuts-9195.html', 'https://stores.joann.com/oh/hudson/171/', 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=phxch&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', 'https://www.seaboardbrewing.com', None, 'http://www.lavenderoldlace.com', '1592066560', 'https://www.sleepcountry.ca', None, 'http://www.picknsave.com/StoreLocator.aspx', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://newserenityspa.com', 'http://www.pesticon.com', 'http://www.sushieye.com/', None, None, None, None, 'http://www.worldoftile.net/wordpress/locations/scottsdale/', None, None, 'https://fitnesstogether.com/plaza-midwood?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://handy-howies-handyman-services.business.site', 'http://nataliesnailsalonlasvegas.com', 'http://fiveguys.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, None, None, 'http://extremepita.com', None, 'http://www.solaruniverse.com/vegas', 'http://www.securitysupplystore.ca', 'http://jackastors.com/location-map/34/', 'http://www.lakenormancottage.com', 'http://sunsetwater.net', 'http://www.loveyoubackboutique.com', 'http://www.coach.com', 'http://www.scottsdaleoralsurgery.com/', None, 'http://www.davidsonday.org', 'http://www.babypointlounge.com', 'http://www.aerushome.com', 'https://www.dtlvarts.com', '1592066878', None, None, 'https://www.uber.com', None, None, 'https://www.theskincentermd.com/cosmetic-plastic-surgery-shadyside-pittsburgh-pa', 'http://www.PrecisionWindowTint.com', None, 'http://www.juicecore.com/', 'http://locations.einsteinbros.com/az/scottsdale/6501-e-greenway-pkwy-bldg-5.html?utm_source=storelocator', 'http://jewelrycitylasvegas.com', 'http://heartlakeauto.net', 'http://www.mcclurecollision.com/', 'http://canyonmidwifery.com', None, None, 'http://www.superstore.ca', 'http://www.atlascinemas.net', 'http://www.fosterinn.com', 'http://5two3.com', None, 'http://www.girlsgolfofphoenix.org', 'http://www.asu.edu/news/forthemedia/asufacts/campuses.htm', 'http://igniteglassandgifts.com', 'http://www.fixsys.ca', 'http://www.skydivelasvegas.com', None, None, 'http://guildresto.com/', 'http://www.focusbenefits.com', 'http://smokespoutinerie.com', None, 'http://www.jcprinting.net', 'http://www.passautos.net', 'http://www.papagiuseppes.com/', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://www.salonbellarouge.com', 'https://my.clevelandclinic.org', 'http://www.littlecaesars.com/', 'http://www.graciebarra.com/ahwatukee-az', 'http://www.marahselegantbridal.com', None, 'http://albertaballet.com', None, 'http://www.eyebrows4us.com', None, 'https://familydentistryofharrisburg.com', '1592067261', 'http://lenasrotianddoubles.ca', None, None, 'http://www.aquasystems.ca/', '1592067298', 'http://www.limoluxusa.com/', None, 'https://stores.cartier.com/united-states/nv/las-vegas/3131-las-vegas-blvd-south', None, 'https://www.reliablecouriers.com/service-area/arizona/phoenix', 'http://www.theoperahousetoronto.com', 'http://www.pinkberry.com/stores/15324', 'http://facejuvenate.com', None, 'http://www.tooltimehandymanservices.com', 'http://www.concordcustomcleaners.com', None, None, 'http://www.iriefoodjoint.com/', 'http://www.laherradero.com', 'http://www.insuremedave.com?cmpid=ram6_blm_0006', None, 'https://www.caseys.com/general-store/wi-de-forest/602-w-north-st/1936?utm_campaign=website&utm_term=organic&utm_medium=referral&utm_source=yelp&utm_content=1936', None, 'https://www.services.gov.on.ca/services/mapLocations.do?action=m', 'http://www.illinipubcycle.com', 'http://www.automotivespecialtyservices.com', None, 'http://mcmanusandcampbellhair.com', 'http://www.lv-pm.com', 'http://www.brunswickvolkswagen.com', 'http://www.tempe.gov/museum', 'http://www.firstchoicedental.com/locations/east-madison-office', None, 'https://smp-ink.com', 'http://www.lilliasdressmaking.com', 'http://www.firstcareclinic.org', 'http://www.redrockcanyonlv.org/hikes-trails/first-creek-canyon', None, 'http://www.discount-drugmart.com', 'http://rumrunnervegas.com/', 'http://www.starbucks.com/', None, None, 'http://thebco.com', 'http://www.oldworldbrewery.com', None, None, None, 'https://www.mattressfirm.com/stores/az/scottsdale/mattress-stores-scottsdale-az-85260-2922.html', None, 'https://www.walmart.com/store/1380/las-vegas-nm/details', None, None, None, 'http://www.flashtelwireless.com', 'http://www.hornekia.com', None, 'http://www.yogurtology.com', None, 'http://www.iliosnoche.com', '1592067721', None, None, None, 'http://baycathospital.com/', 'http://www.sportinglife.ca/', 'http://www.montebellasalon.com', 'http://www.salvationarmyusa.org', 'http://www.crossfitmecklenburg.com', 'http://Jjsgrubbshacknliquor.com', 'http://10minuteoilchangers.business.site', 'http://www.pizzacafe.ca', 'https://www.theneckandbackclinics.com/locations/southwest-las-vegas', None, None, 'http://nextgenelectrical.com', None, None, '1592067863', None, 'http://www.clarkcountynv.gov/Depts/parks/Pages/charlie-frias-park.aspx', None, None, 'http://www.dunkindonuts.com', 'http://www.shopnofrills.ca', 'http://santemedical.ca/', None, 'http://3rdphazebodyoil.net', 'http://www.mogowestlake.com/', 'https://www.23twelveluxe.com', 'http://painting.vegas', 'https://locations.rallys.com/az/tempe/1410-west-university', 'http://www.swisschalet.com', 'http://www.rfadezbarberspa.com', None, 'http://www.bluedogtraining.com', 'http://www.brueggers.com/', 'https://www.fitnessworks.com/locations/dobson-rd-chandler-blvd', '1592068059', 'http://www.freskamediterraneangrill.com/', 'https://www.choicehotels.com/pennsylvania/gibsonia/quality-inn-hotels/pa122/rates', None, None, None, 'http://www.coyoacangrill.com', 'http://www.dobsonspringsapartments.com', None, 'https://sites.google.com/site/brightcleantollesonlaundromat/', None, 'http://www.ihg.com/candlewood/hotels/us/en/tempe/phxbr/hoteldetail', 'http://www.guerlain.com', None, 'https://www.usps.com/', 'http://www.acelebritylimousine.com', 'http://www.laserspecialists.net', 'http://vegasstronglocksmithlv.com', None, None, 'http://www.centremecanix.com', 'http://www.schmitzdoor.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.fourturnberryplace.com/', None, None, 'http://www.lassendas.com', 'http://www.UnitedBloodServicesAZ.org', 'http://www.wildburger.ca', 'https://www.pizzadivinoaz.com', None, 'http://www.chicagojoesrestaurant.com', 'http://flopatio.com/', 'http://www.undrcard.com', 'https://www.cherrylash.com', 'http://restaurants.quiznos.ca/on/mississauga/CLARKSonCROSSING-L5J2Y4', 'http://www.kitcabinets.com', None, 'http://www.nevadaozone.com/', 'http://www.arizonaturnergroup.com', 'http://samwanna.com/', 'https://www.phxfray.com', 'http://www.azdonutco.com/', 'http://www.mtnroadcycles.com', 'http://www.martinisandmagicshow.com', 'http://www.ghhlasvegas.com', None, 'http://www.tradeinautobrokers.com', 'http://www.enterprisecarsales.com/location/39R2/Enterprise_Car_Sales_Bedford?mcid=iyp:277&CUR=yelp', 'https://local.biglots.com/az/tempe/4604?zcp=pd_local_yelp', None, 'https://tonysgaragelv.com', None, 'http://www.papajohns.com', None, 'http://www.searshomeapplianceshowroom.com/locations/phoenix-az', None, None, None, 'http://www.hivehealinghouse.com', 'http://americansprinklerlandscape.com', 'http://www.ckdesignslv.com', 'http://www.lasvegasoralsurgery.com', 'https://www.signaturestyle.com/locations/on/markham/first-choice-haircutters-51352.html', None, 'http://poopertrooperslv.com', None, 'https://www.charlottebraces.com/davidson-office-location', 'http://www.josephiarussi.com', 'http://www.yankeecandle.com', None, 'https://www.369bugs.com', None, None, None, None, '1592068718', 'http://www.frysfood.com/', None, None, 'https://www.lasvegassedation.com', 'https://vitospizzaglendale.com', 'https://www.amazinglashstudio.com/studios/az/phoenix/phoenix-ahwatukee', 'https://www.azpulm.com', '1592068798', 'http://www.cedarbraemall.ca', None, 'http://megansframing.com/', None, 'http://www.summitapartmenthomes.com', 'http://www.vegasgarageco.com', 'http://www.terroni.com/bar-centrale/', 'https://www.windsorstore.com/blogs/store-locator/nv-henderson-henderson-galleria-at-sunset', None, 'http://www.snaponoptics.com', None, 'http://rockinsul.com', 'http://www.thescottsdaleresort.com/restaurants/palm_court/', 'http://www.salsablancagrill.com', 'http://www.acupuncturevegas.com', 'http://www.blvdfoodcourt.com', 'http://www.drwiner.com/', None, None, 'https://lasvegasbankruptcylawyer.co', 'http://www.celebritytanning.com', '1592069006', 'http://beststripclublasvegas.com', 'http://www.hometastingroom.ca', 'http://www.clubredrocks.com/', 'http://greaterthancoffee.com', 'http://www.caferio.com/', None, 'http://www.MountainsideAir.com', 'http://adrenalinerushslingshot.com', None, 'https://www.take5oilchange.com/locations/oh/southeuclid-140/', 'http://www.pizzahut.com/', 'http://www.micabeauty.com', 'https://www.zeppestavern.com', 'http://www.woodworkerssource.com', None, 'https://www.steammaxx.net', 'http://www.arizonaveterinaryoncology.com', 'http://newellfarmersmarket.com/', 'http://www.theheadquarters.ca', None, 'http://www.sunrisekidsdental.com/', 'http://www.azprecisionpet.com', 'https://store.401games.ca', 'http://www.meineke.com/locations/il', 'http://medpost.com/location/surprise-az', None, None, 'http://www.wickedpixiehair.com', 'http://www.poorlittlerichgirlresale.com', 'http://www.karenwrightins.com?cmpid=G2LN_BLM_0006', 'https://lesliescleaningservice.wordpress.com', 'http://www.sacredseed.com', 'http://fiveguys.com/', 'http://www.raginuptown.com/', 'http://www.meadowspediatrics.org', 'https://www.sprint.com/storefronts/bd/sprint-pittsburgh-pa-15235-fcs-102/?ecid=SEO:Yelp:2020_Q1:LastMile', None, 'https://www.wokwithmingchinese.com', 'http://mdcpgh.com/', None, 'http://www.moochieandco.com', 'http://www.coronalandscape.com', 'http://www.devourphoenix.com', None, 'http://www.vegasvacuumsewing.com', '1592069417', None, 'http://www.rossstores.com', None, 'http://www.abercrombie.com', 'https://www.homedepot.com/l/Tempe-K/AZ/Tempe/85283/489?cm_mmc=seo-yelp&', 'http://www.framezillapgh.com/', 'http://carguyscanada.ca', 'https://www.enterprise.com/en/car-rental/locations/us/al/huntsville-madison-55ha.html?mcid=iyp:8368400', None, None, 'https://www.fyhanimalhospital.org', 'http://burritozone.ca/', 'http://www.malayakitchen.com', None, 'https://www.choicehotels.com/CN478?mc=blypypaf', '1592069575', 'https://clearviewplumbing.ca', 'http://www.azvalleydental.com', 'http://www.hiddencoveapartments.com', 'http://platinumhealth.ca', 'http://www.rachellallen.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, 'https://www.jerseymikes.com/3141', 'http://www.rexall.ca/', None, 'http://www.seah.ca', None, 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', None, 'https://www.kohls.com/stores/oh/stow-331.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', None, None, None, None, 'http://www.starbucks.com/store/9334/', None, 'http://pamperedhair.com', 'http://www.31kitchen.com', 'http://www.acsmartialarts.com', None, 'http://www.innovativeeyecare.net', 'https://theresortatsummerlin.com/dining/casual-dining/market-place-buffet-las-vegas/', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.mzdds.com', 'http://www.jeanniesgrooming.com', 'http://www.mobilemassagebychristina.com', 'http://www.acsm-clinic.org', 'https://www.anytimefitness.com/gyms/4149/Champaign-IL-61820/?utm_source=yelp&utm_medium=local', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.jessicahair.ca', 'http://www.teamafterburn.com', None, '1592070253', None, 'https://www.beemasterlv.com', 'https://sierraspringsdental.com', None, 'https://www.alfaromeousaofhenderson.com/contact.htm', '1592070319', 'http://www.boutiquetalulah.com', None, None, 'http://www.spirithalloween.com', 'http://www.rountreeplantation.com', 'http://www.forever21.com', 'http://serviceexpertscleveland.com/', None, 'http://suncontrollandscapes.com/', 'https://locator.bmo.com/Default.aspx?t=bb&lang=en&sid=8803', 'https://locations.wendys.com/united-states/sc/fort-mill/1800-hwy-160-w.', 'http://www.elmayarestaurante.com', None, None, 'https://www.brightnow.com/dentist-near-me/surprise-dentist/40110', 'http://www.lasvegascc.com', 'http://www.landscapeguy.net', 'https://www.cloverleafsuites.com/charlotte/', 'http://www.ihg.com/holidayinnexpress/hotels/us/en/phoenix/phxun/hoteldetail', 'http://myInsightImaging.com/AZ', None, 'http://www.azfaccs.org', 'http://www.phoenixhairextensions.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://rodneygentry.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006430627392&ds_s_kwgid=58700000443429206&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.onearmor.com', 'http://www.opusglow.com', 'http://www.hurleybrokers.com', 'http://www.Anchoranvilcoffee.com', None, 'http://www.greatclips.com/', 'http://sahniendo.com', '1592070593', 'https://www.chevronwithtechron.com', 'https://www.rayspizzacavecreek.com', None, 'http://www.hotelblackfoot.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://wwww.outlawscavecreek.com', 'http://www.beunailsbeauty.com', 'http://www.maggiesdonuts.com', None, 'https://www.johnfloydpaintingllc.com', None, None, 'http://carstarexpress.ca/locations/toronto/', 'http://www.coyoteplumbingandrooter.com', 'http://rothrockdmd.com/', 'http://www.mcmurraystylingcenter.com', 'http://danmarpetresort.com', 'https://www.MedgausDentalCare.com', 'http://www.starbucks.com', None, 'http://www.vietnampalace.ca', None, None, 'http://advancedanklefoot.com', 'http://www.sscandycigar.com/', 'http://signaturewest.net', None, None, 'http://www.burritoboyz.ca', None, None, 'https://www.moonvalleynurseries.com/services/tree-service-nevada?utm_campaign=yelp&utm_source=yelp-nv&utm_medium=tree-care', 'http://www.majesticeyebrowthreading.com', 'https://www.devitomd.com', None, 'https://www.harveys.ca', None, None, 'http://sofiasbraids.com/', None, 'https://www.aderraapts.com', 'http://goodwillsp.org/shop/retail-stores/second-editions', None, 'https://www.mattressfirm.com/stores/az/phoenix/mattress-stores-phoenix-az-85032-7927.html', 'http://www.azdmvaz.com/', 'http://www.TheMovementDanceExperience.com', 'http://www.hurstbakery.com/', 'http://www.sacktimemattress.com', 'http://www.bedbathandbeyond.com', 'http://www.shell.us', 'http://www.tambellinisbridgeville.com', 'https://electriciansmesaaz.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700007898397760&ds_s_kwgid=58700000649767852&ds_e_adid=%7Bcreative%7D&ds_url_v=2&cmpid=ADV_PG_%5B*KeywordID*%5D', 'http://www.redkingkong.com', 'http://www.bestbuy.com', None, 'http://eatnewchina.com', None, None, None, 'http://www.lvpremierpt.com', 'http://www.osteoklinika.com', 'http://koreangrillhouse.com/yonge-bloor-location/', 'https://www.providencedental.com', 'http://www.cabana-restaurant.com', None, 'http://www.dfwfurniture.com', None, 'http://www.impbaseline.com', 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.rumbi.com', 'https://arbatrestaurant.com', None, None, '1592071168', 'http://www.marshallretailgroup.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.firehousesubs.com/locations/oh/mayfield-heights', None, 'https://www.calldunrite.com', 'http://www.childrenslearningadventure.com/', 'http://www.bankofthewest.com', 'http://www.deweyscoffee.com', 'https://mercado-negro-tapas-restaurant.business.site', 'http://dethaicuisine.com', None, None, '1592071267', 'http://locations.harristeeter.com/nc/charlotte/205/', '1592071283', 'http://www.starbucks.com/store/8167/', 'http://www.amhomeservices.ca/', 'http://www.marbleslab.ca', 'http://castlecoffeelasvegas.myrestaurantpage.website', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://bikesharetoronto.com', 'http://storelocator.pepboys.com/pepboys/stores/chandler/chandler/00735', 'http://Www.makeupbynikkip.com', None, None, 'http://www.sevenhillsdentalcenter.com', 'http://www.stclairtanningspa.com', 'http://www.ontimevegas.com', None, None, 'http://www.lasvegasdiningtours.com/', None, 'http://www.airconditioningarizona.com/', 'http://www.spayaz.com/locations/mesa-az/', 'http://www.crossfitvenividivici.com', None, None, 'http://www.oakasushihouse.com', None, None, 'http://southeastlocksmith.com/', 'http://www.arizonacharliesdecatur.com/Food-Drink/Charlies-Market-Buffet', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://atechautocenter.com/', 'https://ad.doubleclick.net/ddm/clk/434184450;236814625;f', '1592071537', None, 'http://www.bonterrafarmmarket.com', None, 'http://solobace.com/', 'https://www.popeyes.com', 'http://holidayhoundhouse.com', 'http://zebreoptique.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', None, 'https://www.ediblearrangements.ca/stores/250-7th-Ave-SW-Calgary-AB-T2P-0W6', 'http://www.kaypacha.ca', None, 'http://whasn.com', 'http://www.winners.ca', 'http://www.elsantocantina.com/', 'http://www.thehomeshopnv.com/', 'https://stores.maccosmetics.com/us/oh/westlake/262-crocker-park-blvd', 'http://www.valueplace.com', 'https://stores.jcpenney.com/pa/pittsburgh/clothing-stores-pittsburgh-pa-2263.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2263', 'http://blackchickenwinecellar.com/', 'http://www.matthewsastrology.com', 'http://www.corrados.ca', 'http://www.apple.com/fashionshow', None, 'http://www.perfectprincesscut.com/', None, 'http://www.kouski.com', None, '1592071798', 'http://www.patdees.com/', 'https://order.subway.com/Default.aspx', 'http://fiveguys.com/', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTVF', 'http://smartypaws.com', '1592071848', None, 'http://reflexologypedicure.blogspot.com/', 'https://www.lhshealth.com/cedarwood-plaza/', None, 'http://www.scottssharpening.com', 'http://www.starbucks.com/store/11425/', 'http://gratifyinggrains.com', 'http://dreamconnections.com', '1592071927', None, 'http://www.actionazcomputers.com', 'http://www.sushixtra.com', 'http://www.pinevillecomputers.com', 'https://corksncrafts.com', 'http://www.pictureshowent.com/home-mesa/', None, 'https://locations.whataburger.com/az/scottsdale/2955-n-hayden-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', None, 'http://www.simplebistro.com', 'http://www.jimmysbackyardbbq.com', None, None, 'http://dentistrylasvegasnv.com', 'http://fishsandwich.ca/', 'http://yoamotacos.com/woodbridge-canada/', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.valentinos.ca', 'https://computerrepairdoctor.com/locations/cleveland/', None, 'http://www.sugarmarmalade.com/', 'http://www.winners.ca', 'http://www.originalchimneys.com/', 'https://www.hellotech.com/us/nc/mecklenburg/charlotte', None, None, 'http://www.champaigntruetires.com', 'http://www.hokkaidoseafoodsushi.com', None, 'http://www.innerstrengthchiro.com', None, None, None, None, 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2fnv%2flas-vegas%2fmidtown&sisearchengine=1524&siproduct=BEMDT_Bus', '1592072228', 'http://www.lettysonshamrock.com', None, None, 'https://www.wyndhamhotels.com/hotel/29495?tel=18882935571&iata=00093785&cid=fe:bh:20161215:yl:pp:hawus:29495', '1592072279', 'http://charlestonantiquemall.com/', 'http://apple.ca/retail/dix30', 'http://majerles.com/goodyear/', 'http://bebosmacshack.com', None, 'http://pamperohio.com/about.php', 'http://www.baystreetvideo.com/', 'http://www.bistro2182.com', 'http://staples.ca', None, 'http://www.uhhospitals.org/locations/uh-st-john-medical-center/st-john-medical-center-emergency-room', 'https://store.vioc.com/oh/brunswick/3675-center-rd-44212-060023', None, None, 'http://www.unionhillscc.com/', 'http://brighteyespa.ca', 'http://www.cosmictreats.ca', '1592072432', 'http://stardustvapes.org', None, 'http://www.nancyscheese.com', 'http://local.fedex.com/nc/concord/SRWA/', None, 'http://www.ososkatepark.com', 'http://www.alligatorpartyrental.com/', 'https://www.photobooths-charlottenc.com', None, 'http://jovanilinens.com', 'http://www.torontopearson.com/en/parking/#%23', 'http://mrgreek.com', 'http://www.thaihana99.com', 'http://www.FlawlessBeautiAndNailBar.com', None, None, 'http://www.cashamerica.com/SUPERPAWN-stores/AZ/PHOENIX/Pho18', 'http://www.holidayinn.com/hotels/us/en/toronto/yyzct/hoteldetail', None, None, 'http://vibrantcare.com', None, 'http://www.planetfitness.com/gyms/mississauga-dundas-st-on-1431?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'http://www.perfumes4u.com', 'http://www.fitness19.com', '1592072690', 'http://www.brisams.com/', None, 'http://www.dmelv.com', 'http://www.picea997.com', 'http://www.les3maria.com', 'http://www.jazz28.com', 'http://www.petland.ca/store-locator', None, 'http://www.tcccollision.com', 'http://www.advantaclean.com/las-vegas-nv', None, 'http://ccshvac.net', 'https://longevitymedical.com', None, 'https://tottoriallergy.com', 'http://richsautomotive.net', None, None, 'http://tasty-n-healthy.com/', 'http://www.reesors.ca/', 'http://www.tacoshuichoaz.com', 'https://www.azcasona.net', 'http://www.diamondtaxi.ca/', 'http://www.noblewindowcleaning.com', 'https://www.holysheetsusa.com', None, 'https://livepalmvillas.com', 'http://thelodgeatgrandtetonlasvegas2.cafesplanet.com', 'https://www.autumnparkcharlotte.com', 'https://www.hubbardswim.com', 'http://www.poolsnbeyond.com', 'http://www.littlerascalsrescue.com/', 'http://cityfreshpastapgh.com', 'http://www.primeroofingaz.com', 'http://www.lasvegasendodontics.com', None, None, 'http://uspalasvegas.com', 'http://stingraysushi.com/', None, None, None, 'http://www.pelicanssnoballs.com/', 'http://lakewoodmusicacademy.com', 'http://www.snvrha.org/index.htm', 'http://eastvalleysewnvac.com', 'http://www.daysinn.com/hotels/hotel-overview?wid=ps:addppc_yelp&tel=18003291073&reg=appc-_-ylPROPP-_-00405-_-diWES&hotel_id=00405&cid=whg_di_yelp', None, 'http://sushiave.net', None, 'http://www.silvercityrestaurant.ca', 'https://www.dontdrivedirty.com/locationsandpricing/?id=904', 'http://pages.wynnlasvegas.com/showstoppers', 'https://bluelakesassistedliving.com', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'http://www.facetofacefotographie.com', 'http://locations.harristeeter.com/nc/concord/202/', 'http://www.crownbuilders.com', 'http://www.mississaugahonda.com', 'https://www.goodfeet.com/charlotte', 'http://doorclosingsystems.com/', '1592073208', 'http://www.expressautospot.com', 'http://www.chestnutresidence.utoronto.ca/', 'http://www.naturalmeats.org', None, 'https://www.pizzaepazzi.ca', None, None, 'https://saenz-cleaning-llc-house-cleaning-service.business.site', 'https://einsteinplumbing.com/las-vegas-nv', '1592073287', 'http://www.thegreatinflatablerace.com/las-vegas', 'http://phenixsalonsuites.com/locations-detail/?id=94', 'http://www.houndhuddle.com', 'http://www.littlecaesars.com', None, '1592073354', None, 'http://www.stripdistrictmeats.com/', None, None, 'http://www.fearlesskittyrescue.org', None, 'http://www.rollinpizzapies.com', 'http://nsbank.com/', 'http://locator.carolinas.aaa.com/locations/pineville/', None, 'http://www.fortefamilypractice.com', 'http://www.arbys.com/', None, 'http://www.beatitudeschurch.org', None, None, 'https://www.alaskaair.com', 'http://www.discounttire.com/dtcs/makeAppointmentForStore.do?sc=NVL-12&utm_source=Yelp&utm_medium=iyp&utm_campaign=YelpIYPSA', None, 'http://www.parisoptique.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7784&localechange=1', None, 'http://thebreakawaybarandgrill.com/', None, 'https://www.riteaid.com/store-details?storeNumber=11467', None, 'http://storelocator.staples.ca/stores/fr_CA/QC/Saint-Laurent/21', 'https://www.supercuts.com/locations/sc/rock-hill/manchester-village-haircuts-82206.html', 'http://www.nacvegas.com', None, 'http://www.peoriashippingservice.com', 'http://www.bravaboutique.ca', 'http://www.saffronlasvegas.com/', '1592073756', None, None, 'http://gammajeansinhomedogboarding.com', 'http://www.eternitynailsspa.com', 'http://www.foodymart.com/', 'http://www.vogue-salon-spa.com', None, None, 'http://freshco.com/stores/bloor-dundas', 'http://www.tiltedgrill.com', 'http://www.northpointeyestudio.com', 'http://www.rightbuyautosales.com', 'https://www.jiffylube.com/locations/az/tempe/1665', 'https://www.choicehotels.com/wisconsin/madison/comfort-inn-hotels/wi415/rates', 'http://www.theinsidecoup.com', 'http://www.healingroots.center', 'http://www.lululemon.com/toronto/yorkdale?cid=yelp', None, 'http://tiffanynailsandspa.ca', None, 'http://www.ivoryparade.com', 'http://shehadys.com', 'http://www.torontonorthmitsubishi.ca', 'https://www.jiffylube.com/locations/nc/cornelius/3148', 'http://www.bridalboutiqueofaz.com', None, 'http://www.portofsubs.com/', None, 'https://yorkvilles.ca', 'http://www.atmospheresalon.com/', None, 'http://www.concordtavern.com', 'http://tyndallfurniture.com', 'http://www.las-vegas-locksmith.us/', 'http://www.saharamedicalinstitute.com', 'http://3vintagechicks.com/', 'http://www.edandmarks.com', 'http://www.eatoooweebbq.com', None, 'http://www.cbdefa.com', 'http://www.kuhars.com', 'https://www.mattressfirm.com/stores/nc/charlotte/mattress-stores-charlotte-nc-28277-5684.html', None, None, 'http://www.lalalunaspa.com', 'http://www.titansgym.com/index.php?Itemid=47&view=article&id=52&option=com_content', 'http://www.quiznos.com', 'http://www.sunpenang.com', 'http://www.stevenvance.com', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', '1592074181', 'http://makerealestateeasy.com', 'https://koliequestrian.com', 'http://www.goldmedalgym.com/chandler', None, 'http://www.freckaplumbing.com', 'http://www.mamaginaspizzeria.com', 'http://www.hotwire-electric.com', '1592074242', None, None, 'https://www.littledealer.com/pages/location%20phoenix', None, 'http://www.cleanandamazing.com', None, 'http://somagrill.com', 'http://www.xtremeperformance.info/', 'https://www.discounttire.com/store/nv/henderson/s/1300?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://www.gabesstores.com', 'http://5-starair.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://arthurmurray.com/locations/las-vegas-west-studio', None, 'http://www.winebeercigars.com', 'https://www.dairyqueen.com/us-en/locator/Detail/2398', 'http://www.glutenzerobakery.com/', None, 'http://www.cowboyciao.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', 'http://www.rebel-auto.com/', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=443', 'http://salsaritas.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.shoptieoneon.com/about-us.html', 'http://remachines.com', 'http://queencityrides.com', 'http://www.mazajloungecalgary.com', 'http://hahnsurvival.com', 'http://www.charmingcharlie.com/', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'https://www.starbucks.ca', None, '1592074521', '1592074530', 'http://www.scooterinvasion.com', 'http://www.zenewicznotary.com', 'http://www.onyxtheatre.com', 'http://www.mirasnails.com', 'http://www.gostorageone.com/self_storage/Las_Vegas_NV/zip_89130/storageone_self_storage/10494', 'https://www.dahntahndonuts.com', 'http://thirdeyebodyart.com', None, None, 'http://www.cjaimnotary.com', 'https://www.smalv.com/OurCenters/HealthCareCenters/Nellis', 'http://www.sinteriordesign.com', 'https://mstarkeychiro.com', 'http://www.provocateurimages.ca/', 'http://www.newwavemarket.com', 'http://www.dentistrybydesignchampaign.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.scratchkitchenclt.com', None, 'http://www.harlemrestaurant.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://classicchinesecuisine.com', 'http://www.thepreeners.com', None, 'http://www.organicmarketplacenc.com', 'https://www.officedepot.com/storelocator/nv/las-vegas/office-depot-6761/', None, None, 'http://www.surprisespringtraining.com', None, 'http://www.pickeringtoyota.com', None, 'http://www.bonniesprings.com/', 'http://www.divasalonspa.com/', 'http://wi.madison.charmingcharlie.com/fashion_accessories_madison_wi_124.html', None, 'http://roseandcrown.com/', None, 'http://www.buffalochipsaloon.com', 'http://rainbowcinemas.ca', 'http://www.grandchinesecuisine.com', 'http://www.azcommercialproperties.info/', None, 'https://bumpandroll.com', 'http://www.springmeadowliving.com', None, 'http://www.brakeshopaz.com', 'https://www.singyourstyle.com', 'http://lajollaexpress.net', '1592074921', None, 'http://calladams.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=10538&localechange=1', None, 'http://www.bulwarkpestcontrol.com', None, None, None, 'https://yourfeetfixer.com/locations/chandler/', 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', None, None, 'http://www.sofashack.com', 'http://mcneelyfamilydentistry.com', 'http://www.ludexauto.com', None, 'https://www.benjerry.com/gastonia', 'http://www.timhortons.com', 'http://raffin.ca', None, None, 'http://www.freshss.ca', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006435132512&ds_s_kwgid=58700000444952194&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.goodspeedperformance.com', 'http://www.ritzcarlton.com/en/Properties/Toronto/Dining/DEQ_Lounge/Default.htm', 'http://www.thesycamoreatscottsdale.com/', 'http://www.cobsbread.com/local-bakery/kingsland-calgary', None, 'https://fratellosdeliohio.com', 'https://www.wyndhamhotels.com/hotel/11185?tel=18007606483&iata=00093785&cid=fe:tl:20170523:yl:pp:tlus:11185', None, 'http://www.desertsparkle.com', None, 'http://allcityrefrigeration.com/', 'http://www.ipl-laserhairremoval.com', None, 'http://www.monicals.com/locations/monicals-pizza-of-tolono/', '1592075253', 'https://mercedes-benz-thornhill.ca', 'https://www.simonmed.com/locations/all/simonmed_imaging___summerlin', None, 'http://www.watericemesa.com', 'http://www.swisschalet.com', None, None, 'http://acehotel.com/pittsburgh', 'http://www.restauranttranscanada.ca', None, '1592075354', 'https://www.batteriesplus.com/store-locator/az/surprise/batteries-plus-616?utm_source=yelp&utm_medium=local&storecode=616', 'http://www.pioneersand.com', 'http://www.discounttire.com/', 'http://www.jonesoptical.com', 'http://www.illsecondthatonline.com', 'https://dealer.sprint.com/experts-choice', 'http://www.veneciagastonia.com', 'https://www.greatwolf.com/concord/dining-shopping/dining/loose-moose-cottage', 'https://www.scottsdale5thapts.com', 'http://www.shanghaipalacerestaurant.ca/', None, 'http://anytimefitness.com/en-us/clubs/582', None, '1592075489', 'http://lakecityair.com', 'http://www.level20lounge.com', 'http://www.vetteculture.net', 'http://chambersphx.com/', None, None, 'http://www.jimmythegreekgroup.ca', 'http://www.element-dayspa.com/', 'http://www.shopbop.com', '1592075568', 'http://www.the505e.com/', 'http://icyroll20.com', 'http://www.verizonwireless.com/stores/arizona/surprise/surprise-319434/', 'http://www.blizzyogurt.com', None, 'http://www.mygnp.com', '1592075632', 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/etobicoke-(1).aspx', None, '1592075661', 'http://homefabricsonline.com', 'https://kayahempco.com', 'http://www.planetfitness.com/gyms/kannapolis-nc-1212?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', '1592075693', 'http://myarizonasmile.com', 'https://www.navyfederal.org/?cmpid=bn%7Cyelp%7Cevgrn%7Cbrn%7Cnull%7Cnull%7C01/13/2020%7Cml%7Cnull', 'http://www.knitandsip.com', 'http://www.carpetmdonline.com', 'http://www.slcrealty.com', None, 'http://www.gvdentalcenter.com', None, 'https://www.apple.com/ca/retail/yorkdale/', 'http://www.pizzapanorama.ca/index.html', None, None, None, None, 'https://easternarizonaexterminating.com', 'http://envoguestyle.com', None, None, 'http://www.greenvalleypres.com', 'https://www.petsuppliesplus.com/Store/OH/Middleburg-Heights/Middleburg-Heights/28?utm_source=Yelp&utm_medium=Store-Page%C3%8A', '1592075851', 'https://www.hermanheylflorist.com', 'http://arizonanationalinsurance.com', None, 'https://www.sunrisepreschools.com/locations/dysart-thomas/?utm_source=yelp&utm_campaign=online_listing_goodyear_287&utm_medium=referral&utm_term=business_url', 'http://neurocnv.com/', 'http://www.showmethatsmile.com', 'http://www.facesbylorri.com', 'http://www.qinaturalfood.com', 'http://www.longos.com/store/storedetails.aspx?store_no=10', 'https://spbarbershop.com', 'https://www.nofrills.ca/store-locator/Jerry%27s+No+Frill%27s+Mississauga/3101?storeId=3101', None, 'https://www.advantage.com/location/las-vegas-mccarran-intl-airport-las/?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_local', 'https://www.walgreens.com/locator/walgreens-133+w+8th+ave-homestead-pa-15120/id=9674', None, None, '1592076007', None, 'http://www.bikesaviours.org/', 'http://saucelv.com/', None, 'http://www.splendidchinamall.ca', 'http://www.libertytintsolutions.com', None, 'http://scottsdale.hyatt.com', 'https://www.aceofheartscpr.net/locations/ace-of-hearts-cpr-las-vegas', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'http://www.re-newcabinets.com/', 'http://www.lloydsrestaurant.ca', 'http://www.medmassage.com', None, 'http://www.familyfarmcreameries.com', 'http://www.bloomingexpressionsflowers.com/', 'https://www.valero.com', 'http://www.norwinbowl.com', 'http://www.smilestraight.com', 'http://www.sunvalleyscooters.com', 'http://www.paperjoy.com/', None, 'https://www.redclayciderworks.com', 'http://arkvegas.com/restaurants_village-streets.htm', None, None, 'http://www.martelsalon.com/', None, 'http://www.theunicornpub.com', None, 'http://www.superstarmovers.com', 'http://www.tempe.gov/parks/', 'https://shop.wirelesszone.com/ma/hudson/234-washington-street', 'http://www.phoenixcarpetrepair.com', None, None, 'http://www.fernandosmexicanrestaurant.ca', 'http://scaasports.com', 'http://www.deervalleyplumbing.com', 'http://www.nspeds.com', 'http://www.americanapparel.net', 'http://www.sahara.bhhsnv.com', 'http://www.nobutei.com/', 'http://www.marvelmasonry.com', '1592076409', 'http://store.bobbiesflowers.com/?srccode=yelp_track', None, 'http://www.cicis.com/locations/nv-las-vegas-9890-s-maryland-pkwy', 'https://www.dontdrivedirty.com/locationsandpricing/?id=907', None, 'http://www.dilisosmeats.com', None, 'http://thebacklotbar.com', 'https://palmbeachtan.com/locations/AZ/Scottsdale-North-Scottsdale-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://www.procarehospicenv.com', '1592076498', 'https://www.harborinnseafoodcharlotte.com', 'https://www.kohls.com/stores/pa/gibsonia-1271.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.nsbank.com/branch/?branch=durango_and_215_south_branch', None, None, None, None, '1592076576', 'http://www.paradisemedspa.com', None, None, 'http://www.torontolivings.com', 'https://www.usps.com', 'http://www.scottspro.net', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=976', 'https://www.epoxylasvegas.com', 'http://www.pappertdentistry.com/', 'http://www.amjcampbell.com', 'http://www.valleyendodontic.com', 'http://www.encorelasvegas.com/#/entertainment/sinatra_dance_with', 'https://www.target.com/sl/glendale/1806', 'https://www.ulta.com/stores/gilbert-az-354', 'http://habitburger.com', None, 'http://www.greatclips.com/salons/6225', 'http://www.heavenscookie.com/', 'http://www.coolcomfort.net', None, 'https://late-night.business.site', 'https://mydeluxecleaners.com', 'http://www.thaiexpress.ca/', 'http://brotherspizzaandwings.ca', 'http://www.thechefon2wheels.com', 'http://www.reedkrakoff.com', None, None, 'http://www.big5sportinggoods.com', 'http://www.onelovegrapplingarts.com', 'http://www.eatatjoeydeli.com', None, 'http://www.montgomeryross.ca/', 'http://intouchcoachingllc.com', 'http://www.grottoshakersquare.com', 'http://www.ddsdiscounts.com', 'http://martininailbar.com', 'http://www.wedjaz.com', 'http://www.kasecustom.com', 'http://amadioranch.com', None, 'http://www.dworksautotintanddetailing.com', 'http://www.toothhealer.com', 'http://www.chbaz.com', None, 'http://gameworldvegas.com/', 'http://hearingassociateslv.com', 'http://www.kildares.ca', 'http://www.tablewinemadison.com', 'http://oohlalash.ca', 'http://www.ezpawn.com', 'http://www.starbucks.com/store/9298/', 'http://www.gnc.com', 'http://conniesjewelrygalleryandclockshop.com/', None, 'http://www.sushikanda.com/laval.html', None, None, 'http://www.quilles440.com/index_files/Page455.htm', 'http://www.oreillyauto.com', 'http://www.maxandermas.com/locations/333038', 'http://www.abreedapartanimalhospital.com', 'http://www.harbsauto.com', 'http://www.imagepluslasvegas.com', None, None, 'http://www.karachikitchen.com/location/brampton', None, 'http://www.flikandcompany.com', 'http://Skingri-la.com', 'http://www.theperfectshadesalon.com', None, '1592077170', None, None, 'http://www.calgaryautoquest.ca', '1592077203', None, None, None, 'https://www.beautybrands.com/store-locator/ahwatukee.do', 'http://www.rochestersdining.com', None, 'http://www.riverset.com', 'http://www.alohacardiology.com', None, 'https://www.ttc.ca/Subway/Stations/Wellesley/station.jsp', 'http://www.restaurantmissitalia.com/fr', 'http://www.charliesbarkpark.com', 'http://www.heartcenterofnv.com', 'http://www.networkautomotive.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.thaidtogo.com', 'www.edelweissimports.com', 'http://www.mcnallypianomovers.com/', 'http://www.daybreaknewmarket.com', 'http://www.wealthsimple.com', None, 'http://algobertoslasvegas.onlunchroom.space', 'http://phoenixcarnival.com', 'http://cabanatanolympia.business.site', 'http://www.bayrischerhof.biz', 'http://challengeyourgame.com/', None, None, None, None, None, 'http://www.phtowers.com/', 'http://www.raniasgrill.com/', None, 'http://www.dibellas.com/club-bread?source=yelp', '1592078201', 'http://www.burritoboyz.ca', None, 'https://locations.jimmyjohns.com/oh/southeuclid/', 'http://rusanscharlotte.com', 'http://www.brooklinbakery.ca', 'http://dukepubs.ca/westminster', None, 'http://www.regmovies.com/Theatres/Theatre-Folder/Regal-Severance-Town-Center-Stadium-14-822?utm_medium=social+network&utm_source=yelp&utm_campaign=yelp+-+description&utm_term=Regal+Cinemas+Severance+Town+Center+14', 'http://www.longjohnsilvers.com/', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', '1592078324', None, 'https://www.ahn.org/location-detail?name=wexford_Health_%21_Wellness_Pavilion', 'http://www.legendswarehouse.ca', None, 'http://www.gordonsjewelers.com', 'https://www.mukujapaneseramen.ca', 'http://www.kyreneinternalmedicine.com', 'https://www.presleyuptown.com', 'http://www.rapaczdental.com', 'http://www.ethicalaz.com/moquinosbodyandpaint', '1592078404', 'http://www.reesors.ca/', None, 'http://www.absorental.com', 'http://isthmustattoo.com', 'http://azhairdoctor.com', 'http://www.natalyapanchenkostudios.com', 'http://www.thrifttown.com/', 'http://www.lecharmeroses.com', None, 'https://www.northpointecle.com', None, None, 'http://robraymagic.com/', 'http://www.busybuggy.com/', 'https://www.lvhomeinspection.com', 'https://store.vioc.com/nv/las-vegas/9631-trailwood-dr-89134-HC0003', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.arsenal201.com', 'http://www.aplusnanniesinc.com', None, 'http://www.waypointvacation.ca', 'https://www.wastedgrain.com', 'http://www.analogcoffee.ca', 'http://www.chandlerneurologyandsleep.com', 'http://rockinrollssushi.com', None, 'http://www.ymcacharlotte.org', 'http://www.pohlenv.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYZXR', '1592078668', 'http://www.myoworksvegas.com/', 'https://www.cottonlanecitrus.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.villagerapartments.com', None, None, None, 'https://shop.nordstrom.com/c/wedding-suite-locations-beachwood-place?utm_source=yelp&utm_medium=organic&utm_campaign=wedd&utm_content=11227&utm_channel=lcl_brand_p&sp_source=yelp&sp_campaign=wedd', 'http://vegascaralarm.com', 'http://www.lubestop.com/Locations/Solon', 'https://www.fatcatsfun.com/gilbert', 'http://www.pinnaclevistavilla.com/', 'http://hharizona.com', 'http://www.sinatralv.com', 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'http://www.mlsalonspa.com', 'http://www.xplozone.com', '1592078860', None, 'http://bodyworksplus.com', 'https://circlecentreforhealth.com', None, 'http://www.hyatt.com/gallery/torphannon/kosher.html', 'http://www.halalbiryanihouse.com', None, 'http://longoortho.com/', 'http://www.emeraldsuites.com', 'http://www.gophoit.com/index.htm', 'http://www.sherwin-williams.com/store-locator/paint-store/Charlotte/NC/2164', None, 'http://www.attentiontoautodetail.com', 'http://www.club120.ca', 'http://www.onereddoorhudson.com/', 'http://pittsburgh.pirates.mlb.com/pit/ballpark/pncpark-events/venues/brewing-hall/', None, 'http://www.sunburstsmokeshop.com', None, None, 'http://www.DrJoyceChang.com', 'http://www.adogsbestfriendlv.com', None, 'http://www.jumpforfun.com', 'http://www.gonsteadperformance.com/', 'http://www.sunkissedtanninglv.com', 'https://www.purebarre.com/location/brecksville-oh', None, 'https://www.moderntortilla.com', None, 'http://bagelnash.ca', None, 'http://www.elevationweightloss.com', 'http://www.homegoodstore.com', None, None, None, 'http://www.frenchconnection.com', 'https://thirteen55motorsports.com', None, 'http://www.biltmorelifestylesre.com', None, 'https://megafurnitureusa.com', None, 'http://brittneywinklerhair.com', 'https://www.noons.co', 'http://www.eurocarsdetail.com', None, 'http://www.nailbrite.ca', 'http://worldofbeer.com/Locations/Gilbert', 'http://smartbuyavenue.com', None, '1592079287', None, 'http://www.pawsnclawsvet.com', 'https://www.smartstyle.com/en-us/locations/nv/las-vegas/located-inside-walmart-3473-haircuts-3334.html', 'http://www.christownspectrum.com', 'http://www.rebarn.ca', 'http://www.champps.com/', None, 'http://www.thepavilionsattalkingstick.com', 'http://www.merriperry.com', 'http://www.willowsdesign.com', None, None, None, None, None, 'http://www.dollartree.com', 'http://labambaburritos.com/', None, 'https://locations.teavana.com/us/az/scottsdale/7014-east-camelback-rd.html', 'http://whiskyattic.com/', 'http://apexendolv.com', 'https://locations.dennys.ca/AB/CALGARY/248179', 'http://www.alphadrone.com', 'http://sushibarkim.com/index.php', 'https://adamsplacerestaurant.wordpress.com', None, 'http://camelbacktailoringandalterations.com', 'http://www.firedupbbq.com', 'https://calgarycanine.com/auburn-bay/', None, 'http://www.oneshotinstallation.com', 'http://donutconnection.com/', 'http://www.goodfaithcasting.com', 'https://lvccld.org', None, 'https://www.sdi.edu', 'http://www.rslcs.org', 'http://www.coinlesslaundry.com/alma-school.html', None, 'http://www.oggiristorante.net/', 'https://www.ph216.com', 'http://www.shopnsavefood.com', None, 'http://www.manchuwok.com/', 'http://billskupa.com', 'http://www.cellphonerepaircomputerrepairlasvegas.com/', 'https://www.smartstyle.com/en-us/locations/nv/north-las-vegas/located-inside-walmart-2592-haircuts-3346.html', 'http://www.zernichlaw.com', 'http://www.greatdjmusic.com', '1592079719', 'https://www.redshoeslv.com', 'http://www.brighton.com', 'http://www.breadandsalt.ca', 'http://www.verizonwireless.com/stores/ohio/westlake/westlake-202670/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://zeroissuebeer.com', 'https://www.corepoweryoga.com/free-week?utm_source=yelp&utm_medium=cta&utm_campaign=chandler-yelp-free-week', 'http://www.ljkusina.com', 'https://www.kungfutea.com', None, None, 'https://www.discounttire.com/store/az/scottsdale/s/1284?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://desertridgemusicacademy.com', 'https://www.popeyes.com', 'http://www.martislounge.com', None, None, 'http://lylesjewelryandcoin.com', '1592079878', None, 'http://www.luckyhuelv.com', None, 'http://www.colonyroofing.com', None, None, 'http://www.mentorsportsbar.com', 'https://www.randrrefrigeration.com', 'http://kaizenautocare.com/body-shop-phoenix-az/', 'http://www.dsw.com', None, None, 'https://www.pristinemobiledetailaz.com', 'http://frysfood.com', None, 'http://www.hotdeallio.com', 'http://locations.michaels.com/NC/GASTONIA/5006/', 'http://www.enochturnerschoolhouse.ca/', 'http://AutoNationSubaruScottsdale.com', None, None, 'https://www.mezu.ca', None, 'http://racquetclubscottsdaleranch.com', 'https://www.stbank.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://kitcarson.snappages.com/', None, 'http://manchuwok.com', 'http://affinityplumbingaz.com', 'http://www.pizzadepot.ca/locationDetails/21', 'http://www.keratinspecialist.com', None, 'http://www.sincityvapor.com/', 'https://www.thebrick.com/pages/store-mississauga', 'http://www.villasatgreenvalleyapts.com', 'http://sojoramenlasvegas.taverndesigner.site', '1592080235', None, 'http://www.timhortons.com/', 'http://www.ahappydogwash.com', None, 'http://www.lids.com', '1592080274', 'http://www.johnrobertsspa.com', 'https://northscottsdale.tapoutfitness.com', 'https://1transmission.wixsite.com', 'http://www.momdocwfw.com/mercy-gilbert', 'https://my.clevelandclinic.org/locations/directions/225-brunswick-family-health-center', 'http://charmeck.org/city/charlotte/Airport/Parking/Pages/CellPhoneLot.aspx', 'http://idealdentistryaz.com/', 'http://www.DNDC.ca', 'http://www.crescentphx.com/', None, 'http://www.regency.edu/campus/tinley-park-illinois.aspx', None, None, None, 'http://www.coffeetime.com', None, 'http://www.connectionsarizona.com', 'https://mollymaid.com/west-glendale-avondale', 'https://www.baskinrobbins.com', 'http://www.maidbrigade.com/az/phoenix', 'http://azcyclery.com/', 'https://stores.guitarcenter.com/las-vegas-lessons?source=4GOA5AJBB', 'https://rvonthego.com/arizona/paradise-rv-resort/?utm_campaign=Yelp_Directory_Listing&utm_medium=directory&utm_source=Yelp', 'http://www.ranchomanana.com/', 'http://www.stratospherehotel.com', 'http://michaeldaily.com', 'http://www.winnerstraining.com', None, 'http://monkpatisserie.ca', 'http://www.mypopcornheaven.com', 'http://www.mc3salon.com/', 'http://www.andersonentertainment.co', 'http://biggunenterprisesllc.com/', None, 'http://www.glenatmesa.com', 'https://www.hilton.com/en/hotels/msnbjhx-hampton-suites-madison-downtown/?SEO_id=YELP-HP-MSNBJHX', 'http://concordncattorney.net', 'http://www.curetondentistry.com', 'http://eastsidemarios.com', None, None, 'https://www.theflowercart.org', 'http://www.serenevegas.com', 'http://www.outback.com/locations/az/surprise', None, None, 'https://haroldstudio.com', None, 'http://alsshoerepair.com', 'https://www.garciadiamondsusa.com', None, None, 'http://bodyworkscenter.net/', 'http://www.barbershopjack.com', None, None, 'http://www.charlotterestore.org', 'http://www.chopstixhenderson.com', 'http://www.billysroadhouse.com', 'http://www.fieldclub.org/', None, 'http://www.xcitemobilegamingcharlotte.com', 'http://www.dentalhygienegroup.ca', 'https://mbveuro.com', 'http://oaksautotruckservice.com/', 'http://www.securitytitle.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/nj/colonia-2471.html?mcid=iyp:8368400', None, None, 'http://www.thecortinaapartments.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044282967372&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.gnc.com', 'http://www.nexuscarelv.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.agramgardencentre.ca', 'http://www.petvethospitals.ca/morningside-pet-hospital/', 'http://www.restomeridiana.com', None, 'http://www.face2toe.com', None, 'http://teslamotors.com', None, 'http://www.lvsmiles.com', 'http://www.starbucks.com', None, 'http://www.theweavescene.com/', None, 'http://athoriagames.com', 'http://america-sunshine-maintenance.com', 'http://www.arizonatmjpain.com', 'http://www.redrocklasvegas.com/dining/cabo.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=RR&utm_content=RR-PROPERTIES&utm_campaign=RR-YELP', 'http://www.christysnursery.com', None, 'http://www.insuranceofvegas.com/?cmpid=wj16_blm_0006', None, '1592081138', 'http://www.discount-drugmart.com', 'http://www.arrowmedaz.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.Rooneys.Market', 'http://www.vitalant.org', '1592081204', 'http://rsecatering.com/', 'http://rebozos.ca', 'https://www.bbbuggy.com', 'http://www.villageinnrestaurants.com', 'http://www.craigroadcarwash.com', 'http://freshsifoods.com', 'http://www.gogibson.com', 'http://www.fatburger.com/home/', 'http://www.pitapit.com/', 'http://www.aumaubree.com/', 'http://oxygenrestaurant.ca', 'https://www.freshthyme.com/store/oh-westlake/', 'http://www.tinderboxcigars.com', 'http://www.brooksidelawnservice.com', 'https://www.truecocktailsbites.com', None, 'https://www.zaxbys.com/locations/nc/denver/7179-highway-73/', 'http://www.lovethreadingbar.com/', None, 'http://truearizonaliving.com', 'http://pigandduke.com/', 'http://ignite.fortis.edu/?BID=3038&VID=SR_CID-000045038666', 'http://www.moes.com', '1592081418', 'http://www.esourceparts.ca', 'http://www.qualityinn.com/hotel-madison-wisconsin-WI089?source=pmfyelp&pmf=yelp', 'http://www.satisfiedfrog.com', None, 'http://www.fceglobal.com/', 'https://www.lifetimefitness.com/en/programs-and-events/lifestyle/lifespa/arizona/goodyear-palm-valley-lifespa-hair-skin-nail-spa-facilities.html', 'http://phoenixmoversunlimitedinc.com', 'https://curaleaf.com/stores/pl/curaleaf-az-central-85040/', 'http://www.arbys.com/', 'http://www.towcompanymesa.com', 'http://thepurrrfecttouch.com', 'https://dutchbros.com/?utm_source=las-vegas&utm_medium=yelp&utm_campaign=website_las-vegas', 'http://www.linatranstudio.com', None, 'http://www.terryalixsalon.com', 'http://www.cupcrazed.com', 'http://shymcgrath.com', 'http://asiannoodletogo.com', None, '1592084427', None, 'http://www.themandrakeclt.com', None, 'http://www.AppleOne.com', 'http://www.originalhoagieshop.com/', None, 'http://www.modernrevivalevents.com/', None, 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, None, 'http://calibercollision.com/locate-a-caliber-collision-center/las-vegas-downtown/', 'http://barksbubblesgrooming.com/', 'http://www.customalterationsbyanne.com', 'http://www.kingcoleducks.com', 'http://climb-cleveland.com', None, 'http://my.clevelandclinic.org/staff_directory/staff_display.aspx?doctorid=4363', 'https://www.americancampus.com/student-apartments/az/tempe/vista-del-sol', 'https://www.perfectpalmssalon.com', 'http://www.finaticsaquarium.com', 'http://www.jrsweets.ca', '1592084616', 'http://www.acuraeast.com', 'http://www.grillcity.com/nevada/bbq-grill/', None, None, 'http://www.cowlickhairsalon.com', None, None, 'http://www.magnoliaroom.com', 'https://www.wyndhamhotels.com/hotel/52962?tel=18003876085&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52962', 'http://www.pro-edgecutleryaz.com', 'http://www.elitedestinations.net', 'http://www.fourstarsushi.ca', 'http://clickserve.dartsearch.net/link/click?lid=43700006430791267&ds_s_kwgid=58700000444977937&ds_url_v=2', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.attractagency.com', 'http://www.valleyrentabox.com', 'http://Www.garbarinos.com', 'https://www.swoon.cc', None, None, 'https://www.lakenormannailsbar.com', 'http://www.studiofontana.ca', 'http://www.e-zrentacar.com/locations/las-vegas-airport', None, 'https://hpautorepair.com', None, None, 'http://www.pizzahut.com/', 'http://thesman.com/las-vegas-meadows', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://affordableweddingminister.com', 'https://www.drnnadi.com', None, None, 'https://www.autozone.com/locations/pa/wilkinsburg/208-penn-ave.html', None, None, None, None, 'https://stores.dickssportinggoods.com/nv/las-vegas/1266/', None, 'http://www.themadisonapartmentliving.com', None, None, 'http://www.entsoa.com', 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', 'http://www.harborcrestapts.com', None, 'http://www.bottega.ca/index.php', 'https://badaxethrowing.com/axe-throwing-vaughan', 'http://www.rockincleveland.com', 'http://www.enatethiopianrestaurant.com', '1592085010', None, 'http://www.thecamptc.com', 'http://portsalonandspa.com', 'http://www.tso.ca', '1592085055', '1592085064', None, 'http://www.jimboys.com', None, None, 'http://workshopcalgary.com', None, None, None, 'http://charandarestaurante.com/', 'http://www.pattersonroofingco.com', 'http://www.armaantasteofindia.com', None, 'http://www.citywidetowing.com', 'https://www.officedepot.com/storelocator/nv/las-vegas/office-depot-3351/', 'https://www.hilton.com/en/hotels/lasethx-hampton-suites-las-vegas-airport/?SEO_id=YELP-HP-LASETHX', 'http://www.achristmasstoryhouse.com', 'http://www.cannabition.com', None, '1592085206', 'https://www.sleepcountry.ca', 'http://www.tenfeettall.ca/', 'http://myezroofsolution.com', 'http://mellomotive.com', 'http://www.sciullispizza.com', 'https://resurrectioncarpetcleaning.com', 'https://www.dodsonbros.com/location/charlotte-south/', 'http://www.russellchiro.net/', None, 'https://www.goldsgym.com/chandler-az/', 'http://fortyrodroadhouse.com', 'http://www.hendricktoyotaconcord.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, 'https://petmedic.vet/hospitals/providence-pet-hospital/', 'http://www.thewholeplate.ca', 'http://www.richardsautoclinic.com/', None, 'http://www.melonhead.ca', 'http://www.harryrosen.com/store/Vaughan%20Mills', 'http://www.rpspasta.com/', 'http://www.themaidcrusade.com', 'https://www.pintocashforgold.ca', 'http://www.willemandjools.com', 'http://www.abrazohealth.com/facilities/ArizonaHeartHospital/home.aspx', None, 'http://www.surefireinstitute.com', 'https://www.choicehotels.com/pennsylvania/coraopolis/comfort-suites-hotels/pa327/rates', 'http://www.kidszonedurham.ca', None, None, 'http://www.kennywood.com', 'http://www.avriorx.com', 'http://www.marcelwaffles.com', 'http://crumblcookies.com', 'http://www.thaiexpress.ca/', 'http://www.justincrain.com?cmpid=RV5O_BLM_0006', 'http://bestwestern.com', None, 'https://www.peoriapines.com', None, 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.aquarelarestaurant.com/Dominican_Restaurant_Toronto_-_Aquarela/Welcome.html', 'http://www.desertbloomskincare.com', 'https://locations.cibc.com/on/toronto/03902?q=Toronto%2C+ON+M4K+1P7&loc=M4K+1P7%2C+ON', 'http://masterwerks.com/', None, 'http://robertostacoshop.com', None, None, None, 'http://www.weprintvegas.com', 'https://angel-dentalcare.com', 'https://www.urbanoutfitters.com/en-ca/stores/downtown-toronto', 'https://www.repairngo.com', 'http://www.oasisislandvegas.com', 'http://www.airconditioningrepairinphoenix.com/', 'http://www.paloverdedentistry.com', 'https://www.benjerry.com/davidson', 'https://www.anytimefitness.com/gyms/690/Fitchburg-WI-53719/?utm_source=yelp&utm_medium=local', None, None, 'http://southwestcajunfest.com', 'http://locations.scootaround.com/lasvegas', 'https://www.tidedrycleanersaz.com/locations/e-frank-lloyd-wright-blvd', None, 'https://www.fragranceoutlet.com/pages/fragrance-outlet-tanger-outlets-westgate-114', 'https://www.exxon.com/en/gas-station/tempe-az-tholdings2165-321058', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.mygym.com/henderson', 'http://www.southwestbikes.com', 'http://www.thepittstopbbq.com/', 'https://www.missionmontessori.com', 'http://www.chandlerjewishpreschool.com', None, 'http://www.chinaonepgh.com', None, 'http://www.pandaexpress.com/', 'http://www.davisvilleparkvets.com', 'https://lawnlove.com/charlotte-lawn-care', None, 'http://www.stitchit.com', None, 'http://kidscuckoosnest.com', None, '1592085834', None, 'http://www.coolsummercamp.ca', 'http://www.phxskin.com/', 'http://www.magnetitecanada.com', None, None, None, None, 'http://www.diamondbackcleaners.com', None, 'http://asurology.com', 'http://www.maxstudio.com', 'http://thefloatfactor.com', 'http://www.hairdistrict.ca', 'http://www.republicservices.com/', None, 'https://order.iamaflowerchild.com', 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', None, 'https://secondcup.com', None, 'http://www.altour.com', 'https://www.gogeisel.com', 'http://www.sincitytint.com', 'http://www.landeseye.com', 'http://www.superstore.ca', None, 'http://www.academyofnails.net', 'http://www.stevenkretchmer.com/', None, '1592086116', None, 'http://www.cineplex.com/Theatre/cineplex-cinemas-scarborough', 'http://www.crowbarlasvegas.com', 'http://azwindowreplacement.com', 'https://www.jrsmulchsales.com', 'http://www.mywellnessplace.com/', '1592086195', None, 'http://www.summerwindsnursery.com', 'http://palace.awardspace.com/', None, None, 'http://www.danieljsalons.com', 'http://www.southhillsortho.com', None, None, 'http://www.blackcatespressobar.com', None, 'http://jtsconcrete.com', 'http://www.magnolialanelv.com', 'http://www.bergstrommadisonchevrolet.com', 'http://www.thevillagedentalcenter.com', None, 'http://www.walterstation.beer', None, 'https://www.officedepot.com/storelocator/nc/kannapolis/officemax-6480/', 'http://www.aztacoshack.com', None, None, None, 'http://www.tranestudio.com/', 'http://www.gabroy.com', 'http://www.doughnc.com', 'http://www.maxmcqs.com/', 'http://www.popsphotoandtrophy.com', 'http://www.jaysatthelake.com', 'http://summerlinbarbershop.com/summerlinbarbershop/', 'http://www.destihl.com/', None, 'http://www.lasvegasskinspa.com/', 'http://www.northhillstoyota.com/', 'http://www.drlomboydc.com', None, None, None, None, 'https://www.hungryhowies.com/store/hungry-howies-00829?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=00829&utm_content=Website_Link', None, None, None, None, 'https://www.kyrene.org/apr', 'http://www.hoopershealth.com/store-information/store-information-brampton/', 'http://www.cosmoprofbeauty.com', 'https://www.unlvmedicine.org/clinic/internal-medicine-0', 'http://www.hairyt.com', '1592086614', 'https://mettlercenter.com', 'http://stlouiswings.com', 'https://bhmanagement.com/communities/az/phoenix/cielo-apartments-1794/', 'http://woodchesterautoglass.com', 'http://castironwaffles.com', 'http://www.caronpartners.com', 'http://www.debruinchiropractic.com', 'http://proshops.bowlersmart.com/locations/nevada/bowlersmart-henderson-pro-shop-inside-strike-zone-sunset-station', None, 'http://www.windowprollc.com', 'http://www.mildreds.biz/', 'http://theupsstore.ca/45', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Phoenix/AZ/8160', 'http://www.hennaflorist.ca', 'http://murphyslaw.pub', 'http://website-3132959076228282333140-nailsalon.business.site', None, 'http://cksbbqlv.com', 'http://thewhitebrickkitchen.com/', 'http://www.teamonsiteconcrete.com', 'http://www.eighty8family.com', 'http://www.romaninconcrete.com', 'http://www.safelite.com', 'http://madison-west.certapro.com/?source=Yelp', 'http://sassygurl.ca', 'https://www.azspiritualmedium.com', 'http://yourbuybacks.com', None, None, None, None, None, None, None, 'http://www.bhfcfinancial.com', 'http://www.angelblessingslv.com/', 'http://www.stonebowlkorean.com', 'http://www.liveattheurban.com', None, '1592086982', 'http://www.bigotires.com/regions/Nevada', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', None, None, None, 'http://phoenix-9thandbell.titleboxingclub.com/', 'http://www.thevineyard.ca/', 'https://www.anytimefitness.com/gyms/2195/Richmond-Hill-ON-L4E-3L6/?utm_source=yelp&utm_medium=local', 'http://www.mygreek2go.ca', 'http://stephenfolsondds.com', None, None, 'http://keyspleaselocksmith.com', None, 'https://www.woodbridgeinteriors.com', 'http://www.powernailspa.com/', 'https://www.homedepot.com/l/N-Scottsdale/AZ/Scottsdale/85260/472?cm_mmc=seo-yelp&', 'http://www.hair2000.ca', None, None, 'http://www.bestchickenwings.com/', 'http://www.faziopizza.com', 'http://www.sprint.com', None, 'http://shusalon.com/', 'https://www.lawkm.com', 'http://www.thetrteam.com/?agent=mahalalel-leli', None, 'http://5starpdrllc.com', None, None, 'http://www.bennettmedical.com', None, 'http://www.sulwer.com', 'http://www.ryerson.ca', 'https://locations.theupsstore.com/wi/fitchburg/2935-s-fish-hatchery-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.modernsalonandspa.com', 'https://scottsdaledaycare.com', 'http://www.venetian.com/casino/race-and-sports-book.html', 'http://www.waterway.com', 'https://parteeputt.com', 'http://www.vapelandphx.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.preferredpublicrelations.com', 'http://www.flashlightfactory.net', '1592087399', None, None, None, 'http://cloisterhoney.com/', 'https://horizoneye.com', 'https://www.autozone.com/locations/nv/las-vegas/1021-w-owens.html', 'http://www.audiprestigeddo.com', None, 'http://www.hpb.com/041', 'https://stores.joann.com/nv/henderson/2015/', '1592087498', 'http://www.buldak.ca', 'https://www.westwindintegratedhealth.com', 'http://www.rtosullivans.com/', 'http://www.thechefandyou.com', 'https://locations.checkers.com/nv/north-las-vegas/2720-n.-las-vegas-blvd/?utm_source=Yelp', None, 'http://magnoliapetwc.com', None, 'http://subway.com/', 'http://gotrmc.org', 'http://www.quiktrip.com', 'http://capitalacupuncture.janeapp.com', None, 'http://www.firehousegrillingco.com', 'http://www.threesheetsdtlv.com', None, None, 'http://www.cccnevada.com/doctor/josette-e-spotts-md-facs/', 'http://www.greatclips.com/', 'http://www.kokobellibagel.com', 'http://jpsushi.ca', 'http://protechautogroup.com', None, None, None, 'http://www.teresaspizza.com/richfield', None, None, '1592087749', None, 'http://www.donsbees.com', 'http://LugariPet.com', 'http://popeyeschicken.ca', 'https://arizona-firearms.com', 'http://www.paramountbutchershop.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Phoenix-AZ-85027/858046/', 'https://www.hertz.com/rentacar/location/unitedstates/northcarolina/charlotte/CLTT12', 'https://www.greendrop.com/calgary', None, None, None, 'http://www.bluelinegrille.com', 'http://www.shakeys.ca', None, 'https://www.lattaplantation.org', 'https://www.cybermax-pc-phone.repair', 'http://www.latan.com', None, None, 'http://web.archive.org/web/20071110183536/www.city.cleveland.oh.us/around_town/city_highlights/landmarks/freestamp.html', 'https://www.jerseymikes.com/3128', None, 'http://www.thecopperpalmsapartments.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.colorglolasvegas.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.thepotatobarn.com', 'https://northernsmokes.com', 'http://www.pandmpizzainc.com', 'http://www.brakemasters.com', 'http://www.towercitycenter.com/', 'http://www.crazymocha.com', 'http://www.bloordovercourtappliances.com', None, None, 'http://www.carmellaspizzagrill.net', 'http://www.thebluefoodtruck.com', 'http://www.kumon.com/PHOENIX-DESERT-RIDGE', 'http://www.creativeleather.com/', None, 'http://www.lecreuset.com/stores/charlotte-premium-outlets-charlotte-nc', 'http://www.ahwatukeechurch.com', 'https://stores.advanceautoparts.com/pa/carnegie/1011-washington-ave', 'https://charlotte-bjj.com', 'https://www.galleriavillage-apts.com', None, None, 'http://www.theairteamlv.com', 'http://www.northcentralsportsbar.com', None, 'https://epdpestcontrol.com', None, '1592091673', None, 'http://www.innerdivafitness.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.waltsbbq.com', 'http://www.lpoecharlotte.com', None, 'https://maps.rbcroyalbank.com/locator/searchResults.php?t=6702', None, None, None, 'https://www.campbowwow.com/cuyahoga-falls/', None, 'http://www.asba.com', 'http://cccwindowcleaning.com', 'http://metrodiner.com', 'http://www.sunnynutrition.com', None, 'https://www.jamescheese.ca', 'http://mattas.com', None, 'http://www.autoglassnow.com/phoenix', 'http://www.auntieleechampaign.com/', None, 'http://evolvedancecenter.com', 'http://www.luigismadison.com/', None, 'https://www.noodlebarphx.com', 'https://elitefloorandfurniturecleaning.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.yogibar.ca/', 'http://www.centerforbehavioralhealth.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://ineffablecaphe.com', 'http://www.thecitycafeaz.com/', 'http://www.floresfamilydental.com', 'https://www.thejoint.com/north-carolina/mint-hill/mint-hill-12029', 'http://sweetrepublic.com', 'http://www.thecaptainsboil.com?utm_source=Yelp&utm_medium=Click&utm_campaign=ON_5313Yonge&utm_content=mainlink', 'http://www.blueheel.ca/', 'http://sweetiefry.com', 'http://www.aecne.com/', 'http://www.carriepatriciastudios.com', None, 'http://www.platosclosetstrongsville.com', None, None, 'http://thegreatgreekgrill.com', None, None, 'http://www.wilderfood.ca', '1592092073', None, 'http://trunightclublasvegas.com', 'http://www.pouroliveoil.com', None, 'http://www.massagebar.com/pit.htm', 'https://penrose.edu/student-salon-and-spa', 'http://www.foodbasics.ca', 'http://www.t-mobile.com', 'http://www.azmulticare.com', None, 'http://www.yourbrowbar.com', 'https://www.advanceamerica.net', 'http://www.blossomdayspa.com', 'http://www.anticaosteria.ca/', 'http://mtholly.us/park_and_facility.php?3', None, '1592092204', 'http://www.ezpawn.com', None, 'http://wigwamcreekdental.com', None, None, 'http://enchantedcastle.info/', 'http://rosenailbarsummerlin.com', 'http://www.greatlakesbeer.com', 'http://habibizcafe.com', 'http://www.villagediscount.com/', 'http://www.tocayopgh.com', 'http://www.shuku2salon.ca', None, 'https://www.britishairways.com', None, 'http://www.elmolinocafe.com/', 'http://www.vedasfitness.com', 'https://www.infuzelv.com', None, 'http://www.winghouseandsportsbar.com/', 'http://www.eyewearatthehamptons.com', None, 'http://www.funkutzkidshairsalon.com/', '1592092400', 'http://expresssmogandmotorworks.com', None, 'https://www.affordablegaragedoor.biz', 'https://www.nhsteelecreekfamilymedicine.org', 'http://www.dhl.ca', 'http://www.azcrittergetter.com', 'http://www.smokesweinerie.com/', 'http://www.pizzapizza.ca/', 'http://www.southlandvisioncare.com', None, None, None, None, None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/1/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'http://iflyelite.com', 'http://www.crossfits3.com', None, 'https://www.argentaapts.com', 'http://www.guitaror.ca', 'http://hotyogaoncrowfoot.com/', 'http://Localocksmiths.com', 'https://www.tillys.com/store?StoreID=0044', 'http://www.woodbineanimal.ca', 'http://www.shilohcommunity.com', None, 'http://www.cochranvwnorth.com', None, None, 'http://www.fulloutperformance.com', 'http://caveatemptorlv.com/', '1592092620', 'http://www.bigalsbargrill.ca', 'http://www.theglostudio.com/', 'http://www.thematka.com/', 'http://thetkto.com/', 'http://www.pestcontrolguaranteed.com', None, 'http://belmontdrivein.20megsfree.com', 'http://marcellospastagrill.com/', 'http://www.roysrestaurant.com', '1592092690', 'http://aldi.us/index_ENU_HTML.htm', None, 'http://petersenicecream.com', None, None, None, None, 'https://www.hilton.com/en/hotels/phxclhw-homewood-suites-phoenix-chandler-fashion-center/?SEO_id=YELP-HW-PHXCLHW', None, 'http://www.apria.com', 'http://www.bk.com', None, 'https://www.caesars.com/caesars-palace/things-to-do/cleopatras-barge?utm_campaign=Yelp&utm_source=yelp&utm_medium=local&utm_term=CocktailCabaret&utm_content=entertainment', 'http://www.keshaschairlv.com', None, 'http://www.thesweetspotcleveland.com', None, 'http://www.3030southwater.com', 'http://www.presotea.ca', None, 'https://www.biscuitsrestaurants.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sunscreensbymark.com', '1592092906', None, None, 'https://www.ahnenchiropractic.com', None, 'http://www.sincerelyyogurt.com', 'http://vipbilliards.ca', 'http://www.roosterscharlotte.com/', None, 'http://www.autosood.com/fr/', 'https://www.capriottis.com', 'http://www.starbucks.com/store/11135/', None, 'http://www.americanapparel.net', 'http://www.searsucker.com/scottsdale/', 'http://www.gogofresco.org', 'http://www.frankstowngardens.com/?srccode=yelp_track', None, 'https://local.fedex.com/az/scottsdale/office-2256/', 'http://www.sheahomes.com/newhomes/charlotte', 'http://www.integratedmha.com', 'http://arcadia.redemptionaz.com/', None, 'http://www.elevationoncentral.com', 'http://www.gilbertappliancestore.com', 'http://www.firelightbarn.com', None, 'http://www.southcharlotte.youvegotmaids.com', 'http://valleyprocessservice.com', None, None, None, 'http://www.foreverdiamondsjewelers.com', 'http://newmanprop.com', 'http://www.platosclosetlyndhurst.com', 'http://drappliance.business.site', 'http://99only.com/store/Las-Vegas---Beltway/', 'http://www.azbreastfedbabies.com', 'http://www.mobileaxs.ca', 'http://firstpictureframe.ca', None, None, None, 'http://advanceddermatologynevada.com/about-us/jan-fu-md-phd-faad/', None, 'http://www.sccarcenter.com', 'https://www.choicehotels.com/nevada/boulder-city/quality-inn-hotels/nv062/rates', None, 'http://www.animalkennelcare.com', 'http://www.mccandlesscrossing.com/', None, 'http://www.kmart.com/stores/pennsylvania/pittsburgh/3529.html', 'http://www.blomedry.com', 'http://www.steve-n-kids.com', 'http://www.brickandbarleyaz.com', None, 'http://www.residoor.com', None, 'http://www.massageheights.com/matthews-festival', 'https://pranaivtherapy.com', None, 'https://dillonmcdonald.vipmtginc.com/apply/', '1592093393', None, 'http://www.spamadison.com', None, None, 'http://www.drsibel.com', 'http://www.MODERNMILK.com', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', None, 'http://www.tacotimecanada.com/locations/alberta/calgary/deerfoot-meadows', 'http://www.apple.com/retail/northlakemall/', 'http://www.starbucks.com/store/10759/', 'http://www.sharkys.com', 'http://www.riggsroadvets.com/', 'http://www.head2toesmassage.com', None, 'https://www.ikea.com/us/en/stores/las-vegas/', 'http://Geeksquad.com', None, 'http://www.nevadaheadache.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.omwest.com', '1592093574', None, 'https://nolmsted.onyxhealthclub.com/contact/', None, 'http://westgatenailsspaglendale.com', 'http://www.stjosephphoenix.org', 'https://www.dickblick.com/stores/ohio/cleveland-heights/', None, 'https://www.familydentalcharlotte.com', 'http://www.swearlv.org', 'http://www.shawarmabox.ca', 'http://Www.summitpsychologicalservices.org', 'http://www.oldnavy.com/products/peoria-az-store-6364.jsp', 'http://www.myrosatis.com/store-details/Mesa%20Greenfield', 'http://www.mamarosespizza.com', None, None, 'http://www.underwrapslv.com', 'http://www.russellhendrix.com', 'http://www.wisconsinfencing.com', None, None, 'http://www.holatapasbar.com', None, 'http://www.autonationtoyotatempe.com/', 'http://www.shawarmaave.ca', None, 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', None, 'http://www.abcsforchildren.com', 'https://locations.sonicdrivein.com/az/tolleson/9552-west-van-buren-avenue.html', 'http://www.lackscleaning.com', 'https://www.psgdental.com/locations/precision-orthodontics-lyndhurst/', None, None, None, 'https://LimitlessGraphix.com', 'http://kachinapainting.com', None, 'http://www.fuchsiaspa.com', 'https://azfunandsun.com', '1592093870', 'http://www.deserttitle.com', None, 'http://www.trundlemanor.com', 'http://www.wrapsodygifts.com', None, 'http://tsujiri-global.com', 'http://www.sundevilbookstores.com', 'http://www.sophiespizza.ca', None, 'http://greenlifelawncare.net/', None, None, 'http://www.xoxolashes.com', 'https://www.freshcoatpainters.com/glendale-az', 'http://thesmokincue.com', None, 'http://www.centennialparkgolfcentre.com', 'https://hcpnv.com/clinic/la-canada-cardiology/', 'http://www.tanglesmadison.com', None, 'http://www.valleyartcenter.org', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.chatime.com', 'https://www.choicehotels.com/arizona/phoenix/econo-lodge-hotels/az191/rates', None, None, 'http://www.angrygorillamarketing.com', None, None, None, 'http://www.premierdentalal.com', 'https://www.americanfreight.com/?utm_medium=Yelp_Website_URL&utm_source=Yelp', 'http://www.bunkerhillgolf.com/bar_and_grille/', 'http://advancedcleaningscottsdale.com', 'http://www.ckokickboxing.com/las-vegas.html', 'http://www.portcityclub.com/', 'https://locations.davidstea.com/on-markham-167', 'https://www.target.com/sl/streetsboro/2157', None, None, 'http://thebutlerdidit.info', 'http://www.rootcandles.com', 'http://mistertransmissionnewmarket.com', 'https://www.oxifresh.com/locations/az/paradise-valley-carpet-cleaning/?utm_source=yelp&utm_medium=social&promo_code=YP5000', 'http://www.lasvegaspropertyvaluation.com', None, 'http://www.mcknight-kitchen.com', 'http://www.aulddubliner.com/', 'http://www.cabanaondumont.com', 'http://www.lubie.ca/', 'http://www.seventattoostudio.com', None, 'http://www.goldwaterbank.com', 'http://yang39y.wix.com/xiangweixuan', '1592094322', None, '1592094333', 'https://www.johnsonlarsendental.com', 'http://www.mcadenvillemotors.net', 'https://locations.churchs.com/az-scottsdale-685', 'http://mauidogrocks.com', 'http://www.socialeatery.ca', 'http://www.newstylenailshenderson.com', 'http://eosfitness.com/location/centennial?utm_campaign=vegas-centennial&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, 'http://www.nicholaselectric.com', 'http://www.appetitebooks.ca/', 'http://www.futureshop.ca/', 'http://storelocator.pepboys.com/pepboys/stores/glendale/63rd_and_bell/00779', None, 'http://www.phoenix.invisiblefence.com', 'http://www.wilddogdigital.com', 'http://www.verizonwireless.com/stores/nevada/las-vegas/katie-203158/', 'https://www.davescosmicsubs.com', 'http://www.cocoshairstudio.com', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'http://pjobrokerage.com', None, 'http://www.valleywidedoor.com', 'http://www.thaifootmassageaz.com', None, None, 'https://marketbrewingco.com', 'http://groutsmithcharlotte.com', 'http://www.bestfuneralservices.com', None, 'http://www.aactionmovers.com', 'http://www.mcsorleys.ca', None, 'http://www.camelviewpt.com', 'http://vaseklaw.com', None, 'http://www.shayacuisine.com', 'http://www.pizzapizza.ca', 'https://www.moesbarbershopaz.com', 'http://www.regencyheightsapartments.com', 'http://www.asiankitchencleveland.com', 'http://www.nationalspecialtypharmacy.com', 'https://www.spencediamonds.ca/location/vaughan-on/?utm_source=Yelp&utm_medium=Display', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', 'http://www.valleypaincenters.com', 'http://www.pipercleo.com', 'http://www.ashleysbeautylounge.com', 'http://mobiletecsmallengines.ca', 'http://www.zamrestaurant.com/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', None, 'http://www.partybusesinvegas.com', 'https://www1.shoppersdrugmart.ca', 'https://www.etiquettesystems.com', None, 'http://www.hardrockhotel.com/las-vegas-restaurants/goose_island_pub', 'https://www.absolutedental.com/locations/las-vegas/2301-e-lake-mead/', 'http://uptowncheapskate.com/charlotte', 'http://www.dontortaco.com', 'http://www.spbcnc.org', 'http://www.seeingdoublecle.com', 'http://www.dogpolitan.com', 'http://www.bistrosirwilfrid.ca', 'http://crispycrust.ca', 'http://www.mcgoverns.biz', 'http://www.cadencehealthcentre.com', 'http://www.besolasvegas.com/', 'http://mayaclubaz.com', 'https://www.occphx.net', 'http://www.gamestop.com', 'http://Www.remosbistro.ca', 'https://www.petsmart.com/stores/us/az/phoenix-store0131.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://www.periospecs.com', None, None, 'http://www.azsingingcrepe.com/', 'http://www.andlerdental.com', 'http://bocalocaburritofactory.com', 'http://www.azbestgaragedoors.com', None, 'http://www.lazeezshawarma.com/', None, None, None, 'https://www.rightautoandtruck.com', None, 'http://www.cielostudios.ca', 'https://kirinbuffet.net', 'http://www.beatleshowvegas.com', 'http://www.metroicare.com/', None, 'https://www.bowdog.ca', None, None, 'http://www.greentreeprinting.com', 'http://www.vientianepalacerestaurant.com', 'http://jep-landscaping.com', 'http://www.germanbreadbakerylasvegas.com/', None, None, 'http://www.maxwellspine.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.greatclips.com/', '1592100552', None, None, 'http://www.drtomgreco.com', 'http://www.deerbrookefarm.com', None, None, None, 'https://americanaddictioncenters.org', 'http://www.marriott.com/cmpts', 'http://www.myspace.com/sweetsandbeats', 'https://locations.jackinthebox.com/us/az/scottsdale/20385-n-hayden-rd', 'http://www.budsayathaispalv.com', 'http://www.coderedplumbing.com', 'http://www.abcbartending.com/vegas/index.htm', 'http://www.starbucks.com', 'http://www.silversagebeef.com', None, 'http://lunchboxcafeaz.com', 'http://www.onedaypittsburgh.com', None, 'http://coolblew.com', None, None, 'http://www.oreilyauto.com', 'http://www.lvchamber.com', None, None, 'http://www.owloptometry.com', 'http://www.approvedaz.com', 'http://www.horizonlaservision.com/', None, 'http://zulmamassagetherapy.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.vapeandco.com', None, 'https://goo.gl/2UEpRZ', None, 'http://www.322PizzaBar.com', '1592100862', 'http://www.blackmountainbmx.com', None, 'http://www.dunkindonuts.com', '1592100900', None, 'http://www.crystalrivergems.com', 'http://swimmingpoolslasvegasnv.com', None, 'https://satruck.org', 'http://www.nickscalgary.com', None, None, 'http://www.caldensebakery.ca/', None, 'http://www.azsprinklerpros.com', None, None, None, 'http://www.oakridge-kennels.com', 'http://www.qdoba.com', 'http://www.arizonalodgingexperts.com/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282947950&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.vegasconciergedoctor.com', None, 'http://www.teaspressa.com', 'http://mdrainlaw.com', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.dynastyirondoors.com', 'http://www.pineviewauto.com', 'https://www.extraspace.com/storage/facilities/us/ohio/kent/501645/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.crazymocha.com', 'https://www.royalautosales.ca', 'https://automotivediagnosticspecialties.com', 'http://www.greatescape-to.com', 'http://www.greatclips.com', None, 'https://rigatonys.com', 'http://www.sunrisemontessori.com', None, None, 'http://asavvyskincare.net', 'http://www.gabyetjules.com/', 'http://www.tcvintageaddiction.com', 'http://www.starbucks.com/store/9298/', 'http://www.CompleteLaserAZ.com', 'http://www.expressscreenprinting.com', None, None, 'http://www.tshirthell.com/', None, 'http://www.katkuphotography.com', 'http://www.greatclips.com', 'http://www.nutbrownschristmastrees.com', '1592101294', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3075+West+Liberty+Avenue-Dormont-PA-15216/storeid=4144?cid=ps_ylp_2020', 'http://www.mrshawarmaairdrie.ca/', None, 'http://www.harborddentistry.com', 'http://porterhouseto.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Pittsburgh-PA-15237/809064/', 'http://www.ihg.com/hotelindigo/hotels/us/en/scottsdale/phxin/hoteldetail', 'https://www.homedepot.ca/en/home.html?eid=LS_Google_7061', None, 'http://www.goldes.com', None, 'http://www.splitsville.ca/locations/woodbridge-bowling', '1592101389', None, 'https://www.target.com/sl/chandler-south/2747', 'http://www.worldscoutingmuseum.org', 'http://fuyaosupermarket.ca', None, 'http://www.scottsdaleaz.gov/parks/scottsdalemall', 'http://www.valleyinteriors.com', None, '1592101459', 'https://www.redradishcharlotte.com', None, 'http://www.matsudasushi.com/', None, 'http://www.starbucks.com/store/10759/', None, 'https://www.wellsfargo.com', 'http://www.stowglass.com', 'http://www.ramsey-auto.com/', 'http://mazzeolaw.com', 'https://www.tech1auto.com', '1592101541', 'https://www.suncountryheatingandcooling.com', None, 'http://www.taylorsflooringaz.com', 'http://www.asgnv.net/', 'https://www.autozone.com/locations/nv/las-vegas/1351-n-eastern-ave.html', None, 'http://cpllabs.com/', 'http://www.dandelionretro.com/', 'http://www.freshkabob.com/', None, '1592101619', 'http://www.restorationhardware.com/', 'http://serenityspapeoria.com/', 'http://www.bannerhealth.com/locations/avondale/banner-physical-therapy-mcdowell-107th?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', None, 'http://www.phoenixcountryclub.com', 'http://www.krauterdds.com', None, 'http://sthielpilates.com', 'http://www.advanceddermatologylv.com/', 'http://www.UltraCleanExpress.com', 'http://www.littlecaesars.com', 'http://evansotg.com', 'http://www.pitabasket.com', None, 'https://www.ulta.com/stores/surprise-az-176', None, 'https://www.scaddabush.com/sherway/', 'http://www.carltonsharley.com', None, 'http://shadysidedetail.com/', 'https://www.hendersondentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://vbarbershop.com/locations/gilbert/', 'https://www.lifestorage.com/storage-units/arizona/phoenix/85234/7110-in-gilbert/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-AZ-7110&utm_content=website-link&howheard=IYE', None, None, 'http://karsonskornbread.com', 'http://www.starwoodhotels.com/sheraton/property/dining/index.html?propertyID=205', 'http://www.pi-co.ca', 'http://www.itsbarmagic.com', None, 'http://www.painconsultantsofarizona.com', 'https://www.popeyes.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://harbour60.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.kidsacademyofcopley.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PITMC', None, 'http://www.marriott.com/hotel-restaurants/phxcb-jw-marriott-scottsdale-camelback-inn-resort-and-spa/blt-steak-scottsdale/125727/home-page.mi', 'http://www.mobilicity.ca', None, 'https://www.cricketwireless.com', 'https://www.ycsignsllc.com', 'http://www.fantasticfiredept.com/', 'http://www.busybodiesplayground.com', 'http://exquisitemobiledetailingllc.com', None, 'http://www.imperialflowershop.net/?srccode=yelp_track', 'http://www.yogaplustoronto.com', 'https://www.myessentia.ca/fr/magasins/laval/', 'https://torontovaporizer.ca', 'http://www.GurrBrothers.com', None, 'https://www.cleoms.com/meet-us/meet-the-doctor/dr-urban-picard/', None, 'http://thaipepperlv.com', 'http://www.orthoarizona.org', 'http://www.tirekingdom.com', None, None, 'http://www.barbsbridalboutique.com/', None, 'http://johnandsonsoysterhouse.com', 'http://www.allamericanlandscape.net/', None, 'http://www.kickerspizza.com', 'http://www.jlcolesalon.com/', 'http://obsessionbeautysalon.com/', 'http://www.allstateselfstorage.com/self-storage/bell-85032', 'http://www.risingphoenixllc.com', '1592102184', 'http://www.rasputinvodkabar.com', 'http://www.hkasiancuisine.com/', 'http://prioryfinepastries.com', 'http://alnolashoerepair.com', None, 'http://mvrwholesale.com/', 'http://www.rankinwomenscenter.com', 'http://www.curvyconsignments.com', 'http://www.krantzelectricinc.com/', 'http://www.gilcontool.net', 'http://www.themobilewebapps.com/repairwatchandjewelry', 'http://7eleven.ca/store-locator/33259', 'http://queencitylawn.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=2767&utm_content=bpas', None, None, None, None, 'http://www.hauntedpittsburghtours.com', 'http://www.pebooks.com/lv2/', 'http://www.kavicwinery.net/5701.html', 'http://www.waincapital.com', 'https://www.dentalsummerlin.com', 'http://slslasvegas.com/thebuffet', 'http://swensonelectric.com', None, 'http://www.westernexterminator.com/las-vegas', 'https://www.lebos.com', 'http://www.figarosbarbershop.com/', 'http://www.totallychiinc.com', 'http://www.fiddlersgreenpub.ca/', None, '1592102421', 'http://www.druglessdrs.com', 'http://www.learningrx.com/akron-bath', 'http://projectrefinedlife.com/group-private-yoga/', 'http://www.essenceairllc.com', 'http://www.amazingyoga.net', 'http://www.FatAngelosPizza.com', 'http://blinkeyewear.ca', None, 'http://www.bufords.kitchen/locations/downtown/', 'http://thenailspapgh.com/', 'http://lawanthaikitchen.com', None, 'http://harrisonautos.com', 'http://www.poppydenvegas.com/', 'http://www.reveregolf.com', 'http://clarkcountypoolandlawn.com', 'http://www.carpetsharkslv.com', 'https://www.summerlin.com/downtown-summerlin/overview.html', 'http://velvetlanecakes.com', 'http://www.cameostudio.com', 'http://1therapeuticmassage.wix.com/massage', 'http://www.muchoburrito.com', 'http://athleticmindedtraveler.com/browse-cities/toronto-on/university-of-toronto-athletic-centre', 'http://californiapools.com', None, 'http://www.apricotflowers.com', 'http://www.avon.com', None, 'http://www.MaderaPoint.com', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.mindfulwellnessaz.com', 'http://www.opalskinwax.com', 'http://www.sunkistpoolslv.com', None, 'http://www.greenroadpediatrics.net', 'http://www.room1six.com/', 'http://corktowndental.com', 'https://www.massageaddict.ca/Toronto-TheBeach', 'https://www.changingtimesdiaperco.com', 'http://www.zefiessmoothies.com', 'http://dynoplumbinglv.com', None, 'https://www.permenterchiropractic.com', 'http://scofieldrealty.com', None, None, 'https://stores.hallmark.com/nv/lasvegas/amys-hallmark-shop-lasvegas-nv-63960.html', None, 'http://www.globex2000.ca/fr', None, 'http://www.awrswheelrepair.com/pittsburgh', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Toronto-ON-M9M2R5/848075/', '1592102864', 'http://www.gnc.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.burritoboyz.ca/', 'https://ca.tokyosmoke.com/pages/queen-west', 'http://copperwillowsalon.com', 'http://www.stoughtonbuyandsell.com', 'https://integritydry.com', 'http://zero20kids.com/', None, 'http://walmart.com/', 'http://www.motel6.com', None, 'http://www.caldensebakery.ca', 'http://appliancewarehouse.com/', None, None, 'http://www.cafedejapon.com/', 'http://drshawnae.com', 'http://parrcabinet.com', 'https://www.towndentalgroup.com', 'http://www.theblindmannc.com', 'https://storelocations.ae.com/us/az/phoenix/4550-east-cactus-road.html', None, 'https://www.springspreserve.org/visitor-information/divine-cafe.html', 'http://www.solutionbuild.ca', 'http://www.northwestlimo.com', 'http://fairtradecoffeehouse.com/', 'http://www.starbucks.com/store/10759/', None, 'https://locations.wendys.com/canada/on/ajax/80-bayly-street-west', 'http://www.fredsheatandac.com/', '1592103130', 'http://newyork51pizzeria.com/', 'http://www.starbucks.com/store/14175/', 'http://www.radichisalon.com', 'https://encompasshealth.com/valleyofthesunrehab', 'http://www.fantasyautoinc.com', 'http://www.elitebusinessfilms.com', 'http://shop.pistolwhippedpastry.com', None, 'http://www.evanjstrong.com/', 'http://www.cbsquaredhairstudio.com', None, '1592103225', 'http://www.clevelandprintwear.com', None, 'http://www.jolantainteriors.com', None, None, None, None, 'http://www.pisausa.com', 'http://www.bashas.com/', 'http://www.playitagainsportscfalls.com', 'http://www.rvpaintrepair.com', 'http://www.janetkirkman.com/', 'http://www.ccvglass.com', 'https://www.massagespacharlotte.com', None, 'https://www.ritasfranchises.com/Wexford', 'http://bogartspet.com', 'https://www.jerseymikes.com/9003', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Tempe-AZ-85213/723082/', None, 'http://www.beyondconceptionmidwifery.com', 'https://www.24hourfitness.com/gyms/las-vegas-nv/rainbow-las-vegas-supersport', 'https://craftypots.studio', None, 'https://ad.doubleclick.net/ddm/clk/318369417;146857521;q?https://www.safelite.com/stores/Mesa?utm_source=yelp&utm_medium=list&utm_campaign=Addressblock&utm_content=85210_AZ', 'https://vspa-massagespa.business.site', 'https://winecollective.ca/highfield-store', 'http://www.thetoothwhisperer.com/index.html', 'https://opticas2020.business.site', None, None, 'https://drbeverlysmathisel.wixsite.com/drbeverlysmathisel', None, 'http://www.echosalonphoenix.com/index.html', 'http://www.guyspizzaco.com', None, 'http://www.dphairdesigns.com/', 'http://www.markmitsuphoenix.com', 'http://beegee-kitchen.com/', 'https://www.wm-columbuspark.com', 'http://www.infinitebeautysalon.com', None, 'http://www.sundevilauto.com/locations/az/mesa/east-mesa-power-26', 'http://www.flowersoftheworld.ca', 'http://patonthebackspa.com', 'http://pdrpizza.net', None, 'http://bankerinsulation.com', 'http://www.mainstreetcupcakes.com', None, 'http://www.veinitymd.com', 'http://mcfaddendalehardware.com', 'http://www.massagenirvana.com', 'http://www.timhortons.com', None, None, 'https://www.lush.ca/on/demandware.store/Sites-LushCA-Site/en_CA/Stores-Info?StoreID=129', 'http://www.acrepair4less.com', 'http://locations.jambajuice.com/ll/US/AZ/Scottsdale/15686-N_-Frank', 'http://www.timhortons.com/', None, 'http://www.GreaterPhoenixAreaRealEstate.com', 'http://xtremeairparks.com', 'https://www.steaknshake.com/locations/nv-las-vegas-las-vegas-boulevard-south', 'http://cambiedesign.com', 'https://ca-en.caudalie.com/boutique-caudalie-sherway-gardens-toronto', None, None, None, None, None, 'https://karachi-kabab-wala.business.site', 'http://www.azcellphones.com/', 'http://kjscurrybowl.com', None, 'https://grantmma.ca', 'http://phoenix-phx.worldairportguides.com/terminals.php', 'http://www.oldtimecandy.com', 'http://americanhomewater.com', 'http://www.footsolutions.com/summerlin', 'http://www.brownsshoes.com', None, None, 'http://eyecandispa.com', 'http://www.seanssalon.com', 'https://www.barlouie.com/locations/us/az/tempe/tempe-marketplace?utm_source=other&utm_medium=listings', 'http://www.bikramyoganphx.com/', 'http://www.nationalgolfacademy.ca', None, 'http://www.luxurymotorsports.us/used-luxury-cars-phoenix/', 'http://zcoilcf.zcoil.com/137/', None, 'http://www.fatbobpizza.com', 'http://www.swarovski.com/en-US/?utm_source=YELP&utm_medium=referral_managed&utm_campaign=YELP', 'http://www.west49.com', 'http://geigerfitness.net', 'http://Clearqualityautoglass.com', None, 'http://www.zenshinasianrestaurant.com', 'http://www.veginyyc.com', 'https://www.californiaclosets.com/locations/greater-phoenix/scottsdale-showroom/?CC5=32008&CC6=30593&utm_source=directory&utm_medium=local&utm_content=AZ166-M1-SR2', 'https://lulasewickley.com', 'http://www.fatbastardburrito.ca', 'http://www.wellsfargo.com', 'https://www.gordmans.com', 'http://www.chagrinfalls.net/', None, None, None, 'http://www.coconutclubbargrill.com', 'http://www.imaginedentistry.com', 'http://oldchicago.com/locations/champaign', 'https://www.interdent.com/gentle-dental/locations/az/mesa/1425-s-greenfield-rd-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.haysroofing.com/', 'https://www.integritycontentsservices.com', None, 'http://smilesourceonchagrin.com', 'http://etonhouse.ca', 'https://tsperformance.net', 'http://www.dressbarn.com', 'http://www.buildabear.com?sc_cid=ds:yelp:profile', 'https://locations.theupsstore.com/pa/harrisburg/4075-linglestown-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://greenwoodrentals.ca/', 'https://www.bikesourceonline.com/articles/bikesource-denver-pg502.htm', None, None, None, 'http://www.signaturehealthinc.com', 'https://www.snakesandlattes.com/location/tempe/', None, 'http://yourspacesalons.ca/tonyc', 'http://www.americanairappliance.com', None, None, 'http://www.theacvh.com', None, 'http://www.thisaint.ca', 'http://www.lasvegaswoodflooring.com/', 'https://www.insomniacookies.com/locations/store/1129', None, 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', 'http://shell.us', 'http://www.ghost-armor.com', 'http://www.thegunsmith.com/thegunsmith.html', 'http://treasureisland.com/restaurant/13/the-buffet-at-ti', None, None, 'https://www.rotorooter.com/murrysvillepa/', None, 'http://harrys-market.business.site', None, 'http://shop.hobbylobby.com', 'http://www.noodles.com/locations/south-end', None, 'https://vindalooindiancuisine.business.site', 'https://www.bowlero.com/location/bowlero-avondale?utm_campaign=webclicks&utm_medium=organic&utm_source=yelp', 'http://yardzatmirabelli.com', None, None, 'http://www.chefclaudio.com', None, None, 'http://rockfords.ca/', None, 'http://dannysdeli.net', 'http://www.foodcity.com', 'https://www.bnc.ca/particuliers/cartes-credit-mastercard/salon-aeroport-montreal.html', 'http://www.allforfun.ca', 'http://www.elyspringvalley.com', 'http://www.clubcorp.com/Clubs/Bear-s-Best-Las-Vegas/', 'http://www.dollartree.com/home.jsp', None, 'http://www.jriac.com', 'http://www.starbucks.com', 'http://www.cevw.com', 'http://realsh1ft.com', 'http://www.premiercustombuilders.com', '1592104749', 'http://www.scorelv.com/', 'http://bellaluna.us', 'http://www.jaaz.org', 'https://local.fedex.com/nc/charlotte/office-0746/', None, 'http://www.habibspersiancuisine.com', 'http://www.YouthfulDaySpa.com', None, 'http://morfinserv.com/', 'http://destinychurchlv.com', 'http://www.cabanasislandrestaurant.com', 'https://www.wyndhamhotels.com/hotel/46608?tel=18003370400&iata=00093785&cid=fe:bu:20170523:yl:pp:bayus:46608', None, 'http://downtowndetailslv.com', None, None, None, 'http://sonoranskydental.com', None, 'http://www.northcourtbeverage.net', 'http://www.restaurantdepot.com', 'http://www.hardrockhotel.com/las-vegas-restaurants/johnny-smalls', 'http://www.phoenixbimmer.com', 'https://azdonerightroofing.com', None, 'http://www.atozpediatricdentistry.com', 'https://www.titc.us', 'http://www.eastsidemarios.com/locations/ontario/toronto/scarborough/', 'https://www.assurant.com', 'http://www.westcheyennepharmacy.com', 'http://taylorhouselaw.com', 'http://kingsfamily.com/?utm_source=yelp&utm_medium=referral', None, None, None, None, None, 'http://romanticaboutique.ca', 'http://www.marriott.com/hotels/travel/cltcm-courtyard-charlotte-matthews', 'http://eagleriderhonda.com', None, '1592105051', 'http://www.hungryhollow.ca/', 'http://www.chandlercourtamc.com/', None, None, 'http://www.ultimatesportscards.com', '1592105091', 'https://www.arizonamovingcompanies.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282947950&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', None, 'https://calibercollision.com/locate-a-caliber-collision-center/charlotte-tyvola-rd', 'http://www.totalcigars.com', 'http://www.madisondancelife.com', 'http://www.pghgrille.com', 'http://www.allianceurgentcare.com', 'http://yafokitchen.com', 'http://www.abcautobodylv.com', None, 'http://www.boracaymassagevegas.com/', None, 'http://www.vetcoclinics.com', 'https://saharalasvegas.com/restaurants-bars/beers-and-bets/', 'http://www.robcoelectriclv.com', '1592105228', 'http://www.veneziabakery.com', None, 'http://www.summerspraytan.com', '1592105273', 'http://www.fresh-carpet-care.com/p/693?utm_source=bing&utm_medium=cpc&utm_campaign=EOW%20-%20Exact%20-%20Phoenix&utm_term=carpet%20cleaning', None, 'http://www.gordonwoodoptical.com', None, 'http://www.winghouseandsportsbar.com/', 'http://www.deltaco.com', 'https://www.obonsushi.com/scottsdale-quarter', 'http://www.valleyprotectiveservices.com/phoenix', 'http://www.wildgamegrillaz.com', 'http://thejewelers.com/venetian.html', 'http://www.sushiakaihana.com', 'http://www.wpahumane.com', 'http://menuderiaguanajuato.com', 'http://lambrettapizzeria.ca', 'http://www.clarkcountycourts.us', 'https://i-hvac.net', 'http://www.ichiriki.ca/', None, 'http://www.wordoflifelv.com', 'http://www.pizzaspano.ca', 'http://www.oakley.com/en/store-finder', None, None, None, 'http://overstockvehicles.com', None, 'http://hophausbrewing.com', 'http://www.pmilv.com', 'http://phoenixportablemobilewelding.com', None, None, 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=106&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-106', 'http://www.oldewinecellar.com', None, 'http://www.east4thstreet.com', None, 'http://www.synergyhomecare.com/agencies/az/sun-city/az02/', None, 'http://www.wildpurveyors.com', 'http://www.4monkeybusiness.com', None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', '1592105619', None, 'http://www.starbucks.com/store/14175/', None, 'https://independenceveterinaryclinic.com', None, 'https://www.choicehotels.com/CN281?mc=blypypaf', None, 'https://opasouvlaki.ca', 'https://www.lasvegassedation.com', 'http://www.mondrianscottsdale.com/mondrian_hotel_scottsdale_redbar.asp', 'http://affordablebailbondsphoenix.com/', None, None, 'http://www.todays-market.com', None, 'http://www.jazhairstudios.com/', None, 'http://www.dwellatregency.com', None, 'http://www.skinplicitylaneese.com', None, None, 'http://www.peterpiperpizza.com/', 'http://www.BlueStateElectric.com', 'http://www.thesourcemenswear.com', 'http://pizzaandbeyond.blogspot.com/', 'https://cobblestone.com/?utm_source=yelp%C3%8A', None, 'http://www.alohausedcars.com', 'https://www.carenow.com/locations/las-vegas/eastern-horizon-ridge', None, None, None, None, None, None, None, None, 'https://www.covenantcare.com/stores/silver-hills-health-care-center/', None, 'https://www.bestwestern.com/en_US/book/hotel-rooms.66086.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66086', 'http://avolsbookstore.com', 'http://www.mrshuizenga.com/', 'https://el-mesquite-cocina-mexicana.business.site', 'http://www.arnoldcrescentanimalhospital.com', 'http://www.valorestaurant.com', 'http://www.estheticssalonandspa.com', '1592105962', 'http://www.vape29.com', None, None, 'http://www.akronmassage.co', 'https://www.eddiev.com/', None, 'http://www.lcbo.com', None, None, 'http://www.dynamitecustomsoundsolutions.com', 'http://lacubana.ca', 'http://www.juiceandjava.ca', 'http://liftedtrucks.com', 'http://www.awildhairvegas.com/', 'https://lasvegasliving.com/properties/inspire/', 'https://www.thesteelecreekdentist.com', 'http://www.abbybelladance.com', None, 'http://www.roomstogo.com/stores/pineville/carolina-place-store/1601#utm_source=yelp&utm_medium=localsearch&utm_campaign=yelp', 'http://www.scottsdaleplaza.com/dining/jds-sports-pub-en.html', 'http://www.winners.ca', 'http://www.michaelangelos.ca', 'https://www.kohls.com/stores/pa/pittsburgh-293.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://lifedesignevents.com', 'http://www.rmcf.com', None, 'http://www.nhlakesidefamilyphysicians.org/locations--hours/prosperity-church.aspx', 'http://www.friartuckonline.com', 'http://www.getacoolbox.com', 'https://www.motel6.com/en/motels.az.phoenix.18.html', 'http://www.rockerzandrollerz.com', None, 'http://www.vegasemployment.com', None, None, 'http://kahunaville.com/', None, 'http://www.bathandbodyworks.com', None, 'http://thepinkdoorcarefree.com', 'http://www.scottsdaleaz.gov', 'http://www.midibistro.ca/', None, None, 'http://www.kingdomtitle.net', 'https://www.tdcanadatrust.com/products-services/banking/index-banking.jsp', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.liveatsanmelia.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=organic', None, 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'https://locations.monro.com/oh/bay-village/609-dover-center-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, 'http://www.oakridgepizza.ca', 'http://www.myspace.com/ronnieslocal069', 'http://barbershopstyles.com', None, 'http://www.spotteddogmenu.com/', 'https://www.susanvthomaslaw.com', 'https://www.rittersdinerpittsburgh.com', None, 'http://www.rocklandtrailers.com', 'http://azcustomcoatings.com', 'https://www.bannerhealth.com/locations/peoria/banner-health-clinic-vistancia', 'http://www.bestarizonacars.com', 'http://www.kay.com', 'http://www.digitaljuiceagency.com', 'http://www.bzbarandgrill.com', '1592106459', None, None, 'http://www.aspenplumbingandrooter.com', 'http://www.allcaremedicine.com/', 'http://patriatoronto.com/', 'http://www.awomansplaceobgyn.com', None, None, 'http://www.karlsqualitybakery.com', None, None, None, '1592106550', 'https://www.popeyes.com', None, None, 'http://www.michaelangelosphotography.com', None, 'https://www.wesoothesoles.com', None, 'http://www.thechickery.com/', 'http://www.dunkindonuts.com', 'http://www.treats.com', 'https://myneighborhoodprimarycare.org', 'http://www.scottsdaleaz.gov/trolley/neighborhoodtrolley.asp', None, 'http://www.lemongrassthaicafe.net', 'https://www.bowlero.com/location/bowlero-via-linda?utm_source=yelp&utm_medium=organic&utm_campaign=webclicks', 'http://www.framesunlimited.com/locations/shaker-heights', None, None, None, 'http://www.nicks-automotive.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.goodwill.ab.ca/goodwill-location/varsity/', 'http://www.icaptureevents.com', 'http://www.trubuildersaz.com', 'http://www.formulakartways.com', None, 'http://chandlerdrivingschool.com', 'https://www.jacksonhewitt.com/Tax-Preparation-Services-Locations/NV/North-Las-Vegas/16008/', 'http://www.sushihousebrampton.ca', '1592106768', 'http://www.thehickorytavern.com/home/hickory_tavern-locations.php?l=Charlotte+-+Providence+Rd.', 'http://www.re-wrap.ca', None, 'http://availvapor.com', 'https://threemonksandaduck.ca', 'http://www.hihealth.com', 'http://www.starbucks.ca', 'https://www.keendogtraining.com', '1592106832', None, 'http://www.kangamooplay.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://sunrisemechanical.com/areas-served/peoria-az/', 'http://www.caminoalnorteapts.com/', 'http://www.bidwellhomeimprovement.com', 'http://www.bookbeautician.com', 'http://www.marcellos.ca', None, '1592106911', 'http://www.theroyalambassador.com', 'http://nextlevelcleaners.com/', None, 'http://www.ogdenfuneralhome.com', 'https://www.starbucks.ca', None, 'http://highlifesmokes.net', '1592106971', 'http://www.goodlifefitness.com/locations/ontario/northyork/don-millseglinton?source=yelp-websitelink-northyorkdon-gl', None, None, 'https://www.elementballantyne.com', 'http://tapeface.tv', 'http://www.hussongslasvegas.com', 'http://tidycleaners.ca', 'http://www.hksarizona.com', 'http://www.prettyininkpermanentmakeupaz.com', 'http://www.thefarmersfork.com', 'http://www.ibarbieri.com', 'http://www.barbiesgrill.com/', None, 'http://www.vegasgoldbuyers.com', None, 'http://www.anthemlandscape.com', None, 'http://www.erinssnug.com/', '1592107116', None, 'http://www.gelato64.com/?page_id=409', None, 'http://mesafamilydentistryaz.com/', 'https://locations.dennys.com/OH/ELYRIA/247158', 'https://www.closetworld.com', None, 'http://segwaylasvegas.com', None, None, 'http://getahaircutlv.com', 'http://www.ticozaz.com', 'http://www.brooksdiamonds.net', '1592107221', 'http://www.giadasitaliancucina.com', 'http://stores.barnesandnoble.com/store/2680', 'http://www.bamboobistrolasvegas.com', 'http://cestbonrestaurant.com/', '1592107261', 'http://www.euro-fine.com', None, 'https://www.wrightlawnv.com', 'http://www.booksamillion.com', None, 'http://ballersbistro.com', None, None, None, None, 'http://www.bestwaypoolservice.com', 'http://www.vfwpost7401.com/', 'http://30minutealterations.com/', 'http://www.auroranaturalmedicine.com', 'http://littlecaesars.com/', None, 'http://www.yaymaker.com', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.SecureDataRecovery.com/Charlotte', 'https://smartstopselfstorage.com/storage-locations/usa/nevada/las-vegas/smartstop-self-storage-4851-e-bonanza-rd', 'http://phoenixarc.salvationarmy.org', 'http://www.timbermesaoutdoors.com', '1592107451', 'https://www.chilis.com', 'http://www.phoenixvirtualtour.com', None, 'http://gorgeouslady.ca', 'http://www.tinderboxcigars.com', None, 'https://primpandblow.com/locations/primp-and-blow-chandler/?UTM=yelp', None, None, 'http://www.aladorpetresorts.com', 'http://www.bcbg.com', 'http://feelslikehomefarm.com/index.html', 'https://www.circlek.com/store-locator/us/ruskin/3602-hwy-41-s/2707247', None, None, None, None, None, 'http://www.kazesushi.ca', None, None, 'http://www.fabulousmanicurebar.com', 'http://www.gtatireandauto.com', 'http://www.mannyforinsurance.com/?cmpid=UGOG_BLM_0006', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', 'http://www.torontoairportlimo-taxi.ca', 'http://Www.FlamingoCoolsVegas.com', 'http://www.secondcup.com/location/toronto-355-danforth-avenue', 'http://www.jdbyrider.com/buy-here-pay-here-pittsburg-monroeville-pennsylvania-pa114', 'http://www.djinlasvegas.com', 'http://freshco.com/stores/bloor-dundas', 'http://www.totalwine.com/store-info/arizona-phoenix-desert-ridge/1003', 'http://www.ggfarm.com/welcome-to-grateful-growers/harvest-moon-g', None, 'http://www.apassionforpetssalon.com', 'http://www.trulythaicuisine.com', 'https://www.scottsdalecfdh.com', 'https://www.legendatarrowhead.com', 'http://vehiclesunlimitedinc.com', 'http://vegasairduct.com', None, None, 'http://www.thetcmclinic.com', None, 'http://www.verizonwireless.com/stores/colorado/denver/16th-st-mall-201560/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://www.plazaloftskierland.com/', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'https://www.neohioeyes.com', 'https://www.enterprise.com/en/car-rental/locations/us/az/tempe-holdeman-5015.html?mcid=iyp:8368400', 'http://www.chelseanicole.com', 'http://www.morningsidenissan.com', None, '1592139482', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/yyzsb-delta-hotels-toronto-east/', '1592139508', None, 'https://www.chocolatebywickerhead.com', None, '1592139537', None, 'https://www.kneadtodestress.com', '1592139573', 'http://www.holeeburger.com/', 'http://www.gttdogs.com', None, 'https://www.faded.club', None, 'http://www.autoberry.ca', None, 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=441', 'http://www.pandaexpress.com/', 'http://www.mistyglenkennel.com', 'http://www.krispykremeaz.com', 'http://nsbank.com/', '1592139673', 'http://www.vivefitness.com/', 'http://www.mchair.com', 'http://purefitnessclubs.com/', 'http://www.bstreetrmt.com', 'http://www.rungreatrace.com/', None, 'http://lasvegas.777rentals.com', 'http://www.philipamodei.com/lasvegashairstudio/contact', 'http://www.moongsushi.com', 'http://www.capitalone.ca', '1592139757', 'https://www.discounttire.com/store/az/scottsdale/s/1284?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://www.cmgfi.com/agents/richard-blair', None, '1592139794', 'http://www.lvboneandjoint.com', None, 'http://www.eatgarbanzoaz.com/', None, None, None, 'https://lumenbrite.com/locations/phoenix/', 'http://www.Topspizza.ca', None, None, None, 'http://chefkennysasianvegan.com', None, 'http://www.safestoreaz.com', 'http://www.foothillsrehab.com/', 'http://www.25thstreetautomotive.com', 'https://cobblestone.com/?utm_source=yelp%C3%8A', '1592139942', None, 'https://www.bankofamerica.com', 'https://www.purebarre.com/location/westlake-avon-oh', 'http://www.tigerslairaz.com/', None, 'http://adavenkid.com/', 'http://www.shoregrillandgrotto.ca', None, 'https://www.advancedfootankle.com', 'http://www.insideoutsports.com', 'http://roysliquor.com', 'http://www.bestchoice4insurance.com', 'http://www.redkphoto.com/', 'http://www.firstchoicedental.com/locations/sun-prairie-office', None, 'https://www.target.com/sl/henderson-lake-mead-pkwy/2404', None, None, '1592140058', 'http://substationaz.com', 'http://searchthec4group.com', None, 'https://www.zimbrickaudiofmadison.com', 'http://www.mercuryautospa.com', 'http://mtclaret.org', None, 'https://www.nothingbundtcakes.com/bakery/nv/las-vegas-eastern?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=las-vegas-eastern_2', 'http://bobbyqgrill.com/', 'https://www.spring-green.com/franchise/urbana-area/', 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/etobicoke-(1).aspx', 'https://www.kohls.com/stores/az/scottsdale-620.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.abovegroundartsupplies.com/', None, '1592140190', None, 'http://www.brighterangels.com/phoenix.html', 'http://croywifi.com', 'http://www.jlbridal.com/', 'http://www.thepaperytoronto.com', 'http://brocach.com', 'http://www.balancingyourchemistry.com/', 'http://deeplyrootedhairstudio.com', None, 'http://www.chennaichettinaadpalace.com', None, None, '1592140276', 'http://gohomeplate.com', '1592140287', 'http://www.lvac.com/locations/central', 'http://www.amici-dinos.com', 'http://greenfieldjewelers.com', 'http://oldnavy.gap.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.spoonandfork.ca/spoon-and-fork-eglinton/welcome', None, 'http://fpcmesa.org', 'http://www.goodlifefitness.com', 'http://www.theecigflavourium.com', 'http://www.milagrorestaurant.com', None, 'https://www.thingsremembered.com', None, 'http://www.sambabraziliangrill.com/', None, None, 'http://fusionpowerco.com', 'http://www.michaelkors.com/', 'https://shop.lululemon.com?CID=ps_YELP_website_org', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://agents.farmers.com/az/tempe/greg-krajewski?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'https://www.sephora.com/happening/stores/pittsburgh-shadyside', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://skyvapeaz.com', 'https://pablocanada.com/shop/', 'http://www.nationalcarmart.com/', 'http://home-savings.com', 'http://chopstickexpresstogo.com', 'http://www.pathwayscounselingsvcs.com', 'http://www.petsittingscottsdale.com/', 'http://www.woodbridgevacuum.ca', 'http://www.golfwildhorse.com/', 'http://www.jennysdispensary.com', None, 'http://thericegarden.com/', None, 'https://locations.jackinthebox.com/us/nc/charlotte/7725-pineville-matthews-rd', 'https://performancetherapymassage.net', 'http://www.shinbay.com', 'http://www.solapizza.com', 'http://www.gilbertserenitydental.com/', 'http://jollyrollsicecream.com', 'http://browsbyhilary.com', None, 'http://www.adventdentistry.com/', 'http://www.mastersgalleryltd.com/', 'http://smog-king.com', None, None, None, 'http://www.a-ztennis.com', 'https://www.cellairis.com/gilbert-cell-phone-iphone-repair-yelp', 'https://made-in-argentina-la-esquina-argentina.business.site', None, 'http://www.mustardseedherbary.com', None, 'http://ivorystar.com/shows/friends-the-musical-parody', None, 'http://FoxtailLasVegas.com', 'http://www.starbucks.com/store/1006250/ca/jean-adam-st-denis/75-avenue-de-la-gare-saint-sauveur-qc-j0r-1r6/les', 'https://www.plumbcrazy4plumbing.com', 'https://www.enterprise.com/en/car-rental/locations/us/al/huntsville-madison-55ha.html?mcid=iyp:8368400', 'http://gamblerscafe.com', None, None, 'http://www.themachatgroup.com', 'http://www.mikesbarbershopmentor.com', '1592140868', None, None, 'http://chaseanddestroy.com', None, None, 'http://andersonac.com', None, 'http://www.phoenixazchinesefood.com', None, 'https://www.allnevadainsurance.com', 'https://thepolishedbrow.com', 'http://muramoto.biz', 'https://palmbeachtan.com/locations/OH/Westlake-Detroit-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=CLE', '1592140999', None, 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', None, 'http://www.campusbodysalon.com', 'http://www.lovesick.com/store-details?StoreID=9020', 'https://www.infinityfinancialmortgagecorp.com', 'http://www.atlwingsyourway.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.lvatvtours.com/', None, None, None, '1592141121', 'http://www.psychiatrylasvegas.com', 'http://www.all-starpediatrics.com', None, 'http://roomofonesown.com/', 'http://www.unityrideshop.com', 'http://villemontpizza.com/index.html', 'http://www.2bmod.com', '1592141176', 'http://washworkslaundry.com', 'https://highweedmedshop.com', 'https://mrsub.ca', 'http://gradeaid.ca', None, None, None, None, 'http://www.mrrooter.com/phoenix', None, None, None, 'https://www.sheetz.com', None, 'https://www.wyndhamhotels.com/hotel/36226?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:36226', None, 'https://www.coffeebean.com/store/usa/scottsdale/promenade-at-scottsdale', 'http://www.clearwash.ca', 'http://www.carolinavaluevillage.com', 'http://www.sincitycrossfit.com/', None, 'http://www.epennzoil.com', 'http://www.bojangles.com', 'https://www.americafirst.com/about/branches/branch-details.cfm?siteNo=233', None, 'https://www.sprint.com/storefronts/bd/sprint-south-euclid-oh-44118-fcs-1799/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://townhall.innis.utoronto.ca', 'http://www.azgreencleaning.com', 'http://pa.pittsburgh.charmingcharlie.com/fashion_accessories_pittsburgh_pa_128.html', None, 'http://www.kickaxeguitars.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YULRI', 'http://www.sunlessboutique.com/', 'https://www.dominos.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://redhousepub.co', 'http://zellaspizza.com', 'http://www.palminaleone.com', 'http://comicgirlcoffee.com', 'http://www.andreawhitten.com/', 'https://www.whitakertire.com/location/mode/1/3701-greensprings-ave-west-mifflin-pa-15122.aspx', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.taylorrentalavon.com', 'http://www.starbucks.com', None, None, 'http://www.urbanaparks.org/facilities/16.html', 'http://www.superdry.com/', '1592141552', None, 'http://hazelgeneralstore.com/', 'https://www.supercuts.com/locations/az/glendale/bell-park-shopping-center-haircuts-80215.html', 'http://www.creativetraveler.com', None, None, None, 'http://www.circabella.com', '1592141628', None, 'http://www.usps.com', 'http://www.aaronbrothers.com', 'http://www.roastmadison.com/', 'https://www.americancampus.com/student-apartments/nc/charlotte/university-crossings-charlotte', 'http://calgary.cnty.com/', 'http://locations.michaels.com/AB/CALGARY/3902/', 'http://www.home-computerhelp.com', 'http://www.thebeverleyhotel.ca', None, 'http://sachisalon.com', None, 'http://www.durafastlabel.ca', 'http://www.vegas-salon.com', None, 'http://firstchurch.church', None, 'http://www.sumitsyoga.com', 'http://www.popeyeschicken.ca', None, 'https://kneaders.com', 'https://www.turkishkebabhouse.net', None, 'http://www.subarusaintejulie.com/en/', 'http://www.eyelidsphoenix.com', 'http://www.victoriassecret.com', None, 'http://totalwellnessacupunctureaz.com/', 'http://www.maxandermas.com/locations/333038', None, 'https://www.berkshirecommunities.com/apartments/nc/charlotte/berkshire-dilworth/?utm_source=Yelp&utm_medium=Referral', 'https://www.walmart.com/store/2050/henderson-nv/whats-new', None, 'http://www.discounttirebellevue.com/', 'http://www.pizzanova.com', 'http://www.7-eleven.com/?yelp=29384', None, None, 'http://www.marriott.com/hotels/travel/yyzmk-courtyard-toronto-markham/', 'https://locations.levi.com/nv/lasvegas/denim_las-vegas_nv_188.html', 'https://www.pizzahut.ca/store-details/store-R32814/pizza-hut-Mississauga', 'http://www.onpointhairandnail.com', None, None, 'http://www.climatechnc.com', None, 'http://www.newgalleryofmodernart.com', None, 'http://www.assemblychefshall.com', 'http://www.wisconsincheesemart.com', 'http://www.gamestop.com', None, 'https://flights.hisgo.com/us/', 'http://www.starbucks.com/store/14887/', None, 'http://www.belgo.ca/', None, '1592142134', None, 'https://www.lasvegasbookkeeping.com', 'http://www.dollarama.com/', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', 'https://www.instantplumbing.ca', 'http://www.tintaz.com/', 'http://www.aneyefordetail.com', 'http://vanlanendds.com', 'http://www.loonspub.com/', None, '1592142216', None, '1592142226', None, 'https://www.bigfrog.com/stores/gilbert-az/', 'http://csrcoffee.com', 'http://www.expert-locksmith.ca', 'http://www.manhattanfishgrill.com', 'https://henleysfloor.com', None, 'http://www.skallywagcigars.com', 'http://www.speedandstrengthuniversity.com', 'http://www.lasvegasrestorationllc.com', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, 'http://steelecreekpediatricdentistry.com', 'http://www.mesafarmersmarketandflea.com', None, 'http://theforumatdesertharbor.com', None, 'http://addisonparkapts.com?utm_source=Local%20Listings&utm_medium=referral&utm_campaign=Top%20Line%20Local', 'http://www.greatclips.com', None, 'http://www.citi.com', None, None, None, None, None, 'http://www.7baker.ca', 'https://team-kaizen.com', None, 'https://donroyaldrycleaner.wixsite.com/website', None, 'http://www.goldstartailoring.com', None, 'http://www.planetfitness.com/gyms/charlotte-whitehall-nc-125?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.gerrysmarine.com', None, None, 'https://www.suretempair.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/oh/elyria-3931.html?mcid=iyp:8368400', 'http://www.krispykremeaz.com', 'http://www.breadandcircus.ca', None, 'https://www.millerstavernlv.com', 'http://www.marcs.com/stores/marc-s-lakewood-grocery-store-44107.aspx', 'http://www.ambrosia.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.awardscheesecakes.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNCY', 'http://www.daleskarrparts.com/', 'http://www.bp.com', None, None, 'http://magictailor.ca', 'http://www.distillerypub.com/', 'http://www.deckscapes.com', 'http://www.chromasalonandboutique.com', 'https://www.bfly.ca/experience-bfly/', 'http://www.thrill-bikes.com', '1592142722', None, 'http://www.gotransit.com/publicroot/en/travelling/stations.aspx', 'http://ModManBarber.com', 'https://chocolateshops.sees.com/az/phoenix/chocolate_shops_phoenix_az_102103.html', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://sunboxphotography.biz', None, 'https://locations.monro.com/oh/painesville/1575-mentor-avenue?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', '1592142811', 'http://www.truemedispa.com/', 'http://www.realfruitbubbletea.com', 'https://az.aaa.com/agent-office-locations/az/phoenix/aaa-phoenix-office', None, 'http://jeroldoptical.com/', 'http://www.redrocklasvegas.com/dining/hachi.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=RR&utm_content=RR-PROPERTIES&utm_campaign=RR-YELP', 'http://jenniferwallens.com', None, 'https://clevelandhungarianmuseum.org', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.lakenormanchrysler.com', 'http://latitude360.com/pittsburgh-pa/', 'http://www.brewercommercialservices.com/', 'http://www.winerack.com', None, 'https://www.radisson.com/madison-hotel-wi-53719/madison', 'http://cityofhenderson.com/henderson-happenings/facilities/recreation-centers/whitney-ranch-recreation-center', None, 'http://Terminalbarbershop.com', None, 'http://www.americanapparel.net', None, 'http://www.mariospgh.com/southside/', 'http://www.shamrockroofingservices.com', 'https://www.amctheatres.com/movie-theatres/phoenix/amc-ahwatukee-24?utm_uid=yelp', None, '1592146361', 'https://www.steinmart.com/?utm_source=yelp.com&utm_medium=social&utm_campaign=yelp_site_store97', 'http://meineke.com', 'http://www.partydesignlv.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.margaritavilleglendale.com/', 'https://www.chimneystax.ca', 'http://www.palmettofamilyhearing.com', 'https://airpromaster.com', 'http://www.fresh-burger.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281380115&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://tempecomputerrepairservice.net', None, 'https://www.lvaspensalon.com', 'http://www.pacetax.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/13777+Pearl+Road-Strongsville-OH-44136/storeid=4300?cid=ps_ylp_2020', None, None, 'http://www.pandalawfirm.com', None, None, 'http://www.dominos.com', 'http://www.azuldayspa.org', None, 'http://www.koa.com/campgrounds/madison/', 'https://tinylovesphotography.com', 'https://www.imyourmechaniclv.com', 'http://www.moes.com', 'http://eng.bigbustours.com/lasvegas/home.html', 'http://www.lindleylawoffice.com', 'http://www.actiontours.com', None, 'http://www.thaizone.ca/', 'https://www.islandthymepub.com', None, None, 'https://www.efirstbank.com/customer-service/find-location.htm?loc=BELL', None, 'http://sagoautoglass.com', 'http://www.truevalue.com', 'http://www.clemcopostal.com', 'http://www.barrymorefurniture.com', 'http://www.plazahotelcasino.com/entertainment/superstars_of_magic', 'http://www.peakchiropracticpgh.com/', 'https://locations.sonicdrivein.com/nv/lasvegas/6301-w.-lake-mead-blvd..html', None, None, 'http://www.scooopskidspa.com/', 'http://www.ladybugsandwiches.com', None, 'http://www.separkmansion.com', 'https://www.ironwoodcrc.com/locations/?locations_doc=scottsdale', 'http://www.goldenlobsters.com', 'http://www.FourSonsLandscaping.net', 'https://www.enterprise.ca/en/car-rental/locations/canada/on/north-toronto-eglington-yonge-c245.html?mcid=iyp:8368400', 'https://www.softsurroundings.com/stores/the-district-at-green-valley-ranch/', 'http://wellnesslasvegas.com', 'http://www.southernshows.com/wch', 'http://omfastfoodrestaurant.com/home.html', 'http://fuerstauto.com', 'https://www.papamurphys.com', None, None, 'http://eeleconomicrestaurantequipment.com', 'http://www.haircoloringinphoenixaz.com', None, 'http://www.oakesappliance.com', 'http://whasn.com/providers/julie-goodell', 'http://www.huntersvilleford.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.carminespizzakitchen.com', 'http://www.homedepot.ca', 'https://www.hendersonmentalwellness.com', None, 'http://onetoone.case.edu/', 'http://www.taqueriapastorcito.com', 'https://www.bakerbros.com/visitus', None, 'http://www.fallsheating.com/', None, 'http://www.starbucks.com', None, None, 'http://houlihans.com/', 'http://www.yogurtini.com', 'http://www.donvalleynorthlexus.com', 'https://www.smartstyle.com/en-us/locations/az/phoenix/located-inside-walmart-2113-haircuts-11301.html', None, 'http://www.bk.com', None, 'http://pizzabytheslice.ca/', 'http://www.shapetrain.com', 'http://coronadelsolhighschool.net/', None, None, 'http://www.wyattworks.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1015+East+Ray+Road-Chandler-AZ-85225/storeid=7849?cid=ps_ylp_2020', 'http://massageaddict.ca/oakville', 'http://Inspiradabarberandbeauty.com', 'http://www.nuvohotel.com', 'https://www.hilton.com/en/hotels/lasruru-tru-las-vegas-airport/?SEO_id=YELP-RU-LASRURU', 'http://www.starbucks.com/store/9059/', 'https://azoralsurgery.com', 'http://www.bethanyanimalhospital.com', None, 'http://www.goodeggdining.ca', 'http://www.nevadadivorce.net', None, 'http://www.classykitchenandbath.com', '1592147262', None, 'http://www.service1stenergysolutions.com/', 'http://www.vesperrx.com', '1592147301', 'http://www.chestnuthillsdental.com/locations-providers/pittsburgh-shadyside/', 'http://www.macssushi.com', 'http://www.nanashake.com', 'http://www.eyepriority.com', 'http://www.northcoastbodymod.com', None, '1592147361', 'http://www.lvcc.com', 'http://storelocator.pepboys.com/pepboys/stores/phoenix/cactus/00747', 'http://www.reginospizza.com', 'http://www.travelbughealth.com', None, None, 'http://www.mysisterscloset.com/', 'http://www.carolsconfectionspgh.com', 'http://www.altaderm.com', 'https://www.ymcacalgary.org/program-descriptions/locations/brookfield-residential-ymca-at-seton/', 'http://www.aaaonestopappliance.com', 'https://www.tonysautoshop.com', 'https://911drivingschool.com/ohio/driving-school-rocky-river-ohio/', 'http://www.gaudinporschelv.com', None, 'http://www.tuesdaymorning.com', 'http://www.tlcflooringlv.com', 'http://www.buddystubbshd.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.potterlawaz.com', 'https://www.maaco.com/locations/nc/charlotte-11619/', 'http://www.bartendingschoolc2c.com', 'http://www.millenniumsalonmadison.com', 'http://www.yak.com', 'http://rackempubandbilliards.com', 'https://restoreandrenewmassage.com', 'http://creativeillusionshairstudio.com', 'https://www.TheUltimatePartyStore.net', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.completevisioncare.com/home.html', None, '1592147593', 'https://www.northwest.com', 'http://www.arizonalullabyguild.com', 'http://www.meltwich.ca/richmondstreet/', 'http://www.aroundtheblock.com', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700006435131063&ds_s_kwgid=58700000444951711&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', '1592147657', None, None, 'http://www.littlecaesars.com/', None, 'http://cherryboomboom.com', None, None, 'http://www.azdivalashes.com', None, 'http://www.drteddysim.com/about-dr-teddy-sim/', 'http://access-handyman.webnode.com', None, 'http://www.roomforhealth.com/', 'https://www.autozone.com/locations/nv/las-vegas/1351-n-eastern-ave.html', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', None, 'https://www.salt-tempe.com', 'https://nature.discoveryplace.org', 'http://www.senjivastudio.com', None, None, None, 'https://www.shijosushi.ca', None, None, 'http://www.qinthaicuisine.com', None, 'http://www.invisiblehandyman.com', None, None, 'http://headtripbrewery.com', None, None, None, 'http://www.444park.com', 'http://www.pueblomedicalimaging.com', 'http://www.the-bag-lady.biz', None, None, None, None, 'http://www.bbqisland.com', 'http://www.ohiocitynails.com', 'https://www.realstar.ca/apartments/on/brampton/188-cityview-apartments/index.aspx', None, 'http://walmart-eyedocs-lasvegas.com/Marks-Sunset', 'https://studios.waxingthecity.com/ll/US/PA/Pittsburgh/3605-Butler-Street?utm_source=yelp&utm_medium=local&utm_campaign=localmaps', None, None, 'http://omnidermatology.com', 'http://locations.in-n-out.com/234', 'http://www.eastvalleyfiduciaryservices.com', '1592148033', 'https://azflorist.com', 'https://www.meineke.com/locations/pa/pleasant-hills-2517/?utm_source=yelp&utm_medium=xvc', 'http://www.ezpoolslv.com', None, 'http://alluregardens.com/', 'http://abovethecrustpizza.com', 'http://www.alldogsgotobev.com/', '1592148088', 'http://itdotrepairs.com', None, None, 'http://www.uwhealthkids.org/locations/kidsdetail.jsp?locationId=59', None, 'http://www.visiblevoicebooks.com', 'http://www.juvenaspa.com', 'http://www.aeortho.com', 'http://www.rosestodaylasvegas.com/?srccode=yelp_track', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.charakcenter.org', None, 'http://gear-oh.com', 'http://www.wfamilydental.com', 'http://salonstylush.com/', 'http://www.westaddisonaz.com', '1592148259', 'https://www.barra-sandusky.com', None, 'http://www.denturerepairinglendaleaz.com', None, None, 'http://www.schneidersauto.com', None, 'https://datarecovery.com/about/toronto/', 'https://stores.jcpenney.com/az/phoenix/clothing-stores-phoenix-az-2889.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2889', 'http://www.picknsave.com', None, None, None, 'https://www.eliminexpest.com', 'http://www.starbucks.com/store/9298/', 'http://www.lightrx.com/mesa', 'https://local.biglots.com/pa/monroeville/1960?zcp=pd_local_yelp', None, 'http://rizzosspaghettihouse.com', 'https://www.statefarm.com/agent/US/NV/Henderson/Carl-Endorf-HYSY71YS000?cmpid=ST0N_BLM_0006', '1592148451', 'http://www.lveyedocs.com', 'https://www.usps.com/', 'http://www.fatbastardburrito.ca/locations/burrito-whitby', None, None, 'https://www.scottsdale-duilawyer.com/?utm_source=scottsdale_location&utm_medium=yelp&utm_campaign=website_link_click', 'http://arizonapianomover.com', None, 'http://www.ctrglass.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', None, None, None, 'http://www.rollcare.ca', 'https://locations.panerabread.com/wi/fitchburg/2960-cahill-main.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', None, 'http://www.speedlube.net', 'http://local.skechers.com/ca/glendale/38/', 'http://www.lowes.com', 'https://www.pinnaclenissan.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'https://www.ulta.com/stores/westlake-oh-1423', 'http://www.azphi.com', None, 'http://www.lacasadelahabana.com', None, None, None, 'https://www.ediblearrangements.com/fruit-arrangements?utm_source=Yelp&utm_medium=Listing&utm_campaign=StoreID206', None, '1592148696', None, None, 'http://www.starbucks.com', '1592148737', 'http://www.getenzo.com', 'http://howlerspittsburgh.com', None, None, None, 'http://l.macys.com/fashion-show-in-las-vegas-nv?cm_mmc=Yelp_Stores-_-LasVegas-_-n-_-532&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Las+Vegas_532&m_ac=yelp_stores&m_ag=n', None, None, None, 'https://local.safeway.com/safeway/az/scottsdale/23565-n-scottsdale-rd.html', 'http://local.firestonecompleteautocare.com/arizona/cave-creek/4915-e-carefree-hwy/?lw_cmp=IYP_YPC_MLP_784291', 'http://www.doordoctor.ca', 'https://www.maccosmetics.com', None, 'http://www.popularpizza.ca/locations.phtml', None, None, None, None, 'http://www.craftcellars.ca', 'http://www.valleylights.com', 'http://www.quickspark.com', 'http://www.atlwings.com', None, 'http://www.OnTheSpotLV.com', None, '1592148965', 'http://www.hillsong.com/phoenix', None, None, 'http://www.divinethreading.com', 'http://www.mayfieldohdentist.com/', 'http://www.scottsdaleaz.gov/sports/aquatics/mcdowell-mountain-ranch-pool', None, 'http://www.ryah.ca/', None, 'http://www.bubblesandbowslv.com', 'http://www.houdini.com', None, None, 'http://mmgyn.com', None, 'https://www.openriceclt.com', 'http://www.mccoolspools.com', 'http://musiccentrecanada.com/', None, 'http://www.gioninos.com', 'http://www.ericpalacioslaw.com', None, 'http://www.embassygrand.com', 'http://www.firstprimerealtygroup.propertyware.com', 'http://www.sierraironlv.com', None, None, '1592149192', None, None, 'https://www.bakerbros.com/glendale', 'https://prairieflowersandgifts.com', 'https://www.sunsetgrill.ca', 'https://americanbuilderslv.com', 'https://www.petsfriend.ca', 'https://cobsdoggersandmore.wixsite.com/website', 'http://jacksaloon.com', 'http://www.thesweetpotato.ca', None, None, 'http://www.breatherefuge.com', None, 'http://www.fantasticsams.com', 'http://www.olsonortho.com', 'http://www.bulldogliquidators.com', 'http://www.ladyyorkfoods.com', 'http://storelocator.pepboys.com/pepboys/stores/las_vegas/east_charleston/00685', 'http://www.acaringnanny.com', 'http://www.markhamtheatre.ca', None, None, 'http://www.mermaidsplashnails.com', 'http://www.stephanoslv.com', None, 'https://www.peppywalnutssweetsboutique.com', None, 'http://www.courtesychrysler.com', 'http://creamistryAZ.com', 'https://www.skininjectables.com', 'http://www.lcbo.com', 'https://www.sprint.com/locations/nv/henderson/334-w-lake-mead-pkwy', None, 'http://nourishmassageandskincare.com', 'http://www.timhortons.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044280206066&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://sidelinestreeservice.com', None, 'http://www.evooristorante.com', 'http://www.budacafe.ca', 'http://leesliquorlv.com/locations/sunset-office/', None, 'https://www.jackentertainment.com/thistledown', 'http://www.orangetreepools.com', 'http://www.panorea.ca', 'http://www.cancleanpros.com', 'http://www.phxestatesales.com/', 'http://www.bareoaks.ca', 'http://www.marriott.com/hotels/travel/phxss-springhill-suites-phoenix-north/', 'http://steviespizza.com', 'http://www.7-eleven.com/?yelp=32265', 'http://tricitytowingaz.com/', 'http://www.vanillabeanonline.com', 'http://www.haggen.com', 'https://www.thebrick.com', 'http://www.appleplumbingaz.com/', 'https://www.americaroofingco.com', 'http://www.broadacresm.com', 'http://www.MobileDogWash.dog', None, None, 'http://www.cozymels.com', 'https://cacao70.com/fr/nos-emplacements/atwater', 'http://www.randstadusa.com/', None, 'http://www.soulas.ca', 'http://www.masalahousepgh.com', 'http://www.creativeformalwear.com/', None, 'http://annandalegolfclub.com/', 'http://drmikepace.com', None, None, None, 'http://www.rona.ca/', 'https://www.riosbraziliansteakhouse.com', 'http://johnnyrockets.com', None, None, 'https://www.goldencorral.com/locations/970/golden-corral-william-penn-highway', 'http://www.glendaleaz.com/parksandrecreation/ThunderbirdPaseoPark.cfm', 'https://www.daveandbusters.com/locations/pineville', '1592150792', 'http://whatisbrewing.com', None, None, 'https://www.klinkecleaners.com/locations/dane-county-locations/monroe-street', 'http://www.ymcacharlotte.org', 'http://vegasvisionoptometry.com', 'http://www.photoandgo.com', None, 'https://the-post-sports-bar.business.site', None, None, 'https://www.chevronwithtechron.com', 'https://www.audioexpress.com/locations/15115-n-hayden-road-scottsdale-az-85260/', 'http://www.kidsgardenlearningacademy.com', 'http://www.iamyoga.ca', 'http://vmacauto.com', None, None, None, 'https://www.mansion54.com', 'http://www.gogopokeus.com', 'http://www.allen-equipment.com/', 'http://tandoorigrillcalgary.com', 'http://www.indulgeculinaryacademy.com', 'http://pillartopost.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=13080', None, 'http://siestalandscaping.com/index.html', 'http://storelocations.ae.com/us/pa/pittsburgh/540-south-27th-street.html', 'http://www.rockstartattoolasvegas.com', None, None, 'http://logansroadhouse.com', '1592150921', None, 'http://www.villageatcobblestonecourt.com', 'http://www.burritogordito.ca', 'http://www.tigbbqlv.com', 'http://www.fishfurniture.com', 'http://www.windowandsolarservices.com', 'http://www.pazzopittsburgh.com/', None, 'http://www.richardstreeservicellc.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://sabalimo.com', None, 'http://www.parkway-sc.com', '1592150982', 'https://www.schlotzskys.com/wisconsin/madison/1532', 'https://stores.anntaylor.com/az/scottsdale/7014-east-camelback-road.html', None, None, 'https://www.wingstop.com/location/wingstop-1394-gastonia-nc-28054/menu', None, None, 'http://www.calgarytransit.com', 'http://www.zeebasleep.com/', 'http://camdenlv.com', 'https://chicagoschickenandwafflescle.com', 'http://www.performancemarineaz.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.starbucks.com/store-locator/store/1016078', 'http://www.mullinspub.com/locations/kington-road/hours-location/', 'http://www.christensenoralsurgery.com', 'http://www.med-place.org', 'http://www.goodearthcafes.com', 'http://www.jkhlawoffice.com', None, None, 'https://prolinewindowtint.com', None, None, 'https://dainties-macaron.business.site', 'http://www.abovebeyondcleaner.com', 'https://www.petsmart.com/stores/us/oh/macedonia-store1851.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', None, None, 'http://www.heartcenterofnv.com', 'http://www.zaksbicycleshop.com', '1592151104', '1592151108', 'http://nevadaorthopedic.com/directory/people/show/thomas-vater', 'http://www.pastelesmary.com/', None, 'http://www.sciullos910.com/', 'http://www.simplyflowersnthings.com?srccode=yelp_track', 'https://www.sonoranspine.com', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', None, 'https://www.wellsfargo.com', 'http://www.weymouthfarms.com', 'http://shawarmaxpress.ca', 'http://thebarrelpub.ca', 'http://www.eurekagifts.com', None, 'https://chatters.ca/stores/chatters-square-one/', None, 'http://www.lviglobal.com', 'http://hamptoncrestenc.com', 'https://www.saucepizzaandwine.com/locations/tempe-tempe-union/', None, None, 'http://tandooristyle.ca', 'http://www.associatedretinaconsultants.com', 'http://www.azeuros.com', None, None, None, 'https://www.supercuts.com/locations/az/peoria/north-valley-power-center-haircuts-82958.html', None, None, 'http://www.chassepermanentmakeup.com', 'https://www.expressoil.com/stores/gilbert/az/3502/', None, 'http://www.bestmatt.com/', None, 'http://www.nonnasplace.ca', 'http://www.commonwealthah.com', 'http://www.cigna.com/cmgaz/locations/phoenix-central-multi-specialty-center', 'http://www.safeway.com', 'http://www.roxxsouthend.com', 'http://www.torontohearingservices.com/location-contact/north-york-ontario-ca/', 'http://www.cookiedoughjo.com', 'http://www.nohiddenfeestickets.com', None, 'https://www.simon.com/mall/southpark', 'http://kintarooh.com', None, 'http://nahasushi.com/', None, 'http://www.princessauto.com', 'http://2ndandcharles.com/', 'http://www.NorterraAH.com', 'http://www.torontomenssuits.ca', 'http://www.metalmorphosis.tv', 'https://homewoodfamilyaz.org', None, 'http://www.bestpittsburghhomes.com', 'http://www.awhlawyers.com', None, None, None, None, 'http://www.rightbynaturemarket.com', 'http://www.centrooptical.com/', 'http://www.grodzinskibakery.com/', 'https://www.spoonerpt.com/locations/goodyear/', None, 'http://www.detailingknights.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.secondcup.com', None, 'http://tinyurl.com/hp44240', 'http://tokyogrill.business.site', None, 'http://www.scoopspittsburgh.com/', None, 'http://www.candlelightersnv.org/', 'http://www.grapevinewinebar.com/', 'https://www.torrid.com/store-details?StoreID=5371', 'https://freedsbakery.com/pages/freeds-dessert-shop', '1592151332', None, None, 'http://www.pandysgardencenter.com', 'http://petgrooming-charlotte.com', 'http://southlandlanes.com', None, None, None, None, 'http://www.truenorthbarbers.com', None, None, '1592151370', '1592151372', 'http://www.sunsolarsolutions.com', None, 'http://www.avenuediner.com', 'https://www.hubbardswim.com', 'http://www.canamrealestateservices.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.dreamdealers.com', None, 'http://www.rapidcollision.com', None, 'http://www.homesteadmaid.com/', 'http://www.jackinthebox.com/', 'http://www.doylechiropractic.com', 'https://dutchbros.com/?utm_source=phoenix&utm_medium=yelp&utm_campaign=website_phoenix', 'http://www.spookyfast.com/', 'http://popeyeschicken.ca/', None, 'http://huntfamilychiro.com', 'http://www.stafflink.ca/', '1592151421', None, 'https://dc-glass-glass-repair-service.business.site', 'http://www.ohiocitypizzeria.org', 'http://www.northyorksheridanmall.com', None, 'http://www.concretemadenew.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.samscyclesupply.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://graphicideals.com/', None, 'https://www.excellenceairllc.com', 'http://www.larryjsauto.com', None, 'http://www.gioninos.com', None, 'https://www.pridestaff.com/tempe', 'https://www.cinemark.com/nevada/century-orleans-18-and-xd', None, 'https://thepetclub.net', None, 'http://www.thedartshop.com/', 'https://www.greentreesportsplexpittsburgh.com', 'http://safe.frontpointsecurity.com/yelpsecurity-us', 'http://www.themagicoflearning.com/', 'http://www.westvalleyent.com/', 'http://www.lvccld.org', 'https://pizzeriamoretti.com/moretti-toronto/', 'http://www.crossfitmte.com', 'http://mcmurraych.com', 'http://murderinkproductions.com', 'http://baskonseaboard.com/', 'http://www.gracefulcakecreations.com/', 'http://www.clasensbakery.com/', 'https://jessicanailinc.wixsite.com/feelingnailsandspa', None, 'http://www.mrlube.com/en/mr-lube-near-you/Locate-a-Mr-Lube/richmond-hill-1070-major-mackenzie-drive-east.aspx?storeid=190&postalcode=L4S%201P3', 'https://www.tacobell.com/food?utm_source=yelp&utm_medium=social&utm_campaign=yelp-pilot-test-menu-static-1-CS&utm_content=paid', 'https://www.charlotteautodetailingservices.com', None, 'http://www.helzberg.com', 'http://www.haircutsarefun.com', 'http://lasvegasyardwork.com', 'http://www.innevation.com', 'https://www.cox.com/local/residential/stores/nv/henderson/545-marks-street?sc_id=dp_z_yext_z_store_19Q2?sc_id=dp_z_yelp_z_store_19Q102', 'http://www.nesbitslanes.com', None, 'http://www.eggsmart.ca', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.artistictlc.com', 'http://www.wendys.com', 'https://www.haggardchiro.com/chiropractor-phoenix/', None, 'http://www.starallison.com', 'http://www.pupusaslafamiliar.com', 'http://usonv.com', 'https://www.devadave.com', 'http://www.evdi.com/?location=chandler', 'http://www.raulsoriginal.com/', 'http://www.omasplaceaz.com', 'http://www.legacynails.net', '1592151592', 'http://www.rvvh.ca/', 'https://chargrilldavidson.com', 'http://tijuanatanaz.com/', 'http://scifest.vegas/contact/', 'http://www.davinciflowershop.com', '1592151608', 'http://www.eventsbyfirstrose.com', 'http://outremont.lapiazzetta.ca', 'http://www.eurowise.com', 'http://www.rocrace.com/charlotte', '1592151621', 'https://www.wellsfargo.com', None, None, 'http://www.powerhousegym.com/schedules-i-407.html', 'http://www.dogsbycindy.com', None, 'http://www.kngmg.com', '1592151643', 'http://www.odelaybagel.com/', 'http://www.harbsbarbershop.com', None, 'http://stevedavishtgac.com', 'https://www.lajoliefemme.ca', 'http://www.alphagraphics.com/centers/westlake-ohio-us649/index.html', 'http://www.hellofacepaint.com', None, None, None, 'https://www.southwestmobility.com', None, None, None, 'http://www.lbvgallery.com', None, 'http://www.pageplusaz.com', 'http://gimelconstruction.homestead.com', 'http://www.gelatostar.com', 'https://www.lawncontrolcenter.com', 'http://www.anotherstory.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.rnrcarpetcleaning.com', 'http://www.heistbrewery.com', 'http://www.sharkystaphouse.ca/', 'https://negretelawfirm.com', None, 'http://samsxpresscarwash.com/huntersville', None, 'http://www.righttime-inc.com', None, 'https://locations.monro.com/oh/cuyahoga-falls/1584-state-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'http://www.yclibrary.org/lakewyliebranchdetails.php', 'http://www.publix.com', 'https://locations.monro.com/oh/north-olmsted/4565-great-northern-blvd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, None, None, 'http://tonysicecream.com', 'http://bigapplebagels.com/', 'http://www.clwplumbing.com/', 'http://fcautobody.com', None, 'http://restaurants.basspro.com/FishCompany/Mesa/Default.aspx', None, 'http://www.bk.com', 'http://www.dom.pitt.edu/id/About.html', 'http://vervewellnesscenter.com/', None, 'http://www.oliveoilaz.com', None, 'http://www.nevadadentalgroup.com/', None, 'http://www.castcadearizona.com', 'http://www.JimmyJohns.com', 'http://www.generationsmedcenter.com', None, 'https://www.circlek.com/store-locator/US/phoenix/4135-w-anthem-way/2709192?location=85086', None, 'http://lotusservicesaz.com', 'http://www.mysteryadventureslasvegas.com', 'http://www.swirlyz.ca', None, '1592151813', 'http://www.kfc.ca/store/sunrige-mall-101-2525-36-st-ne-calgary', 'http://www.torontocarwrap.com/', None, 'http://www.countyofdane.com/lwrd/parks/viking.aspx', 'http://www.golfpinehills.net', 'http://www.ddsdiscounts.com', 'http://www.gailambrosius.com/', 'http://wagntails702.wix.com/vegaspetgrooming', 'http://www.rawfitnesspersonaltraining.com', None, None, 'http://www.vedatakeout.com', None, 'http://www.cocinaelvis.com', 'http://www.filosophy.ca', None, 'https://www.interstatebatteries.com/locations/all-battery-center-las-vegas-nv', 'https://www.frysfood.com/storeHours?store=66000064', None, 'http://www.kitchenova.com', 'http://www.cosmiix.com', 'http://www.7-eleven.com/?yelp=29644', 'http://www.jasonsdeli.com/restaurants/az/glendale/glendale-deli', 'http://www.clevelandtallships.com', 'http://www.eathopscotch.com', 'http://www.RentMontereyApartments.com', 'https://stores.hallmark.com/nc/matthews/jansens-hallmark-shop-matthews-nc-23514.html', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.orthocarolina.com/locations/gastonia', 'http://www.phoenixsymphony.org', 'http://balletaz.org', 'http://www.PazanGallery.com', 'http://www.teaspaceus.com', 'https://www.nsm-seating.com/national-seating-mobility-relocates-phoenix-office/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.gotyoucoveredmusic.com', 'http://handelsicecream.com', 'http://www.babessportsbar.com/', 'http://www.thegreenmango.com', 'http://teamzrestaurant.com', 'https://www.chick-fil-a.com/stapleydrive', 'http://car-detailing-cleveland.com', '1592151935', None, 'http://www.a-zequipment.com', 'http://www.adventurecjd.com', 'https://ad.doubleclick.net/ddm/clk/434423236;236814625;d', 'http://www.hotrocksdiner.com/', 'http://www.studiowe.ca', 'http://www.floorcoveringfactoryoutlet.com/', 'http://www.nutrition51.com', 'https://www.hungryhowies.com/store/hungry-howies-01809?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1809&utm_content=Website_Link', 'http://www.zbakeryandcafe.com', None, 'http://www.payless.com', 'http://www.onefinedayphotography.com', 'http://www.sherwin-williams.com/store-locator/paint-store/Phoenix/AZ/8160', 'http://www.acehardware.com/store-details/07473', 'http://www.lacakery.ca/', 'https://agents.farmers.com/nv/las-vegas/jason-suemoto?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://www.scpchiropractic.com', 'http://www.housepaintinginphoenix.com', 'http://www.autoaccessoriespgh.com', None, 'http://www.baskinrobbins.com', '1592151997', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.tcby.com', 'http://www.creativestonecrafters.com', 'http://www.cocosbakery.com/', 'http://www.salonjovon.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.babyramacanada.com', None, 'https://guardlocksmithgaragedoor.com/locksmith-garage-door-repair-scottsdale-az', 'http://www.rei.com', 'https://www.burnsidebarbecue.com', 'http://www.pizzeriaviamercanti.ca/', 'https://www.sprouts.com/stores/details/-/store-details/view/store/19/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.wilsonniblett.com', 'http://www.zoltons.com', None, 'http://www.hirtsflowers.com', None, 'http://www.badboy.ca/en/pg-storelocator-Mississauga', 'http://www.blm.gov/nv/st/en/fo/lvfo/blm_programs/blm_special_areas/red_rock_nca.html', None, 'https://www.pteglv.com/locations/pts-pub/rainbow-cheyenne/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://www.crabcrackernc.com', None, 'http://magiceric.com/', 'http://www.mathnasium.com/scottsdale', None, None, None, 'http://www.textilemuseum.ca/', 'http://www.tpcmadison.com', 'http://www.freshstartaz.com', '1592152094', None, None, 'http://pizza.dominos.ca/Calgary-Alberta-10108', 'https://techfourlife.com', None, 'http://www.ttroadhouse.com', 'https://www.walgreens.com/locator/walgreens-5975+weddington+rd-matthews-nc-28104/id=10255', None, 'https://www.walgreens.com/locator/walgreens-5400+pearl+rd-parma-oh-44129/id=3314', 'http://www.ryanfootandankleclinic.com', 'http://www.hansabrewery.com', 'http://gabbys.ca/locations/gabbys-kingston-road/', 'http://westcitysnackbar.com', 'http://www.litchfield-park.org/index.aspx?NID=288', 'https://www.petsuppliesplus.com/Store/OH/Parma/Parma/120?utm_source=Yelp&utm_medium=Store-Page%C3%8A', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.centralbarkusa.com/madison-southeast', 'http://www.mattress.org', None, 'http://www.hobbylobby.com/StoreDetail/500', None, 'http://www.hairandcosaloninc.com', 'http://www.dennys.ca', 'http://bangzartandhairsalon.com/', None, None, 'http://www.joeslakewoodcomputer.com', None, 'https://milios.com', 'http://www.thescottresort.com/?utm_medium=enhanced-profile&utm_source=yelp', None, 'http://meskeremincharlotte.com/', 'http://sergio-daae.squarespace.com', '1592152184', 'http://www.courtesydental.com', None, 'http://www.soma.com/store/home.jsp?cmp=MOC_yelp', 'http://www.paradisevalleyhomes-forsale.com', 'https://thelasvegaslounge.com', 'http://www.eandgrealestate.com', 'http://www.laundrybasketvillager.com/', 'http://www.marybekitchen.com', 'http://www.gelatospot.com', 'http://professornailsandspa.com', None, 'http://www.buttermilkfinewaffles.com', 'http://www.tubeaudio.com', 'http://jeffreyngmd.com', 'http://www.lenailsonline.com', 'http://www.gnc.com', None, 'http://www.9round.com/fitness/madison-wi', 'http://www.alltattoos.net', None, 'https://www.officedepot.com/storelocator/az/glendale/officemax-6171/', '1592152244', None, 'http://www.downtownphoenixrentals.com', None, 'http://www.strawberryhedgehog.com', None, 'https://azwestpainting.com', 'http://www.abundance-a.com', 'http://silvertoncasino.com/dining', None, None, 'http://www.nellisfamilyhousing.com/', 'http://www.theloftycupcake.com', 'http://impchandler.com', None, 'http://www.lassendas.com/Casual-Family-Dining-Sports-Bar-Happy-Hour-Mesa-The-Grille-Las-Sendas.html', 'http://www.edgewaterbarandgrille.com/', 'https://www.guardianmortgageonline.com/branch-managers/arizona_mortgage_branch_manager_steve-howard/', None, None, 'http://www.marketdistrict.com/Locations/Detail.aspx?LocationID=7', None, None, None, None, None, None, 'http://ingenuitycleveland.com/', 'http://www.lcbo.com', None, 'http://www.starbucks.com', 'http://www.marcellos.ca/', 'http://www.clevelandgaragedoors.net', 'http://www.africanabbqcurriesdrinks.com', 'http://www.pvbarbers.com', 'https://locations.wendys.com/canada/on/mississauga/6966-financial-drive', 'http://www.PawsPetServicesInc.com', None, 'http://svdppitt.org/stores/store-locations-directions/', 'http://www.newliferoof.com', None, None, None, 'https://agents.farmers.com/az/carefree/jason-grifo?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://venturagrill.com/', 'http://www.libertyburgers.ca', 'http://www.mushkaki.com/', None, None, 'http://www.enchantress.com', 'http://www.belfilandscaping.com', '1592152383', None, None, None, None, 'http://www.rubymarie.com', None, 'https://locations.wendys.com/canada/on/mississauga/6966-financial-drive', 'http://www.dmvnv.com/', None, None, None, None, 'http://www.badattitude.com/', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', 'http://www.ogreatcut.com', 'http://www.analoguegallery.com/', 'http://www.arizonadigestivecenter.com', 'http://www.murrysvillespa.com', 'http://www.bobstvaz.com/', None, 'https://www.rcpw.com', 'https://www.arizonatermitespecialists.com', 'http://www.familypetsalon.com', 'https://novasolutions.ca', 'https://www.kwiktrip.com/Locations/Details/673/', 'http://locations.harristeeter.com/nc/charlotte/205/', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_112', 'http://www.greenleafwholesale.com', None, 'https://nestnailwellness.com', 'http://www.lowes.com', 'http://www.soupersalad.com/', 'http://www.mammaspizza.com', 'http://www.sacredrootsaz.com', 'http://bk.com/', 'http://www.eldoradocasino.com', 'https://locations.wendys.com/united-states/wi/sun-prairie/743-w.-main-st.', 'https://salsalocamexicangrill.square.site', 'http://www.tph.ca/print/toronto-yonge-st-eglinton-ave', None, 'http://www.batteriesandgadgets.com/pages/contact_us', '1592152501', 'http://www.westthirddeli.com', 'http://www.charmeck.org/', None, 'https://growgeneration.com/?utm_source=yext', 'https://oldpinevillepremiumpub.com', None, None, None, 'http://www.swspineandsports.com', 'http://www.thepubat333.com', None, 'http://www.quiznos.com', 'http://www.autohausofnorthscottsdale.com/', 'http://www.deathinvenice.ca', 'https://sobamesa.com', 'https://wongs-chinesedining.com', None, 'http://www.megatunes.com', None, 'http://www.phoenixpaver.com', 'https://www.myplacehotels.com/my-place-hotel-north-las-vegas-nv', None, 'http://www.wehnergroup.com', 'http://liveatwynnpalms.com', None, 'http://www.artisanantiquescleveland.com', 'http://chaletvet.com', None, 'http://www.miasalonstudios.com', None, 'http://www.eatchinakitchen.com', 'http://www.williamsstainedglass.com', None, None, 'http://elcompatireshoplv.com', '1592152616', None, None, 'http://www.tbcmovers.com', 'http://www.residencesonfarmer.com/', 'https://jojosjerky.com', 'https://agents.allstate.com/mike-seifer-henderson-nv.html', 'http://www.domyresume.net', 'http://www.hongkongvintage.com/', 'http://www.newtowndds.com', 'http://www.chuparosapainting.com', 'http://ottavio.ca/contact-greenfield-park-rive-sud/', None, None, 'http://www.charmeck.org/', 'http://www.ae.com/web', 'https://www.loandepot.com/branches/Henderson-NV?sc_camp=DE925F8ECA0F4E0D8092C7E108A615A3', 'http://www.judonspainting.com', None, 'http://www.peoplesmortgage.com/loan-officer-team/broker/Tamiarmijo', None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.pinkhousenc.com/', 'http://www.kobootcamp.com', 'http://www.standa.ca', 'https://www.pepsolar.com', 'http://www.onenevada.org', 'http://623pc.com', None, None, 'http://www.expressionswaxspa.com/about-us.html', None, 'http://www.azburgershoppe.com', 'http://italianvillagepizza.com/', None, 'https://www.newportmotorslv.com', None, 'http://mandys.ca', None, 'http://www.joelcohenmd.com', '1592152774', None, 'https://www.walgreens.com/locator/walgreens-220+s+century+blvd-rantoul-il-61866/id=9965', 'http://topnotchnailsspa.com', None, 'http://www.passengerscafe.com/#hello', None, None, None, 'http://enginehouse.com', 'http://www.cartridgeworldphx.com', 'https://www.cellphonerepair.com/mayfield-heights-oh/', 'http://towingscottsdale.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.essentoronto.com/', 'http://avis.com', 'http://www.z-vw.com//?cs:pro=repman&cs:e=yelp', 'http://www.patriciacakes.com', 'http://www.acepoollv.com', None, None, 'http://www.firepatio.com', None, 'http://jerryrhynescollision.com/locations/pineville-location/', None, None, None, 'http://www.michaelkluthe.com', 'https://www.youfit.com/gym/phoenix-32nd-st-7406', 'http://www.woofgangbakery.com', 'http://www.azandersonanimalclinic.com', None, 'http://Phomono.ca', 'http://www.sunscreens-armadillo.com', None, 'http://www.donatos.com', None, 'http://www.noviamia.ca', 'http://www.paseodelpradoapts.com', None, 'http://www.redrockcanyonlv.org', 'http://Www.Tailsandbones.com', 'https://pawnscottsdale.com', None, None, 'https://www.waltsjewelers.com', None, 'http://www.oasishookahlv.com/hookahcatering.html', None, 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.ontrackdj.com', 'http://vintagelucys.com', None, 'http://www.regissalons.com/salondetail/default.asp?salonid=4818', 'http://www.DJEliteAZ.com', 'http://www.stadiumbargrill.com', 'http://kwcafeterias.biz', None, 'http://www.joycitytires.com', '1592152981', 'http://luckithaibistrolasvegas.justcafepages.site', 'http://www.thebakedbear.com', None, 'http://www.finewinereserve.com', 'http://scfapm.net/', 'http://www.shoppersdrugmart.ca', 'http://www.thrivehomeservice.com', 'http://www.atomiccomics.com', None, None, None, 'http://www.regenesisskinstudio.com/', 'https://www.searshomeservices.com/locations/appliance-repair/pa-pittsburgh/1000-robinson-center-dr', None, 'https://stores.jcpenney.com/pa/monroeville/salon-monroeville-pa-0199.html', None, 'http://www.cosmosrecords.ca', 'https://nothingbundtcakes.com/bakery/AZ/glendale?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=glendale_56', '1592153049', 'http://bentoburrito.ca/', 'http://www.bigming.ca', 'http://www.rolenbrothersfence.com', 'https://juliagrambo.com', 'http://www.automaxxon16th.com', 'http://www.flatlinegrinding.com', 'http://www.hopingfurahome.com', None, 'http://www.UrbanPranaYoga.com', 'http://www.aamfg.com', 'http://www.808smokes.com', '1592153093', 'http://restaurantlabelleprovince.com', None, None, None, 'http://www.winners.ca', 'https://www.mamabaers.com', None, None, 'http://www.risquelv.com/', None, None, None, 'https://www.KFYIadvertising.com', 'http://www.christieclinic.com', 'https://showerdoors.binswangerglass.com/nc/charlotte/?utm_source=yelp&utm_medium=social&utm_campaign=showerglass&utm_content=urllink', None, None, 'http://www.thaitouchkitchen.com/', 'http://www.bk.com', 'http://www.spinbikeshop.com', '1592153170', 'https://www.crawfordtransmission.com', 'http://www.simpleshuttle.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.dreamcarrentals.com/', '1592153188', 'http://www.coldstonecreamery.com', 'http://www.tonicpgh.com', 'http://www.hydroshieldphx.com', 'http://www.bk.com', 'https://wellingtonsofcalgary.com', 'https://store.vioc.com/nc/gastonia/2650-e-franklin-blvd-28052-BD0014', 'http://kimaznailspa.wix.com/kim-az-nails-and-spa', 'http://superstore.ca', 'http://www.kilkearysautobody.com', 'https://skinaddictstudio.com', None, None, None, 'http://autopricecanada.com', 'https://westvalleypawn.com', None, '1592153258', None, 'http://www.clevelandymca.org/branches/westpark', 'https://www.azcontainers.com', None, 'http://www.techspector.com', 'http://www.dunkindonuts.com', 'http://www.napoca.ca/', None, 'http://www.brownleejewelers.com', 'http://www.arbys.com', 'http://www.kearnsandco.co', None, None, 'http://www.tommulligan.com', 'http://www.vegaslimocoach.com', 'https://locations.sonicdrivein.com/nv/lasvegas/6301-w.-lake-mead-blvd..html', 'http://www.westhillvet.com/', 'http://www.mark-taylor.com/arizona/laguna/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.fourpeaks.com', None, None, 'https://www.carewithconfidence.com', 'https://elementsmassage.com/tempe', None, 'http://www.millscleaners.com', None, 'http://cluffmechanical.com', 'http://www.buenoburgeraz.com', 'https://www.cinemark.com/northern-ohio/cinemark-cuyahoga-falls-and-xd', None, 'https://www.pittsburghparks.org/schenley-park-cafe-visitor-center', 'http://www.rossocoffeeroasters.com', 'http://www.edgewalkcntower.ca/', 'http://www.dunkindonuts.com', None, '1592153386', 'http://www.shell.us/', '1592153393', 'http://www.tolmachoff-farms.com', 'https://crisp-pizza.com', 'http://www.comebuy2002.com.tw/', None, 'https://www.perfectpalmssalon.com', 'http://www.regionsbistro.com', 'http://advancedtherapeuticsmassage.com', 'http://www.davidofflv.com', 'http://www.kvegas.com', None, 'http://www.farnsworthrealty.com', 'http://blueagavemadison.com', None, None, 'https://howardair.com/showroom/', 'http://www.photographerjun.com', 'https://www.desertdryclean.com', 'http://workoutanytime.com/mint-hill', '1592153469', None, None, 'https://www.supercuts.com/locations/az/scottsdale/scottsdale-pavillions-haircuts-9672.html', 'http://www.merlenormanstudio.com/nv/las-vegas/mn-10293/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.divasalonspa.com/', None, 'http://www.alexandani.com/pittsburgh', 'http://www.antoniospizza.net', '1592153499', None, None, 'http://www.pittsburghsbestremodeling.com', 'https://cvam.com', None, None, None, 'http://www.magomindfulness.org', 'http://www.twolouiesmarket.com/', 'http://www.runningroom.com/hm/', 'http://www.gamestop.com', 'http://www.fabletics.com', 'http://captiverooms.com', None, 'http://ad.doubleclick.net/clk;266326037;92222803;c?http://www.24hourfitness.com/FindClubDetail.do?clubid=205&edit=null&semiPromoCode=null&cm_mmc=Yelp-_-ClubPage-_-BusinessListing-_-Link', None, 'http://www.7-eleven.com/?yelp=19726', '1592153554', None, 'https://www.kalonsandglidewell.com', 'https://roiproperties.com', 'https://www.truenorthmortgage.ca/?utm_source=Yelp&utm_medium=yelp-ads&utm_campaign=yelp-ont', None, 'http://www.scottsdaleautogroup.com', None, None, 'http://www.papamurphys.com', '1592153591', 'http://www.beneficial-insurance.com', None, 'http://katespade.com', 'http://www.purelifehomeopathy.com', 'https://www.nhrandolphobgyn.org/locations--hours/charlotte.aspx', None, None, 'http://www.olgaallenhairsalon.com', 'http://www.cabowabocantina.com/vegas/', 'https://apricotlaneboutique.com/store/champaign/', 'https://locations.noodles.com/wi/fitchburg/2981-triverton-pike-dr--at-mckee-rd.html', 'http://clickserve.dartsearch.net/link/click?lid=43700044282974602&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.tlchudson.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, 'http://www.gmgbarbershop.com', 'https://communitytirepros.com/tires-auto-repair-6-phoenix-az', 'http://www.calgary.ca/CSPS/Recreation/Pages/Athletic-parks/Glenmore-Athletic-Park.aspx', None, 'http://www.2dadsbouncehouses.com', 'http://www.INTLscottsdale.com/', 'http://www.xingfutang.ca', 'http://www.thelashandsugarcompany.com', None, 'https://stores.hallmark.com/nc/pineville/amys-hallmark-shop-pineville-nc-38803.html', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEAA', None, 'https://www.azmonsterair.com', 'http://knockout-fitness.com/locations/gilbert/', None, None, None, None, None, None, '1592153709', None, '1592153714', 'https://vercini.com/stores/vercini-miracle-mile-shops-las-vegas/', 'http://signaturegalleries.com/ACs_Wyland_Signature.html', None, None, None, 'http://marsiglianospizzeria.com', 'http://www.byashleymarie.com', 'http://www.dominos.com', None, None, None, 'https://www.americasbest.com/store-list/pineville-la-71360/', None, None, 'http://www.kbbicycleshop.com', 'http://mastertune.ca', None, None, 'http://www.stevesknows.com', 'http://www.shopplanetblue.com', 'http://www.89002realtor.com', 'http://www.peoplesutility.com', 'http://www.silverstatecu.com', 'http://www.crossfitmagna.com', 'https://www.olivegarden.com/locations/ab/calgary/calgary-northgate-plaza/4345?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:calgary-ab-4345_pl:locurl_rd:1743', 'http://dishosteria.com', 'http://gcbc.gcu.edu', None, 'http://www.bluewaterimaging.ca/', 'https://bikedepot.com/stores/toronto/', 'http://angelosgilbertbarbershop.com', 'http://www.teacoffee.com', 'https://locations.usbank.com/index/arizona/phoenix/west-bell-rd-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.nevadacoinmart.com', 'http://www.cupcakes-canada.ca', 'http://www.mastertoddy.com/', 'http://www.venetian.com/restaurants/italian/tintoretto-restaurant-bakery.html', None, 'http://www.videoproductionscottsdale.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/PHXEL', 'http://www.wineandspiritfestival.ca', 'https://www.supercuts.com/locations/ca/oakland/rockridge-district-haircuts-8050.html', 'http://www.cvlt.org', 'http://www.9round.com/fitness/madison-wi', 'http://hobakkoreanbbq.com', 'https://beavertails.com/shops/', 'http://friendsofdormontpool.org/pool.html', 'http://www.applyapprovedauto.com', None, 'http://www.customdentures.us', 'http://www.partysavvy.com', 'http://www.oliverannie.com', 'http://abrazodiabetes.com', 'http://www.i9sports.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://www.dunkindonuts.com', 'http://www.crossfitcholla.com', 'http://HairMajesty.mysalononline.com', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', 'http://vespucciconsignment.com', 'http://www.dominos.com', 'http://www.hangoverbail.com', 'http://www.harmonybarandgrill.com', 'http://LiveAtArtessa.com', '1592153958', 'http://drkaylaeckart.com', None, 'https://www.va.gov', 'http://www.starbucks.com/store/9334/', 'http://thesweatlab.com', None, None, None, 'http://www.homesense.ca', None, 'http://www.dominos.com', None, 'http://www.zerorezpgh.com', None, None, 'http://whasn.com/locations/whasn-henderson/', '1592154010', 'https://haircutmenraintreeranchchandleraz.com', 'http://themonticello.com', 'http://www.seagullbook.com/', None, 'https://www.publicstorage.com/arizona/self-storage-mesa-az/85210-self-storage/787?pid=wyelp&CID=1245&CHID=AFL', 'http://protecrepair.biz/', 'https://www.vapeloft.com/vaughan-vape-shop', 'http://www.futons-4-less.com/', 'http://www.unitedcabcle.com', 'http://www.pinkladiesmaids.com/', None, None, 'http://www.heroburgers.com', 'http://www.coinlesslaundry.com', 'http://www.mortgageandcreditpro.com', None, None, 'http://www.caesarspalace.com', 'https://www.gohealthuc.com/north-carolina/charlotte/quail-hollow?utm_source=yelp&utm_medium=referral&utm_campaign=yelp%7Clistings%7C2020-nc-null-novant-011520-quail-hollow', None, '1592154093', None, 'http://www.akanails.com', None, 'http://doorfiftyfive.com/', 'http://opasouvlaki.ca?utm_source=yelp&utm_medium=website-click&utm_campaign=enhanced-profiles', 'http://northamericanspine.com/', 'http://www.nutrishopcharlotte.com/', None, 'http://www.plazaappliancemart.com', None, None, 'http://www.chickrotisseriebar.com', None, 'http://www.losolivosrestaurants.com', None, None, None, 'www.rustycage.ca', None, 'http://www.newcityphx.com/', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.pnponecarecleaning.net/', 'http://ronsserviceandtire.com', 'https://simonmed.com', 'http://www.streetsaw.com', '1592154185', '1592154189', 'https://bosshairsalon.com', '1592154198', 'http://www.speedydetail.ca', 'http://www.dentalspalv.com', None, 'http://www.taketimetorelax.com', None, 'http://www.lvcmc.com', 'http://www.fkaiser.com', None, 'http://www.vinorossorestaurant.com', None, 'http://www.dwellstudentmadison505.com', None, 'http://www.DornHardware.com', None, 'https://www.atmosphere.ca/stores/calgary.html', 'http://anvilbaraz.com', 'http://thetreeguylv.com', None, 'http://www.redrockcanyonlv.org', 'http://www.chachis.ca', 'http://www.decadenceculinary.com', None, 'https://www.sierracanyonaz.com', '1592154281', 'https://providenceeye.com', 'https://locations.sonicdrivein.com/il/savoy/101-calvin-street.html', 'https://cosmomusic.ca', '1592154349', 'http://www.banfield.com/veterinarians/az/phoenix/pho?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://www.desertspringsrealty.com', 'https://www.valuecityfurniture.com/store-locator/pa/pittsburgh/451-clairton-blvd', 'https://www.speedy.com/shops/newmarket/en/', 'https://stores.anntaylor.com/factory/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'http://splashwindowcleaning.com', None, 'http://incascuisine.com', 'http://kierlanddentalcenter.com/', 'http://www.cherrystbbq.com', 'http://www.hznvet.com/', 'https://www.enterprise.com/en/car-rental/locations/us/oh/chardon-3928.html?mcid=iyp:8368400', 'http://www.gansercompany.com', 'http://casaritasmexicangrill.com', None, 'https://www.theelementlv.com', 'http://www.olde8automotive.com', 'https://www.241pizza.com', 'http://kindmedsaz.com', None, None, 'http://www.blimpcitybikeandhike.com', None, None, 'http://www.championfenceinc.com', None, 'https://www.anytimefitness.com/gyms/3397/Montreal-QC-H2W-1Y7/?utm_source=yelp&utm_medium=local', '1592154453', 'http://www.enchantedvalleyacres.com/', 'http://furbabiesresort.com', None, 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=586&utm_content=bpas', 'http://www.visualcreations.vegas', 'http://www.phoenixazhomes4u.com/', '1592154476', '1592154480', 'http://www.phoenixmedicalgroup.com', 'http://www.azvpcp.com', 'https://www.petsmart.com/stores/us/nv/las-vegas-store1153.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'https://www.sellitquickhs.com', 'http://a1carpetmasters.com/', None, 'http://www.arcoflifechiro.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', None, 'http://www.sears.com/stores/north-carolina/monroe/0002008.html', 'http://www.EliteBartend.com', 'http://www.adriaticflooring.com', 'http://www.BombHairVegas.com', 'https://www.sonoraquest.com/find-a-location/sun-lakes-alma-school-riggs/', None, 'https://www.bluebreeze216.com', 'http://www.swgeneral.com', 'http://www.wetpetsandfriends.com', 'http://www.dirkcpa.com', 'http://www.alivedanceworks.com', 'http://www.coldstonecreamery.com/stores/20181?latitude=33.6071829&longitude=-111.9403254', 'http://www.silhouettebridallv.com', 'http://www.alibabas.ca/index.php?route=common%2Flocation_details&loc_id=12', 'http://www.jakesbarvegas.com/', None, 'http://www.sav-onplumbing.com?srccode=yelp_track', None, 'http://www.edelmanplumbing.com', 'http://www.starbucks.com/store/11823/', 'http://www.scentair.com', None, 'https://www.ihop.com/en/restaurants-south-las-vegas-nv/7490-s-las-vegas-blvd-3096', None, 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/mississauga-(2).aspx', 'http://www.aspirekidsports.com', 'http://www.bloodbrothersbrewing.com', 'http://www.sherwin-williams.com/store-locator/paint-store/Gilbert/AZ/8286', 'http://www.blackeagletoronto.com/', 'http://www.maderacabinets.com', 'http://www.citycleaner.com', 'http://todaysinteriors.com/', 'http://4seasonslandscapinglv.com', None, None, 'http://ashleynailsspachandler.com', None, 'https://www.walmart.com/store/1646/mesa-az/details', 'http://www.daphnesbakery.com', 'http://www.vynepowdercoating.com/', '1592154609', 'http://www.cityfresh.org', None, None, 'http://www.crystalsatcitycenter.com', None, None, 'http://www.accountingprose.com/accountingprose-arizona/', 'https://order.pizzapatron.com/menu/phoenix-indian-school-road?utm_source=Yelp&utm_medium=Order&utm_campaign=Ordering', None, 'http://www.beachesbrewing.com', 'http://www.robertocarpetclean.com', 'http://skyharbor.com', 'http://www.brodywebdesign.com', None, '1592154648', None, None, 'http://www.puckettsfarm.com', 'http://www.bishopsplum.com', 'http://www.alleen.com', 'https://www.hilton.com/en/hotels/clemehx-hampton-suites-cleveland-mentor/?SEO_id=YELP-HP-CLEMEHX', 'http://www.ariatroofing.com/', 'http://yazlounge.com', 'http://www.safaripetcenter.com', 'http://www.sukhothaibistro.com/', 'http://teavana.com/ca', 'http://www.furniturestopaz.com/', 'http://www.apexoutdoortreeservices.com', 'http://meridianlv.com', None, None, 'http://www.eddys.com', 'http://www.bobscopyshop1.com', '1592154699', 'http://www.kingspointsmallengine.ca', 'http://www.elpolloloco.com/locations/store/5541', 'http://www.skinstudioaz.com/', None, None, 'http://www.liquordepot.ca/?utm_source=Yelp&utm_medium=Online_Listings&utm_content=Location&utm_campaign=LiquorDepot_AB_Q3_July_September_2017', 'http://www.hcdentallv.com', 'http://www.tomduffyinsurance.com', None, 'https://www.chandleraz.gov/explore/chandler-aquatics/centers-and-pools/folley-pool', 'https://jessieleephotos.com', 'http://Www.crawlingsquidtattoo.com', 'http://www.bojangles.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.airportlimocanada.com', '1592154747', 'http://www.wellsfargohistory.com/museums/phoenix', None, 'https://www.xtremevapour.com', '1592154760', 'http://sunvalleynailspa.com/', 'http://www.quantumcollections.com', None, None, 'http://www.grandstayhospitality.com/find-a-hotel/locations/mount-horeb/overview', 'http://www.swingpt.com', 'https://certapro.com/summerlin/?utm_source=YextNetwork&utm_medium=organic&utm_campaign=YextNetwork', None, 'http://biggscamera.com', 'http://www.deekerssidetracks.com/', 'http://www.humblewinebar.com', 'http://www.theofficepub.ca', 'http://hhtattoo.com', 'http://www.blackphoto.com', None, 'http://kakaallyoucaneat.com', 'http://www.restartbar.com', None, None, 'http://kiplinautorepair.com/', 'http://www.thepizzacompany.biz', 'http://www.checkeredmoon.com', '1592154823', 'http://www.michaelpwebbersondds.com', None, 'http://www.potbelly.com/MayfieldHeights', 'https://www.walgreens.com/locator/walgreens-2363+s+lindsay+rd-gilbert-az-85295/id=5215', '1592154837', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, 'http://www.kirchappliance.com/', 'https://www.erinparktoyota.com', 'http://www.parkatdeervalley.com', 'http://www.westgateaz.com', 'http://www.sugo.ca', None, 'http://bellacinos.com', None, 'http://mintstrength.com', 'http://www.napolivince.ca', 'https://parsleymediterranean.com', 'http://sweeteezsweets.com', 'http://www.pspohio.com/', 'http://www.readytuxedo.com', 'http://www.wildchildclothes.com', 'http://www.mexicanbowl.ca', 'https://bostonpizza.com/en/locations/sunridge.html', None, 'https://www1.shoppersdrugmart.ca', 'http://www.baskinrobbins.com', None, 'https://www.long-mcquade.com/location/Ontario/Brampton', '1592154901', 'http://www.uhhospitals.org/westlake/urgent-care', 'http://www.wholehomeinspectionsllc.com', 'http://www.beautif-eyestudios.com', None, 'http://allbritematerials.com', 'http://www.victoriassecret.com', 'http://www.fransrestaurant.com', 'http://www.fandoctor.com', 'https://www.popeyes.com/store-locator/store/restaurant_83325', 'https://www.usstoragecenters.com/self-storage/arizona/glendale/us-storage-centers/126', 'https://stores.nordstromrack.com/us/az/gilbert/2090-e-williams-field-road?utm_source=yelp&utm_medium=organic&utm_campaign=rack&utm_content=154&utm_channel=lcl_acq_p&sp_source=yelp&sp_campaign=rack', 'http://www.trilogylife.com/communities/arizona/vistancia/', 'http://www.carmellaspizza.com/', 'http://www.skysongcenter.com', 'http://www.sessomsandsons.com', 'http://fossilandhaggis.com/', None, None, 'http://casablancanails.com', None, 'http://www.erbertandgerberts.com/locations/detail/sun-prairie', 'https://cafevialinda.com', 'http://www.tesororanchapts.com', 'http://www.icecreampizzaworks.com', 'http://www.pizzanova.com', 'http://www.te-koop.ca', None, 'http://www.cardealsdirect.ca', 'http://getnutslab.com', 'http://www.artinthepark.org/', None, None, None, 'http://www.CampanasPizza.com', None, None, 'http://www.phoenix.gov', 'http://www.pwaesthetics.com', None, 'https://www.encantopoolcare.com', None, None, 'http://greekstop.ca', 'http://www.mortproductions.com', 'http://trendsettereyewear.com', 'http://www.starbucks.com/store/15887/', None, 'https://www.anytimefitness.com/gyms/4028/Mint-Hill-NC-28227/?utm_source=yelp&utm_medium=local', 'https://www.gowireless.com/stores/az/goodyear/goodyear-estrella-marketplace/', 'http://www.kitchenstuffplus.com', None, 'https://www.choicehotels.com/CN283?mc=blypypaf', 'http://phxfloors.com', 'http://www.tenpennyimc.com', 'http://www.winchellspub.com/', None, None, 'http://www.kisellasurgery.com', 'http://www.thegraniteshopofmadison.com', 'http://www.coffeebean.com', 'http://www.completeeyecareaz.com/', 'http://www.palazzo.com/azure', 'http://www.azinspector.com/', 'http://www.dollartree.com/home.jsp', None, 'http://www.cartersgymnastics.org/', None, 'https://www.extraburger.ca', 'http://www.elhefe.com/scottsdale', 'http://www.baschs.com', None, 'http://www.monroevillekia.com', 'http://farmersfurniture.com/', None, None, 'http://www.abestautorepair.com', 'http://www.northlakechiro.com', 'http://drkennedychiro.com/', 'http://www.artgalleryofmississauga.com/index.html', 'http://www.uptowncharlottecleaner.com', 'http://www.buckdeitzphotography.com', 'http://www.manuelsaz.com/', 'http://www.funkymunky.ca', 'http://tandmautomotive-hub.com', 'http://smokespoutinerie.com', 'http://www.galleriashoppingcentre.ca', 'http://www.functionpilates.com', None, '1592155146', 'http://www.allthingseyeslv.com', None, None, None, 'https://www.mgmgrand.com/en/nightlife.html', 'https://arizona.weidner.com/apartments/az/phoenix/arezzo/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'http://www.flawlessfaceandfigure.com/', '1592155164', 'http://www.clerkofcourt.maricopa.gov/', 'http://www.chaska.ca/', 'http://www.highdesertoralsurgery.com', 'http://www.wins.doitbest.com', 'http://www.gtmassages.com', 'https://www.pioneerfamilypools.ca/locations/newmarket/', None, 'https://www.lvspaandnailsvegas.com', 'http://thebikeproject.org', 'http://www.850Degrees.ca', 'https://stores.joann.com/oh/hudson/171/', None, 'http://www.phoenixlockoutsandroadsideassistance.com', 'http://www.bettercontracting.ca', 'https://www.popeyes.com', None, None, 'http://icirides.com/', None, 'http://www.7-eleven.com/?yelp=29384', 'http://www.ritascleaningllc.com', None, '1592155224', 'http://www.rockbottom.com/', 'https://soswaxlv.com', None, 'http://www.pelicanssnoballs.com/lakewylie123.php', 'http://www.shopmetcalfes.com', None, None, 'https://www.themotherhoodco.com', None, 'https://losgirasoles.us', None, 'http://www.bowlofgreensfinesalads.com', None, 'http://planetpetopia.com', 'http://www.platinumentourage.com', 'http://www.drsteed.com', None, 'http://www.chopchopwok.com/', 'http://www.mizumisushi.ca', None, 'http://www.lachotsmiles.com', 'http://www.laketowne.com', 'http://www.carolynes.com', 'http://www.golfsmith.com', 'http://www.themerchant.biz', None, 'https://thepetclub.com', 'https://www.loszarapes.com', 'http://www.dmvnv.com/locat.htm', 'http://www.mrpuffs.com/fr/', None, 'http://distilledbeautybar.com', 'http://www.intothefirepizza.com', 'http://www.womeninneed.net/', 'http://www.rianahairstudio.com', 'http://planetsmoothie.com/', 'http://www.asianhealingartsandacupuncture.com/', 'http://www.sosappliancerepair.com', 'http://www.greaterlasvegasdental.com', None, None, 'https://www.cellphonerepair.com/phoenix-central-az/', 'http://GreenMaid.Info', 'http://www.goldennugget.com/home/entertainment_attractions/defending_the_caveman/', 'http://www.vegasvacuumsewing.com', None, 'http://www.rockwoodnaturalmedicine.com/', 'http://www.club42.com', 'http://newmarket.colmustardspub.com', 'http://www.lazyloafandkettle.com', None, None, 'http://www.thepilatesbodylasvegas.com', None, None, None, 'http://www.laptopxchange.com/phoenix-az-computer-repair/', 'https://www.4pawspetresort.com/?utm_source=yelp&utm_medium=social', 'https://www.cartridgeworld.com/las-vegas-nv/89102-0318/about-us', None, None, 'http://www.economy-glass.com', 'http://www.novaristorante.ca', None, None, 'http://locations.harristeeter.com/nc/davidson/268/', 'https://dentalsolutionslv.com', 'http://www.doggiedoorsontherun.com/', 'http://www.thaicuisineexperts.com', 'http://www.aztechglasstint.com', None, None, 'http://www.designercakeshop.com', 'http://www.myfoodcity.com', 'http://www.kmart.com/stores/michigan/monroe/4098.html', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'https://www.themetatfashioncenter.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.mcqueenplumbing.com/index.html', None, 'http://communityculinary.org/', 'http://www.classroomcentral.org', None, 'http://www.saltydogpetspa.net', 'http://www.thebodyrx.com', None, 'https://www.moonvalleynurseries.com/services/tree-service?utm_campaign=yelp&utm_source=yelp-az&utm_medium=tree-care', 'http://www.wm.com/', '1592155455', 'https://dema.az.gov/army-national-guard/papago-park/arizona-military-museum', 'http://www.glendalesalonaz.com', 'http://www.hcpnv.com/clinic/wynn', 'http://www.noirtraining.com', 'http://www.heritagecrownaz.com', 'http://massageworkscharlotte.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=803&utm_content=bpas', 'http://www.OrganicMassageCleveland.com', 'http://www.BeefOBradys.com', '1592155482', 'http://www.parktowerapts.com', 'http://chartiersanimalhospital.com/', 'http://ordercasadeplata.com', 'https://www.omniwellnesscenter.com', None, None, 'http://www.horsinaround.ca/', 'https://petvalu.com/location/2034', None, None, 'http://www.karlaskarslv.com/', 'http://www.dogsuppliesvegas.com', 'http://www.savon.com', 'http://www.foodbasics.ca', 'http://www.cryscakecreations.com/', 'https://www.autorepairpittsburghpa.com', 'https://vrplayin.ca', None, 'http://www.philsebastian.com', 'http://www.gvdentalcenter.com/', 'http://www.barringtonplaceapts.com/', 'https://locations.theupsstore.com/az/surprise/15508-w-bell-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.hinkleyslighting.com', 'http://laptopexpert.com', 'http://www.bathandbodyworks.com', 'http://www.jimmysnycpizzeria.com', 'https://locations.theupsstore.com/az/phoenix/4802-e-ray-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.rotaryevolution.net/', None, None, 'http://www.baskinrobbins.com', 'http://www.apple.com/fashionshow', 'http://www.gohunanofstow.com', None, 'http://www.piacere.ca', None, 'http://www.azcenturypoolplastering.com', 'http://www.acclivityhealthcare.com', None, None, 'http://www.gallaghersexpertautorepair.com', 'http://www.containerbaryyc.com', None, None, 'https://www.superstitiongrandantiquemall.com', None, None, 'https://beehiveceilings.com', 'http://www.toppackgear.com', None, 'https://www.medexpress.com/location/pa/pleasant-hills/plp/?utm_source=yelp&utm_medium=paidsocial&utm_campaign=Evergreen', None, None, None, 'http://www.phoenixgreyhoundpark.com', None, '1592155627', 'http://www.autobell.com', 'http://slshotels.com/lasvegas/', None, None, 'https://www.hilton.com/en/hotels/pitsmes-embassy-suites-pittsburgh-downtown/?SEO_id=YELP-ES-PITSMES', 'http://bestchinainc.com', None, '1592155648', None, 'http://www.ArrowAH.com', None, 'http://dswcenter.com', None, '1592155660', None, 'https://tonyspokerschool.com', 'http://www.aetna.com', 'http://www.pizzafire.com', 'http://www.cakesonthemove.com', 'http://www.mandeljcc.org/', 'http://www.shakesandfranks.com', 'http://www.erindaleanimalclinic.com/', None, 'http://www.bloorwestvet.com', None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/501/?utm_source=yelp&utm_medium=listing&utm_campaign=website', None, 'http://www.bestmatt.com/', 'https://agents.allstate.com/wes-cain-gilbert-az1.html', None, 'http://www.scoutandcash.com', None, 'https://soccerpostarrowhead.com', 'http://www.saeedsbarandgrill.com', None, None, 'http://www.thesuntanboutique.com', None, None, 'http://www.childrensurgentcare.com', None, 'http://www.nikospizzalasvegas.com', None, None, 'https://enjoi-nails-salon.business.site', None, None, 'http://poolpartsplus.net/', None, None, 'https://locations.davidstea.com/on-toronto-1', 'http://clickserve.dartsearch.net/link/click?lid=43700006435127445&ds_s_kwgid=58700000444950505&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://myamericanfurnituregalleries.com', 'https://www.circlek.com/store-locator/us/peoria/9476-w-olive-ave/2706468', 'http://www.tikiicevegas.com', None, None, '1592155772', 'https://www.challengerschool.com/campus/nevada/las-vegas/silverado', None, 'http://www.fredmeyerjewelers.com', 'https://shadysidehousecleaning.com', None, 'http://serranosaz.com', 'http://www.expressflooring.com', None, 'http://www.clarkcountynv.gov/depts/clerk/services/pages/civilmarriages.aspx', 'http://griegofamilyeyecare.com', None, 'http://www.thunderbirdobgyn.com', 'http://www.bowkerchiropractic.com/', 'https://www.ventospizzapgh.com', 'http://www.minhchauoptical.com', None, 'http://www.michael-a-anderson.com', 'http://www.thewallpapermarketofmesa.com/', 'https://www.lvac.com/locations/henderson', 'http://subway.com/', 'http://www.sarphx.com', 'http://rokvegasnightclub.com', None, 'http://www.dovespa.ca', 'http://www.reprodux.com/', None, 'http://www.birch-dan.ca', 'http://www.nellosauto.com/', 'https://fantasticworldscomics.com', None, 'https://arizonadental.com/contact/phoenix-az-office/', None, 'http://www.strongsvillevacuum.net', 'http://www.ufixitautomotive.com', 'http://www.greenlinehair.com', 'https://www.lifestorage.com/storage-units/arizona/phoenix/85255/787-near-pinnacle-peak/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-AZ-787&utm_content=website-link&howheard=IYE', 'http://www.fremontweddingchapel.com', 'http://eh25.com/tasting-room/', 'http://www.paramountfinefoods.com', None, 'http://www.stevenjosephphotography.com/reviews', 'http://www.trappedcle.com', None, 'http://roxannesonline.com/', None, 'https://www.medusacle.com', None, None, 'http://www.fikesnet.com', 'http://cobraplumbing.net', 'http://www.tranteam.com', 'http://chinaaaadelivery.com', 'http://www.gangnamstreetlv.com', None, None, None, None, None, None, 'http://www.rapidrehablv.com/', '1592155928', 'https://reviveptandwellness.com', 'http://www.bronzebeautyst.com/', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'http://www.dunkindonuts.com', 'http://www.savers.com', 'http://www.kulinars.net', None, None, 'https://www.tesla.com/findus/location/store/calgary', 'http://www.vibebeautylounge.com', None, 'http://www.sunsetstation.com/entertainment', '1592155959', 'https://www.moneytreeinc.com/branch/north-nellis', 'http://refinedlandscaping.com/', 'http://www.enterprise.com', 'http://www.kidsrkidsmatthews.com', None, None, 'http://www.simsroofingandsiding.com', 'http://www.shoplostfound.com/', 'http://www.galaxylg.com', 'https://www.chopsticksngrill.com', None, None, 'http://www.cal-am.com/communities/tropicana-palms', 'http://www.johnsfamilyrestaurant.com', 'https://www.toronto.ca/data/parks/prd/facilities/complex/337/index.html', 'http://www.onehazelton.com/', 'http://www.handinhandpreschool.com/index.html', 'https://diminishedvaluecheck.com', None, 'http://www.BradyLuxury.com', None, 'http://www.bikramyogamesa.com', 'http://www.niceguystechnology.com', '1592156015', None, 'https://www.kohls.com/stores/oh/macedonia-235.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.castlemegastore.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=phoenix_yelp&utm_content=2017', None, 'http://www.kingwongchinese.com', 'http://l.macys.com/scottsdale-az?cm_mmc=Yelp_Stores-_-Scottsdale-_-n-_-450&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Scottsdale_450&m_ac=yelp_stores&m_ag=n', 'http://www.therisingsun.ca/', 'https://www.ahn.org/locations/healthwellness-pavilion/peters-township', None, 'https://www.impactarcherylv.com', None, None, None, 'http://www.greatclips.com/', None, None, 'https://www.rossoutdoors.com', 'http://www.allianceurgentcare.com', None, None, None, 'https://bounceu.com/matthews-nc', 'http://www.thestudiosalon.biz/', 'http://www.jackieo.ca', None, 'http://www.owlremodeling.com', 'http://wiglewhiskey.com', None, None, None, 'https://store.vioc.com/nc/denver/7575-highway-73-28037-GM0004', None, 'http://www.mymindseyerecords.com', None, None, None, None, 'https://www.themovingmasters.com', 'http://www.rodgunandbarbers.com', 'https://locations.davidstea.com/on-toronto-1', 'http://scottsdaleautoglass.com', None, 'http://ad.doubleclick.net/ddm/clk/283420830;110240480;l', None, 'http://prestigebeautysalo.wix.com/prestigesalon', 'https://www.tileshop.com/store-locator/north+charlotte.do', 'http://www.colkerjanitorial.com/', 'http://www.justforpleasurenc.com/just-for-pleasure-2-north-charlotte-nc/', None, 'https://www.accucaredental.com', 'http://www.eltororestaurante.com', 'https://www.dicksspeedotach.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://trunzosauto.com', 'https://www.originaljoes.ca', '1592156149', 'http://RalphieMayVegas.com', None, 'http://www.greengrotto.ca', None, 'http://carolinashomestone.com', 'https://www.montecarloinns.com/markham-suites', 'https://www.wellsfargo.com', 'https://www.marathonpetroleum.com', 'http://www.recordsreduction.com', 'https://www.rmcf.com/AZ/Tempe50423', '1592156173', None, 'http://www.newcenturyhairdesign.com', 'http://www.bloorwestwinery.com', 'http://www.costco.com/warehouse-locations/scottsdale-az-427.html', 'http://thearomatherapygarden.com', 'http://www.aaronlending.com', 'https://local.biglots.com/nv/las-vegas/4500?zcp=pd_local_yelp', '1592156193', None, 'http://nes-electrical-services.business.site', 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2faz%2fphoenix%2fmesa-west&sisearchengine=1524&siproduct=BEMES_Bus', 'http://othermamalv.com', 'http://www.n15salon.com', 'http://www.eastmesapediatrics.com/Home_Page.php', 'http://www.timhortons.com', 'http://www.shoelessjoes.ca', 'http://www.85254.com', 'http://www.funasiancuisine.com', None, 'http://mazzagarden.com/', 'http://www.littlecaesars.com/Locations/tabid/92/location/28079/Default.aspx?id=MTU1MzAwMTc%3d', 'http://www.accesstohealthcare.org', 'https://www.kidsquest.com/location/redrock/', 'http://www.siegelandcompany.com', 'http://www.miraclesofphoenixfitness.com', None, 'https://www.sheetz.com', 'http://www.elysianatstrose.com/', 'http://Www.federalgalley.org/restaurants/Shaka/', 'http://www.bowiechiropractic.com', 'https://www.topnotchwrapslv.com', 'http://simplicitysalonlv.com/', 'http://www.resumesolutions.ca', 'http://www.steamwayservices.com', None, None, 'https://collegehunkshaulingjunk.com?utm_source=yelp&utm_medium=organic', 'https://www.italianrestaurantrivergate.com', 'https://www.booksamillion.com', 'http://www.starskycanada.com/aboutus.php?s=4', 'http://www.platinumwellness.net', None, None, None, 'http://www.sheratoncavaliercalgary.com/calgary-restaurant', 'http://www.massagealacarter.com/', 'http://www.vintagewireandsupply.com', 'http://www.meyersmanagement.com', 'https://www.orthoarizona.org/physicians/Caleb-Behrend', 'https://heartlandfarmsanctuary.org', 'http://benefitcosmetics.com', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://locator.chase.com/ny/glendale/71-04-woodhaven-blvd', 'http://www.thespiritconnection.com', None, 'http://Adventurehelicoptertours.com', None, 'http://www.studiowish.com', 'http://www.luckybrand.com', 'http://www.glamorgananimalclinic.com', 'http://evermorenevermore.com', 'http://www.flooring2day.com', 'http://www.rosedrivingschool.com', None, 'https://www.marquiscompanies.com/facility/marquis-centennial-hills/', 'https://www.queenofheartscasinoparties.com', 'https://store.vioc.com/wi/stoughton/1108-us-51-138-53589-GR0034', 'http://solylunabeautystudio.com', None, 'http://chinatocleveland.com', 'http://www.coffeedaklak.com', 'https://www.ediblearrangements.com/fruit-arrangements?utm_campaign=StoreID561&utm_medium=Listing&utm_source=Yelp', 'http://www.christophier.com', 'http://www.paramountfinefoods.com', None, 'http://www.ballantynevillage.com', 'https://stores.advanceautoparts.com/nc/charlotte/2910-freedom-drive?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', 'https://www.walgreens.com/locator/walgreens-6649+morrison+blvd-charlotte-nc-28211/id=1376', 'http://andysmechanicalser.mechanicnet.com', None, 'http://www.mattressfirm.com/', None, None, 'https://www.sandmanhotels.ca/hotels/calgary-airport/', 'http://www.MontysFurniture.com', 'http://www.urbanamarket.org', 'https://roadside-picnic.business.site', 'https://www.woolfeyecare.net', 'http://www.neatedupey.com', 'http://www.azpluscleaning.com', 'http://www.JimmyJohns.com', None, 'http://www.doubletreeinspectionsofaz.com', None, None, 'http://www.habitburger.com', None, 'http://www.ShaolinArizona.com', 'http://www.providencecourtapts.net', None, 'http://howardstowingpittsburgh.com/', 'http://www.fantasticsams.com/', None, None, 'http://www.contessacatering.com', 'http://projectmanagementacademy.net/las-vegas/pmp-certification.php', 'http://www.scottsdalelibrary.org/', 'http://richtreenaturalmarket.com/', 'http://www.heightslibrary.org/', 'http://www.alibabas.ca/index.php?route=common%2Flocation_details&loc_id=1', 'http://www.lids.com', 'http://www.terracharlotte.com', 'http://l.macys.com/chandler-az?cm_mmc=Yelp_Stores-_-Chandler-_-n-_-453&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Chandler_453&m_ac=yelp_stores&m_ag=n', 'http://www.galleriaatsunset.com', None, 'http://www.primrosegilbertsantan.com?utm_campaign=localmaps&utm_medium=maplistings&utm_source=yelp', 'http://www.glendalebarber.com', 'http://www.costnerlaw.com', 'http://www.permanentbeautyusa.com', 'http://www.scottsdaleaz.gov/Parks/chap.asp', 'http://www.picknsave.com', None, 'http://freshandeasy.com', None, None, 'https://www.teslamotors.com/en_GB/findus/location/store/rosspark', 'http://bmwofchampaign.com', 'https://pvpd.com', 'http://www.poisonivysalon.com.', 'http://www.rmfenceohio.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://motherstamales.com', 'http://www.pamier-kabob.com/', None, None, None, 'http://www.mabelsbakery.ca/', 'https://www.hilton.com/en/hotels/clepwht-home2-suites-cleveland-independence/?SEO_id=YELP-HT-CLEPWHT', 'http://www.costco.com/warehouse-locations/henderson-nv-673.html', 'http://www.smiledesigncenterlv.com', 'http://www.mononachinastar.com/', None, 'https://www.trendssalonllc.com', 'http://www.heysugarstudio.com', 'https://www.choicehotels.com/arizona/scottsdale/rodeway-inn-hotels/az437/rates', 'http://www.thorespressobar.com', None, 'https://izzysbakeryandcafe.com', 'http://tonydblc.webs.com/', None, 'https://www.teamlogicit.com/PhoenixAZ302/', '1592156578', 'http://clickserve.dartsearch.net/link/click?lid=43700006430431858&ds_s_kwgid=58700000444952719&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', '1592156585', None, 'https://www.palmetto-eye.com', 'http://www.solaireonscottsdale.com/', 'http://www.westernreservehospital.org', 'https://www.sleepcountry.ca', 'http://theupsstore.ca/148', 'http://www.vip-pools.com', 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', None, 'http://7eleven.ca/store-locator/33259', 'https://www.popeyescanada.com/maps/toronto-liberty-village.html', 'http://www.theproducebox.com', 'http://www.elitepestcontrolnv.com', 'https://www.worldwidegolfshops.com/locations/van-s-golf-shops-mesa-801-s--power-road.html', None, 'http://www.wearejoviec.com', 'http://www.zajecpainting.com', 'http://www.spicy-garden.ca/', 'http://www.pittsburghoralsurgery.com', 'https://www.alterationsandcreationsaz.com', 'http://www.marriott.com/hotels/travel/yyzfs-fairfield-inn-and-suites-toronto-airport', 'http://www.wetseal.com', None, 'http://www.gioninos.com', None, None, 'http://www.gioiabeautybarspa.com', 'http://homedetectiveohio.com/', 'http://www.4seasonstreesllc.com', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.buylaptops.ca/', 'http://www.pittsburgh.va.gov', 'https://hoame.ca/contact-us/', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', None, 'http://eatzencore.ca', None, None, None, None, None, 'http://www.teamfreemanhomeloans.com', None, '1592156716', None, 'http://www.snapfitness.com/fountainhills', None, 'http://www.scooterscoffee.com', None, 'http://www.szechuanexpress.com/', 'http://baycoins.wordpress.com/', 'http://www.computerlasvegas.com', None, None, 'http://familylawguys.com/contact/scottsdale', 'https://www.sephora.com/happening/stores/phoenix-jcpenney-at-ahwatukee-foothills', '1592156756', None, 'http://www.swanpizza.com', 'http://cartridgeworldphx.com', 'http://www.amerizonaproducts.com', 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', 'http://www.bridlewoodmall.ca/', None, 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=clein&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', None, 'http://www.faithculture.ca', '1592156791', 'http://www.itsugar.com', 'https://local.fedex.com/az/phoenix/office-5048/', '1592156801', 'http://www.monkeybargym.com', None, None, None, None, 'http://www.quiktrip.com/', 'http://www.signaturewellness.org/', 'http://junipergrill.com', 'http://www.michaudscatering.com', 'http://www.bikeandforth.ca', 'https://www.artlewinbespoke.com', None, 'http://azteamania.com', 'http://www.miegakoreanbbqcleveland.com', None, 'http://goldenasiancuisine.com/', 'http://southparkvet.net/', 'http://www.tamcapturesphotography.com', 'http://www.glotique.com/', 'https://www.t-mobile.com/store-locator/az/chandler/chandler-fashion-center?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://asapcity.com', 'https://www.idealimage.com/medspa-locations/nevada/las-vegas-nv-area/lasvegashendersongreenvalley', 'http://Www.ktsfish-chicken.com', 'http://dreamdinners.com/main.php?page=store&id=53', 'http://www.rockstarhotdogs.com/', None, 'http://www.scottsdaleaz.gov/scottsdale-stadium', '1592156886', 'http://www.discountfirearmslv.com', 'http://www.homelife.ca/office_agents/2164', 'http://www.pillarwestgate.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', 'http://www.treasureisland.com/restaurants/little_richies.php', None, 'https://noracleveland.com', 'http://www.cabelas.ca/find-a-store/calgary/14', 'http://www.nevadagastro.com', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'https://www.extraspace.com/storage/facilities/us/arizona/chandler/1000000272/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', '1592156922', 'http://www.big4motors.com', 'http://telemindclinic.com', 'https://www.wildblue.studio', 'http://theistanbulgrille.com', 'http://www.dunkindonuts.com', 'http://www.fluorescentnails.com', None, 'http://www.leveloneaz.com', 'http://www.summitmedicalweightloss.com/', 'http://hopesbridalnc.com', None, None, 'http://www.meninkilts.com/calgary', 'http://www.harkins.com', None, None, None, None, None, None, 'http://www.marriott.com/hotels/travel/cltfm-fairfield-inn-and-suites-charlotte-matthews/', 'http://luvcuts.com', 'http://www.toronto.ca/parks/parks_gardens/allangdns.htm', 'http://www.abrazadental.com?utm_campaign=local&utm_medium=citations&utm_source=yext', None, None, 'https://www.take5oilchange.com/locations/oh/mayfieldheights-130/', '1592157006', 'https://www.ihop.com/en/restaurants-gilbert-az/920-s-gilbert-rd-1529', 'http://www.murillocarpet.com/', 'https://www.levinfurniture.com/furniture-store/pa/pittsburgh/5438-baum-blvd', None, 'http://www.stanfordcapitalloans.com', 'http://www.jpgexhaust.com', None, None, None, 'http://www.cpr4yourcar.com', 'http://lulyssalonllc.weebly.com/', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/328+Cochran+Road-Mount+Lebanon-PA-15228/storeid=5094?cid=ps_ylp_2020', 'https://www.macu.com/branch-locations/az/gilbert/gilbert-branch/0146', None, 'http://www.JimmyJohns.com', 'http://www.cpr-savers-training.com', 'https://www.paviliondentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.trane.com/residential/en/buying-a-trane/find-a-dealer.html/401830', None, None, 'http://www.carvel.com', 'http://baliandbeyond.ca', 'http://www.karaokerentalphoenix.com', 'http://www.eggsmart.ca', None, None, 'http://www.vogava.com', 'http://www.mrcookschinesecuisine.com', None, 'http://streetsofnewyork.com/', None, 'http://www.truphysicaltherapy.com', 'http://savagewalker.com', 'http://www.tntauctionssnelgrove.com/', 'http://goodworksautorepair.com', 'http://www.hikethislasvegas.com/', 'http://www.pacificforesightsecurity.com/free-security-yelp', None, 'http://www.oneluckypup.com', 'http://www.stovalldentistry.com', 'https://www.dominos.com', None, 'http://www.plushsalonaz.com', 'https://www.kaizenprogressivehealth.com', 'http://www.stoppleworthplumbing.com/', 'http://www.customfitdj.com', None, 'http://www.autodealzaz.com', None, 'http://www.smoothieking.com', 'http://floweramaofglendale.com/', 'http://benedictinewomen.org', None, 'http://www.purevanityspa.com', 'http://www.salonentro.com', 'http://www.shell.us/', 'https://canyonstateclassics.com/contact', 'http://Slicersonmain.com', 'http://www.1brotherspizza.com', None, None, 'https://www.aa.com', None, 'http://www.drlfish.com', 'http://dowdentalcare.com', 'http://www.dignitymemorial.com/funeral-homes/mesa-az/melcher-mortuary-mission-chapel/4527', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.rotorooter.com/scottsdaleaz', 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', None, None, 'https://kottasushi.ca', 'http://www.medvetforpets.com/location/medvet-akron/', 'http://www.internationalhotel.ca', 'http://www.lonemortgage.com', None, 'https://toothworks.com/toothworks-bay-adelaide-dental/', 'http://www.steves-auto-service.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.valleyhomepros.com', 'http://www.smallbarclt.com', None, 'http://www.toscanacondorentals.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281379878&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'https://www.goldbergcentre.com', 'http://www.roosterskountrykitchen.com', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89102/838069/', 'https://www.altierus.org/campus/henderson', 'http://www.coach.com', 'https://medmen.com/stores/scottsdale-north-scottsdale-airport', 'https://hihidonuts.business.site', 'http://www.bulwarkpestcontrol.com/', 'http://ranchoflooringvegas.com', '1592157278', 'http://www.JimmyJohns.com', 'https://www.phoenix.gov/publicworks/garbage/disposable', None, 'http://singsing.com', 'https://aestheticrevolutionlv.com', None, 'https://www.pimtmassage.com', 'http://www.cocosbakery.com/', 'https://www.capriottis.com', 'http://acewasherservice.com/', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'https://www.socialsecuritybranch.com/nv/henderson/henderson-social-security-office/10416-s-eastern-avenue/', 'http://www.collisionrepairphoenix.net', None, None, None, None, 'https://www.olivegarden.com/locations/az/scottsdale/scottsdale-e-cactus-rd/1234?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:scottsdale-az-1234_pl:locurl_rd:1175', 'http://www.jimscoins.com', None, 'http://serenitycleaningaz.com', 'http://www.reddoorliving1.com/delta-lakewood/', 'https://rdavis.amerifirstloan.com', None, None, None, None, 'http://www.tbdnursery.com', None, 'http://www.thescottresort.com/?utm_medium=enhanced-profile&utm_source=yelp', 'https://www.hafezpersianrestaurant.com', 'https://www.zaxbys.com/locations/ga/monroe/195-martin-luther-king-jr-blvd/', 'http://www.yetifirearms.com', None, 'http://www.bombaypalacetoronto.com', None, 'http://lebanesebakery.ca/', 'http://www.midabui.com', 'http://www.whoolisbarandgrill.com', 'https://www.yourcupcakequeen.com', 'http://www.banfield.com/veterinarians/pa/monroeville/moe?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'https://www.vannormanlaw.com', 'http://www.eaglehomeinspections.net', 'https://www.anytimefitness.com/gyms/372/Huntersville-NC-28078/?utm_source=yelp&utm_medium=local', 'http://oralsurgeryaz.com', None, 'http://www.cyclevegas.com', 'http://www.masteryoon.net', 'https://www.brighton.com/viewstore/123', None, None, '1592157435', 'https://www.riteaid.com/store-details?storeNumber=11467', 'http://www.soundplumbinglv.com', 'https://www.envision1361.com', 'http://www.gurusweets.ca', 'http://rebathandkitchens.com', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.nikolastreeservice.com', 'https://www.doctorallenwellness.com', None, 'https://www.dominos.com', 'https://www.ncfdentistry.com', 'http://www.vervehairdesign.com', None, '1592157478', 'http://www.lossombreros.com', '1592157484', None, None, 'https://local.fedex.com/az/tempe/MSCA/', None, 'http://www.ilovesushihenderson.com', 'http://www.okraaz.com/', 'http://www.starbucks.com/store/9298/', 'http://mascariauto.com', 'http://www.azdanceproject.com/', 'http://www.thetilestore.ca', 'http://www.germancarscottsdale.com', 'http://fracturedprune.com/location/shops-norterra', 'http://tasty-chinese.com', 'http://www.haydeedocasarmd.com', None, 'http://bubblesandsqueak.com', 'http://www.greatclips.com/salons/6225', 'https://www.unclelouie.com', None, 'http://www.firemeupaz.com', '1592157549', 'https://www.hilton.com/en/hotels/lasdmhw-homewood-suites-las-vegas-city-center/?SEO_id=YELP-HW-LASDMHW', None, None, '1592157562', 'https://yonge-and-eglinton.orangetheoryfitness.com?utm_campaign=yelp&utm_term=072017&utm_medium=print&utm_source=redirect&utm_content=yelp_print', 'http://www.beerexpressisbest.com', None, None, 'http://www.omahasteaks.com/stores', None, 'http://www.commonwealthbar.ca/', 'http://www.silverminesubs.com/', 'http://www.presotea.ca', 'http://www.myharu.ca', 'http://www.piedmontgoldexchange.com', '1592157598', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://crclasvegas.com', 'http://www.wildwingrestaurants.com/franchisees/newmarket-north/', 'http://www.apples4you.com/', None, None, 'http://www.marinersmadison.com/', 'http://www.century21.com', 'https://www.mcdonalds.com/us/en-us/full-menu/123dollarmenu.html?cid=PS:GCM_MOP:NB::Yelp:All', 'https://www.choicehotels.com/CN387?mc=blypypaf', 'http://www.vegasinkandtoner.com/', 'http://www.johnvarvatos.com/storedetails?StoreID=5656', 'http://www.westsidecommunitymarket.org', 'http://richnerair.com', None, 'https://www.kwiktrip.com', '1592157649', 'http://www.starbucks.com', 'http://www.timhortons.com', None, 'https://www.thejoint.com/north-carolina/huntersville/birkdale-12006', 'http://www.lukehousemadison.org', 'http://www.greatclips.com/', 'http://local.firestonecompleteautocare.com/arizona/phoenix/3638-e-thomas-rd/?lw_cmp=IYP_YPC_MLP_13315', 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, 'http://www.smokeandjoelounge.com', None, 'http://botecolv.com', 'http://www.keelinglawoffices.com', 'https://puredentalaz.com/home.htm', 'https://www.thewestlake.ca', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.greyhound.com/social?utm_source=Book%20now&utm_medium=Yelp&utm_campaign=permanent%20link%20', 'http://aaronmachineshop.com', 'http://www.vegasprohvac.com', None, None, None, 'http://www.torontoimageworks.com', None, None, 'https://desertrad.com/locations/southwest-wynn-road', None, 'http://rocksolid-surfaces.com', None, None, 'https://www.lappeneyecare.com', 'https://www.bentosushi.com', 'http://www.drrusselo.com', None, 'http://www.dunkindonuts.com', 'http://nouveauxclt.com', None, None, 'http://Pizza-wings.com', 'http://tracysautobody.com', None, None, 'https://www.marquiscompanies.com/facility/marquis-plaza-regency/', 'http://www.nascarspeedpark.com/nc.aspx', None, None, 'https://www.gowireless.com/store/stores/Arizona/Mesa/Mesa---MonteVistaVillageCenter----602609', 'http://www.kwikfill.com', 'http://www.usps.com', 'http://www.dolcegelato.net', 'https://www.hilton.com/en/hotels/pitokhx-hampton-pittsburgh-university-medical-center/?SEO_id=YELP-HP-PITOKHX', 'http://www.exclusivetailoring.com', 'https://www.martinandmacarthur.com/', None, 'http://www.marleypark.com/', 'http://www.corepestsolutions.com', 'https://www.marcustheatres.com/', 'https://www.dippindots.com/loc/ll/US/NC/Charlotte/6801-Northlake-Mall-Dr_', 'http://www.healthydentalalternatives.com', 'http://www.northmountainvisitorcenter.org', '1592157819', None, 'https://www.fjtesq.com/?npcmp=dir:local:4407007:89145', None, '1592157829', 'http://www.mwuclinics.com', 'http://www.windowprollc.com', 'http://www.azairductcleaning.com', None, 'http://www.aamcolasvegas.com', 'http://www.casinonight2you.com', None, None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.dairycream.org/', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_426', 'http://www.ntb.com/store/8522/bedford-oh-44146-2252?utm_source=yelp&utm_medium=maps&utm_campaign=yelp-maps', None, 'http://www.lovebailbonds.vegas', 'http://downtownfood.ca/', '1592157877', '1592157879', None, 'http://www.apollos.com', None, None, 'http://www.dairyqueen.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://bennysva.com', 'http://urbantealoft.com/', None, None, 'http://www.25celciusbeautybar.com', 'http://www.simplicitylaser.com/locations/nevada/henderson/', 'http://www.hopkinsautospa.com', 'http://www.junk-rite.com', 'https://locations.dennys.com/NV/LAS-VEGAS/247828', None, 'http://www.arborealty.com', None, 'http://www.9dragonskungfu.com/', None, 'https://www.storquest.com/self-storage/az/tempe/9052', None, 'http://www.nailswithlovelv.com', 'http://www.daveyplumbingpittsburgh.com', 'https://birdysoulfood.com', 'http://motorsandmoreinc.com', None, 'http://www.steaknshake.com/locations/24097-steak-n-shake-us-hwy-17-s-brunswick', 'http://surprisefamilymedicine.com', None, 'http://www.czcustom.com', 'http://Dogfather.me', 'http://clickserve.dartsearch.net/link/click?lid=43700006430792752&ds_s_kwgid=58700000444978432&ds_url_v=2', 'http://www.kelseys.ca/location_region.php?province=on&s=1&city_drop=North+York', None, 'http://jianphoto.com/', 'http://www.worldofdecorauctions.com', 'http://www.goodwillsp.org/shop/retail-stores/pineville/', None, None, None, 'http://www.childrensorchard.com/stores/las-vegas-nv/', None, 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', 'https://www.pescadoborracho.com', 'http://www.spiceshack.ca', None, None, None, '1592158252', None, None, None, 'http://www.extendasuites.com', None, None, 'http://www.docautorepairs.com', 'https://www.signaturestyle.com/locations/oh/medina/famous-hair-medwick-marketplace-haircuts-67153.html', None, '1592158276', None, 'https://www.luxor.com/en/nightlife.html', '1592158284', 'http://www.southstburger.com/location/35-first-commerce-dr/', None, 'http://shopmelrosevintage.com/', 'http://www.mrsub.ca', 'https://imwireless.net/locations/concord-ma/', 'https://lasvegaspainreliefcenter.com', None, 'https://www.extraspace.com/storage/facilities/us/arizona/mesa/501578/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.fastfreshfoods.ca/', None, 'http://www.zazaespressobar.com', 'http://www.snapfitness.com/gyms/lasvegas-nv-89147/7089', 'http://www.forever21.com', 'http://www.ssmokincafe.com', None, None, None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Carnegie-PA-15106/067741/', 'http://www.saloncafe.net', 'https://www.blumenailsspasolon.com', 'http://www.gehendricks.com/', 'https://www.norfolkhall.com', 'http://missionptaz.com', 'http://www.visithenderson.com/todo/special-events/stpatricksdayparade', 'http://www.skinapeelbeautybar.com', 'http://www.ccdcranch.com', 'http://www.craftdogs.com/', None, 'http://www.tomatoesiipizzeria.com/', 'http://yesidobridals.com/', 'https://www.lumberliquidators.com/ll/stores/az/mesa/1845-south-power-rd?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1388', None, None, 'http://www.walmart.ca', 'http://www.papajohns.com', 'http://www.partycity.com/storedetail.do?id=842', 'http://locations.tccrocks.com/az/scottsdale/8980-e-indian-bend-rd.html', None, None, None, '1592158380', 'http://www.raxlenpharmacy.com', 'http://www.anjapparcanada.ca', None, 'http://www.dennys.com', 'https://www.ispynserve4u.com', 'http://mainstreetautoservices.com', 'http://www.dennyscarwash.com', None, 'http://www.bk.com', 'http://www.eatgenos.com', None, 'https://www.seeeyewear.com/locations/pittsburgh', None, 'https://www.piesbyinge.com', 'https://locations.nekterjuicebar.com/ll/US/AZ/Tempe/2000-E_-Rio-Salado-Pkwy_*-Ste_-1213?utm_source=yelp&utm_medium=link&utm_campaign=Tempe&utm_content=URL', 'http://kohinoortempe.com', 'http://idolizespa.com', 'http://www.naturaltress-salon.com', 'https://www.sunrisepreschools.com/locations/baseline-gilbert/?utm_source=yelp&utm_campaign=online_listing_gilbert_280&utm_medium=referral&utm_term=business_url', None, None, 'http://www.peridotsweets.com/', None, 'http://www.thefitmenswear.com', 'http://thegrillat11union.com', 'http://www.papajohns.com/', None, 'https://www.victoriassecret.com', 'http://www.lostdutchmanroasters.com', 'http://www.petplanethealth.com/stores/village-plaza/', None, None, 'http://www.upinvapor702.com', 'https://www.ronmarhoferchevy.com', None, None, 'http://www.sanealcamera.com', 'https://www.bankofamerica.com', 'http://www.thesinginglady.com', 'http://clickserve.dartsearch.net/link/click?lid=43700006435127049&ds_s_kwgid=58700000444950373&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.arizonaeyecenter.com', None, None, 'http://www.thespicetrader.ca/', None, None, 'http://www.harkinstheatres.com/', 'https://chefmayra.com', None, 'http://www.westmore.ca', None, 'http://www.LaRoccaKitchen.com', 'http://www.craquedecreme.com', 'https://heartandlungnv.com/physicians/profile/Arnold-Chung-MD', None, 'http://www.loveandkissespetsitting.net', None, None, 'http://www.tsgservicesaz.com', 'https://www.staples.com', None, None, None, 'http://carlosthecomputerguy.com/', 'http://info.tradesecrets.ca/locations-reviews/upper-canada-mall/', None, 'https://www.shapirolawaz.com', 'http://www.azpatiocover.us', 'http://bevvyuptown.com', 'http://Www.solasalonstudios.com/salon-professional/liz-mervar1', None, 'http://www.yangschicken.ca', None, 'http://www.dsw.com', None, None, 'http://birdhauscoffee.com', None, 'http://www.jysk.ca', None, 'http://www.puffnstuffstores.com', 'https://www.brakesplus.com', 'https://locations.noodles.com/pa/pittsburgh/476-mcmaster-way.html', 'http://www.michaeltoddsfurniture.com', None, 'http://www.dentistryintoronto.com/', 'https://jmichaelrealestate.com', None, 'https://splittingtimber.com/axe-throwing-phoenix/', 'http://www.crabhouselv.com', None, 'http://wisconsincutlery.com/', None, 'http://www.jerseygiant.ca', 'http://www.mazdaofrichmondhill.com', 'http://www.quiktrip.com', 'http://www.dimsumchineserestaurant.com', 'http://www.marcustheatres.com', None, 'http://kaiburcoffee.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.shishaliciouscafe.ca', None, 'http://charandamexicaneatery.com', 'http://www.rideprlimo.com/scottsdale-limousine/', 'http://www.distillata.com', 'http://www.sobeys.com/en/stores/sobeys-aurora-bayview/', 'http://www.acanac.com', 'https://www.ubreakifix.com/locations/goodyear', 'https://www.robainalaw.com', 'http://www.newdimensionsauto.com/', 'http://www.martinmarineaz.com/', None, None, 'http://www.rexall.ca/', 'http://www.baierlhonda.com', 'http://www.pizzalafelice.com', None, 'https://www.olivegarden.com/locations/wi/madison/madison-east-towne-mall/1300?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:madison-wi-1300_pl:locurl_rd:1227', 'https://www.chilis.com', 'http://www.npfy.org', 'http://www.bedbathandbeyond.com', 'http://www.andersons.pub', 'https://www.lowes.com/store/AZ-Scottsdale/1850', 'http://www.playfactoryparty.com', None, 'http://cityh2o.com', 'http://www.zooecomuseum.ca/', 'http://www.shamsflorist.ca/?srccode=yelp_track', 'https://massagetherapistscottsdale.com', 'http://www.wilshirepines.com', 'http://www.modernfurniturestudio.com', 'http://allontarioapplianceclinic.ca', 'http://www.bridalalterationspittsburgh.com/', 'https://perfumecollection.ca', None, 'https://www.homedepot.com/l/Ray-Road/AZ/Chandler/85226/476?cm_mmc=seo-yelp&', 'http://www.bornandraisedlv.com', None, 'https://www.statefarm.com/agent/us/nv/las-vegas/ladonna-koeller-p89cl1ys000?cmpid=jxep_blm_0006', 'https://www.jewelrymedinaohio.com', 'https://eastendpittsburgh.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'https://www.choicehotels.com/wisconsin/madison/cambria-hotels/wi143/rates', 'http://www.cafe-mirage.com', None, 'http://adagestudios.com', 'https://www.masteroftintllctempe.com', None, 'http://lsmchiro.com/clinics-office-hours/sunprairie.html', 'http://plumbing-repair-now.com', 'http://spookysswirls.com', 'http://www.sunserimove.com', None, 'http://www.nandos.ca/restaurants/bay-street', 'http://www.massageheights.com/south-pointe-plaza', 'http://www.a-zsources.ca', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', 'http://www.smilereef.com', 'http://simplycardio.com', None, 'http://kismetandclover.com/', 'http://www.assouline.com', 'http://www.pkcreek.ca/', 'http://www.internationalgalleriesurbana.com', 'http://www.regularsbar.ca', 'http://www.6DegreesAZ.com', 'http://www.oldnavy.com', None, 'http://www.dmcsystemsllc.com', 'http://www.jumpstartchildcare.com', None, 'http://www.villapizza.com', 'https://www.yourlistingexpert.com', 'http://stores.dsw.com/usa/nv/henderson/dsw-designer-shoe-warehouse-stephanie-street.html', 'https://www.usps.com/', None, 'https://eastvalleyrv.com', None, '1592158816', None, None, None, 'https://www.aircanada.com/ca/en/aco/home/fly/premium-services/maple-leaf-lounges/maple-leaf-lounge-details.html#/!lounge@toronto', 'http://chickswithspiritualgifts.com', None, 'http://www.costco.com/', '1592158831', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.tyddvegas.com', 'http://www.samfawazphotography.com', None, None, 'https://www.schendelpest.com', 'http://www.honeybeeac.com', None, 'https://www.gavishpropertymanagement.com', 'http://www.topnailsaz.com', 'http://www.katesissons.com', 'http://www.JimmyJohns.com', 'https://plexustechnology.com', 'http://www.thedirtydrummer.com', 'http://pac.unlv.edu/index.php', 'http://www.enterprisecarsales.com/location/5482/Enterprise_Car_Sales_Las_Vegas?mcid=iyp:277&CUR=yelp', 'http://www.happitimedaycare.com', None, '1592158875', 'http://www.wyldewoodillinois.com', None, None, 'http://www.tribridpersonaltraining.com', 'http://www.kw.com', '1592158885', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.azheart.com/', None, 'http://www.bellabridesmaids.com/', 'http://www.dollargeneral.com/', None, 'http://www.plazahotelcasino.com/entertainment/the_best_little_wh', 'http://www.yofreshyogurtcafe.com', 'http://mdskinlounge.com', None, '1592158911', 'https://www.itdrsolutions.com', 'https://www.papajohns.com', 'http://hiltongardeninn3.hilton.com/en/hotels/ontario/hilton-garden-inn-toronto-oakville-YTOTVGI/index.html?SEO_id=YELP-GI-YTOTVGI', None, 'https://www.emser.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', None, 'http://www.pitapitusa.com/', None, None, None, 'http://www.tlcdentalcarelv.com', None, 'http://www.chiefsautowash.com', 'https://www.gamesourceinc.net', None, None, None, 'http://www.bigotires.com/Locations/89032', 'http://www.mbenergetics.com', None, '1592158958', '1592158961', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.goldenchinarancho.com', 'http://www.starbucks.com/store/14175/', 'https://www.cellphonerepair.com/monroeville-pa/', 'http://vbarbershop.com/locations/kierland-scottsdale/', 'https://agents.allstate.com/padma-schaudt-las-vegas-nv.html', 'https://www.petsmart.ca/stores/ca/on/oakville-store2002.html?utm_campaign=us-lcl-srv-eg&utm_medium=listing&utm_source=yelp', '1592158979', 'http://www.designedbydyani.com', 'http://spadinaoptometry.ca/', 'https://www.wellschiro.com', 'https://www.justonemoretavern.com', 'https://www.sprouts.com/stores/details/-/store-details/view/store/7/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.framestorepittsburgh.com', None, 'http://totemappliance.com/', 'http://www.silvertoncasino.com', 'http://playnwisconsin.com', 'http://www.tokasticksgolf.com/index.php', 'https://local.biglots.com/nv/las-vegas/4500?zcp=pd_local_yelp', 'https://www.dwellarizona.com', 'http://achermanlaw.com/', 'http://www.odellconstructioninc.com', None, 'http://www.thebownesiangrocer.ca', 'http://outlandcigars.com', 'http://stlouiswings.com', 'http://www.brokeassphone.com/broke-ass-phone-locations/broke-ass-phone-rocky-river-ohio', 'https://www.valleywidecooling.com', None, 'http://www.amrracing.com', 'http://www.cmlibrary.org/locations/branches.asp?id=13', None, 'http://nakedpizza.biz', 'http://www.meltinyourmouthcupcakes.com', 'https://local.fedex.com/az/phoenix/office-5048/', 'https://locations.usbank.com/index/arizona/gilbert/east-baseline-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.dominos.com', 'http://www.treeworkersofphoenix.com', 'http://d-moes.com/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://dansdogsdiner.com', 'http://www.saucepizzaandwine.com/locations/scottsdale-scottsdale-waterfront/', None, 'http://Www.tomjames.com', 'http://www.healinginspirations-wi.com/', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', '1592159067', 'https://stores.advanceautoparts.com/pa/etna/500-butler-street?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'https://local.fedex.com/nc/charlotte/office-0746/', 'https://northvalleysurgerycenter.com', 'http://www.burritozone.ca', 'http://www.enterprise.com/car_rental/deeplinkmap.do?cnty=US&gpbr=53W9&bid=004&arc=YAPL100&cm_mmc=Yelp-_-Local-_-National-_-null', 'http://pddlv.com', 'https://hairbykee.wixsite.com/hair-by-kee', None, None, '1592159093', 'http://www.trianglebarswissvale.com', None, None, None, 'http://freshco.com/stores/bloor-dundas', 'http://www.bojangles.com', 'http://www.mark-taylor.com/arizona/san-sonoma/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.midhafurniture.com', None, None, 'https://www.lashandskinlab.com', None, None, None, None, 'http://www.serviceking.com/locations/service-king-bell-rd?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', None, None, 'http://alatisinspectionservice.com/lp/home-inspection', 'https://www.moes.com/find-a-moes/pennsylvania/bridgeville/938?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', None, 'http://www.wagg-n-walk.com', 'http://www.clevelandcabby.com', 'https://www.chaoshomeloans.com', 'http://www.marondahomes.com', 'http://www.morocochocolat.com/', 'http://anytimefitness.com/North-Royalton', 'http://qualitysystemsac.com', 'https://local.safeway.com/safeway/az/sun-city-west/13503-camino-del-sol.html', 'http://www.buckinghamsports.ca/view/buckinghamsports/the-penalty-box', 'http://geckohomeservices.com', 'http://www.republicwesthome.com', 'http://www.pawlorpets.com/', 'https://www.sevabeauty.com/location/az/avondale-w-rancho-santa-fe-blvd/', 'http://www.tjmaxx.com', None, None, 'http://www.barleyhousecleveland.com/', 'http://www.landmarkcinemas.com/bolton/contact-us', '1592159181', 'http://www.irenevoo.com', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', None, None, None, 'http://www.cumbraes.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, 'http://www.curtisnewsomedds.com', 'http://walmart.com/', 'https://www.scottsdalemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://www.casinomontelago.com', None, None, None, 'http://www.jaguarnorthscottsdale.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Mesa-AZ-85210/723081/', None, 'http://www.cityofmadison.com/parks/parks/park.cfm?id=1368', None, 'https://www.viomedspa.com', 'http://www.paradisetileandnaturalstone.net', None, 'http://www.kinglawofficespc.com', 'http://www.innovateframe.com', None, 'http://www.happyhourarizona.com/champions-sports-saloon', 'http://www.millcrestpark.com', 'http://www.atlanticbay.com', 'http://www.fiddlersdream.org', 'http://www.nationaleventstaffing.com', 'http://holisticfit.life', 'http://www.atwistedtulip.net', 'https://www.carmax.com/stores/7284?adcode=YLP7284&utm_campaign=LG_AFF&utm_source=LG_AFF_YLP&utm_medium=local&utm_content=shop', None, None, None, 'https://stewardship.pro', 'http://www.golfsmith.com', 'http://www.duskscottsdale.com', None, 'http://www.metrosouthbridge.com', 'http://www.pizzahut.com/', 'http://www.80OnGibsonapts.com?campaign_code=yelp_80_On_Gibson_NV_89012', 'http://reliableappliance.ca', 'https://www.legendsofchampaign.com', 'http://www.timhortons.com/ca/en/locations/get-directions.php?id=101357', None, None, 'http://www.spotlesssquadlv.com', None, 'https://asquaredmathtutoring.com', 'http://www.autoxpertslv.net', None, 'http://www.shanklelaw.com', None, 'http://www.cajuncrawfishLV.com', 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=1165', 'https://ohiosbestmassage.com', 'https://handandstone.com/locations/az-glendale', '1592159307', '1592159309', 'http://inesdisanto.com/', 'https://www.mandalaybay.com/en/nightlife/eyecandy-sound-lounge.html', None, 'http://affordablemoverslv.wixsite.com/website', None, 'https://pizzapit.biz', 'http://www.piedmontexpresscaresuttonroad.com/', 'http://www.distinctiveweddingvideos.com', None, 'https://affderm.com/locations/tempe-office/', 'http://vegasproperty.management', 'http://www.gandmstaffingsolutions.org/', 'http://www.bellagio.com/attractions/gallery-of-fine-art.aspx', 'http://www.structube.com', None, 'http://www.autocash.com/auto-cash-in-las-vegas?utm_campaign=places&sf_traffic=paid_listing_yelp&utm_medium=listing&tmx_id=d2c199521a9a9f&utm_source=yelp', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.theavenuebanquethall.com', None, None, None, 'http://trutea.ca', 'http://www.rbomotorsports.com', None, '1592159362', 'http://thebagelshoppe.weebly.com/', 'http://www.freewayinsurance.com', 'http://www.persianbijar.com', 'https://www.phoenixmusiclessons.com', 'http://www.discoverydetectivegroup.com', 'http://www.fortinos.ca/en_CA/storedetail.484.html', 'http://www.motel6.com/reservations/motel_detail.aspx?num=4742', 'http://www.its2di4.com', None, 'http://pawminderspluspetsitters.com', 'http://www.fiatusaofstrongsville.com', 'http://www.skinsopretty.com', 'https://www.ontario.ca/page/serviceontario', 'http://www.northphoenixcampground.com', 'http://www.packbrotherscollisioncenter.com/', 'http://www.gusnewyorkpizza.com/', None, 'http://www.canadianpardons.ca', 'http://www.computerezearizona.com', None, 'http://www.vashticosmetic.com', None, 'http://fitforlifefood.com/', 'https://www.meijer.com/shop/en/store/307/?cmpid=OLA:040319:YelpGrocery', None, 'http://www.suepembertoneducation.com', 'http://www.disktopia.com', 'http://www.wjbonline.com', 'http://www.atriasrestaurantpncpittsburgh.com', 'http://www.discotech.me', 'http://www.myturnlv.com', None, 'http://www.catchacritter.us', None, None, 'http://www.ourfamilyeyecare.com', 'http://www.goodwillaz.org/', 'http://www.BroadstoneParadiseValley.com/?utm_medium=listing&utm_source=yelp', 'http://uptownnailsalon.wix.com/uptown-nail-salon', 'https://www.sticyyc.com/locations-1/', 'http://www.harveyswineburger.com', 'http://esoslifestyle.com', 'http://www.smithenvironmentalsolutions.com', 'https://beyondextensions.mayvenn.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.cardinalewaymazdapeoria.com/?ibp-adgroup=yx_target', None, '1592159462', 'https://local.albertsons.com/az/peoria/8240-w-deer-valley-rd.html', None, 'http://www.plastikwrap.com', 'https://www.petsuppliesplus.com/Store/OH/Lakewood/Lakewood/128', 'http://www.thunderbirddental.com', 'http://poolservlv.com', 'https://www.heinens.com/Heinens-of-Hudson', 'http://www.jackscornertaplkn.com/', None, None, 'https://www.wyndhamhotels.com/hotel/03725?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:03725', 'https://innovativecleaningsvs.com', 'https://www.starbucks.ca/store-locator/store/1017580/bayview-village-2901-bayview-avenue-toronto-on-m-2-k-1-e-9-ca', 'http://www.statusinkllc.com', 'http://bellagiopizza.ca', 'http://www.puptowncharlotte.com', None, 'http://www.starbucks.com/store/10311/', 'http://avis.com', None, 'https://locations.postnet.com/nv/las-vegas/10120-w-flamingo-rd', 'http://aftbar.com/', 'https://local.skechers.com/az/phoenix/', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.autozone.com/locations/az/peoria/8669-nw-grand-ave.html', 'https://smartmoveinsurance.com', None, None, 'http://www.the-idea-store.org', 'http://www.dunkindonuts.com', 'https://www.i9sports.com/ProgramDirector/309/', 'http://www.rainrestaurant.ca', 'http://www.davidandgoliathtees.com/', 'http://www.starrdickensortho.com/', 'http://mickeles.ca/', None, 'http://www.westsidelaser.ca', 'http://www.lcbo.com/lcbo-ear/jsp/storeinfo.jsp?STORE=629&language=EN', 'https://gilbert.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', None, 'http://www.pitapit.com', 'http://www.cottonwoodstationeatery.com', 'http://superdiscounttrans.com/', '1592159557', 'http://www.bowlingwithleverage.com', 'http://www.saloncentric.com/', 'https://www.carolinashealthcare.org/locations/detail/carolinas-healthcare-urgent-care-union-west', 'http://www.autosonic.ca', None, 'https://www.heirloomtoronto.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66065.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66065', 'http://philipsplastics.com/', 'http://www.bookmans.com', 'http://www.wordofmouthdentistry.ca', 'http://www.starbucks.com', None, 'http://www.yofreshyogurtcafe.com', 'http://www.breadheadslv.com', 'http://www.luxor.com/attractions/attractions_king_tut_museum.aspx', 'http://www.crazymocha.com', 'http://www.vuaurology.com', 'http://www.costco.com/', 'http://www.mydcdental.com', 'http://www.ritzcarlton.com/en/hotels/charlotte/dining/punch-room', 'http://phoenicianestates.com/', 'http://www.bountifulfruitarrangements.com', '1592159602', None, None, None, None, None, 'http://www.azpowerwashpros.com', 'http://www.fishcreekvets.com/', 'https://haircutmensoutheuclidoh.com', 'http://Www.mixnmatchausa.com', None, 'http://consolidatedauto.com', None, 'http://www.cookinsgroup.com', 'https://stores.bestbuy.com/il/champaign/2117-n-prospect-ave-46.html/?loc=ns100&ref=NS', 'http://www.nextcare.com/locations/az/peoria-glendale/?utm_source=Yelp&utm_medium=Local&utm_campaign=peoria_glendale', 'http://darkhorsesportsbar.com', None, None, 'http://www.catseyevintage.ca', 'http://www.searsgaragedoors.com', '1592159643', 'http://www.westcoastkids.ca', 'http://heavenlyhairbodysalon.com', None, None, None, '1592159653', 'http://www.cyclonespitas.com', None, None, None, 'http://www.valledelsol.com', None, 'http://www.villageinn.com/', None, None, None, 'http://www.tascsolutions.org/locations/phoenix-central', 'http://hooverdamstore.com', None, 'http://www.elitepodiatry.net', None, '1592159685', 'http://www.cleverchefscatering.com/', 'http://www.successjustclicks.com', None, 'http://www.theterracegardens.com', 'http://www.nortownfoods.com', '1592159695', None, 'http://www.chinadragonbedford.com', 'http://www.choiceautorepairaz.net', 'http://gpsandtrack.com', 'http://www.pancheros.com/locations/tempe', 'http://www.oldvegastavern.com', 'https://store.vioc.com/wi/madison/3594-e-washington-ave-53704-GR0030', '1592159710', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.youfit.com/gym/scottsdale-scottsdale-rd-7346?utm_source=local&utm_medium=yelp&utm_campaign=club7346&utm_term=website', 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.budget.com/', 'https://www.glamparlor.com/?utm_source=Yelp&utm_campaign=Main_Web', None, 'http://www.balanicustom.com/custom-suits-cleveland/', None, 'http://www.cox.com', 'http://www.westendcomics.ca', 'http://www.greenhousemontessorischool.com', 'http://carpetwarehousecleveland.com', None, None, 'http://www.ontrac.com', None, 'http://www.jacobsonbrosdeli.com', 'http://www.altrahomedecor.com', None, None, None, None, 'https://p3hp.org/nevada/p3-medical-group/our-locations/business-park/', 'http://lvmpd.com/', None, None, 'http://www.fastforwardskate.com', 'http://www.midoco.ca', None, None, 'https://www.starbucks.ca', None, None, 'http://www.dunkindonuts.com', 'http://www.findyourcenter.com/', 'http://www.neocoffeebar.com', 'http://www.locksmithbetty.com', None, None, '1592159796', 'https://www.walgreens.com/locator/walgreens-4667+william+penn+hwy-murrysville-pa-15668/id=11768', 'http://www.thespa-sunprairie.com', 'http://www.rushhourlegal.com', 'https://locations.jackinthebox.com/us/az/gilbert/4361-e-baseline-rd', None, 'http://www.celsiussushibar.com', '1592159814', None, None, None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.gamestop.com/store/us/az/glendale/1867/arrowhead-town-center-gamestop', None, None, 'http://www.SonicAutoHouse.com', None, 'http://www.bayviewmooreauto.com/', 'http://www.jessicabashaw.com', 'http://www.thewoodenvine.com', None, 'https://merkatoethiopianrestaurantlasvegas.com', 'http://www.whitevacuum.com', None, None, None, None, 'http://www.beattheheatpoolservice.com', 'http://www.carmellaspizzagrill.net', 'http://www.tphc.ca', 'http://www.tomscountryplace.com', 'http://www.balaton-restaurant.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.autoexinc.com/', 'https://creativenailsandspascottsdale.com', None, 'http://www.canadiantire.ca/en/store-locator/ab/calgary-south-ab.store.html', 'http://rusticsocial.ca', 'http://wolfwatersports.com', 'https://www.extremearizona.com', 'http://www.theBooKnack.com', None, 'https://elementsmassage.com/fitchburg', None, 'https://www.copart.com/locationDetail/47', 'http://tigotrattoria.ca', None, 'http://www.nevadadesertmermaids.com/', 'http://www.caesarsuniforms.com', 'http://www.wingsntings.ca/', 'http://www.boulderzclimbing.com', 'http://www.SonoranVein.com', 'http://www.goodlifefitness.com', 'http://nievemalandra.com', None, 'http://www.scottsdalespice.org', None, 'http://l.macys.com/madison-wi?cm_mmc=Yelp_Stores-_-Madison-_-n-_-228&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Madison_228&m_ac=yelp_stores&m_ag=n', 'http://www.superiortire.ca/locations/mode/7/208-king-rd-oak-ridges-on-l4e2w1.aspx', 'http://thearizonafishingguides.com/index.php', 'http://www.chapmancollision.com/', 'http://www.quiktrip.com/', None, None, None, 'http://www.littlecaesars.com', 'http://www.onereddoorphotography.com', 'http://www.circuscircus.com/casino/slots_a_fun.aspx', None, 'http://www.milgard.com', 'http://benetflorentine.com', 'http://www.sbistroaz.com', None, 'http://locations.avis.com/us/az/chandler/c5a.html', 'http://www.rockcitydance.com', None, 'http://www.washwerksexpress.com', 'http://AutoFinanceNC.com', 'http://www.mrhandyman.ca/calgary-south', None, 'https://arizona.weidner.com/apartments/az/mesa/pala-mesa-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', None, 'http://www.lesterklebedpm.com', 'http://www.bk.com', 'https://www.bathandbodyworks.com', None, 'http://www.brakemasters.com', 'http://clearfloat.ca', None, 'http://www.watchdogpestcontrol.com', None, 'http://www.budgetbridal.vpweb.com', 'http://advancedheartcare.com/', 'https://moxies.com/?utm_source=yelp&utm_campaign=yelp_enhanced_profile&utm_content=landing-page', 'http://www.ihlcanada.com', 'http://www.parmalaserwash.com', None, 'http://www.anatoliaturkishfood.ca', None, None, 'http://calgarylibrary.ca/locations/shaw', 'https://www.payless.com/stores-location/NV-LAS-VEGAS-5798.html', 'https://peoria.uptownjungle.com', None, 'http://www.rimrepairrx.com', None, 'http://www.apria.com', None, 'http://www.santinocatering.com', 'http://www.fusionsincsalonanddayspa.com', 'http://www.qualityonepool.com', None, 'http://zenarizona.com/', None, 'http://empireplumbingac.com/', 'http://www.contourmedical.com', 'https://website-18496976604484669992-seafoodrestaurant.business.site', 'http://www.swimmingpoolrepairphoenix.com', 'https://locations.visionworks.com/ll/US/OH/Beachwood/26300-Cedar-Rd', 'http://www.barrebodystudio.com', 'http://www.wwsdental.com', 'http://robertostacoshop.com', None, None, 'http://pacificeastcoventry.com/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4391+East+Washington+Avenue-Las+Vegas-NV-89110/storeid=3172?cid=ps_ylp_2020', 'http://www.labonitadulceria.com', 'http://www.retinalconsultantsaz.com', 'http://www.irideworldwide.com', 'https://titleboxingclub.com', 'http://www.serviceking.com/locations/service-king-north-las-vegas?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'https://www.elegance-living.com/senior-living/nv/las-vegas/wentworth-las-vegas', 'http://www.cafekrishnaaz.com', 'http://mountaincreektrees.com', None, 'http://www.batw.ca/', None, '1592160137', None, 'https://locations.vitaminshoppe.com/nv/lasvegas/vitamins-supplements-las-vegas-nv-394.html', None, None, 'http://www.secondsoleohio.com/#!akron/c1fn6', None, None, None, None, 'http://www.purpleorchidflowers.ca/', None, 'http://www.bluepandalv.com', 'http://calgary.treehouseplay.com/', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'http://www.ncdot.org/dmv/', 'http://www.2meatballz.com/', 'https://www.fit4less.ca/locations/ontario/north-york/409?utm_source=yelp&utm_medium=cta&utm_term=torontolawrence-f4l', 'http://www.mrhero.com/', 'http://www.truevalue.com/', 'http://www.pombalensebakery.com/', None, 'http://www.peoriacpr.com/', 'http://www.terribleherbst.com/', None, 'https://www.smartstyle.com/en-us/locations/on/brampton/located-inside-walmart-3135-haircuts-5184.html', 'http://www.massagebymichael.vegas', 'https://www.myeyedr.com/locations/ballantyne?utm_source=Yelp&utm_medium=Referral&utm_campaign=ballantyne', None, 'http://www.reebokcrossfitramsay.com', 'http://blackowlrealtylv.com', 'http://www.speducci.com/', None, 'https://deadcityvegas.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://bostonpizza.com/centropolis-laval', 'https://satruck.org', 'http://www.richbeckguitars.com/', None, 'http://www.inspiringsmilesdentistry.com', None, 'http://www.morcillapittsburgh.com', 'http://www.kids-supergym.com', None, 'http://www.deserthottubs.com', 'http://www.dimaggiostire.com/', 'https://local.fedex.com/nv/las-vegas/office-1441/', None, None, None, 'http://budstreecare.com', None, 'http://www.saintsandsinnershairdesign.com', 'http://www.woodiesautoservice.com', None, 'http://www.gap.com?tid=gpme000290', None, 'http://rawfitnesslv.com', 'http://www.starbucks.com', 'https://cocosgelatooh.com', 'http://www.sissybeautysalon.com', 'http://m.primarycarewalkinmedicalclinic.com', None, None, 'https://www.sprint.com/storefronts/bd/sprint-matthews-nc-28105-fcs-2516/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.xiangcunfaxian.com/', 'http://Www.LVLegalHelp.com', 'https://scottsdale.andaz.hyatt.com/en/hotel/dining/weft-warp-art-bar-kitchen.html', 'http://www.salzmann.com', None, 'http://glossitdetailstudio.com/', 'http://www.trcaparks.ca', 'http://www.eastofchicago.com', None, 'https://chocolateshops.sees.com/pa/pittsburgh/chocolate_shops_pittsburgh_pa_pa-17.html', 'http://houseoffitness.net/', None, 'http://www.circletree.net', 'https://www.atmosphere.ca/stores/calgary.html', 'http://www.burgers2beer.com/', 'http://www.shipleyortho.com/', None, None, 'http://www.mamathaikitchenaz.com', 'http://www.junctionatantiquity.com', 'http://www.vendettatowing.com', None, None, 'http://www.centennialwellnesscenter.com', 'http://thebiomedcenter.com/', 'http://www.LandisReedHomes.com', 'https://www.thegibsoncharlotte.com', 'http://www.vientianepalacerestaurant.com', 'https://www.allauramedical.com', None, 'http://www.benefitcosmetics.com/?ext_id=yelp_ulta_010713_Yelp', 'http://www.cnroofing.com', None, 'http://www.sabas.com', 'http://www.greatclips.com/', 'http://www.mammothairlv.com', 'http://www.daleshirleys.com', None, 'http://www.baldwinbrothersautomotive.com', 'http://www.gyroworldohio.com/', None, None, None, 'http://www.bistro-jules.com', None, 'http://onsitebuilds.com/', 'http://www.livewellarborsteelecreek.com', 'http://www.augiessportsgrill.com/', None, None, 'https://www.gowireless.com/stores/az/gilbert/gilbert-gilbert-road/', None, None, 'http://www.arrabiatas.com', None, 'http://www.heroburgers.com', 'http://www.amesachiropractor.com', 'https://onginstitute.com', 'https://www.woodlandestatesapartment.com', 'http://www.bamboobakery.com', 'http://www.olsonglassandmirror.com', 'http://www.carolinafoxs.com', 'http://www.kingwongchinese.com', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.theminderbinder.com', 'http://www.wpinsure.com', 'http://www.sagastasalon.com', 'http://velveteenrabbitlv.com', None, None, 'http://pesterminate.ca', 'http://downtown.muramoto.biz', None, 'http://nevadacheapcars.com/', 'https://www.laserquest.com/nv-lasvegas', None, None, None, None, 'http://smogbusters.com', None, 'http://www.theriverrockgrille.com', 'http://www.myresortspa.com', None, 'http://nalysfloralshop.com', 'http://www.plantitsucculent.com', None, 'http://www.greatclips.com/salons/1552', 'https://www.suburbanpropane.com/locations/phoenix-az/', 'http://www.windycityair.net', 'http://www.lavidamassageshadysidepa.com', None, None, '1592160519', 'http://www.upandawaylandscaping.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, None, 'http://www.sincerelyyogurt.com/locations/peters-township', 'http://www.classickids.biz', 'http://mastercrafttires.com/', 'http://glenwoodacresrvpark.com', 'http://forecloseduponpets.org', 'https://www.mediweightloss.com/locations/scottsdale?_ppc=Z8FWKRXKT5', 'http://galaxiedinerwoodbridge.com', 'http://yeoldsoapmill.com/touch', None, None, 'http://www.1866staydry.com', None, 'http://www.sarphx.com', None, None, None, 'https://homesbyclaudiamarion.kw.com', 'http://cloud9windows.com', 'http://www.bevmo.com', None, None, None, 'http://www.tailgatersilprimo.com', 'http://www.rainbow.travel', 'http://www.timhortons.com/', 'http://www.noblebeastpets.com', None, 'http://compadresmexicanrestaurantwi.business.site', '1592160596', '1592160598', 'http://www.sunnymorning.ca', None, 'https://www.westvalleytech.net', 'http://www.lasvegasbraces.com', 'http://www.cicispizza.com', None, None, 'https://stores.dickssportinggoods.com/az/phoenix/365/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281360123&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.gamestop.com', None, 'http://www.apple.com/retail/scottsdalequarter/', '1592160628', 'http://www.mark-taylor.com/arizona/san-marquis/?utm_content=visit-website&utm_medium=listing&utm_source=yelp', '1592160634', 'http://www.quiktrip.com/', 'http://www.yellowcheckercabchampaign.com/', 'https://www.lowes.com/store/PA-Monroeville/1660', 'https://megafurnitureusa.com', 'http://greenbeautyspa.com/', 'https://www.wyndhamhotels.com/hotel/52964?tel=18666154223&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52964', 'http://www.itsnewtomeconsignments.com', 'http://www.valueworld.net/', 'http://www.lunarossaristorante.com/', None, 'https://www.bannerhealth.com/locations/mesa/banner-urgent-care-1', None, 'http://www.planetfitness.com/gyms/willowick-oh-985?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://jakesautorepair.com/', None, None, 'http://www.flippnburgers.ca', 'https://camp.nc', 'http://www.lenotrebistro.com', None, 'http://www.yourfloorz.com', None, '1592160693', 'http://www.paradisejuice.com', None, 'http://www.nailsforyou.ca', 'http://www.vintagebymisty.com', None, 'http://www.chezcora.com', 'http://www.thehealthygreek.ca/', 'http://www.upmc.com/locations/hospitals/western-psychiatric/Pages/default.aspx', None, 'https://www.scottsdalecc.com', 'https://www.anytimefitness.com/gyms/92/Phoenix-AZ-85048/?utm_source=yelp&utm_medium=local', 'http://www.tccarpetcare.com/', 'http://www.phohongnc.com', 'https://us.louisvuitton.com/eng-us/point-of-sale/usa/louis-vuitton-cleveland-saks?utm_source=yelp&utm_medium=organic&utm_campaign=store_listing_storepage', '1592160727', None, 'http://www.nice1nails.com', None, 'http://www.travelsuitesinn.com', None, 'http://priscillaspantrypetsupplies.business.site', None, None, 'http://www.avidalaserandspa.com/', None, None, 'http://www.city-tavern.com', 'http://www.lockquest.com', None, 'https://www.plannedparenthood.org/health-center/arizona/scottsdale/85251/scottsdale-health-center-2809-90030?utm_campaign=scottsdale-health-center&utm_medium=referral&utm_source=yelp.com&utm_content=link', None, None, None, None, None, 'http://tirerepairlasvegas.com', 'http://www.tangdynastyurbana.com', None, None, 'http://www.thegridcl.com', None, 'http://www.janasredroom.com', 'http://atrackout.com', None, None, 'http://www.mancavecigarlounge.com', 'http://quicklaneoflasvegas.com/', 'https://locations.theupsstore.com/nv/north-las-vegas/5575-simmons-st?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://consulmex.sre.gob.mx/lasvegas/', 'http://www.azstoragedoctor.com', 'http://www.greenstreetcommunitiesinc.com', 'http://www.pcpropane.com', None, None, "http://charmeck.org/mecklenburg/county/ParkandRec/Greenways/OpenGreenways/Pages/MallardCreekandClark'sCreek.aspx", 'http://www.paradisevalley.edu/', 'http://www.curiouskidspreschoolaz.com/', 'http://www.gardenialanephotography.com', 'http://www.kandrpainting.com', 'http://bestestiedayspa.com', None, None, None, 'https://locations.panerabread.com/oh/hudson/118-w-streetsboro-street.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', 'https://www.usps.com/', 'http://www.apartmentsnow.online', 'https://www.5280mexicangrub.com', 'http://Destinations.rei.com/Arizona', 'http://www.ferrarimaseratioflasvegas.com', None, 'http://www.justrightlandscape.com', 'https://arizonapain.com/contact-us/chandler/', 'http://www.degnanlawaz.com', '1592160875', None, 'http://azvideomarketing.com', None, 'https://www.marks.com/en/stores/north-york-2625d-weston-road-14-15-crossroads-centre.html', 'http://toterafinefoods.ca', 'http://www.moonhaven.com/', None, 'http://www.torontoguesthouses.com', 'http://massagemaniaglendale.com', None, None, None, None, 'http://www.plannedparenthood.org/health-center/ohio/rocky-river/44116/rocky-river-health-center-2418-91230', None, 'http://www.totaltransit.com', 'https://www.marriott.com/hotels/travel/phxsh-springhill-suites-scottsdale-north/', 'http://www.skatenrd.com', 'https://blueboxmovingcompany.com', 'http://lamsaz.com', 'http://www.designsbylanae.com', None, 'http://arborstownhomes.com', 'http://www.europebound.com/', 'http://www.acehardware.com', 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', 'https://azvent.com/locations/mesa.php', 'http://usacollision.com', None, 'http://www.falloutgames.net', 'http://www.shaanxibiangbiang.com', None, 'http://www.brettbaughman.com', 'http://swgclv.com', 'http://www.kromerradio.com', 'http://www.acmteam.com/acm/outside_home.asp', 'http://www.orvis.com/madison', 'https://www.retreataptsmcalpinecreek.com', None, 'http://www.elvischapel.com', 'http://www.ubreakifix.com/locations/peoria/', None, 'http://www.superchargefoods.com', 'http://www.thesakeexpress.com', None, 'http://www.topnoodle.ca', 'http://www.jazzberryteahouse.com', 'http://www.berkshireplaceapartments.com/', 'http://www.horizonhonorselementary.org', 'https://www.enterprise.com/en/car-rental/locations/us/pa/irwin-4021.html?mcid=iyp:8368400', 'http://www.4colorprint.com', 'https://deserttropicspools.com', 'http://martyscycle.com/', 'https://www.ttc.ca/Subway/Stations/Bay/station.jsp', 'http://www.sunsetgrill.ca', 'http://www.sothlv.com', 'http://www.palms.com/?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.ekgproperties.com', None, None, 'https://www.loandepot.com/branches/las-vegas-nv?sc_camp=DE925F8ECA0F4E0D8092C7E108A615A3', None, 'http://www.newmexicangrill.net', '1592161032', None, None, 'http://www.lenasflowersandgifts.com', 'http://www.cell2fix.com/queen-410-brampton/', None, 'http://www.tonymorenosalon.com', None, None, None, None, 'http://www.callister-law.com', 'http://www.smokeybones.com/', 'http://franklinPhotographyAz.com', '1592161065', 'http://www.thelastwordcharlotte.com/coffee-press', 'http://totalofficenc.com/', None, None, 'https://www.kaltire.com/en/stores/locations/?source=yelp_track&StoreID=864', 'http://www.corinthiapeoples.com', 'http://www.madurbanbees.com/', None, None, '1592161087', 'http://www.petopia.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.BarrowNeuro.org', '1592161101', 'http://www.southpittsburghvet.com', 'http://www.safeguardministorage.biz', 'http://www.lasvegasnvpoolservice.com', None, 'http://www.shopthepig.com', 'https://www.fatburger.com/locations/rancho', 'http://www.preferredparking.com', 'http://charkha.us/', 'http://www.hardboiledinc.com/', 'http://fasttowingaz.com', 'http://www.mephisto.com/us', None, 'https://locator.chase.com/oh/akron/50-s-main-st', 'http://www.lous.ca/', 'http://www.stratospherehotel.com/Food-Drink/Dining/Carvery-108', 'http://onestopnutrition.com', 'https://www.mimiscafe.com/locations/sunset-station/', 'http://www.nice1nails.com', None, '1592161151', None, 'http://diamondcarwash.ca', 'http://delucasplaceinthepark.com', None, 'https://www.torontobuffing.com', 'http://collinscleaners.com', None, 'http://xquiziteaffairz.com', None, 'https://locations.checkers.com/nv/las-vegas/4175-e-desert-inn-rd/?utm_source=Yelp', 'https://www.tedwiens.com', 'http://visionsource-southhills.com', 'http://www.timhortons.com/', 'http://www.arizonaeyedoc.com', None, 'http://www.mrhandyman.com/western-dane-county?sppccampaignid=67352&SPPC=Offline&L=true', 'https://www.whitbylibrary.ca', 'http://www.dangfinerentals.com', None, 'https://pizzariviera.com', 'http://eyetique.com', None, None, 'http://www.haulnassproductions.com/contactus.html', None, 'http://www.kinginklasvegas.com', 'http://www.hartwooddentists.com', 'https://www.ahwfac.com/', 'http://www.southparkoralsurgery.com/', 'http://store-locator.barnesandnoble.com/store/2039', 'http://www.imaginationchildcare.com', 'http://www.palazzo.com/entertainment/shows/baz.html', 'http://www.culturefreshlife.com', None, 'http://lvshotokan.com/', 'http://woolfeye.com', None, 'http://www.cloudadagents.com', 'http://www.viautorepair.info/', None, None, 'http://www.littlecaesars.com', 'http://terros.info', None, 'https://ad.doubleclick.net/ddm/clk/434334113;236814625;y', None, None, 'https://campbellssweets.com/locations/campbells-sweets-factory-ohio-city/', None, 'http://charmeck.org/mecklenburg/county/ParkandRec/Greenways/PlannedGreenways/Pages/McDowellCreekGreenway.aspx', None, None, 'http://www.italianvillagepizza.com/store-locator.html', 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', 'http://www.coldstonecreamery.com', None, 'http://www.smileworkscalgary.com', None, 'http://www.bannerhealth.com/locations/phoenix/banner-physical-therapy-mcdowell?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', None, 'https://rosyeyebrowthreading.business.site', 'http://www.sushiichibanchampaign.com', 'http://www.suenosimports.com', 'https://www.wellsfargo.com', '1592161298', None, 'https://www.davey.com/eastpittsburgh?source=Yelp', 'https://www.aria.com/en/restaurants/aria-patisserie.html', 'http://www.dpdough.com', 'http://www.us.allsaints.com/', 'https://www.payless.com/stores-location/OH-PARMA-6259.html', None, None, None, 'https://www.chick-fil-a.com/scottsdale101', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://thenakedbbq.com', 'http://www.ensurco.ca', 'https://www.mattressfirm.com/stores/az/scottsdale/mattress-stores-scottsdale-az-85266-6664.html', 'http://www.tenthousandvillages.com', 'http://chatime.com', 'http://www.childcareproviderlasvegas.com', 'http://www.meckabc.com', 'http://storelocator.pepboys.com/pepboys/stores/pittsburgh/library_road_service_center/01425', 'http://www.obskincare.com', None, None, None, 'http://www.valleyacademy.com', 'https://www.freshlandflowers.ca/?srccode=yelp_track', 'https://www.saladmeister.com', 'http://yogalovepgh.com', 'http://www.countyofdane.com/lwrd/parks/lake_farm.aspx', 'https://silvertoncasino.com/dine/dining-in-las-vegas-on-a-budget/', 'http://lasilmosoptical.com', 'https://www.solasalonstudios.com/salon-professional/lainie-schmidt2', 'http://winghartburgers.com/', 'http://www.bobzip.com', 'http://absoluteautoservicelv.com', 'http://tacoskissiphoenix.lunchroomconstructor.website', 'http://www.adelaideclub.com', 'http://www.drrudisill.com', None, None, None, 'http://www.maxandermas.com/locations/333038', 'http://mariannasalterations.com', 'http://www.bossplow.com', 'http://www.liquordepot.ca/?utm_source=Yelp&utm_medium=Online_Listings&utm_content=Location&utm_campaign=LiquorDepot_AB_Q3_July_September_2017', 'http://www.assuranceexteriors.com', None, 'http://www.shell.com', None, 'http://www.table51.com', 'http://www.recreation.gov/recAreaDetails.do?contractCode=NRSO&recAreaId=7&contractCode=131', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.wildpcs.com', 'https://ad.doubleclick.net/ddm/clk/301028788;127996861;v#/Live%20Weekend%20Entertainment', None, 'http://www.reebokcrossfitlibertyvillage.com/', 'http://www.pittsburgh-serenity.com', 'http://www.torontopubliclibrary.ca/detail.jsp?Entt=RDMLIB014&R=LIB014', 'http://dogboneranch.com', 'http://www.cleanjuice.com', 'http://dilworthcoffee.com', 'http://www.rompdogs.com', 'https://www.haircp.com', '1592161456', 'https://www.surlatable.com/browse/storeLocator/storeHome.jsp?storeId=140', '1592161460', 'http://filigreebeds.com', 'http://www.colonyamerican.com/?utm_source=Yelp!%20Regional&utm_medium=Text%20Link&utm_campaign=Yelp!%20Charlotte', None, 'http://www.juicebaratlvac.com', 'https://canyoncreekrestaurant.ca/vaughan/', None, None, None, None, None, 'http://www.acehardware.com/store-details/09471', 'http://www.timhortons.com/', 'http://www.barrospizza.com', '1592161494', 'http://smokeworldtempe.com', 'http://www.bestdoggonebookkeeping.com', 'http://www.banfield.com/veterinarians/az/phoenix/pho?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://alphaomegafg.com/', 'https://www.enterprise.ca/en/car-rental/locations/canada/on/north-toronto-eglington-yonge-c245.html?mcid=iyp:8368400', 'https://www.aqua-tots.com/surprise', 'http://www.kortmaninc.com', None, 'http://browsthreadingsalons.com', 'http://www.strideawayequestriancenter.com', 'http://www.mattressfirm.com/', None, None, None, 'http://www.saigonphocuisine.com', 'http://www.salvationoutdoor.com', 'http://us.christianlouboutin.com/us_en/', None, 'http://www.pizzaiolo.ca/?utm_source=Yelp', 'http://edhardyshop.com', 'http://www.parkshairhaven.com', None, 'http://www.hautechixvegas.com', 'http://integrity.helpusell.com', 'https://www.avis.com/en/locations/us/nv/las-vegas/qn4', 'http://www.matabar.ca', 'http://www.vivaldisalonsuites.com', 'http://ritzfurnitureplanet.ca/', 'http://movingteamsix.com', None, None, 'http://www.ecocleaners.ca/', 'https://locations.wendys.com/united-states/oh/parma/6970-ridge-road', 'https://www.discounttire.com/store/az/mesa/s/1294?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://biltmorenailbar.com', '1592161583', None, None, 'http://rumerzpgh.com/', 'http://www.suite114.ca', 'http://www.myspace.com/giant45', None, None, 'https://www.homedepot.com/l/Peoria/AZ/Peoria/85345/453?cm_mmc=seo-yelp&', None, None, '1592161607', None, 'https://www.k9viplv.com', 'https://www.hockeyworld.com/index/page/store_details/store_id/10', 'http://www.peninsulaluggageonline.com', 'https://www.phoeniciangaragedoorandrepair.com', 'http://www.LuminosityWellnessCenter.com', None, 'http://www.easternpodiatry.com', 'http://www.peppersfreshmarket.com', 'http://www.tremontscoops.com', 'https://local.albertsons.com/az/mesa/2727-n-power-rd.html', None, 'http://moviescoop.com', 'http://waikikigoldsilver.com', 'http://applefactory.com', 'http://www.brustersicecream.com', 'http://bigjohnspizza.ca', None, None, '1592161649', None, 'http://www.bigboy.com/', 'https://www.bluemercury.com/?utm_source=2016-Yelp&utm_medium=social&utm_content=49&utm_campaign=general', '1592161660', 'http://www.heritageatdeervalley.com', 'http://www.cellaxys.com', 'https://ireh-salon.business.site', 'http://noblecleaners.vegas', 'http://www.tuesdaymorning.com', 'http://specsonbloor.ca', 'http://www.estheticsbycristina.ca/', 'http://www.silkmanlawfirm.com', 'http://hubcapannielasvegas.com', None, None, 'http://www.eastcalgaryregistry.com', 'http://local.firestonecompleteautocare.com/ohio/akron/1245-firestone-pkwy/?lw_cmp=IYP_YPC_MLP_2046', 'http://www.fallsicecream.com', 'http://www.revitasize.ca/', 'http://www.elitefitnesstrainingandcoaching.com/', None, None, 'http://www.atozmovinganddriving.com', None, None, None, 'http://www.lagravineseappraisals.com', 'http://www.custombuiltpizza.com/', 'http://www.jeanvegetariankitchen.ca', '1592161722', 'http://beautyinvegas.com', 'http://www.countyofdane.com/lwrd/parks/indian_lake.aspx', 'http://www.phillipscliniclv.com', 'https://www.mattressfirm.com/stores/oh/streetsboro/mattress-stores-streetsboro-oh-44241-7782.html', 'http://www.hollywoodtans.com', 'http://budget.com', 'http://www.saltcreekofficefurniture.com', 'http://www.happycampersdental.com', None, None, 'http://www.pizzanova.com/locations/mississauga.html', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, 'http://h1racing.com', 'http://pdqmechanicalllc.com', 'http://www.edenwellnessvibration.com', 'http://yardbusterslandscaping.com', None, None, 'http://www.uwhealth.org/locations/detail.jsp?locationId=254', None, None, 'https://www.backfithealth.com', None, 'http://microsoft.com/en-ca/store/locations/ab/calgary/chinook-centre/store-1053?pointit_retailsustain_sitelinks_Yelp', 'http://ruggedmoosedecor.com', None, 'http://simplekitchento.com', 'https://www.republicservices.com?utm_source=yelp.com&utm_medium=referral&utm_campaign=Yelp_Profile_Click&utm_term=gTepcuWPpm5jqrVjM0gwew', 'https://www.standardplumbing.com/locations/PE', 'https://local.fedex.com/az/phoenix/office-5048/', '1592163820', None, None, 'https://ollyshoes.ca/pages/yonge-street-store', 'http://www.waterfrontoronto.ca/explore_projects2/west_don_lands/don_river_park', 'http://www.biggamblesrummagesale.com', None, None, None, None, 'http://www.arizonabouncearound.com', 'http://lavidamassageballantynenc.com', None, 'http://adultmart.com?utm_source=yelp&utm_medium=social&utm_campaign=yelptest&utm_content=url', 'http://www.firewatercalgary.com/', 'https://www.hertz.com/rentacar/location/unitedstates/arizona/phoenix/PHXC04', 'http://www.sameatshops.ca', 'http://mywheelguylv.com', 'https://locations.theupsstore.com/az/tempe/1753-e-broadway?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'http://www.shopsteelcity.com', 'http://www.RiteRug.com', 'http://www.thegolfdome.com', 'http://hianddrycleveland.com', 'https://plan.dignitymemorial.com/yp/plan/?id=542949820&utm_source=yp.com&utm_campaign=ypm&utm_medium=display&utm_term=386&utm_content=mlp', 'http://www.eurogyro.com/', 'http://www.marriott.com/hotels/travel/cltfa-fairfield-inn-and-suites-charlotte-arrowood/', '1592163884', 'http://www.logojoy.com', 'https://locations.theupsstore.com/az/scottsdale/16211-n-scottsdale-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.donttreadonbeans.com', None, None, 'http://www.sleepcountry.ca/', 'http://allcountypet.com', 'http://glenmeadowsauto.mechanicnet.com', 'http://www.parloursalon.com', None, 'http://www.merchantoftennis.com/', 'http://www.neshoffscarpetcleaning.com', 'http://www.fortmcdowelladventures.com', 'http://www.psychicamandamarquez.com', 'http://www.tricountymg.com', 'https://www.alaschools.org/arizona/az-schools/gilbert-k-6', None, 'https://pastureandplenty.com', 'https://palacestation.sclv.com/Gaming/Bingo?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', None, 'http://www.cupidstoybox.com', None, 'http://thesanddollarlv.com', 'http://www.usysnv.net/page/show/110658-bettye-wilson-soccer-complex', 'http://www.themeadowsschool.org', None, 'https://www.esso.ca/en/gas-station/richmondhill-on-beavercreekesso-200302403', 'http://www.studiocsalonsaz.com/studio-c', 'http://www.clorebeauty.com', '1592163956', 'http://klimateking.com/', 'http://www.spinco.ca', 'http://infinitevapor.com', '1592163966', 'http://www.thebrewteabar.com', 'http://www.chopchopchinesekitchen.com', 'http://www.chilitos.ca', 'http://thewienerscirclelv.com/', 'http://www.hob-nobs.com/', 'http://www.legendaryhomeservice.com', 'http://DefiantPestControl.com', 'https://www.napoli.vegas', 'http://www.brunswickkidds.com', 'http://www.ctrca.com/', 'https://www.trekbikes.com/us/en_US/retail/madison_west/', 'http://www.streetsofnewyork.com', 'http://nbdmd.com/', 'http://phoenix.gov', 'http://www.azpodiatrists.com', 'http://www.larryscocktails.com', 'http://www.cookout.com', 'http://spandexcity.com', 'http://www.casalinochiropractic.com', None, 'http://www.samdaebbq.com', 'http://balancingpawsdogtraining.com', None, '1592164037', 'http://www.revivme.com/las-vegas?utm_source=yelp-org&utm_medium=webclicks', '1592164042', 'http://www.beelogicpestcontrol.com', 'http://www.creativestyleslv.com', None, 'http://Www.bellaskinsalon.com', 'https://www.chick-fil-a.com/rossparkmall', '1592164058', None, 'http://www.bluejadechinese.com', 'http://nswhobgyn.com', 'http://brewsters.ca/', '1592164073', 'http://www.luxebrides.vegas', None, None, 'http://paradisebakery.com', None, 'http://actoneyouththeater.org', 'http://www.cityofhenderson.com/henderson-happenings/facilities/overview-of-pools/pool-locations/whitney-ranch-indoor-pool', 'http://www.7-eleven.com/?yelp=29384', 'http://www.johnniewalkerrv.com/index.php', 'http://www.ColleenSellsAZ.com', None, None, 'https://www.haircutmengibsoniapa.com', 'https://www.payless.com/stores-location/IL-CHAMPAIGN-5067.html', '1592164114', 'https://www.cafetempeaz.com', None, None, 'http://www.bartongrouphomesearch.com', 'http://foleysac.com', 'http://www.foodandwineexpo.ca/sitepages/', None, None, '1592164141', None, 'http://www.jaginchystuff.com', 'http://www.tremontpointe.com', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975319&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.qashtonbeauty.com', 'http://www.phoenixairport.hilton.com', 'http://www.sweetaffairsbakeshop.com', 'https://www.verizon.com/home/storelocator/pa/pittsburgh-15206/', '1592164167', None, 'http://local.firestonecompleteautocare.com/arizona/peoria/10661-n-84th-ave/?lw_cmp=IYP_YPC_MLP_11932', None, 'http://www.azcoolmotors.com', 'http://www.shoproka.com/web/', 'http://www.gostorageone.com/self_storage/North_Las_Vegas/zip_89084/storageone_self_storage/10487', 'https://www.anytimefitness.com/gyms/1820/Markham-ON-L6B-0P2/?utm_source=yelp&utm_medium=local', 'http://www.pinkpolishnailspa.com', None, None, 'http://www.boyacklawgroup.com', 'http://dekra.us/en/nevada/las-vegas-3209-n-rainbow-blvd/', 'http://www.goplaycanada.com', 'http://www.crispfoods.com', 'http://www.restechservices.net', 'http://www.polepositionraceway.com', '1592164211', None, 'http://geraldinetoronto.com/', 'http://www.bakerperformingarts.com', 'https://dutchbros.com/?utm_source=las-vegas&utm_medium=yelp&utm_campaign=website_las-vegas', None, None, None, 'https://tortillafeliz.com', 'http://www.charlotterecyclery.org', 'http://www.maxmoving.ca', None, 'http://www.daggerdiesel.ca', 'https://my.clevelandclinic.org/locations_directions/Regional-Locations/beachwood-fhsc', None, 'http://www.cannonpharmacies.com', 'http://www.bettysonking.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.waffluv.com/find-us', 'http://www.dunkindonuts.com', 'http://www.ytr.com', 'http://www.scoopnswirl.com', 'http://www.piedmont-eyecare.com', 'http://www.thefoodconnectionlv.com', None, 'http://www.starbucks.com/', 'http://www.copperrestaurant.com', None, 'http://www.peachykleenlaundry.com', 'https://wildfire.sclv.com/Wildfires/Wildfire-Rancho?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://extremepita.com', None, 'http://www.axelmortgage.com', 'http://www.hobbylobby.com/StoreLocator/Search?Latitude=33.2774819&Longitude=-111.73165570000003&SearchOn=85297&SearchDistance=50', None, None, None, 'http://www.myerstar.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', '1592164296', 'http://www.sportclips.com/', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975058&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, '1592164307', 'http://www.busterstransmission.com', 'http://www.the-alley.ca', 'http://www.mifamiliamexicanfood.com', 'http://Www.thebubblegumgallery.vpweb.com', None, None, 'http://www.lincolnlandexpress.com/index.php', 'http://www.kaiulanidetailing.com', 'http://restaurants.quiznos.com/oh/lorain/amherst-44053', 'http://www.angelscremationandburial.com/', None, None, None, 'http://www.luxepethotels.com', None, 'http://www.officedepot.com/storelocator/az/chandler/officemax-6515/?cm_mmc=GMB-_-6515-_-Website-_-NS', 'https://www.hughesfloorcovering.com/yelp-offer', 'http://www.pinesatcarolinaplace.com', None, None, 'http://www.charlotteobserver.com', None, 'http://www.tmdish.com', 'http://www.insideoutpatio.ca', 'http://www.firecreekcoffee.com', None, None, 'http://www.QuickKeyLocksmithLasVegas.com', 'http://www.loftea.ca', 'http://sandersonplumbinglv.com', None, 'http://www.sopra.ca', 'http://mylifelinemd.com/', 'http://EUsweets.com', 'https://vinoandcanvas.com', 'http://www.desertsunpeds.com', 'http://www.allstardiscountmuffler.com', '1592164392', None, None, 'http://www.pokerpalace.net/', 'http://www.pizzapros.ca', 'https://www.thebso.com/locations/on/toronto/beauty-supply-outlet-haircuts-11813.html', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_112', 'http://www.salstuscangrill.com/', 'http://www.arizonahikebikeboard.com', 'http://www.providentlawyers.com', None, 'http://www.unicarcollision.com', 'http://kickstandcharlotte.com/', 'http://www.american-self-storage.com/avondale', None, 'http://www.TopScottsdaleHomes.com', 'http://www.ssrealtylv.com', 'http://www.thelimited.com', None, 'http://www.shawarma-palace.com', 'http://www.picturepeople.com/default.aspx?PK=e3adeef4-54b4-43c6-990a-619531e6637b', 'http://www.linsseafoodbuffet.com', None, 'http://www.whichwich.com', 'http://redlinediagnosticsllcauto.com', 'https://www.walmart.com/store/5085/belmont-nc/whats-new', None, 'https://stores.whitehouseblackmarket.com/boutique/white-house-black-market-kierland-commons', 'http://www.theconcordcove.com/', None, 'http://thenailchamber.com', '1592164461', None, 'https://www.paradisegrapevine.com', 'http://www.chrispuhlmanflowersandgifts.com/?srccode=yelp_track', 'http://phbalancedpool.com', 'http://www.timhortons.com', 'http://petemac.com/gilbert/', 'https://millersalehouse.com/locations/henderson/?utm_source=Yelp&utm_campaign=Yelp', 'http://nailing.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://vehbrothers.com', None, None, None, 'http://www.noizeboyz.com', 'http://www.originalchimneys.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.blackhawkcc.com', 'http://www.dollargeneral.com', 'http://kingdom-restaurant.business.site', 'https://www.choicehotels.com/illinois/urbana/comfort-suites-hotels/il366/rates', None, 'http://www.sunburstmusic.com', 'http://torontoauto.autotrader.ca/contact-form.htm', 'http://www.thecleverkoi.com/', 'http://www.madcitypetclinic.com', 'http://www.tandori.ca', 'http://www.waitressworldllc.com', 'http://homeinsulationaz.com', 'http://www.woodworkersemporium.com', None, 'https://www.elnuevotaquito.com', None, None, 'http://luxenailsspaphoenix.com', 'http://www.masakosushibar.ca', 'http://www.pandawokscarborough.com', None, 'http://www.tacobell.com', None, 'https://www.sunbritedental.com', 'http://atticsalt.co', None, None, 'http://www.drfineberg.com', 'http://www.uscryotherapy.com/location/scottsdale-az/', 'http://www.sureshotbilliardsaz.com', 'http://uspizzasurpriseaz.com', 'https://www.statefarm.com/agent/US/NC/Charlotte/Jack-English-YKJTS2RZ000?cmpid=M8V7_BLM_0006', None, 'http://www.bedbathandbeyond.com', 'http://www.arizonaministers.com', 'http://www.usairways.com?c=maplisting_Yelp_21041', 'http://www.arbys.com', 'http://www.kuohua.com', 'https://brighthomeenergy.com', None, 'https://www.meineke.com/locations/pa/monroeville-512/?utm_source=yelp&utm_medium=xvc', None, '1592164595', None, '1592164600', 'http://www.shoelessjoes.ca', 'http://azsportsmedicine.com', 'http://www.decadentdesserts.ca', '1592164609', 'http://www.whynotpizzaclt.com', None, 'http://www.diventures.com', 'http://www.ncix.com/', '1592164621', None, None, None, None, 'http://locations.michaels.com/oh/avon/1532/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.allamericanmodernsportsgrill.com', 'https://www.windsorstore.com/blogs/store-locator/az-scottsdale-scottsdale-fashion-square', 'http://www.trumphotels.com/las-vegas', 'http://www.grldcheese.com', 'http://www.wowwash.com', 'http://www.zoyogurt.com/arcadia/', 'https://hatfieldmedicalgroup.com/red-mountain-primary-care-arizona/', None, 'http://www.lafitness.com/Pages/clubhome.aspx?clubid=669', None, 'http://shop.tuesdaymorning.com/', 'https://www.pizzaville.ca/stores/view/4335-bloor-street-w', 'https://www.meijer.com/shop/en/store/146/?cmpid=OLA:040319:YelpGrocery', None, 'http://local.skechers.com/AZ/GILBERT/237/', None, 'http://www.arbys.com', None, None, 'http://www.gratefulgardens.net', 'http://www.wynnlasvegas.com/Shows', 'http://www.recess.net/', 'https://www.aaautocarelv.com/locations/decatur-blvd/', 'http://www.remnanthealth.com', 'http://computerdocnc.com', None, None, 'http://www.papajohns.com', 'https://bananarepublic.gap.com', 'https://www.boehmerheating.com', 'http://fancynailsnorthlasvegas.com', 'https://www.vibenailbarlv.com', None, 'http://www.littlemountaincc.com', None, 'http://azglaucomaspecialists.com/our-locations/phoenix/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://icedoutbarbershop.com', 'http://www.moxies.ca', 'http://www.villamadina.com', 'https://www.davidyurman.com/stores/s/david-yurman-the-forum-shops-at-caesars?ecid=listing%20marketing%20store%20LasVegas%20all%20082216%20all', 'http://www.coconutsaz.com', 'http://www.superstarcarwashaz.com', None, 'http://naildistribution.com/', 'http://www.bk.com', 'http://www.screenlounge.ca/', 'http://beachpizzavegas.com/', 'http://www.anthropologie.com', 'http://www.comicheaven.com', 'http://www.quiktrip.com', 'http://richardsandoval.com/commissary', 'http://www.coldstonecreamery.com/stores/20030', None, 'http://lvbraces.com', 'http://www.silverspoonphoenix.com', 'http://www.ashleyfreehan.com', 'https://radiantdivine.com', 'http://www.dahmensathawks.com', 'http://www.kidsu.ca/', 'https://culinaryhc.com', None, None, 'http://www.luckybreakaz.com', 'http://www.3dudesquilting.com', 'http://www.theprincessmargaret.ca/en/pages/default.aspx', None, None, 'http://www.estrellamountain.edu/', 'http://www.greendrycleaning.ca', 'http://www.westshorehome.com', 'http://www.greatclips.com', None, 'http://laurel-md.yellowusa.com/', 'http://www.honestjohnspgh.com', None, 'http://www.tandoorichaska.com', 'https://www.tempemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', None, 'http://myporcellis.com', None, 'http://www.JimmyJohns.com', None, 'https://www.godsgardentreasures.com', None, None, 'http://www.burns-scalo.com', 'http://www.mrsteamcarwash.ca', None, 'http://www.nouvellemariaspa.com/', 'http://rinckerlaw.com', 'http://www.drchemdry.com', 'http://nationsfreshfoods.ca', 'https://www.kohls.com/stores/nv/lasvegas-1372.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.fourpeaksrentals.com', 'http://www.canuckamusements.com', 'https://greenvalleyranch.sclv.com/Dining/Hanks?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYZGS', 'https://stores.bestbuy.com/nv/las-vegas/6950-arroyo-crossing-pkwy-1044.html/?loc=ns100&ref=NS', '1592164907', 'http://www.roccuzzorealty.com', 'http://www.surprisechildcare.com', 'http://www.shuttlersuniform.com', None, 'http://www.ecochoicewindows.ca', 'http://www.sushiandrolls.ca', None, 'http://www.backofhouse.ca/', 'http://www.handmunlimited.com', None, None, 'http://www.asimpleaffairlv.com', 'http://www.kegsteakhouse.com/en/locations/ON/toronto/mansion-keg/', 'http://www.spinellikia.com', 'http://www.saborcharlotte.com', 'http://www.kctowinginc.net/', 'https://www.crudowine.com', 'https://www1.shoppersdrugmart.ca', 'http://www.jaipurgrill.com', None, 'http://madisoneyecare.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.blackbearcharlotte.com/', 'http://mycupoftea.ca', '1592164974', None, 'http://olympiclimoservice.ca', 'http://www.beddingtons.com', 'http://villagecp.com', None, None, 'http://www.aonefingerprinting.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66054.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:66054', 'http://elementsdrycleaner.com/', 'http://www.hungfooktong.ca', 'http://www.ayscater.com', '1592165005', None, 'http://www.mwuclinics.com', None, None, 'http://amelia-c.com/', 'https://www.hilton.com/en/hotels/pitmcht-home2-suites-pittsburgh-mccandless-pa/?SEO_id=YELP-HT-PITMCHT', 'http://www.dunkindonuts.com', 'http://local.buckle.com/PA/PITTSBURGH/415/', 'http://www.pleasantlakeapartments.com', 'https://www.supercuts.com/locations/az/phoenix/village-square-haircuts-9668.html', '1592165031', 'http://www.ateamair.net', 'http://GoldenRuleFB.com', 'https://locations.jackinthebox.com/us/nv/boulder-city/1101-nevada-hwy', None, 'http://www.greengardenflowers.ca/', None, 'http://local.skechers.com/AZ/GILBERT/237/', None, None, 'http://www.bestbuy.com', 'http://www.drshalupal.com/about.html', 'http://www.downwarddoggrooming.ca', None, 'http://www.exxonmobilstations.com/home.php?c=20140424134755&lang=en-us&storenum=757cfd0e77ee7a4ea34c6a6a556f7058', 'https://www.staples.com', 'http://www.federicosmexicanfood.com/', 'http://www.rainbowchiropracticcenter.com', 'http://www.janefendelman.com', None, 'https://glendaleparts.com', None, '1592165090', 'http://www.hairhappiness.com/ca/index.php', None, 'https://www.walmart.com/store/2112/scottsdale-az/details', 'https://www.druryhotels.com/locations/pittsburgh-pa/drury-plaza-hotel-pittsburgh-downtown', 'https://sincityent.com', 'http://www.bondplace.ca', 'https://allyouneedsmokeshop.com', 'http://www.suntechnc.com', 'http://www.mariepain.com', 'http://lazarbakery.ca', 'http://www.dollargeneral.com', 'http://www.platinumblondestudios.com', None, 'http://Chandlersmilesbrighter.com', None, 'http://www.meritagehomes.com/', None, '1592165136', 'http://fiveguys.com', None, None, None, 'http://www.barbarycoastcasino.com', 'http://abcsanitary.com', 'http://meatmarketvintage.com/', None, 'http://coconutcleaningco.com', None, 'http://bostonmarket.com', 'http://drlaffa.com', 'http://www.davescosmicsubs.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=723&utm_content=bpas', 'http://www.lrlv.com', None, None, 'http://www.bluediamondairductcleaning.com', 'http://www.castelloristorante.com', 'http://bit.ly/103bJzP', 'http://www.usps.com', 'http://aromawcs.ca', 'http://lifeguardsolutionsllc.com', None, 'http://www.chandlermall.com', None, 'http://www.aceretailer.com/aceonwilly/', 'https://reliancehomecomfort.com/toronto/', None, 'http://pizza.dominos.ca/Mississauga-Ontario-10348', None, 'https://www.clearchoice.com/locations/dental-implants-phoenix/?trans_id=webdisyelp&utm_campaign=Yelp&utm_source=YelpListing&utm_medium=Listing', 'http://www.4BCafe.com', 'http://www.onabun.ca', 'https://roxybar.ca', 'http://watchoutaz.com', '1592165236', 'http://www.drglick.com', 'http://licecrew.com', 'http://www.cafecentralclt.com', 'https://www.thebrick.com/pages/store-calgary-sunridge', '1592165249', None, 'http://wolfgangpuck.com/restaurants/casual-dining/wolfgang-puck-express/7730', '1592165258', 'https://www.dsw.ca/en/ca/', 'http://www.studio223az.com', 'https://locations.theupsstore.com/az/phoenix/1934-e-camelback-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.andoverwoodsapts.com/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.7-eleven.com/?yelp=29384', 'http://www.evelinecharles.com/', 'http://www.bffgyms.com', 'http://bbdrybar.com/', 'https://stores.advanceautoparts.com/nc/charlotte/2910-freedom-drive?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'http://drakids.com', 'http://Www.lasvegaspsychic.com', 'https://www.vegasdeserttours.com', 'https://www.lasikvisioninstitute.com/lasik-eye-surgery/tempe-surgeon-center/?subid=TMP&venid=YLP31', 'http://www.7-eleven.com/?yelp=29384', 'http://www.la-mestiza.com/', None, None, 'http://check-please-cafe.business.site', 'http://www.sweetitis.net', 'http://www.papyrusonline.com/', 'http://cutenailscharlotte.com', 'http://www.drumsnflats.com', 'https://www.uhaaz.com', 'http://www.butlerplaza.net', 'http://www.filibertos.com', None, None, 'http://nynbeauty.com/', 'http://www.tastebudsyogurt.com', '1592165348', 'http://pellathehandyman.webs.com/', 'http://www.azmediquip.com', 'http://www.efilenevada.com/', 'https://himali-kitchen.business.site', None, 'https://gynecarelv.com', None, 'http://www.annschophouse.com', 'http://www.fitnessinmotionlv.com', 'http://wwwtowingwithtlc.wixsite.com/website', 'https://repestservices.com', 'https://www.ulta.com/stores/charlotte-nc-590', None, 'https://local.safeway.com/safeway/az/chandler/4970-s-alma-school-rd.html', 'http://www.dogsbollockspub.com', 'http://www.freshii.com', 'http://www.havencraft.org', 'http://www.ampm.com', 'http://www.rockyrivervineyards.com', 'http://www.gleemhomecleaning.com', 'http://www.peets.com/?cm_mmc=yelp*peets*8_28_12_homepage*na', None, 'http://www.aubreyshouseofroots.com', 'https://steakfrites.ca/fr/restaurants/detail/saint-sauveur/', 'http://www.theartshoppe.com', None, 'https://www.twiceasnicefood.com', 'http://motorlessmotionbicycles.com', 'http://www.bathandbodyworks.com', 'http://www.tayloredphotomemories.com', 'https://www.moes.com/find-a-moes/ohio/mentor/6467?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', 'http://bk.com/', 'http://www.marriott.com/hotels/travel/cltsd-springhill-suites-charlotte-uptown/', 'http://www.a1carlinemuffler.ca', None, 'http://www.sharpmyknife.com', None, 'https://www.arizonapartybike.com/scottsdale/', 'http://bradyhendersoninspections.com/', 'https://www.jamba.com/nc/charlotte/charlotte-douglas-int-l-airport-613', None, None, 'https://www.take5oilchange.com/locations/oh/bedfordheights-121/', 'http://www.ullaeyewear.com', None, 'http://www.lunaacupunctureaz.com', 'http://dignityvethospital.com', 'http://bodywaxinglasvegas.com', None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpwestorganic', 'http://www.legermainhotels.com/en/torontomls/', 'http://www.realcoolhvac.com/', 'http://www.meckafitness.com', 'http://vitalitybowls.com/locations/las-vegas/', 'http://www.southtrailhyundai.com', None, 'http://www.midas.com/pittsburgh/store.aspx?shopnum=6365&dmanum=688', 'http://www.ttc.ca', 'http://grainger.com/', 'http://www.thelakeshorespa.com', None, None, 'http://www.laurentianlanes.com', 'http://www.Park2300.com', 'https://classicwaterheaters.com', None, None, 'http://carpetcleanertoronto.ca', 'http://www.charlesalexanderdistribution.com', 'https://www.planetfitness.com/gyms/monroe-nc?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.thnclinic.com', 'http://www.rosenails.ca', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'http://www.jetluxuryresorts.com', 'http://www.arwright.ca', 'http://www.tugawaytowing.com/home.html', 'https://www.ttc.ca/Subway/Stations/Ossington/station.jsp#Schedule_', 'https://tempestoysterbar.com', None, 'http://www.locksmithsmadison.com', 'http://spicensabzi.us', 'http://www.moviesoncentral.com', 'http://www.jamisymonssalon.com/', None, 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.saloncercone.com', None, 'http://eggsquis.com/fr/restaurants.php?restaurant=lasalle', 'https://www.montecarlo.com/en/restaurants/brand-steakhouse.html', None, None, None, 'http://www.carwashcleveland.com', 'http://sophiainsapphire.com', 'http://russospizza.net/', 'http://www.secondcup.com', 'http://edeninlove.com', 'http://www.rogersrepairservice.com/', 'http://doubletopping.com', 'http://www.charlotteautogroup.com', None, 'http://www.rebelbook.com', 'https://www.anylabtestnow.com/franchises/huntersville-28078', 'http://www.eatnpark.com', 'http://www.doubledoorinn.com', 'http://robertostacoshop.com', 'http://www.spirithalloween.com', 'http://www.marriott.com/hotels/hotel-rooms/yyzmd-courtyard-toronto-mississauga-west/', 'http://www.dunkindonuts.com', 'http://clevelandsteamercleaner.com', None, 'http://oaklandhallinn.com/', 'http://www.starbucks.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.startrekexp.com', None, 'http://us60selfstorage.com', None, None, 'http://www.jensontotalservices.com', 'https://www.olivegarden.com/locations/nv/las-vegas/las-vegas-cheyenne-ave/1363?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:las-vegas-nv-1363_pl:locurl_rd:1274', '1592165677', 'http://305kustomzdesigns.com', 'http://www.arthurmurrayscottsdale.com/', '1592165689', None, 'http://southwestsmilesdental.com/', 'http://www.prestigemusicacademy.com', None, None, None, None, 'https://www.wontonking.com', None, 'http://www.dunkindonuts.com', None, None, 'http://www.notecollection.com', None, None, 'http://rajafoods.ca', 'http://www.phoenixalterations.com', None, 'http://www.insurewithmelissa.com?cmpid=d3u7_blm_0006', 'http://www.cafe-mirage.com', 'http://livingyogacenter.net/', 'http://deserttintlasvegas.com', 'http://www.restoreandreplenish.com', 'http://www.palaciosupholstery.com', 'http://elisabat.com/', 'http://www.yogurttimedt.com', 'http://cityproperty.com', 'http://www.Ricosacai.com', 'http://www.alternativepaths.org', 'http://deerfootmeadowsoptometry.com/seton-optometry-location-and-hours/', None, 'https://thefaceshop.ca/en/pacific-mall-center-32-32-32-32/', 'http://www.silverminesubs.com/', 'http://seasaltbar.com', None, None, 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', None, 'http://www.menchies.com/frozen-yogurt-shops/frozen-yogurt--northland-plaza-ab', 'http://www.fosdalbakery.com', 'http://www.BombHairVegas.com', '1592165799', None, '1592165804', None, 'http://www.justroughinit.com', None, 'https://www.hazukiphotography.com', None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://style-encorenorthhills.com', 'http://matchacafemaikoclt.com', 'http://Hydra-tattoo.com', 'http://www.sunautoservice.com/auto-repair/nv/las-vegas/las-vegas-tropicana-52', None, 'http://www.chura-hair.com', 'http://www.spoiledchics.com', 'http://www.docwalters.com', 'http://www.sushionbloor.com/', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.litehouse.com', '1592165859', 'http://www.specializedflooringaz.com/', None, None, None, 'http://www.crywolfclothing.com', 'http://www.disantoimmigrationlaw.com', None, 'http://www.rossstores.com', 'http://www.popuppizzalv.com', 'http://www.thomaseyecare.org', None, 'https://budgetblinds.com/DowntownPittsburgh/?utm_source=yelp&utm_medium=naf-listing&utm_campaign=NAF-LOCAL-YELP', None, 'https://www.hankinsplasticsurgery.com', None, None, None, 'http://www.spirithalloween.com', 'http://www.heersmanagement.com/properties/property/desert-point-apartments', 'https://www.hilton.com/en/hotels/cltephf-hilton-charlotte-executive-park/?SEO_id=YELP-HI-CLTEPHF', 'https://www.foodandthingsliquoranddeli.com', 'https://www.hairclub.com/locations/on/toronto/hair-loss-clinic-242/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_242', 'http://tubbystubs.com', None, None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://www.dipsidoo.com', 'https://www.annaksalon.com', 'https://www.republicservices.com?utm_source=yelp.com&utm_medium=referral&utm_campaign=Yelp_Profile_Click&utm_term=a7viRDrdChmaTJUqBxg4EQ', 'http://www.urbanairgoodyear.com', 'http://ajsalterations.com', 'http://www.nevadacleanup.com', 'http://qdoba.com', 'http://www.purplepenguinsnow.com/', 'https://www.vietbeautysupply.com', 'http://www.nashvillenorth.ca/', None, None, None, 'http://www.lauerrealtygroup.com', 'http://tpmlasvegas.com', 'http://www.clerebar.com', None, None, None, 'https://www.elizabethtatelaw.com', 'http://www.elegancehaircare.com', None, None, 'http://yourspacesalons.ca/tonyc', 'http://www.greatclips.com/salons/3065', None, 'http://www.cherokeehillsgolf.com/restaurant', 'http://www.housetohomeinspection.com/', 'http://prairiefirechampaign.com/', None, None, 'http://cervantesrestaurant.com/', 'https://www.doanrestoration.com', 'http://www.tjbakerconst.com', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.lasvegasartificialgrass.com/', None, None, '1592166030', None, None, 'http://www.sonobello.com/locations/cleveland/?utm_source=yelp&utm_medium=site&utm_campaign=yelp_loc_page_url', None, None, None, None, None, 'https://www.kitchenstuffplus.com/storelocator/#kspST020', 'http://punjabibynature.ca/home.html', 'https://www.charlottefoot.com', '1592166055', 'http://www.kentlanes.com', 'http://www.pghtacotruck.com', 'http://www.tiltonlmt.wixsite.com/mysite', 'http://www.ghadirmeatmarket.com', 'http://www.starbucks.com/store/15887/', 'http://www.pizzanova.com', None, 'http://actionjacksfurniture.com/', 'http://taylorandcolt.com/commerce-court-toronto/', None, 'http://www.blindstogo.com/en/stores/east-mississauga?ua=L4X+1M1&rds=25', '1592166085', 'http://www.menchies.ca/frozen-yogurt-shops/frozen-yogurt-creekside-crossing-ab', 'https://www.searshomeservices.com/locations/appliance-repair/nv-las-vegas/4000-meadows-lane', 'http://www.sundeviltrophy.com/', None, '1592166100', 'http://www.lakeshorepediatric.com', None, 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'http://www.popularlube.ca', 'https://www.southhillspetresort.net', None, 'http://www.plumbperformingartscenter.com/', 'https://www.vaqueroscarneasada.com', 'http://www.giantlasvegas.com', None, 'http://www.richsrestaurant.com/', 'https://riosecco.com', None, None, 'https://www.ritasfranchises.com/Brodheadsville', 'http://www.drivingtoindependence.com/', 'http://www.piazzamanna.com', 'http://www.lcbo.com', 'http://www.wxdx.com/main.html', 'http://thebelsize.pub', 'http://www.bannerhealth.com/locations/gilbert/banner-physical-therapy-gateway?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', 'http://www.caromonthealth.org/Locations/Hospitals/CaroMont-Regional-Medical-Center.aspx', 'http://tigerlily.com', 'https://www.mielesitalian.com', None, 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', None, 'http://www.peaceofmindandbodymassage.com', 'https://www.eatnpark.com', 'http://scarletdoorcafe.com', 'https://stores.maccosmetics.com/us/nv/las-vegas/3200-s-las-vegas-blvd', 'http://www.dominos.com', 'http://www.vinylwraptoronto.com', 'http://www.brillorestoration.com', None, 'http://www.dunkindonuts.com', None, '1592166203', 'https://www.ediblearrangements.com', 'http://www.ecosmartpros.com/', 'http://www.accurateautoglass.net', 'http://jennysalterations.wordpress.com/', 'http://www.whiskersandpawsvet.com', 'http://happyfeetnailsspa.com', 'http://www.stonefamilyservices.com', None, 'http://www.animalhospitalatgrayhawk.com', 'http://www.restaurantsawadee.com/', 'http://toronto.thepint.ca', None, 'http://cutie-giftshop.business.site', 'http://signaturebarbershop.net', 'http://cuyahogalibrary.org/', 'https://www.wellsfargo.com', None, None, 'http://mrniceguygames.wpengine.com/contact-us/', 'http://www.dunkindonuts.com', 'http://fiveguys.com/', 'http://www.papajohns.com/', None, 'http://www.sonicdrivein.com/', 'http://602autosports.com/', 'http://www.salonescapeaz.com', 'http://www.rocklobster.com.php53-12.ord1-1.websitetestlink.com/ossington/', 'https://www.4wheelparts.com/stores/north-carolina/4wp-charlotte-nc-54', 'https://www.petsmart.com/stores/us/az/phoenix-store0131.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://www.daango.com', 'http://restaurantspago.com', None, None, 'https://www.amr.net/home/lasvegas', 'http://www.hennashoppe.com', None, 'http://www.lickshomeburgers.com', 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'https://www.pizza73.com', 'http://www.cigna.com/cmgaz/locations/stapley-multi-specialty-center', 'http://tempeimportsaz.com', 'http://www.restaurant-divino.com/accueil', None, 'http://www.big5sportinggoods.com', None, 'http://www.idoupdolasvegas.com', 'http://www.westlibertyanimalhospital.com', None, None, 'http://www.anteloelectric.com', 'http://www.happyhomeservicesaz.com', 'https://www.upmc.com/locations/community/south-side', None, None, 'http://www.newhighend.com', None, 'http://www.Visionworks.com', 'http://patio-pleasures.com/', None, 'https://www.unlv.edu/donnabeamgallery', 'http://www.virtualworldmadison.com', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', None, None, 'http://desertwestobgyn.com', 'https://www.twistedmindssmokeshop.com', 'http://www.PalmLane.ca', None, 'https://www.sportchek.ca', None, 'http://www.mezzaninehair.com', 'https://www.fatfreddyscatering.com', 'http://www.asepestandweedsupplies.com', 'https://www.ihg.com/holidayinnexpress/hotels/us/en/rantoul/cmirt/hoteldetail', None, None, 'http://www.babassupperclub.com', 'http://www.champagnestrands.com', 'http://www.boulevardvegas.com', '1592166417', 'http://bloomcigar.com', 'http://extensionroom.com', 'https://www.fascinations.net', 'http://www.theskullzsalon.com', None, 'http://www.indianpunjabibazaar.ca', 'https://www.bobevans.com/our-restaurants/locations/22?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', None, 'http://bestasianmassagelv.com', 'https://locations.davidstea.com/ab-calgary-111', 'http://www.papajohns.com', None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEWI', 'http://www.vivintsmarthomesecuritycalgary.com', None, 'https://www.terravita.com/neighborhood', None, '1592166467', None, 'http://www.kelseys.ca/', 'http://ago.ca/ago-bistro', '1592166475', None, 'http://www.cbcautism.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'https://www.circlek.com/store-locator/us/tolleson/11450-w-southern/2701130', None, 'http://www.simpleaz.com', None, '1592166498', 'https://scottsdaleearnoseandthroat.com/about-us/meet-our-team/cheryl-a-mackechnie-md', '1592166505', 'http://www.lvcabs.com', None, 'https://hpautorepair.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FPHXPC', 'https://www.purebarre.com/location/san-tan-village-az', 'http://www.canadapost.ca/cpotools/apps/fpo/personal/findPostOfficeDetailPrint?outletId=0000104239', 'http://www.onlinetickets.com/', 'http://www.cwpress.com/', 'http://thesaucebse.com', 'http://www.caferio.com/', None, None, 'http://www.heldtlumber.com', '1592166545', 'https://www.afw.com/stores/az/glendale', 'http://www.tenniswerx.com', 'https://www.publicstorage.com/north-carolina/self-storage-waxhaw-nc/28173-self-storage/2073?pid=wyelp&CID=1245&CHID=AFL', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', 'http://www.purcelltire.com/retail/retail-locations.aspx?store=85', None, None, 'http://www.envynailspaclt.com', None, 'http://www.organicgarage.com', 'http://abbottcourtesyplumbing.com', None, 'http://www.americanrealtylv.com', 'http://www.angelosohio.com/', 'http://www.sipcoffeeandbeer.com', None, 'http://www.osteriausa.com', None, 'http://www.homegoods.com', 'http://www.vegas-asian-massage.com/', 'http://www.cristinahindslaw.com/', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.shangrilabanquet.com', 'http://japanesespecialties.blog.fc2.com/', 'http://www.gordonbiersch.com', 'http://www.awildhairaz.com', 'http://www.switchbladesalon.com', 'https://www.dairyqueen.com/us-en/locator/Detail/?store-id=5175&localechange=1', 'http://www.theloreleiboutique.com', 'http://www.chandleraz.gov/tumbleweed', 'http://www.pizzahut.com/', None, 'http://www.harrisonlawaz.com', 'https://www.menswearhouse.com/store-locator/2513?utm_source=yelp&utm_medium=listing&utm_content=2513&utm_campaign=AZ', 'http://palacenaillounge.net', None, 'http://www.falconlandscapes.com', 'http://www.trueharmonywellness.com', 'http://lodgecharlotte.com', None, 'http://www.moonflightcoffee.com', 'http://subamanautocare.com', 'http://www.theotheroomlv.com', None, None, 'https://tannertileandstone.com', 'http://rebathandkitchens.com', 'https://www.target.com/sl/mccandless/1218', None, 'https://jamboree-bagel.business.site', None, None, 'https://www.arrowheadccaz.com', None, 'http://floydsbarbershop.net/', 'http://www.livemagnoliaterrace.com', None, 'http://www.oasisspasalon.com', 'https://www.homedepot.com/l/Bethel-Park/PA/Bethel-Park/15102/4176?cm_mmc=seo-yelp&', 'http://www.platosclosetcuyahogafalls.com', 'http://www.quesada.ca', 'http://www.sdsdental.com', 'http://www.ipdiet.com', 'http://www.NEwomensclinic.com', None, None, None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700044280216620&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.customcarconcepts.net', None, 'http://www.fabricboutique.com/', 'http://stores.save-a-lot.com/crafton-pa', None, 'http://www.acu-painclinic.com/index.html', 'https://mollymaid.com/southwestern-charlotte-fort-mill-rock-hill', None, 'http://99only.com/store/Mesa---Main/', 'http://www.regaleaz.com/', 'http://www.centennialcollege.ca', 'http://www.carinobistro.ca', 'https://www.ridgeatrobinson.com', 'https://www.integramedical.com', 'https://local.fedex.com/oh/fairlawn/office-1229/', 'http://www.timhortons.com/', 'http://www.graftmadison.com/', 'http://www.arcadiafamilyclinic.com', 'http://www.bayequityhomeloans.com/gilbert', None, None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=12197&localechange=1', None, 'http://www.islandpethospital.com', None, None, 'http://www.marshallscanada.ca', 'http://www.tzatziki.ca', None, 'https://www.thejoint.com/arizona/mesa/red-mountain-48026', 'http://www.callcoolaid.com/', None, 'http://www.hoerner.ca', 'http://www.greatlakesoutdoorsupply.com', 'http://www.thefatbelgian.com/', 'http://thailandspamassage.com', None, 'http://www.surgicalweightcontrolcenter.com', None, 'http://www.sonoranorthotrauma.com', None, None, 'http://www.vcahospitals.com/elliot-park/?utm_source=yelp.com&utm_medium=referral&utm_term=VCA_Elliot_Park_Animal_Hospital&utm_campaign=main_url', 'http://www.eyebrowsrus.com', 'http://www.herriotts.com', 'http://www.longsontherapy.com', None, None, 'http://www.ardene.com', None, None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.dominicspizzamedina.com', 'http://atrautoglass.com', None, 'http://livingwellhomes.com/apartments/details/escondido_apartments', 'http://www.swisschalet.com', 'http://justincookhomes.com', 'http://octanecf.com/', 'http://www.theladysilvia.com/', 'http://www.whitfillnursery.com/', None, None, 'http://theodorewilliamslaw.com/', 'https://upscalemixologist.com', None, 'http://fancynailsaz.com', 'http://www.vivalasguestlist.com', 'http://www.dwr.com/category/find+a+studio/scottsdale.do?mobile=off', None, 'http://gilbert.donutworryaz.com', 'http://greenleafeinncharlotte.com', 'http://www.ahsappliance.com/', None, 'https://www.villageeastcleaners.com', None, None, 'http://info.lululemon.com/stores/us/las-vegas/fashionshowmall', 'http://www.involtini.ca/', 'http://www.blackforestbooksandtoys.com/', 'http://drjimshipley.com', None, 'http://www.arizonaorganic.com/', 'http://www.jasontheplumberllc.com', None, 'http://www.benfranklinplumbingcharlotte.com', 'http://www.aznypdpizza.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://Geostacos.com', 'https://www.thingsremembered.com', 'http://bikepirates.com/', 'http://manhattanpizza4lv.com', 'http://www.greatclips.com/salons/9261', None, 'http://www.divinedoggies.ca', 'http://www.desertdove.com', 'http://www.LocalMotionPGH.com', 'http://www.shopchristophertodd.com', 'http://www.umamipoke.com', '1592167022', 'http://www.marathon.com', 'http://www.timhortons.com/', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.premiercoachworks.com', None, 'http://swigs.ca', 'https://globalgifts.ca', None, 'http://www.sfxphx.org', None, 'https://www.napaonline.com', None, '1592167056', 'http://directpcc.com/', 'https://www.cricketwireless.com', 'http://www.gotaticket.net', None, None, None, 'http://www.rentalstempe.com', None, 'http://www.costco.ca/warehouse-locations/vaughan-woodbridge-on-547.html', 'http://www.momosushilasvegas.com', 'http://threeamigoscharlotte.com', 'http://www.sincitycomputerservices.com', 'https://www.petahead.ca', None, 'http://www.acropoliscg.com', 'https://www.interdent.com/gentle-dental/locations/az/phoenix/702-e-bell-rd-dentist-office/', 'http://www.foundery.is/', 'http://waxhawdentalcenter.com', 'http://www.trainwithgabe.com', 'http://laplanchagrills.com', None, 'http://eastwestblinds.com/?utm_source=Yelp&utm_medium=Yelp%20CPC&utm_campaign=Yelp%20East%20West%20Blinds', 'http://bespokeinn.com/', 'http://www.dunkindonuts.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.craigrd.com/', None, 'http://www.cash4chaos.com', None, None, 'https://www.autozone.com/locations/az/scottsdale/7401-e-mcdowell-rd.html', 'http://www.acebartending.com', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'http://newbeginningscounselingcenters.com', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://stockmanos.com', None, None, 'http://www.blingtour.com', 'http://charlotteswim.com/', 'http://www.chiro-phoenix.com', None, 'http://alterios.webs.com/', 'http://azteccaraudio.com', 'http://www.arizonaveterinaryoncology.com', 'http://www.robertcoomergroup.com', None, 'http://www.portofsubs.com', 'http://gastondentalassociates.com/', 'http://www.aapglv.com', 'http://www.practiceofpositivity.com', 'https://hirehometeam.com', 'https://www.kohls.com/stores/nv/henderson-667.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.hillsideappliancerepair.com', 'http://www.lorain.lib.oh.us', 'http://www.dunkindonuts.com', None, 'http://www.Konbeasushi.com', 'https://www.walmart.com/store/1598/phoenix-az/whats-new', 'http://www.agaveyaguacate.blogspot.com/', 'http://www.periodentalimplant.com', 'http://www.nandos.ca/restaurants/calgary-17th-coming-soon', 'https://www.signaturestyle.com/locations/on/richmond-hill/first-choice-haircutters-51417.html', 'http://aproposcreations.com/', 'http://www.theshopprinting.com', 'http://fritesstreet.com/', 'http://www.cyanimalclinic.com', 'http://mrsouvlaki.ca/en/locations/ontario/hudson%E2%80%99s-bay-centre.html', None, 'http://local.buckle.com/OH/WESTLAKE/419/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', None, 'http://www.starbucks.com/store/9334/', None, 'http://rebeltoronto.com', 'http://mikasgreek.com', None, 'http://www.premiumgranite.com', '1592167276', 'http://pouloslawfirm.com', None, None, None, 'https://andersondentalaz.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.devicepitstop.com', 'http://www.keungsrestaurant.com', None, 'https://www.chickasawdemolition.com', None, 'http://washmeohio.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://Lasajspa.com', 'http://transnationalmoving.com', None, 'http://www.pinkberry.com', 'http://www.paradiseflowermarket.net', None, None, 'http://spamontst-hilaire.com', 'http://www.aamco.com/centerinfo.asp?centerNum=4128569500', None, 'http://www.banfield.com/veterinarians/nc/cornelius/coe?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', None, None, 'http://www.wbt.com', 'http://tanakasmartialartslasvegas.com', None, 'http://www.arrowprofessionals.com', 'http://www.princetonclubxpress.net/', 'http://www.parkersdowntown.com', '1592167562', None, None, None, None, 'http://www.hibachigrillphoenix.com', None, 'https://www.kaltire.com/en/stores/locations/?StoreID=694&source=yelp_track', '1592167582', 'http://www.theartshoppe.com', 'http://www.stjohnsmusic.com/locations/calgary', None, None, None, 'http://www.spanishtrailcc.com', None, 'https://www.kruyerdental.com', None, 'http://www.fellinislv.com', '1592167611', 'http://sincitybrewtours.com', None, '1592167616', None, 'http://www.caseysdrafthouse.com', None, 'http://www.tradesecrets.ca/', None, 'http://otrocafe.com', 'http://www.lvnailspanv.com', 'http://www.letseat.at/paparazzipizzeria#home', None, None, 'http://lapaloma.ca/about-us/', '1592167648', 'http://www.healthyplanetcanada.com', None, None, None, 'https://www.aplaceformom.com/assisted-living/arizona/phoenix', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', '1592167664', 'http://www.Kaizensushi.ca', None, 'http://helpinghandhandymanservice.com/handyman-services/', None, 'http://www.furniturebank.org', 'http://www.verizonwireless.com/', 'http://www.buddhabodyyoga.ca', None, None, None, 'https://www.rodiziogrill.com/henderson-nv/', 'http://www.sunridgemazda.com', None, 'http://www.breastforbaby.com', 'http://www.bowlingdynamics.com/', 'https://www.americanrealtyacademy.com', 'https://www.ashleycourt-apartments.com', None, 'http://www.lasvegashairandmakeup.com', None, None, 'http://www.campbowwow.com/summerlin', 'https://centennialhillsrealty.managebuilding.com/Resident/PublicPages/Home.aspx', 'https://www.anytimefitness.com/gyms/2194/Cottage-Grove-OR-97424/?utm_source=yelp&utm_medium=local', None, 'http://www.glickdental.com/', 'http://Lawdawgtrailtours.com', 'http://www.citymarketdeli.com', None, 'http://www.regalnails.com', 'http://www.tipitytoenails.com', 'https://www.foodforthought.ca/contact-us/markham', 'http://www.salonlola.net', None, None, 'https://www.publicstorage.com/arizona/self-storage-phoenix-az/85027-self-storage/1113?pid=wyelp&CID=1245&CHID=AFL', None, None, None, 'https://bridalxoxo.com', 'http://www.statewidelightinglasvegas.com', 'https://www.enterprise.com/en/car-rental/locations/us/oh/mentor-3921.html?mcid=iyp:8368400', 'http://www.cozinedental.com', 'http://www.lessermeansmore.com', 'http://www.drinkbambu.com', None, 'http://www.villaserenahenderson.com', None, '1592167790', 'http://www.sevencalgary.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.hilton.com/en/hotels/cltsohw-homewood-suites-charlotte-southpark/?SEO_id=YELP-HW-CLTSOHW', None, 'https://www.menswearhouse.com/store-locator/2503?utm_source=yelp&utm_medium=listing&utm_content=2503&utm_campaign=AZ', 'http://www.eastwestbank.com', 'https://stores.godiva.com/us/nv/las-vegas/3200-las-vegas-blvd-s.html', None, 'http://www.fracturedprune.com/location_charlotte.php', 'http://www.arizonapoolpros.com', 'http://www.kleinhenzjewelers.com', None, 'http://www.eastvalleypetdoors.com', 'http://www.chatpattachowk.ca', None, 'https://www.bostondentalgroup.com/locations/happy-dental-west-valley/', 'http://www.dasonii.com', 'http://gourmetschnitzelhouse.com/', 'https://www.kohls.com/stores/pa/monroeville-289.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.parasitetesting.com', 'http://www.picnicfood.ca', 'http://counseling.healingcouples.org', 'http://www.ssrtowing.com/', None, '1592167861', 'http://www.Vegasjj.com', 'http://www.jerianchiropractic.com', None, 'https://www.sprint.com/storefronts/bd/sprint-chandler-az-85226-fcs-1172/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.starbucks.com', 'http://fitnesstogether.com/waxhaw?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://www.carefreecoverings.com/home', None, 'https://www.sunsetgrill.ca', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://arizonareptilecenter.co/', 'http://www.tastidlite.com/', None, None, 'http://www.victoriassecret.com', 'http://www.arrowoodmed.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.indiascafeandkitchen.com', None, 'http://www.dollartree.com', 'http://www.soshandyman.ca', 'http://www.greencleanphoenix.com/', None, None, None, 'http://paninisgrill.com/locations/euclid', None, 'http://www.cellxperts.ca', 'http://www.mamarobertos.com', 'http://www.subaruofmaple.com/', '1592167950', 'http://www.aaronvargaphotography.com', 'http://modernvascular.com/modern-vascular-of-glendale/', None, '1592167966', 'http://www.yourserenity.ca', 'http://fiveguys.ca', 'http://www.eddiebauer.com', 'http://cosmonailsmesa.com', 'http://www.christieclinic.com', None, 'http://vacationownershipconsultants.com', 'http://www.mcdonalds.com', 'http://www.wardenandsons.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.affirmhartman.com', 'http://rdnplumbing.net', 'http://www.allegromusicaz.com', 'http://www.bevsbeadz.com', 'http://www.phatpackshow.com/', None, 'http://www.bdcdental.ca', 'http://www.sweetnote.ca', 'https://4tenrealestate.com', 'https://www.painteddesertgc.com', 'http://www.marinoswickliffe.com/', 'http://www.delimarkcafes.com/477mtpleasant.html', None, 'https://www.r2smile.com', 'http://www.yelp.com/events/charlotte-hops-and-shops-2', '1592168035', 'http://www.rentcoldair.com/', None, 'http://7keystohealthinc.com/contact-us/', None, 'http://www.massageaddict.ca/mississaugameadowvale', 'http://www.stripper101.com/', None, None, 'http://www.quiktrip.com', 'http://pageperpage.com', 'http://www.stephencooley.com', '1592168063', 'http://www.torontopubliclibrary.ca/detail.jsp?R=LIB060&Entt=RDMLIB060', 'http://tradition.market', 'http://www.thepeacepipe.net', 'http://www.lasvegassuites.com/', None, None, None, '1592168082', 'https://locations.theupsstore.com/pa/pittsburgh/4885-mc-knight-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.hoopershealth.com', 'https://arizona.weidner.com/apartments/az/phoenix/roosevelt-row/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'http://www.centuryelectricllc.com', None, 'http://www.vieweyecare.com/', None, None, None, 'http://www.carusogourmetpizza.com/', 'https://locations.wendys.com/united-states/az/goodyear/1178-n-litchfield-rd', None, None, 'http://www.massimodutti.com/ca', 'http://www.restaurantrouge.com/', 'http://www.thesportspub.com/', 'http://www.ektaxandaccounting.com', 'https://locations.vitaminshoppe.com/az/scottsdale/vitamins-supplements-scottsdale-az-571.html', 'http://www.AgaveTreeAndLandscape.com', 'https://www.marks.com/en/stores/rexdale-etobicoke-55-woodbine-downs-blvd.html', None, 'https://www.thearrogantbutcher.com', 'http://www.adenearthworks.com', None, None, None, None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', 'http://www.roserunkennel.com', 'https://www.thebso.com/locations/on/aurora/beauty-supply-outlet-haircuts-11847.html', 'http://www.allfireduplv.com', 'http://www.visitingangels.com/AZ', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3075+West+Liberty+Avenue-Dormont-PA-15216/storeid=4144?cid=ps_ylp_2020', 'http://www.conns.com/store-locator/north-carolina/charlotte-28273', None, None, None, None, 'http://www.superstarcarwashaz.com/w_services2.php', 'http://www.goldstockjewelers.com', None, 'https://www.uhaul.com/Locations/Self-Storage-near-Litchfield-Park-AZ-85340/721037/', 'https://locations.theupsstore.com/nv/henderson/631-n-stephanie-st?utm_source=Yext&utm_medium=organic&utm_campaign=Listings', None, 'http://eastvalleypulmonary.com', 'https://www.gucci.com/us/en/store/7014-east-camelback-road?utm_source=yelp_us&utm_medium=storemarketing&utm_campaign=retail_storepages_marcomm&utm_content=23016_bookapp', None, 'https://www.ritasice.com', None, 'http://www.joinrmxprofessionals.com', None, None, 'http://bartlegibson.com', 'http://www.progressivehome.com', 'http://www.elpolloloco.com/locations/store/3855', 'http://www.problindrepair.com', 'http://stacyneal.wvmbmesa.com', 'http://www.hyattcarcompany.com/', None, 'http://www.thebeehivegifts.com/', None, None, 'http://luckyboyburgershopphoenix.bestcafes.online', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Chandler-AZ-85226/922030/', '1592168214', 'http://jwmechanical.com', 'http://www.dogdaystempe.com', None, 'http://www.venetian.com', 'http://villageclubs.com', None, None, 'https://www.titlemax.com/store/phoenix-e-warner-az-41-title-loans/?utm_source=yelp&utm_medium=listing&utm_campaign=places&tmx_id=771c0159ac754cf&sf_traffic=paid_listing_yelp', 'https://risasdental.com/find-a-location/tempe/?utm_campaign=LocationPage&utm_medium=TempeYelpWebsite&utm_source=Yelp', 'https://www.robinsplumbing.com', None, 'http://www.kierlandcigars.doodlekit.com/', 'http://www.rtthomegroup.com', None, 'http://www.samifruits.com', None, 'http://www.zeppes.com', 'http://www.southshoresah.com', '1592168256', 'http://www.meetfreshcanada.com', 'http://www.chandlerfamilycare.com', 'http://www.dadanddaughterphotography.com', 'https://www.themattressmania.com', None, 'http://uberadventures.net', 'http://megatirecalgary.com', None, 'https://stayatthei.com', None, 'http://www.harborfreight.com', '1592168284', None, 'https://runpigrun.ca', None, 'http://pranamanana.com', 'http://www.isscogenerator.com/main/', None, 'http://www.runplusfun.com', '1592168301', 'http://www.bowlology.com', 'https://www.ulta.com/stores/henderson-nv-1191', 'http://chase.com', '1592168311', 'http://www.floorsforless.com', None, 'http://www.3030dundaswest.com', 'http://www.krankensigns.com/', 'http://www.laurellake.com', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', 'http://www.graceaz.com', 'https://www.roostersmgc.com/locations/nc/huntersville/torrence-village-haircuts-13194.html', 'http://www.greatclips.com/', 'http://omegamassage.ca/', None, 'http://www.wiler.org', None, 'http://azrattlers.com/', 'http://hawksgym.com', None, None, None, '1592168350', 'http://xaviermortimer.com', 'http://www.teejaysbrampton.com', 'http://leesliquorlv.com/locations/the-crane-bar', 'http://portofinocrs.com', 'http://www.mississauga.ca/', 'https://www.foresthillortho.com', None, 'http://my.clevelandclinic.org/departments/eye', '1592168367', 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', None, 'http://www.bk.com', 'http://www.urbanhomeandgarden.com', 'http://N8tiveArts.com', 'https://www.publicstorage.com/south-carolina/self-storage-lake-wylie-sc/29710-self-storage/2566?pid=wyelp&CID=1245&CHID=AFL', 'http://www.valleyairworks.com', None, 'https://www.vagaro.com/nailsbyyulie', 'http://www.schaffers.com/', None, 'https://www.eaglerider.com/phoenix', None, 'https://www.SearsHomeServices.com/locations/appliance-repair/nv-henderson/1245-w-warm-springs-rd', 'https://www.wellsfargo.com', 'http://www.makeupbykyra.com', 'http://encorecatering.com/', 'http://www.cbdemporiumaz.com', '1592168410', 'http://pawntnt.com', 'http://baywoodclinic.com', None, None, 'http://www.corvobartoronto.com', 'http://www.spicersprinklerworks.com/', None, 'http://www.expertnailandspa.com', 'http://cleaningservicecharlotte.net', None, 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', None, None, None, 'https://www.pods.com/locations/united-states/nevada/las-vegas', 'http://www.blackmountainortho.com', 'https://advancedvisioninstitute.net', 'http://christophersfinedrycleaning.ca', None, 'http://www.canvasgallery.ca', 'http://browart23rosspark.com', None, 'https://local.safeway.com/safeway/az/phoenix/810-e-glendale-ave.html', 'http://www.epiclandscapelv.com', 'http://local.buckle.com/nv/las-vegas/457/', 'http://italianoasispizzeria.com', 'http://www.lmca.org/about/lmcafe.cfm', None, 'http://www.flutedmushroom.com', None, 'http://cleaningwonders.com', 'https://www.msisurfaces.com/phoenix-countertop-showroom-distribution/?utm_source=yelp&utm_medium=cpc&utm_campaign=yelp_monthly', 'http://www.memphisstationbar.com/', 'http://www.berdenas.com', 'http://www.tacobell.com', None, None, 'http://www.jacobsonbrosdeli.com/', 'http://www.dornellas.com/', None, 'http://www.bikebike.ca', None, 'http://www.beautybrowsalon.com', 'http://www.theupsstore.ca/60', 'http://www.azmistsystems.com', 'http://www.ajsspamillennium.com/', None, 'http://www.dsw.com', 'http://www.elitefamilydental.com', None, 'http://solita.ca', None, 'http://www.hugoboss.com', 'https://horsebacktrailriding.com', None, 'http://www.cpk.com/location/details/Chandler', None, 'http://www.salonlevel6.com', 'http://www.twainandswensonstorage.com', None, None, None, None, 'http://www.fanniemay.com/', 'http://tavernedelaferme.ca', None, None, 'http://www.essentialskinaz.com', 'http://www.newcreationssalon.com', 'https://www.nevadalegalforms.com', 'http://www.masterhardwarepa.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.theinnersleeve.com', '1592168571', 'http://www.serviceking.com/locations/service-king-falconis?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'https://cinemark.com/nevada/century-16-santa-fe-station-and-xd', 'http://www.metrogreenscape.com', 'http://mkautomotive.com', 'http://www.teuschertoronto.com', None, None, None, None, 'http://www.dininginaz.com', 'http://sc-clinic.com/clinic/spadina-and-bloor/', None, None, 'http://www.maxumconstruction.net', 'https://www.walmart.com/store/6586/concord-nc/whats-new', 'http://www.smartpointresearch.com', 'https://www.cfshops.com/toronto-eaton-centre.html', 'http://www.elmnt.ca', 'http://dubaijewellers.ca', 'http://www.pizzabreak.ca/', 'http://labahiaasadosymariscoslasvegas.cafesonly.online', None, None, None, None, '1592168629', 'http://www.precisiontune.com/ConcordNC', 'http://beyerapplianceservice.com', 'http://www.completemarine.com', 'https://www.food4less.com/', None, None, None, 'http://sundevilpool.com', '1592168649', 'https://www.handandstoneballantyne.com', 'https://blackbeardiner.com/location/glendale/', 'http://fastlanepizza.com', 'http://www.arizonasrealty.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, None, None, None, 'http://k9-games.com', None, 'http://www.carolinashealthcare.org/charlotte-medical-clinic', 'http://www.gmurrayphoto.com', 'http://stcharlesonline.org', 'http://www.classiccakesaz.com', 'http://www.tiptopautocollision.com', None, 'http://www.fresh-burger.com/', 'http://www.lakesidesportschiro.com', 'http://flobarlounge.com', None, None, 'http://www.fat-tuesday.com', 'http://www.stowintegrityauto.com', 'http://experiencetheblackout.com/', 'http://www.charlotteappliancerepairmen.com', None, '1592168709', 'http://www.redtailgolfclub.com', 'https://www.balancinghormonesnaturally.com', 'http://www.emb-plus.com', 'http://famouslaffa.com/', 'http://stores.sportsauthority.com/az/phoenix/sporting-goods-phoenix-692.html', 'https://cigarsdaily.com', 'http://www.centralphoenixanimalhospital.com', 'https://lvfamilyvision.com', None, 'http://www.cuttingedgecalgary.com', 'http://www.rentaffordable.com', 'http://www.sunnydayslearningcenter.com', 'http://www.oasisspringsbuilders.com', 'http://www.surf-ski.com/', 'http://www.kwakballet.com', 'http://www.setmefree.ca', 'http://cashcanada.com/pawn-shop-location/cash-canada-forest-lawn', 'http://universalenroll.dhs.gov', 'http://www.kfc.com', None, 'http://www.haircutie.com', 'http://www.mcdonalds.com', 'http://frugalmuse.com', None, 'https://www.snaderlawgroup.com', 'http://www.acehardware.com/store-details/15917', None, 'http://www.stlouiswings.com', 'https://www.abuelos.com/restaurants/chandler/', None, 'http://www.shenanigansbarandgrille.com', None, 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', None, 'https://www.anytimefitness.com/gyms/3912/Litchfield-Park-AZ-85340/?utm_source=yelp&utm_medium=local', None, 'http://wynnfitness.com/location/mississauga', None, 'http://newvisionpropertymanagement.com/', 'http://www.islanddentalcenter.com', 'http://www.hootersarizona.com', 'https://www.circlek.com/store-locator/us/scottsdale/20721-n-scottsdale-rd/2705915', None, 'https://lernerandrowe.com/?utm_source=Yelp', None, 'http://www.murrayhillfoods.com', '1592168804', None, 'https://locations.panerabread.com/il/aurora/1304-fox-valley-center.html', 'http://Therapeuticretreat.Com/', 'http://www.instantbeautyspa.com', None, 'http://www.nest.to/', 'http://alexblackgolf.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.verizonwireless.com/stores/ohio/streetsboro/streetsboro-357953/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.dollarama.com', 'http://www.gustafsonlawoffice.com', 'https://www.core704.com', 'https://dealer.sprint.com/experts-choice', None, 'http://www.dragonscandy.com', None, 'http://www.suncitysummerlinflorist.com/?srccode=yelp_track', 'http://www.smoknpig.com/', 'http://www.rahrawrahpetfood.com', None, None, 'https://arizona.weidner.com/apartments/az/phoenix/trailside-at-hermosa-pointe-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'https://dentistryofthecarolinas.com/locations/monroe/', None, 'http://www.timothys.ca', 'https://www.nofrills.ca/store-locator/?utm_campaign=Loblaws&storeId=1384&utm_medium=LPM&utm_source=G', None, 'http://www.lu-az.com', 'https://www.circlek.com/store-locator/us/glendale/5848-w-olive-ave/2706657', 'https://tahdesign.com', None, None, 'https://www.alphagraphics.com/centers/charlotte-north-carolina-us287.html', None, None, None, 'http://www.sabas.com', None, None, 'http://www.picknsave.com', 'http://locations.harristeeter.com/nc/harrisburg/92/', '1592168889', 'http://all-tranz.com', None, 'http://www1.shoppersdrugmart.ca/en/store-locator/store/359', 'http://thefrogandcrown.com', 'http://popeyeschicken.ca/', 'http://www.muchoburrito.com', 'http://www.starbucks.com/store/17296/', 'http://www.stratospherehotel.com/american_superstars.html', 'http://www.foodlion.com', None, 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', 'http://www.hayashihibachi.com', 'http://www.aboutfaceaz.com', None, 'http://www.pizzanova.com', 'http://www.westowntire.com', '1592168925', 'https://www.potterybarn.com', 'http://www.burritoboyz.ca', 'http://www.oldnavy.com', 'https://firstcu.net', None, None, '1592168939', 'http://www.dontortaco.com', None, 'http://astoriacafemarket.com', 'http://downtownbrewfestival.com', None, 'http://www.papajohns.com', 'https://el-pollito-charro.business.site', None, 'http://www.asiaaz.com', 'https://www.currys.com', 'http://www.discount-drugmart.com', None, None, 'https://www.alamo.com/', 'https://www.coffeeatrio.com', 'https://www.studioemphotography.com/photographer-fairmont-scottsdale-princess', None, None, 'http://www.louiesnoodle.com', None, None, 'http://desertcovenursingcenter.com', 'http://parkviewmanor.ca', None, 'https://www.lumberliquidators.com/ll/stores/az/phoenix/2120-s-7th-street?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1019', None, None, 'http://www.chinapearl1.com/home', 'http://amoving.company', None, 'http://www.macgyverautobody.ca', None, 'https://local.fedex.com/oh/north-olmsted/office-1234/', None, 'http://www.poochesnpalsinc.com', 'http://www.hmsmokeshop.com', 'http://www.bankofamerica.com', 'http://www.a2bmovers.com', 'https://www.casareynapgh.com', None, 'http://www.smilecreationsaz.com', 'https://curaleaf.com/stores/nj', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.rounduplakecampground.com/', 'https://www.sunlifemaids.com', 'https://www.chevronwithtechron.com', 'http://sugarshacksweets.net/', None, 'http://www.stuffingtonbear.com/', None, 'http://www.myflightsurgeon.com', None, 'http://www.stavrossportsbar.com', 'http://www.teffngreens.ca', 'http://www.modern.yoga', 'http://www.fillsgas.com', '1592169057', 'https://local.fedex.com/pa/pittsburgh/office-2064/', 'http://www.scoopnyc.com', 'http://ourjourneytraining.com', None, 'https://www.camelbackautoglass.com', 'http://www.tempespeedandperformance.com', None, 'http://www.spartantrainingchandler.com/', 'http://www.tbirdmed.com', None, None, 'https://www.petsuppliesplus.com/Store/OH/Elyria/Elyria/123?utm_source=Yelp&utm_medium=Store-Page%C3%8A', 'http://www.enzositalianmarket.com', 'http://www.drlonlawrenz.com', 'https://www.anylabtestnow.com/franchises/las-vegas-89146/', 'https://stores.jcrew.com/en/south-hills-village', 'http://coachzinger.com', 'http://www.footsensation.ca/', 'http://www.assistbooks.com', None, 'http://www.goodlifefitness.com/locations/ontario/toronto/kingliberty?source=yelp-websitelink-torontoking-gl', 'http://www.mcdonalds.com', 'http://www.adore2.com', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://transcanadacarrental.com', 'http://www.berryvinegifts.com', 'http://www.thewellnesscenter.org', 'http://www.lux-spa.com', 'http://www.shopkawaii.com/', '1592169123', 'http://www.playitagainsportsmadison.com', None, 'http://www.audichandler.com/index.htm', None, 'https://www.peets.com', None, None, 'http://www.hollandresidential.com/talise', None, 'http://www.insurrectionaleworks.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.domesticblissdesign.com', 'http://taniahairbraiding.com/', None, 'http://www.graceinlasvegas.com', 'http://www.damcomputermedic.com', 'http://lokacupunctureclinic.com', None, 'http://www.artistsplay.com/hub-cafe', None, None, 'http://www.thrifty.com/?campid=DS:YL_THR_20150109_HP', None, 'https://www.wingstop.com/location/wingstop-421-phoenix-az-85015/menu', None, 'http://clickserve.dartsearch.net/link/click?lid=43700006430792986&ds_s_kwgid=58700000444978510&ds_url_v=2', 'http://cogburnsbar.com', None, 'http://www.kidscoop.org/', 'http://sangrock.com', 'http://www.expertconcretecoatingsinc.com', None, 'http://www.braizenfoodtruck.com/wp1/', 'http://www.drraykim.com/', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.cjsitalianice.com', None, 'http://www.awinadvancedautobody.ca', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Calgary-AB-T2H1L6/840051/', 'http://www.doddslaw.com', 'http://www.stacyspitstopbbq.com', 'http://www.hbotofaz.org', 'http://www.flowershoppetempe.com', 'http://www.moderationsuit.com', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.montzauto.com/', None, '1592169237', 'http://www.foodkingmenu.com', None, 'https://cortland.com/apartments/cortland-whitehall/', None, 'http://tiffanynailshairscottsdale.com/', 'http://www.chocolatsfavoris.com', 'http://www.yogadigs.com/', None, 'http://www.estheticaglenmore.com/index.php', 'http://www.richmondhilltoyota.com/', None, 'http://www.advant-edgecurbing.com', 'http://www.juicestandard.com', None, 'http://nlasvegasvets.com', 'https://footsolutions.com/peoria/', None, 'http://www.themanorspa.ca/', 'http://www.intownsuites.com/locations/charlotte-northeast-chn/', None, None, None, '1592169289', 'http://www.allearzsavannahs.com/', 'http://www.CoronadoHeights.com', 'http://www.aimeecaballero.com', 'http://www.pinklotusmenu.com', 'http://www.myscrubsandmore.com', 'http://www.parmatireservice.net', None, 'http://myvillaroma.com/', 'https://boulderoaksca.com', 'http://www.livingstoninnmadison.com', '1592169315', 'http://www.duffsfamouswings.ca', 'https://www.sourceforsports.ca/en-CA/Stores/3-Adrenalin-Calgary-Source-For-Sports.aspx', None, 'https://budgetbicyclectr.com/used-bicycle-showroom-museum', 'http://arizonagoldendoodles.com', 'http://www.einsteinbros.com', None, 'http://chickenfootestatesales.com', 'http://www.joyli.net', 'http://stuartdayguitars.com', 'http://www.foodlion.com', 'http://www.mellowbentertainment.com', 'http://avenueb-pgh.com', 'http://www.superelectric.tv/pinball', 'http://www.backonthebar.com', 'http://www.dickssportinggoods.com/', None, None, None, 'https://www.6ixcycle.com/contact-us/', '1592169360', 'http://azcountertop.com', 'http://www.turnberrytowers.com', 'http://www.liftedtrucks.com', 'http://elwoodmotorsportsandcollision.com', 'https://www.wellsfargo.com', 'https://www.pier1.com/on/demandware.store/Sites-pier1_us-Site/default/LocalStore?storeId=1243', None, 'http://www.tntmobiledetail.com', 'http://mariscoensenada.com', 'http://www.rachelsyoungatart.com', None, 'http://www.outback.com/locations/az/gilbert', None, 'http://redmodernfurniture.com', 'http://www.savorcafecharlotte.com', 'https://www.finessecateringandevents.com', 'https://www.petpeoplestores.com/rocky-river-store.html', 'https://www.gregbryandds.com', 'https://www.searsclean.com/pittsburgh-pa', None, 'http://www.cobsbread.com/home/', 'http://www.mcnallyauto.com', None, '1592169413', 'https://palmbeachtan.com/locations/NC/Indian-Trail-Old-Monroe-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=AUG', 'http://www.monroevilleconventioncenter.com', 'http://www.moes.com', None, 'https://www.umcsn.com', 'https://www.chuckecheese.com/?utm_source=yelp.com&utm_medium=referral&utm_campaign=Branded_Profile_CTA&utm_content=Branded_Profile_CTA_399', 'http://www.markhambowl.com', 'http://www.maidpro.com/henderson-nv', 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=119&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-119', 'http://www.hairrehabaz.com', 'https://abraauto.com/abra-auto-location/1051-charlotte-nc-albemarle-road', None, 'http://www.gingerflowerstudio.com/', 'http://domenicspub.com/', 'http://www.angelpark.com/-ninos-kitchen', 'http://www.redonionlounge.com', None, 'https://introboutique.com', 'https://www.faema.ca', 'http://johnnyvegasgrill.com/', 'http://www.bakingatitsbest.com', None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Ocotillo.aspx', None, 'http://www.thedistrictatscottsdale.com', 'https://www.marriott.com/spas/lasjw-jw-marriott-las-vegas-resort-and-spa/spa-aquae/5013971/home-page.mi', None, None, 'http://www.chopstickfoods.com', 'http://www.denimanddiamondsmesaaz.com', 'http://www.azenergy.com/', None, 'http://motorlessmotionbicycles.com', None, 'http://Centurionmgmt.com', 'http://www.crossfitwestvalley.com', '1592169495', 'http://www.oldnavy.com', 'http://www.palaroos.com', '1592169503', 'https://www.libertytax.com/income-tax-preparation-locations/19248.html', 'http://www.timhortons.com', 'http://www.gusbrown.com', None, '1592169513', 'http://www.citypharmacylv.com', 'http://lm2investments.com', 'http://www.silverstatesoccerleague.org', None, None, 'http://www.xlntshutters.com', None, None, 'http://www.tomaydo.com/', None, None, 'https://www.gallusdetox.com', 'http://www.laveenalterations.com', None, 'https://www.redrobin.com', 'http://elcortezhotelcasino.com/', 'http://www.fasttracksolar.com/', 'http://www.maricopaac.com', 'http://www.pagesbooks.ca', 'http://www.shaneonealphotography.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://sites.google.com/publikfire.com/publikfire', None, 'http://www.patatemallette.com/fr', 'http://palmvalleyrehabilitation.com/?utm_source=yelp&utm_medium=organic', 'http://criticalaz.com', 'http://sweetfrogyogurt.com/locations/store/tarentum-pa', 'http://www.kmart.com/stores/nevada/las-vegas/3680.html', 'http://www.snugasabug.com/store', 'http://www.luplv.com', 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-bethany-home?utm_source=yelp&utm_medium=local&utm_campaign=Honorhealth_Medical_Group_Bethany_Home', 'http://www.greatclips.com', 'http://www.toysrus.com/family/index.jsp?camp=MISC:yelp:store:page:tru:111910&categoryId=2255956', '1592169583', None, 'http://www.cherrybombcoffee.ca', None, None, 'http://www.courtneyvista.com', 'http://lisasean.com', 'https://www3.hilton.com/en/hotels/nevada/elara-by-hilton-grand-vacations-center-strip-LASCSGV/index.html?SEO_id=YELP-GV-LASCSGV', 'https://shopused.com', 'http://swingshiftauto.com', 'http://nail-salons.dapeem.com/', None, None, 'http://www.lcbo.com', 'https://www.chick-fil-a.com/lee', 'http://www.starbucks.com/store/9298/', 'http://www.sidestogobbq.com/bbq-location/cleveland-hopkins-international-airport', 'http://www.itsjustlunch.com/', 'http://onestopnutrition.com', None, None, None, 'http://www.jamminjerkvegas.com', 'https://www.lifestorage.com/storage-units/north-carolina/charlotte/28270/109-near-matthews/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NC-109&utm_content=website-link&howheard=IYE', 'https://www.marriott.com/hotels/hotel-information/restaurant/yyzda-delta-hotels-toronto-airport-and-conference-centre', None, 'http://www.dragonpearl.ca', None, 'https://www.gianteagle.com/Stores/Store/?storeId=8%2F', None, None, None, None, 'http://www.adampinksalon.com', 'https://essencevegas.com/locations/las-vegas-strip/', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.blushlasvegas.com', 'http://www.autoglassfactoryaz.com', 'http://www.womensinternational.com', 'https://www.advancedmotionchiropractic.com', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.blastedbarley.com', 'http://www.cvcclub.com', 'http://www.falafelcafecleveland.com', 'http://www.munails.com', 'https://www.gotenda.com', None, 'http://www.coffeerushcafe.com', 'http://www.azdeq.gov/EmissionsTest', 'http://www.RedesignUpholstery.com', None, 'http://www.olbrichbiergarten.com', 'http://doubletree3.hilton.com/en/hotels/ontario/doubletree-by-hilton-hotel-toronto-downtown-YTOCSDT/index.html?SEO_id=YELP-DT-YTOCSDT', 'http://www.northshorecommercialdoor.com', 'http://www.thewhiningpig.com', None, 'http://www.eastmeatswesthalal.com/', None, 'https://www.speedy.com/shops/toronto-danforth/en/', 'http://sandiegonailsphoenix.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, 'http://www.ingleside.com', 'https://soslawnmaintenance.com', None, None, 'http://www.deltamechanical.com', 'https://www.wyndhamhotels.com/hotel/02493?tel=18002521709&iata=00093785&cid=fe:ra:20161215:yl:pp:ramus:02493', '1592169733', 'https://www.speedway.com', 'http://www.gianteagle.com', 'http://www.theredzonemadison.com/', 'https://azcdps.com/locations/paradise-valley/', 'http://www.kitchenland-lv.com', 'http://www.squirrelhilloptometrist.com', 'https://stepuptowellnessnow.com', 'http://www.WestonTeamMortgageLoans.com', 'http://www.dreamlanes.com', '1592169756', None, 'http://www.eatfromatruck.com', None, 'http://www.coffeebytes.net', 'http://tahoepower.net/', None, 'http://www.cafedepariscanada.com', 'http://www.mecklenburgbridal.com', None, 'https://www.lasvegashomes.vegas', 'http://www.DrThompsonAZ.com', None, 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', 'https://www.ups.com/ca/en/help-center/contact.page', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.capstonerealtypros.com', 'https://ad.doubleclick.net/ddm/clk/459148250;262878939;b', None, None, None, 'https://www.clubpilates.com/tempe/', 'http://www.starbucks.com/', None, 'http://www.7-eleven.com/?yelp=29384', 'http://www.tribeamericaleathers.com', 'http://www.bravobridals.com', 'http://www.ipolished.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.vegasonerealty.com', 'https://procellmotorsports.com', None, 'http://www.subzimandi.ca', None, None, None, 'http://www.starbucks.com', 'http://theflatsatgranitereef.com', 'https://www.budsglassjoint.com', 'http://www.city-tavern.com', None, 'http://www.onceuponachildstow.com/', 'http://www.sonoranlifestyle.com/', 'http://auraspacalgary.com/', 'http://tfwestrella.com', 'http://www.lasvegaspaininstitutes.com', 'https://locations.monro.com/oh/fairlawn/111-ghent-road?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', None, 'http://www.doctorsofhair.com/', None, '1592169853', 'https://www.purebarre.com/location/central-phoenix-az', 'http://goodnbr.com', 'http://www.warholdmechanicals.com/', None, 'http://gmhairdesigncolorspecialist.com', 'http://www.michaeljohnsoncpa.com', 'http://robertsidell.com', 'http://www.lakeeriecrushers.com', None, 'https://www.wellsfargo.com', None, 'https://local.fedex.com/nc/charlotte/office-0746/', 'http://www.mrsub.ca', '1592169882', None, None, 'http://www.tempetrophy.com', 'http://www.pvlaserlipo.com/', 'https://www.quicknclean.net', 'https://www.signaturestyle.com/locations/on/toronto/first-choice-haircutters-5677.html', 'http://www.proficientclean.com', 'http://www.einsteinbros.com', 'https://www.kurevapes.com', None, 'https://unlvmedicine.org/family-medicine', 'http://www.7-eleven.com/?yelp=32265', None, 'https://theeggworks.com', 'http://www.aamcophoenixcentral.com', 'http://www.somacafe.com', 'http://www.ronherman.com', 'http://www.culligantotalwater.com', 'http://www.theweaveshop.com/north-carolina-charlotte/', 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', 'http://www.starbucks.ca', 'https://www.bluevaportech.com/', 'http://www.missbrow.com/', 'http://gqcutslasvegas.com', 'http://lasvegas.carenow.com/location/ann-simmons', 'http://allure4me.com', None, None, None, 'http://snvscooters.com', 'http://www.bownessauto.com', None, 'http://cowbridgefurniture.com', 'https://www.staples.com', 'http://www.youngbeautifulsalonspa.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://grilladestorino.com/fr/grillades-torino-place-vertu/', 'https://candy-cravers-coffee.business.site', 'https://www.xpo.com', 'https://www.supercuts.com/locations/oh/lakewood/lakewood-city-center-haircuts-83162.html', 'http://www.carters.com', 'https://arrowheadcoin.com', None, 'http://bytecle.com', 'http://www.bombayschutney.com', 'http://www.bookparkngo.com/index.php?main_page=charlotte', None, 'http://www.familydollar.com', 'http://thedetroiterlakewood.com', 'http://www.crownandanchorlv.com/', 'http://www.papajohns.com/', 'http://www.keffervw.com', 'https://www.choicehotels.com/arizona/scottsdale/comfort-suites-hotels/az816/rates', 'http://www.nardisfinedining.com', 'http://balilaffa.com', 'http://www.cabarruseye.com', 'https://stores.advanceautoparts.com/az/scottsdale/15475-northsight-blvd?utm_content=listing-2017-12-21&utm_medium=local&utm_source=yext&utm_campaign=aap', 'http://www.studiojewelerswi.com', 'http://www.mootowncreamery.com', None, 'https://locations.vitaminshoppe.com/oh/strongsville/vitamins-supplements-strongsville-oh-854.html', 'https://6ixtips.ca', 'http://www.pickeringhonda.com/', 'http://www.ericwelchpainting.com', 'http://www.harborfreight.com', None, 'http://www.childsplayaz.org', 'https://www.beallsplumbing.com', 'http://www.tatesac.com', None, None, 'http://www.roiimprovementsllc.com/', None, None, 'https://nevadaoralandfacialsurgery.com', 'https://www.barrhouseonmain.com', 'http://www.bankandbaronpub.com/', '1592170054', 'http://bluediamondlv.com', None, 'http://www.jfdayspa.com', 'http://www.pizzanova.com', 'https://www.thegalleryapthomes.com', 'http://www.elyriafence.com', 'http://www.bakerykaffeehaus.com', 'http://www.drteddysim.com/about-dr-teddy-sim/', 'http://www.gopropoolservice.com/', 'http://www.redressnyc.com/', 'http://charmeck.org/mecklenburg/county/ParkandRec/Parks/ParksByRegion/NorthRegion/Pages/Jetton.aspx', 'http://www.kbhome.com/new-homes-phoenix', 'http://www.freshbonsainails.ca', None, '1592170086', 'https://www.ediblearrangements.com/stores/2875-W-Ray-Rd-Chandler-AZ-85224', None, None, None, None, 'https://www.prosourcewholesale.com/showrooms/oh-prosource-of-cleveland', 'http://www.merrymaids.com', 'https://www.purchasegreen.com/locations/henderson-nv/', None, 'http://the-toy-box.com', None, None, None, None, 'http://www.buckle.com/', 'http://www.nostalgic-furniture.com', '1592170122', None, 'http://www.maximes.ca', 'http://www.allanimalsrescue.com', 'http://www.streetsvilleflorist.com/?srccode=yelp_track', 'http://www.roadrunnerexpresscarwash.com', 'http://www.cafemia.com', None, None, None, None, 'http://Tshsalon.simdif.com', None, None, 'http://www.rossstores.com', None, 'http://www.dirtyfoodto.com', None, None, 'https://www.bellissimafashions.com', None, 'http://www.dollarama.com/', None, None, 'http://www.azdentalheights.com', 'https://www.discounttire.com/store/az/mesa/s/1294?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', 'http://clickserve.dartsearch.net/link/click?lid=43700006430627338&ds_s_kwgid=58700000443429188&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.urbancrawfishstation.com', 'http://www.foothillsrehab.com/', 'http://www.holidayinn.com/hotels/us/en/brampton/ybmca/hoteldetail', 'http://www.eatkickshaw.com/index.php', 'http://www.hannahscaramelapples.com', 'http://center-for-growth.com', 'http://www.lux.salon', None, 'http://robertostacoshop.com', 'http://wagnwash.com/locations/phoenix-az/', 'http://www.thepoolmagician.com', 'http://pierrefonds.carstar.ca', None, 'http://www.riosantafeapts.com/?campaign_code=yelp_Rio_Santa_Fe_AZ_85323', 'http://www.arrowheadmartialarts.com', 'https://www.houseofhandsome.com', None, 'http://kristapsons.com', 'http://www.oldcreditbrewing.com', 'http://www.applehillanimalhospital.com/', 'http://www.ortconline.com', 'http://www.dasgasthaus.ca/', 'http://www.campthunderbird.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.34182.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:34182', None, 'http://www.starbucks.com/store/9298/', None, None, None, 'http://www.cronometro.com', None, 'http://www.salonmichaels.com', 'http://www.burlingtoncoatfactory.com/store-details-Store-213--5959-West-Sahara-Avenue--Las-Vegas--NV--89146-3126.aspx?id=2d282a9a-6b10-4437-bb42-7194af655b83&from=89146', 'http://www.fracosandfroccs.com/', 'http://luchitashudson.com', None, None, 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', '1592170265', 'http://www.hm.com', None, 'http://cranberry.sagahibachi.com', 'http://www.tortoisesupply.com', 'http://www.dominos.com', 'http://www.anchorsuprescuegroup.org', 'http://www.lovemycolon.com', None, 'https://grayhawkgolf.com/dining/the-morning-joint', 'http://etherealnails.com', '1592170292', 'https://www.niudanoodles.com', 'http://www.southwestrainbow.com', None, 'http://ad.doubleclick.net/ddm/clk/283420830;110240480;l', 'http://www.foodtruckcaravan.com', 'http://www.portraitinnovations.com/location/143/OH/Mayfield%20Heights/44124/Mayfield-Town-Center', None, 'http://www.hudsonoverheaddoors.com', 'http://www.meltingpot.com/greenvalley', None, 'http://aguamentiaz.com', None, None, 'http://www.rue21.com', '1592170325', 'http://www.valleyvistapoolcare.com', 'http://www.spirale.net/', 'http://fuegobistro.com/fuegotaco/', 'http://www.liasian.com', 'http://DeconResponseteam.com', None, 'http://www.gurtnerdental.com', 'http://www.timhortons.com/', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', 'https://www.oakandfort.ca/', 'http://jcls.jazzercise.com/facility/fountain-hills-jazzercise-center-14600', None, 'https://www.marriott.com/hotels/hotel-information/restaurant/cltad-ac-hotel-charlotte-city-center/', 'http://www.dvmcvet.vetsuite.com', None, 'http://plumbingsource.net', 'http://www.middleburgheights.com/?q=rentals_community_center', 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', None, 'http://www.sparkletime.net', None, 'http://www.playitagainsports.com', 'http://www.hrcleveland.com/tparrot.php', 'http://www.desertstorage.com/las-vegas-nv-boat-rv-storage', '1592170380', 'http://www.nakedcitylv.com', None, 'http://www.bathandbodyworks.com', None, 'http://www.hoipolloicafe.com', None, 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'http://www.plazahotelcasino.com', None, None, 'http://www.thespiritbox.com', 'https://www.patagoniaelements.ca/store/content/10/Calgary/', '1592170409', None, None, 'http://www.countrytireautomotive.ca', 'https://www.royaldistributing.com', 'https://www.presidentialpools.com/gilbert-swimming-pool-builder', 'http://www.gingersdesserts.com', None, None, None, 'http://www.vegasairductcleaning.com', None, 'http://www.betterlifemobility.com', 'http://www.azoral.com/', 'https://www.bellacinosofstow.com', '1592170442', 'https://totalimageoakville.biz', 'http://www.dayonepizzadelivery.com/', None, None, 'http://desotocentralmarket.com', None, 'http://www.eyedoctorchandleraz.com', None, 'http://eosfitness.com/location/pecos?utm_campaign=vegas-pecos&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, '1592170464', None, None, 'https://stores.jcpenney.com/pa/pittsburgh/clothing-stores-pittsburgh-pa-2263.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2263', None, None, 'http://www.thepamperedpinkie.com/', '1592170483', None, None, 'http://www.flightcentre.ca/', 'http://www.marshallsonline.com', '1592170491', 'https://www.gilbertaz.gov/departments/parks-and-recreation/parks-facility-rentals/parks-info-ramada-rentals/parks/crossroads-park', 'http://www.pair.com', '1592170499', 'https://organicnaillounge.business.site', None, None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.mathnasium.com/northwestlasvegas', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.eddietaxplus.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=3317&localechange=1', 'http://www.heroburgers.com', 'https://www1.shoppersdrugmart.ca', 'http://www.sackstierney.com', 'http://www.beastmodeonline.com', 'https://www.waxmans-carpet.com', 'http://myazinjury.com', 'http://www.mrappliance.com/east-valley', None, 'https://www.aritzia.com/en/store?StoreID=aritzia-bloor-flagship-toronto', 'http://www.northcalgarypediatricdentistry.com', 'https://skylineliquorstores.com', None, None, 'https://womensfitnessclubs.com/ajax/?utm_source=yelp&utm_medium=search&utm_campaign=yelpcampaign', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.siennalv.com', 'http://www.tomohibachiandsushi.com/', 'http://www.marriott.com/phxrn', None, None, 'http://www.avis.ca/', None, 'http://www.barriquesmarket.com', 'http://www.spirithalloween.com/', None, 'http://www.bigbenbarber.com', 'http://www.andoaz.com', 'http://www.klassykidsacademylv.com/', 'http://www.freddysusa.com', 'http://repweddings.com', None, None, 'https://www.voguerecoverycenter.com', '1592170593', None, None, 'https://www.planetfitness.com/gyms/streetsboro-oh?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', None, 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=13080', None, 'https://hawkesclinic.com', None, 'https://www.johnnyscleveland.com', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Sun%20Prairie/WI/3289', 'http://libertyshawarma.com', None, 'http://www.altatempe.com', None, 'https://angel-dentalcare.com', 'http://www.pvcglendale.com', 'http://www.letspartyconsultants.com', 'https://www.wyndhamhotels.com/hotel/53380?tel=18003876085&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:53380', None, 'http://www.v-dubfurniture.com', 'http://www.sovpreschool.org', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.physio-logic.com', None, 'http://www.buca.ca/king.html', 'https://www.100thbgrestaurant.com', 'http://www.arizonaeyes.net', '1592170657', None, 'http://www.dollarama.com/', None, 'http://www.foxspizzamtwashington.com', None, 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', None, None, None, 'http://www.GoodByeDebt.com', 'http://www.starbucks.ca', 'http://www.icecastlearena.com', 'http://www.bestmatt.com/', None, 'http://www.fix8fitnesscafe.com', None, 'http://proem.org', None, None, None, 'http://www.sojoblau.com/', None, 'http://www.canyonstateedu.com', 'http://www.weavebaratl.com', 'http://www.assemblychefshall.com', None, 'https://dealer.sprint.com/preferred-wireless/find-a-store/67331-bridgeville-pa-15017', None, 'https://www.hydroplumbinginc.com', 'http://www.johnwallickjewelers.com', 'http://www.roastednutfactory.ca', 'http://www.hottopic.com/', 'http://www.azulorganicskincare.com', None, None, 'http://www.joyestes.net', 'http://www.jambajuice.com', None, 'http://www.traditionalelectric.ca', 'http://www.hatchyoga.com', None, 'http://www.harbsauto.com', 'http://www.TheKyotoBowl.com', 'http://www.sunnyviewdental.ca/georgetown', 'http://www.inkbombaz.com', 'http://locations.traderjoes.com/oh/woodmere/676/', 'https://www.starbucks.ca/store-locator/store/1017580/bayview-village-2901-bayview-avenue-toronto-on-m-2-k-1-e-9-ca', None, None, 'http://www.bulkbarn.ca/', None, None, 'https://www.supercuts.com/locations/az/surprise/albertsons-shopping-center-haircuts-82334.html', None, 'http://www.applewoodstorage.com', None, '1592170796', None, 'https://theprimerestaurantphx.wordpress.com', None, 'http://fitlifefusion.com', 'http://pazzoscatering.com', 'http://kaffirlimeindonesiancuisine.blogspot.ca', 'http://www.comicoasis.com', None, 'http://www.bigbztexasbbq.com', 'http://www.mrservice.ca', 'http://perennial-yoga.com', 'http://www.lovenspoonfuls.net', 'https://www.davidwmartinlaw.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.kireitoronto.com/', 'https://www.directcarpetone.com', 'http://www.bestratereferrals.com', None, '1592170850', 'http://www.dimpflmeierbakery.com', None, 'http://fabulashclt.com/', 'https://www.georgesflowers.net/?srccode=yelp_track', None, 'https://mintdentallasvegas.com', 'http://www.summerlink.com/wp-content/uploads/2010/03/2010ParkMap', None, 'http://www.southpittsburghvet.com', None, 'http://www.hpsc.ca', 'http://atlanticdentalphoenix.com/', 'http://www.middletonsports.com', 'http://www.californiathai.ca', 'http://www.albertaparks.ca/fish-creek.aspx', None, 'http://summitrestorationlv.com', 'http://serviceprowashandlube.com', 'http://bobcoffeebar.com', 'https://www.lhmfordmesa.com', 'http://www.dailyfreshgrillandcafe.com', 'http://www.houseofblues.com/lasvegas/crossroads', 'http://www.owensautorepair.com', None, None, None, 'http://www.halisangels.com', '1592170925', 'http://www.elixirspa.ca', 'http://www.phillyssportsgrill.com/', 'http://www.charlottecomedytheater.com/', 'http://www.dutildenim.com', 'http://www.treasureisland.com/restaurants/khotan.aspx', 'http://painstopclinics.com/pain-clinic-uptown/', None, 'https://www.modlash.net', None, None, 'http://www.cledogwalk.com', 'http://drivetime.com', '1592170963', 'http://www.bankofamerica.com', None, 'http://www.skyloft.com', 'http://nodayoga.com/', '1592170978', 'http://www.boosterjuice.com', 'http://www.letusrollyouone.com', None, 'https://www.anytimefitness.com/gyms/1069/Pineville-LA-71360/', 'https://iflycu.com', 'http://www.ysl.com/gb', 'http://www.southsidemotorspgh.com', 'https://www.ribeirojiujitsu.com/las-vegas-nevada.html', 'https://www.wellsfargo.com', None, 'https://medmen.com/stores/tempe-airport-phoenix-sky-harbor', None, None, 'http://www.allisonlovesfinejewelry.com', None, 'https://www.uhaul.com/Locations/Truck-rentals-near-Champaign-IL-61820/760043/', 'http://www.ojoslocos.com', 'http://beitshofarot.com', 'http://www.key.com', 'http://www.rainbowchiropracticcenter.com', 'http://www.rivenailandskincarespa.com', 'https://www.pancheros.com/locations/fitchburg', 'https://www.smilesquadlv.com', 'http://www.southfacese.com', 'https://www.bankofamerica.com', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=977', None, '1592171054', None, 'http://Rentmyrims.com', '1592172433', 'http://www.nevadaspca.org', None, 'http://extremecleantoday.ca/', None, 'http://www.runningroom.com', 'http://www.primarypetcare.com', 'http://www.extremeautocustoms.com', 'http://www.drivingforce.ca', 'http://www.clarkbrands.com', 'http://www.ilovemichaels.com/', 'https://hansenfamilyplumbing.com', 'https://www.ductz.com/north-phoenix-deer-valley/', 'http://www.klosekniturbana.com/', 'http://purebarre.com', None, 'http://renewedbeautyspecialists.com/', 'https://elevationchurch.org/locations/blakeney/', 'http://www.goodlifefitness.com/', 'https://www.selfstoragesolutions.com/self-storage/az/gilbert/85297/421?utm_source=yelp&utm_medium=cpc&utm_campaign=Gilbert', None, None, None, None, 'http://www.drainproplumbing.com', 'http://www.Charleys.com', 'http://www.mtsystems.ca/', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.olivegarden.com/locations/az/scottsdale/scottsdale-e-cactus-rd/1234?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:scottsdale-az-1234_pl:locurl_rd:1175', None, 'http://www.woofswigglesnwags.com', None, None, None, 'http://www.lasvegasnannies.com', 'http://www.sixthcitycycles.com', 'https://fancypetsaz.com', 'http://haymakersportsgrill.com', 'http://Elegantentrysarizona.com', 'http://www.avalonmeatcandy.com', None, 'http://www.theargusrealestatecompany.com', '1592172552', 'http://www.pizzasola.com/eastside.html', 'http://www.arizonabartenders.com', None, None, None, 'http://www.morningstarstorage.com/location/308/sc/fort-mill/carowinds/?ref=yelp', 'http://www.dynamitemontessori.com/', None, 'http://aandbroofing.net', 'https://www.kenganleynissan.com', 'http://www.janthonysalonspa.com/', 'http://rawlicious.ca/canada/locations/markham', '1592172590', 'http://www.illinistor-it.com', None, 'http://www.truevalue.com/mcfarlandtruevalue', None, None, None, None, 'https://www.brakesplus.com', 'https://www.scottsdaleaz.gov/parks/scottsdale-civic-center', 'http://www.athenaswindows.com', 'http://fishkinggrill.com', 'https://locations.davidstea.com/ab-calgary-111', 'http://www.salaradesserts.com', None, None, 'http://www.belle-image.ca/', None, None, 'http://bootlegsmokehouse.com', 'https://www.jasonsdeli.com/restaurants/maryland-pkwy-deli', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', '1592172650', 'http://gotorickys.com', 'http://www.morrowparkcityapts.com', 'http://epicentrenc.com', 'http://www.marketonmacleod.com', '1592172664', None, 'http://subsecreto.com', 'http://www.Barking-Dogs.com', 'http://www.punjabitadka.ca/', None, None, 'http://southeastcrabfeast.com/', 'https://www.tanrepublic.com/locations/nevada/las-vegas/maryland-parkway', 'http://www.mattressfirm.com/', 'http://www.greathousesportsgrill.com', 'http://www.thaihouse.us/', 'https://www.aircanada.com/us/en/aco/home.html', 'http://www.substationii.com', 'http://www.greatwallchinese-lasvegas.com', 'https://www.childhelp.org/advocacy-centers/childrens-center-of-arizona/', None, 'http://www.therefinerycc.com', 'http://onsitetinter.com', 'http://harvestcompassioncenter.org', None, None, None, 'http://www.ohstudiolv.com', 'https://sgplaw.com', 'http://www.failteirishpub.ca/', None, 'https://stores.loft.com/outlet/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'http://www.paradisebakery.com/', 'http://www.osigym.com', None, None, 'https://www.uniquenailbargateway101.com', 'https://www.quarryparkchiropractic.com', 'http://www.easternfootspa.ca', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://paintingwithatwist.com/avon/', 'http://www.urepmaids.com', 'http://www.entebella.com', None, None, 'https://www.springboardrecovery.com', None, 'http://www.pekingexpress.ca', None, None, 'http://www.red9ine.com', 'http://swift-moving.com', 'http://www.frontgate.com/Frontgate/US/CustomerService/Scottsdale/landing-path', 'http://www.greatexpressions.com/dental-offices/ohio-offices/severance', None, None, 'https://www.ulta.com/stores/macedonia-oh-1004', None, None, 'https://www.sitesmartmarketing.com', '1592172812', 'https://www.hilton.com/en/hotels/phxsrhx-hampton-suites-phoenix-surprise/?SEO_id=YELP-HP-PHXSRHX', 'http://patriciangrill.com/', 'http://www.1800waterdamage.com/phoenix-az', None, None, 'http://www.theelranchomotel.com', 'http://falbobrospizza.com', None, 'http://www.noahsnaturalfoods.ca', 'http://vitalshipmed.com', 'https://www.walgreens.com/locator/walgreens-4900+library+rd-bethel+park-pa-15102/id=11296', None, 'http://www.aboutadog.ca/', None, 'http://www.palacestation.com/entertainment/jacks/?utm_term=PS&utm_content=PS-PROPERTIES&utm_medium=SOCIAL&utm_source=YELP&utm_campaign=PS-YELP', None, 'http://www.telaverdeapts.net', 'http://www.skinpossible.ca', None, 'https://www.fordonfifth.com', None, None, 'http://www.gamestop.com', None, 'http://www.rentawrecktoronto.com', 'http://www.fitworks.com/rockyriver', 'http://www.calderawealth.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://extendedstayhotels.com', 'http://jc101services.com', 'http://www.palladinoscleaning.com/', 'https://www.haydenparkapartmenthomes.com', 'http://www.cyclepathnorthyork.com/', 'http://www.starbucks.com/store/69863/', None, None, 'https://www.visioncenterslv.com', None, 'http://makeupmagic.net', None, 'http://therapeuticmethod.net', '1592172922', 'http://www.tlc-airconditioning.com', None, 'https://urbanlivestockvet.com', None, 'https://www.nvisioncenters.com/locations/las-vegas/nevada-eye-care-east/?utm_source=directory&utm_medium=yelp', 'https://www.spectrumdermatology.com', 'https://www.intownsuites.com/extended-stay-locations/arizona/phoenix/west-2/', 'https://beautyenhancementsbyd.com', 'http://www.veneziabakery.ca', None, None, 'http://www.dentscharlotte.com', None, 'http://www.pethavenaz.com', None, 'http://exclusivelytankless.com', None, None, 'http://insurance-agency.amfam.com/AZ/tryna-hoerger-owens/default.aspx?sourceid=agp0001015578', 'http://laterna.ca/', None, 'http://affordableframingaz.com', None, 'http://blackcablimo.wix.com/blackcab', '1592172980', 'http://www.newfoodorder.com', 'http://lolasgelato.ca', 'http://www.tonibulloni.com/', None, None, None, 'http://www.rebath.com/location/lasvegas', 'http://www.acuhealings.com/', 'http://viapanini.ca', None, 'https://www.wyndhamhotels.com/hotel/14408?tel=18008899735&iata=00065402&cid=fe:hj:20170523:yl:pp:hjus:14408', None, None, 'http://ccsunscreens.com/', 'http://tazikiscafe.com/location/charlotte', '1592173013', 'http://www.effortlessmobileauto.com/servicesPhoenix/servicesPhoenix.html', 'http://fukuburger.squarespace.com', None, 'http://www.empowerpt.com', None, 'http://khalsamontessori.org/', 'http://www.mayfieldnissan.com/', 'http://quintealias.com', 'http://www.starbucks.com/store/9229/', None, 'http://www.fatburger.com/home/', 'http://cvdentalcare.com/contact-us-citrus-valley/', 'http://www.bonniesyarncrafts.com', 'http://www.nedrebos.com', 'http://www.ThePMFirm.com', None, 'https://www.xmsweetconnections.com', None, '1592173062', None, 'http://www.ivandianaphotography.com', 'http://www.dunkindonuts.com', '1592173071', 'http://www.cbac.com/sun-prairie?utm_source=Social&utm_medium=Yelp', 'http://www.threebrothersbarandgrill.com', None, 'http://www.cabinforlife.ca', None, 'https://www.azsolarconcepts.com', 'http://mwsvegas.com', 'http://scottsdalecosmeticdr.com', 'http://www.southsidejewelers.com', None, 'https://www.drtroell.com', 'https://www.coolrunningsautomotiverepair.com', 'http://www.stop-n-go.com', 'https://www.zimbrickchevrolet.com', 'http://www.preciouspets702.net', 'http://www.structube.com', 'http://www.middleroadbarbershop.com', 'https://karenshallmarkshops.com', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.springdayspa.com', None, 'http://ridgeparkurgentcare.com', 'https://www.spectrumeyecarenc.com', 'http://vinoandpanino.com', 'http://orangeleafyogurt.com', 'http://weddingsbybonnie.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.richardmilleusa.com', 'http://www.chaneyscollision.com', 'http://www.townshoes.com/', 'http://www.indulgeyourselfsalon.com/', None, 'https://www.luxurypad.vegas', 'http://wanleeloy.com/', 'http://mindbrainlv.com', 'https://www.eatnpark.com', 'http://lerock403.com/', 'http://www.quiznos.com', 'http://www.crossfitculmination.com', None, None, 'http://www.shadowridgeapartmenthomes.com', None, 'http://le-cue.com/', 'http://www.carolinashealthcare.org', 'http://americanbrosplumbingco.com', 'http://www.thepalmsapts.com', None, 'http://www.bk.com', 'http://www.daracap.com', 'http://www.selectphysicaltherapy.com', 'https://www.premierworkspaces.com/nevada/henderson-augusta-park?utm_campaign=Yelp&utm_source=Yelp%20Organic', 'http://www.aulaz.com', 'https://locations.whataburger.com/az/goodyear/14251-west-indian-school-bypass.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', 'http://bullandfinchbars.com', 'http://www.harkinstheatres.com/theatreDetails.aspx?theatreId=4527', None, 'http://www.stfrancisdesaleslv.org', 'http://www.SallyBeauty.com', None, None, 'http://www.ljsilvers.com', None, '1592173210', 'www.brokenplate.ca', None, 'https://www.taylorstmgmt.com', None, 'http://www.swrandalltoys.com', 'http://www.factorydirect.ca', 'https://champsfamilyautomotive.com', 'http://www.orbitzcleaning.com', None, 'https://www.choicehotels.com/pennsylvania/pittsburgh/sleep-inn-hotels/pa744/rates', 'http://www.krispykreme.com', None, 'http://www.jacksonpt.com', None, None, None, None, 'http://www.6az.com/', 'http://www.ynhcclinic.com/', 'http://www.altafillmore.com', None, 'http://www.untitledandco.com', 'http://www.thesodashop.co', 'https://www.themarketphx.com', 'http://www.dqlunchforless.com', 'http://www.yourmovingsolutions.com/?utm_source=Yelp', 'http://miketheplumbercharlotte.com', 'http://www.lasvegasjeepandoffroad.com', 'http://www.oldnavy.com/products/las-vegas-nv-store-5811.jsp', 'http://www.starliquor.ca/', 'http://www.thefrenchbee.com', 'http://www.jeffsnotary.com/', None, None, 'http://pettis-pizza.com', None, 'http://www.rajdhanisweets.ca', 'http://www.simsgm.com/', None, 'https://joefilter.com', 'https://www.maaco.com/locations/az/glendale-11376/', 'http://www.pandaexpress.com/', 'http://www.meineke.com/', 'http://www.dunkindonuts.com', 'http://www.urbanhakka.ca', None, 'http://www.sweetrubyjane.com', None, None, None, 'http://www.legacybungalowsapts.com', 'http://www.getbooked.com', 'https://www.moes.com/find-a-moes/pennsylvania/murrysville/3428?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', 'http://www.mrpuffs.com/', 'http://www.dibeads.com', 'https://www.carstar.ca/en/locations/ab/calgary/carstar-calgary-sunridge-16118/', None, 'http://www.colormebella.com', 'https://www.wilhelmautomotivelitchfield.com', 'http://www.valleverdecarwash.com', None, 'http://www.lapetite.com/your-local-school/las-vegas-nv-7300/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', 'https://local.fedex.com/nc/gastonia/MRNA/?cmp=LOC-1001545-3-2-971-1110000-US-US-EN-YEXTLISTFEDEXSC', None, 'http://www.escapeloungelasvegas.com', 'http://azlifts4less.com', 'https://scottsdaleair.com', None, None, 'http://www.studiofivehair.com', None, 'http://www.insidethebungalow.com', 'http://www.health4lifenv.com', 'http://www.llbcpa.com/#dxEvq', 'http://www.fantasticsams.com', 'http://www.greatclips.com/', None, 'http://www.carolinafireplacerepair.com', None, 'http://www.dickies.com', None, 'http://www.cavabienspa.com/', None, None, None, 'http://beijingchinesecuisine.com', 'https://www.shoeshow.com', 'http://www.orangeleafyogurt.com', 'http://www.starbucks.com/store/8054/', None, None, None, 'http://www.sunshinemoon.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.stevesultimateair.com', 'https://www.wingstop.com/location/wingstop-1117-cleveland-oh-44128/menu', None, 'http://www.inspectslasvegas.com', None, None, None, 'http://www.barefoot-acupuncture.com/', 'http://www.kxyorkville.com/', 'http://flamebroilerusa.com', None, None, None, 'http://www.dragongemchinese.com/', 'https://www.collegecutiescleaningcars.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.fieldservicesaudio.com', 'http://www.lasvegastransportation.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.redrockcanyonlv.org', 'http://www.yongesheppardcentre.com', None, None, 'http://locations.jjill.com/az/scottsdale/65', 'http://www.savourfinefoods.com', 'http://www.twosphotobooth.com', 'https://ccv.church/locations/midtown-phoenix', '1592173504', None, None, None, 'http://www.pmcpropertygroup.com/', 'http://www.citywidemovers.ca/', 'http://www.apnursery.com', 'https://www.extraspace.com/storage/facilities/us/california/glendale/299955/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, 'http://www.sndk9.com/', 'https://local.pharmacy.safeway.com/az/scottsdale/7920-e-chaparral-rd.html', 'https://zoeskitchen.com/locations/store/az/mcdowell', '1592173529', None, 'http://kickinkitchenfoodtruck.com', None, 'http://www.lxvestudios.com', 'http://www.themugshottavern.com/', 'http://www.kravelasvegas.com/', None, 'http://www.noodlewok.com/', 'http://www.gigiscupcakesusa.com/ahwatukee-arizona', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.classictattoowilloughbyoh.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.davidduncanhouse.com', 'http://www.Totallymadeover.com', None, 'http://www.cccakesbybrandi.com', 'http://www.thankqrentals.net', '1592173562', None, None, None, None, 'http://www.grcogrill.com', 'http://www.cantinalaredo.com/location/scottsdale-arizona/', 'http://www.japaneats.ca', None, 'https://www.esso.ca/en/gas-station/vaughan-on-majormacvelloreesso-200302606', 'http://www.jordansrefurbishing.com', 'http://www.gyrostown.com', 'http://www.eddiebauer.com', None, 'http://www.serenityspalounge.com', 'http://www.timhortons.com', 'http://www.iphotoshop.com/', 'http://h5helicopters.com', 'https://www.fahadrestaurant.com', 'http://www.relocationservicescanada.ca', 'http://redcanyondentistry.com', 'http://www.quaker.org/pghpamm/', 'http://www.loveyourposture.com', 'http://www.manilabbq.com', 'http://www.kelseys.ca', None, 'http://www.baskinrobbins.com', 'https://www.boutiquewebsites.ca/website-designs.html', None, 'http://www.decatarina.com', 'http://www.felixswimschools.com/locations-schedules/woodbridge', 'http://www.magictouchmobiledetail.com', 'http://www.shenandoahmill.com', 'http://www.lovinghut.ca', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FLASHH', 'http://www.molonlabe.com', None, None, 'http://www.bundlebuilders.net', 'http://www.thebrewteabar.com', '1592173643', 'http://thecellarsalon.com', None, 'http://carambamex.com', 'http://villagepubcasino.com', None, 'http://www.cainpark.com', 'http://www.fairwaysapartmenthomes.com', 'https://www.athome.com/store-detail/?StoreID=North%20Carolina-Charlotte&utm_source=yelp&utm_medium=paidprofile', 'http://www.midas.com/toronto/store.aspx?shopnum=9827&dmanum=857001', None, 'https://www.bannerhealth.com/locations/phoenix/surgicenter', 'http://www.bestdealsauto.ca', 'http://www.dunkindonuts.com', None, 'http://dekra.us/en/nevada/henderson-26-w-horizon-ridge-pky/', 'http://www.pizzapizza.ca/', 'http://www.urbanfarm.org', 'http://fairgreenlawncare.com', 'http://www.flamingicecube.com', 'http://tnailslasvegas.com/', 'http://countrybarpeoria.com/', 'http://www.magicofjasonbird.com', 'http://kidsinclearningcenters.com/chandler-watermark/', 'http://www.jimenezarmsinc.com', 'https://www.squawpeakhilton.com/dining?_ga=2.50950569.1853949617.1562623293-1431667612.1561583712', 'http://www.madisontranquilitymassage.com', None, 'http://spgilbert.com', 'http://www.siamthaipgh.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.doraziolawgroup.com', 'http://www.highlandinnlasvegas.com', 'https://www.uhaul.com/Locations/Trailer-Hitches-near-Charlotte-NC-28217/780053/', 'https://www.walgreens.com/locator/walgreens-6906+university+blvd-moon+township-pa-15108/id=10917', 'http://www.rinxtoronto.com', 'http://kingskitchen.org/', 'http://www.wholehealth4life.org', None, 'http://www.velocityautobody.com', 'http://www.hotpinkteahouse.com', None, 'http://www.showtimetourslv.com/', 'https://www.dominos.com', None, '1592173746', 'https://us.petvalu.com/location/6058', 'http://illinois.edu', None, 'http://www.arizonaeliteproperties.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', '1592173758', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.missspice.ca', 'http://www.24-7electricalservices.com/', None, 'http://www.familydollar.com', 'http://bchcares.org', 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', None, 'http://www.unforgettablecoatings.com/', 'https://www.purebarre.com/location/fort-mill-sc', 'http://www.starbucks.com/store/9289/', '1592173785', 'http://someburros.com/', 'https://chatters.ca', 'http://salsaritas.com', 'http://www.barbiesgrill.com', None, 'http://www.cleyards.com', 'http://www.onenevada.org', 'http://www.spectrac.com', 'http://www.hortfrontiers.com', None, 'http://pursuitcf.com', None, None, 'http://www.qcreflex.com/', 'http://www.azwhiteningspa.com/index.html', 'http://www.houseofathens.com', 'http://www.firstclassprinting.ca/', None, 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'https://crissangel.com', 'http://TignorGroup.com', 'http://www.davismagic.com/', 'https://www.kellerhearing.com', 'http://www.cannabisclinics.ca/medical-marijuana-canada-clinic/#1440516223690-62dfe946-9e59', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreId=976', 'http://www.oasisbeautyspa.ca', 'http://www.sushiplus.ca/', '1592173848', '1592173850', 'http://www.searshometownstores.com/home/az/fountain-hills/1841', '1592173854', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', None, None, None, 'http://www.furnitureworld1.com', None, None, 'http://www.bellagioapts.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', None, None, None, 'https://cell-computers-and-games.business.site', '1592173881', 'https://mamajuliannes.com', 'https://backyourhero.com', 'http://www.theplacerestaurant.ca', 'http://www.urbanbulkandrefill.com', 'https://www.lumberliquidators.com/ll/stores/nc/matthews/11101-east-independence-blvd?utm_source=yelp&utm_medium=local&utm_campaign=yext&utm_content=1264', None, 'http://www.marengospa.com', 'http://dialabottle.ca', None, 'https://ad.doubleclick.net/ddm/clk/434342189;236814625;k', None, None, 'http://riotanlasvegas.com/wordpress1/', None, 'http://www.alyciamoore.com', 'http://abdappraise.com', 'http://www.fatbastardburrito.ca', 'http://abakeshop.com', 'http://www.mesabilliards.com', None, 'http://www.marbleslab.ca', 'https://townandcountrybarbershop.com', None, 'http://www.covenantday.org', 'http://www.lakenormanpaintandbody.com', None, None, 'http://www.elitemassagecalgary.ca', 'http://www.cccnevada.com/doctor/g-h-kashef-md/', None, 'https://www.huntington.com/Community/branch-info?locationId=23476', None, 'http://www.bivfaz.com', 'http://www.metro.ca', 'http://www.vegashot.com', 'http://www.crossroadsvet-painesville.com', 'http://www.baskinrobbins.com', None, 'http://www.XtremeExtensions.com', 'http://www.arbys.com', 'http://www.scottsdaleaz.gov/preserve.asp', None, '1592173969', None, 'http://vivaviolins.com/', 'http://www.fionnmaccools.com/locations/brampton-ray-lawson', 'https://www.wingstop.com/location/wingstop-421-phoenix-az-85015/menu', '1592173979', 'https://www.sprouts.com/stores/details/-/store-details/view/store/2/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'http://www.maidpro.com/phoenix-north', 'http://www.kingputtlv.com', 'http://mysuboxonedoctor.com', 'http://www.ivylasersalon.com', 'http://www.kofeinstitute.com', 'https://www.extraspace.com/storage/facilities/us/nevada/las_vegas/300031/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.laredosrestaurante.com', 'http://www.wizardofsuds.net', 'http://www.ontarioparks.com/park/voyageur', None, 'https://excelprimarycare.com', None, None, 'https://www.malakjewelers.com', 'http://www.clevelandishome.com', 'http://www.pattysheart.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.marrakechrestaurant.ca', 'http://the-yellow-penthouse.business.site', None, None, None, 'http://www.jjbrewco.com', 'http://www.piccoloantiquemall.com', 'http://www.divinethreading.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.allthingstea.ca', 'http://www-rivera-custom-painting.com/PROFILE', 'http://www.moxieinkdesigns.com', 'http://www.royaljade.ca', 'https://honorhealth.com/locations/primary-care/honorhealth-medical-group-east-tempe', 'https://www.napaonline.com', None, None, 'http://www.abpreschool.com/', 'http://www.ryanssalons.com', 'http://www.scottsdaleaz.gov/parks/cactuspool.asp', 'https://carolinacleanerscharlotte.com', 'http://www.goldsgymcalgary.ca', None, 'https://diceandbeansboardgamecafe.com', None, None, 'https://www.desertfinancial.com/Home/BranchDetails?branchId=1', 'http://www.trendautomotive.com', None, None, 'http://www.isaacjewelers.com', None, '1592174106', 'http://www.ashfordplaceapartments.com', 'https://store.vioc.com/nv/las-vegas/9631-trailwood-dr-89134-HC0003', '1592174114', '1592174118', None, None, None, 'http://pitapitusa.com', None, 'http://meridianlv.com/', None, 'https://www.casinoarizona.com/dining/eagles-buffet/', None, None, None, 'http://www.haggen.com', 'http://www.brickpointingpittsburgh.com', None, None, None, 'http://pennhills.org', 'http://rrrc.run', 'http://www.sunterramarket.com', 'http://www.pastorsautoservice.com', 'http://lvhottubs.com', 'http://l.macys.com/university-heights-oh?cm_mmc=Yelp_Stores-_-UniversityHeights-_-n-_-578&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=University+Heights_578&m_ac=yelp_stores&m_ag=n', None, 'http://www.mychickenshack.com', 'http://angelschristianacademy.com/landing.php', None, 'http://www.housing.wisc.edu/universityapartments/eheights', 'http://www.evaszabo.com', 'http://www.dentalemergencyservices.ca', None, None, None, '1592174202', 'https://container36.wixsite.com/wtrn', 'http://www.artisanhairstudio.net', 'http://www.desertheatwindowwash.com', 'http://www.thedoctorfence.com', None, None, '1592174224', None, 'https://www.choicehotels.com/north-carolina/charlotte/suburban-hotels/nc533/rates', 'http://www.goodmanslandscape.com/', None, None, None, '1592174241', 'http://www.getthelook.com/salondetail/default.asp?salonid=61146', 'http://www.adamsroadsidetowing.com/', None, None, None, 'http://www.aalltechmobile.com', 'http://www.hoopdome.com', None, 'http://www.nrsinjurylaw.com', None, 'http://www.coldwaterroofing.com', 'http://www.MassagesByChelsa.squarespace.com', 'https://www.wallacebethelparkflower.com', None, 'https://www.richerlifecounseling.com', 'https://someburros.com', None, 'http://marcelos-az-barbershop.business.site', None, None, 'http://cowtownbeefshack.ca', 'http://cooperscarpetandtilecleaningaz.com', 'http://alexbeauregard.com', None, None, '1592174306', None, None, None, 'http://www.wrapcitydesigns.com', None, 'http://www.southshoresah.com', 'http://www.balzacs.com/locations/distillery-district/', '1592174326', 'https://npidb.org/organizations/physician_assistants/family_363lf0000x/1386110443.aspx', None, None, None, None, None, None, 'https://www.americandiscountfoodsaz.com', 'http://www.JenniferKostuchowski.MyHomeHQ.biz', 'http://www.sweetdaddycupcakes.com/locations.html', None, None, 'http://www.drbarin.com', 'https://nuancelasvegas.com', '1592174369', None, 'https://www.petsmart.ca/stores/ca/on/scarborough-store2009.html?utm_campaign=us-lcl-srv-eg&utm_medium=listing&utm_source=yelp', 'https://goppa.org', None, None, 'https://www.perplexitygames.com', 'http://www.printystamp.com', 'https://alfredopatisserie.com', 'http://ipacupuncture.com/', None, 'https://hirosushi.to', 'http://www.verizonwireless.com/', 'https://www.peopleready.com/locations/las-vegas-nv-1144', 'http://hancockfabrics.com/', None, 'http://www.welltracnv.com', 'http://www.greenstarautocenter.com', 'http://www.pmkbar.com', None, None, None, None, 'http://www.bhhsnv.com', 'http://www.dhslv.com', None, 'http://www.airportparkingreservations.com/lot_ez_airport_parking_yyz', None, 'http://www.andysfishandchips.com', None, None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1212+South+Greenfield+Road-Mesa-AZ-85204/storeid=5026?cid=ps_ylp_2020', None, None, 'http://www.alwaysjumping.com', 'https://www.mimiscafe.com/locations/glendale/', 'https://www.streetsideclassics.com', 'https://healinghandshannah.com', 'http://www.wendys.com', 'https://nationwidevision.com/locations', 'http://www.canadiantire.ca/en/store-details/on/toronto-main-danforth-on.store.html', None, 'https://globalautoscarrentals.com', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'http://www.gnc.com/', 'https://www.residentculturebrewing.com', 'http://www.ucalgaryrecreation.ca', 'http://www.sheaparc.com', None, 'http://www.windowuniversecleveland.com', 'http://www.marvelousmousetravels.com', None, '1592174529', 'http://www.eraagelessfuture.com', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', None, 'http://www.gap.com', '1592174544', 'http://www.floweramacleveland.com', 'http://www.mycdi.com/arizona/insight_imaging_-_arrowhead/', None, None, 'http://www.perfectblend.ca', 'https://locations.theupsstore.com/pa/moon-township/5990-university-blvd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://www.laseraway.com/locations/nv/las-vegas/?utm_source=yelp+organic&utm_medium=website+link', 'http://www.fabfred.com/locations/trailwood-drive/', None, 'http://www.srsengineshop.com', 'http://studiooftens.com', '1592174581', None, 'http://www.pcfarm.ca', None, 'https://www.cartelcoffeelab.com/pages/5th-avenue', 'http://www.disantopizza.com', None, 'http://www.heritagefishnchips.com', 'http://www.needle-pin.com', 'https://speeders.ca/calgary/', 'http://www.countryhillsmassage.com', 'http://www.fedmc.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360246&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://tempeautoshop.com', None, 'http://www.silverstatecu.com', 'https://www.monicals.com/locations/monicals-pizza-of-st-joseph/', 'http://www.steveedaniellehairandmakeup.com/', 'http://www.myhomegroup.com', 'http://all4you.cleaning', None, 'http://www.drkennedychiro.com', 'https://www.petsmart.com/stores/us/az/surprise-store1040.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', '1592174651', 'http://www.moestire.com', 'http://theartpartystudio.com/', 'https://www.handandstone.ca/locations/richmond-hill', 'http://www.smallcakesphx.com', 'http://www.ratatouillerestaurante.com', 'https://rosyeyebrows.com', 'http://www.phoenixcylinderhead.com', 'http://www.gobencars.com/', 'http://www.bellamiahairstudio.com/', 'https://www.hotelartskensington.com', 'http://www.qimontessori.com', 'http://www.AZbodySugaring.com', 'https://www.ceenta.com', 'https://www.firehousesubs.com/locations/nc/indian-trail', None, 'http://startersaz.com', None, None, 'http://pdubsbbqlasvegas.mypub.website', 'http://porterchiropracticaz.net', 'http://www.alexanderslv.com/', '1592174720', 'http://www.milestonesrestaurants.com/', 'http://www.eddiesalehouse.com/', None, 'http://www.xclusivesparlor.com/', 'http://www.christianjamesart.com/#!/page/183244/home', 'https://www.wyndhamhotels.com/hotel/49487?tel=18002521709&iata=00093785&cid=fe:ra:20170406:yl:pp:ramus:49487', 'http://www.torontomarriagecounselling.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.asianimages.biz', 'http://www.sallybeauty.com', 'https://www.jeansflowers.org/?srccode=yelp_track', 'http://www.lassendasspineinstitute.com', None, 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', None, 'http://www.caplans.ca/', None, None, 'https://www.ewingirrigation.com', 'http://www.six02.com/scottsdale', None, None, None, 'https://internetmarketingteam.com', 'http://lucerestaurant.net', 'http://www.penn-station.com/', '1592174806', 'http://loandaodentistry.com', '1592174812', None, None, 'https://bmcmattress.com?utm_campaign=PhoenixJanMar2018&utm_medium=organic&utm_source=Yelp&utm_content=website-organic', 'http://www.homesurfacesaz.com/', 'http://thelivingroomhairstudio.com', 'http://www.lecreuset.com/stores/tanger-outlets-phoenix-glendale-az', 'http://clinicalsolutionsskincare.com', 'http://www.danier.com', 'http://www.hollisfamdental.com', 'https://drivingmba.com/?utm_source=Yelp&utm_medium=CPC&utm_campaign=Yelp-Ads', 'https://genkijapanese.info', 'http://www.contactlensking.com/', 'http://www.head2toessalon.com', None, None, None, 'http://americarestaurant.ca/', None, 'http://www.amresupply.com/locations/calgary', None, None, 'https://dekalash.com/find-a-studio/pennsylvania/wexford/', None, 'https://www.mjchristensen.com', 'http://www.pusateris.com', 'http://ProCS.us', 'http://swandetailing.com', 'https://ironglovesboxing.com/', 'http://www.bellunionautoservice.com', 'http://www.shoepalace.com?utm_source=yelp&utm_medium=yelp149', 'https://axethrowingpgh.com/booksouthhills/', 'http://www.tuttifruttidejeuners.com/en', 'https://www.courtesyjeepsuperstitionsprings.com', None, 'http://www.londoncleaners.com', None, 'http://www.helvighealth.com/', 'http://www.bebe.com', '1592174941', 'http://www.verizonwireless.com/stores/arizona/scottsdale/promenade-330170/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.lilliputhats.com/', 'https://www.number7honda.com', None, None, 'http://www.wineamplified.com/', 'http://www.glennstoolservice.com', 'http://www.thevictorianmonkey.com', None, 'https://fntarizona.com/scottsdale-kierland', None, 'http://www.anthemchildcare.com', 'http://www.nicols.ca/', 'http://blueskyoils.com', None, 'http://faceofanangel.net', 'http://www.intersport.ca', 'http://shessheik.com', None, None, None, None, 'http://www.adventcoworking.com', 'https://www.walmart.com/store/2512/phoenix-az/details', 'http://www.universalmotorcarslv.com', 'http://northpawdogs.com', '1592175027', 'http://www.pawsinthebath.com', 'http://www.auntieannes.com', 'https://www.walmart.com/store/2256/henderson-nc/whats-new', None, None, None, None, 'http://www.scavolinitoronto.com', 'http://holidaypetfestival.com', 'http://www.precisealterations.com', None, '1592175060', '1592175063', None, None, None, 'http://cateringcompanieslasvegas.com', 'https://www.wyndhamhotels.com/hotel/09499?tel=18003291073&iata=00065402&cid=fe:di:20161220:yl:pp:dius:09499', None, 'http://www.samosagrill.com/', 'http://www.lvcrystalnailspa.com', 'http://www.jamesgallery.net', None, None, 'http://www.thekentstage.com', None, 'http://noirtacgroup.com', 'https://www.goodyearmoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://yuriyscustomjewelry.com', None, 'http://www.vdevmaison.com', 'http://www.newprairieconstruction.com', 'http://www.parkroadhealingarts.com/', 'https://www.ecolinewindows.ca/location/calgary/', None, 'https://www.target.com/sl/mesa-west/1429', None, None, None, None, None, 'http://www.peterparvez.ca', None, 'http://www.teavana.com', 'http://www.lifecaremedicalrx.com', 'http://www.saladessensations.com', 'https://www.jerseymikes.com/3020', 'https://www.uneedmartialarts.com', '1592175182', 'http://aldi.us/index_ENU_HTML.htm', 'http://lincolndoor.com', 'http://bk.com/', None, None, 'http://designerartdirect.com', 'http://www.eljesseautorepair.com/', '1592175206', 'http://www.marriott.com/hotels/travel/yyzbp-fairfield-inn-and-suites-toronto-brampton', 'https://bolmergc.com', 'http://www.snapfitness.com', 'http://www.arrowheadlakesdentistry.com', 'http://www.simonesmtlebanon.com', 'http://www.jewelrysalonlv.com', 'http://www.whitehousechicken.com', 'http://www.carlitosmokes.com', None, None, 'http://www.riderscycleboard.com', None, None, 'http://www.conns.com/store-locator/nevada/las-vegas-89121', 'http://www.glowbeautyloungellc.com', None, 'http://www.sugarmarmalade.com/', None, None, None, None, 'http://scottsdalelavishnails.com/', 'http://www.bambooandspine.com', 'http://www.eyeglass-store.com', 'http://www.pioneercardiovascular.com', 'http://www.davisvillemedicalclinic.com', None, 'http://www.robbellplumbingservices.com', None, None, 'https://oralsurgerylv.com', 'http://www.ritzcarlton.com/en/Properties/Phoenix/Dining/Bistro24/Default.htm', None, None, 'http://envynailsspa.ca', 'https://happytreeslandscaping.com', None, None, None, 'http://kimmiesrestaurant.com', 'http://www.7thdegreeclothing.com', None, None, 'http://www.168sushibuffet.com/mississauga', 'http://www.chipofftheblocklv.com', None, 'http://www.ezpawn.com', 'http://www.applebees.com', 'http://www.portofsubs.com', 'http://www.thecharlotteforum.com', 'https://lastresrosasllc.yahoosites.com', 'http://www.amayaexpress.com/', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Phoenix-AZ-85023/858021/', None, '1592175367', 'https://agents.farmers.com/nv/las-vegas/yebel-escalona?SourceID=AMPALFYEAG&utm_source=Yelp&utm_medium=Local', 'http://buddysautorepair.com', 'http://www.mactechcomputer.com', 'http://www.azmountainviewmassage.com', 'http://vandykeaesthetics.com', None, 'http://www.faithfulwordbaptist.org', None, 'http://www.calgarylabservices.com/', 'http://onestoprealtylv.com', 'http://farmhousefoods.com/', None, None, 'http://koreangrillhouse.com/yonge-bloor-location/', 'http://rollinrustgarage.com', None, 'http://vintageconservatory.com', 'http://phatbuddhatattoos.com', 'https://www.241pizza.com', 'http://www.DentistryoftheCarolinas.com', None, 'http://www.rockstaraccountantsllc.com', 'http://www.chandleraz.gov/default.aspx?pageid=625', None, 'http://www.onemanandahammer.com', None, 'http://www.sushinotori.com', '1592175450', 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.scriptpharmacy.ca/', 'http://www.pancheros.com/locations/urbana', 'http://www.cultivategoods.com', 'http://www.starbucks.com', 'http://www.reynoldsplumbingllc.com/', 'http://www.clevelandcupcakecompany.com/', 'http://www.esmil.com', 'https://www.destinationhotels.com/scottsdale-resort/dining/bar-six40', 'https://www.wwwilliams.com/location/phoenix-az/', None, 'http://www.quiktrip.com', '1592175489', 'http://paradisegreens.com', 'http://www.aerobatics.com', None, None, None, 'http://bellybombz.com/lasvegas', None, 'http://www.madtownrentals.com/', 'http://www.eyedistrictbyjo.com', 'https://www.bathandbodyworks.com/store-locator', None, 'https://www.zamzamhalalmeat.com', 'http://www.walmart.ca/en/walmart-meadowvale-supercentre/3054', None, None, 'http://www.perfectcredit60days.com', 'http://todaysauto.ca', None, None, None, 'http://stores.tuesdaymorning.com/nc/matthews/home-decor-matthews-nc-char0146.html', None, None, 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', None, 'http://www.tntwindowtint.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, 'http://galaxyktv88.com', 'http://www.hollandmarshsoupfest.ca/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.cellularsourcestore.com', 'http://aldi.us/index_ENU_HTML.htm', 'http://www.openstudioartcafe.com', None, None, '1592175611', 'http://www.haifengchinese.com', 'http://www.centennialpediatricslv.com', None, None, 'http://www.mississauga.ca/portal/discover/lakeviewgolf', 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'http://www.abceurodeli.com/', None, '1592175639', None, 'http://www.malkinlegal.com', 'http://www.nextlevelpaintersaz.com', None, None, None, '1592175656', 'https://www.centurylink.com', None, 'https://www.mytightface.com', None, 'http://www.suncoastaz.com/', 'https://www.indochino.com/showroom/charlotte', None, 'http://www.indulgeandco.com', None, 'http://tradeandlore.com', None, 'http://heartlandgsbeautybar.simplybook.me/sheduler/manage', 'https://www.emser.com', 'http://www.quiktrip.com/', None, 'https://www.smithsfoodanddrug.com/storeHours?store=70600376', None, 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', '1592175713', 'http://www.The-Dawg.com', 'http://www.southtempeapartments.com', 'http://shop.hobbylobby.com', 'http://www.thefitalterations.com', 'http://www.vpicasso.com', None, 'http://www.healingheartspeds.com/', 'http://sportsperformanceandspine.com', 'http://www.racewaychrysler.com', 'http://www.fendi.com/ii/store-locator', 'http://clickserve.dartsearch.net/link/click?lid=43700044280205718&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', '1592175749', 'http://www.barbqueboss.com', 'http://adjaafricanbraiding.com', '1592175758', 'http://beyondexpectationdental.com', None, 'http://www.petvethospitals.ca', 'http://www.azenaphoto.com', 'http://www.axisgear.ca', 'https://www.trebilcockcoffee.ca', 'https://www.thetouchofitaly.com', 'http://Rickslawnandlandscaping-lv.com', '1592175785', 'http://www.onceuponachildmacleod.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.espressoroyale.com/', 'http://dtrschooloftrucking.com', 'http://newoutlookopticalnv.com', None, 'http://sushi930.com', 'https://www.mediweightloss.com/locations/ballantyne?_ppc=Z8FWKRXKT5', None, 'http://www.richardsgardencenter.com', None, 'https://locations.levi.com/nc/charlotte/denim_charlotte_nc_258.html', 'http://mojoscoffeeandmore.com', 'http://www.barefootpoolman.com', 'http://www.lushexpress.com', None, 'http://www.stclair.org', None, '1592175848', 'http://www.palmettopethospital.com', 'http://www.mountainsidechiropractic.com', 'http://www.exclusiveballoons.com', 'https://diamondsstrip.ca', 'http://valleywidehouses.com', None, 'http://www.dayspring-umc.org', None, 'https://www.northtownvethospital.com', None, 'http://www.candy-addict.com', 'http://www.harthouse.ca', 'http://www.elitephoenixseo.com/', None, 'http://shea-blvd-chiropractor.com/', 'http://www.alluniformwear.com', None, 'http://www.draskovich.com', None, 'https://lvcollectables.com', 'http://jonesmoverz.com', None, 'http://www.gobeautybar.com', None, 'http://calgary.regency.hyatt.com/en/hotel/home.html?src=corp_yelp_businesslisting_calrc_en_homepage_desktop&code=YB', None, None, 'http://www.thesundanceclinic.com/', 'http://www.sunvalleyfamilyoptometry.com', 'http://www.vipbeautysalon.org', 'http://www.murdockmeals.com', None, 'http://www.pghfilmmakers.org', 'http://pferguson.pillartopost.com', None, 'http://www.dunkindonuts.com', None, None, None, None, 'http://www.masterpieceescapes.com', 'https://nothingbundtcakes.com/bakery/AZ/goodyear?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=goodyear_138', 'http://www.bobbiepinbeauty.com', 'http://www.firkinpubs.com', 'http://murphyusa.com', 'http://Swanpizza.com', 'http://www.advancedpreventivemedicine.com/', 'http://www.starbucks.com/store/6597/', 'http://www.infinityhd.tv', 'http://www.starbucks.ca', 'http://www.lasvegashomewatchers.com', 'http://torontopawn.ca', '1592179247', 'http://www.vegasdiscountnutrition.net', 'http://www.bulkbarn.ca/en-ca/index.html', 'http://adventuredogpark.com', 'http://www.timhortons.com/ca/en', 'http://www.apple.com/retail/southpark', 'http://www.iltartufo.ca', 'http://www.hydrotherapiesplus.com/', 'http://www.desertspringschiro.com', 'https://www.pizzahut.ca/store-details/store-R42101/pizza-hut-Etobicoke', 'http://www.thetirexpress.com', 'http://www.tuckedawaycraftkitchen.com', 'http://www.renewedlifechiropractic.com', 'http://www.phxplastering.com', None, None, 'http://www.muchoburrito.com', 'http://www.theriverwoodconservancy.org/', None, None, None, 'http://blackforgecoffee.com', 'https://www.slatescottsdaleapartments.com/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://locations.dennys.ca/AB/CALGARY/248179', None, 'http://www.babysafehomes.com/united-states-baby-proofing-locations/child-proofing-phoenix-arizona', 'http://www.clevelandmoto.com', 'http://www.applerepaircanada.ca', 'https://www.druryhotels.com', 'http://www.sabaiaz.com', None, 'http://www.sunsationalswimschool.com/', 'http://www.lapetgrooming.com', 'http://www.gamestop.com/store/us/az/scottsdale/2760/sonoran-village-gamestop', None, 'http://www.ezyes-takeout.com', None, 'http://www.trendvape.com', 'https://glassnickelpizza.com/location/sun-prairie/', 'http://www.arizonarvresorts.com/Pioneer', None, '1592179997', 'http://www.myhealingplaceaz.com', None, 'http://www.restaurantconfluence.com', None, 'http://www.bioprollc.com/', 'http://www.valleyofthesunpharmacy.com', 'http://www.arizonasandwichco.com', None, 'http://www.frogbox.com/toronto', 'https://sushisanrestaurant.business.site', 'http://www.jwoodflooring.com', 'https://www.lefrancisstudios.com', 'http://www.aboutudental.com', 'http://www.kokomogrills.com', 'http://www.ivanhairsalon.ca', '1592180030', None, 'http://www.granadalakestempe.com', 'http://www.capellaeyecare.com', None, None, 'http://www.onthelistdenver.com', None, 'https://www.lvscc.com/south-rancho', 'http://www.lotsmartsolarscreens.com', 'http://www.ontariotrails.on.ca/trails/view/cheltenham-hills', 'http://georgesbrasserie.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://freshandeasy.com/', None, None, 'http://www.northwoodslanes.com', 'http://cleanmycarpetlv.com', 'http://www.nonnispizzamenu.com', None, 'http://globalbikes.info/', 'https://locations.wendys.com/united-states/az/phoenix/9905-w.-camelback-rd.', '1592180077', None, '1592180081', 'http://www.hotdogonastick.com', None, 'http://www.luchitas.net/elyria.php', None, None, None, 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', None, None, 'https://www.fitnessworks.com/locations/dobson-rd-chandler-blvd', None, None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.buckysportabletoilets.com', None, None, 'https://www.mirage.com/en/restaurants/osteria-costa.html', 'https://www.cheyennedentalgroup.com/?utm_source=yelp&utm_medium=link&utm_campaign=yelp', 'http://www.atobmove.com', 'https://locations.waxcenter.com/nc/charlotte/charlotte-arboretum-0169.html', None, None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Yucca.aspx', None, None, 'http://www.qsdiner.com', 'http://www.pb.com', 'http://www.amenzone.com', 'http://www.cremacoffee.ca', 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpsouthwestorganic', 'http://www.blueberryhillrestaurants.com/', 'http://www.wishacake.ca', 'http://www.littlecrittersvet.com', 'http://www.c3lasvegas.com', '1592180154', None, 'https://www.thebrick.com/pages/store-north-york', 'http://www.rightc3.com', 'http://imboardgames.com', 'http://www.bwilliamsdds.com', 'http://www.stephita.com/', 'http://www.underdogpetrescue.org', 'http://www.canadianamphitheatre.net/', 'http://www.anthropologie.com', 'http://www.noblecare.us/', None, None, 'http://titanhomeimprovements.com', 'http://nvjawdoc.com/', 'https://www.redlobster.ca/locations/list/on/brampton/368-queen-street-e', '1592180189', None, 'http://www.scottsdale-sc.com', 'http://www.zappos.com', 'http://www.naturalgourmetfoods.com/', 'http://www.walgreens.com/locator/Walgreens-10503-W-Thunderbird%2C-Suite-101B-Sun-City-AZ-85351/id=10018', None, '1592180205', 'http://www.pittsburghcitywide.com', None, 'http://www.inspectioneeringpgh.com/', 'http://www.arizonacharliesboulder.com/Food-Drink/Sourdough-Cafe', 'http://www.regency.edu/campus/madison/?guest', None, 'https://www.leprovencalbistro.com', '1592180219', 'http://www.fallasstores.com', None, 'http://sabiskitchen.com/', 'https://www.nofrills.ca/store-locator/Enzo%27s+nofrills+%28Yonge+St.%29/0770?storeId=0770', 'http://www.sassiskin.com', '1592180232', None, 'http://www.kyrene.org', 'https://www.aqualineplumberselectriciansacrepairscottsdaleaz.com', None, '1592180244', None, None, 'https://bananarepublic.gap.com', 'http://www.artisticintegrity.ca', 'http://www.plumberphoenixboydco.com', '1592180257', 'http://www.goldengeckocoffee.com', None, 'https://www.kfc.ca/store/3199-blvd-taschereau-greenfiled-pk', 'http://www.carolynannryan.com', 'http://www.hyperproductionsdjs.com', 'http://www.menchies.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.coldstonecreamery.com/stores/20030', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.globalpetfoods.com', 'https://www.nbc.ca', None, 'https://www.maids.com/106/?utm_source=Yelp', 'http://www.preferredhomecare.com/', 'http://www.lighthouseNMT.com', None, 'http://www.coiffeurssalon.com', None, 'http://www.ganleywestsidehyundai.com', 'http://www.marcsimonlaw.com', None, 'https://www.lamaithaicuisineaz.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=1943&localechange=1', None, 'http://www.currentdentallasvegas.com', 'http://www.coupebizzarre.com', None, 'http://hallfamilyfarm.com/', 'http://www.ohiyo68.com', 'http://www.knuckleheadstobacco.com', 'https://www.livingspaces.com/stores/phoenix', None, 'http://aldi.us/index_ENU_HTML.htm', 'https://www.prestigecleaners.com', '1592180331', 'http://www.allwashedwindowcleaning.com', 'http://www.chiropractichealthmatthews.com/', 'https://www.phxrisingfc.com/stadium-information', 'https://www.ritasice.com', 'http://logansroadhouse.com', 'http://bluetruckwindowcleaning.com', 'https://restaurants.applebees.com/az/tempe-88083.html?utm_campaign=Local&utm_medium=Display&utm_source=Yelp', 'http://moonlitetavern.com', 'http://www.fiveguys.com', None, 'http://www.HealthyOutlookFamilyMedicine.com', 'http://www.fresh52.com', 'http://www.meckneurology.com', 'https://www.erieinsurance.com/about-us/locations/charlotte', 'https://www.rosewoodcandlebar.com', 'http://www.lvzendog.com', None, 'http://www.centralazsupply.com', 'http://www.costco.com/warehouse-locations/henderson-nv-673.html', 'http://www.coldstonecreamery.com/stores/20030', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, None, 'https://www.revampextensions.com', 'https://breezemassage.wixsite.com/85301', 'http://www.costcutters.com', None, 'http://www.rodefshalom.org', 'http://www.haychiro.com', None, 'http://www.nailsupspahuntersville.com', 'http://fountainhillsbowling.com', '1592180403', 'http://azmrfixit.com/', 'https://probateattorneyvegas.com', 'http://www.siegelsuites.com/rent-apartment-las-vegas/paradise-las-vegas-apartments.php', None, None, 'http://www.shopthemensroom.com', None, None, '1592180420', 'http://www.lilbaci.com', 'http://www.toronto.ca/parks/prd/facilities/complex/1099/index.htm', None, None, 'https://ezautosaleslv.com', 'http://www.dmscshea.com', 'https://haircutmenbillgreenpittsburghpa.com', 'http://www.dentalcaretoronto.com', None, None, '1592180441', 'http://www.mirrorbombstudio.com', 'http://www.2020visionphotos.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YULMV', None, 'https://www.tgifridays.com/locations/1110/fridays-robinson-pennsylvania', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYZTS', 'http://www.annalakshmi.ca', 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://lvcraftshows.com', None, 'http://ktran1401.wix.com/pamperednailsalon', None, None, 'http://www.poolman.com', 'https://www.travelex.com/stores/phoenix-sky-harbour-international-airport-terminal-4-checkpoint-b-phoenix-az/7283', None, 'http://oldnavy.gap.com/customerService/store.do?storeid=6528', None, None, 'https://www.cantina15eleven.com', None, None, 'http://locations.truereligion.com/az/scottsdale/51049/', 'http://www.nolldentistry.com', 'http://www.annexales.com', 'http://dekra.us/en/nevada/las-vegas-6020-n-decatur-blvd/', 'http://www.swanksalon.info', 'https://visionsource-loepervisionteam.com', 'http://hoopssportsbar.com', 'http://www.ArgonPest.com', None, None, None, 'http://www.arizonainterlocks.com/tempe', 'http://www.MellonsPub.com', 'http://arichtreatments.com', 'http://www.deserttailsanimalclinic.com', 'http://www.billsbarandburger.com/venues/pittsburgh/', '1592180530', 'http://www.hcxcharlotte.com/carmel-commons-hair-salon/', 'https://bignons.com/s-tryon-location', '1592180537', 'https://nothingbundtcakes.com/bakery/NC/charlotte?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=charlotte_145', None, 'https://stjudesranch.org/', 'http://www.graciebarra.com/phoenix-az', 'https://www.outback.com/locations/oh/cuyahoga-falls', 'http://www.pizzapizza.ca', 'http://www.eatatfellinis.com/', 'http://www.santurbanospa.com', 'http://www.theartofshaving.com', None, None, 'http://www.sugarpeony.com', 'https://www.lifecelebrationpetcrematory.com', 'http://www.hassanphoto.com', 'https://www.littlejohnslawns.com/', 'http://www.tiptoptrough.com', None, None, None, 'https://www.sephora.com/happening/stores/pittsburgh-shadyside', 'http://clevelandcinemas.com/location/6347/Chagrin-Cinemas-Showtimes', 'http://www.lifetimeeyecarecenter.net', None, '1592180585', 'http://www.wetseal.com', 'http://www4.ncsu.edu/~jtdemars/theshakeshack.html/', 'http://www.usfries.com/', 'https://www.carx.com/madison/5633-odana-rd/', 'http://www.ihg.com/staybridge/hotels/us/en/madison/msnhc/hoteldetail', 'https://siennajewelsalon.com', 'https://andersonsdelivery.com', 'http://www.azwood.com/', None, '1592180608', 'https://www.sunbaraz.com', 'http://www.splishsplashpools.com', 'http://www.swisschalet.com', 'http://www.hutchsbicyclegarage.com', None, 'https://www.levinfurniture.com/furniture-store/oh/solon/6130-kruse-drive', 'http://waternicetempe.com', 'http://lafontanagelati.com/', None, 'https://www.honorhealth.com/locations/primary-care/honorhealth-medical-group-jomax?utm_source=yelp&utm_medium=local&utm_campaign=honorhealth-medical-group-jomax', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', '1592180634', 'http://www.mezes.com', 'https://www.crystalglass.ca/location/calgary-south', None, None, 'http://bikebarnaz.com/', None, None, 'http://www.tashabradyphotography.com', None, 'http://www.sunrisecarpetupholsterycleaning.com/', '1592180654', 'http://www.geniecarwash.com', 'http://www.phoenixlocksmithoncall.com', 'http://www.bronzfx.com', None, 'http://www.urbancurls416.com', 'http://www.einsteinbros.com/', 'https://www.chick-fil-a.com/surprisemarketplace', None, 'http://www.andrewblairs.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.caferio.com/', 'http://spencerstv.com', 'http://www.cuttlefishscottsdale.com/', 'https://www.terragreenhouses.com/terra-milton-garden-centre', 'http://www.performanceext.com', None, None, 'http://charlottenc.gov/cats/rail/lynx-blue-line/Pages/parkwood-station.aspx', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.cinemark.com/nevada/century-16-south-point-and-xd', 'http://www.bgchenderson.org', 'http://popeyeschicken.ca', None, 'http://www.shanghaiwokbyby.com', 'https://www.phoenixmechanicshop.com', 'http://www.mysichuanpalace.com', None, 'http://www.lowes.com', 'http://www.extremepestcontrolLasVegas.com', 'https://www.questdiagnostics.com/home/patients?utm_source=yelp&utm_medium=social&utm_campaign=pscprofiles&utm_term=psc&utm_content=patients', None, 'http://www.sidernobakeryllc.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'https://www.wolbertautobody.com', 'http://www.mscarolsgymkids.com', None, 'https://www.redlobster.com/locations/list/nc/concord/8012-concord-mills-blvd', 'http://www.crazydaisieslv.com', 'http://abbeyroseflorist.com', 'https://positive-movement-logistics.business.site', 'http://www.delaneyauction.com', None, 'http://www.freakstattooemporiumnv.com', 'https://www.oxifresh.com/locations/nc/charlotte-carpet-cleaning/?utm_source=yelp&utm_medium=social&promo_code=YP5000', None, None, None, 'http://www.alnoorkebab.com', None, 'http://www.avalondentistry.ca/', 'http://jwoodsonderm.com', None, None, 'http://www.palmcleaningsystems.com/wp/', None, None, None, 'http://www.innovativevapors.com', 'http://www.naturalbeautyhairboutique.com', 'http://www.bethelbarbershop.com', 'http://www.livescanlasvegas.com', None, 'https://hotelxtoronto.com/en/falcon-skybar.html', 'http://www.danielsdiscount.com', 'http://www.thesmilecreators.com', None, None, None, None, 'http://www.koolthairestaurant.com', None, None, 'http://www.cowboycooling.com', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', '1592180808', 'http://www.jimpeplinski.ca', 'https://www.avis.ca/car-rental/location/CAU/CA/PQ/Dollard-des-Ormeaux/PCQ', 'http://www.vasilis-yyc.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.avecpanacherestaurant.com', 'http://www.shell.com', 'https://www.creativekidslearningcenter.com/schools/las-vegas-nv-6705/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhancedprofile', None, None, '1592180834', 'http://www.eatrastapasta.ca', 'http://www.chmuseums.org', 'http://www.settlersridgepa.com/', 'http://www.rightgear.net', 'http://www.bodyrevolution.net', 'https://istanbulsofra.com', 'http://www.3riversartsfest.org/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/32331+North+Scottsdale+Road-Scottsdale-AZ-85262/storeid=7839?cid=ps_ylp_2020', None, None, 'http://www.nvcoin.com', 'http://specialtyrvaz.com', 'http://www.azultimatechiropractic.com', 'http://www.moldanddefectlawyer.com', None, 'http://www.ultrasmoothskin.com/', 'http://www.tbmelectriclasvegas.com/', None, None, None, 'http://www.imperialpalace.com/casinos/imperial-palace/restaurants-dining/ginseng-3-detail.html', 'http://www.ventureoutrvresort.com', None, None, 'http://smittysresidentialpainting.com', 'http://royalthaihakka.com', 'http://www.instantac.com', None, 'http://www.rawfitnesslv.com/summerlin.html', 'https://calgarymovers.net', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89084/600-in-north-las-vegas/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-600&utm_content=website-link&howheard=IYE', None, None, 'http://www.cobrinhabjjlasvegas.com', 'http://www.rooterranger.com', 'https://www.ladym.com', 'http://www.doubledouble.ca', 'http://www.orleanscasino.com/entertain/bourbon-street-cabaret?disid=wmm_yelp_to_corporate_bourbonstreetcabaret_organiclink', None, 'http://pawncorp.com', '1592180934', 'http://www.visionexpress.ca', 'http://www.mashawiaz.com/', 'https://www.pearsoneyecare.com/location/superstition-springs-mall-mesa/', 'https://trivenisupermarket.com', 'https://leader-barber-style-shop.business.site', 'http://www.unclesmoke.com', None, 'https://www.lasvegasbouquet.com', None, 'http://sachisalon.com', 'http://www.bixelsstudios.com', 'http://www.hairsalonlulu.com', None, 'http://www.thriftypropane.com/', 'http://www.balloonatix.com', None, None, 'http://www.starbucks.com/store/9298/', 'http://mississaugaemission.ca', 'http://www.dirtyat1230.com', None, 'http://www.11BravoMobileAuto.com', 'http://www.lagoeastbank.com', None, 'http://www.vitabellasalonandspa.com/', None, 'http://www.climatechangeyyc.com', 'http://fingerprintingink.com', 'http://www.oudejansinteriors.com', None, 'http://www.japco.ca', None, '1592181015', 'http://www.explace.on.ca', 'http://www.kensautorepair-hub.com', 'https://shop.nordstrom.com/store-details/nordstrom-scottsdale-fashion-square?utm_source=yelp&utm_medium=organic&utm_campaign=fls&utm_content=380&utm_channel=lcl_brand_p&sp_source=yelp&sp_campaign=fls', 'http://www.sfagentonline.com?cmpid=FVUJ_BLM_0006', None, 'https://www.aqua-tots.com/ahwatukee/', 'https://myhoneybakedstore.com/charlotte/114', None, 'http://phoenix.gov/parks', None, None, None, 'http://www.dryaged.ca', None, 'http://www.cpk.com/Location/Details/Ross-Park', None, 'http://www.chavezboxinggym.com', 'http://www.curvyconsignments.com', None, None, 'http://www.swlegalgrp.com/', None, None, 'http://www.superstore.ca', None, 'http://www.luxor.com/amenities/amenities_the_spa.aspx', 'http://chucksroadhouse.com', None, None, 'https://www.restaurant-mesa.com', 'http://www.gap.com/products/westlake-oh-store-7181.jsp', 'http://www.gotitcoverednv.com', None, 'http://www.ihg.com/holidayinnexpress/hotels/us/en/mentor/mntoh/hoteldetail', None, None, None, None, 'http://nanadeesdiner.com', 'http://shopping.schubachstore.com/', None, None, 'https://aarons-ark-mobile-veterinary-services.business.site', None, 'http://www.eldoradocasino.com/dining.cfm', 'http://www.commutehome.com/', None, 'http://www.badgergymnastics.com', 'http://www.hightvet.com', None, 'https://www.wedgewoodweddings.com/venues/arizona/palm-valley?utm_source=yelp&utm_medium=social&utm_campaign=palm-valley-yelp', 'http://Inglewoodpizza.ca', 'http://www.gianteagle.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.boulangerielefournil.com/', 'https://www.walmart.com/store/4148/charlotte-nc/details', 'http://www.byrdspub.com', None, 'http://www.headwaystudio.com', 'http://www.bronzecafe.com', None, 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'https://www.choicehotels.com/arizona/phoenix/sleep-inn-hotels/az140/rates', 'http://www.alcornhair.com', 'http://www.topshotlasvegas.com', None, 'http://www.bigmouthluckdumplinghouse.com/', 'http://www.matsudasushi.com/', None, '1592181177', 'http://nile-seafood-market-restaurant.business.site', 'http://versatileautoappearance.com', 'http://www.pizzahut.com/', 'http://www.walmart.ca/en/richmond-hill-supercentre/3195', None, 'https://attorneysforfreedom.com', 'https://havensmedispa.com', 'http://www.bannerhealth.com/locations/glendale/banner-physical-therapy-eugie?utm_source=GMB&utm_medium=Listing&utm_campaign=PT%20WebCTA', 'http://www.allproawninglv.com', None, 'http://www.woodbinetoyota.ca', None, 'http://www.ihg.com/holidayinnexpress/hotels/us/en/calgary/yyccg/hoteldetail', 'https://sukiesbridal.com', None, 'http://green-pepper-ot-xanh-asian-cuisine.business.site', 'http://www.sodadispenserdepot.com', 'http://www.7-eleven.com/?yelp=36121', 'http://www.medusaslv.com', 'http://www.cozzeria-pizzeria.com', 'https://www.alanhuirealty.com', 'http://www.clevelanderbar.com/', 'http://www.streamersicecream.com/', 'http://www.fraziersbarbershop.com', None, 'http://www.mufflermasters.com', None, 'https://functionbar.ca', 'http://worldsalon.ca', 'http://www.PetLossAtHome.com', 'http://www.summerlindance.com/', 'http://voxcurvy.com', 'http://www.paintingbyhector.com', None, None, 'https://marsautoservice.net', 'https://www.wyndhamhotels.com/hotel/08797?tel=18007606483&iata=00093785&cid=fe:tl:20170523:yl:pp:tlus:08797', None, 'https://locations.paychex.com/us/pennsylvania/pittsburgh', 'http://www.ymcacharlotte.org', 'https://www.chandlerair.com', 'http://www.poshmommy.ca', '1592181276', None, 'http://www.beancurd.biz', 'http://hummusfactorylv.com', 'http://gilberttowncommons.com/?utm_campaign=YelpWebsite&utm_medium=YelpBusinessProfile&utm_source=Yelp', 'http://www.starloancenters.com', 'http://www.pnc.com', 'http://skincarecleveland.com/index.html', None, 'https://www.kalologie-lasvegas.com', None, None, 'http://www.westlibertycycles.com', 'http://www.marriott.com/hotels/travel/phxcs-springhill-suites-phoenix-chandler-fashion-center/', 'http://Chagrin-Falls.certapro.com/?source=Yelp', 'http://www.keepitcut.com', 'https://www.lugayfamilydental.com', 'http://k-momo.com', 'http://www.executivelounges.com', 'http://www.trulyengravables.com', 'http://www.ecodryrestoration.com', 'https://www.target.com/sl/arcadia-crossing/950', 'http://www.hobbylobby.com', 'http://www.hubnest.com/', 'http://fandombar.com', None, None, None, 'http://www.gianteagle.com', 'https://www.lasvegas-propertymanagement.com', 'http://www.boxheartgallery.com', 'http://corneaaz.com', 'http://www.aveda.com', 'http://www.dralbertli.com', 'http://www.thegatewaybarandgrill.com/', 'http://www.meckabc.com', None, 'http://sugarshackrecords.net', 'http://www.starbucks.com/store/89998/ca/vmr-lucerne/2362-chemin-lucerne-ville-mont-royal-qc-h3r-2k5', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', 'http://www.bennysbar.net', 'http://www.ringaudio.com', None, 'http://dlautomotive.com', 'http://401dixieinfiniti.ca/', 'http://www.operationhalloween.com', 'http://www.fetchpetcare.com', 'http://bluebirdcarpet.com', 'http://auclv.vegas', 'http://www.coachcanada.com/coachcanada/ss.casino.asp', 'http://www.novellenail.com', None, 'https://ad.doubleclick.net/ddm/clk/414776926;215768644;y', 'https://www.apexskin.com/locations/solon-oh-dermatology-office/', 'https://vyiannibas.wixsite.com/juliana', '1592181407', None, 'https://ca.louisvuitton.com/fra-ca/magasin/canada/louis-vuitton-montreal-ogilvy?campaign=b_yelp/storepage', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700007898397760&ds_s_kwgid=58700000649767852&ds_e_adid=%7Bcreative%7D&ds_url_v=2&cmpid=ADV_PG_%5B*KeywordID*%5D', 'https://Scottsdalequarter.com', 'http://www.healthpalace.ca', 'http://www.aamcogilbertchandler.com', None, 'https://www.hungryhowies.com/store/hungry-howies-01823?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1823&utm_content=Website_Link', None, 'http://www.generate.to', None, 'http://campagnolotoronto.com/', '1592181442', 'https://www.24hourfitness.com/gyms/las-vegas-nv/silverado-las-vegas-sport', 'http://www.captainscovenc.net/', 'http://sanctuaryongreen.com/', 'http://www.nulawn.net', 'https://www.simonmed.com/locations/all/simonmed_imaging___greenfield', None, None, None, 'http://www.papajohns.com', None, 'http://restaurantgalleria.ca/fr/', None, 'http://www.sbcfoods.com/', 'https://freshfixofheights.com', 'http://www.myautoamerica.com', None, 'http://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLEEM', 'http://www.fonda-las-cazuelas.com', 'http://zaxe.ca', 'http://www.jmphoenixcriminalattorney.com/', None, 'http://www.aztrailstravel.com', 'http://www.allstartaxi.ca', 'http://www.autobell.com/ab_locations/333-n-new-hope-rd/', 'https://stopandshop.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.bergamotmassage.com', 'http://rossispizzamonona.com/', 'http://www.lakeeriemonsters.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Las-Vegas-NV-89128/027192/', 'http://go.georgestreetphoto.com/l/9752/2014-06-04/m1c3t', None, 'http://www.danspoolcare.com', 'http://www.earlycollegenv.com', 'https://www.oliversplumbing.com', 'https://www.lowes.com/store/AZ-Surprise/1607?cm_mmc=YEXT-_-SiteLink-_-1607-_-Lowes', 'https://stores.guitarcenter.com/avondale?source=4GOA5AJBB', 'https://www.perfectteeth.com/office/az/surprise/85379/waddell/', 'http://www.conciergemakeup.com', None, 'http://soilextractors.com', 'http://greengarageaz.com', '1592181555', None, 'https://www.apparelprousa.com', None, 'http://www.adamahmadison.com', None, None, 'http://www.dieselworldwide.ca', 'http://heritagetattoolv.com', None, 'http://www.phoeniximprovfestival.com', None, 'https://www.cosmeticlasercenters.com', None, 'http://www.zapitwash.com', 'http://therocksportsbarandgrille.com', 'https://www.sprint.com/storefronts/bd/sprint-phoenix-az-85037-d-4610/', None, 'http://www.taverninthehills.com', 'https://dream-cut-barber-and-stylist.business.site', None, None, 'http://www.solsticesunglasses.com/', None, None, 'http://www.patisseriefleur.com', '1592181622', 'http://locations.michaels.com/AB/CALGARY/3902/', 'https://www.bestbuy.ca/en-CA/stores/store-locator.aspx?StoreID=62', 'http://www.kristenmarieweddings.com', 'https://www.eatnpark.com', 'http://www.middletonpower.com/', 'http://www.lakeviewautopartsinc.com', 'http://www.avedamontreal-lifestyle.com', 'https://www.laplazataqueria.com', 'http://discountfloorsaz.com', 'https://www.bannerhealth.com/locations/mesa/banner-urgent-care-1', 'https://www.shopfuego.com', None, 'https://www.labl.ca', None, None, 'http://www.cortezindustriesllc.com', None, 'http://www.skilletsforbreakfast.com/', None, None, 'http://lvfurnituredirect.com', 'https://www.petalsofloveflorist.com', 'https://www.aairofcharlotte.com', 'http://launchhouse.com/', None, None, 'http://www.friendlys.com/', None, 'http://american1stheating.com', 'http://chase.com', '1592181697', 'http://www.bcbg.com', 'http://www.signaturerentalsbyowner.com', 'http://www.traditionalstitches.com/', 'https://www.supercuts.com/locations/az/peoria/north-valley-power-center-haircuts-82958.html', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.mixxiemixologist.com', 'http://carolinawinesandspirits.com', None, 'http://www.altf.com', 'http://www.classiccarspa.com', None, 'https://palmbeachtan.com/locations/AZ/Tempe-East-Broadway-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://www.trustedepa.com', None, '1592181735', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', '1592181744', 'https://www.activedecorating.com', 'http://www.TheWomansCenter.com', None, 'http://moderneraweddings.com', 'http://www.mrdetailohio.com', 'http://winginout.com', 'http://www.bigotires.com', 'https://superiorplay.com/dealerlocator/north-carolina/charlotte', 'http://www.bk.com', 'http://www.lasvegasvasectomy.com', 'http://www.wildbunchdesertguides.com', '1592181773', None, None, 'http://avonurgentcare.net/', None, None, 'http://www.mimiscafe.com', 'http://tenrenstea.com/start/', None, 'https://santafestation.sclv.com/gaming/sports-book?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.look-stylesociety.com', 'http://www.callsp.com', 'http://www.datadoctors.com/locations/101', 'https://cbdemporium.com', 'http://www.ledolci.com/', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.marriott.com/hotels/hotel-information/fitness-spa-services/details/phxcb-jw-marriott-scottsdale-camelback-inn-resort-and-spa/5014642/', 'http://www.cocokala-lasvegas.com', None, None, None, '1592181829', None, 'http://www.twintranslations.com', 'http://www.target.ca', None, None, 'http://www.proximityatpapago.com/index.aspx', 'https://www.massageaddict.ca/calgary-sagehillcrossing/', 'http://www.donaldjpliner.com', None, 'http://bombatacos.com/fairlawn-oh', 'https://www.vitbeo.com', 'http://lanierparking.com/', 'http://SellWithOne.com', 'http://www.phoenixtitleloan.com/', 'https://www.eyesonhayden.com', 'http://CATCh.theater', 'http://www.goldenharboronline.com', 'https://www.kohls.com/stores/az/gilbert-621.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.nuturfofarizona.com', 'http://www.topgearcarwash.net', 'http://www.sansotei.com', None, 'https://www.walgreens.com/locator/walgreens-2324+w+war+memorial+dr-peoria-il-61614/id=6408', None, None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.bestbuyflooringcenter.com', 'http://www.flowerchildvintage.com/', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', None, 'http://www.theglassguru.com/scottsdale/', 'http://www.clevelandrockgym.com', 'https://jessebrowns.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://www.colossalcupcakes.com', 'http://www.phoenixpediatrics.com', 'http://www.klearlykleaned.com', 'http://811kitchenbarlounge.com', 'http://www.blacklanterntattoo.com', None, 'http://www.dedicatedaudio.com', 'http://www.flipsideburger.com', None, 'http://www.teamorthodontics.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.appleone.com/?yelp_track', 'http://www.confectionarycupboard.com/', 'http://www.neworientalrecipes.com', None, None, None, 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', None, 'http://www.beckhamspub.com', 'http://www.7-eleven.com/?yelp=29644', 'http://www.goldentrianglebike.com', None, 'http://www.wildsagert8.com/', None, 'https://www.grifolsplasma.com/en/-/mesa-az', 'http://www.handsoncharlotte.org', 'https://blackbeardiner.com/location/las-vegas-tropicana/', 'http://www.resumerick.com', 'https://locations.davidstea.com/on-thornhill-17', 'http://www.trinitybellwoods.ca/', 'http://www.threes-company.com', None, 'http://www.braidyjewelers.com', 'https://acrirlty.com', None, 'http://popeyeschicken.ca', 'http://www.aactionmovers.com', 'http://www.lovhairbymarianne.com', 'http://www.pinklime.com', 'http://www.hanshiktaco.com/', 'http://www.littlecaesars.com/', 'https://local.firestonecompleteautocare.com/nevada/las-vegas/3770-s-maryland-parkway/', None, None, 'http://www.precisionguitaraz.com', 'http://www.elitehomeinspections.us', None, 'http://www.vantageloftslv.com/#Home', None, 'https://makeupbyalmah.business.site', 'http://www.salutelounge.ca', 'https://arizonaurologyspecialists.com/contact-scottsdale-thompson-peak', 'http://www.starbucks.com/store/11823/', 'http://www.azmobilereflash.com', 'http://www.baskinrobbins.com', None, 'http://www.fye.com', 'http://www.ampm.com', None, None, 'http://www.nygh.on.ca', 'https://sheridannurseries.com/store-locations/toronto-garden-centre/', None, 'http://www.weathervaneplayhouse.com', None, 'http://www.littleotterswim.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.ebtfloral.com/', 'https://bostonpizza.com/en/locations/sunridge.html', None, 'http://www.sayteaonbloor.com', 'http://www.anewtable.ca', 'http://www.bathurstbowlerama.com/', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/109+Dabney+Drive-Henderson-NC-27536/storeid=7055?cid=ps_ylp_2020', 'http://www.lesleyscreativecakes.com', None, None, 'http://www.gandermountain.com', 'https://housedoctoraz.com', 'http://www.greatclips.com/', None, None, 'https://locations.cariboucoffee.com/az/phoenix/2080-w.-northern-ave.html', None, 'http://oriondetailing.co', 'https://business.google.com/website_shared/launch_bw.html?f=vegas-noodle-house.business.site', None, 'http://www.southernpawsanimalhospital.com', 'http://www.pnpkaz.com', 'http://www.eventprostrategies.com', 'http://www.penn-station.com/westvirginia/', '1592182138', None, 'http://www.bohme.com', 'http://www.fatzombietattoo.com', 'http://www.globalmarketretail.com', None, None, 'https://www.wyndhamhotels.com/hotel/28120?tel=18885951878&iata=00093785&cid=fe:mt:20180223:yl:pp:MTUS:28120', '1592182158', 'https://www.amctheatres.com/movie-theatres/phoenix/amc-westgate-20?utm_uid=yelp', None, 'http://Www.piedmontvets.com', 'http://www.chachago.ca', 'http://rebelluxeaz.com', 'https://www.kingdomplumbing.com', None, 'http://www.sincitypc.com', 'http://www.weprintwhatyouwant.com', None, 'https://ballantynecleaners.com/drop-store', None, None, 'http://www.fastfreshfoods.ca/', 'https://comfortcare.ca', 'http://www.calgary.ca/CSPS/Parks/Pages/Locations/SW-parks/North-Glenmore-Park.aspx', None, 'http://www.dontbebroke.com/?utm_source=Yelp&utm_medium=Location%2BPage&utm_campaign=Yelp', 'http://bobochinabuffet.com/', '1592182216', None, 'http://www.quiznos.com', 'http://www.bolocakery.com', 'http://benvenutos.com/locations/middleton', 'http://yourvalleyproperty.com', None, None, None, 'http://www.mehowey.com', 'https://www.alfredoshairsalon.com', None, 'http://www.flooptical.com/index.html', 'http://www.fineskincare.ca', 'http://www.iautocars.com', None, None, None, 'http://www.bk.com', '1592182267', 'https://www.walgreens.com/locator/walgreens-1801+philo+rd-urbana-il-61802/id=11368', None, 'http://www.closetworld.com', 'http://www.chicsincity.com', 'http://www.terribleherbst.com/', 'http://www.altardstate.com/', None, None, 'http://www.madisonpubliclibrary.org/monroe-street', None, 'http://www.corekitchenaz.com', None, 'https://stores.jcpenney.com/nv/henderson/clothing-stores-henderson-nv-2626.html', 'https://gymvmt.com', 'http://www.nynyhotelcasino.com/entertainment/bars/bar-at-times-square.aspx', 'http://www.symdonchevy.com/?cs:pro=repman&cs:e=yelp', 'http://www.dgapplianceandair.com', None, None, None, None, None, None, 'http://imageeyecarenv.com', 'http://www.mondaynightauction.com/', None, None, 'http://www.pool-spaoutlet.com', 'https://ameritekusa.net', 'http://www.nepps.net/contact-us', None, 'http://www.diamondcharlotte.com/', 'http://www.caferumba.net/', 'http://www.libertystationtavern.com', 'https://surrealnightlife.com/las-vegas/', None, None, 'http://7eleven.ca/store-locator/33259', 'http://www.thestretchdoc.com', None, 'http://www.drumandmonkey.ca/', None, 'http://www.thelivermorepgh.com', None, 'http://www.marriott.com/hotels/travel/pitel-springhill-suites-pittsburgh-bakery-square/', 'https://gelsbystormi.com', 'http://www.starbucks.com/store/9298/', 'http://www.magicwindow.ca', 'https://www.wellsfargo.com', None, 'https://www.marcosorder.com/?store=3582', None, 'http://yorktaps.ca', 'https://www.lasikmd.com/fr?utm_source=yelp&utm_medium=cpc&utm_campaign=calgary&utm_content=general-link&p_utc=YELP_paid', 'https://www.extraspace.com/storage/facilities/us/nevada/north_las_vegas/300030/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, None, 'http://www.fleetfeetcleveland.com', 'https://www.walgreens.com/locator/walgreens-8816+e+pinnacle+peak+rd-scottsdale-az-85255/id=3602', None, None, 'https://local.lenscrafters.ca/on/thornhill/one-promenade-circle.html', None, "https://www.brueggers.com/?utm_source=yelp&utm_medium=homepage&utm_campaign=mayfield_heights&utm_term=Bruegger's%20Bagels%20Home%20Page", 'https://locations.wendys.com/united-states/al/madison/8814-hwy-20-w', None, 'https://www.bathdepot.ca/bath-depot-vaughan', 'http://www.lupitafernandezins.com?cmpid=d92d_blm_0006', 'http://www.aaatowing.ca', None, 'http://www.gilbertdentalcenter.com', 'http://fix8fitnesscafe.com/', 'http://www.jaroblue.com', 'http://fairwayindependentmc.com/', 'http://www.pittsburghwinery.com', 'https://www.wow1day.com/locations/house-painting-york?utm_campaign=yelp&utm_source=york&utm_medium=ubl', None, 'https://5starautocol.com', 'http://www.metrolinatire.com', None, None, 'http://palmerslawncare.net', None, None, None, 'http://www.tiffanymaidservice.com', 'http://www.reallysmooth.com', 'http://studio-kim.business.site', None, None, 'http://www.phohoa.com/', None, 'http://softmechanic.com', 'http://mastairaz.com', None, 'http://robertostacoshop.com', 'https://www.sonoraquest.com', 'http://www.dollar.com/Locations/gen.aspx?locationId=YTOC18', 'http://www.cpllabs.com/locations/patient-service-centers/cheyenne-psc.aspx', 'http://www.nguyenhuong.ca/menus.htm', 'http://www.fashionq1982.com', None, 'http://www.maddenbros.com', '1592182541', None, 'http://TresaApts.com', 'http://www.sanriva.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281379494&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, None, 'http://www.chevron.com', 'http://www.ruelo.com', 'http://burgerking.ca', None, None, 'https://www.uptowncleveland.com', None, 'http://www.dollartree.com/home.jsp', None, 'http://aerosportsparks.ca/brampton/', 'http://hummingbirddental.ca/', 'http://www.pghtours.com', '1592182590', 'http://www.amigotireslv.com', 'http://www.majesticmemorieslv.com/', 'http://www.eliteappliance.ca/', 'http://www.timhortons.com/', 'http://www.absolute-offroad.com', 'http://locations.harristeeter.com/nc/charlotte/205/', 'http://www.deanshousestrongsville.com', 'http://www.sushiakaihana.com', None, None, 'http://www.sunocoinc.com', None, 'http://lvmpd.com/', 'http://lvdancesport.com', 'http://blasioleseatery.com', 'http://qualitycarstereoaz.com', 'http://www.sambucagrill.com', None, 'http://manhattandelimentor.com/', None, 'http://www.rockler.com/retail/stores/pa/pittsburgh-store', 'http://www.taxcomexicangrill.com', 'http://www.mobilespaelite.com/', 'https://www.walgreens.com/locator/walgreens-8500+w+cheyenne+ave-las+vegas-nv-89129/id=4197', 'http://www.bkreflex.com', 'http://Www.haringtonsplumbing.com', 'http://rjcrosby.com', None, 'http://www.coremotus.com/book-now', None, 'http://www.mathnasium.com/ocotillo', 'https://www.cleaneatz.com', None, 'http://www.gunsmithlasvegas.com', '1592182676', '1592182679', 'http://www.wachovia.com', 'http://www.secondcup.com', 'http://www.thelezzet.com', None, 'http://www.polishfestivalaz.org/', 'http://www.cmkpool.com', 'http://www.anniesgourmetitalian.info', None, 'http://www.cannabispatientcenter.com/', 'http://www.traderjoes.com', 'http://www.abcvalet.com', 'http://www.popshoppinautorepair.com', 'http://www.gnc.com', 'http://www.boldbeautyco.com', 'https://stores.thenorthface.com/WI/MADISON/USA24/', 'https://www.bannerhealth.com/locations/phoenix/banner-university-medical-center-internal-medicine', None, 'http://kimpovietnamese.com', 'http://familysbarbershop.com', 'http://www.olio.ca', None, 'http://www.dominos.com', 'http://onestopnutrition.com/', 'http://www.platinumcars.ca', 'http://www.fantasticfleamarket.com/', 'http://www.sapporopgh.com/', 'http://www.uniquedentalaz.com', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', None, None, 'http://www.theyogurtbaraz.com', 'http://www.papajohns.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/7499+West+Bethany+Home+Road+Corner+Of+75th-Glendale-AZ-85303/storeid=7872?cid=ps_ylp_2020', 'https://www.alohacleanersaz.com', 'https://www.mingsbubbletea.com', 'http://www.gokartslasvegas.com', 'https://www.brakemasters.com/location/5180-w-peoria-glendale-az/', '1592182778', None, 'http://www.cocochocolates.ca', 'http://www.showcaselandcare.com', None, 'http://www.bluewaveautospa.com', 'http://starfishandcoffeeshop.com', 'http://cohoots.com', None, 'http://shop.trisports.com/tempe', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282975370&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://www.nhmintviewobgyn.org/', None, 'http://www.stewartdetailing.com', None, 'http://bsalon11.com', 'http://www.northcoastlaser.com/', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7781&localechange=1', None, 'http://www.starbucks.com/store/9334/', 'http://kasianchiro.com', None, None, None, 'http://rtcsnv.com', None, 'http://monkeysushi.ca', '1592182939', 'https://www.bellagio.com/en/restaurants/sadelles.html', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'http://www.siegelsuites.com', 'http://www.bankofamerica.com', 'http://www.sloppykisseslv.com/', None, None, 'http://anthemhillsmusic.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.eliterepeatconsign.com/', 'https://www.zoup.com', 'http://thegymsportsbars.com', 'https://blackbeardiner.com/location/las-vegas-tropicana/', None, None, None, None, None, 'http://www.pachamamadc.com', 'http://www.econrads.com', '1592182988', None, 'https://www.jerseymikes.com/3128', 'http://www.indacutbarbershop.com/', 'http://www.pdubsbbq.com', 'http://www.bonaterrapgh.com', 'http://www.cessiespizza.com/', 'http://www.mark-taylor.com/arizona/ravenwood-heights/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.greatermatthewshabitat.org', None, 'http://www.modernchairrental.com', 'http://www.rhbarkleyhvac.com', None, None, 'http://www.riolasvegas.com/casinos/rio/restaurants-dining/hamadas-asiana-detail.html', 'http://accentricsalons.com', None, None, 'https://www.foodsocietyx.com', 'http://www.kay.com', 'https://www.eglintonsquare.ca', 'http://www.nicholascoffee.net', 'http://pinevilleplace.com', 'http://www.furry-pals.com', None, 'http://www.fantasticfades.ca', 'http://stratospherehotel.com/Food-Drink/Roxys-diner', 'http://www.villapizza.com', 'http://phoenixshoulderandknee.com/', '1592183053', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.sjnc.org/', '1592183060', None, 'http://fixautousa.com/las-vegas-north', None, 'http://www.lasvegas-cars.com', 'http://www.brimhallwellnesscenter.com', 'http://www.english.fgs2.ca/', None, None, 'https://locations.churchs.com/nc-charlotte-598', 'http://www.europeanautosalon.com', 'https://www.interdent.com/gentle-dental/locations/az/peoria/14155-n-83rd-ave-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.secondheartyoga.com', None, 'https://www.infinitiofbrampton.com', None, 'https://www.jlrichardscatering.com', 'https://madison.madhouseexperience.com', 'http://www.tempeuniversityinn.com', None, 'http://www.andersonaudiology.com', 'http://www.fionnmaccools.com', None, 'http://www.gelogisticsolutions.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.hairclub.com/locations/on/mississauga/hair-loss-clinic-228/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_228', 'http://www.hotyogatnt.com', 'https://locations.theupsstore.com/pa/mcmurray/4017-washington-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.cinemark.com/theatre-detail.aspx?node_id=97350', 'http://www.cuyahogalibrary.org/Branches/South-Euclid-Lyndhurst.aspx', 'http://www.wepoundemfishing.com', 'http://www.serviceking.com/locations/service-king-puritas?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'http://www.alsautorepair.mechanicnet.com', 'http://novabellasalon.mysalononline.com', 'http://www.cleatsinc.com/', 'https://phoenix.legacytraditional.org', 'https://beautyboxbyella.business.site', 'http://costcutters.com/salondetail/default.asp?salonid=62180', 'https://www.lafolieresto.ca', None, 'http://www.costco.com/WarehouseLocatorDetailsView?warehouseNumber=1121&storeId=10301&catalogId=10701', 'http://lakesidegrillestrella.com', 'http://www.barrecleveland.com/', 'http://www.cafemoka.us', 'http://www.midas.com/lasvegas/store.aspx?shopnum=1192&dmanum=480', 'http://www.verizonwireless.com/stores/arizona/tempe/tempe-marketplace-1088402/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://junskoreanlv.com/', 'http://www.smoothieking.com', 'http://restorecoldpressed.com', None, 'https://www.choicehotels.com/CN280?mc=blypypaf', 'http://www.earlyeducationpros.org', 'https://www.milansalonaz.com', 'http://www.tappsbar.com/', None, 'http://www.makisushi.ca', 'http://vibrantcare.com', None, 'http://www.marshallscanada.ca/en/directions.aspx?sid=741', 'http://www.thornhillskinclinic.com', 'http://www.union.lib.nc.us', '1592183182', 'http://buysellrent702.com', None, 'http://www.sugarshackaz.com', 'http://www.pizzahotwings.com', 'https://www.pizza73.com', 'http://allpages.com/', 'http://hairtiquedesignersinc.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.hunkmansion.com', 'http://www.nellos-pizza.com/', None, None, 'http://www.andrewleohairsalon.com', None, 'http://www.fullhousedesserts.ca', 'https://www.symphonywed.com', 'http://www.homesteadkennels.com', None, None, 'https://www.lasvegaseliteoffroad.com', None, None, None, 'http://www.dogworksmpg.com', 'http://www.rosarium.ca', 'http://www.piecesunimagined.com/', None, 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', None, 'http://www.groundsguys.com/madison', 'http://www.lovetowax.com', 'https://nailuxx-nail-salon.business.site', 'http://www.mooreinjurylaw.com', 'http://myfriendsrestaurant.com/site/', None, 'https://www.tillys.com/store?StoreID=0045', 'https://www.mosquito-authority.com/locations/scottsdale-az/', 'http://www.thesteammasters.com', 'https://www.thetreesurgeonpa.com', 'http://www.albertsgifts.com', None, None, 'https://www.usps.com/', 'http://www.kingorthodontics.com', None, None, 'http://tailsandscales.ca/', '1592183275', None, 'http://www.papajohns.com/', 'http://www.buonafurniture.ca', 'https://local.biglots.com/ky/henderson/190?zcp=pd_local_yelp', 'http://www.freshlysqueezed.ca', None, 'https://stores.nordstromrack.com/us/oh/westlake/30121-detroit-rd?utm_source=yelp&utm_medium=organic&utm_campaign=rack&utm_content=277&utm_channel=lcl_acq_p&sp_source=yelp&sp_campaign=rack', None, 'https://celebritytheatre.com', None, 'http://www.ryanhomes.com', 'http://www.bwmpetresort.com', 'http://www.diblasisbakery.com', None, None, 'http://clickserve.dartsearch.net/link/click?lid=43700044281380850&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.goldenspoon.com/', 'http://www.starrtreatments.com/', 'https://greatchoicechiro.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044281360228&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://cardenaslawgrouplv.com', 'http://www.coldstonecreamery.com/stores/20008', None, 'http://www.circlek.com/', 'http://www.vegasdiscountnutrition.net', 'https://islandpacificmarket.com/stores/listing/tropicana', 'http://www.madisonwindowtint.com', 'http://www.brueggers.com', None, 'http://giu.ca', None, None, 'http://www.paintingcontractorsohio.com', None, 'http://www.hmsmokeshop.com', 'http://www.sunsetboatrv.com', 'http://www.welderup.com', None, 'http://www.revolution-games.com', 'http://www.rocksidefamilydentalcare.com', 'http://www.accphx.com', 'https://locations.monro.com/pa/sewickley/209-ohio-river-blvd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'http://www.lumosarts.com', None, 'http://www.saswax.com', 'http://www.crutcherauto.net', 'http://www.puttersvegas.com', 'https://palmbeachtan.com/locations/AZ/Peoria-West-Thunderbird-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'http://clevelandhenna.com', None, None, 'http://www.sageandsandgrill.com', 'http://www.fillsgas.com', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', None, None, 'http://www.tonycordellphotography.com', '1592183388', None, 'http://www.cornerbarrestudio.com', None, 'http://www.bk.com', 'http://www.balletbarreworks.com', 'https://www.ghcscw.com/clinic/Hatchery%20Hill%20Clinic', 'https://www.la-z-boy.com', 'http://www.ecig-emporium.com', 'http://abcpoolservice.net', 'https://etobrestaurant.business.site', 'http://www.otherwideshoes.com', 'https://catchersfish.com', 'http://www.tigerkimstkd.com', 'https://www.take5oilchange.com/locations/oh/euclid-125/', None, 'http://www.esspa.net', 'http://www.allsewingmachinerepairs.com', 'https://www.sunrisepreschools.com/locations/broadway-mcclintock/?utm_source=yelp&utm_campaign=online_listing_tempe_274&utm_medium=referral&utm_term=business_url', None, 'http://planetzong.com/', 'https://www.choicehotels.com/pennsylvania/pittsburgh/comfort-inn-hotels/pa354/rates', 'http://www.starbucks.com/store/8054/', 'http://www.pillarcrowncourt.com', None, 'http://www.greenvalleyranchresort.com/dining/sushi-sake.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://CallAdvancedPro.com', 'https://www.bagelnash.ca', 'https://www.homedepot.com/l/Arrowhead-Ranch/AZ/Glendale/85308/475?cm_mmc=seo-yelp&', None, 'http://www.mattressfirm.com/', None, None, 'http://www.thepowerplant.org', '1592183453', 'http://www.donsbusbox.com', 'http://cheshirecataz.com/', 'http://www.dentistinstow.com', 'http://www.antiquecenterofstrabane.com', 'http://www.pointmanagementgroupllc.com', 'http://brightonevents.ca', 'http://www.autorepairaz.com', 'http://www.timelesshealthclinic.com', None, 'https://www.flyswoop.com', 'http://southpointcasino.com/dining/michaels-gourmet-room?cid=ypmic', None, None, None, None, 'http://www.bedfordnissan.com/', '1592183486', None, 'http://www.planetbouncepgh.com', None, None, None, 'http://www.jrlighting.com', 'http://monsoongroup.ca', '1592183503', 'http://www.familydollar.com', 'http://www.sachisalon.com', None, 'https://www.stancatos.com', 'http://www.substationii.com', None, 'http://www.ccdgp.com/', 'http://uhaul.com', 'http://wienerschnitzel.com/', 'https://www.wherefamiliesgrow.com', 'http://www.corksplayhousesquare.com', 'http://www.tempedogs.com', None, 'http://www.blowit.ca', None, 'http://www.thewokker.com', None, 'http://www.gianteagle.com', 'http://www.calgarybikram.com', 'http://www.fortresscreditpro.com/', 'http://www.honeymee.com', 'http://schnitzel.ca/', 'http://www.freshens.com', None, None, 'http://www.squashblossomaz.com', 'http://www.saharacenterlv.com/', None, 'http://www.unitedmayflower.com?Cpgn=Yelp', None, None, None, 'http://www.finewineandgoodspirits.com/webapp/wcs/stores/servlet/StoreCatalogDisplay?storeId=10051&catalogId=10051&langId=-1&utm_source=yelp&utm_medium=profile&utm_campaign=fwgsyelpfy17', 'http://ahwatukeehoa.com', None, 'https://www.scenariopgh.com', 'https://www.carowinds.com/things-to-do/dining', None, None, 'https://www.petsmart.com/stores/us/az/glendale-store0157.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://fastfix.com', 'http://www.starbucks.com/store/11823/', 'http://www.shapirocho.com', 'https://www.hungryhowies.com/store/hungry-howies-01824?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=1824&utm_content=Website_Link', None, 'http://www.metromoversgtanorth.ca', None, 'https://westernhonda.com', None, 'http://www.quiktrip.com', 'http://www.jeffspeakmanlv.com', 'http://www.fmentertainment.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=891&utm_content=bpas', 'https://www.gggflorist.net', None, None, None, 'http://www.birtsbistro.com', 'https://www.thepercantileandcreamery.com', 'http://www.geniusnetworkevents.com', 'https://www.esportafitness.com/Pages/clubhome.aspx?clubid=531', 'https://www.pinnaclemoderndentistry.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.glazecraze.ca', None, 'http://adfarm.mediaplex.com/ad/ck/24347-208272-18214-3', 'http://charlotteplumbing.com', 'https://stores.loft.com/outlet/us/nv/las-vegas/7400-las-vegas-blvd-south.html', 'https://www.wellsfargo.com', None, 'http://www.guess.com', '1592183651', 'http://www.therosehairstudio.com', 'http://www.cookmortgagegroup.com', None, 'http://www.LasVegasSegway.com', 'http://bennettautocare.mynapatools.com', None, None, None, 'http://www.kfc.ca/store/27-ruth-ave-brampton', None, 'http://www.lcbo.com', 'http://88noodlepapalasvegas.cafescentral.xyz', 'http://www.endurancerehab.com', 'http://www.rennsportire.ca', 'http://ladydianascakes.com/', 'https://locations.wendys.com/united-states/oh/cleveland/13939-euclid-ave.', None, 'http://www.MDA-Dentist.com/', 'https://ad.doubleclick.net/ddm/clk/300113982;127047679;f', None, 'http://www.toucheh.com', 'http://www.deathsdoorspirits.com', 'https://www.appliancepartscompany.com', 'http://www.toronto.ca/parks/island/', 'https://www.cinemark.com/arizona/cinemark-mesa-16', 'http://www.madisoninternationalspeedway.com', 'http://www.koolkatpgh.com', 'http://www.appliancesrepaircalgary.ca', None, None, None, 'http://www.reno4men.com/', None, 'http://scottsdale.house.hyatt.com/en/hotel/home.html?src=corp_yelp_businesslisting_phxxs_en_homepage_desktop&code=YB', 'http://www.servicecanada.gc.ca/tbsc-fsco/sc-dsp.jsp?lang=eng&rc=4802', 'http://ginassalonandwigs.com', None, 'http://phxnightclub.com/', '1592183735', None, 'http://www.heersmanagement.com/properties/property/del-mar-terrace-apartments', 'https://www.minutesuites.com', 'http://www.urban-baker.ca/', 'http://www.cactuslv.com', 'http://www.kandagreenhouse.com', 'http://englishhedgerow.com', 'http://www.universalcupcoffee.com', 'https://www.carmax.com/stores/7197?adcode=YLP7197&utm_campaign=LG_AFF&utm_source=LG_AFF_YLP&utm_medium=local&utm_content=shop', None, 'https://locations.dignityhealth.org/st-joseph-s-westgate-medical-center-glendale-az', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/1015+East+Ray+Road-Chandler-AZ-85225/storeid=7849?cid=ps_ylp_2020', None, None, 'http://yotapros.com/', 'https://www.giantmanhattanpizzaphoenix.com', None, None, 'http://www.masalakitchen.ca', 'http://www.zachmonroytraining.com/', None, 'http://www.livinggrace.net', 'http://homewoodsuites3.hilton.com/en/hotels/ontario/homewood-suites-by-hilton-toronto-markham-YYZHWHW/index.html?SEO_id=YELP-HW-YYZHWHW', 'https://stores.columbia.com/chandler-factory-store', 'http://www.wegotgamez1.ca', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', None, None, '1592183793', 'http://www.murphyappliance.ca', 'http://www.silkroadsushiandsteakhouse.com', 'http://www.mycdi.com/arizona/locations__services/', None, 'http://www.mypeacepreschool.com', 'https://www.inschoolwear.com', 'http://www.silverminesubs.com/', 'http://www.Arizonarvservice.com', 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'http://www.saharavegas.com/food/HouseofLords/', 'http://www.applianceservicecentre.ca', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.lesolson.com', 'http://www.phoenixpublicmarket.com/', None, None, None, None, 'http://www.theprophysiques.com/#home-top', 'http://www.eta-az.com', 'https://www.pizza73.com', None, None, 'http://radiocitypizza.com', 'http://www.yaharabay.com', None, None, 'http://www.ontariocourts.on.ca/scj/en', None, 'http://www.primroseestates.com', None, 'http://www.zonacaffe.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.jcsgrillhouse.ca/markham/', 'http://www.womensspecialtycare.com', None, 'http://www.ipetgrooming.com', None, 'http://www.perkykettle.com', None, 'http://www.tazzyandboo.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://alorestaurant.com/', None, 'http://Vegasonlinerealty.com', None, 'http://www.cynthiaschinese.com', 'https://locations.traderjoes.com/az/gilbert/258/', 'https://www.carlsjr.com/?utm_source=Yext&utm_medium=Visit%20Website&utm_campaign=Homepage', 'http://www.bestfloorcoatingsaz.com', 'http://www.scottsdalememorycare.com', 'http://www.evans-properties.com', 'http://www.lidlounge.com', 'https://www.supercuts.com/locations/az/surprise/albertsons-shopping-center-haircuts-82334.html', 'http://www.apnihavelly.com/', 'http://www.swisschalet.com', None, None, 'http://www.yangteashop.com', '1592183914', None, 'https://www.originaljoes.ca', None, None, 'http://www.BASISgoodyear.org', 'https://www.goodyear.com/en-US/tire-shop/Phoenix+AZ+85016-6808+US/000121764', 'https://www.pizzahut.ca/store-details/store-R42601/pizza-hut-Stouffville', None, 'http://www.foodlion.com', 'http://www.kegsteakhouse.com/en/locations/ON/toronto/esplanade-keg/', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', 'http://www.patelbros.com', None, 'http://blomedry.com/locations/blo-strongsville-ohio/', 'http://stylehousebeautybar.com', 'http://calgarysbestpubs.com/joyce/', None, 'http://www.artbuscoffee.com', None, 'http://www.galleriasm.com/branch/store/th', 'http://www.morcandirect.com', None, 'http://daluhairdressing.com', None, 'http://www.theencorequartet.com', 'http://www.unclebenspawnshop.com', 'https://www.olivegarden.com/locations/az/phoenix/metro-center/4410?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:phoenix-az-4410_pl:locurl_rd:1754', 'https://www.daveandbusters.com/locations/canada-oakville', 'https://www.westerndental.com/en-us/find-a-location/arizona/phoenix/13651-n-35th-ave', None, 'http://cargilleyecare.com', 'https://www.charmecochinchine.com', 'http://locations.michaels.com/nc/charlotte/5032/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://womensbookstore.com/', None, None, 'http://www.stellamiawestlake.com', 'http://cahairsalons.com', None, 'http://www.svahayogacenter.com', None, 'https://www.choicehotels.com/north-carolina/pineville/quality-inn-hotels/nc384/rates', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'https://www.theartofvapes.com', 'https://www.jetspizza.com/stores/illinois/champaign/512-s-neil-st?utm_source=Yelp-Ads&utm_medium=Yelp-Ads-SEO&utm_campaign=IL-008&utm_content=Website', 'http://www.cactuscrossfit.com', 'http://www.safeway.ca', 'http://justdogsgourmet.com', None, 'http://www.studio480thesalon.com/', None, 'https://southcharlotteservices.org', None, 'http://btnfurniturerepair.com/', 'http://www.bojangles.com', 'http://www.mississauga.ca/portal/discover/braebengolfcourse', 'http://exclusivecustomslasvegas.com', 'http://www.somersfurniture.com', 'https://no900.com', None, None, None, None, None, 'https://www.stateandmain.ca', 'https://www.ttc.ca/Subway/Stations/Pape/station.jsp', 'http://www.washwag.com', 'http://www.ritasfranchises.com/ArcadiaTowneCenter', 'http://www.loova.com', 'http://www.xpandrealty.com', '1592184055', 'http://www.petagreemobilespa.com/', 'https://www.simplyravishingsalon.com', 'http://www.dreamtherapies.com/', None, 'http://www.perrinofurniture.com', 'https://www.guaranteedrate.com/loan-expert/joecordone', 'http://www.strudelteig.com', '1592184073', 'http://www.bremec.com/BremecOnTheHeights.htm', 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.arizonaroofingcompany.com/', None, 'http://www.anthropologie.com/', '1592184083', 'http://www.firstcu.coop', 'http://www.gristle.catering', 'http://www.hartswastedspace.com', 'https://www.chilis.com', 'http://www.elementalplumbing.com', 'http://www.northmountaingrille.com', None, 'http://www.meridiancu.ca', 'http://www.elitemoversofcleveland.com', 'http://www.jackalopejacks.com/', None, 'http://www.avenueliving.ca', None, 'https://www.hudsonsrestaurant.com', 'http://www.yojuicylife.com', 'http://bloorwestcosmetics.ca/', None, 'https://locations.nekterjuicebar.com/ll/US/AZ/Phoenix/4340-Indian-School-Rd_*-Ste_-19?utm_source=yelp&utm_medium=link&utm_campaign=Indian-School&utm_content=URL', 'http://www.vintagethai.ca', None, '1592184122', 'http://www.artistsuncorkd.com', None, 'http://www.tiedemannglobe.com', 'http://saveamuttkennel.info', None, 'http://www.bedsidemanor.com/', 'http://www.silversevenscasino.com/', None, 'http://www.lobbyrestaurant.com/', None, 'http://lindeyslakehouse.com', None, 'https://www.ubreakifix.com/ca/locations/thornhill', 'http://kwikkleaning.webs.com/', 'http://www.gemoftheearth.com', 'http://www.charlottelaserdentist.com', None, None, None, 'http://www.exquisitenailsandspa.ca/', None, 'http://www.jfcsaz.org/site/c.ajIQK6NMLfJ0E/b.6211177/k.2AE8/Contact_Us.htm', 'https://tabletennisandmore.com/phx/tabletennisandmore/', None, 'http://www.7-eleven.com/?yelp=22886', None, None, 'http://www.bodytherapyaz.com', None, 'http://www.westvalleypetsitting.com', None, 'http://www.capitano.ca', 'http://www.renusskincare.com', 'http://scorpionridge.com', 'http://www.cfcgiftcard.com', 'http://www.sandovalkarate.net', 'https://abrazomedicalgroup.com/amg-parkway/robert-l-bierman-md/', 'http://www.lifeisbeautifulsalon.com', 'http://www.upnsmokepitbbq.com', 'http://www.kensingtonmakeup.com', 'http://www.saryocafe.com', None, 'https://www.interdent.com/gentle-dental/locations/az/phoenix/4910-e-chandler-blvd-dentist-office?utm_source=organic&utm_medium=yext', None, 'http://www.reposesalonandspa.net', None, None, 'http://www.xingfutang.ca', None, None, 'http://www.challengerbasic.com', 'http://www.farrarjewelers.com/', 'http://www.oakandvine.ca', None, None, 'http://northernhillsdental.com', 'http://www.madvapesindianland.com', 'http://4sonsstores.com/29', None, 'http://www.cashamerica.com/SUPERPAWN-stores/NV/LASVEGAS/LV02', 'https://www.ubreakifix.com/ca/locations/avenueroad', None, 'https://www.hamiddonair.com', 'https://only-inspections.com', 'http://www.restauranttrentina.com/', None, 'https://www.petsuppliesplus.com/Store/SC/Fort-Mill/Fort-Mill/4189', None, 'http://www.skinnywimpmoving.com', 'http://www.auntieannes.com', None, 'http://www.toshoknifearts.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216608&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://wisconsinrepair.com', None, 'http://www.valleysolaraz.com', 'https://www.walgreens.com/locator/walgreens-1159+e+southern+ave-mesa-az-85204/id=3670', 'https://www.cineplex.com/Theatre/cineplex-cinemas-winston-churchill', 'http://www.peaksathleticclub.com', 'https://www.servproscottsdale.com', 'https://www.ventitobakery.com', 'http://www.northscottsdalehyundai.com/', None, 'http://www.metrofireaz.com', None, 'http://www.wingstop.com/', 'http://www.sallybeauty.com', None, None, 'http://hrhc.lv/PinkTaco', 'http://www.addisononerinmills.com/', None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'http://www.heartpathacupuncturemadison.com/', '1592184312', 'http://www.carolinaexpressclinic.com/', 'http://www.insuranceinwhiteoak.com?cmpid=ts6n_blm_0006', 'http://www.apachewellscountryclub.com', 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/mississauga-2191-dundas-street-e.aspx', None, 'http://www.bathandbodyworks.com', None, 'http://www.bestbuy.com', 'https://iisnv.com', 'https://www.topmasterlocksmith.com', 'http://www.lowes.ca', 'http://www.primrosehuntersville.com?utm_source=yelp&utm_medium=maplistings&utm_campaign=localmaps', 'https://stores.jcpenney.com/wi/madison/clothing-stores-madison-wi-0891.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-0891', 'http://www.mcdonalds.com', 'https://1stpetvet.com/locations/north-valley-az/', None, 'http://www.rxboilerroom.com', None, 'http://www.brakemasters.com', 'http://www.familydollar.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://liasgardenbanquethall.com', 'https://www.menswearhouse.com/store-locator/2505?utm_source=yelp&utm_medium=listing&utm_content=2505&utm_campaign=AZ', 'http://freshmexican.wix.com/hermanos', 'http://www.miramaestudio.com/#!services/c166e', None, None, 'http://www.harrisgrill.com', 'http://www.tiffany.ca/', 'http://www.saloonofmtlebanon.com', 'http://priorityonefit.com', None, 'http://www.miseryinktattoobodypiercing.com/', 'http://hongto.weebly.com/', 'https://solsouthwestkitchen.com/#locations', 'http://nexusrehabspine.com/phone/index.html', 'http://www.vivaladivalv.com', 'http://www.eastcoastweapons.com/', 'http://www.arizonaglasspros.com', None, 'https://www.fishertools.com', None, 'http://panzapiena.ca', None, 'http://www.sunocoinc.com', 'http://redwagondiner.com/', None, 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', '1592184420', None, 'http://local.firestonecompleteautocare.com/arizona/peoria/10661-n-84th-ave/?lw_cmp=IYP_YPC_MLP_11932', 'http://www.starbucks.com', 'http://www.thenailroomlv.com', 'http://www.nunziospizzeria.com', None, 'https://shopmarketandco.com/merchant/seed-to-sausage/', 'http://www.clarkcountynv.gov/Depts/parks/Services/pages/picnic-area-reservations.aspx', 'http://www.azstudentmovers.com', 'https://www.itcu.org/about/locations/green-valley', 'http://www.runningroom.com/hm/inside.php?id=2332', None, 'http://www.pareagyros.com', 'http://paladinautomotivelasvegas.com', 'http://automaticgaragedoorcompany.com', 'http://www.azcentral.com', 'http://www.gotourscanada.com', 'http://commercialcenterdistrict.com', 'https://thesaguaro.com/scottsdale/#hotel-spa', 'http://www.einsteinbros.com', 'https://www.desertstateinspections.com', 'http://local.firestonecompleteautocare.com/north-carolina/charlotte/5340-e-independence-blvd/?lw_cmp=IYP_YPC_MLP_10790', 'http://www.clarkcountynv.gov/Depts/parks/Pages/clark-county-museum.aspx', 'http://www.goodlifefitness.com', 'http://www.eSMIL.com', None, 'https://moxies.com/restaurant-locations/ontario/richmond-hill-beaver-creek', 'https://twomaidsgilbert.com', None, 'https://my.clevelandclinic.org/locations_directions/regional-locations/avon-lake-fhc', None, None, 'https://www.redlobster.ca/locations/list/on/toronto/20-dundas-street-west', 'http://www.dvfmaz.net', None, 'http://thevaporloft.com/', 'http://www.dineoncampus.com/UNCCharlotte/', 'http://www.celebrationsbarandgrill.com', 'http://heyrestaurant.tumblr.com/', 'http://www.dunkindonuts.com', None, None, None, None, 'http://www.clarkcountynv.gov/depts/parks/Pages/parkslocator.aspx', 'http://www.dillonsontopofcentral.com/', 'http://www.omegahomeimprovements.com', 'http://talleyproperties.com', None, 'http://www.chandelier-hair.com', 'http://www.serenityflorals.com', 'http://hbsalonnc.com', 'https://www.crawfordsautoservice.com', 'http://www.newsons.ca', 'https://www.tempevistabyavanti.com', None, 'http://www.leons.ca/', 'https://www.giuseppeson28th.com', 'https://diamondlaw.ca/toronto', None, None, 'https://www.greulichs.com', None, None, 'http://www.scottsdaleskininstitute.com/', 'https://germancarspecialist.com', 'https://locations.wendys.com/united-states/pa/swissvale/1703-s.-braddock-ave', 'http://www.greekvillagegrille.com/', 'http://www.neptunehairstyling.com', 'http://www.dollartree.com', None, 'http://www.trucksonlysales.com', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Calgary-AB-T2Z4M8/058072/', 'http://www.karenshairdesigns.com/', 'http://mothernaturesdiapers.com/', 'http://www.arizdental.com', 'http://www.johnnysonfulton.com', 'https://agents.allstate.com/adam-brown-las-vegas-nv-14515251.html', 'http://www.lasvegasdiscountbeds.com', 'http://www.cactuspools.com/', 'http://www.skytecsecurity.com/', 'http://paintingcontractorglendale.com', 'http://www.carters.com/home?id=carters&cm_mmc=Retail-_-Carters-_-Yelp-_-April', None, 'http://azempiredoors.com', 'https://www.millers.com', None, 'https://thegoatbarandgrill.com', None, 'https://davids-diner.business.site', 'http://www.clevelandart.org', None, 'http://theclevelandarcade.com/', 'https://locations.jackinthebox.com/us/az/chandler/900-n-cooper-rd', 'http://www.gap.com', 'http://www.janetscustomsewing.com', '1592184613', 'http://xtremecleanmobile.com', 'http://sfsourdougheatery.com/', 'http://www.randitas.com', 'http://steliotesdentalspa.com/', None, 'http://www.sushisushi.ca/', None, 'http://www.duzz.ca', None, None, 'https://arizona.weidner.com/apartments/az/phoenix/skyline-lofts-0/?rccustomid=Mjc%3d-uldpF8AWDTo%3d', 'https://dealer.sprint.com/experts-choice', 'http://www.aztecatowingarizona.com', 'http://mannysdiscount.com', None, None, 'http://www.sterlingsteelecreek.com', 'https://www.nofrills.ca/store-locator/Dennis%27+nofrills+%28Erin+Mills+Parkway%29/0700?storeId=0700', 'http://www.libertyfordbrunswick.com/', 'http://pro-techtransmission.com', 'http://www.cozumelrestaurante.com', 'https://www.choicehotels.com/pennsylvania/pittsburgh/quality-inn-hotels/pa476?source=pmfyelp&pmf=yelp', 'http://www.scottmillerdds.com/', 'https://www.escapecampervans.com/locations/las-vegas-nevada/', 'http://automotiverepairphoenix.com', None, 'http://www.bathandbodyworks.com', None, None, None, 'http://www.a1qualitycare.com/', 'http://madisongaragedoorservices.com', 'http://local.firestonecompleteautocare.com/arizona/mesa/1553-s-crismon-rd/?lw_cmp=IYP_YPC_MLP_784290', 'http://petstellthetruth.com/', 'https://www.circlek.com/store-locator/us/chandler/1405-n-alma-school-rd/2705973', 'https://www.dentistryatgreenway.com', 'http://duckyschampaign.com', 'http://www.paddyspourhouse.com', None, 'http://www.saffrontakeout.ca', 'http://www.overflowcafe.com', 'https://www.bankofamerica.com', 'http://msquarecoffeeco.com/', 'http://Clearqualityautoglass.com', 'http://www.nicolebodyworks.com', 'https://www.homedepot.com/l/Avon-Vail/CO/Avon/81620/1525?cm_mmc=seo-yelp&', '1592184707', None, None, 'http://99only.com/store/Phoenix---12th-St./', 'http://www.oasispediatrics.com', 'http://www.krispykreme.com', 'http://www.calgaryscrapautoremoval.com', None, None, None, 'https://jamesgoodnow.com', 'http://www.nps.gov/lake/index.htm', 'http://www.aiellospizzeriallc.com', None, 'https://local.fedex.com/az/glendale/office-1471/', 'http://pecoslawgroup.com', 'https://www.handlebarj.com', 'http://www.courtoneaz.com/', 'http://www.desertrad.com', 'http://trubliss.com', 'http://hotduckaz.com', 'http://www.starbucks.com/store/14887/', 'https://glassdoctor.com/henderson', 'http://www.momdocwfw.com/scottsdale', 'https://www.calforex.com/en/locations/foreign-exchange-vaughan-on-vaughan-mills/', 'http://www.truwest.org', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.designateddriversinc.com', None, 'https://f45training.com/southwestlasvegas/home', None, None, 'http://bellavitaitaliangrille.com/', None, 'http://www.mark-taylor.com/arizona/san-lagos/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.azhomedefense.com', '1592184786', None, 'http://www.hooters.com', 'http://www.atlccleaningcompanyllc.com', 'http://christinanailboutique.com', None, None, 'http://tamidairiki.md.com/', 'http://www.azallergy.com', 'http://www.DymeBoxing.com', None, 'http://www.toplimolasvegas.com', 'http://livingwellhomes.com/apartments/details/brookfield_homes', 'http://www.dermsolutionsaz.com/', 'https://wickenburg-alternative-medicine.business.site', 'http://www.lapetite.com/your-local-school/north-las-vegas-nv-7306', 'http://lasvegasstripweddings.com/', None, None, 'http://www.newamericanagent.com/steveschauer', 'http://www.ocean-prime.com/locations-menus/phoenix', None, 'http://tajindiancuisineaz.com', 'http://www.adidas.com/us/storefinder', None, '1592184835', 'http://ranchosecurity.com/', None, 'http://www.foodlion.com', 'http://www.thebeadery.ca', 'http://www.othermothersaz.com', None, 'http://www.esmokercanada.com', None, 'http://www.carmodysgrille.com', None, 'http://littlemacau.com/', 'http://AnimalCareClinicInc.com', None, None, None, 'https://www.hilton.com/en/hotels/phxruru-tru-gilbert/?SEO_id=YELP-RU-PHXRURU', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.tocasierraspa.com', 'http://www.buffalowildwings.com/?utm_source=yelp&utm_medium=yelp_profile&utm_campaign=yelp_locations', None, 'http://www.fortmcdowellcasino.com/home.php', 'http://www.pridebeautystudio.com', 'http://www.restaurantrouby.ca', 'http://sawasdeethaitogokitchen.com', 'https://mountainsidefitness.com/locations/carefree/', 'http://www.copps.com/', 'http://locations.saksoff5th.com/en/phoenix', 'http://www.canadiantire.ca/en/store-locator/ab/calgary-south-ab.store.html', 'http://www.hansonwater.com', 'http://www.gussdiner.com/', None, 'http://ladoniacafe.com/', '1592184905', None, None, 'https://www.dynacare.ca/specialpages/secondarynav/find-a-location/toronto-center-(1).aspx', 'https://www.floristinrockyriveroh.com/?srccode=yelp_track', 'http://www.menasgrill.com', 'https://gohoneyhut.com', 'https://footsolutions.com/phoenix/', None, None, None, 'http://www.minervaindiancuisine.com/', 'http://savoypgh.com/', None, 'http://www.peoples-gas.com', 'http://www.shopjustice.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'http://www.bollywood-bites.com', None, 'http://myazlawyers.com/', 'http://heartschoices.com/calgary-farmers-market/', None, 'http://www.dicey.biz', 'http://crazyburrito.ca', 'http://www.Montblanc.com', '1592184965', 'http://www.beadworldinc.com', 'http://www.lvcharleston.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, None, None, None, None, 'http://www.muchoburrito.com', 'http://www.pitaland.ca', None, None, None, 'http://www.rideabike.com', 'http://www.shicklaw.com', 'http://www.revivestainlesssteel.com', 'http://www.thebespokeman.com', 'http://www.cavecreeklock.com', 'http://www.chestermerelanding.com', 'https://www.canyonstateauto.com', 'http://6ixbar.com', 'https://www.regmovies.com/theaters/ua-showcase-theatre-8/C00542780062', None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7784&localechange=1', 'https://www.aldoshoes.com/us/en_US/store-locator/store/2103', 'http://www.kingsrestaurant.net', None, None, None, 'http://www.sarahhorne.org', None, 'https://www.rentvegas.org', 'https://cocktailantabli.com', 'http://dosgringosaz.com', 'https://www.regus.com/?utm_medium=portal&utm_source=YelpNA', None, None, None, 'https://www.fsequence.photography', 'http://www.petgroominglv.com', None, 'http://www.mississauga.ca/portal/residents/mississaugavalleylibrary', None, 'https://oakarizona.com', 'https://www.nightowlcontractors.com', None, 'http://robertostacoshop.com', 'http://gravady.com', 'http://www.humblepieusa.com/', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=14057&localechange=1', None, 'http://www.farwellpools.com', 'http://www.bin110.com', 'http://www.crocs.com', 'http://tricitycardiology.com', 'http://www.larkspurcatclinic.com/', 'https://www.sunrisechildrenshospital.com', None, 'http://www.theroofdoctorlv.com', None, 'http://www.altacamelback.com', 'https://locations.timhortons.com/ca/on/oakville/494-dundas-st-w.html', 'https://www.currys.com', 'http://perfecthomefurniture.com', None, 'https://www.abraauto.com/abra-auto-location/1058-concord-nc', 'http://www.patricksimonedds.com', 'http://www.lvendocrinology.com', 'http://www.chandlerapthomes.com', 'http://www.desirableskinaz.com', 'http://www.flooringsurfacesinc.com', 'http://www.travelgolfcenter.com', None, 'https://www.jetblue.com', 'https://www.bahamabreeze.com/locations/nv/las-vegas/las-vegas/3020?cmpid=br:bb_ag:ie_ch:dry_ca:BBQ121LOCURL_dt:20200605_sn:yelp_gt:las-vegas-nv-3020_pl:locurl_rd:1013', 'http://www.weknowurban.com/Phoenix/', 'http://www.thorncliffebowlerama.com/', 'http://www.queenspectacle.com/', 'http://www.linenchest.com', 'https://www.redrobin.com', None, None, '1592185140', 'http://lemontblancbanquets.ca', 'http://nolimitska.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.highlandfarms.ca', 'http://www.firkinpubs.com/thefrogandfirkin/', None, 'http://www.papamurphys.com/', None, None, 'http://www.dacordplumbing.com', 'https://www.championwindow.com/Cleveland/?lpc=UU-OD-Cleveland-Yelp&source=ol-yel01&utm_source=yelp&utm_medium=local&utm_campaign=Cleveland', 'http://bundtz.com', 'https://caringhandsvetlv.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://clearchoicetechnical.com/service-locations/las-vegas', 'http://www.devinetouchmassage.com', 'http://www.nirvanahome.ca/', 'http://www.robertgodridgephotography.com/', 'https://www.maidsinblue.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.enblumcatering.com', 'https://atriumhealth.org/locations/detail/carolinas-medical-center-mercy', 'http://www.gilbertazpediatricdentist.com/', 'http://www.screwielewies.com/', '1592185190', 'http://www.sapporolv.com', 'http://www.galaxyhobby.ca/', 'https://azpyramidservices.binbooker.com', None, None, None, 'https://www.modernround.com/locations-peoria.html', 'http://livingwelltakeout.net/', 'http://www.balletossington.com/', None, '1592185212', 'https://agents.farmers.com/az/chandler/joe-baselice?SourceID=AMPALFYEAG&utm_medium=Local&utm_source=Yelp', None, 'https://www.get-go.com', 'https://www.ulta.com/stores/glendale-wi-646', 'https://www.cox.com/local/residential/stores/az/gilbert/2150-east-williams-field-rd?sc_id=dp_z_yext_z_store_19Q2?sc_id=dp_z_yelp_z_store_19Q36', 'https://www.olivegarden.com/locations/az/phoenix/metro-center/4410?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:phoenix-az-4410_pl:locurl_rd:1754', None, 'http://www.passionfordesserts.ca', 'https://www.aqua-tots.com/chandler/', 'https://www.handy.com/yelp50cc?utm_source=yelp&utm_medium=CPC&utm_campaign=TOR-Handy-Yelp', 'http://www.xcash.ca', None, 'http://www.mysweetbasil.com', 'https://www.sassoon-academy.com/en/academy/north-america/toronto', None, None, 'http://www.toytownplaycenter.com', 'https://my.clevelandclinic.org/locations_directions/Regional-Locations/twinsburg-fhsc', None, 'http://www.tabbyandjacks.com', None, None, 'http://enhancedcare.ca', None, 'https://www.gianteagle.com/Stores/Store/?storeId=224', None, 'http://www.kay.com', None, 'http://www.jimshorkeyford.com/?ibp-adgroup=YX_Target', None, None, None, 'https://www.granitetransformations.com/location/granite-transformations-of-northeast-ohio/?utm_source=Yelp&utm_campaign=2018', 'http://www.greentreefootandankle.com', 'https://medmen.com/stores/las-vegas-the-strip-paradise-and-harmon/', None, None, None, None, None, 'http://www.paradisehalls.com', 'http://www.jamrockcafe.com', None, 'http://www.ocadu.ca/', 'http://www.bathandbodyworks.com', 'http://www.hollywoodtans.com', 'http://gehomeservicesllc.com', 'http://eosfitness.com/location/peoria?utm_campaign=peoria&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', 'https://www.firehousesubs.ca', 'https://nationwidevision.com/locations', 'http://www.doubledspizzas.com', 'http://www.waxandlash.ca', 'https://www.tidesonthomas.com', 'http://l.macys.com/superstition-springs-center-in-mesa-az?cm_mmc=Yelp_Stores-_-Mesa-_-n-_-459&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Mesa_459&m_ac=yelp_stores&m_ag=n', 'http://fiveguys.com/', None, 'http://www.fastlanecoatings.com', None, None, None, None, 'http://peignedancestudios.com', None, 'http://www.scottsdalesilveradogolfclub.com/', None, 'http://www.fithcc.com', 'https://tattoocorner.business.site', None, 'http://www.fransrestaurant.com', 'http://www.thelittleclinic.com', None, 'http://www.copperstatehandymanservices.com', None, 'http://www.soniasbridalstudio.com', 'http://sollidcabinetry.com', 'http://www.dukephototempeaz.com', None, 'http://www.city.cleveland.oh.us/CityofCleveland/Home/Government/', '1592185371', 'http://www.saukpointvet.com', 'https://www.duckdonuts.com/location/orange-village-oh-2/', 'https://www.starlitebbq.com', None, None, 'http://www.tartucollege.ca/', 'http://sanctuary-salon-spa.com', None, 'http://www.swignsweets.com', 'http://englishbayfish.ca', 'http://www.riderta.com/facilities/airport', None, 'http://www.arbys.com/', '1592185397', 'http://northmesaauto.com', None, 'https://www.buffalowildwings.com/en/food/?utm_source=Yelp&utm_medium=organic&utm_campaign=yelp', 'https://www.phillipslaw.com/phoenix-arizona-personal-injury-lawyers', 'http://www.momdocwfw.com/estrella', 'https://thelearningexperience.com/preschoolsuccess?utm_source=Yelp&utm_medium=Referral&utm_campaign=Sponsored+Business+Listings', 'http://www.harveyspaint.com', 'https://www.screensunlimited.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.bombayexpresstoronto.com/', None, 'http://www.dominos.com', 'http://www.beautyavenue.us', 'http://www.thethirdestimate.com/', None, None, 'http://1stchoicemechanicalaz.com', 'https://redrock.sclv.com/Dining/Grand-Cafe?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.glockmeister.com', 'http://greenjadechampaign.com', None, '1592185448', None, 'https://escapetheroom.com/phoenix/', None, 'http://www.atthebarns.org/', 'http://www.thebagelcafelv.com/', None, 'http://www.spicypickle.com/restaurants/arizona.html', None, None, 'http://www.dollarama.com', 'http://www.aaaactiondoors.com', None, None, None, 'http://www.lobtoronto.com', 'http://www.happyspizza.com', 'https://locations.comerica.com/location/75th-thunderbird', None, 'http://servitaxlv.com', 'http://www.azdot.gov/mvd', 'http://www.perinatal.com', 'https://www.imagedentallasvegas.com/?utm_source=yelp&utm_medium=link&utm_campaign=yelp', 'http://www.pghoptical.com', None, None, 'https://www.mckenziesmidtown.com', 'http://www.redsearestaurant.net', 'http://www.mallorcarestaurant.com', 'http://broadviewbeauty.ca', 'https://www.homecitybnb.com', 'https://local.fedex.com/oh/bedford-heights/LNNA/', 'http://www.guschaletrestaurant.com', None, 'http://www.tinyboxvapeshop.com', 'http://www.barrettfinancial.com', '1592185518', None, 'http://www.delanolasvegas.com/en/nightlife/skyfall-lounge.html', None, '1592185526', None, None, 'http://www.openarmspreschool.ca', 'https://www.jerseymikes.com/9004', 'https://locations.sonicdrivein.com/az/mesa/3033-e-mckellips-road.html', 'http://www.phoenixfamilymedical.com/', 'http://www.capitaldrip.com', None, None, 'http://www.handelsicecream.com', None, 'http://www.kaleidoscope-living.com', '1592185552', 'http://www.head2toechiro.com', 'http://www.eastvalleyfloors.com', 'http://www.scrapbookbarngilbert.com', None, 'http://www.nextcare.com/locations/az/peoria-glendale/?utm_source=Yelp&utm_medium=Local&utm_campaign=peoria_glendale', 'http://www.relaxologywellness.com', 'https://theoaksevents.com', 'http://www.imsiamlasvegas.com', None, 'https://www.restaurantohsushi.com', 'http://www.troplv.com/meetings-and-events/havana-room', None, 'http://www.lvms.com/bullring', 'http://www.antoniospizza.net', 'http://arbortrimming.com', None, 'https://www.venetian.com/towers/the-palazzo.html?utm_source=yelp&utm_medium=partner&utm_campaign=businesslisting', None, None, 'http://www.rajdhanisweets.ca', 'http://www.cashmanphoto.com', None, None, 'https://agent.amfam.com/nv/north-las-vegas/colleen-drinnon/', 'http://www.grandtouringautos.com/', 'https://locations.wendys.com/united-states/az/laveen/3520-w-baseline-road', 'http://www.fuzesalon.ca', 'http://gerchickrealestate.com/', 'https://greenvalleyranch.sclv.com/?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://hamptoninn3.hilton.com/en/hotels/ontario/hampton-inn-by-hilton-toronto-airport-corporate-centre-YTOSOHX/index.html?SEO_id=YELP-HP-YTOSOHX', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', None, 'http://www.everythinginsight.net/our-practice.html', None, None, None, None, None, 'http://www.petco.com', 'http://www.willietys.com/', 'http://www.sidebarphoenix.com', None, None, 'http://www.pandahobby.store', 'https://dealer.sprint.com/experts-choice', 'http://illinishuttle.com', None, 'http://www.instylenail.ca/', 'http://www.mcdonalds.com', 'http://www.clhairartstudio.com', None, None, None, 'http://www.garciaswm.com/', None, 'https://locations.einsteinbros.com/us/az/surprise/13746-w-bell-rd', 'http://www.michaelsbridalgalleria.com', 'http://www.penn-station.com', 'http://www.prmvineyards.com', 'http://Mindbodysoles.net', 'http://EvansLasVegas.com', None, None, None, 'http://www.carolinavaluevillage.com', 'https://locations.whataburger.com/az/scottsdale/2955-n-hayden-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', 'http://www.coolclubs.com', 'http://Azkickbacktours.com', 'https://www.phoenix.gov/parks/sports/complexes/desert-west', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.trilliumweightloss.com', 'http://www.runningwiththebullsusa.com', 'http://saporebyzafferano.ca/', None, None, 'http://www.thehkan.com', 'http://www.hayleylynnedoeshair.com', None, 'http://www.cotswoldfarmersmarket.com', 'http://www.kaleidoscopejuice.com/', 'http://www.harrysgrilleandtavern.com', None, 'http://local.biglots.com/az/gilbert/1913?zcp=pd_local_yelp', 'http://www.scottsdaleobgyns.com/russellBartelsMD.html', 'http://www.yourcarinsiders.com', 'https://www.supercuts.com/locations/pa/pittsburgh/shadyside-plaza-s-highland-haircuts-8953.html', '1592185734', None, None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://www.yknotparty.com', None, None, 'http://local.firestonecompleteautocare.com/ohio/north-olmsted/4580-great-northern-blvd/?lw_cmp=IYP_YPC_MLP_14796', 'https://www.saksfifthavenue.com/locations/s/sherwaygardens', None, None, 'http://www.vintagehomecharlotte.com', 'http://Www.aglowbeautybarlv.com', 'http://www.carolinaherrera.com', None, 'http://www.activegreenross.com', None, None, 'https://locations.whataburger.com/az/mesa/1947-w-broadway-rd.html/?utm_source=yextpages&utm_medium=locallistings&utm_campaign=yextpages', None, None, 'http://locopatron.com/', 'http://livewellmassage.vpweb.com', None, 'http://www.mammaspizza.com', 'http://www.victoiredayspa.ca', 'http://www.outback.com/locations/nv/las-vegas-craig', 'http://bandnautosalesllc.weebly.com', 'http://us.shop.ecco.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://firenzastone.com/', None, None, 'http://auxiliarycrossfit.com', 'http://www.capitolpetroleumllc.com/site/', 'http://www.pogllc.com', None, None, 'http://www.legendscafelv.com', 'http://www.sweetlifegelateria.com', 'http://www.clutchlandscaping.com', 'https://www.lesinsulaires.ca', 'https://www.circlek.com/store-locator/us/scottsdale/20721-n-scottsdale-rd/2705915', 'http://mine-sushi.com/', '1592185823', 'http://www.tsingtaomesaaz.com', 'http://www.Friendly-Wireless.com', 'http://www.bmvbooks.com/', 'http://www.smoochiepoochiedoggrooming.com', 'http://www.LaurenParis.com', 'https://www.wyndhamgardencalgaryairport.ca', None, 'https://www.calgarycoop.com/stores/shawnessy/', None, None, 'http://bosadonutsaz.com', 'https://www.ganleylincolnohio.com', 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/brampton-7688-hurontario-street.aspx?postalcode=L7A+3R9&storeid=66', None, 'http://www.thestratford.org/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', None, 'http://www.eddiebauer.com', 'http://www.tempritelv.com', None, 'https://www.saksfifthavenue.com/locations/s/phoenix', 'http://www.carolinavet.com/locations/matthews-nc/', 'http://aestheticbodycontouring.net', None, 'http://www.ganleysubaru.com', 'https://vbarbershop.com/location/chandler-fulton-ranch/', 'http://www.bellanailshop.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://hashtaggallery.com', 'https://locations.theupsstore.com/nc/matthews/2217-matthews-township-pkwy?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', None, 'https://www.uhaul.com/Locations/Truck-Rentals-near-Henderson-NV-89011/807058/', 'http://www.heislersmarket.com', '1592185901', None, 'https://www.esso.ca', 'http://www.hogwildpitbbq.com', None, None, '1592185915', 'https://www.kohls.com/stores/az/glendale-652.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.indiasflame.com', 'http://www.brightoncollisioncenter.com/', None, 'http://workoutquest.com', 'http://www.eagerbeaverplumbing.com', 'http://allens-pool-catering.business.site', '1592185934', 'http://www.ostlerfamilydental.com', 'https://chutneyb.com', 'http://www.ae.com', None, 'http://vspconsignment.com', 'https://www.zipscarwash.com/location/rock-hill-sc/', 'http://www.teresaspizza.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.morphebrushes.com', None, 'http://schwans.com', 'http://lovekimnailsspa.wix.com/home', None, 'http://www.earthangelsholistic.com', None, None, 'http://8thwonderstudio.com', 'http://www.bowtiepizza.com', None, 'http://www.eggsmart.ca', None, None, 'http://www.chapters.indigo.ca/home/storelocator/storeDetails/929/', 'http://www.donvalleyvw.net', 'https://druxys.com/location/2-st-clair-ave-colonia-place/', 'http://www.aemediaproduction.com', 'http://www.wingmachine.com', 'http://www.lvmis.com', None, 'http://www.vegaspubcrawler.com', None, 'http://www.jeromesalon.com', 'http://www.transformerstation.org/', 'https://lasvegas.adamevestores.com', 'http://www.dvtowing.com', None, None, 'http://Longevityinfitnessexcellence.com', 'http://www.smilebeautifuldentistry.com', 'http://www.marriott.com/hotels/travel/yyzap-courtyard-toronto-airport', 'http://www.urbanbloomcurls.com', 'https://www.willowpark.net', None, 'http://local.biglots.com/az/phoenix/4085', None, None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://rogersandhollands.com/east-towne-mall', 'https://starliterestaurant.ca', 'https://texasstation.sclv.com/Dining/Slices-Pizzeria', 'http://www.oldmillgm.ca', None, 'http://www.maac.com/alabama/huntsville/colonial-grand-at-madison?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://www.drbarbaranewman.com', 'http://www.clecabinets.com', None, 'http://www.mcmagain.com/home.html', 'https://locations.theupsstore.com/nc/charlotte/2820-selwyn-ave?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.bloorfitness.com/', 'http://www.kouskousexpress.com', None, 'http://www.sweetalamode.ca', None, None, None, None, 'http://www.globalpestlasvegas.com', None, 'http://www.foliclesalonandspa.com/', 'https://locations.usbank.com/index/nevada/las-vegas/summerlin-smiths-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://www.givemiacookie.com', 'http://www.chatime.com', 'http://www.therobotoproject.com', 'https://www.bushmillsirishpub.com', 'http://ladlows.com', 'https://sfaservice.com', None, None, 'https://luxxestudio412.glossgenius.com', 'http://www.alankells.com', 'http://www.gianteagle.com/store-locator', 'http://www.marytripi.com', 'http://rajplasticsurgery.com/', None, 'http://www.deltaco.com', 'http://www.steelcitysteak.com', 'http://nickelsdeli.com/saint-laurent/', None, 'http://momentumsalon.com', 'https://octapharmaplasma.com/donor/center/18568', 'http://www.mastercleaners.us', 'http://www.justfresh.com', None, 'https://www.gowireless.com/stores/az/phoenix/phoenix-ahwatukee-foothills/', 'http://shop.helixtelecom.com/', 'http://www.CerberusDesigns.ca', 'http://www.betsysicecream.com', 'https://www.newyorknewyork.com/en/nightlife/pour-24.html', 'http://pinogelato.com', None, 'http://www.vergerlacroix.ca/', 'http://www.starbucks.com/store/113209/', 'http://www.alexreidosleitoes.com', 'http://www.habitatlivingsound.com', 'http://www.attackpestcontrollv.com', 'https://www.kindercare.com/our-centers/mesa/az/000632?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=000632', 'http://www.pawsplaygrounds.com', None, 'http://www.zagg.com', 'https://www.donmillersubaruwest.com', None, 'http://www.ArcadiaCoveApartments.com', None, 'http://www.bodyheattanning.com', 'http://www.litespressobar.com', 'http://www.barbiespetsitting.com', 'http://www.hungryhut247.com', 'http://www.restaurantmochica.com/en/toronto', '1592186182', None, 'http://www.evoautoaz.com', '1592186188', 'http://phochay88.com', None, 'http://www.rockyscustomdetailing.com/', 'http://www.azeyes.com', 'https://www.therackandcue.com', 'http://www.backdoorlv.com', None, 'https://www.capriottis.com', None, 'https://lemonsharkpoke.com/locations/', 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://www.usps.com', 'https://www.labcorp.com', None, None, 'http://www.omzhair.com', 'https://www.marcs.com/stores/marcs-middleburg-hts-southland-grocery-store-4413', None, 'http://woodsrestaurant.ca/', None, None, None, 'http://fourpalmsautosales.com', 'http://gesu.com/', None, 'https://www.purdys.com/shops/details/79', 'http://www.scilv.com', 'http://tjsbutcherblock.com', 'http://aquatherapyclinics.com/', 'https://locations.costavida.com/costa-vida-a2660367671e', 'http://ragingspoon.ca/', 'http://www.queenfishandchips.com/', 'http://www.brownleejewelers.com', '1592186258', 'http://www.gypsybarphoenix.com', 'http://www.tbsstrong.com', 'http://www.titanrelocationservices.com', 'http://www.majormilliken.com/', 'http://www.bestautobuy.com/', 'http://Www.GotGoddessHair.com', 'http://www.cleatswings.com', 'http://outpostcoffee.com', None, None, 'http://www.ampm.com', 'http://www.photoantiquities.org', 'http://www.atmosphere.ca/', 'http://affordablelakenormanshuttle.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'https://stores.mooresclothing.com/ab/calgary/415---3545-32nd-ave--ne.html?utm_source=yelp&utm_medium=listing&utm_content=41&utm_campaign=AB', 'http://www.disciacca.com', None, None, None, '1592186300', 'https://sparklenailsbar.com', 'http://southpointcasino.com/dining/michaels-gourmet-room?cid=ypmic', 'http://www.fiveyearsyoungerfacial.com', 'http://www.thecrepecafe.com', 'http://locations.harristeeter.com/nc/charlotte/205/', None, 'http://www.JohnsonEstateLaw.com', 'http://walmart.com/', None, None, None, 'https://www.pizza73.com', 'https://www.batteriesplus.com/store-locator/az/tempe/batteries-plus-333?utm_source=yelp&utm_medium=local&storecode=333', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', None, '1592186333', 'https://dwss.nv.gov', None, None, 'http://noftz.com', 'https://santanvillage.benefitbrowbars.com', '1592186344', 'http://www.cardiacsolutions.net', 'http://natureworksbest.com/', 'http://www.mdclv.com', None, 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, 'http://www.petsmart.com', None, None, 'http://www.sawhilbanquet.com', 'https://lernerandrowe.com/?utm_source=Yelp', 'http://www.maccosmetics.com', None, 'http://www.midas.com/calgary/store.aspx?shopnum=9304&dmanum=151', 'http://www.timhortons.com', None, 'https://www.rolex.com/rolex-dealers/dealer-locator/retailers-details/rolexboutiquelvluxury.rswi_17986.lasvegas.unitedstates.rolexdealers.com', 'http://www.vosgeschocolate.com', 'http://www.a2zrm.com/', None, None, 'http://www.pittsburghprestigephotobooth.com', 'http://www.carmeldayspa.com', 'http://reddiservices.com/phoenix', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.abcpediatricshenderson.com/', 'http://www.wolsteincenter.com', 'https://stores.chicos.com/s/the-summit-at-scottsdale', 'http://www.monsterpgh.com', 'http://www.soccerzonevegas.com', 'http://www.northhillpizza.com', 'http://www.oxigenspa.com', None, 'https://eyetique.com/squirrel-hill', None, 'http://www.azresearchcenter.com/', '1592186416', 'https://www.cielohendersonapts.com', 'http://www.baileyorthoaz.com/', 'http://www.hobbypeople.net', 'https://www.holidayinn.com/redirect?path=hd&brandCode=hi&hotelCode=clesv&regionCode=1&localeCode=en&cm_mmc=mdpr-_-yelp-_-enhanced_profiles', 'https://mandarinrestaurant.com/mississauga/', 'http://www.switchconsignment.com', 'http://www.porschenorthscottsdale.com', 'http://www.carolinapacknc.com', None, None, 'http://www.guyandgalsalongilbert.com', '1592186443', 'https://www.wilhelmautomotive.com/locations/tatum-ranch/', None, None, 'https://twomenandatruck.com/movers/il/champaign', 'https://castlerents.com', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', 'http://www.dukescycle.ca', 'http://greatwhaletattoo.com', '1592186463', None, 'http://www.trilliummoving.com', None, 'https://www.dominos.com', 'http://local.buckle.com/AZ/TEMPE/385/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', 'http://myorangeleaf.com/frozen-yogurt-store/AZ/chandler/2875-west-ray-rd%2c-suite-1?id=8', 'http://www.17thavechiropractor.com', 'http://www.abiaz.com', 'http://www.thevenetian.com/RIVA.aspx', '1592186483', None, None, 'http://vwnorthscottsdale.com', '1592186491', '1592186493', 'http://www.gordsappliance.com', 'http://www.robinsongarden.com', 'http://www.cityandeast.com', 'http://ccvet.com', 'http://www.treatmentsalon.com', 'http://www.cottonon.com', 'http://fixauto.com/scarborough-town-centre', None, None, 'http://www.whitetoronto.com/', 'http://www.ercfixtv.com', 'http://www.jimmyscoffee.ca', 'http://www.ngvlanes.com', 'https://www.anytimefitness.com/gyms/288/Las-Vegas-NV-89139/?utm_source=yelp&utm_medium=local', 'http://inandoutmoving.ca/', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.madisonavecakes.com', 'http://www.manestudio.ca', 'http://blackcatpgh.com', None, 'http://www.smileshopdental.com', 'https://www1.shoppersdrugmart.ca', 'http://www.bullfrogspas.com', 'http://www.childrenslearningadventure.com/', None, '1592186547', 'http://www.desimoneglobalmarketing.com', '1592186550', 'http://www.timetoronto.com', 'http://restaurant.chineseonlineorder.com/goldengatehenderson', 'http://restaurantletandoor.ca', 'https://www.autozone.com/locations/az/phoenix/3908-e-thomas-rd.html', 'https://www.snapdragon-salon.com', 'http://www.thetamalestore.com', 'http://www.peterpiperpizza.com/', None, 'https://mononabarbershop.com', 'http://www.carolinasdish.com/', 'http://www.sparkleanlaundryphoenix.com/', 'http://www.lagunaspanv.com/', None, 'http://www.foxfamilydental.net', None, '1592186583', 'http://www.usps.com', 'http://www.sprint.com', None, 'http://islandheatdinnershow.com', 'http://flashforwardsites.com', 'http://www.bishoplawoffice.com', 'http://www.grandavenueproduce.com', '1592186601', 'http://www.balconycleaning.net', 'https://www.mapleteahouse.com', None, '1592186607', 'http://www.thechesterfieldshop.com', None, 'https://cftn.com', 'https://stoughtontireandautorepairinc.com', 'http://www.jiffylube.com', 'http://www.starbucks.com/store/9298/', 'http://www.thaidelightaz.com', 'https://www.redmountainweightloss.com/scottsdale/', 'https://www.libertysquare-apartments.com', '1592186628', 'http://guireitoronto.com', '1592186632', None, 'http://k1speed.com/phoenix-location.html?utm_source=Yelp', 'http://www.jasesouder.com', 'http://www.thecalgarygroomroom.com', None, 'http://www.littleones.ca', 'http://www.leagueofvapes.ca', 'http://www.scottsleedmd.com', None, 'http://mex-city.com/', None, 'http://www.azrapidmaintenance.com/index.html', 'http://www.coveredwagonaz.com/', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', None, 'https://www.thepaincenter.com/?utm_source=yelp&utm_medium=referral&utm_content=arizona-gilbert', 'http://ritzasiaaz.com/location.php/', 'http://www.meineke.com', 'http://www.soraonrose.com', None, 'http://www.ritzcarlton.com/en/hotels/canada/toronto', 'http://paradisevalleyseptic.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281357318&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://torrancepharmacy.com', None, 'http://www.littlecaesars.com', 'http://www.hummusbowls.com', '1592186689', 'http://www.audioworkx.com/', 'http://restaurantvago.com', None, 'https://www.batteriesplus.com/store-locator/nc/gastonia/batteries-plus-423?utm_source=yelp&utm_medium=local&storecode=423', 'https://beezybooth.com', 'https://www.montecarlo.com/en/nightlife/brand-lounge.html', None, 'http://www.sovarizona.com', 'https://www.orangetheory.com/en-us/locations/arizona/phoenix/2501-west-happy-valley-road-suite-32-1020/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.stylevsalon.com', None, 'https://www.ahn.org/location-detail?name=AGH_Suburban', 'http://shopurbo.com', 'https://www.josbank.com/store-locator/mt-lebanon-pa-356?utm_source=yelp&utm_medium=listing&utm_content=356&utm_campaign=PA', 'http://www.campers-village.com', None, 'http://www.bouldercreekgc.com', None, None, None, 'http://www.bigalscanada.com/stores/locations/NorthYork/northyork.html', '1592186732', None, 'http://www.oakdalepodiatrist.com', 'https://www.penrosecharlotte.com', None, 'http://www.wildwingrestaurants.com/franchisees/brampton-trinity-commons/', 'http://www.scottsdalehillsah.com', 'https://www.sundancecinemas.com/', '1592186748', None, None, None, 'http://www.opticgallerysummerlin.com', None, 'http://www.cottonlanervresort.com', '1592186762', 'http://franklygoodcoffee.com', '1592186767', 'http://www.thebeerstore.ca/locations/sheppardwillowdale', 'https://www.moxieservices.com/locations/phoenix/?utm_source=yelp&utm_medium=cpc&utm_campaign=2019_cpc', 'http://www.valleymedped.com', None, '1592186777', 'http://www.camelotinnelyria.us', 'http://www.kevinpowellmotorsportscharlotte.com', None, 'http://www.spiceasianbiz.com', 'http://www.santabarbaracatering.com', 'https://www.gamestop.com/store/us/wi/madison/740/prairie-towne-center-gamestop', 'http://www.lasvegasspinesurgeon.com', 'http://songbirdartistry.com', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'https://budgetblinds.com/Goodyear/?utm_source=yelp&utm_medium=naf-listing&utm_campaign=NAF-LOCAL-YELP', None, None, 'https://www.amazinglashstudio.com/studios/nc/charlotte/park-towne-village', None, 'http://www.therollhouse.com/wickliffe', 'http://www.dunkindonuts.com', 'https://www.purebarre.com/location/chandler-az', None, 'http://www.eliteacademyofgymnastics.com', 'http://advancedmetal.com', 'http://www.pitaland.ca', None, 'http://www.evex.ca', None, None, 'https://www.anytimefitness.com/gyms/92/Phoenix-AZ-85048/?utm_source=yelp&utm_medium=local', None, 'https://aamcobayarea.com/locations/concord', 'http://www.starbucks.com', 'http://www.spectrumbrands.com/', None, 'http://www.bellybumpsandbabies.com/', None, 'http://www.theknittingroom.ca', 'http://www.simplesimonpies.com', 'http://www.ohiojunkforce.com', None, 'http://www.publix.com', '1592186858', None, 'http://www.furniturestorelasvegas4u.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://tumbleweedhotel-hub.com/', None, None, 'http://www.polishdnailbar.com', 'http://uppercrustwinebar.com', 'https://www.kfc.ca//store/1595-cote-vertu-st-laurent', None, None, None, 'http://www.25liberty.ca', 'http://www.yehyogourt.com/', 'http://www.lasvegasfloral.com/?srccode=EF_YELP', 'http://www.wigsrustoronto.com', 'https://www.walgreens.com/locator/walgreens-3025+las+vegas+blvd+s-las+vegas-nv-89109/id=6167', 'https://ramclean.com', None, None, 'http://mintlocker.com', None, None, 'http://www.greenfieldgroomingsalon.com', None, '1592186914', 'http://www.tankstap.com/', None, 'http://www.ouestantoniospizza.com', 'http://www.badwaterbrewing.com', 'https://local.albertsons.com/az/chandler/3145-s-alma-school-rd.html', None, None, 'https://quantumcleaning282.wixsite.com/website', 'https://locator.chase.com/ny/glendale/71-04-woodhaven-blvd', None, 'http://www.milestonesrestaurants.com/', None, None, None, 'http://www.wmdouglas.com/charlotte', 'http://www.nvscc.com', None, None, 'http://www.planetdog.ca', 'https://www.arizonarestorativepsychiatry.com', None, None, 'http://www.mevamekitchenexpress.com/', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', None, 'http://nevadaeyephysicians.com', 'http://www.ranchorelaxotogo.com', 'https://hakkasan.com/las-vegas/', 'http://www.dysartdental.com', 'http://stacyannsscissorhands.com', 'https://www.supercuts.com/locations/az/scottsdale/scottsdale-pavillions-haircuts-9672.html', 'https://locations.churchs.com/nv-las-vegas-8616', 'http://www.landmarkautoanddiesel.com', 'http://www.familyvideo.com', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://stores.dickssportinggoods.com/az/scottsdale/1194/', 'http://www.drallenhuioptometry.com/', 'http://www.fabfred.com/locations/ft-apache/', 'http://www.theteakeeper.com/', 'https://agents.allstate.com/juan-deglane-glendale-az.html', 'http://www.ctctree.com', 'http://www.saharaair.com', None, None, None, 'http://www.holtrenfrew.com/store/holt/stores/hr2/VaughanMills', None, 'http://www.bennett2rentit.com', 'https://locations.theupsstore.com/az/scottsdale/16211-n-scottsdale-rd?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=11088&localechange=1', '1592187020', 'http://www.auburnbayvet.com', 'https://www.swiftaviation.com', 'http://www.sundevilauto.com/locations/az/scottsdale/north-scottsdale-25', None, None, 'http://www.pinotspalette.com/southend', 'http://www.pinkynails.ca', 'http://smartrepairsaz.com', 'http://www.lizyhoeffer.com', 'https://www.spiritsandtales.com', '1592187046', 'http://www.greatclips.com/salons/4614', 'http://www.dunkindonuts.com', None, None, 'http://walkinwalkin.com', None, '1592187060', None, None, None, 'http://www.seespotclean.com', 'http://gogreenac.com/', 'https://iworkoncars.com', 'https://silvernuggetlv.com', None, 'http://www.bk.com', 'http://www.mgmgrand.com/restaurants/crush.aspx', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNMW', 'http://www.AbsolutePetCareAZ.com', 'http://www.scrubtastic.com', 'http://broadbentdentistry.com/', 'https://www.bestbuy.com/site/electronics/geek-squad/pcmcat138100050018.c?id=pcmcat138100050018', 'http://www.honmachirestaurant.com', 'http://www.bluearrowrecords.com/', 'http://www.phoenixrackandaxle.com', None, 'http://www.parkcentraldeli.com', None, 'https://www.mygabes.com', 'http://thriftdlux.com/', None, 'http://www.clinicsalveo.ca', 'http://www.pandora.net/en-us/stores/united-states/85254/pandora-scottsdale-quarter', None, 'https://www.sprint.com/storefronts/bd/sprint-north-las-vegas-nv-89032-fcs-4123/?ecid=SEO:Yelp:2020_Q1:LastMile', 'http://www.housemaster.com', None, None, 'http://nodabarkandboard.com/home.html', 'https://www.destinymartialartsacademy.com', 'http://emakravmaga.ca', 'http://www.stateoftheartdesigners.net', 'https://www.chevronwithtechron.com', 'http://www.bigjimsrestaurant.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=626&utm_content=bpas', None, None, 'https://www.lasikvisioninstitute.com/lasik-eye-surgery/cleveland-surgeon-center/?venid=YLP31&subid=CLE', None, '1592187147', None, 'http://www.verizonwireless.com/stores/arizona/mesa/val-vista-204086/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', None, 'http://www.midwoodsmokeshack.com', 'http://www.chapters.indigo.ca', 'https://www.vistasureno.com', 'http://www.therumormillmarket.com', 'http://www.drywallperfection.com', '1592187163', 'http://artscapeyoungplace.ca', 'https://www.supercuts.com/locations/oh/rocky-river/rocky-river-haircuts-81630.html', 'http://www.katieskorner.com', 'http://www.camh.net', 'http://www.southwestfunctionalmedicine.com', None, None, 'http://www.lionsmanegrooming.com', 'http://www.amazinginc.net', 'https://haircutmenfairlawnoh.com', None, 'http://www.cherryberrieslasvegas.com', 'http://www.shopwoc.com', 'https://www.amcofdv.com', 'http://www.starbucks.com/store/10759/', 'http://usmaniagrill.ca', 'http://www.igoturback.com?cmpid=GSDG_BLM_0006', 'http://www.cal-am.com/communities/chaparral-village', 'http://azmajorsoccerleague.com', None, 'http://www.valleygoldmine.com', 'http://www.saludjuicery.com/aspinwall/', 'http://www.undefeated.com', None, 'http://www.whatalesya.com', 'https://www.burialplanning.com/cemeteries/resthaven-memory-gardens/', 'http://collegestreetanimalhospital.com', 'http://pueblosereno.com/', 'http://www.autonationbuickgmchenderson.com', None, 'http://fortmill.burnbootcamp.com/', 'http://www.orangedumpster.com/', 'http://portofsubs.com', 'https://www.newdermamedlaserclinic.com', None, None, 'http://www.svmmjcc.com', 'https://stores.bareminerals.com/wi/madison/us-2313/', None, 'http://www.rdquake.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sears.com', None, 'https://www.storage-solutions.org/storage/arizona/storage-units-peoria/8266-W-Lake-Pleasant-Parkway-115', 'http://www.darinaelectrolysis.ca', 'https://www.allamericanpubclt.com', 'http://www.sasshoescharlotte.com', 'http://taylormadecustompools.com', 'http://www.kitchen73.ca/laval', 'http://colorworkslandscape.net', 'http://www.bodyrestorationmassage.com', 'http://fujiyamacalgary.com', 'http://www.castleapartments.net', 'https://www.myeyedr.com/locations/independence-boulevard?utm_source=Yelp&utm_medium=Referral&utm_campaign=independence-boulevard', 'http://carlretterdebtattorney.com', None, 'https://zelaya-jr-painting.com', 'http://www.swarovski.com/Web_CA/en/index', 'http://www.trapezeu.com', None, 'http://www.marshallsonline.com', 'http://kokofitclub.com', 'https://www.thirdfederal.com?=ad.doubleclick.net/ddm/clk/458420725;262568795;w', 'http://www.tropicalsmoothie.com', 'http://www.jonesysclassiccleaners.com/', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', None, 'http://tradelandsaz.com', None, 'http://www.sweetfixbakery.com', None, 'https://www.cellphonerepair.com/charlotte-university-city-nc/', 'http://www.happyvalleyplumbingllc.com', 'http://www.oneofakindpets.com/', 'http://jacksonswash.com', None, None, 'http://www.talkingstickresort.com/spa.aspx', 'http://nathansfamous.com', 'http://fiveguys.com/', None, None, None, None, 'http://christensenconstruction.net', '1592187329', 'http://www.realliving.com/sj-fowler-real-estate', 'http://www.gnargnartours.com/', None, 'http://www.apachestone.com', 'http://www.doddsdoors.com', 'http://www.subway.com', 'http://www.canyon-pt.com/location/surprise-office/', 'http://trustarts.culturaldistrict.org/production/37630', None, 'https://stores.savers.com/nv/lasvegas/savers-thrift-store-1047.html', 'https://milk-n-more.business.site', None, 'http://www.sfldentistry.com', 'https://extremewheels.com', None, 'http://www.JimmyJohns.com', None, None, None, 'https://www.thehouseofused.com', 'http://www.nhhuntersvillepedsandim.org', 'http://www.origintmassage.com/', None, 'http://www.hskaraoke.com', 'http://mesa.floorstogo.com', 'http://www.nveyesurgery.com', 'http://www.stoutflooring.com', None, None, 'https://square.site/book/0GPTRC4PSJ0WB/sunny-girl-spray-tan-phoenix-az', None, 'http://www.leanonmeals.ca', 'http://www.colossusgreektaverna.com/locations/colossuspc/contact/', 'http://sonoranserenityspa.com/', '1592187400', 'http://www.blm.gov', 'https://thrifty.com/loc/ll/CA/OT/TORONTO-~-FRONT-ST-(OT)/330-FRONT-STREET-WEST', 'http://www.benchworksteering.com/', 'http://visit.brownmackie.edu/SERCH/dynamic/bmc-3-step.aspx?RIFID=3322&source=YELP&sourceCode=YELP&cid=SERCH_BMWA_096_SRCH_002&SearchEngine=Yelp&utm_source=Yelp&utm_medium=cpc&utm_campaign=Yelp', None, 'http://www.suzannewigginton.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.carepatrol.com/', '1592187418', None, None, 'http://artgeckoshop.com', None, None, None, 'http://www.loftsonthomas.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/pa/turtle-creek-4072.html?mcid=iyp:8368400', None, 'http://www.thebierhausbistro.com/', 'http://www.wheatbyjay.ca', None, None, None, '1592187446', '1592187449', None, 'http://brathausaz.com/brat-haus-events/', None, 'http://www.stardustbuilding.org', None, 'http://www.blackcanyonconferencecenter.com', 'http://www.pittsburghyellowcab.com', None, 'http://www.casbahrestaurant.ca', 'http://www.solosmallmotors.ca', 'http://www.empirepizzabar.com', 'http://www.jacksonswash.com/location-84th-avenue', 'http://www.dakinemobiledetailing.com', 'http://www.adrenalinetoronto.com', 'http://www.drlockney.com', 'https://locations.churchs.com/az-phoenix-681', 'http://venues.calgarystampede.com/venues/bmo-centre/', 'https://bikanersweetsandrestaurant.ca', '1592187485', None, 'http://www.prayosha.ca/', None, 'http://blackcanarycoffee.com', 'http://www.massagewellvegas.com', 'http://www.brookdaledentalcare.com', 'http://www.mcdonalds.com', None, 'http://www.customcaninecompany.com', 'http://www.dcslv.org', None, 'http://www.terribleherbst.com/', None, None, None, None, 'http://gocheckers.com/team/game-guide', 'http://paradisemelts.com', 'http://www.tribebarrestudio.com', 'http://www.fuguasian.com/menu.aspx', None, None, 'http://www.billygrahamlibrary.org', 'http://www.purolator.com/purolator/ship-track/find-location.page?location=ridRC0715', None, 'http://www.fedex.com', None, 'https://www.snwa.com', 'http://www.freshandeasy.com', None, 'http://www.orthopaedicspecialists.org', None, 'https://www.kwiktrip.com/Locations/Details/650', 'http://www.phoenixbedbugexpert.com', None, 'http://www.videoconversionexperts.com/', 'https://www.cash1loans.com/locations/arizona/phoenix-thomas.aspx', 'http://local.firestonecompleteautocare.com/ohio/cleveland/6874-pearl-rd/?lw_cmp=IYP_YPC_MLP_1694', 'http://www.canadiantire.ca/en/store-details/on/warden-eglinton-on.store.html', '1592187562', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://supremetouch15.wixsite.com/supreme-touch/contact', 'http://www.perryellis.com?utm_source=Yelp&utm_medium=Social&utm_campaign=PE_Yelp_BioCTA', 'http://www.mrtire.com', None, 'http://www.sunbirdgolf.com', None, 'http://www.mocca.ca', 'https://www.pizzaville.ca/stores/hours/2889-derry-road', None, 'http://www.maranellocafe.ca', 'http://www.torontosteamnclean.ca', 'http://www.jogoyaseafoodbuffet.com/', None, 'http://www.joefresh.com', 'http://ladolcepesca.com', 'https://www.theshadestore.com/showrooms/nevada/las-vegas?utm_source=yelp&utm_medium=cpc', 'http://www.tristatetowingandrecovery.com', None, '1592187599', 'http://www.royalvinewinebar.com/', None, 'http://SecondsCity.com', 'http://www.ljsilvers.com', None, 'http://scacciaunion.ca', 'http://truhitfitness.com/north-phoenix/', 'https://orlandoautobody.com', 'http://www.coyoteoaties.com', 'https://www.jiffylube.com/locations/az/phoenix/2098', 'http://www.bigmanspizza.ca', 'https://www.dogtrainer-charlotte.com', None, None, 'http://www.goodwillaz.org/location/goodwill-mckellips-mesa/', 'http://www.azspinalcare.com/', 'http://www.ranee.ca', 'http://www.thecornflower.com', 'http://www.springspreserve.org', None, 'http://www.dakotamoversaz.com', None, '1592187645', 'http://www.bownesshealthfood.ca', 'https://rapidresponders.com', None, 'https://amazingleakdetection.com', None, 'http://www.midas.com/cleveland/store.aspx?shopnum=6355&dmanum=212', 'https://www.fountainpalmsapts.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=organic', 'http://www.bojangles.com', 'http://www.themagicshoplv.com', None, 'http://www.sewmuchfun4you.com/', None, 'http://www.hickorynutgolfcourse.com', 'http://www.mrsbridalboutique.com/', 'https://www.harveys.ca', 'http://www.AZLawnGuys.com', '1592187676', 'https://www.spencediamonds.ca/location/calgary-ab/?utm_source=Yelp&utm_medium=Display', 'http://www.LasVegas1RealEstate.com', None, 'http://www.samosafactory.com', 'http://www.thebrubakerspub.com', None, 'https://www.homestreet.com', 'http://www.nikosgardening.ca/', None, 'https://www.victra.com/?utm_content=Yelp&utm_medium=URL&utm_source=Yelp', 'https://www.mindbodhiyoga.com', 'http://www.virginamerica.com/', None, None, 'http://www.health-a-licious.com', 'http://lacampagnaonline.com', 'http://www.canampm.com/sun-valley-ranch', 'http://www.ranefamilyinsurance.com', None, 'https://stores.nyxcosmetics.com/directory/nv/las-vegas/3663-south-las-vegas-blvd.html', 'http://trilogycle.com', None, 'http://www.bloomersto.com', 'https://www.walmart.com/store/5334/aurora-co/details', 'http://www.fithcc.com', None, 'http://99only.com/store/Las-Vegas---Beltway/', '1592187730', None, 'http://www.vbinspect.com', 'http://www.makinolasvegas.com', None, 'http://samspizza.co', 'http://www.swimmerinsurance.com', 'http://www.lipsboutiquebyjka.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', None, 'http://www.themarvelexperience.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.bannerhealth.com/locations/banner-urgent-care-6', '1592187760', 'http://www.taitjointreplacement.com', None, 'https://www.weecarelv.com', None, 'http://www.pizzanova.com', None, None, 'http://www.entebellamedical.com', 'http://www.deansgreenhouse.com', 'http://www.jazz.fm', 'https://locations.churchs.com/az-phoenix-7235', 'http://www.hashhouseagogo.com/', 'https://www.wyndhamhotels.com/hotel/15081?tel=18008899735&iata=00093785&cid=fe:hj:20170523:yl:pp:hjus:15081', 'http://www.hawaiianbarbecue.com', None, 'http://www.pizzahut.ca/', None, None, 'http://www.hawaiipoke.ca', None, 'http://www.bogdananghelmd.com', 'http://www.thegreenhousetavern.com', 'http://www.farmgirlsfood.com', 'http://www.museums.nevadaculture.org/nsmlv-home', 'http://www.graysbooks.com', None, 'https://www.clubpilates.com/fortmill', 'http://www.swc', 'http://www.4feldco.com/locations/madison/', 'https://www.simon.com/mall/arizona-mills/stores/disney-store-outlet-the', None, 'http://www.raymillphotography.com', 'http://www.thomasyu.ca', None, 'http://www.animalinnlv.com', None, None, 'http://www.ajlawplc.com', 'http://www.metrobark.com', 'https://www.get-go.com/', None, 'http://www.clevelandairport.com', None, 'http://jbamotors.com', '1592187838', 'http://www.wagonthedanforth.com/', None, 'http://www.toryburch.com', 'https://samurairamen-sushi.com', None, 'http://pizza.dominos.ca/Mississauga-Ontario-10348', 'http://www.universalscreenarts.com', None, None, None, 'http://www.keybank.com', None, 'https://cannacabana.com', '1592187864', 'http://www.sweetpeas.ca', 'http://cornercafewi.com/', 'http://www.restaurantbellissimo.com/fr', 'http://pigtailsandcrewcuts.com/charlotte-cotswold', None, 'http://www.calldanielhull.com?cmpid=F27M_BLM_0006', 'http://www.sidestreetpubandgrill.com/', None, 'http://www.DesertFitnessAZ.com', 'http://www.beardedchefaz.com', 'http://www.WickedBakery.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.gumbyspizzamadison.com/', None, None, 'http://www.makeafoodiecall.com', 'http://www.dunkindonuts.com', None, 'https://www.shadowmountaindentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.biennails.com', 'https://www.americasbest.com/store-list/avon-oh-44011/', 'http://www.wynnlasvegas.com/#Shopping/blacksatin', None, None, 'http://farmersagent.com/gelfond', None, 'http://cherryblossompools.com', 'http://www.eatnpark.com', None, 'http://www.activekidspediatrics.com', 'https://daisybluesummit.business.site', 'http://www.cedarcenterhardware.com/', 'http://www.hectorsmovingservices.com', 'http://locations.maurices.com/nv/lasvegas/clothing-stores-lasvegas-nv-1751.html', None, None, 'http://www.sonoramesquitegrill.com', 'http://www.airbornetrampoline.ca/locations/woodbridge/', 'https://www.enterprise.com/en/car-rental/locations/us/az/mesa-alma-gardens-5050.html?mcid=iyp:8368400', 'https://arizonaorganix.org', 'http://www.coinlesslaundry.com/23rd.html', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://littlerascalslearningcenter.com', 'http://www.barreljunction.com/location', None, None, '1592187957', None, None, 'http://www.kensingtonmakeupacademy.com', 'https://www.greulichs.com', None, 'http://www.eastwinddentallv.com', None, 'http://www.metroparktune.com', 'https://locations.theupsstore.com/az/gilbert/70-s-val-vista-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.trinitycathedral.com', 'http://kokosmiles.com', None, 'http://www.millerswaterdamageandrestoration.com/henderson-nv.html', None, 'http://ltgrill.com/', 'https://eastvalley.bintheredumpthatusa.com', 'http://www.swgas.com', 'http://www.FletcherHeightsAH.com', None, 'http://www.mooredalesailing.com', 'http://www.disgraceland.ca/', 'http://www.michaelkors.com/stores/united-states/nv/las-vegas/michael-kors-palazzo', None, 'https://www.kurevapes.com', 'http://www.sossystems.com', None, None, 'https://stores.petco.com/nc/gastonia/pet-supplies-gastonia-nc-1586.html', None, 'http://www.Azbiltmorebarbershop.com', 'https://www.zaxbys.com/locations/nc/concord/7781-gateway-ln-nw/', '1592228118', 'http://eyelovebeautybar.com/', 'http://www.justrolldup.com', 'http://www.7-eleven.com/?yelp=22886', None, 'https://www.simonmed.com/locations/all/simonmed_imaging___estrella', None, 'https://www.gostorageone.com/self-storage/nv/las-vegas/maryland-pkwy/', 'http://www.cineplex.com/Theatre/cineplex-odeon-ajax', None, None, None, 'https://www.zimbrickhonda.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', 'http://www.kgautoworkspgh.com', 'https://www.americafirst.com/about/branches/branch-details.cfm?siteNo=104', 'http://www.goldenwestmanagement.com/', None, None, 'http://www.adega9.com', 'http://www.freshandwild.ca', 'http://www.scottsdalerestaurants.co', None, 'http://headtotoeinc.com', 'https://theeggworks.com', 'http://www.aqualunch.ca', 'http://www.steakonastone.us', 'http://www.thesavannah.com', None, None, 'http://www.ktnv.com/', None, 'https://honeycombsaloncollective.com', None, 'http://sunwest-az.com', 'http://www.rustikrotisserie.com', 'http://www.eliesfamilyrestaurant.com', 'https://www.lennar.com/New-Homes/Nevada/Las-Vegas/Las-Vegas/Mountains-Edge/Monterey-Ranch-The-Gardens/INHC', 'https://playgroundsdartzone.com', 'https://stores.peoplesjewellers.com/on/markham/5000-highway.html?cid=YEXT', 'http://www.tuttisantibynina.com', 'http://www.redsealnotary.com', 'http://www.cheekychicbb.com/', None, None, None, 'http://www.garciabailbonds.company', 'http://www.el-chameleon.com', 'http://www.miamistarglo.com', 'http://www.felinefabulous.com/', 'http://firebyjames.com', 'http://www.lumiermedicalspa.com', 'https://www.thaithaispaandmassage.com', 'http://www.lisaeleyphotography.com', 'https://corebodypilatesaz.com/locations/north-gilbert-pilates-studio', 'https://stores.thenorthface.com/NV/LAS-VEGAS/USA314/', 'http://local.firestonecompleteautocare.com/ohio/rocky-river/21210-center-ridge-rd/?lw_cmp=IYP_YPC_MLP_2089', 'https://leancanteengcu.wixsite.com/healthy', 'http://www.hargta.com', 'http://www.vanish-ink.com', 'http://www.elsombrerovegas.com/', 'http://www.SkatelandMesa.Com', 'http://www.aiellospizza.com', 'https://www.tristastevensonstylist.com', 'http://www.marisolcu.org/', 'http://www.theforumbarpar.com', 'http://www.usps.com/', None, 'https://www.carcolv.com', 'http://www.blacks.ca', 'http://barredefinitions.com', None, 'https://nailsatthelakes.business.site/?m=true', None, 'http://www.menschssecurityfilms.com', 'https://hovdeproperties.com', 'http://www.carolinasmedicalcenter.org/body.cfm?id=14', None, 'http://www.dentproaz.com', 'http://www.littlebangkokthairestaurant.com', 'http://thegallerysouthend.com/', 'http://www.desertsandsrvresort.com', None, None, 'http://www.townofyork.com', None, None, 'http://www.onedaytours.com', None, 'http://www.abercrombie.com', None, None, 'http://www.elpolloloco.com/locations/store/3859', 'https://dripedgeroofing.com', '1592228355', '1592228358', 'http://www.sleepnumber.com/?utm_source=Yelp&utm_medium=CPC&utm_term=CTA&utm_campaign=9f4Lf6iyxkIXfQw-hozi2w&cm_mmc=Yelp-_-Yelp-_-WebsiteLink-_-9f4Lf6iyxkIXfQw-hozi2w-904825&tag=904825', None, 'http://www.fairviewmall.ca/EN/Directory/Stores/Pages/NormaReed_1084A.aspx', None, None, None, '1592228370', 'http://www.qualityhomeinspectionspittsburgh.com/', 'https://mattressbyappointment.com/locations/nv/lasvegas/', 'http://retinanevada.com/', '1592228381', 'https://drcooperortho.com', None, 'http://www.brookcreekapartments.com', None, 'https://www.chevronwithtechron.com', 'http://www.citi.com', '1592228400', None, 'http://www.hobobillstrainhousellc.com', 'http://www.maidstr.com', 'http://www.teavana.com', 'http://www.webuycars.com/location', 'http://drphonerx.com', 'http://www.jaquelineandrea.com', 'http://sc-clinic.com/clinic/bay_and_college/', 'http://www.quailhollowresort.com/page/dining', None, None, 'http://www.chtools.ca', 'http://www.silvercollectionatthepark.com/?utm_knock=y', None, 'http://www.nvcoin.com', None, 'http://www.petco.com', None, 'https://www.giggles.ca/fr', 'https://www.associatedbank.com/location/WI/madison/2001-londonderry-dr', None, 'http://www.cascadeautogroup.com', 'https://eatcopperbranch.com', 'http://www.ZahlerProperties.com', 'http://www.somethinscooking.com', 'https://www.levinfurniture.com/furniture-store/oh/mayfield-hts/6051-mayfield-rd', '1592228466', None, 'http://www.bvinsurance.ca', None, 'http://www.csn.edu/library', 'http://www.waxon.ca', 'https://www.davidsbridal.com/?utm_source=Yelp&utm_medium=OrganicSearch&utm_term=84&utm_content=DisplayURL&utm_campaign=WebsiteListing', 'http://affyspremiumgrillscarborough.com', 'http://www.asapcity.com', '1592228491', 'http://www.libertymutual.com/insurance-office-madison-wi', 'https://www.ihop.com/en/restaurants-south-las-vegas-nv/7490-s-las-vegas-blvd-3096', None, None, 'www.ripspub.com', 'https://melodys-chair.business.site/#gallery', 'https://www.upack.com', '1592228513', 'http://www.customcoaches.ca', 'http://www.midabui.com', 'http://www.bostonsgourmet.com/', 'https://www.arrowheaddentalgroup.com/?sc_cid=yelp:o:listing&_vsrefdom=yelp', 'http://www.saberhealth.com', None, 'http://www.trainersclubaz.com', 'http://acdentalvegas.com', 'http://www.crateandbarrel.ca/Stores/Store.aspx?storeid=201', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'https://impactptaz.com/scottsdale/', 'http://choicehotels.com/Suburban', None, '1592228544', 'http://www.plasticmaninc.com', 'http://www.mnmshop.com', None, 'http://www.desertstorage.com/scottsdale-az-storage-units', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', None, 'http://www.clubmonaco.ca', 'https://parkwoodslaundry.com', None, None, '1592228568', None, 'http://venushomesaz.com', None, None, '1592228579', 'http://fabulousfloorscleveland.com', 'http://www.clorebeauty.com/', 'http://bluediamondbestbitedental.com', 'http://www.wildeeasttownehonda.com', None, None, 'http://www.bootmaster.com', 'http://www.freevegasclubpasses.com/yelp', 'http://www.planetfitness.com/gyms/glendale-downtown-az-1110?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://booyah-inc.com', 'https://locations.wendys.com/united-states/nv/las-vegas/9385-south-eastern', '1592228604', 'https://www.autonationbuickgmcwestsahara.com', None, 'http://www.highlandparkclubapartments.com', 'http://www.russellaquatics.com', 'http://www.eatatbobsdiner.com', 'http://www.huffsautomotive.com', '1592228619', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', None, 'http://www.freds-flowers.com', 'http://www.handandstone.ca/locations/jarvis-adelaide', 'https://www.azmobiletanning.com', None, 'https://www.audinorthlake.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', None, 'http://www.bunkhousekids.com/', 'http://www.flooringamericaofchandler.com/', 'http://bestbridalandtuxedo.com/', 'http://mcghies.com', None, 'http://www.lhsstudio.com', 'http://summerlinsmiles.com', 'http://alfornopizza.com/', 'http://www.thebatteryguy.com', 'http://www.piola.it', 'http://www.thaihousecalgary.com', None, 'https://agents.allstate.com/rosalie-gonzalez-las-vegas-nv1.html', 'https://www.champssports.com', 'http://www.greenlifecafemadison.com', None, None, None, 'http://www.napolipizzasqhill.com/', 'http://www.amptover.com', 'http://www.bronzecafe.com', 'http://www.envisionnv.com', None, 'http://www.conspirephoenix.com', 'http://findlayhonda.com', 'http://www.LendingPathway.com', None, None, 'https://www.walmart.com/store/2768/mesa-az/details', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://islingtongolfclub.com', None, 'http://HealthySkinLasVegas.com', 'http://cherryst.ca/', 'http://www.popsexotics.com', 'http://StrawberryPropertyManagement.com', 'https://www.extraspace.com/storage/facilities/us/pennsylvania/west_mifflin/1000000568/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', None, 'http://floresdecoloresart.com', 'http://www.chicos.com/store/home.jsp?cmp=MOC_yelp', None, None, 'http://www.cbdamericanshamanlv.com', None, 'http://www.sheacoco.ca', 'http://dollarcanada.ca', 'http://www.sapphirerealtyaz.com', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://greenwithenvyaz.com', None, None, 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'https://www.redlobster.ca/locations/list/on/toronto/20-dundas-street-west', 'http://lemodernjapanese.com', 'http://www.edwardsmovinglabor.com', 'http://www.deforestfamilyrestaurant.com/', 'http://www.stusmotorcycletowing.com', 'http://www.foxpeterson.com', 'http://luckyspizza67.com', 'http://www.catboardingcharlotte.com/', None, 'https://familybraces.ca', 'http://www.letsdodinneronline.com', None, None, 'http://allstarhairnailslasvegas.com', 'http://www.coldstonecreamery.com/stores/22323', None, 'http://www.straightedgewallpapering.ca/default.htm', 'http://www.tsalonmesa.com', 'https://spiceclub.foodticket.ca/foodticket/cgi/bestel.cgi', None, 'http://mardaloopwellness.com', 'http://www.vegasCapital.com', 'http://7eleven.ca/store-locator/33259', None, 'http://shaggychic.com', 'http://www.chicagodeepdishpizza.com/contact-us/calgary-ne', 'http://www.abbott-trophies.com/', 'http://www.aspca.org', 'http://archibaldmicrobrasserie.ca/index.php/fr/restaurant-blainville.html', 'http://cpm-apts.com', None, None, 'http://poshnailscharlotte.com', 'http://www.sallybeauty.com/', 'https://www.4wheelparts.com/stores/ohio/4wp-cleveland-oh-51', 'http://www.davesducts.com', None, None, 'http://www.trianglerealty.us', 'http://www.findlaychevy.com/', 'http://mountainpie.com', 'http://www.Districtlending.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://www.redmangousa.com', 'http://wtfexp.com', 'http://www.mi5tentertainment.com', None, 'https://www.bmoharris.com', 'http://www.ymcacharlotte.org', None, 'http://sphinx-cafe.com/', '1592228860', 'http://subway.com/', None, 'https://stoneandvine.com', 'http://www.papajohns.com', None, 'http://www.deltaco.com', 'http://illinishop.com', 'http://www.canadiantire.ca/en/store-details/on/warden-eglinton-on.store.html', 'http://medifastarizona.com', 'http://www.amorino.com', 'http://pdmtailor.com', '1592228887', 'https://www.casamiascottsdale.com', None, 'http://www.paragonphysicaltherapy.com', 'http://www.newviewrealtygroup.com', None, 'http://charlotteorthodontists.com/', 'http://www.clarkcountycourts.us/', 'http://www.redspaceevents.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://www.steaknshake.com/locations/23413-steak-n-shake-kenyon-road-urbana', 'https://www.cresthavengolf.ca', '1592228921', 'https://locations.wendys.com/united-states/ga/brunswick/164-altama-connector', 'https://www.palladiumeventcenter.com', None, 'http://www.royalganesha.com', 'http://mlk-smog-check.business.site', 'http://www.wemovewi.com', None, None, 'http://www.stevensplumbingac.com', 'http://akronfitnessandwellness.com/', None, 'https://www.wellsfargo.com', 'http://www.portofcleveland.com/environment-infrastructure/cleveland-lakefront-nature-preserve/', 'https://ad.doubleclick.net/ddm/clk/434046915;236895156;q', None, 'http://tiltedkilt.com/richmond-hill/', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.investarrealestate.com/agent?xid=040101-01&name=Jacqueline-Kenoun', 'http://www.mirage.com/casino/race-and-sports-book.aspx', None, 'http://www.lilacbakeryaz.com', 'http://www.vegasvalleyphotography.com', 'http://desertmesapainting.com', 'https://www.chick-fil-a.com/arboretum', None, 'http://www.usmenuguide.com/honmachi.htm', 'http://www.cox.com', 'http://www.sgi-usa-lasvegas.org/', 'http://www.pancho-and-leftys.com', 'http://www.shell.com/', 'http://www.lofthairlounge.ca', 'https://www.metro.ca/trouver-une-epicerie/44', None, None, 'http://www.vfwpost3513.com', None, 'https://lasvegasurology.com', 'http://www.animalhousevetqueencreek.com', 'http://www.CLTvictor.com', None, None, 'https://www.shearexcellence.com/dilworth/', None, 'https://www.choicehotels.com/CN803?mc=blypypaf', 'https://thejoyfullkitchen.com', 'http://www.dbksalon.com', 'http://qdoba.com', None, 'http://www.francescasfaceliftoflasvegas.com', None, None, 'https://local.safeway.com/safeway/az/gilbert/90-s-val-vista-dr.html', None, None, 'http://massagemaniaglendale.com', 'http://www.chivino.com', None, 'http://www.integritytransportationpgh.com', None, 'http://www.dynastybuffetnc.com', 'http://www.dvf.com/', 'http://www.casitassanmarcos.com', 'https://sincitymoto.com', 'http://www.bodyandsoultherapy.ca', None, 'http://www.grapevinecabinets.com', 'http://www.arizonadanceartistry.com', 'https://www.simonmed.com/locations/all/simonmed_imaging___metro_ii', 'http://www.nps.gov/jaga/', None, 'http://www.foothillsautoglassphoenix.com', 'http://www.iloveichibanaz.com', None, None, None, 'http://lahainagrillLV.com', 'http://www.adcopaint.com', None, 'http://everydaycafepgh.com', '1592229087', 'https://desertcastlepc.com', 'http://www.stonecreekroofingaz.com', None, None, 'http://www.bfshomewarranty.com', 'http://thomashinds.ca', 'http://www.dermatiquemd.com', 'http://www.bathemp.ca', 'https://nice-garden-landscape.business.site', 'http://www.browart23.com/location.html', None, 'http://www.millerframe.com', 'http://www.JimmyJohns.com', 'http://www.fabriziovegas.com', 'http://chensnoodlehouse.business.site', None, None, 'http://www.ridersinn.com', 'http://www.radjewellery.com', 'https://www.statefarm.com/agent/us/az/phoenix/eric-jackson-x4xfz21d000?cmpid=r6sy_blm_0006', 'http://yelp.com/elite', 'http://www.itsabaotime.ca', 'http://www.azchoicechiro.com', 'http://www.apple.com', '1592229145', 'http://www.superstarcarwashaz.com', 'http://futuredirectionsspa.com', 'http://www.heatherkphotography.net', 'http://www.cvcheart.com/scottsdale-princess-office/', 'http://jfkfinancial.com', 'http://www.canadiantire.ca', None, None, None, '1592229167', 'http://www.samuraicomics.com', None, 'https://nationwidevision.com/location/las-vegas-vision-center-silverado-ranch', '1592229176', None, 'https://ad.doubleclick.net/ddm/clk/435408715;237471434;h?https://www.hrblock.com/block-advisors/local-tax-offices/arizona/gilbert/67-s-higley-rd/2404?otppartnerid=9308&campaignid=pw_mcm_9308_9763', 'http://www.menchies.com', 'https://www.getpreloved.com', 'http://www.thainaka.ca/', 'http://retail.monroehardware.com/', 'http://cualv.com', 'http://www.starbucks.com', 'https://www.pteglv.com/locations/sean-patricks/deer-springs-5th-st/', 'http://www.mooseandbeargrill.com', 'http://assortedtablewine.com', '1592229204', 'https://www.haircutmenblakeneycharlottenc.com', 'https://www.basculebrewpub.com', 'https://www.aramarkuniform.com/our-locations/aramark-uniform-services-phoenix2', None, 'http://myloannailslasvegas.com', 'https://www.walmart.com/store/1093/rantoul-il/whats-new', None, None, None, None, 'http://www.mrsub.ca', 'https://www.metropcs.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.francisandsonscarwash.com/category/7889/francis-and-sons-car-wash-of-peoria-az.htm', 'http://www.swisschalet.com', None, 'http://www.nail-pretty.com', 'http://www.southernhighlandscyclery.com', None, 'http://www.wpxi.com', 'http://docromeopark.tk', 'http://guitartreemusic.com', None, 'http://www.dennysallnighter.com/', 'http://www.queencityclosets.com/', 'http://onestopnutrition.com', 'https://www.flavours-ajax.com', None, None, 'https://heightspestcontrol.com', 'http://vsnails.com', 'http://www.desertflowerseniorliving.com', 'http://www.reclaimedart.org', 'http://www.drdonnawoo.com/', 'https://www.sunsetgrill.ca', 'http://cuttingedgebarbershoplv.com', 'http://www.lllreptile.com/', 'http://www.cencispizza.com/wexford', '1592229284', 'https://www.gowireless.com/stores/az/scottsdale/scottsdale-scottsdale-fiesta/', 'http://haneysfamilyjewelers.jewelershowcase.com', 'http://www.evpools.com', None, None, 'http://www.activatechiropracticclinic.com', 'http://glendale.mensvitalitycenter.com/', 'https://www.potterybarn.ca', 'http://www.learningexpress.com/store/robinson/', 'http://www.letzrolloffroad.com', 'http://quickresponseplumbing.ca', 'http://hamptoninn3.hilton.com/en/hotels/arizona/hampton-inn-and-suites-phoenix-goodyear-PHXGYHX/index.html?SEO_id=YELP-HP-PHXGYHX', None, None, 'http://www.districtflats.com', None, 'http://www.urbanashop.com', None, 'http://www.goodwill.org', 'http://lvfurnituredirect.com', 'http://locations.maggianos.com/us/ohio/beachwood/beachwood', 'http://www.mobil1mentor.com', 'https://cocofreshtea.ca', 'https://www.uhaul.com/Locations/Truck-Rentals-near-Gilbert-AZ-85234/922024/', '1592229337', None, None, 'http://bk.com/', '1592229346', 'http://hkrealtygroup.com', 'https://www.sparkletts.com', None, 'http://isabelcleaning.blogspot.com/', None, None, 'https://www.jackentertainment.com/cleveland/', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'https://www.90degreesapartments.com', None, 'http://www.fourlakesfamilydental.com', None, 'https://www.choicehotels.com/north-carolina/matthews/sleep-inn-hotels/nc032/rates', None, 'https://valleywiderestoration.com', None, 'http://fitnesstogether.com/matthews?utm_source=Yelp%20Web&utm_medium=Yelp%20Web&utm_campaign=Yelp%20Web', 'https://www.albertsons.com/home.html', None, 'http://www.bk.com', 'https://www.videogameswap.ca', 'http://www.summitdentallv.com', 'http://pekingman.ca', 'http://ismiledentists.com', 'http://handymanphoenix.com', None, 'http://www.vocellipizza.com/pleasanthills_pa', 'https://stores.sportsmans.com/sportsmans-warehouse/us/az/avondale/10145-w-mcdowell-road', 'http://bennybbq.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://thepetclub.com', 'http://www.zuppasdeli.com/', 'http://bacusbroshotdogs.com', 'http://www.toysrus.com/family/index.jsp?categoryId=2255956&camp=MISC:yelp:store:page:tru:111910', 'http://www.cityofmadison.com/parks/parks/park.cfm?id=1207', None, 'http://www.dominos.com/', 'http://www.mindfulmotionphysicaltherapy.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.centralonbroadway.com/', 'http://www.stashcharlotte.com', 'https://locations.sonicdrivein.com/sc/tegacay/3039-highway-160.html', 'http://www.jtblandscapers.com/', 'http://www.exxonmobilstations.com/1721617-tesoro-62540-las-vegas#_62540-mobil-las_vegas', 'http://www.newimagephotography.com', '1592229448', 'http://sweetvirginiawax.com/', None, 'http://www.perfectflooringpgh.com', 'http://www.macmediainc.com', 'https://alohanailsblakeney.com', 'http://www.rinaldisdeli.com', 'http://www.wizardworld.com/cleveland.html', 'http://www.restaurantcrescendo.com/', None, 'https://www.performancemuffler.net', None, 'http://royal10cigar.com/', 'http://ashavecomics.com', 'http://www.brdiamondsuite.com', 'http://www.laurasecord.ca', '1592229483', 'https://5thavedeliandgrill.com', '1592229487', 'https://www.simon.com/mall/south-hills-village', 'http://www.positivechanges.com', 'http://www.aeacarizona.com', 'http://ajiichibansushi.com/', 'https://www.clearchoice.com/locations/dental-implants-las-vegas/?trans_id=webdisyelp&utm_campaign=Yelp&utm_source=YelpListing&utm_medium=Listing', None, 'http://www.dunkindonuts.com', None, 'http://www.superpumper.com', '1592229508', 'https://www.stance.com/retail/stance-scottsdale', 'https://www.tombstonetactical.com', 'http://www.lebouquetfloristandgifts.com', 'http://www.azmagician.com', 'http://www.whitehouseblackmarket.com/store/home.jsp?cmp=MOC_yelp', 'http://www.monroevillechryslerjeep.com', 'http://www.familylifecenters.org', 'http://locations.harristeeter.com/nc/charlotte/205/', 'http://365gymllc.com', 'https://milios.com', None, 'http://www.padellaonavenue.com', None, 'http://crurestaurant.ca', 'http://www.arizonapaintingcompany.com/', None, 'http://www.equityapartments.com/Arizona/Phoenix-apartments/Scottsdale/San-Marcos-apartments.aspx', '1592229547', 'http://www.calakmulaz.com', 'http://www.pamperedpoocheslasvegas.com', None, 'http://torontovintage.ca', 'http://www.preservationcleaning.ca', 'https://www.homedepot.com/l/West-Aurora/IL/Aurora/60506/1957?cm_mmc=seo-yelp&', None, None, 'http://www.glowbeautycharlotte.com', 'http://www.attorneyramsey.com', None, None, None, 'http://www.kjsdental.net', '1592229581', 'http://www.buchanans.ca', None, 'http://www.khaanakhazanaaz.com/', None, 'https://www.torrid.com/store-details?StoreID=3512', 'http://www.croatiarestaurant.ca/', 'http://kblossomshop.com', 'http://www.phoenixlosburritos.com', 'http://drorona.com/', None, None, 'https://www.signaturestyle.com/locations/wi/madison/cost-cutters-maple-grove-shopping-center-haircuts-16214.html', None, None, 'http://www.mountainpark.org', 'http://www.alamo.com/', 'http://sweettomatoes.com', 'https://nixrockstargarage.com', 'http://glownailbarhenderson.com', 'http://www.heartbreakersalon.ca', 'http://lahmajounsevan.com/', 'http://www.tapino.com', 'http://www.hotdogonastick.com', None, 'http://www.ScruffyToFluffyLV.com', 'http://www.sagamorehillsanimalhospital.com', '1592229642', None, 'https://www.wellsfargo.com', 'http://www.balzermotorworks.com', None, 'http://www.pdqstores.com/gmaps/Wisconsin/MadisonArea/131.html', 'http://www.meadows-chiro.com', 'http://www.levinfurniture.com/furniture-store/pa/pittsburgh/5846-forbes-ave', None, 'http://www.futurekiddie.com', None, None, None, None, 'http://www.starbucks.com', None, '1592229673', 'http://southpointcasino.com/landing?cid=ypsp', None, 'http://www.shigatsusushi.com', 'http://lasvegasballroomdancing.com', None, None, 'http://www.brakemasters.com', '1592229689', 'http://www.homehardware.ca/en/dealer-microsites/1582-7/contact-information.htm', None, None, None, 'http://www.ascentcle.church', 'https://www.5spice.ca', '1592229702', 'http://www.hairbymegananton.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.charlottepediatricclinic.org', 'http://mastcoffee.ca', 'http://strutbridalsalon.com/', 'https://doctors.bannerhealth.com/provider/Daniel+Arrhenius/776226', 'http://www.bk.com', None, 'http://www.ultralighting.ca', None, 'http://ak-chinpavilion.com', 'http://www.zara.com', 'http://amyhogan.ca', 'http://www.chilis.com/', 'http://www.rainstormcarwash.com', 'http://Moreheadtavern.com', 'http://www.camelbackwomenshealth.com', 'https://az.aaa.com/agent-office-locations/nv/las-vegas/aaa-west-lake-mead-branch', 'https://www.luvasub.com', 'http://www.citysmokeclt.com', None, 'http://www.nailparade.com', 'http://stores.sallybeauty.com/az/mesa/beauty-supply-mesa-az-3292.html', None, 'http://www.Jerksogood.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.accuratecargo.com', 'https://losprimosmeats.com', 'http://www.grottolounge.com', 'https://www.plazaexecutive.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=KL%20execsuite', None, 'http://matthewscarwash.com', 'http://www.bbqislands.net', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://zscreamandbean.com', 'http://www.waxmuseum.net/', 'https://www.dominos.com', None, 'http://www.mccleanerslasvegas.com', 'http://jcls.jazzercise.com/facility/carnegie-jazzercise-center', 'http://duclostoys.com', 'http://www.therightguyshvac.com', 'http://www.thegourmetcave.com', 'http://tremontfarmersmarket.com/', 'http://www.nemdelight.com', None, 'http://www.mohaveeyecenter.com', 'http://www.yumeisushi.ca', None, 'https://www.early-bird.ca', None, '1592229815', 'http://www.pbjpittsburgh.com', 'https://www.starbucks.ca', 'http://clickserve.dartsearch.net/link/click?lid=43700006435129272&ds_s_kwgid=58700000444951114&ds_url_v=2&matchtype=%7Bmatchtype%7D&device=%7Bdevice%7D', 'http://www.conditandassoc.com', None, 'http://www.immortalsinconline.com', 'http://www.butterchickenroti.com', 'http://www.kirklands.com', 'http://www.eatgarbanzoaz.com/', 'http://thevertigogames.com', 'http://www.stadtcafetoronto.com', 'https://locations.waxcenter.com/nc/charlotte/charlotte-arboretum-0169.html', None, 'http://www.privyskincare.com', 'https://www.koekkoekfood.com', 'http://www.burgundysrestaurant.com/', None, 'http://www.usagymaz.com', 'http://www.bodhitreeacupuncture.com', 'http://www.tendertouchmoving.com', 'https://thaiexpress.ca', 'http://www.tenniscabana.com', '1592229868', 'http://www.yogurt-land.com', None, 'http://drivenfix.com', 'http://www.rocknes.com/index.htm', None, 'http://www.econolodge.com/hotel-rock_hill-south_carolina-SC139?source=pmfyelp&pmf=yelp', 'http://www.petco.com', 'http://www.countryinns.com/scottsdaleaz', 'http://www.bigsurffun.com', 'https://studiocpb.mysalononline.com', None, 'http://www.acoresrotisseriechicken.com/', None, 'http://www.oglolas.com', 'https://www.idealimage.com/medspa-locations/pennsylvania/pittsburgh-area/pittsburghwexford', None, 'http://www.bojangles.com', 'http://www.5cinco5.com', 'https://www.indiapalace.ca', 'https://crownpublichouse.com', None, None, 'https://twomaidswexford.com', None, 'http://www.luxdesign.ca', 'https://www.arizonaanimalwellnesscenter.com', 'http://www.clarkcountynv.gov/Depts/parks/Pages/winchester-cultural-center.aspx', 'http://www.renewalbyandersen.com/window-company/720-phoenix-az.aspx/?utm_source=Yelp&utm_medium=localization&mm_campaign=B5863EAA37E0931E94FEB25E4DEEDB54&mm_replace=true&engName=social_Yelp', 'http://www.shopjustice.com', None, None, 'http://www.perfecttimingentertainment.com', 'http://www.bsyhairstudio.com', None, None, 'http://www.welcomepharmacy.ca', 'http://www.friendshipsuites.com/', None, 'https://locator.chase.com/az/cave-creek/28238-n-tatum-blvd', None, '1592229955', 'https://www.panerabread.com/en-us/home.html', '1592229959', 'http://www.crossfitfury.com', 'https://order.qdoba.com', 'http://infinitydentallv.com/', 'http://www.maintenance-pro.com', None, 'https://www.brennansmarket.com', 'http://jetspizza.com', 'http://www.onthegrill.us', 'http://www.myrivertowne.com', 'http://www.plapreschool.com', 'http://www.dpauljewelry.com', None, None, None, 'http://www.forgoodnesscakesaz.com', '1592229997', None, None, 'https://www.eyeglassworld.com/store-list/las-vegas-(s)-nv-89121/', None, 'http://www.swansofinglewood.ca', 'https://www.aqualineplumberselectriciansacrepairphoenixaz.com', 'https://www1.shoppersdrugmart.ca', 'http://www.elirelocation.com', None, None, 'http://www.premiumsweets.ca', 'https://earls.ca/locations/calgary-tin-palace', 'http://www.beadworldinc.com', None, 'http://www.eastsidecannery.com', None, None, 'http://www.pooranivilaas.ca', 'https://www.navyfederal.org/branches-atms/locations/usa/az/phoenix/cmb/', 'http://www.landainails.com', 'https://www.accountingprincipals.com/locations/mesa-az-accounting-staffing-agencies', 'http://tendershawarma.ca', 'http://www.cherryaz.com', 'http://www.theindiegathering.com', None, 'http://www.prontomexicanfood.com', 'http://www.greatclips.com/', 'http://lasvegasrealestatetoday.com', 'http://www.terribleherbst.com/', 'http://freshdelicious.net/', 'http://www.selfconnection.ca', 'https://dealer.sprint.com/experts-choice', 'http://greatlakesbakingcompany.org', None, 'http://www.threedogbakerylkn.com', 'http://www.kiedrowskibakery.com/', None, 'http://www.boernercompany.com', 'https://www.ihop.com/en/restaurants-streetsboro-oh/9700-state-route-14-3606', '1592230084', 'http://pavelsjewelry.com/', 'http://www.suspectvideo.ca', 'http://www.tomasos.com', 'http://www.orenisbetter.com', 'http://az-ent.com', None, None, None, 'http://www.thespottedcouch.com', None, 'https://giorgio.ca/fr/restaurants/succursale/laval/', None, 'http://chiropractorpeoriaaz.com', None, 'http://www.gandgmarine.com', 'http://www.azdot.gov/mvd/', 'http://reeis.com', None, 'http://www.cuffsclothing.com', 'http://www.candomoving.com', 'http://www.cappysonwalnut.com', None, None, 'http://baraongles.ca', 'http://www.ellenmerkinod.com/', 'http://www.lynnbuss.com?cmpid=lvji_blm_0006', 'https://www.unitypoint.org/madison/home-health.aspx', None, None, 'https://www.ditkasrestaurants.com/Locations/Wexford', 'http://embassysuites3.hilton.com/en/hotels/pennsylvania/embassy-suites-by-hilton-pittsburgh-international-airport-PITATES/index.html', '1592230150', 'http://greenvalley.schoolofrock.com/?labelSource=Yelp&utm_campaign=yelp&utm_source=yelp&utm_medium=yelp&utm_content=yelp', None, 'http://www.lamexicana.ca', 'http://www.mymexquite.com', None, None, 'http://www.bayti.ca', None, 'http://www.rhythmcatsshow.com', None, 'http://bcpoolsupply.com', 'http://www.besolasvegas.com/', 'http://www.allclimatenc.com', 'https://www.wellsfargo.com/locator/bank/32__CATAWBA__ST_BELMONT_NC_28012/', 'http://www.innzonerainbow.com', 'http://www.airparkbike.com/', 'http://www.tolmtaz.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/3825+Monticello+Boulevard-Cleveland+Heights-OH-44121/storeid=3337?cid=ps_ylp_2020', 'http://www.breathinghands.com', 'http://www.tropicalsmoothiecafe.com', 'http://phoenixnap.com/', 'http://www.canadel.ca', None, 'http://www.rogerkdowney.com/?cmpid=rkr3_blm_0006', 'http://www.caldensebakery.ca/', None, 'http://www.cegdealers.com', 'http://www.toysrus.ca', None, None, None, None, None, 'http://www.lvprestigecleaning.com', None, 'http://www.chtoyota.com', 'https://www.petersendean.com/?utm_source=yelp&utm_medium=referral', 'https://locations.waxcenter.com/pa/mccandless/mccandless-0287.html', 'http://www.turcopersian.com', 'http://www.haji-baba-tempe.com/Haji-Baba.com', None, None, 'http://www.jojosmenu.com', 'http://www.proficientclean.com', None, 'https://www.popeyes.com/store-locator/store/restaurant_82776', 'https://www.mattressfirm.com', 'http://www.snohc.com', 'http://www.napaautopro.com/auto-repair/alberta/calgary-renfrew-auto-service/', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://www.ezpawn.com/locations/nv/henderson/10075-eastern', None, 'http://www.ultimatecrc.com', 'http://www.camelbackcab.com/', 'https://www.duluthtrading.com/locations/?city=Belleville&StoreID=004&street-address=1107RiverSt.', 'http://www.chandlersmiles.com', '1592230275', 'https://www.forever21.com', None, None, None, 'http://www.nextcare.com/locations/az/98th/?utm_sourceYelp&utm_mediumLocal&utm_campaign98th', None, None, 'https://www.mattressfirm.com', 'http://www.gianteagle.com', None, 'http://www.boltonpizzapanini.ca', 'http://dms.netmng.com/si/cm/tracking/DGTrackingRedirect.aspx?siclientid=2918&siredirect=http%3a%2f%2fwww.extendedstayamerica.com%2fhotels%2foh%2fakron%2fcopley-east&sisearchengine=1524&siproduct=2557_Bus', None, 'http://www.thanglong.ca', 'http://azfamilydentistry.com', 'http://www.deltaco.com', 'http://www.awakennutrition.com', 'http://www.alphasdiscoveryclub.com/indoorplayground', '1592230318', 'http://www.flourandbarley.com', None, 'http://www.nevadacareerinstitute.com', None, 'https://www.enterprise.com/en/car-rental/locations/us/az/fountain-hills-5029.html?mcid=iyp:8368400', 'http://sonedental.com', 'http://www.get-go.com/storelocator/storedetails.aspx?storeid=294', None, 'https://www.goodyearautoservice.com/en-US/shop/AZ/Goodyear/000908232-85338-1430', None, 'http://www.atlasespressobar.com', 'http://www.foothillsrehab.com/', None, 'http://www.aliveandfreehypnosis.com', None, 'http://www.pizzaiolo.ca/?utm_source=Yelp', None, 'http://floorshop.ca', None, 'http://www.southcharlotteanimalhospital.com', None, 'http://www.morfurniture.com', 'http://www.royalpgh.com', 'http://navarautoandtire.com', 'https://www.jerseymikes.com/8009', '1592230379', 'https://www.hilton.com/en/hotels/clehtht-home2-suites-cleveland-beachwood/?SEO_id=YELP-HT-CLEHTHT', 'http://www.kay.com', '1592230386', 'https://www.davidcreechmd.com', 'http://www.rabbatphoto.com', 'http://eddysautoservice.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYZOT', 'http://www.estrestaurant.ca', 'https://www.180automotive.com', 'http://www.proaquatic.ca/', 'http://www.fastbiryani.com', None, 'http://asairoofing.com', 'http://www.lightrx.com/arrowhead', None, 'http://demetres.com', 'http://www.cre8iveevents.com/', 'https://www.4patientcare.ws/wsv3pro/web/webschedulerv3.aspx?SessionID=2050212', 'https://www.bcautospecialists.com/index.php', 'http://www.JimmyJohns.com', 'http://www.stonetheatres.com/#/sun-valley-14/4579599036', 'https://canyonstateac.com', 'http://www.inexcreativefloors.com', 'http://www.hotelvalleyho.com/vh-spa', None, 'http://www.rhodesranchgolf.com', 'https://www.isluxury.com', 'http://www.yogastyles.com', 'https://pho-oc-vietnamese-restaurant.business.site', None, None, 'http://www.arrowheadoralsurgery.com', '1592230459', 'http://www.fnfcarpetcleaning.com', None, None, 'http://www.slashsalon.com/', 'https://www.mgmresorts.com/en/hotels/united-states/mgm-northfield-park.html', 'http://musa-restaurant.business.site', '1592230477', 'http://www.wellsfargo.com', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://gourmetgringos.com', 'http://www.enlivant.com/communities/arizona/kingswood-place-surprise/?utm_campaign=local&utm_medium=Y&utm_source=listings', 'http://www.tiltedkilt.com/', 'http://www.azskindoc.com', 'https://www.tranquilwatersmedspa.com', 'http://www.KulturaToronto.com', None, 'http://www.kristinachartier.com', '1592230508', 'https://www.theprosperitygoddess.com', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FCLTCG', 'http://www.walmart.ca/en/walmart-meadowvale-supercentre/3054', 'http://www.harlemglobetrotters.com/', 'http://www.affordableappliancerepair.net', 'http://phoenixchildrens.org?utm_medium=local-listing&utm_source=chatmeter', None, None, 'http://www.avenuetaphouse.com', 'http://www.torrid.com', 'https://whiteoakanimalsafehaven.com', None, 'http://www.campusdish.com/en-US/CSW/UNLV', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.boomerangpilates.ca', 'http://www.lvsubzero.net', 'http://genghisgrill.com', 'http://www.pinkhouseboutique.com', 'http://www.sdmattress.com', 'http://www.prairiewindsvet.ca', None, None, None, None, 'https://www.pteglv.com/locations/pts-gold/st-rose-eastern/', None, 'http://theloftmedspa.healthcare', None, 'http://gottabegarys.com', None, 'http://www.thebiddlelawfirm.com', None, '1592230586', None, 'http://www.cookiesbygeorge.com', None, 'http://aquinolawgroup.com', 'http://skinnywimpmoving.com', 'http://locations.whichwich.com/AZ/Tempe/Tempe-Gateway', 'http://clevelandfurniturebank.org', 'http://thedizzy.ca/', 'http://www.libertyclinic.com', 'http://www.windowtreatmentslasvegasnv.com', None, 'http://www.portofsubs.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', 'http://www.familydollar.com', 'https://visionsource-studio-2020.com', 'https://locations.traderjoes.com/az/tempe/93/', None, 'https://www.galleryhiart.com', 'http://dyslexiacenter.com', 'http://www.latinmasstimes.com/Nevada/446', None, 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=891&utm_content=bpas', 'http://pipscafe.com', None, None, 'http://www.royaloakdentalcalgary.com', 'https://www.choicehotels.com/arizona/tempe/comfort-inn-hotels/az498/rates', None, None, None, None, 'http://www.aquasealwaterproofing.com', 'http://www.lovinghut.ca', 'http://www.rhinoexterior.com', 'http://store.concept1.ca/store/', 'http://havenhg.com', 'http://WorldsTallestChiropractor.com/', 'http://www.leahs.ca', 'http://thevaporsocial.com/', 'http://www.drpalluck.com', 'http://www.sunsetstation.com/dining/hooters/?utm_source=YELP&utm_medium=SOCIAL&utm_term=SS&utm_content=SS-PROPERTIES&utm_campaign=SS-YELP', 'http://www.newsonganthem.org', 'http://www.rosynails.ca', 'http://www.touchofthai.info', '1592230681', 'http://www.statebicycle.com', 'https://www.bodynbrain.com/scottsdale', 'http://greatscience.com/explore/exhibits/william-g-mather-steamship', 'http://www.arizonacharliesdecatur.com/', 'http://www.trainerjo.ca', 'http://www.myhenrybrown.com/?cs:pro=repman&cs:e=yelp', None, None, 'http://www.g1tour.com', None, 'https://urgentvet.com', 'https://www.mgmgrand.com/en/entertainment/grand-garden-arena.html', 'http://www.blimegreen.com', None, 'http://www.murgatroid.to', 'https://www.segwayofscottsdale.com', 'http://www.iandjfountainrestaurantaz.com', '1592230719', None, None, 'http://www.renegadecoffeeaz.com', None, 'http://www.earnhardtcjd.com', None, 'https://www.target.com/sl/north-olmsted/2016', 'http://schnitzelhub.com/', None, 'http://terramassagepgh.com', 'http://madjackspub.com/', None, None, '1592230745', 'https://www.mountainshadows.com', 'http://www.dukesupperdeck.com', None, 'http://www.qdoba.com/', None, 'https://nevadaeyephysicians.com/Henderson-Office', 'https://www.lebos.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'https://www.jetblue.com', 'http://doughboyspizzaandwings.com/', 'http://www.gordon-law-offices.com', 'http://www.callahans-flooring.com', 'https://madison.floorcoveringsinternational.com', None, None, 'http://www.festivalofcolorsusa.com/festival-of-colors/festival-colors-las-vegas-nv/', 'http://www.gigglingtomatoes.com', 'https://www.dominos.com', 'https://jose-cansecos-showtime-car-wash.business.site', 'http://lvpaiutesmokeshop.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FMSNME', 'https://wholehealthphoenix.com', 'http://www.salondecheveux.com', 'https://www.sportchek.ca/stores/hillcrest-mall.html', 'http://www.thebrooklineplumber.com', 'http://www.arcadiafarmscafe.com', None, None, 'http://lasvegashears.com/audiologist-las-vegas/', 'http://carlitosmokes.com', None, None, 'http://www.mypittsburghlocksmith.com/', 'http://goodwillsp.org/shop/retail-stores/weddington-road', None, 'http://www.magiclaserandaesthetics.com/', None, 'https://www.nextdoorwinebar.ca', 'http://store-locator.barnesandnoble.com/store/2212', None, 'http://www.fallasstores.com', None, None, 'http://www.coopercoons.com', 'http://www.aatexlawn.com', 'https://zacharylawgroup.com', 'https://local.executivecoin.com', 'http://www.azrealsolutions.com', 'http://www.thunderbirdim.com', 'https://www.insynco.com', 'http://www.drstevensabatino.com', None, 'http://www.lasvegasdogtrainers.com/', 'http://progressivedentalandortho.com/', 'https://www.chapmanhyundaiphoenix.com', None, None, 'http://www.a-natural-u.com', 'https://www.redstarvapor.com/stores/vape-shop-gilbert/', 'http://www.hcpnv.com', 'http://www.twtdbooks.com', 'https://www.stylistics.net', None, 'http://www.ashandkris.com', None, 'http://Www.SerenityRichmond.com', None, None, 'http://www.patrickssigns.com', 'http://www.havanacafe-az.com/', 'http://www.mogheltandoori.com', 'http://alpremium.ca/', 'http://www.wincofoods.com', None, None, None, None, None, 'http://www.mygreentreeapts.com', 'http://www.bearcreekaz.com', None, 'https://fixautousa.com/las-vegas-craig', 'http://www.inmotionstores.com', 'http://www.cadencecoffee.com', None, None, 'https://randrpaintingaz.com', 'http://www.labcorp.com', 'http://www.sunautoservice.com/auto-repair/nv/henderson', None, 'http://www.dubinwellness.com', None, 'https://www.alamo.com/en_US/car-rental/locations/location-details/US/PITT71-pittsburgh-international-airport.html?iataNumber=AL022237&affiliateRes=Y&cm_mmc=Yelp-_-IYP-_-National-_-null', 'http://www.pitaland.ca', 'http://www.tuttigelati.com/', 'http://www.burroburrito.com', None, 'http://terravitavet.com/', None, 'https://www.wingilariver.com/index.php/vq-vee-quiva-hotel-casino/dining/fatburger', 'http://imperialexcavatingaz.com', None, 'http://www.bartholomewphotography.com', 'http://lucillemadison.com', None, None, None, 'http://www.starbucks.com/store/9334/', 'http://www.decoraport.ca/francais/', 'http://www.johnnyrockets.com', 'http://boucherville.delfriscos.ca', 'http://beerbatter.ca', 'http://www.quiznos.ca', None, None, None, 'http://withoutwalls.tv/', 'http://Www.kristinlloy.com', 'http://www.letswrap.biz', 'http://www.truevalue.com/hometruevalue', 'http://www.allmetalsrecyclingllc.com', None, 'http://www.jumprestaurant.com/', None, None, 'http://www.chandlercustomcycles.com', None, 'http://www.bwh-services.com', None, 'https://locations.dignityhealth.org/oasis-hospital-phoenix-az', None, 'https://www.smithsfoodanddrug.com/stores/details/706/00370', 'https://locations.panerabread.com/oh/bainbridge-township/8480-east-washington-street.html', None, 'http://www.fred-vincentvw.com/', 'http://www.audreyjanellebeauty.com', None, 'http://www.lauraleesalon.com', 'https://www.bartlett.com/locations/Phoenix-AZ.cfm/?yelp_ads_lop%C3%8A', 'http://appliance-repair-pittsburgh.pa-biz.com', None, 'http://www.starbucks.ca', 'https://sandees-water-n-ice.business.site', None, 'http://www.orvis.com/cleveland', 'http://www.acehardware.com/store-details/06234', 'http://www.fantasticsams.com', None, 'https://www.availvapor.com/north-olmsted', 'http://www.georgiossalonspa.com/', 'http://www.sexyplusclothing.com', '1592231074', None, 'http://eatmegourmetcatering.com', 'http://www.onceuponachildstow.com/', None, None, 'http://www.pianotuneraz.com', 'http://www.framersworkshop.net', None, 'http://www.sunglasshut.com/us/?cvosrc=display.yelp.LasVegas', None, '1592231097', None, None, 'https://www.mandalaybay.com/en/nightlife/rhythm-riffs.html', 'https://www.walmart.com/store/4356/las-vegas-nv/details', 'http://www.bosmeadery.com/', 'https://www.steaknshake.com', None, 'http://www.dolcelucano.com/', None, 'http://storelocator.pepboys.com/pepboys/stores/phoenix/cactus/00747', 'http://www.jumparoundpartyjumpers.com', 'http://www.NextLevelTreeLLC.com', None, 'http://www.jimmyscoffee.ca', 'http://www.bigotires.com/regions/Nevada', None, 'http://www.onegooddoctor.com', 'https://www.aqualityelectric.com', 'http://www.actionappliancepitt.com', 'http://www.fetishfalls.com', None, 'https://www.camdenliving.com/scottsdale-az-apartments/camden-foothills?cm_mmc=Yelp', None, 'http://www.themilkshakefactory.com', 'http://www.opalasvegas.com/', None, 'http://relaxologywellness.com', 'http://www.greatclips.com/', 'http://l.macys.com/biltmore-fashion-park-in-phoenix-az?cm_mmc=Yelp_Stores-_-Phoenix-_-n-_-451&m_sc=sem&m_sb=yelp&m_tp=text&m_cn=Phoenix_451&m_ac=yelp_stores&m_ag=n', None, None, 'http://hmshost.com', 'http://www.pagoda.com', None, 'http://www.2fixmyair.com', 'http://lazioitalianbakery.com/', 'https://www.flyfrontier.com', 'http://warraichmeats.com/', 'http://bhelwalaindianfood.com/', 'https://www.enterprise.com/en/car-rental/locations/us/nv/north-las-vegas-craig-rd-5404.html?mcid=iyp:8368400', None, None, None, 'https://azvent.com/locations/estrella.php', None, None, 'http://www.ahealershand.com/', None, 'http://www.thedelevagroup.com', '1592231200', None, None, 'http://www.libertywildlife.org/', None, None, 'https://www.emilianos.net', None, 'http://www.klassynailscalgary.com', 'https://stores.cortfurnitureoutlet.com/arizona/chandler/150-south-kyrene-suite-1/?utm_source=yelp&utm_medium=organic&utm_campaign=localmaps', 'http://www.pintoranch.com', None, 'https://locations.timhortons.com/ca/on/markham/5000-hwy-7-e.html', 'http://www.institutefordriverssafety.org/', None, 'http://allautobattery2007.com', 'http://www.paulstailors.com/', None, None, 'http://www.purplesagesalon.com', 'http://www.rockwoodnaturalmedicine.com/', None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'https://www.westelm.com', 'http://robertostacoshop.com', None, 'http://www.banfield.com/veterinarians/nc/charlotte/bky?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89015/601-in-henderson/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-601&utm_content=website-link&howheard=IYE', 'http://banditolv.com', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=352&utm_content=bpas', 'http://www.bigotires.com/regions/Nevada', 'https://www.appletreedentalforkids.com', 'http://www.lizzynco.com', 'http://www.skylarparkeraz.com', None, 'http://www.btvape.com', 'http://www.tenrestaurantaz.com', 'http://www.zoestrunk.com', None, 'https://www.popalock.com', 'http://www.zaborfh.com', 'https://www.sunamg.com', None, 'https://www.awomanschoiceinc.com', 'https://www.walmart.com/store/2720/madison-ms/whats-new', 'http://www.cannonschophouse.com', 'http://www.school-for-massage.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216494&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', None, 'http://www.aquariuslandscapinglasvegas.net', None, 'http://www.snowdentistry.com', None, 'http://www.westhillsnissan.com', 'http://www.marcos.com/', 'http://bermanchiro.net', None, 'http://www.lcbo.com/lcbo-ear/jsp/storeinfo.jsp?STORE=629&language=EN', 'http://www.myazsmile.com', 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89148/591-near-spring-valley/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-591&utm_content=website-link&howheard=IYE', None, '1592231336', None, None, 'http://agentpestcontrol.com', 'http://www.gilleyslasvegas.com', 'http://www.drsmarkantone.com', 'http://www.dollarama.com', 'http://www.vbsconstruction.com', 'https://locations.wendys.com/canada/ab/calgary/7109-macleod-trail-s-w', 'http://www.rockruninn.com', 'http://www.jamburritos.com', 'http://www.icenowaz.com', 'http://www.thebarrelinn.com', 'http://www.pierdeorleans.com', None, 'http://www.transitionsquad.com/', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.hennashoppe.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'http://www.buffalophoto.net', 'http://www.skyzone.com/ca/mississauga', 'https://www.destinationhotels.com/tempe-mission-palms/dining/tempe-restaurants', 'http://www.summitfamilydentist.com', 'http://www.notoriousautodesigns.com', 'https://www.wellsfargo.com', 'http://www.walmart.com/', '1592231397', 'https://centralusa.salvationarmy.org/danecounty', 'http://www.thetheodore.beer', 'http://www.michaelruffattodmd.com', 'http://www.quiktrip.com', 'http://www.metropolis-living.com/', 'http://www.buildabear.com?sc_cid=ds:yelp:profile', None, '1592231415', 'http://lagourmandinebakery.com', 'https://www.artclassesforkids.com', None, 'https://www.coulternissan.com', 'http://www.harborfreight.com', 'http://www.loveandcaresitters.com/', 'http://scottsdaleprinting.org', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.wallacefloral.com', 'http://kellyclark.com', 'http://www.papa-luigi.com', None, 'https://www.wellsfargo.com', 'https://nativegrillandwings.com/location/pebble-creek/?utm_source=Yelp&utm_medium=Local&utm_campaign=Pebble_Creek', 'http://www.skinnydippingpools.com', 'http://www.kfc.com', '1592231454', 'https://eyetique.com/squirrel-hill', None, None, 'http://www.bargainballoons.ca', 'http://www.sonicdrivein.com/', 'https://www.erbertandgerberts.com/locations/?store_address=53703', 'http://www.papagnos.com', None, 'http://www.johnnymcguires.com/vegas/', 'http://www.beaumontalignment.com', None, 'https://www.dapperandstout.com', 'http://www.uhhospitals.org/north-olmsted/urgent-care', 'https://www.barnettfalls.com', None, 'http://www.greatdanepub.com/', 'http://www.skodagordon.com', 'http://www.raffmobilemechanic.com', '1592231495', None, 'http://www.maac.com/south-carolina/columbia/the-fairways?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'https://www.walgreens.com/locator/walgreens-8816+e+pinnacle+peak+rd-scottsdale-az-85255/id=3602', None, 'http://www.dannburg.com', None, None, None, 'http://routes.ca', 'https://locations.tiaa.org/nc/charlotte/8500-andrew-carnegie-blvd?tc_mcid=bn_yelplocal_0618', 'http://sweetpeacafe.moonfruit.com/', 'http://www.EveryKidsDentist.com/?sc_cid=Yelp_Listing', 'http://www.healthyplanetcanada.com/stores/index/view/store/13', None, 'https://squarefeat.com', 'http://www.advancedtherapeutics.ca', 'http://www.eighteeneight.com/beachwood-oh', '1592231531', 'https://www.moes.com/find-a-moes/pennsylvania/wexford/100189?utm_source=yelp&utm_campaign=yelp&utm_medium=referral', None, 'http://www.arizonabestmaids.com', None, 'http://www.amournailsandspa.com', None, None, 'https://www.hilton.com/en/hotels/phxgthx-hampton-suites-phoenix-gilbert/?SEO_id=YELP-HP-PHXGTHX', 'https://www.cooperfamilydentistryaz.com', None, 'http://www.hendersonchevrolet.com', 'http://www.dentalassistingaz.com', 'http://www.senecacollege.ca', None, 'http://philthyphillys.com', 'https://www.kaltire.com/en/stores/locations/?StoreID=694&source=yelp_track', 'https://www.restoration1ofgreaterlasvegas.com', None, 'http://www.arizonalasik.com/', 'http://www.downtownhomeinn.com', 'https://stores.petco.com/pa/pittsburgh/pet-supplies-pittsburgh-pa-1812.html', 'https://www.saigeboutiquesalon.com', 'https://barbaraannsalterations.com', 'http://www.vivitaspa.com', None, 'http://footreflexologychandlergilbertaz.com/', 'http://www.rycoplumbing.com', 'https://www.petpeoplestores.com/strongsville-store.html', 'http://www.paulsacehardware.com', None, '1592231602', 'http://www.thegrubclub.com/a/daddydukes', 'http://mesaunderground.com/', 'https://refinedlightingaz.com', 'http://www.thechildbirtheducator.com', None, None, None, None, 'http://www.desertstorage.com/scottsdale-az-storage-units', None, 'http://www.petiterestaurant.ca', None, None, '1592231633', 'http://www.falcononeclt.com', 'http://www.fracascleveland.com/', 'https://stores.arhaus.com/oh/akron/2246-west-market-street.html', 'https://www.publicstorage.com/north-carolina/self-storage-charlotte-nc/28210-self-storage/1196?pid=wyelp&CID=1245&CHID=AFL', 'https://www.lolplay.ca', 'https://www.desertfinancial.com/Home/BranchDetails?branchId=101', 'http://www.somethingnewmedia.com', 'http://www.mughaircuts.com/', 'http://www.pcmloan.com', None, None, 'http://sanfrancescos.com', 'https://www.thephoenician.com/phoenician-tavern/', 'http://www.aeopowersports.com', 'https://www.redlobster.com/locations/list/az/peoria/7921-west-bell-rd', 'http://locations.arbys.com/us/nv/north-las-vegas/1465-w-craig-rd.html', None, 'http://www.usps.com/', 'https://az.aaa.com/agent-office-locations/az/phoenix/aaa-paradise-valley-auto-repair', None, 'http://www.ellamentssalon.com', 'https://shuyi.ca', 'http://www.macxn.com/stores/madison-wisconsin', 'http://pedispanailbar.com', 'http://www.vaudreuilvolkswagen.com', 'https://www.anytimefitness.com/gyms/2160/Scottsdale-AZ-85266/?utm_source=yelp&utm_medium=local', 'http://www.mcdonalds.com', 'http://www.patriotmovers.us', 'http://www.furnitureappliancemart.com/current/Stores/Madison_West_Wisconsin_Furniture_Store.aspx', '1592231701', 'http://www.yorkvilleespressobar.com', None, 'http://www.italianvillagepizza.com/', None, None, None, 'http://www.rossstores.com', None, 'http://www.honeycombedhair.com', None, None, 'http://frostyvalleygolf.com/', 'http://www.builderzwarehouse.com', 'https://www.callawayirrigation.com', 'http://www.Boothbuilt.com', 'https://www.lalasercenter.com/centennial/?utm_source=yelp&utm_medium=centennial&utm_campaign=centennial%20dermatology%20and%20laser%20cancer', None, None, 'https://www.idealimage.com/medspa-locations/nevada/las-vegas-nv-area/lasvegassummerlin', 'http://www.epoxyfloorsphx.com', None, 'http://www.bmdprinting.com', 'http://www.warfieldhairbar.com', 'http://www.jandsautoservice.com', 'http://link.hertz.com/link.html?id=46190&LinkType=HZLK&TargetType=Homepage', 'http://Ljhaircuts.com', 'https://www.caesars.com/flamingo-las-vegas?utm_campaign=Yelp&utm_source=yelp&utm_medium=local&utm_term=FlamingoLasVegas&utm_content=hotel', 'http://www.bitsandbytesllc.com', 'http://www.timhortons.com/', 'https://stores.dickssportinggoods.com/nv/las-vegas/1266/', 'https://www.checkcity.com/locations/nevada/lake-mead-and-torrey-pines/', 'https://www.zerorezmadison.com', 'http://www.sheppardvillagedental.com', None, 'http://www.avenue.com', '1592231786', None, '1592231790', 'https://www.camdenliving.com/tempe-az-apartments/camden-tempe', None, 'http://www.qdoba.com/', None, None, 'http://www.pizzahut.ca/', 'http://www.cafelabella.com/', None, 'http://www.agassiprep.net', 'http://www.scarlettplus.com', 'https://gilberteast.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.planetfitness.com/gyms/madison-west-broadway-wi-797?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://lisnoodles.com', 'http://idealcustomflooring.net', '1592231823', 'http://www.picanhalv.com', None, None, 'http://elpalenqueeastlake.com', None, None, 'http://universitycoinandjewelry.com', 'http://www.rajskitchennc.com', 'http://www.strongsvilleheating.com', None, 'http://www.thebutcherspantry.com', 'https://locations.dignityhealth.org/emergency-room-at-arizona-general-hospital-glendale', 'http://www.artisticalloys.com/', 'http://www.silkmedspa.ca/', 'http://www.arizonalodgingexperts.com/owner-support/scottsdale-az-property-management/', 'http://allntattoo.com', 'http://www.belleetboeuf.com', 'http://www.spice9cuisine.com', 'http://lensfactory.ca', 'http://www.mark-taylor.com/arizona/stone-oaks/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'https://www.aldoshoes.com/ca/en', 'http://www.shopartmart.com', 'http://www.womeninneed.net/', 'https://www.atozdiscountdepot.com', 'http://lifetimegaragedoorsaz.com', 'https://www.alphaandomegadogtraining.com', 'http://ajustbrewcoffee.org', 'https://www.brooklynbedding.com/showrooms/arizona/phoenix-buckeye/?utm_source=yelp.com&utm_medium=referral&utm_campaign=yelppage', 'http://www.yellowjersey.org/', None, None, 'http://www.cafelouise.ca', None, None, 'http://www.izzazu.com', 'http://www.posterjack.ca', 'https://local.firestonecompleteautocare.com/nevada/las-vegas/3770-s-maryland-parkway/', 'http://orangeleafyogurt.com/frozen-yogurt-store/OH/gates-mills-/6679-wilson-mills-rd?id=384', 'http://www.zaidmoving.com', 'http://www.exoticnailstouch.com/', 'http://www.cartelbrewery.com', 'http://www.greasemonkeyhuntersville958.com', 'http://dresslersrestaurant.com', 'http://superdupermovers.ca', 'http://aquasportscuba.com', 'http://mortlandpt.com/', 'http://www.firstimpressionnails.ca', 'http://www.coffeepubs.ca', 'http://www.empirebagels.com/', None, None, 'http://www.mcramyun.com', 'http://www.ledesmafootandankle.com', 'https://www.bobevans.com/our-restaurants/locations/146?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'https://www.papillon.com/las-vegas-tours/?utm_source=yelp&utm_medium=cpc&utm_campaign=papillon-las-vegas', 'http://www.bytesbackdatarecovery.com', 'https://www.macu.com/Branch-Locations/AZ/Mesa/Mesa-Branch/0033', None, None, None, None, 'https://www.regmovies.com/theatres/regal-manor-twin/1841?utm_medium=social+network&utm_source=yelp+listing&utm_campaign=Regal+Manor+Twin', None, 'http://www.barneys.com/location#!/stores/14', 'http://www.getenergysmarter.org', 'http://hooterscasinohotel.com', None, None, None, None, 'http://www.dianasseafood.com', 'http://www.theloadeddog.ca/', 'https://www.restaurantszibo.com/les-restos/boisbriand', 'https://www.prolinkprotection.com', None, 'http://www.hairhappiness.com', None, 'https://www.playfalcongolf.com', 'http://www.cpracademylv.com', 'http://www.quiktrip.com', 'http://www.kennelcarevet.com', None, 'http://www.athletico.com/locations/urbana/', 'https://www.sgtclean.com', 'http://brucehalllaw.com', 'http://www.westernskiesgolf.com/', 'http://homewoodsuites3.hilton.com/en/hotels/ontario/homewood-suites-by-hilton-toronto-mississauga-YTOHWHW/index.html?SEO_id=YELP-HW-YTOHWHW', None, 'https://www.medicentres.com/chiropractic/', None, '1592232037', 'http://home.vivoli.ca/', 'http://www.ncdentist.com', None, 'http://www.poetryjazzcafe.com/', 'https://locations.tropicalsmoothiecafe.com/nv/henderson/10612-south-eastern-avenue', 'http://www.malleys.com', None, 'http://www.junk-works.ca/locations/toronto-north', None, None, 'https://www.frontieraptmovers.com', 'https://www.hattendo.ca', None, 'http://www.housecallnotary.ca', 'http://bunsofsteelbootcamp.com/', '1592232072', None, 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=118&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-118', 'http://thekeyrealtyteamaz.com', 'http://www.arcadiachiro.com', None, 'https://www.180smoke.ca/vape-shop-etobicoke-islington-electronic-cigarettes', None, 'http://www.soulpepper.ca/', None, 'http://www.autotruckandfleetservices.com', 'http://hiltongardeninn3.hilton.com/en/hotels/north-carolina/hilton-garden-inn-charlotte-concord-CLTCOGI/index.html?SEO_id=YELP-GI-CLTCOGI', 'http://igniteperformance.net', 'http://www.smartbookingtravel.com/', None, 'http://www.sashandbustle.com/', 'http://www.bestratemtg.com', 'http://hibachisan.com/', None, None, 'http://www.acneadvisors.com', None, None, 'https://www.classictan.biz', 'http://www.cityofhenderson.com/henderson-happenings/parks-and-trails/locations-and-features/reunion-trails-park-and-amargosa-trailhead', 'http://juicdlife.com', '1592232129', 'http://www.tutortime.com/your-local-school/phoenix-az-6086/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', 'http://www.letstalktherapy.com', 'http://www.downtownerlv.com', 'http://www.melrosekitchenaz.com', 'http://www.somethinsweet.ca', None, None, 'http://www.climateprollc.com/', 'http://Www.artistichandsmassage.com', 'https://locator.chase.com/az/scottsdale/15415-n-65th-pl', 'http://www.scottsdaleeyesurgerycenter.com', 'https://www.brewerairporttoyota.com', 'http://springfieldgolfresort.com/', None, 'http://www.bodyworkscentral.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.elpasobarbeque.com/', None, 'http://www.mitchiesmunchies.com', 'http://www.jardinalex.com/?srccode=yelp_track', 'http://www.homesmartinternational.com', 'https://mkbjj.com', 'http://www.outside-inspirations.com', 'http://www.mayereyecarelv.com', 'https://kimmynailsandspa.weebly.com', 'http://www.tree-removal-phoenix.com', None, None, 'http://www.e-mg.com', 'http://www.amayaexpress.com/', None, 'http://www.winners.ca', None, 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FLASGR', '1592232211', 'https://www.loblaws.ca/store-locator/Glen+Erin+Drive%2C+Mississauga/1011?storeId=1011', 'http://www.tamalerialaurita.com', None, '1592232222', None, 'http://www.CharlotteHealthCenter.com', 'http://www.ihg.com/intercontinental/hotels/gb/en/cleveland/clehb/hoteldetail', 'http://www.rockymountaintails.ca', None, 'http://www.meineke.com', 'http://www.semprepizzapasta.com', 'http://symposiumcafe.com/woodbridgerestaurants.html', None, 'https://elementsmassage.com/Phoenix-Arcadia', 'https://graylinelasvegas.com', '1592232247', None, None, 'http://iwearthebest.com', 'http://www.repeteslv.com', 'http://www.managestaff.com', None, 'http://www.urbanpalm.com', 'http://www.livelifeblended.com', 'http://www.empire.edu/cosmetology-schools/north-carolina/charlotte-pineville', None, 'https://rightspacestorage.com/self-storage-locations/nevada/las-vegas-nv-storage-units/3043-N-Pecos-Rd/', None, 'http://www.phxortho.com', None, 'http://www.petvalu.com', 'http://hummusfactorylv.com', None, None, 'http://www.pelicanssnoballs.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/4755+West+Ann+Road-North+Las+Vegas-NV-89031/storeid=2989?cid=ps_ylp_2020', 'http://nvpicapart.com', 'http://www.mrosmow.com/', 'http://www.lofgreens.com/', None, 'https://rtcatcafe.org', 'https://www.surpriserental.com', 'http://tjmaxx.tjx.com/store/stores/storeLocator.jsp?_requestid=62549', 'http://www.brooksbrothers.com/?CMP=SMC_Yelp_6006', 'http://www.btbouncies.com', 'http://www.tinnenfamilychiropractic.com', 'http://www.blushlane.com', 'http://archwaychandler.greatheartsacademies.org', None, None, 'http://www.grillies.ca', None, 'http://www.backcarechiropractic.com', 'https://www.mypetmarket.com', 'http://www.visionworks.com', None, None, 'https://www.anytimefitness.com/gyms/288/Las-Vegas-NV-89139/?utm_source=yelp&utm_medium=local', 'http://www.happyhauscleaning.com', None, 'http://www.steaknshake.com/locations/23949-steak-n-shake-cherry-road-rock-hill', 'http://www.irepex.com', 'http://glutenfreegoat.com', 'http://www.rossstores.com', 'http://www.dogdayslasvegas.com', None, 'https://www.allvalleycarcare.com', 'http://www.thepigeonguy.com', None, 'http://www.etchsalon.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.sugarfactory.com/', 'http://clickserve.dartsearch.net/link/click?lid=43700044281357789&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'https://locations.jackinthebox.com/us/az/phoenix/620-e-buckeye-rd', None, 'http://quicksmiles.com', 'http://www.southbrookpumpkins.com', 'http://www.starbucks.com/store/9298/', None, 'http://www.fender.com', 'http://www.arizonaveterinaryoncology.com', None, 'http://chollafamilydentistry.com', 'http://www.aliveprintshop.com', 'http://www.skipro.com', 'https://www.cyclegear.com', 'http://sunriseautosaleslv.com', '1592232418', None, 'http://www.warmspringssurgicalcenter.com', None, 'http://www.safewaytire.net/locations/mode/22/4623-superior-avenue-cleveland-oh-44103.aspx', 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=8248', None, 'http://radioshack.com', None, None, None, 'https://ad.doubleclick.net/ddm/clk/300150600;127077036;j', None, 'http://www.kinospainting.com/', 'http://www.themercurylounge.com', 'http://www.westcanadian.com', None, '1592232457', None, 'http://www.tequilaranchcleveland.com', 'https://www.peaceloveandlittledonuts.com/locations/hudson-oh', 'http://www.azsportscenter.com', 'http://www.UnitedBeautySupply.com', None, 'http://www.phoenixpubliclibrary.org/Locations/Pages/Cholla.aspx', None, 'http://www.chilinoodlecorner.com/', None, 'https://3scocoffeebar.com', 'http://leafandbloom.com', 'https://www.fastmed.com/urgent-care-centers/phoenix-az-walk-in-clinic-maryvale-parkway/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', 'https://www.homedepot.com/l/Cave-Creek/AZ/Phoenix/85022/480?cm_mmc=seo-yelp&', 'https://www.wyndhamhotels.com/hotel/30821?tel=18882935571&iata=00093785&cid=fe:bh:20161215:yl:pp:hawus:30821', 'http://mcithedoctorsoffice.ca/patients/37-clinic-Mt.-Pleasant', None, None, 'http://www.sabatinospizza.com', '1592232509', 'https://chatters.ca', None, None, 'http://www.greenvalleyrange.com', 'http://www.gangnamasianbbqdining.com', 'http://www.gemini-arcade-palace.com/', 'https://www.mcalistersdeli.com/locations/nc/cornelius/19930-west-catawba-avenue-suite-150-1067', 'http://www.ProOutdoorServices.com', 'http://www.winners.ca', 'http://la-dulce-espera.business.site', 'https://ad.doubleclick.net/ddm/clk/434331165;236814625;c', 'https://cortatefurniture.com', 'http://a519chocolate.com', 'http://gregstruckandautorepairil.com', None, 'http://www.visionsourcewestsahara.com', None, None, 'http://mobileglowaz.com/', 'http://donutparlor.com', 'http://www.greenvalleyobgyn-nv.com', 'http://www.awhitepondparadise.com', 'https://www.circlek.com/store-locator/us/glendale/5848-w-olive-ave/2706657', None, 'http://latinoservices.net', 'https://www.xtendbarre.com/studio/charlotte', 'http://sharpedgebeer.com/index.php/locations/brasserie', 'http://www.cavopgh.com/', 'http://www.ymcaofpittsburgh.org/sampson/', 'http://www.planetfitness.com/gyms/las-vegas-flamingo-nv-611?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', 'http://www.lovesac.com/', 'https://www.samsclub.com/sams/pagedetails/content.jsp?pageName=aboutSams&couponId=YLPADS&PID=YLPADS', None, 'http://thewingfactory.ca', 'https://lmp360media.com', 'http://locations.in-n-out.com/234', 'http://www.Learn2PlayNow.com/', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.JimmyJohns.com', 'http://www.lakeforestcc.org', 'http://www.brittwiththebrush.com', 'http://www.gentlemenscutlakewylie.com/', 'http://www.premierfoliage.com', 'http://www.playacabana.ca/', None, None, 'http://areyoukenilworthy.com/', 'https://www.interstatebatteries.com/locations/all-battery-center-peoria-il', 'https://www.homedepot.com/l/Arrowhead-Ranch/AZ/Glendale/85308/475?cm_mmc=seo-yelp&', None, 'http://www.scottclarknissan.com/?utm_source=yelp&utm_medium=business_url&utm_campaign=local-citations', None, 'http://www.goodlifefitness.com/locations/ontario/northyork/don-millseglinton?source=yelp-websitelink-northyorkdon-gl', 'http://fightertownpaintballpark.com/', '1592232641', 'http://www.barbecuedoctor.com', 'http://www.zinburgeraz.com/locations/gilbert', 'http://www.ifrrealestate.com', 'http://www.botanyflowers.ca', 'http://www.homedepot.com/l/Curity/ON/Toronto/M4B-3L8/7012', 'http://dimurasmoving.com', 'http://www.academyofballroomdance.com', 'https://www.redroof.com/extendedstay/hometownestudios/property/AZ/Phoenix/HTS1059', 'https://mhausnailbar.com', 'http://www.cataldi.ca', 'http://www.thenorvalpancakefactory.com', 'http://www.greenmontvet.com', 'http://realalbertamoving.com', 'https://www.hopewayfoundation.org', None, 'http://www.midas.com/phoenix/store.aspx?shopnum=1241&dmanum=684001', 'http://www.oomomostore.com', 'http://yuphas.com', 'http://lekensington.ca', 'http://us.christianlouboutin.com/ca_en/', 'https://www.redlobster.com/locations/list/il/champaign/1901-n-prospect-ave', None, None, 'http://www.calgaryexpo.com', 'http://www.cheesecakeetc.biz', None, 'http://www.cocktailemporium.com', 'http://west50.ca/', 'http://www.donvalleyendoscopy.com', None, 'https://www.interdent.com/gentle-dental/locations/az/peoria/14155-n-83rd-ave-dentist-office?utm_source=organic&utm_medium=yext', 'http://www.chillinaz.com', 'https://www.oberweis.com/ice-cream-and-dairy-stores/Champaign', 'http://www.orthodoc.aaos.org', None, 'http://www.pregocalgary.com', 'https://www.canadiantire.ca/en/store-details/on/markham-on0.store.html', 'http://www.robstreeremoval.com', 'http://www.sodexousa.com/', 'http://www.tempestlukeshospital.com/', 'https://ikonick.com', '1592232742', 'http://www.yourvacationhome.ca', None, 'http://crockerparkwinefestival.com', 'http://svpsports.ca/stores/steeles', 'http://www.calgarycoop.com/stores/brentwood', 'http://www.papaceo.ca', 'http://www.bulkbarn.ca/en-ca/index.html', 'http://www.alphagraphics.com/centers/tempe-arizona-us004/index.html', 'http://www.floppypawspetsupplies.com', 'http://www.phxsecrets.com', 'http://www.apria.com', 'http://www.onceuponachildscottsdale.com/', 'https://www.belfor.com/en/us/belfor-usa-offices/nevada/las-vegas', None, 'https://www.riteaid.com/store-details?storeNumber=11467', 'http://www.adept.fit', 'http://cibocalgary.com', None, 'http://www.thoseguysmotorcycle.com/', 'https://chadloveservices.com', 'http://phoenixtenniscenter.org', 'http://lagerheads.us', 'http://pizza22ribs.com', 'http://www.funnymagicforkids.com', 'http://barksidebistro.com', '1592232800', 'https://summithomesnv.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B22965780.251857559;dc_trk_aid=448264643;dc_trk_cid=119964345;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', 'http://www.frock.ca', 'http://www.myspace.com/rockyseastside', 'http://www.smgvegas.com', 'http://www.toronto.ca/parks/prd/facilities/complex/467/index.htm', 'http://www.monro.com', 'http://www.wildwingrestaurants.com/franchisees/angus-glen/', 'https://www.goodeyes.com/locations/phoenix/?utm_source=yelp&utm_medium=local&utm_campaign=clinic-phoenix&utm_content=website', 'http://www.einsteinbros.com/', 'http://www.jiffylube.com', 'https://www.moonvalleynurseries.com/locations/northwest-las-vegas?utm_campaign=yelp&utm_source=yelp-nv&utm_medium=rancho', 'https://la-canasta.com', None, 'https://locations.panerabread.com/oh/sheffield-village/5382-abbe-rd.html', '1592232836', 'http://www.frizzhairexp.com', None, None, None, 'https://vespersalon.com', 'http://www.michaelhill.ca/on/demandware.store/Sites-MichaelHillCA-Site/en_CA/Stores-Details?StoreID=148', None, 'http://rudyspub.org', 'http://www.lavishmotoring.com', 'http://www.warehousegroup.ca/el-furniture-warehouse-queen-st', 'http://www.naillacquersalon.com/', 'https://www.benjaminfranklinplumbing.com/las-vegas/', 'https://stores.pepboys.com/nv/las-vegas/506-s-decatur-blvd.html', '1592232869', 'https://www.metropolish17.com', 'http://www.streetsofnewyork.com', None, 'http://www.thepostalroute.com', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://theritzoflasvegas.com/', 'http://www.greentreehealthandmassage.com', 'http://www.dmvnv.com/', None, None, None, 'http://www.hotelarts.ca', 'http://www.runnymedewalkinmedical.com/', None, None, None, 'http://www.vitalitytap.com', 'http://www.bk.com', None, 'http://sekaisushi.net', 'https://www.frysfood.com/storeHours?store=66000623', None, 'http://ptownbar.com/', 'http://www.limozarizona.com/', 'http://www.lakeshorewellness.ca', 'http://en.hestiatea.com', 'http://www.snapfitness.com/torontohighpark', None, 'http://www.yogapulsestudio.com', 'http://www.veraiconsalonspa.com', '1592232943', None, 'http://www.jaybuickgmc.com', None, 'https://www.menzieschryslerdealer.com', 'http://www.dickblick.com/stores/pennsylvania/pittsburgh/', 'http://www.handandstone.ca/locations/aurora', None, None, 'http://BluBurger.com', 'http://www.nurturedmama.com', 'http://www.cwtow.com', None, 'http://www.fitnesscafemenu.com/', None, 'http://www.vegasmartialarts.com', None, 'http://drtangeyecare.com', 'http://b2burgersbrews.com', 'http://www.copyrite.ca', None, None, 'http://wingardsalon.com/', 'https://hilalgrill.com', None, 'http://www.extremeenergypros.com', 'http://www.ciaobabycatering.com', 'https://baycity-movers.com', None, 'https://www.bannerhealth.com/locations/peoria/banner-urgent-care', None, 'https://www.associated-dental.com/practices/associated-dental-care-chandler', 'https://www.starbucks.ca/store-locator/store/1020633/177-holland-street-east-177-holland-street-east-bradford-on-l-3-z-1-x-8-ca', None, 'http://www.spencersonline.com', None, None, 'http://w.subway.com', 'http://myomedicalmd.com', None, 'http://www.karma.salon', None, None, 'http://powerautobrokers.com/', None, None, 'https://www.ihg.com/holidayinnexpress/hotels/us/en/las-vegas/lasqr/hoteldetail', 'http://robertostacoshop.com', None, 'http://www.allbrightsmile.com', None, 'http://www.publicenemytattoo.net', 'http://www.lucysseacove.com', None, None, 'https://www.nofrills.ca/store-locator/Mario+%26+Selina%27s+nofrills+%28Royal+York+Rd.%29/1371?storeId=1371', None, 'http://beersoftheburgh.com', None, 'http://dnister.ca', 'http://www.wilsonparkerhomes.com', 'http://www.priyashairsalon.com', 'http://www.patoisjamaicanrestaurant.com', 'http://www.thebeautyspotstore.com', 'https://opdaz.com', 'http://www.bearmountainsports.com', 'http://www.foresightchiropractic.com/', 'http://www.palominolv.com/', 'http://www.ralphlauren.com', 'https://wildfire.sclv.com/Barleys?utm_source=Yelp&utm_medium=banner&utm_campaign=yelp-organic', 'http://www.lawrysonline.com/lawrys-primerib/las-vegas', None, None, '1592233109', 'http://www.chnv.org/OurCare/HealthyLivingInstitute.aspx', 'http://www.dunkindonuts.com', 'http://602massage.com', 'http://www.zaffron.ca/', 'http://www.bodyvision.com/arrowhead-studio/', None, 'http://www.hhgregg.com', None, 'https://cityofspice.com', None, 'https://varmintgone.com/locations/varmint-gone-matthews-north-carolina/', 'http://eastlakecanoe.com/', 'http://www.auroraarizona.com', 'http://www.papamurphys.com/', 'http://www.marcstreeservice.com', 'http://www.majesticpalmsapartments.com/', 'http://www.peoplesjewellers.com', None, 'http://www.tempe.gov/city-hall/community-services/community-recreation-centers/diablo-stadium', 'http://stonecreekfurniture.com', 'http://www.centralavenuedental.com', 'https://www.planetfitness.com/gyms/kent-oh?utm_source=yelp&utm_medium=referral&utm_campaign=Yelp_evergreen', '1592233167', 'https://www.atozlandscapinglv.com', None, 'http://www.sundevilauto.com/locations/az/chandler/chandler-22', 'http://www.wickeremporium.ca/', None, 'https://paulmitchell.edu/pittsburgh', 'https://www.chapters.indigo.ca/en-ca/home/storelocator/storeDetails/939', None, '1592233187', 'http://www.rainbowfeatherco.com', None, 'https://www.coca-colacoliseum.com', 'http://www.masterworksautomotive.com/', 'https://www.gianteagle.com/Stores/Store/?storeId=5831', 'https://www.goldfishswimschool.com/wexford/', 'http://www.elcampesinospgh.com', 'http://www.noforksgivenclt.com', None, 'http://www.gokartslasvegas.com', None, None, 'http://www.jacksonlimousines.com', 'http://www.thelashlounge.com/salons/waverly/', None, 'http://www.ACabLV.com', 'https://www.jumagallery.com', 'http://www.suncityawning.com', 'http://www.carrvalleycheese.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'https://www.ashleyfurniturehomestore.com/furniture-stores/united-states/nv/henderson/henderson?cmpid=2016YELP', None, 'http://ratewisemortgage.com', None, None, 'http://www.krispykreme.com', 'https://www.oyorooms.com/76426?utm_campaign=US_PHX004&utm_medium=listing&utm_source=yelp', 'http://www.comda.com/en_ca/', 'https://www.veinscarolina.com', 'http://wedosystems.com', 'https://stores.bestbuy.com/az/avondale/10035-w-mcdowell-rd-1109.html/?loc=ns100&ref=NS', '1592233263', 'http://safransmarket.com', 'http://www.kitchenbathroomremodeling.net', 'http://www.atengarofalodental.com', 'http://diamondnailsofmahomet.com', 'https://www.orangetheory.com/en-us/locations/arizona/surprise/16846-w-bell-rd-suite-106/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.pizzuvio.com', 'http://www.ayrsleynailbar.com', 'https://www.t-mobile.com/store-locator/nv/las-vegas/boulevard-mall-in-line?cmpid=WTR_SE_YXT_U98MLWB9027148', None, 'http://www.sushimaster.ca', 'http://silverstatelocksmith.com', 'https://svdpmadison.org/shop/verona/', None, 'http://www.totalsportsmedicine.com', 'http://www.brazascharlotte.com', 'http://www.nutrition-rush.com/stores.html', 'http://brazilianblowoutsbykatie.com', 'http://www.stammhouserestaurant.com/', 'http://www.eyetecheyes.com', None, 'http://www.girly-girlz.com', None, 'http://www.gatesmillsplace.com', 'https://www.flawlessbyhanna.com', None, 'http://www.sdpetresort.com', 'https://locations.togos.com/ll/US/AZ/Scottsdale/7369-N_-Via-Paseo-Del-Sur*-Ste_-104', 'https://www.aireserv.com/red-mountain', 'http://www.ampm.com', 'http://www.custommedicalmassage.com/pre-and-perinatal.html', 'http://www.nanakcarwash.com', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'https://masteldrycleaning.com/?utm_medium=CTA&utm_source=VYRLYelp', 'http://www.verizonwireless.com/stores/arizona/mesa/val-vista-204086/?INTCMP=INT-LOS-NON-EN-MAKEAPPT-06212015-1LOS1-RE', 'http://www.gotransit.com', 'http://www.geckoshadesolutions.com', 'https://www.blissnailspasc.com', 'https://locations.timhortons.com/ca/on/oakville/494-dundas-st-w.html', None, 'https://locations.massageenvy.com/pennsylvania/bethel-park/5223-library-road.html', 'http://blueiguanapools.com', 'http://www.heirloomsalon.net', None, 'https://www.absolutedental.com/locations/las-vegas/1860-e-charleston-blvd/', 'http://www.kingswayvanlines.com', 'http://www.todaysmenu.ca', 'http://lasvegasnora.com/home.html', None, 'http://www.maac.com/florida/orlando/retreat-at-lake-nona?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://www.7-eleven.com/?yelp=19726', None, 'https://www.lifestorage.com/storage-units/nevada/las-vegas/89015/601-in-henderson/?utm_source=yelp&utm_medium=referral&utm_campaign=YELP-NV-601&utm_content=website-link&howheard=IYE', None, 'https://www.calgarycoop.com/stores/shawnessy/', 'http://www.lawroofing.net', 'http://www.palaisroyale.ca/', 'http://www.blackpeppergrill.com', '1592233399', 'http://www.copps.com', 'http://www.greeningenuity.ca', None, 'https://modellacewigsandhair.com', 'http://www.lebowitzdental.com/', 'http://JamesonAndStaggs.com', 'http://www.atlas-autobodyinc.com/', 'http://www.eyebrowsnatural.com/', 'https://www.miragegm.com', 'https://www.hilton.com/en/hotels/clttohx-hampton-suites-charlotte-ballantyne/?SEO_id=YELP-HP-CLTTOHX', None, 'http://www.designingwomanohio.com', None, 'http://www.southwestjudo.com', 'http://www.healingedgeaz.com', None, None, 'http://stores.fossil.com/az/scottsdale/scottsdale-fashion-square-8304', 'http://www.mrfallstree.com', None, None, None, None, None, None, 'https://www.hilton.com/en/hotels/pitsrhx-hampton-suites-pittsburgh-airport-south-settlers-ridge/?SEO_id=YELP-HP-PITSRHX', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.sweetlanasbridal.com', 'http://www.nakedpizza.biz', None, None, 'http://www.generaltsoskitchen.com/', 'http://www.alteredimage.net', 'http://www.azclar.com', None, 'http://www.songlowboutique.com', '1592233480', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'https://www.excellenceindentistry.com', 'http://www.roofitright.com', 'http://www.3riverslocksmith.com/', 'https://locations.dollartree.com/il/champaign/546/', None, None, None, 'http://www.belllexusnorthscottsdale.com', 'http://www.osbornplace.com', None, 'http://tavernamericana.com/', 'http://www.paybackpest.com', 'http://www.donscigars.com/', 'http://parkinn.com/hotel-torontoca', 'https://www.carolinaasthma.com/location/eastover/', None, 'http://www.lickshomeburgers.com/', None, 'http://www.atsol.com', None, None, None, 'http://oxurban.com', 'http://salonliftohio.com/', 'https://www.cvs.com/minuteclinic/clinic-locator/nc/belmont/2233.html', None, 'http://www.dairyqueen.com', '1592233542', 'http://www.thecarolinadental.com', 'http://www.dentistryofarizona.com', 'http://goodearthcoffeehouse.com', 'http://www.newmarketinn.com/', 'http://www.gilbertperfectskin.com', 'http://www.dermimatch.com', 'https://joeyrestaurants.com/location/joey-crowfoot', 'http://www.health.gov.on.ca/en/public/programs/telehealth/', 'http://www.proarmament.com/index.html', '1592233565', 'http://www.bodricksbbq.com', None, 'http://www.yalebrothers.com', 'http://www.supremeshutters.ca', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://excel-pro-services.com', 'http://www.minpermanentmakeup.com', 'http://www.brainandspineaz.com', None, 'http://www.harmonyvet.com/', 'https://m.costco.com/warehouse-locations/summerlin-las-vegas-nv-685.html', None, 'http://SolisWaverly.com', None, 'http://www.ziggystavernpgh.com', 'http://mustangairmechanical.com', 'https://zestyclean.com', 'https://mvpmodernbarbers.com/bridgeland/', 'http://www.arizonahomeworks.com', None, 'http://www.perfectionistdetailing.com', None, 'http://www.mkt.com/desertrootskitchen', None, 'http://www.rotorooter.com', None, None, None, 'http://www.nafloans.com/', None, 'http://www.ztrip.com', None, 'http://www.umc.org', '1592233641', 'https://www.getgocafe.com/stores/pa/pittsburgh/baum-blvd-getgo/3064', '1592233647', None, '1592233652', 'http://onlyfaces.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=14314', 'http://www.erx247.com', 'http://www.marcos.com/', 'http://www.azextensionsbyjess.com', None, None, 'https://locations.dennys.com/OH/PARMA/247530', None, 'http://www.thaionyonge.com', None, 'http://www.pawstwoclaws.com', '1592233683', 'http://www.reliable.vegas', 'http://www.a1thevacuumshop.com/', 'http://www.zumiez.com', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://www.francisandsonscarwash.com', 'http://arizonapinatas.com', None, 'http://saachispa.com', None, 'http://www.papaprimoz.com', 'http://www.thevoodoolounge.ca/', None, None, 'http://www.themesshall.ca', 'http://goldstarsvc.com', None, 'http://www.neatfreakz@vpweb.com', 'http://www.cityofmadison.com/parks/find-a-park/park.cfm?id=1139', 'http://www.eggspectation.com', 'http://www.yamasushihouse.com', 'https://mylocalmcds.com/willow-pass-concord', 'http://www.tlcpedsaz.com/', None, 'http://www.lvthaimassage.com', 'http://www.frenchconnection.com', 'https://www.giascanvas.com', 'https://affordablebathroomsofaz.com', 'https://www.regmovies.com/theatres/regal-franklin-square/1845?utm_medium=social+network&utm_source=yelp+listing&utm_campaign=Regal+Franklin+Square', None, 'https://www.optumcare.com/state/specialty-care/specialist-locations/rose-parkway-oncology.html', 'http://levetto.com/', None, 'http://www.richlane.ca', 'http://www.pureandsimple.ca', 'http://www.moneyinminutesnv.com', 'http://www.MyMightyMoversAz.com', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://nathansfamous.com', 'http://www.wvimaging.com/', 'http://www.motimahal.ca', 'http://4lifenaturalfoods.com', 'http://www.uptowncarservice.com', 'http://www.classicmobileairbrushtanning.com', None, 'https://www.walmart.com/store/1464/charlotte-nc/whats-new', 'http://www.medinaoh.org/Parks/Reagan_Park', 'http://westendfood.coop/farmers-market', 'http://www.larichs-pizza.com', None, 'http://aaapws.com', 'http://www.3Brewers.ca', None, 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', None, 'http://blushcleveland.com', None, '1592233815', None, None, None, 'http://northcarolinafertility.com', 'http://www.lasvegascustomloans.com', 'http://www.tacomamaonline.com', None, 'http://www.jacksonswash.com', 'http://perfectpastryonline.com', 'http://www.clevelandexpress.net/index.html', 'http://www.azturfmasters.com', 'http://sidewalkcafeeuclid.com', 'http://mpmvegas.com', 'http://www.streetsofnewyork.com', 'https://www.thejoint.com/arizona/surprise/surprise-48024', 'http://www.playderecord.com', 'http://www.tranquil-bodyworks.massagetherapy.com', 'http://elsombrero.ca/', None, None, 'http://vipbestlasvegaselectrician.com', None, 'http://www.creativedds.com', None, 'http://www.apexendo.com', 'http://senortequilacantinagrill.com/', 'http://abloomsalon.com', 'http://marketcitycaffelasvegas.mypub.website', 'http://www.starbucks.com', 'http://www.quiznos.com/', 'http://www.mesaaz.gov/parksrec/Parks/MountainView.aspx', 'http://www.victoriasfamily.com', None, None, 'https://northtryonlaundromat.com', None, 'http://www.theastrologystore.com/', None, 'http://www.elomsbraiding.com', 'http://www.downtowntorontochiropractor.com', None, None, 'http://www.matthewsfarmersmarket.com/Index.html', 'https://www.wyndhamhotels.com/hotel/52963?tel=18666154223&iata=00093785&cid=fe:lq:20190403:yl:pp:LQUS:52963', 'http://www.secondlookbooks.org', 'http://www.firstqualityroof.com/', None, 'http://www.visionquestbooks.com', None, None, None, None, 'http://www.sallybeauty.com', 'https://www.bowlero.com/location/bowlero-christown?utm_source=yelp&utm_medium=organic&utm_campaign=webclicks', 'http://www.babysbadassburgers.com', 'http://www.phoenixelectricianstoday.com', 'http://www.saulterstreetbrewery.com', None, None, None, 'http://www.labcorp.com', 'https://www.giovannisonline.com', None, None, 'http://www.marbleslab.ca', 'https://www.lush.ca/on/demandware.store/Sites-LushCA-Site/en_CA/Stores-Info?StoreID=129', 'http://www.eatatovereasy.com', 'http://chirotrendz.com/', 'http://www.greenvalleyranchresort.com/dining/china-spice.php?utm_source=YELP&utm_medium=SOCIAL&utm_term=GVR&utm_content=GVR-PROPERTIES&utm_campaign=GVR-YELP', 'http://www.directv.com', 'https://www.gymboreeclasses.com/en/locations/NC/Charlotte/?utm_campaign=Charlotte&utm_medium=website&utm_source=Yelp&utm_content=yelp+page', 'http://brosandbettys.com/', None, 'http://www.lemonfreshcleaning.com', 'http://jetaction.com/', None, None, 'http://www.getitfixed.me', 'http://primozzpizzeria.com', 'http://www.cascadenc.com', 'https://www.fantasticsams.com/about/regions/phoenix-az/fantastic-sams-mesa-az-4', 'http://www.isaute.ca', 'http://www.piesquared.com', 'http://www.thewhitenailsspa.com', 'http://www.bestmassagecenter.com', None, '1592234018', None, 'https://www.brakemasters.com/new-site/stores/1901-n-scottsdale-rd-scottsdale-az/', 'http://happyjoes.com', None, 'http://www.ArrowheadScoopers.com', 'http://www.neatmethod.com', None, 'https://www.usps.com/', 'http://hancockfabrics.com/', 'http://www.albertayogacollege.com', 'http://www.sterling2cleaners.com', 'http://www.royalyorkmeatmarket.com', None, None, None, 'http://signaturenorthwest.com/agent/andrew-lunsford/', 'http://www.madcatpets.com', 'http://www.chasin-tails.ca', 'http://www.roxannesdriedflowers.com', 'https://www.amco-autoglass.com', None, 'http://handymanserviceslasvegasnevada.com', 'http://www.arrowheadnails.com', 'http://www.mcsorleysrestolounge.com/', 'http://www.cityofhenderson.com/parks/parks/sonata_park.php', 'http://www.heartlandpethospital.com', 'http://www.cruisin7th.com', '1592234082', 'http://www.abbotthearingaidcenter.com', 'http://www.townetavernrestaurants.com', None, 'http://www.subvetclinic.net', 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', 'http://wendyandersonlaw.com', 'http://www.globalpetfoods.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://nailskathylinhdesign.com/', None, 'http://www.meadowslv.com', 'http://www.ccrfurniturecleaning.com', None, 'https://www.bostonmarket.com/location/university-heights-oh-483/', 'http://www.senecacollege.ca', 'https://www.walmart.ca/en/stores-near-me/markville-mall-supercentre-3053', '1592234121', 'https://www.munichmotorco.com/service-your-car', None, 'http://www.guardianautoglassphoenix.com', 'http://www.thetacobros.com', 'https://www.popeyes.com', 'http://www.impactps.com/', 'http://www.phoenixmentalist.com', 'https://cagiautoglass.com', None, None, 'https://periodontics-dentalimplants.com', 'http://www.nowandthenretro.com', None, None, None, '1592234160', 'http://www.lesportsac.com', 'https://lvsmilecenter.com', None, 'http://www.mysewingroom.ca', 'https://ad.doubleclick.net/ddm/clk/458685338;262921492;w', None, None, 'http://www.shawnessysouthvet.com/', 'http://www.jandbappliance.com', None, None, 'http://www.piscessushi.com', None, 'http://www.techniquerestaurant.com/', 'http://www.scubamagic.biz', None, None, None, None, None, 'https://www.blackmetalfirearms.com', 'https://www.walgreens.com/locator/walgreens-4965+w+bell+rd-glendale-az-85308/id=3049', 'https://locator.chase.com/nv/henderson/637-n-stephanie-st?jp_cmp=rb/LocalListings/aff/branch/na', 'http://www.laalv.com', None, 'http://www.lasvegasswimacademy.com', 'https://www.target.com/sl/waterfront/1253', 'https://www.cloudcarpetonelasvegas.com/about-us/las-vegas', 'http://phoenixcenterforthearts.org/', 'http://www.phoenixknifehouse.com', None, 'http://www.calranch.com', 'http://honeydont.work', None, 'http://www.orobrewing.com', None, 'http://seefuhair.com', 'https://www.petsmart.com/stores/us/az/scottsdale-store0130.html?utm_source=yelp&utm_campaign=us-lcl-srv-eg&utm_medium=listing', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.youngshawaiianstylekitchen.com', 'http://www.brickyardgrounds.ca/', 'http://www.donrandleslawoffice.com/', 'http://johnnyrockets.com', 'https://www.planchatoronto.ca', 'http://www.pawgo.co', None, 'https://www.hertz.com/rentacar/location/unitedstates/northcarolina/charlotte/CLTT12', 'http://www.mtnparkchiropractic.com/', 'http://aquatechplumbingaz.com', None, None, 'http://www.larrygjones.com', 'http://www.myguardianselfstorage.com', '1592234284', None, 'http://www.orleanscasino.com/entertain/bowling?disid=wmm_yelp_to_corporate_bowlingcenter_organiclink', 'http://www.starbucks.com/store/15154/', 'http://www.thefrenchbee.com', None, 'https://finercustomjewelry.com', 'http://colournailssalon.com/', 'http://yummymarket.com/', None, None, 'http://robertostacoshop.com', None, 'http://azpermanentcosmetics.com/', 'http://www.midas.com/toronto/store.aspx?shopnum=9259&dmanum=857001', 'http://www.ocotilloeyecare.com/', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.amerisent.com', 'http://backwards.club', 'https://antikka.ca', 'http://www.turnstyleconsign.com/ahwatukee-phoenix.php', None, None, 'http://www.pjsdayspa.com', '1592234337', 'http://thebrowbomb.com', 'https://www.dunnedwards.com/directory/store-list/summerlin-078', 'http://drscottminer.com', 'https://www.peoriaford.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', None, 'https://www.crashofrhinospainting.com', 'https://www.kohls.com/stores/nv/lasvegas-668.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'http://www.thebakeryphx.com/', 'https://www.usps.com/', None, None, 'http://www.burlingtoncoatfactory.com', 'http://dalesbodyworksllc.com/', 'http://www.petworksco.com', None, 'http://yumyumsofcle.com', 'http://www.thedogdasher.com', 'http://www.hartshornstudios.com/', 'http://europeanpaverssouthwest.com/', None, 'http://www.maleesonmain.com/', 'http://www.equityresidential.com/', 'https://www.publicstorage.com/arizona/self-storage-gilbert-az/85233-self-storage/850?pid=wyelp&CID=1245&CHID=AFL', 'https://www.flavorsoflouisianacajun.com', 'http://www.madbatterbakers.com/', None, 'http://tempefarmersmarket.com/', 'http://www.8bitentertainment.com', 'http://machinodonuts.com', '1592234413', None, 'http://www.westmarine.com', 'https://www.nasm.org', None, None, None, 'https://locations.jackinthebox.com/us/nc/charlotte/7725-pineville-matthews-rd', None, 'https://locations.dennys.com/PA/NEW-KENSINGTON/249565', None, None, None, 'https://www.serviceheroesaz.com', 'http://zabblv.com', None, '1592234448', 'http://www.quiznos.com/', 'http://www.workhorsemarine.com', None, 'http://www.readytorolltire.com', 'https://www.beefjerkyoutlet.com', '1592234462', None, None, '1592234467', None, 'http://christophersfinedrycleaning.ca', None, None, 'http://www.donatos.com', 'http://menu6.org', 'https://www.charlottemeetings.com/charlotte-convention-center', 'http://www.lubestop.com/Locations/Wadsworth', 'http://www.papamurphys.com/', 'http://www.adidas.ca', 'http://www.arizonastoragecenters.com', 'http://www.excelauto.biz', 'http://www.edingersurgicaloptions.com', 'https://www.949atthedistrict.com', None, 'http://www.lvtattoo.com', 'http://www.hobbylobby.com/StoreLocator/Search?Latitude=43.1438143&Longitude=-89.35114950000002&SearchOn=+53704&SearchDistance=25', 'http://www.tacorrito.ca/', 'http://www.pintville.com', '1592234514', None, 'http://www.functionalaestheticdentistry.com', 'http://www.bsjplumbing.com', 'http://www.unitedmessengers.com', 'http://www.kelilabee.com', 'http://www.lovecloudvegas.com', 'http://www.vaco.com', 'http://www.aptfind.com', 'http://www.piesunlimited.com', None, 'http://www.vitalant.org', 'https://locations.usbank.com/index/arizona/tempe/east-elliot-rd-safeway-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', '1592234547', None, 'https://agents.allstate.com/the-bell-agency-henderson-nv.html', None, 'http://www.heroburgers.com', 'http://www.kidsrkidslasvegas.com', 'http://artistichair.studio', None, None, 'https://www.qualtire.com', None, None, '1592234573', 'http://www.animalsupplyhouserockhill.com/', 'http://www.muchoburrito.com', None, 'http://suitenectarsalons.com', 'https://www.filibertos.com', None, 'http://www.truwest.org', 'https://www.dipnailboutiqueocotillo.com', 'http://www.paradisedentalgroup.com', None, 'http://dreamydrawdental.com', 'http://www.mcmurray-manalodental.com', 'http://www.thompsonsautorepair.com', '1592234607', None, None, 'http://www.homewatchcaregivers.com/phoenix', 'http://www.tailgateclothing.com', 'http://www.buybuybaby.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.uwhealth.org/locations/detail.jsp?locationid=124', None, 'https://davidguyettphotography.smugmug.com', 'http://www.midge.ca', None, 'http://mkhouseconsulting.com', 'http://www.torontosculpturegarden.com', None, 'http://EventRents.net', None, 'http://usedcars12.com', 'http://www.azartrestoration.com', 'https://www.circlek.com/store-locator/us/peoria/9476-w-olive-ave/2706468', None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, 'http://waxingaroundtown.com', 'http://www.xtremehomeservicesaz.com', 'http://lasmejorespupusas.weebly.com/', 'http://www.redendos.com', 'http://www.CrowdControlLV.com', 'https://www.kaltire.com/en/stores/locations/?source=yelp_track&StoreID=867', 'http://rickjohn.com', 'http://www.pplusic.com', 'http://www.jabzboxing.com/glendale', None, 'http://www.madisonautodetail.com', 'http://www.fourquet-fourchette.com', 'http://www.dazzlingrestaurant.com', 'http://montereyvetclinic.ca', 'http://www.mrbsfamoussale.com', 'http://www.carserviceoflasvegas.com', 'http://www.suntreehealingarts.com', None, 'http://cirqnightclub.com', 'http://www.ampm.com', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, 'http://www.zabas.com/', 'http://summerwindsnursery.com/az/home?utm_source=yelp&utm_medium=text_link&utm_campaign=website', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=%2Fhotels%2Ftravel%2FYYCWA', 'http://www.santanfamilymedicine.com', 'https://www.circlek.com/store-locator/us/gilbert/888-s-higley-rd/2703373', None, 'https://canyoncreekrestaurant.ca/square-one/', 'http://www.artificialgrassinstallers.com', 'http://www.customcyclerescue.com', 'https://www.usstoragecenters.com/self-storage/nevada/las-vegas/us-storage-centers/135', 'http://www.inland-group.com/location/phoenix/', 'http://www.phxprep.com', 'http://www.DLCphoenix.com', 'http://www.babiesrus.com', 'http://www.ashtangayogalv.com', 'http://upmccancercenters.com/network/upmchillman.cfm', 'https://www.swimco.com/?utm_campaign=Market&utm_source=YELP', None, 'http://www.the-alley.ca', 'https://stores.advanceautoparts.com/pa/murrysville/4010-william-penn-highway?utm_content=listing-2016-12-29&utm_medium=local&utm_campaign=aap&utm_source=yext&utm_term=yelp', None, 'http://www.donalfonsotoronto.com', 'https://locations.jackinthebox.com/us/ca/glendale/1200-w-glenoaks-blvd', 'https://bikeworldlv.com', None, 'http://www.mathnasium.com/litchfieldpark', 'https://haircutmenbocaparklasvegasnv.com', 'http://www.partycity.com', 'http://www.donuttyme.com', '1592234787', '1592234790', None, None, None, '1592234800', None, 'http://www.scottsdaleaz.gov/parks/mountain-view-park', None, 'https://www.gowireless.com/stores/az/fountain-hills/fountain-hills-fountain-hills-plaza/', 'https://www.acesdental.com/areas-served/dentist-north-las-vegas-nv/?utm_campaign=yelpnlasvegas&utm_medium=organic&utm_source=local', 'http://www.rivierahotel.com/las-vegas-entertainment/crazy-girls.aspx', None, 'https://clevelandhomes.com', 'http://www.clipncleandoggrooming.org', None, 'http://www.centurylink.com', 'https://www.chick-fil-a.com/indianland', 'http://www.stabley.net', 'http://www.ledolci.com/', 'http://www.kemptonpt.com', 'http://www.bk.com', 'http://www.schneidersdairy.com', 'http://desertfoothillsgardens.com', 'https://aguayobunch.wixsite.com/a1electric', 'https://www.gianteagle.com/Stores/Store/?storeId=3043', 'http://www.chandlerpediatricdental.com', 'http://www.thereikigal.com', None, 'http://www.slepakinsurance.com?cmpid=RZYD_BLM_0006', None, None, 'http://www.colussy.com/?cs:pro=repman&cs:e=yelp', '1592234868', None, 'http://www.racersedgeperformance.com', 'http://www.waterconnectionaz.com', 'http://www.goindustrial.ca', 'http://www.bohofurnituregallery.com', 'http://www.dannysfamily.com/', 'http://www.sammysstudentexchange.com', 'http://bellagio.com/nightlife-diversions/hyde.aspx', 'http://thetoytank.com', 'http://www.ironcityarmory.com', 'http://journeyyogawellness.com', 'http://www.gatewayanimalohio.com/?utm_source=yelp&utm_medium=cpc&utm_campaign=jan16', 'http://www.rocksolidconcrete.com', None, 'http://www.ajimenu.ca', 'http://www.precisionpoolcarellc.com', None, '1592234912', 'https://app.visionnv.com', 'https://www.calgarycoop.com/stores/shawnessy/', None, 'http://www.amcofh.org', 'http://www.ironoakshoa.com', 'http://www.z-teca.com', 'http://kendalaquatics.com', 'http://www.hunnerspizza.com', 'http://www.thedistrictatgvr.com/', 'http://www.apple.com/retail/glendale/', 'http://www.marquisdiagnosticimaging.com', None, 'http://www.thewirelesscenter.com', 'https://www.extraspace.com/storage/facilities/us/arizona/phoenix/501788/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'https://athleta.gap.com/stores/wi/madison/athleta-7080.html', 'https://www.usps.com/', 'http://katespade.com', None, 'http://www.nicksauto-tech.aaro.ca', 'http://www.hairmonet.com', 'http://www.greenwaymoving.ca', None, None, 'http://www.winemandental.com', None, 'http://www.mfmassage.com', None, 'https://bluepearlvet.com/hospital/avondale-az/', 'http://www.antheaspa.com', 'http://www.thechateauaz.com', 'http://www.coronetpizzeria.com', None, 'http://www.sabuddyrestaurant.com', 'http://www.avis.com', 'http://www.hendrickmotorsports.com', None, 'http://www.legacyjewelersofpittsburgh.com', 'https://www.dabuzispizza.com', None, 'http://alysskincare.com', 'http://vikingfishandchips.net', 'https://www.applebees.com/en/restaurants-champaign-il/2121-n-prospect-93091', 'http://www.thenailtique.com', 'http://www.shopsys.ca/', '1592235014', None, 'https://www.extraholidays.com/las-vegas-nevada/worldmark-las-vegas-boulevard?lid=yelp&utm_medium=referral-affiliate&utm_source=yelp.com&Identifier=yelp&hotelid=77219&utm_campaign=localpage-worldmarklasvegasboulevard&promoid=localpage-worldmarklasvegasboulevard', None, None, 'http://www.stmlv.org', None, 'http://www.purrclothing.ca', 'http://call4handymanlv.com', 'http://305kustomzdesigns.com', None, 'http://www.theoriginaldicksspeedotach.com/', 'http://visitpittsburgh.com', 'https://batlgrounds.com/axe-throwing-yorkdale', 'http://www.MyLasVegasChiropractor.com', 'http://www.planetdailies.com', 'http://liquorlibrarylv.com', None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, None, 'http://tofoodfest.com/', 'http://snipssalonclt.com/', '1592235069', 'https://www.spencersonline.com/store/Ross-Park/32682040.uts', 'http://thestudioacademyofbeauty.com/locations/chandler-campus/', 'http://www.tinytots.ca', 'http://www.supremecollisioncentre.ca', 'http://www.mobilia.ca', 'http://www.westernexterminator.com/chandler', 'http://www.yorkmillspa.com', 'http://www.stampsautoqueencreekqueencreekaz.com', None, 'http://www.vitalitymwi.com', None, 'https://www.thespacelv.com', 'http://www.rosenbaumrealtygroup.com/', 'http://coolpools.services', None, 'https://www.wellsfargo.com/locator/bank/?slindex=1&il=EN', 'http://longwongsofnorthphx.com', 'http://www.winedowntc.com', 'http://www.mitcoremodeling.com', None, 'http://www.holdenarb.org/home/', 'http://kmedspa.com', None, 'http://www.quickerrooterplumbing.ca/', 'http://pittsburghrugcompany.com', 'https://suncountry.com', 'http://www.fujimiddleton.com', 'http://www.thebeergrowler.net/charlotte', None, 'https://sandpipers.ca', None, 'http://www.starbucks.com', 'http://www.picazzos.com', 'https://www.easystreetgalleria.com', None, None, '1592235155', 'http://www.goodlifefitness.com/locations/ontario/mississauga/toronto-pearson?source=yelp-websitelink-torontopearson-gl', 'https://www.pteglv.com/locations/pts-gold/st-rose-eastern/', None, 'http://www.weldrickanimalhospital.com/', 'https://www.cosmopolitanlasvegas.com/resort/spa?utm_source=yelp&utm_medium=organic&utm_campaign=yelp', 'http://www.idotactical.com', 'http://www.progressive.com/locations/mayfield-heights-campus3/', 'https://elitevacuumsplus.com', 'http://www.vha.ca/', 'http://mwtravelvegas.com/', 'http://www.ezpawn.com/locations/nv/las-vegas/6032-cheyenne', 'https://www.eggsupgrill.com/locations/charlotte-nc/', 'http://www.popculture-comics.com', None, 'http://www.wimpysdiner.ca', 'http://www.shearshades.com', 'https://www.oroexpresschandler.com', 'http://www.forcesofnature.ca/', 'http://tommysdrivein.com', 'http://www.saintbernadette.com', None, 'http://steelcreationsfab.com', 'http://www.VerveStudio.net', 'http://www.performancemufflershop.com', 'https://po-ke.ca', None, 'https://www.usps.com/', 'http://grandcourtmesa.com', 'https://azfoodcrafters.com', 'http://www.ohiocitycycles.org', 'https://www.biltmorecounseling.com', 'http://www.budgetmoversaz.com', 'http://bluewavepoolsaz.com', 'https://www.haggar.com/stores/las-vegas-nv/?utm_campaign=websitelink&utm_medium=Referral&utm_source=yelp', None, 'https://www.coultertempe.com', 'http://www.swimzone.info', 'http://walmart-eyedocs-lasvegas.com/Eastern-Russell', None, 'https://www.calgarypromovers.com', 'http://www.gyrotime.com', 'http://www.dinaalonzibridal.com', 'http://www.massagelifewellness.com', None, 'http://www.shellhospitality.com', None, 'http://www.lakemetroparks.com/parks-trails/farmpark', 'http://www.charlottegeneraldentist.com/', 'http://tomyumthaiaz.com', '1592235283', '1592235286', 'https://www.30sixnoda.com', None, 'https://www.laserquest.com/on-whitby', 'https://yogatothepeople.com/tempe-az/', None, 'http://hairbygk.com/', 'http://www.crookedcue.ca/', None, 'http://www.mission4th.ca', 'http://www.lisatrainer.net', 'http://www.greektownrestaurant.com', None, 'http://arizonasports.com', 'http://www.itmp.com', None, 'http://www.scarboroughnissan.com', None, 'https://www.walgreens.com/locator/walgreens-401+w+main+st-waunakee-wi-53597/id=5669', 'http://www.andrewsco.com', 'https://www.sheridancollege.ca', 'http://www.bloombrossupply.com/', 'https://www.esso.ca/en/gas-station/richmondhill-on-beavercreekesso-200302403', 'http://www.eatatgreenmix.com', 'http://www.seasonsboulangerie.com', 'https://tanwithkare.com', '1592235353', None, 'http://bestnails.webs.com/', 'http://Brilliantnailsspa.com', None, 'http://www.lowes.com', '1592235367', 'http://www.lasvegascaralarms.com', 'http://scottsdaleshadowsarizona.com', None, 'http://www.jjcontractingnv.com', 'http://www.dualimagesmiles.com', 'https://agents.allstate.com/j-c-alvarado-charlotte-nc1.html', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://mnandipies.com', None, 'http://www.ArnoldSmithLaw.com', 'http://www.customcobblersofmadison.com', 'https://stores.anntaylor.com/nv/las-vegas/1013-south-rampart-blvd.html', 'http://www.suncityupholstery.com/', None, 'http://www.dropshade.com', 'http://popeyes.com/', 'http://gyro-spot.com', 'http://www.aircanada.ca', 'http://3apools.com/', 'http://www.associatedasset.com', '1592235418', 'http://www.terribleherbst.com/', 'http://www.studio262salon.com', None, 'https://ad.doubleclick.net/ddm/clk/434191077;236814625;i', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', None, 'http://www.mattressfirm.com', 'http://www.monkeypantsbar.com/', '1592235439', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://westvalleydentalaz.com/', 'https://abraauto.com/abra-auto-location/1051-charlotte-nc-albemarle-road', 'http://www.mrantenna.com', 'http://www.hlwp.co', None, 'http://www.sweetlifebakeshoppe.com/', 'http://www.brueggers.com/', 'http://footsolutions.com/strongsville', None, 'https://locations.dennys.ca/AB/CALGARY/248179', 'http://www.whitneymurphyfuneralhome.com', 'https://essencevegas.com/locations/henderson/', None, 'https://accutempaz.com', 'http://www.chefbar.ca/', 'http://www.mtlebanon.org/index.asp?NID=25', 'http://www.faregamecc.com', 'https://www.umcsn.com/Quick-Cares/Quick-Care-Locations-Summerlin.aspx?intMenuID=112&intPageID=197', 'http://www.safeworksillinois.com', 'https://www.hendrickhonda.com?utm_source=weblistings&utm_medium=organic&utm_campaign=hendricklocallistings', 'http://www.access1alarm.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://vegasnailspa.com', 'http://sarahdiaz.com', None, 'http://www.theblackbullpub.ca', 'http://www.upmc.com/locations/community/family-health-centers/Pages/lawrenceville.aspx', None, None, None, None, None, None, None, 'http://www.vaughnautoglass.com', 'http://www.experttechauto.com', 'http://am.pizza', 'http://dennys.com', 'https://www.smithsfoodanddrug.com/storeHours?store=70600371', 'http://www.themodclub.com/', None, 'https://www.enterprise.com/en/car-rental/locations/us/nv/las-vegas-n-rancho-dr-5427.html?mcid=iyp:8368400', 'http://www.valleyfirefightermovers.com', 'https://www.choicehotels.com/north-carolina/charlotte/sleep-inn-hotels/nc005/rates', None, None, 'http://vcacanada.com/royaloak', 'http://dbarkadz-filipino-cuisine.business.site', 'https://www.chilis.com', None, 'http://nakedbeautybar.com/', None, 'http://www.lakesidespa.ca', 'http://www.trinitybellwoodsflea.com/', 'http://www.drcarolford.com', '1592235578', 'http://sinfulskinaz.com/', 'http://www.elxrjuicelab.com', 'http://www.laide.ca', 'https://totalstoragesolutions.com/storage-locations/arizona/peoria-storage-units/7590-w-olive-ave', None, None, 'http://www.ebgames.com', 'http://www.atomicdogboutique.com', None, 'http://www.championappliancecare.com', 'http://www.starbucks.com/store/351/ca/sunridge-mall2/2525-36th-st-n-e-160-calgary-ab-t1y-5t4', 'http://www.grafgrowers.com', 'http://www.gameworks.com/', 'http://www.septic-systems-calgary.ca/', 'https://www.americanfreight.com/?utm_medium=Yelp_Website_URL&utm_source=Yelp', None, 'http://hmartca.com/hmart/?page_id=17', 'https://ad.doubleclick.net/ddm/clk/458685305;262921492;q', 'http://pastorcameron.com', 'http://www.leslievilledentistry.com/', None, 'http://joemommasgrille.com', 'https://www.cathaybank.com', 'https://www.verizon.com/home/storelocator/pa/pittsburgh-15206/', 'http://www.cartridgeworld.com', 'http://www.electricdaves.com', 'http://www.mistwoodapartments.com', 'http://daveandbusters.com/?utm_source=DB&utm_medium=Text&utm_term=DB&utm_content=WebsiteLink&utm_campaign=DB_DB_Text_WebsiteLink', 'http://www.callitspring.com', 'http://wreckamendedautobody.com', 'http://www.thisiscleveland.com', 'http://www.chevron.com', 'http://www.marijuanadeliveryline.com/shop', 'https://www.plantarestaurants.com/location/planta-queen/', 'http://www.kumon.com/SUMMERLIN', None, 'https://locations.wendys.com/canada/qc/lasalle/8699-boul.-newman', 'http://www.kabobnmore.com', None, None, 'http://www.kfc.ca/store/sunrige-mall-101-2525-36-st-ne-calgary', 'http://perpetualfitnesscle.com', 'http://sfdslv.org', None, 'https://whitesintegrityauto.com', None, 'http://www.tommiegirlphotography.com', 'https://www.sprint.com/storefronts/bd/sprint-phoenix-az-85022-fcs-1908/?ecid=SEO:Yelp:2020_Q1:LastMile', 'https://www.ceenta.com/locations/huntersville', 'http://flyingmonkeytruck.com', None, 'http://www.allysphotography.ca', 'http://www.nailganic.me', '1592235698', 'https://bailbondsusa.com', None, 'https://fullypromoted.com/locations/scottsdale-az/', '1592235706', None, 'https://www.localfirstaz.com/events/fall-festival', None, None, None, 'https://sugardaddyscafe.com', 'http://vitalitymds.com', 'http://www.muramoto.biz', 'http://www.7forallmankind.com', 'http://www.susysoup.com', 'http://treesbymikearizona.com', '1592235733', 'http://www.pilatesofcharlottenc.com', '1592235738', 'https://www.hungryhowies.com/store/hungry-howies-00876?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=00876&utm_content=Website_Link', 'http://www.riddleroom.ca/', 'http://www.torchedgoodness.com/', 'http://www.precisionspecialtypharmacy.com', 'https://www.lhmnissanmesa.com', 'http://bit.ly/RealtyExecsTempeTower', 'https://smallpeoplepreschool.com', None, 'https://www.bobevans.com/our-restaurants/locations/571?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.wetzels.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=8658&localechange=1', None, 'http://Bundtsnbutter.com', 'http://www.arizonajewelryappraisals.com', 'http://paisleymonkey.com', 'https://www.euroworks.ca', 'http://www.maltzmuseum.org/', 'http://www.akronphotographer.com', None, 'http://www.firegrill.com/', 'http://www.hairoriginalsalonstudio.com', 'http://azdollhouse.com/', 'http://www.gracelutheranhenderson.org', 'http://thepopplacestore.com', None, 'http://www.widmaierchiropractic.com', None, None, 'http://www.toronto.ca/parks/prd/facilities/complex/287/index.htm', 'http://tokyodiscountvegas.com', None, None, None, None, 'https://www.redrobin.com', 'http://www.pranjbeauty.com', None, None, 'https://www.enterprise.com/en/car-rental/locations/us/nc/charlotte-south-woodlawn-53c4.html?mcid=iyp:8368400', 'http://www.structube.com', 'http://popeyes.com', 'http://in.puma.com', 'https://www.discounttire.com/store/az/gilbert/s/1418?utm_source=yelp&utm_medium=social&utm_campaign=YelpBusLinkClick', '1592235832', 'http://totum.ca', None, None, 'http://www.bestofthebestdjs.com', None, 'http://www.gungarage.com', 'http://www.sunsetvillaaz.com', 'http://cornerstonegiftsandpo.com', None, None, None, 'http://calibercollision.com/locate-a-caliber-collision-center/surprise', 'http://www.desertstorage.com/scottsdale-az-storage-units', 'http://www.scottsdalecc.edu', 'http://www.cupkateslv.com', 'http://www.caesarspalace.com/restaurants/empress-court.html#.Ugfc5su9KSM', 'http://www.brossardbagel.com', 'http://www.mrchickencle.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://locations.avis.com/us/az/chandler/ca3.html', 'http://www.modoyogalasvegas.com', '1592235882', '1592235884', 'https://www.valleyhealthphysicians.com/find-a-doctor/hazem-afifi', 'http://Surprisecomics.com', 'http://www.smileycrane.com', None, 'http://allstateappliancelv.com/', 'http://johnnyrockets.com/', None, 'https://www.mesaparks.com/parks-facilities/parks/countryside-park', 'http://www.monarkhome.com/showrooms/las-vegas', 'http://www.steinmart.com', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/109+Dabney+Drive-Henderson-NC-27536/storeid=7055?cid=ps_ylp_2020', 'http://www.dillonazrealty.com', 'http://www.arizonaplumbingservices.com', '1592235919', 'http://www.lasvegaspilatesyoga.com', 'http://www.deeppilates.com', 'http://www.irepairicustomize.com', None, 'https://www.hilton.com/en/hotels/clttght-home2-suites-charlotte-i-77-south-nc/?SEO_id=YELP-HT-CLTTGHT', 'http://www.capitalacctpc.com/', None, 'http://www.jamminjumperoos.com/', '1592235936', 'http://www.frozen75.com', 'http://Tamerabbit.com', 'https://bounceu.com/gilbert-az', 'http://www.bullseyesvc.com', None, 'http://popeyes.com', 'http://www.fastfreshfoods.ca/', None, 'http://www.palmsouthwest.com', 'http://www.jeguitars.com', 'http://www.carrefourrichelieu.com/fr', None, 'http://Www.2womenandasolution.com', 'https://www.simonmed.com/locations/all/simonmed_imaging___maryland', 'http://www.RobinsonAutoDetails.com', 'https://stores.fossil.com/az/gilbert/santan-village-8252/', None, '1592235970', 'http://www.cleveland.hfitcenter.com/', 'http://www.bk.com', 'http://phoenixcoffee.com', None, None, None, 'https://www.lovencarevet.com', 'http://www.saintjohncathedral.com', 'https://www.ryokujudo.com', None, 'http://www.tacotimecanada.com/locations/ontario/ajax/ajax', '1592235997', None, 'http://qdoba.com', None, 'http://roomsonline.ca', 'http://www.prada.com/', 'http://www.cigna.com/cmgaz/locations/san-tan-medical-office', 'http://www.innergroovebrewing.com', 'http://www.drtorgerson.com', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://freshandclear.com', 'http://www.starbucks.com/store/9298/', None, 'http://www.kingbalancing.com', None, None, 'http://www.atlwings.com', 'http://www.cuttyfresh.com', 'https://cleancutlandscaping.us', 'http://www.luxuryhomesofhenderson.com', None, 'http://www.sagebrushranch.com/', 'http://snapfitness.com/waxhawnc', 'http://www.bergo.ca', 'http://www.smilesavvydentistry.com/', 'https://www.purebarre.com/location/dana-park-az', 'https://www.hooters.to/toronto-airport', 'http://www.zenq.ca', 'http://www.drshaunachristensen.com', None, None, 'http://www.kingflyspirits.com', 'http://www.theradiodoctor.com', 'https://www.tirediscountdepotinc.com/cleveland', 'https://www.bobevans.com/our-restaurants/locations/357?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.strattonrestoration.com', 'http://www.Liorthebaker.com', 'http://rmtmassagetherapytoronto.com', None, 'http://fiscalfitnessphx.com/', 'http://banderarestaurants.com/locations/scottsdale/', 'https://www.state48brewery.com', None, '1592236095', 'http://www.quiktrip.com', 'http://www.almikestavern.com', 'http://www.earnhardttoyota.com', 'http://www.clairhursteyecare.com', 'http://mybodyworkz.com', 'http://www.envoquemd.com/', 'http://rvcomplete.com', None, 'https://www.steinmart.com/?utm_source=yelp.com&utm_medium=social&utm_campaign=yelp_site_store48', 'http://www.familyfocusedeyecare.com', 'https://www.obonsushi.com/scottsdale-quarter', None, None, 'https://www.choicehotels.com/arizona/youngtown/quality-inn-hotels/az453/rates', 'http://pizzeriaenzo.com', 'http://www.fitzgeraldsbnb.com', 'http://www.steeltownpaintball.com', '1592236142', 'http://www.timelesstattoolv.com', 'https://www.starbucks.ca', None, 'http://www.landmarkworldwide.com/customerservice?utm_source=yelp_sea&utm_campaign=support&utm_medium=website&utm_content=https', 'http://www.southtowndentalcare.com/', 'https://shinynailsonhappyvalleyglendale.com', 'http://www.foothillsrehab.com/', 'http://www.kay.com', None, 'http://www.azcustomhats.com', None, 'http://www.gokartsatpolsonpier.com', 'https://www.jamesacarytreeservice.com', 'https://artscommons.ca/rentals/venues/max-bell-theatre/', 'http://www.medoneaz.com/', 'http://www.destinocalgary.com/', None, 'http://www.futureshop.ca/', 'http://www.thingsremembered.com', 'http://www.restoamir.com/', 'http://www.kay.com', 'http://www.sunguntanning.com', None, 'https://www.scottsdalemusicaltheater.com', 'http://www.terrysauto.extremewrench.com', None, 'http://www.tupelohoneyteas.com', 'http://www.cliftonmartini.com/', 'http://www.oddballpets.com', None, 'http://www.lcbo.com', 'http://www.azrollerderby.com', 'https://www.chilis.com', 'http://www.brooksbrothers.com/?CMP=SMC_Yelp_6038', None, 'http://thecornerspot.ca', 'https://www.wyndhamhotels.com/hotel/03733?tel=18005369326&iata=00093785&cid=fe:se:20161215:yl:pp:seus:03733', 'https://stores.bareminerals.com/az/chandler/us-3/', 'http://deluxenailsbar.com', 'http://www.watsonsglen.com', None, 'http://www.2andhighbar.com', 'http://www.alpineroofing.ca', 'http://www.klingshirnwine.com', 'https://www.theranchestates.com', None, None, None, 'http://www.crowntownathletics.com', None, None, None, 'https://www.westgateresorts.com/hotels/nevada/las-vegas/westgate-las-vegas-resort-casino/', 'http://www.marshallsonline.com/store-finder/204', 'https://www.bellamirageapts.com', 'http://www.dvpeds.com', None, 'http://sololiquor.com', None, 'http://www.shampoochgrooming.ca', 'https://keepitcut.com', None, 'http://www.benjerry.com/', None, 'http://www.weissmeats.com', 'http://www.thegreenhouseflorist.com', None, 'http://www.halecoffee.com', 'http://www.mycompletepoolcare.net', 'http://www.ranchodetiarosa.com/rancho-chandler.asp?expanddiv=all', 'http://www.mokshayogaaurora.com', 'http://mrlube.com', None, 'http://www.textbookpainting.com', 'http://www.hansenortho.com', 'https://www.highlandparknorthlake.com', 'https://www.ulta.com/stores/madison-ms-477', 'http://www.saltrivereyecare.com', None, 'http://www.knuckledownsaloon.com/', None, 'http://www.westsidemarket.org/vendor.aspx?id=69', None, None, 'http://deseretindustries.org', 'http://www.marcs.com/stores/marcs-north-ridgeville-grocery-store-44039.aspx', None, None, 'http://www.phoenix.edu', None, None, 'https://www.marcos.com/locations/nc/gastonia/w-garrison-blvd', 'http://www.steelcityflooring.com/', 'https://vrjunkies.com/vr-junkies-az/', 'http://sears.com/stores/massachusetts/cambridge/0001343.html', 'http://www.legionbrewing.com', None, None, 'http://www.dukesbread.com', None, 'http://www.jellybeenz.com', 'http://www.urbanbean.ca', None, 'http://www.labellavitacleveland.com', 'http://www.sash.ca', 'https://www.choicehotels.com/CN659?mc=blypypaf', 'http://www.aceparking.com/downtown-phoenix-parking.com', 'http://www.careytechsolutions.com', None, 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', None, None, 'http://armorpestdefense.com', 'http://www.heidisbrooklyndeli.com', 'http://ilanmassagestudio.com', 'http://bdcustomtattoos.com', 'http://www.ntb.com/store/6234/mayfield-heights-oh-44124-2905?utm_source=yelp&utm_medium=maps&utm_campaign=yelp-maps', None, None, '1592236410', None, '1592236415', None, 'http://www.hightails.ca', 'https://www.agelesskarate.com', 'http://www.mpautobodyrepair.ca', 'http://www.carlstreeservice.net', 'http://www.tailoredeventslv.com', None, '1592236432', 'http://www.solairespa.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.exosathome.com', 'http://dentalbobw.wix.com/denturerepairs', 'https://www.sleepez.com/?utm_campaign=Yelp&utm_source=yelp&utm_medium=referral', 'http://hamiltonortho.com', 'http://www.rbinc-sports.com', 'https://www.pikebeverage.com', 'http://ccandcompanysalonspa.com/', None, 'https://www.cleanjuice.com/locations/uptown/', 'http://srwc.unlv.edu/', 'http://www.theronceyhotel.com/home.html', None, None, 'http://www.702-pools.com', None, 'http://www.belmontapts.com', 'https://www.thehealingquest.com', 'http://robertostacoshop.com', 'http://www.charlottegolf.com', 'https://www.haircutmencorneliusnc.com', None, 'http://www.tymaz.com/', None, None, None, 'http://www.coquettepatisserie.com', None, None, 'http://www.alisfashion.com', None, 'http://www.yogurtys.com', 'https://torontoanimaleyeclinic.com', None, 'http://www.teamrossman.com', 'http://www.firehousemovingco.com', None, 'http://constanthomecomfort.com', None, 'http://www.wcequestrian.com', 'http://warehousewars.net', None, None, 'http://www.baselineflowers.com/', 'http://luccaclothing.com/', 'http://www.ilovekickboxing.com/kickboxing-studio/index.php?id=374?utm_source=review&utm_medium=yelp&utm_campaign=referral%C3%8A', 'http://www.azdmovers.com', None, None, 'http://www.harveycatshotel.com/', 'http://www.bk.com', 'https://www.allinclusiveconcretesystems.com', 'http://www.raintreerestaurantchagrinfalls.com/', 'http://nisstech.com', None, 'http://www.ciaociaopizza.com/', 'https://www.bestcharlottemassage.com', 'http://www.LisaYoungDDS.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=1962&localechange=1', 'https://fullpoolservices.com', 'http://www.theparlorlv.com', None, 'http://tonysautoworks.net', None, 'http://nolinesbydesign.com', 'http://local.buckle.com/AZ/GLENDALE/331/?utm_source=Buckle&utm_medium=Website&utm_campaign=Store+Locator', None, '1592236581', None, None, 'http://wyattorthodontics.com', 'http://www.staxburgerbistro.com', None, 'http://www.progressivepavers.com', 'https://www.storagewest.com/facilities/baseline/?utm_source=YelpCTA&utm_medium=referral&utm_campaign=baseline-business', 'http://www.crowdpleezersbarbershop.blogspot.in/', 'http://www.AltierDental.com', 'https://www.victoriassecret.com/store-locator#/store/442', 'http://www.wickedclub.com', None, '1592236611', 'http://www.containerstore.com/locations/showStores.htm?zipCode=28211&searchArea=100', None, None, 'https://lljcustoms.com', None, 'http://www.kohakitchen.com', 'https://locations.wendys.com/united-states/pa/bridgeville/1125-washington-pike', 'http://toba.ca/', None, None, None, 'http://www.renue.ca', 'http://www.mobilehomedepot.net', 'https://www.rubytuesday.com/', 'http://www.odiagasautosales.com', None, None, 'http://www.rivieraopticare.com/', 'http://www.showcasemotorspa.com/', None, 'http://www.azsleepyteepee.com', 'http://giantjones.com', None, 'http://www.marcos.com/locations/oh/berea/w-bagley-rd?s=true', 'http://www.pimadental.com', None, None, None, 'https://www.centerforsightlv.com', None, None, 'http://stores.bestbuy.com/122/', None, 'http://www.averyonthegreen.com', 'http://www.krispykrunchy.com/locations/wy-market', None, 'http://www.thecamptc.com/location-gilbert', 'http://www.saigonpearl.ca', 'http://www.posaresalons.com', 'http://www.sanrio.com', 'http://Islandstylepools.net', 'https://turningpointbeauty.com', None, 'http://www.reneesmassotherapy.com', 'http://johnnyrockets.com', None, 'http://funkytoque.com/wellington-accueil/', 'https://www.smithsfoodanddrug.com/storeHours?store=70600354', None, 'http://therabbitspot.com/the-rabbit-hole/', 'http://www.psdm-madison.com', 'http://www.tizzerts.com/', 'http://flahertytree.com', 'http://www.saddlebroncgrill.com', 'http://www.dwmeats.com/', 'http://www.malleys.com', 'https://www.sprint.com/storefronts/bd/sprint-goodyear-az-85338-d-8157/', 'http://expertcarcareandtransmissions.com', 'http://www.themakeden.com/', None, 'http://www.arbys.com', None, None, None, 'http://www.cupcakejunkiecafe.ca/', None, 'http://www.carolinascornerstone.org', None, 'http://joinswift.com/landing-pages/DR/student/yp/careerhorizon/lp', 'http://monogramcoffee.com', 'http://www.littlecaesars.com', None, 'https://www.fastmed.com/urgent-care-centers/scottsdale-az-walk-in-clinic-mcdowell-road/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', 'http://www.stagsfamilychiropractic.com', None, 'http://www.luxhairsalon.com', 'http://www.fiveguys.com', 'http://www.smittys.ca', 'https://locations.dennys.com/MO/INDEPENDENCE/248531', '1592236785', 'http://www.wellnessmedspacosmetics.com/', 'http://sungshouse.com', None, 'http://www.gaatithaikitchen.com', 'http://www.starbucks.com/', 'http://www.augustoalgusto.com/fr/', None, 'http://www.thegroceryohc.com', 'https://www.mcalistersdeli.com/locations/il/peoria/4810-north-university-street-1289?utm_campaign=yelp&utm_medium=referral&utm_source=yelp&utm_content=1289', 'http://www.kangsbba.com', 'https://www.4patientcare.ws/wsv3pro/web/webschedulerv3.aspx?SessionID=2050207', 'http://noahsarkpediatrics.com/', 'http://eosfitness.com/location/north-scottsdale?utm_campaign=north-scottsdale&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', 'https://www.jandbhauling.com', 'http://goodboydogs.ca', None, 'https://www.tajo-one.com', 'https://www.barkbusters.com/dog-training-charlotte-south', 'http://studiobell.ca', None, 'https://stores.jcrew.com/en/crocker-park?srccode=X%7CPMG%7CNaturalSearch%7CLocal%7CYelp%7C677', None, 'http://www.allseasonsmaintenanceandlandscaping.net/', 'http://www.reborncabinets.com', 'http://mikadosushirobata.dining.ca', 'http://www.orchidspa.net/', None, 'https://www.rolandswigsalonmatthews.com', 'http://www.vegashottubs.com', 'http://hammertoronto.com', None, 'http://www.badassjacks.ca', 'https://www.hilton.com/en/hotels/phxmdgi-hilton-garden-inn-phoenix-midtown/?SEO_id=YELP-GI-PHXMDGI', 'https://www.180smoke.ca/vape-shop-etobicoke-islington-electronic-cigarettes', None, 'http://www.burtonstudio.vegas', 'http://mississaugabarbershop.com', None, None, 'http://www.terribleherbst.com/', None, 'http://www.alovingleash.com/', None, 'http://emeraldcomputers.com/', None, 'http://www.cookout.com', 'http://www.acupuncturecenteraz.com/', 'http://www.TheKyotoBowl.com', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216347&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https:%2F%2Fwww.pizzahut.com', 'http://skintologyrx.com', 'http://leafandplate.tumblr.com', 'http://www.ballyslasvegas.com/restaurants/sea-the-thai-experience.html#.UkB2BsZnGoM', 'http://www.beinlovedesigns.com', 'https://www.athome.com/store-detail/?StoreID=Pennsylvania-Pittsburgh&utm_source=yelp&utm_medium=paidprofile', 'https://locations.theupsstore.com/pa/white-oak/1985-lincoln-way?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://eclipseautobody.com/', None, None, 'http://www.saltchaletarizona.com', '1592236923', 'http://www.rlemb.com', None, 'http://earlymercy.com', 'http://www.lpmproduction.com', 'http://www.nobullburgers.ca', None, None, 'http://www.wilsonstaekwondo.com', 'https://www.autozone.com/locations/az/avondale/10815-w-indian-school-rd.html', 'http://www.chilis.ca', 'http://eighteeneight.com/north-scottsdale-az/?utm_source=yelp&utm_medium=organic&utm_content=weburl', None, 'http://www.zenlia.com', None, None, 'http://www.bowwowdoggiedaycare.com', 'http://www.teakwoodstavern.com/', 'https://rolliciouscreamery.com/madison-1', 'http://www.goldcoastcasino.com', 'http://www.beinsoccer.com', None, None, 'http://www.vegasthedj.com', 'https://local.fedex.com/oh/mayfield-heights/office-1225/', 'https://stores.golfgalaxy.com/nc/charlotte/3039/', 'https://www.positivelyfrosted.com', None, 'http://www.azactorsacademy.com', None, None, 'http://www.rockymountainsoap.com/', 'https://pracharlotte.com/location/charlotte-cotswold-eastover/', 'https://www.taravietnamesegrill.com', None, 'http://www.ubreakifix.com/locations/pittsburgh', None, 'http://www.splaw.net/', 'https://www.olivegarden.com/locations/az/mesa/southern-fiesta-mall-west-mesa-alma-school-rd/1661?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:mesa-az-1661_pl:locurl_rd:1517', 'http://www.portofsubs.com/', None, None, 'http://www.concentra.com', 'https://www.deshonpullenlaw.com', 'http://www.mcfarlandlibrary.org/', 'https://www.wellsfargo.com', 'http://www.LegalRide.com', '1592237028', 'https://www.trainerpaul.com', None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', '1592237043', 'http://www.cefiore.com', 'http://www.roncesvallesanimalhospital.com/', 'http://jokercafe.info/', 'http://www.traderjoes.com', 'http://www.alohayogurt.com', 'http://www.outdoorlivinglv.com', 'http://www.litespressobar.com', None, 'http://www.mikesgranite.com', None, None, 'https://www.spectrumoralandfacialsurgery.com', None, 'http://www.mpi-services.com/', None, 'http://www.senseandcolor.com/weddings', 'http://gexwomenscare.com/', 'http://www.alleghenyvet.com', 'http://www.reclaimedsecrets.com', 'http://www.desertskyfamilydental.com', 'http://www.greenleafarrowheadranch.com', 'http://www.discount-drugmart.com', 'http://www.trueshinerimrepair.com', 'http://www.brars.ca', 'http://www.eagleindustriescorp.com', 'http://www.discountcab.com/', 'http://muddypawslv.com', 'http://cherryontop.net/', None, 'https://www.zoup.com', 'http://www.beckseuropean.com', 'http://www.ejsushi.com', 'https://locations.monro.com/pa/crafton/2175-noblestown-rd?utm_source=yelp&utm_medium=referral&utm_campaign=yelp', 'https://egl-junk-removal-lasvegas.business.site', 'http://pandora.net', None, None, 'http://www.drpease.com', 'http://www.zimbrickbuickeast.com', 'https://www.gessweinsclocks.com', 'https://www.huntington.com/Community/branch-info?locationId=625', 'http://www.tremontmusichall.com/', None, 'https://killumpestcontrol.com', None, 'http://fitnessconnection.com/clubs/southpark/', None, 'http://smiledocsaz.com', 'http://www.pinkislaserspa.com', 'http://www.empowerpt.com', '1592237155', None, 'http://www.artofshoerepair.com', 'http://www.starbucks.com/store/1013599/', 'https://www.londondrugs.com/london-drugs-store-030-calgary/heritage-plaza-030.html', 'http://www.libertymutual.com/brian-weaver', 'http://www.beansproutscafe.com', 'http://www.capitalone.com', 'http://www.kidzrockprep.org', 'http://www.lvsurgical.com', 'http://goharvestmarket.com', '1592237180', 'http://www.scottsdaleautosalon.com/home', 'http://www.studioadt.com/', 'http://www.bamdogwash.ca', 'http://www.staytruebodyart.com', 'http://www.liveatthepavilions.com', 'http://www.restaurantkarma.com', None, 'http://ifadevegas.com', 'http://www.loirerestaurant.ca/', 'http://www.flrpropertymgmtphoenix.com/', 'http://www.mrchickencle.com', None, 'http://cibronebakery.com/index.html', 'https://locations.timhortons.com/ca/on/richmond-hill/1070-major-mackenzie-dr.html', None, None, 'http://www.newlookhairnailandspa.com', 'https://www.elitemusic.ca', None, None, 'http://pageskensington.com/', 'https://www.bobevans.com/our-restaurants/locations/266?utm_source=yelp&utm_medium=listing&utm_campaign=fall_fy20', 'http://www.deseretfamilymedicine.com', 'http://algobertoslasvegas.onlunchroom.space', None, 'http://www.prettyyongething.com', 'http://www.solarpenny.com', 'http://www.southernnevadalocksmith.com/', 'http://www.wongsphoenix.com', 'http://www.dsw.com', 'https://palmbeachtan.com/locations/AZ/Peoria-West-Thunderbird-Road/?utm_source=Yelp&utm_medium=banner&utm_content=website&content=PHX', 'https://mountainsidefitness.com/locations/gilbert/', None, 'https://ad.doubleclick.net/ddm/clk/434177199;236814625;r', 'http://stlouiswings.com', None, 'http://www.posaresalons.com/', 'http://actionappliances.net', 'http://www.riteawayservices.co', None, None, None, 'http://www.buymaytag.ca', None, 'http://www.zibabeauty.com/', 'http://www.eternalhealthandwellness.ca', 'http://hoangthynailsspa.com/', 'http://www.kimmyztatumpoint.com', 'http://www.fisherkingwinery.com', 'https://lasvegas.colormemine.com', None, 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://www.surlatable.com/browse/storeCalendar.jsp?storeId=137', None, 'http://www.spalv.com', 'http://www.fhtaz.org', 'http://www.shell.com', 'http://hgsystemintegrators.com', 'http://littlawfirm.com/', None, 'https://duckdonuts.com/locations/huntersville-nc-rosedale-shopping-center/', None, 'http://pittsburghpetgrooming.com', 'http://www.drkintonis.com', 'http://www.daniadown.com', None, 'http://www.azrealtycare.com/', 'http://www.basspro.com/webapp/wcs/stores/servlet/CFPageC?storeID=55&storeId=10151&langId=-1&catalogId=10051&tab=3&appID=94', 'http://www.omgbakedgoodness.com', None, None, None, 'https://www.usps.com/', 'http://drupatis.com', '1592237356', 'http://www.launicahuntersville.com/', 'http://www.ardene.com', 'https://www.roots.com/on/demandware.store/Sites-RootsCA-Site/default/Stores-StorePage?storeId=187&cm_mmc=YELP-_-CANADA-_-WEBSITEURL-_-187', 'http://yourazbraces.com', 'http://www.parkwaychiropractic.net', 'https://cutrateglass.com', '1592237372', 'https://www.kohls.com/stores/nv/lasvegas-669.shtml?utm_source=rio&utm_medium=referral&utm_campaign=yelp', 'https://www.centropolis.ca', None, 'http://www.mvnailspa.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, 'http://www.wildwestmercantile.com/cgi-bin/Category.cgi?category=home&type=store', 'http://www.rosetuxedoaz.com', 'http://www.starbucks.com/store/9681/', 'https://www.therange702.com', 'http://www.mccalls.ca', 'http://www.chefkebab.ca', 'http://drjsoffice.com', '1592237404', 'http://www.waterjack.com', 'http://gonyfl.com', 'https://www.walgreens.com/locator/walgreens-275+davison+dr-sun+prairie-wi-53590/id=3291', 'http://www.kwalitysweets.com', 'http://velocitycx.com/', 'http://www.carolinashealthcare.org/cabarrus-pediatrics-map-and-directions', 'http://www.sheawomenscare.com', 'https://www.anytimefitness.com/gyms/2744/University-Heights-OH-44118/?utm_source=yelp&utm_medium=local', None, 'https://signup.myiclubonline.com/iclub/newAgreement/0389/selectPlan.htm', 'http://www.yogurtvi.com', 'http://www.venetian.com/nightlife/bars-and-lounges/the-bourbon-room.html', 'http://tropicaldelightscharlotte.com/', 'http://www.countryinns.com/champaignil', '1592237438', None, 'http://www.thecandybar.ca', None, 'http://www.pdqstores.com/gmaps/Wisconsin/MadisonArea/135.html', None, 'https://pho-fordays.com', None, 'https://www.sonoraquest.com/find-a-location/glendale-59th-ave-eugie/', 'http://www.queencitytours.com', None, None, 'http://www.atriasunlake.com', 'http://www.duetcatering.com', 'https://www.target.com/sl/charlotte-midtown/2244', 'http://www.starbucks.com/store/9298/', 'https://www.chick-fil-a.com/arboretum', 'http://marusakebar.com/', None, 'http://www.salazarusedcars.com', 'http://www.arizonadeck.com', None, 'http://www.jerusalemgrill.com', None, 'https://www.circlek.com/store-locator/us/phoenix/19830-n-7th-st/2706643', 'https://www.drtarapeyman.com', '1592237494', None, 'http://www.redcareers.com/kwrpp/', 'http://www.spiedini.com/', 'http://www.marvelousepoxyflooring.com', 'https://order.littlecaesars.com/en-us/stores/15150003/10720-e-southern-ave/mesa/az/menu', None, 'https://bhmanagement.com/communities/the-venue-on-camelback/', 'http://www.greencardadvice.com', 'http://erabyciara.com/', 'http://www.zocalobistro.com', 'http://tyrell710.wix.com/fingertips2thesoul', 'http://www.marshallsonline.com/', 'http://650cafe.com/', 'http://www.astaraskincare.com', 'http://www.arrowheaddermatology.com', '1592237535', 'http://www.serviceking.com/locations/service-king-penn-hills?utm_source=yelp&utm_medium=local&utm_campaign=yelp-profile', 'http://locations.michaels.com/az/mesa/2778/?utm_campaign=Google+Places&utm_medium=Maps&utm_source=Google', 'http://www.centurylink.com', 'https://locations.jackinthebox.com/us/ca/concord/1710-monument-blvd', None, 'http://www.dads-smokeshop.com', 'http://www.friendlyneighborhoodhelp.com', 'https://www.olivegarden.com/locations/oh/parma/parma-town-mall/1128?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:parma-oh-1128_pl:locurl_rd:1083', 'http://www.beautifulskinbylucy.com', 'http://www.ritasfloraldesigns.com', 'https://ad.doubleclick.net/ddm/trackclk/N3493.Yelp/B24050224.272429163;dc_trk_aid=466899273;dc_trk_cid=127976389;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=', None, 'http://www.arcelectriccompany.com', 'http://www.7-eleven.com/?yelp=29384', 'https://locations.massageenvy.com/arizona/scottsdale/8752-east-shea-blvd-ste--c-3b.html', None, None, 'https://redrock.sclv.com/Gaming/Race-and-Sports-Book?utm_source=Yelp&utm_medium=organic&utm_campaign=fb&utm_content=listinglink', 'https://www.target.com/sl/las-vegas-blue-diamond/2164', 'https://www.arashijapansteakhouse.com', 'http://www.mgpaz.com', 'http://www.silversnail.com', None, None, 'http://www.ajsfinefoods.com', 'http://www.lasvegasguitarteacher.com', 'http://www.trilogypoolsaz.com/', 'http://poshpooches.ca', 'http://www.aleromoving.com/', None, 'http://www.litchfieldauto.net', 'https://www.crowncustomtattoo.com', 'http://www.illinismilesdental.com', None, 'http://www.udallattorneys.com', 'http://www.rvmastertechs.com', None, 'http://www.buffaloscafe.com', 'http://www.bengioielli.com', None, 'http://www.moulinbouge.com', 'http://birkdalevillage.net/portfolio/birkdale-village-cleaners', 'http://Munhallvet.com', 'http://www.hillhurstmortgage.com', 'https://www.zaxbys.com/locations/sc/fort-mill/3401-hwy-21/', None, 'https://www.bestwestern.com/en_US/book/hotel-rooms.03157.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:03157', 'http://ilfornello.com/richmond-hill/', None, None, None, '1592237668', 'http://www.bigotires.com/regions/minneapolis-metro', None, None, None, 'http://Www.electrikneedletattoo.com', 'http://www.apachetaxi.com', 'http://www.ibdaz.com', 'http://www.gfsbjj.com/', None, 'http://www.matthewsalehouse.com', 'http://www.antoniospizza.net', None, 'http://eastvalleyurban.com/phoenix/stella', 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', None, None, 'http://DDLDental.com', 'http://familydentalstudio.net', None, None, 'http://www.friendsofthewild.org', 'http://www.libertymutual.com', None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.einsteinbros.com/', 'http://www.paradisebaypetresort.com/', 'http://www.everlastingchanges.com', 'http://www.quiktrip.com', 'https://www.lesliespool.com/?utm_source=yelp&utm_medium=social%20media', 'http://thebeatbooth.com', 'http://babycakeswilloughby.vpweb.com', 'http://www.arizonastone.com', 'http://www.newdirectionfertility.com', 'https://www.vegaslashlounge.com', '1592237755', '1592237758', 'http://www.thecurzon.ca', None, 'http://www.samsbarbershops.com', 'http://www.imeldasmith.com', 'http://tinkers-handyman-service.business.site', 'http://www.starbucks.com', None, None, 'http://www.shaggyshaggyshake.com', None, None, None, None, None, 'https://www.supercuts.com/locations/nv/henderson/eastern-commons-haircuts-9195.html', None, None, None, 'http://pho87.net', None, 'http://poolpartsplus.net/', 'https://www.dominionselfstorage.net/self-storage/az/queen-creek/dominion-self-storge-queen-creek/unit-sizes-prices', None, 'http://www.sherwin-williams.com/store-locator/paint-store/Las%20Vegas/NV/8114', 'https://www.greatvaluestorage.com/self-storage/las-vegas-nv-92971', 'https://konjikiramen.com', 'http://yackodentistry.com', 'https://www.podiatryinc.com', 'http://www.freewayinsurance.com', 'https://chocolateshops.sees.com/az/phoenix/chocolate_shops_phoenix_az_102103.html', 'https://acesandarrows.com', 'http://www.noisette.ca/index.html', 'https://locations.cinnaholic.com/ll/CA/ON/Scarborough/6015-Steeles-Ave-E', 'https://www.sunnysidehonda.com', None, None, None, 'http://www.bwsweetsbakery.com', '1592237855', None, 'http://www.charlotte.studio-cellar.com', 'http://www.xooro.com', 'https://www.sonoraquest.com/find-a-location/phoenix-by-appointment-only-tatum-shea/', 'https://www.cellphonerepair.com/charlotte-uptown-nc/', 'https://ad.doubleclick.net/ddm/clk/458685299;262921492;c', 'http://www.somunsuperstar.com', '1592237871', 'http://www.archeryhq.com', 'https://www.agoracleveland.com', 'http://victoryenergywellness.com/', 'http://www.geeksnthings.com', 'https://nodabrewing.com', None, 'http://phoenix.sentrymgt.com', 'http://www.caringheartsanimalclinic.com', 'http://www.stellavapors.com', None, 'http://www.prohibitionclt.com/', 'http://scenicbrewingcompany.com', None, None, None, 'http://justpuckit.com', 'https://www.avioflower.com', 'http://www.bbhairringtons.com', 'https://www.olivegarden.com/locations/oh/north-olmsted/north-olmsted-great-northern-mall/1133?cmpid=br:og_ag:ie_ch:dry_ca:OGQ121LOCURL_dt:20200604_sn:yelp_gt:north-olmsted-oh-1133_pl:locurl_rd:1087', 'http://www.chousky.com', 'https://speedstarautotransport.com', 'http://www.kolornkiln.com', 'https://www.b1locksmith.com/chandler-locksmith', 'http://www.gamestop.com', 'http://www.gamestop.com', 'http://www.littlecaesars.com', 'http://www.fetaolivesgrill.com', 'http://pittsburgh.vettechinstitute.edu', 'http://luna-street-food.business.site', 'http://centurymill.com', 'http://www.BeestungLingerie.com', 'http://www.rapunzelhair.ca', 'http://www.reliableautolv.com', 'http://extremevaluestore.com/', None, 'https://www.happyaxe.com', 'https://kipc.vegas', None, None, None, None, 'http://www.coastappliances.com', 'http://www.phoenix.egoscue.com', 'http://www.bartonsbakeshop.com', 'http://www.banfield.com/veterinarians/wi/madison/emn?utm_content=yelp_general&utm_medium=PAID&utm_source=yelp&utm_campaign=YelpPaid', 'http://www.homehardware.ca/en/dealer-microsites/1274-8/contact-information.htm', 'http://www.europeanmotorstudio.com', 'https://wildfire.sclv.com/The-Greens', 'http://www.boosterjuice.com/', '1592237980', None, 'http://uniquepatiocreations.com', 'http://www.wexlerstavern.com/', 'http://theplacearizona.com', None, 'http://www.mtlebanonfloral.net/?srccode=yelp_track', '1592237997', 'https://thepetclub.com', 'http://www.encompasshealth.com', 'http://www.dimanddensum.com/', 'http://www.wingatehotels.com/hotels/hotel-overview?wid=ps%3Aaddppc_yelp&tel=18885953511&reg=appc-_-YLPROPP-_-30442-_-winwes&hotel_id=30442&cid=whg_winwes_yelp', 'http://dekra.us/en/nevada/las-vegas-3430-e-tropicana-ave/', 'http://lasvegassolarinstallation.com', 'http://www.hairpieceslv.com', 'http://www.thecharlestondtlv.com', '1592238018', 'https://www.caferio.com/locations/eastern', 'http://www.skysalon.com', None, 'http://www.tourneau.com/custserv/locate_store.cmd?storeNumber=33', 'https://www.bootbarn.com', '1592238033', 'http://www.modelcitizentoronto.com/', 'http://www.epiphanychandler.org', 'http://www.trueblueautomotive.com/', 'https://www.anatomica.ca', 'http://churchschicken.ca', 'http://facialsbysofia.com', None, 'http://www.davidesbakeryandcafe.com/', 'http://www.calgarygold.ca', 'http://www.laroccaspizzeria.com', 'http://www.rowhousecinema.com', 'http://www.buddhapie.ca', '1592238062', '1592238065', 'http://www.sunshinegrandcanyon.com', 'http://www.handysacehardware.com', 'http://www.honeybutterbakery.com/', None, 'http://carrepairvegas.com', 'http://www.tortoiseandharesports.com', None, None, 'http://www.buyerschoicelv.com/', None, None, 'http://www.erieislandcoffee.com', 'http://www.baytowne.net', 'https://www.sephora.com/happening/stores/gilbert-san-tan-village', None, '1592238099', 'http://www.glendaleca.gov/', 'http://www.youreliteplumber.com/', 'http://bestimpressionscaterers.com/main.php', None, None, None, None, None, 'http://www.newvisionaz.org', 'https://www.publicstorage.com/nevada/self-storage-las-vegas-nv/89146-self-storage/1477?pid=wyelp&CID=1245&CHID=AFL', None, None, 'http://larmco-oh.com', 'http://www.maisonettedistillery.ca', None, 'http://www.colonandrectalclinic.com', 'http://www.sirpizzafranklinpark.com', 'https://www.t-mobile.com/store-locator/nv/henderson/galleria-at-sunset-mall?cmpid=WTR_SE_YXT_U98MLWB9027148', 'http://www.dressbarn.com', 'http://www.choicehotels.com/en/choice-privileges', None, 'http://lrcorralescatering.weebly.com/', 'https://www.floxondanforth.com', None, 'https://dresspeopleltd.com/', None, 'http://Leonscaribbean.com', None, 'https://www.zippssportsgrills.com', 'http://www.clancyspubscottsdale.com/', 'http://www.alimentarimadison.com', None, None, 'http://www.atlashomeinspections.com', None, 'http://zinicolaitalian.com', 'http://www.sunsetclubcharlotte.com/', 'http://www.airspecialiste.com/', 'http://www.deltaco.com', 'http://www.wincofoods.com/', 'http://www.DFgym.com', 'https://www.smilesbysmith.com', 'http://www.drbahmanguyuron.com', 'http://salonstudiowest.com', None, '1592238192', None, 'http://royaldutchgrooming.net', 'http://www.dentistsonbloor.ca/', 'http://www.presidentspizza.com', 'https://www.bestwestern.com/en_US/book/hotel-rooms.03133.html?iata=00171910&ssob=BLBWI0005G&cid=BLBWI0005G:yelp:businessurl:03133', 'http://www.scottsdaledentalcare.net', None, 'http://www.nailsforyou.ca', 'http://www.zumiez.com', None, 'http://www.expressgaragedoorservice.com', 'http://acebroslv.com', 'http://www.tacarbon.com', 'https://www.onemainfinancial.com/branches/PA/Pittsburgh/15234/4721', 'http://www.starkmans.com', 'https://preciousmetalsnv.com', 'https://www.extraholidays.com/las-vegas-nevada/worldmark-las-vegas-spencer-street?lid=yelp&utm_medium=referral-affiliate&utm_source=yelp.com&Identifier=yelp&hotelid=103712&utm_campaign=localpage-worldmarklasvegas-spencer&promoid=localpage-worldmarklasvegas-spencer', None, 'https://www.lovestbrides.com', 'http://www.sakunthaionline.com/', 'http://www.gocomtrans.com', 'http://www.sanremobakery.com/', 'https://piedmontmaids.com/local-house-cleaning/nc/monroe', 'http://www.fluidmotionpgh.com', 'http://www.blisscupcakes.com/', None, 'http://www.tanprousa.com', None, 'http://www.rebekhasushi.com', 'http://www.crispfoods.com', 'http://www.usps.com', 'http://www.lvmpd.com', None, None, None, 'http://www.park2go.ca', 'http://www.horstmancomputers.com', None, 'https://www.walmart.ca/en/whitby-supercentre/3113', None, 'http://colosseumtraining.com', 'https://www.kindercare.com/our-centers/phoenix/az/000477?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=000477', 'http://clickserve.dartsearch.net/link/click?lid=43700044280216206&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'http://www.anytimedent.com', None, 'https://www.westelm.com', 'http://www.grayhawkbeerco.com', 'http://www.nextcare.com/locations/az/phoenix-thomas/?utm_source=Yelp&utm_medium=Local&utm_campaign=phoenix_thomas', 'http://goclarendon.com/restaurant', None, None, None, 'http://www.quiktrip.com', 'http://charliepalmersteak.com/locations/las-vegas', 'https://www.sprouts.com/store/az/laveen/laveen//?utm_campaign=website&utm_medium=listing&utm_source=yelp', 'http://www.thomasharman.com?cmpid=czhr_blm_0006', None, 'https://www.autoactionaz.com/dealership-locator/locations-1909-west-van-buren-street-phoenix-az/', 'http://www.ridgecleaners.com', 'http://www.plantworks.com', '1592238330', 'http://www.powerhouseplatinum.com/', 'https://thefortunatefox.com', 'http://www.ghazale.ca/', 'http://www.outback.com/locations/nv/las-vegas-craig', None, 'http://www.boom973.com/index.aspx', 'http://www.maac.com/texas/dallas/colonial-grand-at-hebron?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', 'http://ArizonaSailingAdventures.com', None, 'https://myhearingcenters.com/locations/nevada/summerlin', None, None, 'http://www.wellerracing.com/', 'http://www.timhortons.com', 'https://local.fedex.com/nc/pineville/PNVA/?cmp=LOC-1001545-3-2-971-1110000-US-US-EN-YEXTLISTFEDEXSC', 'http://primmed.org', None, 'https://www.cleanjuice.com/menu/', 'http://www.innovativehclinic.com', 'http://www.quickchangeoil.com/', 'http://www.azmortgagebrothers.com', 'https://valleywisehealth.org/locations/community-health-center-el-mirage/', 'https://www.menswearhouse.com/store-locator/2504?utm_source=yelp&utm_medium=listing&utm_content=2504&utm_campaign=AZ', 'http://www.liveatavanteapts.com', 'https://www.staples.com', 'http://www.northamerican.com/?refcode=844-434-1236&utm_source=yelp&utm_campaign=desktop', None, 'http://99only.com/store/Las-Vegas---Beltway/', 'https://theresortatsummerlin.com/dining/casual-dining/market-place-buffet-las-vegas/', 'https://www.jph4ever.com', None, 'http://www.imsaz.com', 'http://www.afabaz.com', None, 'http://www.perfectlookhairbraiding.com', 'http://www.regissalons.com/salondetail/default.asp?salonid=448', 'http://armor-plumbing-boiler.com/index.php', 'http://www.greatwraps.com', 'http://www.steepery.com/', 'http://nailsqueentoronto.ca', None, 'http://www.rodneysbybay.com', None, None, 'http://www.fitnesspropelled.com/', 'http://www.hawksbar.com', None, None, 'http://www.shakeitupbartending.com', 'http://www.salts-of-the-earth.com', None, None, 'http://store-locator.barnesandnoble.com/store/2039', 'http://www.vbflv.org', 'http://www.outlettags.com', 'http://www.702wingspot.com/', 'http://www.kitchenconnectionllc.com', 'http://www.lvparking.com', 'http://www.radair.com/', None, None, None, 'http://magroclothing.com', 'http://www.furnituretoronto.com', None, None, 'http://www.k2elec.com/', 'http://www.delanolasvegas.com/pool/delano-beach-club.aspx', 'http://woksupreme.com/', 'http://www.markandterri.com', 'http://www.goodlifefitness.com/locations/ontario/vaughan/milani-blvd?source=yelp-websitelink-vaughanmilani-gl', 'http://www.thegreatsteak.com/stores/7525', 'http://www.newdragonexpress.com/', 'http://www.daalroti.ca/', None, None, 'http://www.yourstarpupil.com', 'https://www.mccarran.com/Fly/WhichTerminal.aspx', 'http://www.lasvegascounselingcenter.com', None, None, 'http://www.glastek.com', '1592238519', None, None, 'http://www.burlingtoncoatfactory.com', '1592238526', 'https://www.galleriamc.com', 'https://www.tcclinic.com', 'https://www.midas.com/phoenix/store.aspx?shopnum=1109&dmanum=684001', 'http://az.scottsdale.charmingcharlie.com/fashion_accessories_scottsdale_az_45.html', 'http://www.shell.com', 'http://www.lapetite.com/your-local-school/monona-wi-7148/?utm_source=yelp&utm_medium=paid-directory&utm_campaign=enhanced-listing', None, 'https://www.jabzboxing.com/locations/32ndandshea/', None, 'http://www.tipsetreats.com', None, 'http://www.timhortons.com', 'http://www.amazingphotocamera.com/', None, 'http://www.waddellslonghorncorral.com/', 'http://www.desertsplashadventures.com', 'http://suburban-sanitation.com', None, None, None, 'http://www.thegilcreaseorchard.org', None, None, 'http://www.yasisplace.ca', None, 'http://philhousegrill.com', '1592238583', 'http://www.opticcreations.com/', 'http://www.trainingrevelations.com', None, 'http://www.statclinix.com', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, '1592238598', 'http://locations.daltile.com/az/tempe/118', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, '1592238610', 'http://www.josephsfoodandcakes.com', None, 'http://www.rowdyhair.com', 'http://www.starbucks.com/', 'http://jackastors.com/location-map/18/', None, 'https://www.choicehotels.com/ohio/wickliffe/econo-lodge-hotels/oh335/rates', None, 'http://www.blondesalon.ca', None, 'https://www.papajohns.com/international', 'http://luv2play.com/surprise/', 'http://independenceplumbing.net', 'https://pawn1st.net/our-locations/pawn1st-surprise-arizona/', None, 'http://www.lasvegasprostho.com', 'http://www.isshoniramen.com', None, 'http://www.eatatnakama.com', 'http://www.macklems.com/', '1592238663', 'http://www.towerofjewels.com/', 'https://stores.dickssportinggoods.com/oh/aurora/121/', 'http://www.lomography.ca', 'https://bostonpizza.com/fr/locations/st-bruno.html', None, 'https://www.kindercare.com/our-centers/goodyear/az/301663?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=301663', None, 'http://www.tinytooth.com', 'https://custommufflersaz.com', None, 'http://www.hairsmith.co', 'http://www.medicineshoppe.com/pharmacy-locations/pennsylvania/mt-washington/home', 'http://www.virgoesthetics.com', 'http://fawnboutique.ca/', 'http://www.btrustsupermarket.com/', None, None, 'http://www.allhoursplumbinginc.com', 'http://www.pridemgmt.com', 'https://thrifty.com/loc/ll/CA/OT/TORONTO-~-DANFORTH-(OT)/1245-DANFORTH-AVE', 'https://www.wisconsinrvworld.com', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.sfnevada.com?cmpid=UN7I_BLM_0006', None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://www.shibuyala.com', 'http://www.bell.ca/internet', '1592238730', None, None, 'http://www.deltaco.com', 'http://www.pizzapizza.com', 'https://habanerofreshmex.com', 'http://www.golfanything.com', None, 'https://la-coppa.com', 'https://www.montanas.ca', '1592238753', 'https://www.rallyhouse.com/rally-house-parma', 'http://balloonscharlotte.com', 'http://tton10.wix.com/pizanocustomautobody', 'http://www.secondcup.com/location/toronto-355-danforth-avenue', 'http://www.mmpguns.com', None, 'http://www.hair-dosalon.com', 'http://www.oxfordcleanersaz.com', None, 'http://hookahlounge.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=7781&localechange=1', 'http://www.silverstatevapor.com', None, 'http://www.sessionstattoo.com', '1592238788', None, None, None, 'http://wokintercuisine.com', 'http://sushi-ten.com/', None, None, 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=2179&localechange=1', 'http://madisoncircusspace.com', None, None, 'https://myhoneybakedstore.com/charlotte/114', 'https://wexford.orangetheoryfitness.com/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', None, None, None, None, 'http://www.marshallsonline.com', 'http://www.tiashortys.com', 'http://www.mariscosacapulco.org/Mariscos_Acapulco/Inicio.html', 'https://cmwireless.com', None, None, None, None, 'http://thelocalbarberandshop.com', None, 'http://www.mcgrathcpa.net', 'http://www.northranchvet.com/', 'http://www.unosalonlv.com', 'http://michelsbakerycafe.com/?q=node/1/en/', 'https://www.ebgames.ca/', 'http://www.drivesafeaz.net', 'http://www.journeys.com', 'https://www.engagestudio.com', None, 'http://www.azcostumeshop.com/', 'http://www.painsetsaveurs.com/', 'http://www.extraordinarytoys.com', 'https://www.dairyqueen.com/ca-en/Locator/Detail/?store-id=12605&localechange=1', None, 'http://www.fabricland.ca', None, 'http://www.quiktrip.com/', 'http://www.ahwatukeehoa.com', None, 'http://chase.com', None, 'http://Butlerfamilydentalaz.com', None, None, 'http://www.haydenelectricinc.com', 'http://www.parmatireservice.net', 'http://www.marcos.com', None, 'https://www.eliterooterphoenix.com', None, None, 'http://www.ilovepokebar.com', 'http://sushi-mori.ca', 'https://www.propertymanagementscottsdaleaz.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.jnphoenixfamilylawyer.com', 'https://www.vans.com', 'http://www.rowefarms.ca', 'https://www.sephora.com/happening/stores/hillcrest', None, None, 'https://www.cubesmart.com/nevada-self-storage/las-vegas-self-storage/90.html?utm_source=yelp&utm_medium=seo&utm_campaign=yelp-test-bp', 'https://shakeshakeseafood.com/contact_monroe', 'http://www.montessorivisions.com', None, None, None, 'http://www.MarshProperties.com', 'http://www.setispolishboys.biz', 'http://www.ctuckerandassociates.com', 'https://www.everbrookacademy.com/our-schools/charlotte-nc-4608/?utm_campaign=enhanced-listing&utm_medium=paid-directory&utm_source=yelp', None, 'http://www.costco.ca/', 'http://www.azdermacare.com/', 'http://www.yumbudz.ca', 'https://theeggworks.com', 'http://www.forgetaboutitsupperclub.ca', 'http://www.lbcexpress.com', None, 'https://www.reflectionzdetail.com', 'http://link.hertz.com/link.html?LinkType=HZLK&TargetType=Homepage&id=46190', 'http://www.lebelandcrowe.com', 'http://www.greencleanerslv.com', 'https://drgeoffreysherivf.com', 'http://www.beadworksjo.com', 'http://www.craigscustomupholstery.com', 'https://www.walmart.com/store/5066/avon-oh/details', None, None, 'http://www.philotavern.com/', 'http://www.abercrombiekids.ca/', None, 'http://www.jessieraesbbq.com', 'http://www.vetemergency.ca', 'http://www.carolinadoughnuts.com', '1592239007', 'http://www.starbucks.com', 'http://www.thespainthehollow.com/', '1592239015', 'http://www.bestclean-llc.com/', 'http://galenaeuropeanspa.com', 'https://www.kidsquest.com/location/boulder/', 'http://www.vsschnitzelhouse.com', None, None, 'http://www.fluidconcepts.net', 'https://www.myfitfoods.com/', 'http://kathysalterations.biz', None, 'https://www.cvs.com/store-locator/cvs-pharmacy-address/16545+East+Palisades+Boulevard-Fountain+Hills-AZ-85268/storeid=9279?cid=ps_ylp_2020', 'http://reflectionsautodetailing.me', None, 'http://www.thecannerygrill.com', None, None, None, 'http://www.paintboy.com/', '1592239059', None, None, 'http://arizonawindowwashers.com', 'http://www.courtyardatencanto.com/', 'https://titanicaz.com', None, 'http://labayoucasino.tumblr.com/', None, 'http://www.sapporoscottsdale.com', None, None, 'http://kappatoys.com', 'http://www.shoelabworld.com', 'http://www.goodwillaz.org', 'http://bk.com/', 'http://www.soldentalaz.com', 'http://www.lasvegaskidsrkids.com', 'http://www.caninecountry.ca/', 'https://www.halibuthouse.ca/locations/halibuthouse560mulockdrive/', 'http://www.pinoyscoffeeandboba.com', 'http://cp.cuyahogacounty.us/internet/index.aspx', None, 'http://www.airdrieautotrends.com/', 'http://www.leonesflorist.com', 'http://www.wavescoffee.com', 'http://viennamarketbroadway.com', None, 'http://www.desertskysurfaces.com/', None, 'http://www.chinachilirestaurant.com', 'http://www.bethanychristianschool.org', 'http://www.mazonac.com', 'http://stores.dickssportinggoods.com/az/mesa/725/', 'http://charlottefiberglass.com', None, 'http://www.chinapalacesewickleypa.com/', 'http://www.aceaxethrowing.com', 'http://www.exlacademy.com', 'http://www.deltastore.com', 'http://www.desertsunpt.com', 'https://bloomingnails-nailsalon.business.site', 'http://www.caremd.com/', 'http://giovanniandcompany.com/', None, None, 'http://www.razzles.biz/', None, None, None, 'http://www.dollartreecanada.com', 'http://www.socialsolutionsaz.com', 'http://www.springdayspa.com', None, 'http://www.bbkingclubs.com/', 'http://haciendasolanoresort.com', 'http://www.xperiencevapes.com', 'http://www.leakpoint.com', 'https://www.abccollisionrepair.com/?utm_source=yelp&utm_medium=listing&utm_campaign=yelp-business-listing', 'https://stores.loft.com/us/pa/homestead/126-west-bridge-street.html', None, None, 'http://eosfitness.com/location/tempe?utm_campaign=tempe&utm_medium=display&utm_source=yelp&utm_content=7-day-pass', None, None, '1592239206', None, 'https://dealer.sprint.com/experts-choice', 'http://local.biglots.com/oh/stow/453', 'http://www.yayaspizza.com/', 'http://www.portioncontrolnv.com', None, '1592239219', 'http://www.usmgcorp.com', None, None, 'https://www.boydautobody.com/locations/calgary-1st-st-ab', 'http://www.tranquil-lily.com', 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'https://www.aireserv.com/north-valley', None, 'http://www.bluerosepottery.com', None, 'https://locations.panerabread.com/oh/highland-heights/6345-wilson-mills-road.html?utm_medium=display-ad&utm_source=paid-digital&utm_campaign=yext&utm_content=local-search', 'http://www.retroburger.ca/', 'http://crunchbistro.com', 'http://www.valuevillage.com', 'http://www.bleedingrosetattoo.com', None, None, None, 'https://www.goodyearautoservice.com/en-US/shop/AZ/Glendale/000908234-85308-2429', 'http://www.lisboabakeryanddeli.com', None, None, 'http://www.swingpgh.com/', 'http://www.daydreamsdropin.com', 'http://valleypoolcleaning.com/', None, None, 'http://www.vaultnc.com/', 'http://www.tonicsalonnv.com/', '1592239283', 'https://airpromaster.com', 'http://www.queencityautosalesnc.com', 'http://pageperpage.com', None, 'https://www.firehousesubs.com/locations/oh/willoughby-commons', '1592239298', None, 'http://laptoprepairs.com', None, None, 'http://www.lashliqueur.com', 'http://www.airportgatecafe.com/', 'http://www.naturallyyours.ca', 'http://www.shopkkbloom.com', 'http://www.honestyelectricllc.com', None, 'http://www.husd.org', 'http://www.ucalgaryrecreation.ca', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.exacttemphvac.com', None, None, 'http://www.madisonorientalwok.com', 'http://www.crossfitarchangel.com', None, 'http://www.jolieannephotography.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'http://www.rockabettyssalon.com/', 'http://c2cnd.org', 'http://www.fantasticsams.com/', None, 'http://www.gamestop.com', 'https://www.safelite.com', 'http://cleanenvision.com', 'http://www.ahugheshairdesign.com', None, 'http://www.filibertos.com', 'http://www.kmjwebdesign.com', None, 'http://www.dillysdeli.com/', None, 'http://www.kimchikoreahouse.ca/', None, None, 'http://www.mississauga.ca/portal/discover/parksandgardens?paf_gear_id=9700018&itemId=300019', 'https://www.publicstorage.com/arizona/self-storage-mesa-az/85210-self-storage/787?pid=wyelp&CID=1245&CHID=AFL', '1592239382', 'http://www.teavasee.com/', 'http://www.popularpizza.ca/', None, 'http://www.steffiblackcoaching.com', 'http://www.virginiajohnson.com', 'http://www.kindercollege.ca', None, 'http://www.toysrus.com/family/index.jsp?categoryId=2255956&camp=MISC:yelp:store:page:tru:111910', 'http://www.gdpeds.com/', 'http://www.coldwaterspringsapts.com/', 'http://www.eyecarecenter.com/optometrist-locations/matthews-mint-hill-rd/?address=28105', 'http://www.minus5experience.com/', None, None, 'http://premiercleaners.com', None, 'http://www.theherbcard.com', None, None, 'http://www.maac.com/north-carolina/charlotte/post-ballantyne?utm_medium=cpc&utm_source=yelp&utm_campaign=2019', None, None, 'https://www.unlv.edu/dental', 'https://www.thestonecollection.com/phoenix', 'http://www.sweetpetes.com', None, 'https://www.aaautocarelv.com/locations/decatur-blvd/', None, 'http://www.allnationsdriving.ca', 'http://icehousebrooklyn.com/', 'http://www.carusogourmetpizza.com/', None, None, 'http://www.sparrshspa.com', None, 'https://www.mezzo1.com/?utm_source=yelp', 'http://www.jimtuckerroofing.com', 'https://www.redmountainweightloss.com/ahwatukee/', None, 'http://www.jackinthebox.com/', None, 'http://www.thebeautyloftaz.com', 'http://www.papadelspizza.com/', 'https://www.maricopa.edu', None, None, 'https://www.lasereyecenteraz.com/eye-doctors/kianoush-kian-m-d/', 'http://www.Altus-HPO.com', 'http://yourspacesalons.ca/natural-solutions', None, None, None, 'http://www.buckle.com', 'https://locations.wendys.com/united-states/nc/charlotte/7920-cambridge-commons-drive', 'https://www.glassdoctor.com/phoenix', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=1131', 'http://goodfoodmadison.com', None, None, 'http://www.suicidejims.com', None, None, 'http://www.thelaughshopcalgary.com', None, 'http://www.skyzone.com/phoenix', None, None, 'http://www.animalcareclinicinc.com/henderson-animal-hospital', None, 'http://www.center-towing.com', 'http://www.regency.edu/campus/north-olmsted-ohio.aspx', 'https://www.purdys.com/shops/details/79', 'http://www.hottopic.com', 'http://www.carolinatotalcare.com', None, 'https://locations.jackinthebox.com/us/nv/las-vegas/1591-s-main-st', 'http://archstonecare.com/', 'http://www.kibosushi.com', 'http://rulysauto.com', None, 'https://www.earthfare.com/stores/', 'http://www.allapplianceneo.com', 'http://denver.unique-accounting.com', None, 'https://www.wyndhamhotels.com/hotel/33877?tel=18003291073&iata=00093785&cid=fe:di:20161220:yl:pp:dius:33877', None, 'https://vansoncuisine.ca', 'http://www.donsappliances.com', 'http://alphillipslv.com/', '1592239567', 'http://www.larrystow.com', 'https://www.marleneharriscol.com', 'http://www.consonantskincare.com', None, 'https://www.tgifridays.com/locations/1742/fridays-charlotteuniversity-north-carolina', 'https://locations.eatandys.com/ll/US/NC/Concord/8835-Christenbury-Pkwy', 'http://jmacvideo.com', 'http://www.aosmlv.com', 'http://enticare.com', 'http://www.marriott.com/hotels/travel/pitmv-springhill-suites-pittsburgh-monroeville/?scid=bb1a189a-fec3-4d19-a255-54ba596febe2', 'http://www.theciderfarm.com', 'https://www.presslyanimalhospital.net', None, 'http://www.remax.com/realestateagentoffice/lasvegas-nv-89128-sandrathomas-id11154472.html', 'http://www.meadowlarkestables.com/', 'http://www.salon-atmosphere.com/', 'http://chicagobrewinglv.com', None, None, None, 'http://www.mrlube.com/en/Mr-Lube-Near-You/Locate-a-Mr-Lube/mississauga-2191-dundas-street-e.aspx', 'http://www.doitbest.com', 'http://sterlingssteakhouse.ca', 'http://www.saloncollage.com', 'http://www.eatatbobsdiner.com/', 'http://www.kamasutrarestaurant.ca', 'http://www.swisschalet.com', 'http://www.dunkindonuts.com', None, 'http://locations.atipt.com/grayhawk-scottsdale-az', 'http://www.magicvaporllc.com', 'http://www.starbucks.com', 'http://www.AllBreedMadison.com', 'http://www.montanameatco.com', 'http://bk.com/', 'http://www.starbucks.ca', None, 'http://www.autobell.com/ab_locations/750-n-church-st/', None, None, 'http://www.marriott.com/hotels/travel/cleta-towneplace-suites-cleveland-airport', None, None, None, 'http://workoutanytime.com/charlottewest', 'http://carolinasmiledesignsc.com', 'https://depotbarbershop.business.site/?m=true', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', '1592239672', 'http://www.libertyiron.com', 'http://aaffordablebabysitting.com', 'http://www.quesada.ca', 'http://bk.com/', 'https://locations.theupsstore.com/wi/sun-prairie/1223-w-main-st?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'http://www.kay.com', 'https://www.ugg.com/stores/s/fashion-show-mall?latitude=36.127172&exclude=18231&page=1&longitude=-115.171661&storesPerPage=3', 'http://www.creolecajunbistro.com', None, None, None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpsoutheastorganic', 'http://www.wolfcamera.com', 'http://www.corygoldbergmd.com/', 'http://www.harborfreight.com', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.paksweets.com', 'http://www.airpads.com', None, 'http://www.norterrafamilymedicine.com', None, 'http://www.kensingtonbrewingcompany.com', 'http://25chiropractic.com/', '1592239728', 'http://coffeedaklak.com', None, None, 'http://freddyrguez.googlepages.com/', 'https://www.crossborderpickups.ca', 'http://www.dilworthbilliards.com/', None, 'http://www.ahghomewarranty.com', None, None, '1592239754', 'http://www.guggenheimhermlasvegas.org', 'http://www.pearl-factory.com', None, 'https://local.biglots.com/pa/pittsburgh/1478?zcp=pd_local_yelp', 'https://www.navyfederal.org/?cmpid=bn%7Cyelp%7Cevgrn%7Cbrn%7Cnull%7Cnull%7C01/13/2020%7Cml%7Cnull', 'http://www.sallybeauty.com', 'http://www.salonfair.com', 'https://sunvek.com', 'http://www.midas.com/calgary/store.aspx?shopnum=9304&dmanum=151', 'http://www.bakersbookkeeping.com', None, 'https://phoenix.swimspas.us', None, 'http://torontopremiumoutlets.ca/outlet/toronto', 'http://www.midwifeskaleidoscope.org', 'http://www.redrockfencingcenter.com', 'http://www.azairworks.com/', 'http://auraapts.com', None, 'http://www.discountroofingnv.com', 'http://www.drvinyl.com', 'http://www.newskyrestaurant.ca', None, 'http://www.portauthority.org', 'http://www.downtownbrewfestival.com', None, 'http://www.globallylocal.ca', 'http://www.kakabeautyspa.ca', None, 'http://www.noelinwheeler.tattoo', 'http://concourseroffler.blogspot.com', 'http://www.rexall.ca/', 'http://www.trailsidetreasures.com', 'http://www.nevadatirecity.com', 'http://www.cornerstonewomenscare.com', 'http://www.easyhome.us', 'http://www.mondashaircare.com', 'https://www.desertfinancial.com/Home/BranchDetails?branchId=1', 'http://newks.com', 'http://www.dcspoolbarriers.com', 'http://www.drrochlin.com', 'https://www.fit4less.ca/locations/alberta/calgary/408?utm_source=yelp&utm_medium=cta&utm_term=calgarymacleod-f4l', 'http://www.julietlefleur.com', 'http://www.arizonahairco.com/', 'http://www.dunkindonuts.com', 'http://montana-hookah-longue-and-smokeshop.business.site', 'http://miniso.ca', 'http://www.westairaviation.com', 'https://www.sunrisepreschools.com/locations/baseline-gilbert/?utm_source=yelp&utm_campaign=online_listing_gilbert_280&utm_medium=referral&utm_term=business_url', 'http://www.mackdentistry.com/', 'http://thayersselectmeats.com', 'http://undergrounddiesels.com', '1592239872', 'http://www.samuraicomics.com', '1592239878', 'http://www.michaelkors.com/stores/united-states/nv/las-vegas/michael-kors-palazzo', 'http://willytechshop.com', 'http://www.vernspizza.com', 'http://www.clearviewinstitute.com/', 'http://www.berkeleyplace-apts.com', '1592239893', None, None, 'https://venecia-beauty.business.site', 'http://www.haircutmenriverbendcalgaryab.ca', 'https://www.hungryhowies.com/store/hungry-howies-01805?utm_source=Yelp&utm_medium=Referral&utm_campaign=Yelp_CTA&utm_term=01805&utm_content=Website_Link', 'https://sophiaslounge.com', 'http://www.taxicabchandler.com', 'http://www.bestchickenwings.com/', 'http://www.athletico.com/locations/tempe/?utm_campaign=yelp-listing&utm_medium=web&utm_source=yelp-tempe', 'http://www.solsticerestaurant.ca/', 'http://www.azclubluxx.com', 'http://www.sethkoreytennis.com', None, None, 'http://www.flairtrade.ca', None, None, None, 'http://www.vegasmartialarts.com', 'http://riobel.ca', None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, None, None, None, 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', 'http://www.hairea51.net', 'http://www.dmvdrop.com', 'http://carolklemp.wix.com/aefitness', 'http://www.3Brewers.ca', 'http://www.lisaturley.com', 'http://oldtownweddingcenter.com/', None, None, None, None, None, 'https://zoomrentals.com/toronto-airport-car-rentals', 'http://www.rehabplusscottsdale.com', 'http://azdirectmobility.com', 'http://www.torpedocomics.com', None, 'https://www.aladdinswindowtinting.com', None, None, 'https://www.ewingirrigation.com', None, None, None, '1592239999', 'http://www.dapaulsalon.com', None, 'http://productnightclub.com', 'http://desertmesapool.com', None, 'http://www.cltchristmasvillage.com', None, '1592240015', 'http://www.animalhospitalhighpark.com/', 'https://bhrcenter.com/location/boca-park-nv/', 'http://www.conns.com/store-locator/arizona/scottsdale-85250', None, None, None, 'http://www.bestfootspascottsdale.com/', 'http://elitepetcare.com/', 'http://www.reservesquareapts.com', 'http://www.livelovelashe.ca', 'http://www.ibrandyourbusiness.com', 'http://wingsonbroadway.com/', 'https://mugshotzofficial.wixsite.com/mugshotz', 'https://www.luckysmarket.com', 'http://linkfinancialadvisory.com', 'https://www.gocleancredit.com', 'http://www.rothmanawningco.com/', 'http://www.fredastaire.com/scottsdale', 'http://www.sweettomatoes.com', None, None, 'http://www.bellairconditioningaz.com', None, 'http://www.arbys.com', None, 'http://tempeautorepairshop.com', None, 'http://martinjlevinedds.com/', '1592240069', 'http://www.carolinaurology.com', 'http://www.whiteningxpressions.com', 'http://www.chevron.com/', None, 'http://www.jambajuice.com', 'https://www.honorhealth.com/locations/hospitals/scottsdale-thompson-peak-medical-center?utm_source=yelp&utm_medium=local&utm_campaign=Scottsdale_Thompson_Peak_Medical_Center', '1592240085', 'https://locations.wafflehouse.com/nc-pineville-1157', None, 'http://www.lasvegasapartments-r-us.com/Mirabella-Apartments-for-Rent-in-Las-Vegas.html', '1592240094', 'https://www.supercuts.com/locations/nv/las-vegas/parkway-retail-center-haircuts-82614.html', 'https://www.waterstreetpizzeria.com', 'https://familymoversexpress.com', None, None, None, None, None, None, 'https://www.mcdonalds.com/us/en-us/restaurant-locator.html?cid=RF:Yelp:::RODBOGONov', 'http://www.asphaltpavingranaldi.com', 'https://locations.jackinthebox.com/us/az/peoria/8236-w-deer-valley-rd', 'http://www.tjmaxx.com', None, None, 'http://www.swarovski.com/en-US/?utm_source=YELP&utm_medium=referral_managed&utm_campaign=YELP', 'http://www.chiefdodge.com/', 'https://www.titlenine.com/store-locator/title+nine+-+madison.do', 'http://www.tiremanusa.com/locations.aspx', None, 'http://www.wolfsonmedicalcenter.com', 'http://greenwayfamilyautoandtowing.com', 'https://www.orangetheory.com/en-us/locations/arizona/peoria/25314-n-lake-pleasant-pkwy-suite-1260/?utm_source=redirect&utm_medium=print&utm_campaign=yelp&utm_term=032016&utm_content=yelp_print', 'http://www.woodysdogbath.com', 'http://www.mayfieldbrainard.com', None, None, 'http://www.scrubsandbeyond.com', 'http://fitnessfactorycharlotte.com/', 'http://azsportsmedicine.com', 'http://www.idealservices.com', None, None, None, 'https://www.brasstapbeerbar.com/Charlotte', 'http://www.seaminglystraight.com', None, 'http://www.whitfillnursery.com', None, 'http://www.chip-n-daleslandscaping.com', 'http://www.illinitechcenter.com', None, None, 'http://www.villageicecream.com', '1592240192', None, 'http://www.carpet-cleaning-pittsburgh.com', 'http://www.thelowcarbgrocery.com', 'http://lezah.ca', 'https://www.brakesplus.com', 'http://paulmitchell.edu/phoenix', None, None, None, None, 'http://www.alohabbqgrill.com', 'https://www.esso.ca/en/gas-station/etobicoke-on-geesso-200302279', 'http://www.edencafelounge.ca', 'http://www.signsbytomorrow.com/mesa/', 'http://www.borders.com', None, None, 'https://locations.theupsstore.com/nv/las-vegas/840-s-rancho-dr?utm_campaign=Listings&utm_medium=organic&utm_source=Yext', 'https://peoriafootandanklespecialists.com', None, 'http://tshirtdinervegas.com', None, None, '1592240240', 'http://lebistrodubeaujolais.com/', 'http://farmershouse.ca', 'http://www.sarkujapan.com/', None, None, 'http://www.charlotterusse.com', None, 'http://kpiteophotography.com/', 'http://www.aceshighpartyrental.com/', 'http://www.harveys.ca', 'https://www.walgreens.com/locator/walgreens-11001+s+eastern+ave-henderson-nv-89052/id=6545', 'https://www.altimadental.com/locations/first-canadian-place', 'http://www.rockcitytaphouse.com', 'http://www.thelucky20cafe.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/YYCSC', None, None, 'https://www.yeahyeahspizza.com/toronto', None, 'https://locations.panerabread.com/oh/parma/8109-w-ridgewood-dr.html', 'http://www.clearsound.co', 'http://www.sunrisecoffeelv.com', 'http://www.herbergertheater.org', None, None, 'https://pittsburghmoving.net', 'http://www.phoenixcakecompany.com', 'http://www.timothys.ca', 'http://www.olivetree-apartments.com', None, None, 'http://www.ltayh.com', 'https://www.walgreens.com/locator/walgreens-1745+e+southern+ave-tempe-az-85282/id=1197', 'http://dobsonranchdentalcare.com', None, 'http://www.elitedancestudiolv.com', 'http://www.boosterjuice.com', 'http://www.meckabc.com', 'https://www.realestatelawfirmphoenix.com', '1592240325', 'http://www.goodyearsfamilymedicine.com', 'http://www.drdanstrinkoski.com', 'https://www.petpeoplestores.com/avon-store.html', 'http://www.bedfordautocare.com', 'http://www.shininghopefarms.org/', None, 'http://www.freshandeasy.com/', 'http://www.csuohio.edu/artgallery', 'http://www.desertpalettepainting.com/', 'http://www.premierairsolutions.com', '1592240349', 'http://www.desertroselandscaping.com', '1592240354', 'https://locations.usbank.com/index/kentucky/henderson/old-orchard-branch.html?utm_source=yelp&utm_medium=business%20listing&utm_campaign=visit%20website', 'http://www.dmconcrete.biz', 'http://bluestardiner.ca/', None, 'https://sweetiepiescatering.com', 'https://www.azbouncepro.com', 'http://www.ritters.com', None, 'http://Arizonasoldsisters.com', 'http://www.restaurantdepot.com', 'http://pizzafire.com', 'https://www.ikea.com/us/en/stores/tempe/', None, 'http://www.regaloaksdental.com/', 'https://www.signaturestyle.com/locations/pa/penn-hills/borics-hair-care-penn-hills-shopping-center-haircuts-75141.html', 'http://99only.com/store/Las-Vegas---Beltway/', None, 'https://www.jovannalv.com', 'http://www.coppercanyondentalaz.com/', '1592240397', 'http://journeyeducation.org', None, 'http://www.tandyleatherfactory.com', None, None, None, 'https://local.fedex.com/nv/las-vegas/office-1441/', 'http://www.madameboeuf.com', 'https://www.fitzroyrentals.com', 'http://www.resumetarget.ca/location/ontario-on/toronto/', '1592240420', 'http://callnoworriesjanitorial.com', 'https://www.autozone.com/locations/in/avon/7474-e-us-36.html', None, 'https://www.zoup.com', 'http://www.thedistrictatgvr.com/alsGarage.php', 'http://www.kixs.ca', 'http://www.immigroup.com', 'http://www.burlingtoncoatfactory.com', None, None, None, 'http://www.swensensaz.com', None, 'http://www.arrowheadautocare.com', None, 'http://www.lucasautomotive.ca/', 'http://lululemon.com/toronto/queenstreet?cid=yelp', None, 'https://www.nothingbundtcakes.com/bakery/az/surprise?utm_source=yelp&utm_medium=brandedprofiles&utm_campaign=surprise_300', None, 'http://www.rnbflooring.com', 'http://www.fortpittblockhouse.com/', 'https://www.hilton.com/en/hotels/cltbahw-homewood-suites-charlotte-ballantyne-nc/?SEO_id=YELP-HW-CLTBAHW', 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', 'http://www.cortebellagolfclub.com', 'http://www.popsbarber.com', None, '1592240482', None, 'http://www.greenbeauty.ca', 'http://24hrlockoutsmadison.com/', 'https://kevindeon777.wixsite.com/remodel', 'https://www.needforsweetslv.com', 'http://suzieqluxe.com', None, 'http://www.torontoislandsup.com', None, 'http://www.azhumane.org', 'http://www.providenceprepschool.com/', '1592240510', 'http://kierlandresort.com/fine-dining-at-scottsdale-restaurants/scotch-library/', 'http://www.vocellipizza.com/mccandless_pa', 'http://shawarmaroyale.com', 'http://bullseyervrepair.com', None, None, 'https://www.westelm.com/stores/ca/on/toronto-shops-at-king-liberty/', 'http://www.sterlescountryhouse.com', 'http://www.dawgdayzcare.com/', 'http://www.superiortire.ca/locations/mode/4/3-laidlaw-blvd-markham-on-l3p1w5.aspx', 'https://experigreen.com/service-area/charlotte-nc/', 'http://www.michaelscateringevents.com', 'http://www.footdocvegas.com', 'http://www.InfinityAcuAZ.com', 'http://www.eggsmart.ca', 'http://www.cakestoyou.net', None, None, None, 'http://www.thecorneralley.com', 'http://www.pinkpetalsflorist.com', 'http://www.thecroftdowntown.com/', 'http://www.dbacks.com/', 'https://www.smilefitnessdentalcenter.com', 'https://stores.savers.com/nv/lasvegas/savers-thrift-store-1047.html', 'https://www.appliancepartscompany.com', 'https://www.marriott.com/setSCtracking.mi?scid=dfedf519-6161-4ae6-be84-577f767ed579&mid=/hotels/travel/CLTWS', None, 'http://www.MortgageAssociatesOntario.com', None, 'https://locations.rallys.com/oh/parma/6201-brookpark-road/?utm_source=Yelp', None, 'https://www.bestbuysoccer.com', 'http://www.aeropostale.com', 'http://www.alternativehealingconnection.com/', None, 'http://www.mastercleaningaz.com', 'http://www.thepoddrop.com', 'http://www.dozoaz.com/', 'http://luckysloungelv.com', 'http://www3.hilton.com/en/hotels/north-carolina/hilton-charlotte-center-city-CLTHHHF/index.html', 'http://bestofsevenbarbers.com/', 'http://www.lorispetagree.com', 'https://deltaco.com', 'http://firstwindfitness.com', 'https://lakehealth.org/Locations/TriPoint-Medical-Center', None, 'http://www.swisschalet.com', None, None, None, None, '1592240633', 'http://www.midaslocator.com/AutoRepairFinder/tabid/151/Default.aspx', 'http://www.pricklypeachwaxbar.com', None, 'http://asappoollv.com', 'http://www.snipits.com/locations/avon/', 'http://www.timhortons.com/', 'http://www.43north.biz', 'http://www.mixxmethod.com', 'http://www.rizegaragedoor.com', None, 'http://www.papagostables.com', None, None, None, 'http://theswedishconnection.net', 'https://www.bestwestern.com/en_US/book/hotel-rooms.66083.html?iata=00171890&ssob=BLBWI0003G&cid=BLBWI0003G:yext:website:66083', 'http://federicosmexicanfood.com', None, None, 'http://www.calvarychristiandaycare.com/', 'http://www.phxdj.com', None, 'https://www.ezwindowsolutions.com/cleveland/', 'http://www.mrappliance.com/pittsburgh', None, 'http://www.opsiseyecare.com', 'http://www.avanteoptics.com', None, 'https://www.chronicinktattoo.com/locations/tattoo-shop/chronic-ink-midtown-toronto-yonge-eglinton/', None, 'http://www.adaptiveforce.com', None, 'http://www.thecatnfiddlepub.com', 'http://www.valleyymca.org', 'http://www.ladybugpestcontrol.com/', None, None, '1592240712', None, 'http://www.greenfieldcitrus.com', 'http://www.prada.com', None, None, None, 'http://www.towncentredentalclinic.ca', 'http://www.hobbyhobby.com', None, 'https://southpointcasino.com/entertainment/grandview-lounge/?cid=ypgl', 'http://crystalclearreflectionsaz.com', 'http://danielacaltarumd.com/', None, 'http://www.cleantheworld.org/', 'https://www.famousfootwear.com/?partnerid=YELP', 'http://www.newhorizonlittlelearners.com', 'https://www.hairclub.com/locations/nv/las-vegas/hair-loss-clinic-292/?pub=loclisting&utm_source=yext&utm_medium=loclisting&utm_campaign=YX_292', 'http://galatimarketfresh.com', '1592240751', None, 'http://www.toysrus.ca', 'https://www.cvs.com/store-locator/cvs-pharmacy-address/9302+Olde+Eight+Road-Northfield+Center-OH-44067/storeid=4301?cid=ps_ylp_2020', None, 'http://www.salonritualsaveda.com', 'http://www.jetfuelcoffee.com', None, 'http://www.mrhero.com', 'http://www.lululemon.com/thornhill/promenade?cid=yelp', 'http://www.destinationweddingphotographycalgary.com', 'http://www.golfthebridges.com/barandgrill/', '1592240777', 'http://funtropolis.ca', 'http://www.bilmarcompanies.com', None, 'https://www.aznaturalselections.com', 'http://locksmith4Uvegas.com', 'http://scotiaplaza.kidsandcompany.com', None, '1592240796', 'http://www.greatclips.com', 'http://gremassagetherapy.com', 'http://www.fortunenails.ca', 'https://locations.massageenvy.com/arizona/peoria/10006-w-happy-valley-parkway.html', 'https://www.extraspace.com/storage/facilities/us/north_carolina/charlotte/1000000404/?cid=org::ld&utm_source=yelp&utm_medium=referral&utm_campaign=org::ld', 'http://www.kodisnaturalpetfoods.com', None, 'https://www.papamurphys.com/store-finder/position?utm_source=yelp&utm_medium=2018ads&utm_campaign=SC011', 'http://www.arizonahairco.com', None, None, 'http://www.desi.bar', None, 'http://mkt.com/mobile-bubbles', 'https://www.walmart.com/store/3473/las-vegas-nv/whats-new', None, 'https://www.pizzahut.ca/store-details/store-R31411/pizza-hut-Toronto', 'http://www.waborasushi.com/', 'http://salon7000.com/', None, None, 'http://www.whitecastle.com', None, None, None, '1592240855', None, None, '1592240862', None, None, 'http://www.averyonthegreen.com', None, 'https://www.southviewdentallv.com/index.php', None, None, 'http://www.firstcu.net', 'http://www.anytimegaragedoor.com/', 'https://sardellaspizza.com', None, 'http://www.panevinorestaurant.net/about.html', 'http://www.newhealthdentalaz.com/', None, 'http://www.hatleystore.com', 'http://xtremeairaz.com', 'https://www.phoenixconventioncenter.com/orpheum-theatre', None, 'http://www.veronadentalcare.com', 'http://www.canyonridge.org', 'http://mint-landscape-creations-llc.business.site', None, None, 'http://www.academyofsound.org', None, 'http://lowell.wisc.edu', None, None, 'http://www.chandlervisioncenter.com', None, 'http://www.sugarmarmalade.com', 'http://shoparooni.com', 'http://www.dieselbarbershop.com', 'http://www.allcitytransportation.com', None, 'http://premierdental702.com', 'http://www.dominos.com', 'http://www.lazylizardaz.com/', 'http://www.qqteaplus.com', 'http://snailmotorsports.com/', 'https://www.autozone.com/locations/az/phoenix/8325-w-camelback-rd.html', 'http://www.lacasalatina.net', 'http://www.itzzapizza.com', 'https://www.aritzia.com/en/store-locator?q=M9C+1B8&view=results', 'https://www.plannedparenthood.org/health-center/wisconsin/madison/53713/madison-south-health-center-3937-91860?utm_campaign=madison-south-health-center&utm_medium=referral&utm_source=yelp.com&utm_content=link', None, None, 'http://www.azkarateacademy.com', 'http://olivebranch412.com', None, None, 'http://susiesestateaz.com/estate_sales', None, None, 'http://www.thaiexpress.ca', 'https://www.advantagestorage.net/self-storage/az/phoenix/advantage-storage-moon-valley/', 'https://florist.1800flowers.com/us-nv/1-800-flowers-shoppe-las-vegas', 'http://www.weavebaratl.com', 'http://www.migrationlaw.com', 'http://www.iceburghs.com', 'http://www.drgreencerts.com', 'http://modernnailsandlash.com', 'https://recreationalartstudio.com', None, 'http://design-ramon.com', 'http://www.sallybeauty.com', 'http://www.just4funsportingclub.com/', 'https://www.pandaexpress.com/?utm_source=yelp&utm_medium=profile&utm_campaign=1378&utm_content=bpas', 'http://www.lush.ca', None, 'http://www.madsens.ca/', 'http://www.sportracks.com', None, 'https://www.fatburger.com/locations/rancho', 'http://www.azsuperiorhomeinspections.com', 'http://www.earnhardtcadillac.com/', None, '1592241033', 'http://barneyslounge.ga', 'http://brianwilliamsdds.com/', 'http://www.simplyflowerstoronto.com', None, 'http://www.arizonaautowashscottsdale.com', 'http://www.dunkindonuts.com', 'http://www.PattersonHomesAZ.com', None, 'http://www.outback.com/locations/oh/independence', 'https://pablocanada.com', None, 'http://www.mychiropractoraz.com', 'http://www.hollospapercraft.net/', 'https://www.searsauto.com/stores/6601?psid=campaigncode-6601', 'http://www.sayyestoherdress.com', None, 'http://www.eyestudio.ca', 'http://www.highlandfarms.ca', 'https://tendollarshirtstore.com', 'http://www.cineplex.com/Theatre/cineplex-odeon-eglinton-town-centre-cinemas', 'http://www.myhusky.ca/station_locator.html?provinceID=AB', 'http://peaksandrafters.com', 'http://www.junkmastersinc.com', None, 'https://www.filerchiropracticpittsburgh.com', 'http://www.westsidemarket.org/vendor.aspx?id=77', 'http://www.ttc.ca/Subway/Stations/St_Patrick/station.jsp', 'http://www.gzcomics.com/locale.html', 'http://www.deltaco.com', 'https://jade-chinese-kitchen.business.site', None, 'http://www.bojangles.com', 'http://Clocktowerbar.ca', 'http://valleyislepumping.com/', 'http://www.quiznos.ca', None, None, None, 'http://www.bigotires.com', None, 'http://stpaulpgh.org', 'http://coopersbbqco.com/', 'http://www.ecokingaz.com', None, None, 'http://www.powervac.ca/power-vac-toronto-gta', None, 'http://www.myphoenixair.com', 'https://www.gianteagle.com/Stores/Store/?storeId=6299', 'http://www.Team1stRealEstate.com', 'http://www.fickleonline.com', 'https://vcahospitals.com/bakerstown?utm_source=yelp.com&utm_medium=referral&utm_term=VCA_Bakerstown_Animal_Hospital=main_url', 'http://www.jglennhome.com', '1592241164', '1592241167', 'https://www.doggiedistrictaz.com?utm_source=yelp&utm_medium=social', None, None, 'http://changjiangbestmadison.com', 'https://www.theacteam.com', None, 'http://www.eatoooweebbq.com', 'http://fourcornersbakery.com', None, 'http://www.paintstoretoronto.com', 'https://stores.toysrus.com/az/goodyear/baby-store-goodyear-az-b5641.html', 'https://www.fiix.io', 'http://www.visualharmonyphotography.com/', 'https://landmarkautoanddiesel.com', 'https://www.lafitness.com/Pages/clubhome.aspx?clubid=830', 'https://finedetails.ca', 'http://www.tbdc.com', 'https://www.liveedgelust.com', 'https://www.kinjosushiandgrill.com', 'http://GrizzlyAC.com', 'http://www.opticausa.com', None, 'https://www.simonmed.com/locations/all/simonmed_imaging___sun_city_peoria', 'http://www.advancedhearinggroup.com', 'https://sacredpi.com', None, 'https://www.transformyourself.ca', None, 'http://clickserve.dartsearch.net/link/click?lid=43700044282967270&ds_s_kwgid=58700005061605699&ds_url_v=2&ds_dest_url=https://www.pizzahut.com', 'https://arirangkoreanbbq.business.site', 'http://www.marqueetheatreaz.com', None, 'http://peoriaartisanbrewing.com/tap-room/', 'http://www.holeechow.com', 'https://www.abcgaragedoor.biz', 'http://webersvintageicecream.com', None, 'http://www.leewiyoga.com', None, 'http://www.lucapizza.ca/', None, 'https://www.wangskitchenpa.com', 'http://northofbombay.com', 'http://www.eandrsweetery.com', 'http://indodesignerrugs.com', 'http://www.landmarkhomeserv.com', None, None, 'http://www.saharaautolv.com', 'http://www.drlizcruz.com', 'https://www.wingstop.com/location/wingstop-230-glendale-az-85306/menu', 'http://www.northcoastjawcenter.com', None, '1592241292', None, 'https://www.loblaws.ca/store-locator/Loblaws+-+St.+Clair+Ave+West/1212?storeId=1212', None, 'http://glassking.com', None, 'http://www.glamoursalonandspa.ca', None, 'http://www.CapstoneBrokerage.com', 'http://www.myrivertowne.com', 'http://www.hotrodpiercingcompany.com/index.html', 'http://www.tuckyshotdogs.com', None, 'http://www.getdynamicdetailing.com', None, 'https://www.sprouts.com/stores/details/-/store-details/view/store/20/?utm_source=yelp&utm_medium=listing&utm_campaign=website', 'https://www.walgreens.com/locator/walgreens-3945+e+southern+ave-phoenix-az-85040/id=4422', 'http://local.tiresplus.com/store/Madison-East?lw_cmp=YLP_TP_53714&utm_source=YLP&utm_medium=TP_53714', None, 'http://www1.shoppersdrugmart.ca/en/store-locator/store/359', None, 'http://www.theyardpgh.com/market-square', None, None, None, 'http://www.nissanairport.com', 'http://pilateselementlv.com', 'http://bodega13.com/', 'http://freedomclothingcollective.com', 'http://trackofficefurniture.com/', None, 'https://locations.visionworks.com/ll/US/PA/Pittsburgh/120-Fifth-Ave-Pl', 'http://www.healthree.ca', None, None, '1592241373', 'http://www.glamourco.com', 'http://www.goodlifefitness.com/locations/ontario/toronto/kingliberty?source=yelp-websitelink-torontoking-gl', 'http://www.rei.com/stores/paradise-valley.html', 'https://www.chipotle.com?utm_source=Yelp&utm_medium=Yelp-listing&utm_campaign=cmg_yelp_listings', None, 'http://www.cornerstone-familypractice.com/', 'https://www.mcdonalds.com/ca/en-ca/promotions/mccafefebruaryevent.html', None, None, None, 'http://www.fitkitchen.ca', 'https://www.mastercuts.com/locations/nv/henderson/mastercuts-galleria-at-sunset-haircuts-3331.html', 'https://locations.wendys.com/united-states/pa/mcmurray/3500-washington-rd', 'http://www.mccormickandschmicks.com/Locations/pittsburgh-pennsylvania/pittsburgh-pennsylvania/FifthAve.aspx', 'https://www.strongsville.org/departments/parks-recreation', None, 'http://www.ppcp.org', None, '1592241411', None, 'http://www.allcomfortservices.com/', 'http://www.eliteautoaz.com', 'http://www.stacysplacescottsdale.com', 'http://ductmanofnevada.com', 'http://www.compolastastypizza.com', None, 'http://www.brighton.com', 'http://www.maclinford.com', None, 'http://www.regalcarpetcleaners.com', None, 'https://mrjims.pizza/149.cfm', 'http://www.amberoaksrestaurant.com/', None, None, 'http://www.birkdaledental.com', 'https://www.fastmed.com/urgent-care-centers/chandler-az-walk-in-clinic-west-ray-road/?utm_source=Yelp&utm_campaign=BizURL&utm_term=YelpOrganicListing', None, 'http://www.levelsfitness.net', 'http://www.borealis-salon.com', 'https://www.nickandmikebarandgrill.com', 'http://uniongrilltap.com', 'http://lululemon.com', None, 'http://www.pacificfitnesslv.com/', 'http://hairbyhall.schedulista.com', 'http://www.kierlandresort.com/dining/nellie-cashmans/', 'http://denierlaundromat.findalaundry.org', None, 'https://landing.terminix.com/affiliates?marketingpartner=yelp&campaign=yelpnortheastorganic', 'http://www.myvalleyagent.com', 'http://www.littleotterswim.com', None, None, 'http://www.peoriakayakrental.com', 'https://www.enterprise.com/en/car-rental/locations/us/nc/charlotte-south-woodlawn-53c4.html?mcid=iyp:8368400', 'http://www.kingwestvets.com', None, 'http://www.pointerscrapbooking.com/', 'http://www.cocktailemporium.com', None, 'http://www.nyahsmobilegrooming.com', None, 'http://www.gaetanosmayfield.com/', 'http://www.pool-mart.ca', None, 'http://tullys2.com/', 'https://www.signatureplasticsurgery.net', 'http://www.spinink.ca', 'http://www.wildhorsepassresort.com/kosin', 'http://sol-nails-spa.business.site', 'https://pandalibre.com', None, 'http://www.awaxmanlaw.ca/', None, 'http://www.nellisasc.com', 'https://shop.nordstrom.com/st/last-chance-clearance-store?cm_mmc=yelp-_-ycm-_-rack-_-780', None, None, 'https://www.canadianair.info', 'http://toothfairychildrensdental.com', None, None, None, 'http://www.handandstone.ca/locations/bloor-west', None, None, 'https://www.aldi.us/en/weekly-specials/our-weekly-ads/', 'http://www.europeanlv.com', 'http://www.priorityhealthgroup.vegas', 'http://www.freewayinsurance.com', 'https://locations.wendys.com/united-states/oh/stow/1051-w.-graham-street', 'http://www.foodlion.com', 'http://www.joedufek.com?cmpid=CKMF_BLM_0006', 'https://www.orleanscasino.com/dine/casual-dining/medley-buffet?disid=wmm_yelp_to_corporate_medley_organiclink', 'https://phoenixtrimatree.com', 'http://www.greenmangopest.com', 'https://locations.sonicdrivein.com/az/glendale/13751-west-glendale-avenue.html', 'https://www.kindercare.com/our-centers/mesa/az/301365?utm_campaign=kc-lg&utm_source=yelp&utm_medium=pro&utm_keyword=&utm_platform=&utm_matchtype=&utm_adgroupid=&utm_content=301365', None, 'http://www.sweetmangoohio.com', 'http://www.kfc.com', None, 'http://www.altfitpilates.com', 'http://www.nakedmagicians.com', None, 'http://prestocalzonebakery.com', 'http://www.starnail.com/', '1592241614', 'http://serenityhospiceaz.com/home/1792243', 'http://www.vans.com', 'http://familysqueezedlemonade.com/', 'http://codyscavecreek.com/', None, 'http://www.dwcpinc.com', 'http://www.glendaleazsda.org/', 'http://www.timhortons.com', 'http://www.bankofamerica.com', None, None, 'http://spa4pawsaz.com', 'http://www.turnhousegrille.com', 'https://www.dairyqueen.com/us-en/Locator/Detail/?store-id=2486&localechange=1', 'http://encompasshealth.com/desertcanyonrehab', 'http://www.doc4women.com', 'https://www.bloomingdaysfloral.com', 'http://www.maddissouthernbistro.com/', None, 'http://www.cabcpr.com', 'http://www.upcinematics.com', 'http://drjazzdetailing.com', 'http://www.thedogstop.net/', 'http://www.azbone.com', 'http://www.cleatswings.com', 'http://loparos.localmobil.com/', 'http://www.scottsdale-dentistry.com/index.php', 'http://treasureisland.com/restaurant/17/the-coffee-shop', None, 'http://www.adelaidehallto.com', None, 'http://www.vantagehotels.com/hotel-details.cfm?idp=30026', None, None, 'https://www.tacobell.com/food/breakfast?utm_campaign=e3e4breakfast&utm_medium=social&utm_source=yelp&utm_content=breakfastmenu', None, None, 'http://www.einsteinbros.com/', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://stores.jcpenney.com/nv/lasvegas/clothing-stores-lasvegas-nv-2848.html?cm_mmc=YEXT-_-LOCAL-_-PROFILE-_-2848', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'https://www.fineartspermanentcosmetics.com', None, 'http://www.zyng.com', 'https://www.downeastbasics.com', None, '1592241718', 'http://www.enlightanspraytan.com', None, None, None, 'http://paparocks.com/', 'https://www.wyndhamhotels.com/hotel/13899?tel=18002521709&iata=00093785&cid=fe:ra:20161215:yl:pp:ramus:13899', 'http://qnailscottsdale.com/', 'http://www.economytowinglv.com', None, None, None, 'http://www.centrestauto.ca', None, 'https://www.NEXAMortgage.com', 'http://www.presstalent.com', 'http://www.k9couture.ca', None, 'https://www.sleepcountry.ca', None, 'http://joynailsspa.business.site', None, 'http://www.mipueblo.us', '1592241770', 'http://www.cafe-mirage.com', 'http://www.sammyspizza.com/', 'https://www.starbucks.ca/store-locator/store/1003/u-of-toronto-warren-stevens-bldg-55-harbord-street-toronto-on-m-5-s-2-w-6-ca', 'http://undergroundbutcher.com', 'https://stores.chicos.com/s/chico-s-the-district-at-green-valley-ranch', None, 'http://www.diamondloungeaz.com/', 'http://www.marcs.com/stores/marc-s-willowick-grocery-store-44095.aspx', 'http://www.boosterjuice.com/', 'http://www.smoothieking.com', 'http://www.mensclothingaz.com', None, 'http://thebodyshopgilbert.com', 'http://www.mark-taylor.com/arizona/san-hacienda/?utm_source=yelp&utm_medium=listing&utm_content=visit-website', 'http://www.amfamilyspine.com', None, 'http://www.clarkcountynv.gov/vote', None, 'https://www.gianteagle.com/Stores/Store/?storeId=440', 'http://www.ocarizona.com', 'http://www.citynationalarenavegas.com', 'http://www.accesshealthdental.com', None, None, None, 'http://www.amscharters.org/peoria', 'http://www.oldchicago.com/mesa-superstition-springs', None, None, 'https://thrifty.com/loc/ll/US/AZ/PHOENIX-AP/1805-E-SKY-HARBOR-CIRCLE-SOUTH', 'http://www.celebritypatios.com/', 'https://www.walmart.com/store/1464/charlotte-nc/whats-new', 'http://www.arizonafireplaces.com', None, 'http://www.calvarycenter.com', 'http://www.bigabaldis.com/', 'http://www.bigplusfashions.com/', 'http://www.shampoochdogspaw.com', None, 'http://lowestpricedepot.com', None, None, 'https://www.shortrunprintingltd.com/Quote/index.php', 'http://www.palazzo.com/', 'http://superiorplumber.ca', 'http://www.chambersobgynlv.com', 'http://stanleysteemer.com', 'https://www.laytondentistrync.com', 'https://www.sonoraquest.com/find-a-location/scottsdale-scottsdale-rd-mescal/', 'https://buffetmandarin.weebly.com', 'http://www.americandrain.com', 'http://www.realfruitbubbletea.com', 'https://www.PUREfectReflection.com', None, 'https://www.telus.com', 'http://millermassage.com', 'http://www.weststar.org/pages/about/b_sunset.html', None, 'http://shanghai1938.biz', 'http://www.qualitycleanohio.com', 'http://www.silverstatedental.com', '1592241931', 'https://www.eddiev.com/locations/nc/charlotte/charlotte/8521?cmpid=br:ev_ag:ie_ch:dry_ca:EVQ420LOCURL_dt:20200511_sn:yelp_gt:charlotte-nc-8521_pl:locurl_rd:1019', None, '1592241939', None, 'http://desertwindawards.com', None, None, 'http://mintlounge.ca', 'http://dsw.townshoes.ca', None, None, None, 'http://www.peterpiperpizza.com/', None, None, 'http://shawbuttebarbershop.com/', 'http://www.christopherspub.net', 'http://locations.arbys.com/us/wi/sun-prairie/829-bunny-trail.html', 'http://www.mysweetblessingsbakery.com', 'http://kaizokupokebowl.com', None, None, 'http://www.coveralllasvegas.com', 'https://mikeshaulingservices.com', None, 'http://www.5rchathai.com', 'http://www.pittsburghplayhouse.com', None, 'http://stapleygaragedoor.com', 'http://www.mbco.ca/', 'http://www.union.wisc.edu/', 'http://www.icedenscottsdale.com', 'http://www.sunchain.com', None, 'http://www.laylaspalace.com', '1592242020', None, 'http://www.ackermanhomeinspections.com/', 'http://www.mysisterscloset.com/', None, 'http://www.big5sportinggoods.com', 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', None, None, 'http://subway.com?utm_source=yelp&utm_medium=display&utm_term=order&utm_content=yelp-partnership-gm&utm_campaign=fwh-core-menu-cross-window-2020&cid=0:im:0:subway-com:dsp:nat-us-v1&segment_code=0', 'http://www.indacutbarbershop.com/', None, 'http://www.monkeesofcharlotte.com', 'http://www.financeofamerica.com/jrencher', 'http://www.littlecaesars.com/Locations/tabid/92/directions/85240/Default.aspx?id=MTUxNTAwMDQ%3d', 'https://www.worldmarket.com?camp=ban:Yelp:16_Evergreen&utm_medium=display&utm_source=Yelp&utm_campaign=16_Evergreen', None, 'https://www.reginellapizzeria.com', 'http://www.eastsidemarios.com', 'http://www.pittsburghcyclecenter.com', None, None, 'http://www.fleetfeetcleveland.com', None, 'https://www.brightnow.com/dentist-near-me/chandler-dentist/40050?utm_campaign=local_profiles&utm_medium=profiles&utm_source=yelp', 'http://www.securitydatadestruction.com', 'http://www.innovativehealinglv.com', 'http://www.urbandog.ca', 'https://woodiesautoservice.com', 'https://www.scene75.com/cleveland/eat-drink/food-truck-alley/', 'http://massagemaniaglendale.com', 'http://www.bottleshop48.com', 'http://www.chrislopez.net?cmpid=H3LE_BLM_0006', 'http://www.dunkindonuts.com', 'http://a-zhandymanservicesllc.com', 'https://www.faucetdoctorplumbingaz.com', 'http://signaturenevada.com', None, 'http://www.msfstrong.com/', 'http://www.parkbluesky.com', 'http://www.TheSchielkeGroup.com', None, 'http://www.motel6.com/reservations/motel_detail.aspx?num=2012', 'https://www.simondentalcare.com', 'http://sandblastinglasvegas.com', 'http://www.thegreatsteak.com', 'http://www.linsasianfusion.com', 'http://www.rubinsfurniture.com', None, 'http://www.hairstory.ca', None, 'http://www.abcfest.com', 'http://www.mybenzparts.com/', None, 'http://www.MariachiColonial.com', None, 'http://www.marbleslab.ca', 'http://www.lushfloral.com', 'http://www.llcrv.com/', 'https://madisonsbarandgrill.netwaiter.com/monroeville', 'http://www.ogradysoncollege.com', '1592242202', 'http://www.4naturesfinest.com', 'http://marcellopointeclaire.com', None, 'http://www.barargon.com', None, None, 'https://bodybeautifullasermedi-spa.com/locations-near-me/laser-hair-removal-bethel-park/', 'http://www.footlocker.com', 'http://www.daverubinsonpools.com', 'https://www.decathlon.ca/fr/c/135-decathlon-brossard', 'http://spicnspanspotlesscleaning.webs.com', None, 'https://www.circlek.com/store-locator/us/mesa/2005-w-broadway-rd/2706660', 'http://www.rowefarms.ca', 'http://www.spectaclelovesyou.com', 'http://www.boasteak.com/balboa', 'http://hawgndawgexpress.com', None, None, 'http://sfsourdougheatery.com/', 'http://foosing.weebly.com', None, 'http://www.encorecleveland.com/', 'https://www.joysquad.com/locations/las-vegas-photo-booth-rental', None, None, 'http://chase.com', 'http://www.ginzacuisine.ca', '1592242278', None, 'http://www.peaksathleticclub.com', 'https://www.hilton.com/en/hotels/phxsaht-home2-suites-phoenix-airport-south/?SEO_id=YELP-HT-PHXSAHT', 'http://southpointcasino.com/dining/coronado-cafe?cid=ypcor', 'https://www.cash1loans.com/locations/arizona/phoenix-thomas.aspx', 'http://www.co-operateon.com', None, '1592242301']

In [125]:
len(urls)

23551